In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.optimize import minimize_scalar

from src.rent_vs_buy import (
    RentVsBuy,
    rent_vs_buy_breakeven_objective_closure,
    render_plotly_html,
    optimization_per_year,
    breakeven_per_year,
)

## Setup
We can use `RentVsBuy`'s `calculate` function to simulate a financial scenario:

In [2]:
# We will reuse these kwargs later on.
kwargs = dict(
    home_price=1_000_000,  # Value of the home
    financed_fees=0,  # Any fees that are financed (i.e. VA Loan Fee)
    years=20,  # Compute earnings over X-year period
    mortgage_rate=0.06342,  # Mortgage rate
    downpayment=0.2,  # Downpayment amount
    pmi=0.005,  # PMI expressed as a percentage of the total loan value
    length_of_mortgage=30,  # 30-year fixed mortgage
    home_price_growth_rate=0.08,  # Annual appreciation of the home
    rent_growth_rate=0.01,  # How much rent increases annually
    investment_return_rate=0.08,  # How much can we make in the market annually?
    investment_tax_rate=0.15,  # Long term capital gains tax
    inflation_rate=0.02,  # How much does inflation go up each year?
    filing_jointly=True,  # Tax filing status
    property_tax_rate=0.008,  # Annual tax rate on the home
    marginal_tax_rate=0.02,  # Tax bracket based on income
    costs_of_buying_home=0.01,  # How much do we pay at closing to buy?
    costs_of_selling_home=0.06,  # Seller fee
    maintenance_rate=0.005,  # How much we put away each year for maintenance (roofs, appliances, etc.)
    home_owners_insurance_rate=0.004,  # Insurance on the home.
    monthly_utilities=0,  # Utilities for the home (usually a wash with renting utilities)
    monthly_common_fees=350,  # HOA fees, etc.
    monthly_rent=3550,  # How much is the current rent?
    security_deposit=1,  # Proportion of monthly rent paid in the first month
    brokers_fee=0.00,  # Brokers fee (if applicable)
    renters_insurance_rate=0.01,  # Rental insurance
)
# Create our RentVsBuy calculator.
rent_vs_buy = RentVsBuy().calculate(**kwargs)
df = rent_vs_buy.df
df

,annual_per,per_inv,home_value,first_month_home_value,ppmt,ipmt,pmt,maintenance,insurance,property_taxes,...,total_rent_assets,home_opportunity_cost,rental_opportunity_cost,home_opportunity_cost_fv,rental_opportunity_cost_fv,home_opportunity_cost_fv_post_tax,rental_opportunity_cost_fv_post_tax,home_cumulative_opportunity,rental_cumulative_opportunity,buy_vs_rent
0,0,239,1.000000e+06,1.000000e+06,745.705185,4228.000000,4973.705185,416.666667,333.333333,666.666667,...,0.0,209604.871852,0.000000e+00,1.025843e+06,-0.000000e+00,871966.474075,0.000000e+00,8.719665e+05,0.000000e+00,-8.719665e+05
1,0,238,1.006667e+06,1.000000e+06,749.646237,4224.058948,4973.705185,416.666667,333.333333,666.666667,...,0.0,3154.871852,0.000000e+00,1.533824e+04,-0.000000e+00,13037.503844,0.000000e+00,8.850040e+05,0.000000e+00,-8.850040e+05
2,0,237,1.013378e+06,1.000000e+06,753.608117,4220.097068,4973.705185,416.666667,333.333333,666.666667,...,0.0,3154.871852,0.000000e+00,1.523666e+04,-0.000000e+00,12951.162759,0.000000e+00,8.979551e+05,0.000000e+00,-8.979551e+05
3,0,236,1.020134e+06,1.000000e+06,757.590936,4216.114249,4973.705185,416.666667,333.333333,666.666667,...,0.0,3154.871852,0.000000e+00,1.513576e+04,-0.000000e+00,12865.393470,0.000000e+00,9.108205e+05,0.000000e+00,-9.108205e+05
4,0,235,1.026935e+06,1.000000e+06,761.594804,4212.110381,4973.705185,416.666667,333.333333,666.666667,...,0.0,3154.871852,0.000000e+00,1.503552e+04,-0.000000e+00,12780.192188,0.000000e+00,9.236007e+05,0.000000e+00,-9.236007e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,19,4,4.765810e+06,4.549220e+06,2573.532960,2400.172225,4973.705185,1895.508239,1516.406591,3032.813182,...,0.0,7596.642466,0.000000e+00,7.801254e+03,-0.000000e+00,7596.642466,0.000000e+00,2.971966e+06,0.000000e+00,-2.971966e+06
236,19,3,4.797582e+06,4.549220e+06,2587.134082,2386.571103,4973.705185,1895.508239,1516.406591,3032.813182,...,0.0,7596.642466,0.000000e+00,7.749590e+03,-0.000000e+00,7596.642466,0.000000e+00,2.979562e+06,0.000000e+00,-2.979562e+06
237,19,2,4.829566e+06,4.549220e+06,2600.807086,2372.898100,4973.705185,1895.508239,1516.406591,3032.813182,...,0.0,7596.642466,0.000000e+00,7.698269e+03,-0.000000e+00,7596.642466,0.000000e+00,2.987159e+06,0.000000e+00,-2.987159e+06
238,19,1,4.861763e+06,4.549220e+06,2614.552351,2359.152834,4973.705185,1895.508239,1516.406591,3032.813182,...,0.0,7596.642466,0.000000e+00,7.647287e+03,-0.000000e+00,7596.642466,0.000000e+00,2.994755e+06,0.000000e+00,-2.994755e+06


## Calculations
**High level overview**

First, model the cash flows over the life of the home. These cashflows should incorporate any inflation or appreciation that occurs with time.  For this we need:
- Value of the home
- Liabilities of the home
- Assets the home
- Liabilities of renting
- Assets of renting

Next, we can calculate the opportunity cost of either renting or buying. This is the difference in money (i.e. $8000 monthly house payment - $3000 monthly rent payment) that we can invest for the remaining time of the home, appreciated at market rates.

Below, we show the output of these various calculations for the given scenario.

### Home Value
How much will the home be worth when we sell it?

In [3]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Home Value": df.home_value,
                "Initial Home Price": df.home_value[0],
            }
        )
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="3be091d1-ab8e-451d-b2e9-1e6812e2c22f" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("3be091d1-ab8e-451d-b2e9-1e6812e2c22f")) {                    Plotly.newPlot(                        "3be091d1-ab8e-451d-b2e9-1e6812e2c22f",                        [{"hovertemplate":"variable=Home Value\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Home Value","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Home Value","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[1000000.0,1006666.6666666666,1013377.7777777778,1020133.6296296295,1026934.520493827,1033780.7506304524,1040672.6223013218,1047610.4397833305,1054594.509381886,1061625.139444432,1068702.6403740614,1075827.3246432217,1082999.5068075098,1090219.50351956,1097487.6335430234,1104804.2177666435,1112169.5792184211,1119584.043079877,1127047.9367004095,1134561.5896117457,1142125.3335424906,1149739.5024327736,1157404.432448992,1165120.461998652,1172887.9317453096,1180707.1846236116,1188578.5658544356,1196502.4229601317,1204479.1057798658,1212508.9664850647,1220592.3595949654,1228729.641992265,1236921.17293888,1245167.3140918058,1253468.4295190843,1261824.8857158783,1270237.0516206508,1278705.2986314548,1287230.0006223312,1295811.5339598134,1304450.2775195453,1313146.6127030088,1321900.9234543622,1330713.596277391,1339585.0202525735,1348515.5870542573,1357505.6909679524,1366555.7289077386,1375666.10043379,1384837.2077700153,1394069.4558218152,1403363.2521939606,1412719.007208587,1422137.1339233106,1431618.048149466,1441162.1684704623,1450769.9162602655,1460441.7157020005,1470177.9938066802,1479979.180432058,1489845.708301605,1499778.0130236156,1509776.5331104395,1519841.7099978423,1529973.9880644944,1540173.814651591,1550441.6400826015,1560777.9176831522,1571183.1038010397,1581657.6578263799,1592202.042211889,1602816.7224933014,1613502.1673099236,1624258.8484253227,1635087.240748158,1645987.8223531456,1656961.0745021666,1668007.4816655142,1679127.5315432844,1690321.715086906,1701590.5265208187,1712934.4633642908,1724354.0264533858,1735849.719963075,1747422.0514294954,1759071.5317723583,1770798.6753175072,1782603.999819624,1794488.026485088,1806451.2799949886,1818494.2885282882,1830617.5837851434,1842821.7010103776,1855107.1790171135,1867474.5602105607,1879924.3906119643,1892457.2198827106,1905073.6013485952,1917774.0920242523,1930559.252637747,1943429.647655332,1956385.8453063674,1969428.41760841,1982557.9403924658,1995774.9933284153,2009080.1599506047,2022474.0276836087,2035957.1878681658,2049530.235787287,2063193.7706925352,2076948.395830485,2090794.718469355,2104733.349925817,2118764.905591989,2132890.0049626026,2147109.271662353,2161423.333473435,2175832.822363258,2190338.374512346,2204940.630342428,2219640.2345447107,2234437.8361083423,2249334.0883490643,2264329.648938058,2279425.1799309785,2294621.3477971847,2309918.8234491656,2325318.28227216,2340820.4041539743,2356425.8735150006,2372135.3793384335,2387949.61520069,2403869.2793020275,2419895.074497374,2436027.7083273563,2452267.8930495386,2468616.345669869,2485073.7879743343,2501640.94656083,2518318.552871235,2535107.34322371,2552008.058845201,2569021.4459041688,2586148.25554353,2603389.2439138195,2620745.172206579,2638216.8066879553,2655804.918732542,2673510.284857425,2691333.6867564744,2709275.911334851,2727337.7507437496,2745520.0024153744,2763823.4690981433,2782248.958892131,2800797.285284745,2819469.2671866426,2838265.7289678873,2857187.5004943395,2876235.4171643015,2895410.319945397,2914713.055411699,2934144.4757811106,2953705.438952984,2973396.8085460034,2993219.45393631,3013174.2502958854,3033262.078631191,3053483.8258220656,3073840.384660879,3094332.6538919513,3114961.538251231,3135727.948506239,3156632.80149628,3177677.020172922,3198861.5336407414,3220187.2771983463,3241655.192379668,3263266.2269955324,3285021.3351755026,3306921.4774100054,3328967.6205927385,3351160.738063357,3373501.8096504454,3395991.8217147817,3418631.7671928797,3441422.645640832,3464365.4632784375,3487461.233033627,3510710.974587184,3534115.714417765,3557676.4858472166,3581394.329086198,3605270.2912801052,3629305.4265553057,3653500.7960656746,3677857.4680394456,3702376.5178263746,3727059.0279452167,3751906.088131518,3776918.795385728,3802098.2540216325,3827445.5757151097,3852961.8795532105,3878648.292083565,3904505.9473641217,3930535.9870132157,3956739.5602599704,3983117.823995036,4009671.9428216694,4036403.0891071474,4063312.443034528,4090401.1926547578,4117670.5339391227,4145121.67083205,4172755.8153042626,4200574.187406291,4228578.015322332,4256768.535424482,4285146.992327311,4313714.638942826,4342472.736535778,4371422.55477935,4400565.371811211,4429902.474289953,4459435.157451886,4489164.725168231,4519092.490002685,4549219.77326937,4579547.905091166,4610078.224458439,4640812.079288162,4671750.826483416,4702895.831993305,4734248.47087326,4765810.127345748,4797582.194861387,4829566.076160462,4861763.183334865,4894174.93789043],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Initial Home Price\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Initial Home Price","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Initial Home Price","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### House Liabilities
What will we need to pay to upkeep the home?

In [4]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Monthly Fees": df.monthly_common_fees,
                "Insurance": df.insurance,
                "Property Taxes": df.property_taxes,
                "Maintenance": df.maintenance,
                "Closing Costs": df.buying_closing_costs,
                "Mortgage Principal Payment": df.ppmt,
                "Mortgage Interest Payment": df.ipmt,
                "Total Mortgage Payment": df.pmt,
                "Utilities": df.home_monthly_utilities,
                "Downpayment": df.down_fee,
                "PMI": df.pmi,
                "Sellers Fee": df.sellers_fee,
                "Loan Payoff": df.loan_payoff,
            }
        ),
        log_y=True,
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="5b1b54b1-960c-4030-8076-3378506d2151" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("5b1b54b1-960c-4030-8076-3378506d2151")) {                    Plotly.newPlot(                        "5b1b54b1-960c-4030-8076-3378506d2151",                        [{"hovertemplate":"variable=Monthly Fees\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Monthly Fees","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Monthly Fees","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,350.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,357.0,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,364.14,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,371.42280000000005,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,378.851256,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,386.42828112,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,394.15684674240003,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,402.039983677248,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,410.080783350793,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,418.28239901780887,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,426.64804699816506,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,435.18100793812835,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,443.88462809689094,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,452.7623206588288,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,461.8175670720053,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,471.05391841344544,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,480.47499678171437,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,490.0844967173487,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,499.8861866516957,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296,509.8839103847296],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Insurance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Insurance","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Insurance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,333.3333333333333,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,360.99983560250325,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,390.9626439151032,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,423.41235054021695,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,458.5553668112633,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,496.61523610053496,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,537.8340557699745,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,582.4740171431652,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,630.8190732942369,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,683.176745262429,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,739.8800781815702,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,801.2897597673425,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,867.7964146379398,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,939.8230890622142,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1017.8279419406884,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1102.3071591366684,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1193.7981096953993,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1292.882764027855,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1400.191395802097,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979,1516.40659108979],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Property Taxes\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Property Taxes","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Property Taxes","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,666.6666666666666,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,721.9996712050065,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,781.9252878302063,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,846.8247010804339,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,917.1107336225266,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,993.2304722010699,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1075.668111539949,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1164.9480342863303,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1261.6381465884738,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1366.353490524858,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1479.7601563631404,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1602.579519534685,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1735.5928292758797,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,1879.6461781244284,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2035.655883881377,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2204.6143182733367,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2387.5962193907985,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2585.76552805571,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,2800.382791604194,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958,3032.81318217958],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Maintenance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Maintenance","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Maintenance","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,416.6666666666667,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,451.2497945031291,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,488.703304893879,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,529.2654381752712,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,573.1942085140792,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,620.7690451256688,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,672.2925697124682,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,728.0925214289565,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,788.5238416177962,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,853.9709315780362,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,924.8500977269629,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1001.6121997091782,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1084.745518297425,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1174.7788613277678,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1272.2849274258608,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1377.8839489208356,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1492.2476371192492,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1616.1034550348188,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1750.2392447526215,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377,1895.5082388622377],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Closing Costs\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Closing Costs","line":{"color":"#FFA15A","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Closing Costs","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Mortgage Principal Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Mortgage Principal Payment","line":{"color":"#19d3f3","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Mortgage Principal Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[745.7051850946391,749.6462369978644,753.6081173603989,757.590936260648,761.594804358786,765.6198328998216,769.6661337166979,773.73381923339,777.8230024680379,781.9337970360821,786.0663171534179,790.2206776395733,794.3969939208991,798.59538203377,802.8159586278189,807.0588409691672,811.3241469436898,815.611995060287,819.92250445418,824.2557948902204,828.6119867662146,832.9912011162742,837.3935596141746,841.8191845767351,846.2681989672228,850.7407263987652,855.2368911377816,859.7568181074457,864.3006328911433,868.8684617359731,873.4604315562474,878.0766699370229,882.7173051376399,887.3824660952919,892.0722824286058,896.786884441241,901.5264031255128,906.290970166031,911.0807179433586,915.8957795376887,920.7362887325462,925.6023800184971,930.4941885968951,935.4118503836294,940.355502012907,945.3252808410448,950.3213249502901,955.3437731526524,960.3927649937646,965.4684407567561,970.5709414661551,975.7004088918038,980.8569855527967,986.0408147214439,991.2520404272473,996.4908074609043,1001.7572613783354,1007.0515485047204,1012.373815938568,1017.7242115558024,1023.1028840138752,1028.5099827558888,1033.9456580147535,1039.4100608173608,1044.9033429887809,1050.4256571564765,1055.9771567545486,1061.5579960279974,1067.1683300370041,1072.8083146612498,1078.478106604235,1084.1778633976378,1089.9077434056949,1095.667905829593,1101.4585107119033,1107.2797189410148,1113.1316922556193,1119.0145932491887,1124.9285853745118,1130.8738329482162,1136.8505011553466,1142.8587560539536,1148.8987645796974,1154.9706945505027,1161.074714671202,1167.210994538238,1173.3797046443729,1179.5810163834185,1185.8151020550054,1192.0821348693667,1198.3822889521507,1204.7157393492625,1211.0826620317234,1217.4832339005607,1223.9176327917262,1230.3860374810301,1236.8886276891167,1243.4255840864535,1249.9970882983498,1256.6033229100076,1263.2444714715862,1269.9207185033142,1276.6322495006034,1283.3792509392142,1290.1619102804275,1296.9804159762602,1303.8349574746949,1310.7257252249487,1317.6529106827625,1324.6167063157209,1331.617305608599,1338.6549030687406,1345.7296942314592,1352.8418756654728,1359.9916449783645,1367.179200822075,1374.40474289842,1381.6684719646382,1388.9705898389711,1396.3112994062699,1403.6908046236317,1411.109310526068,1418.5670232321972,1426.0641499499798,1433.6008989824659,1441.1774797335884,1448.7941027139796,1456.4509795468234,1464.1483229737287,1471.8863468606442,1479.6652662038032,1487.4852971356895,1495.3466569310522,1503.2495640129323,1511.1942379587417,1519.1808995063534,1527.2097705602446,1535.2810741976555,1543.3950346747893,1551.5518774330449,1559.7518291052802,1567.9951175221004,1576.2819717182056,1584.6126219387365,1592.9872996456816,1601.4062375243097,1609.8696694896253,1618.3778306928775,1626.930957528089,1635.5292876386247,1644.1730599237949,1652.8625145454935,1661.5978929348648,1670.3794377990262,1679.2073931277941,1688.0820042004743,1697.0035175926746,1705.9721811831514,1714.988244160705,1724.0519570310935,1733.163571624004,1742.323341100036,1751.5315199577494,1760.7883640407276,1770.0941305446813,1779.4490780246088,1788.8534664019699,1798.307556971905,1807.8116124105013,1817.365896782092,1826.9706755465845,1836.6262155668473,1846.3327851161184,1856.0906538854565,1865.9000929912422,1875.7613749826996,1885.6747738494828,1895.6405650292786,1905.659025415459,1915.7304333647785,1925.8550687051106,1936.033212743218,1946.265148272565,1956.5511595811863,1966.8915324595728,1977.2865542086229,1987.7365136476146,1998.24170112224,2008.8024085126722,2019.4189292416627,2030.0915582827047,2040.8205921682274,2051.606328997838,2062.44906844659,2073.349111773333,2084.306761829054,2095.32232306532,2106.3961015427208,2117.528404939374,2128.719542559479,2139.9698253419047,2151.2795658688374,2162.649078374455,2174.078678753664,2185.568684570876,2197.119415068832,2208.731191177472,2220.404335522842,2232.1391724360824,2243.936027962409,2255.795229870188,2267.7171076600516,2279.701992574037,2291.75021760479,2303.8621175048306,2316.0380287958433,2328.27828977803,2340.5832405395063,2352.953222965756,2365.3885807491306,2377.8896593983904,2390.4568062483118,2403.0903704693324,2415.790703077264,2428.5581569430274,2441.393086802471,2454.2958492662233,2467.2668028295916,2480.306307882549,2493.414726719708,2506.5924235504194,2519.8397645088826,2533.1571176643147,2546.544853031171,2560.0033425794404,2573.532960244971,2587.134081939868,2600.8070855629194,2614.5523510101198,2628.370260185205],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Mortgage Interest Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Mortgage Interest Payment","line":{"color":"#FF6692","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Mortgage Interest Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[4228.000000000001,4224.058948096776,4220.097067734241,4216.114248833992,4212.110380735854,4208.085352194818,4204.039051377942,4199.97136586125,4195.882182626602,4191.771388058558,4187.638867941222,4183.484507455067,4179.308191173741,4175.10980306087,4170.889226466821,4166.646344125473,4162.38103815095,4158.093190034353,4153.78268064046,4149.44939020442,4145.093198328425,4140.713983978366,4136.311625480465,4131.886000517905,4127.436986127417,4122.964458695875,4118.468293956858,4113.948366987194,4109.404552203497,4104.836723358667,4100.244753538393,4095.628515157617,4090.987879957,4086.322718999348,4081.632902666034,4076.918300653399,4072.178781969127,4067.414214928609,4062.6244671512814,4057.8094055569513,4052.968896362094,4048.102805076143,4043.210996497745,4038.2933347110106,4033.349683081733,4028.379904253595,4023.38386014435,4018.3614119419876,4013.3124201008754,4008.236744337884,4003.134243628485,3998.004776202836,3992.8481995418433,3987.664370373196,3982.4531446673927,3977.2143776337357,3971.9479237163046,3966.6536365899196,3961.331369156072,3955.9809735388376,3950.602301080765,3945.195202338751,3939.7595270798865,3934.2951242772792,3928.801842105859,3923.2795279381635,3917.7280283400914,3912.1471890666426,3906.536855057636,3900.89687043339,3895.227078490405,3889.527321697002,3883.797441688945,3878.037279265047,3872.2466743827367,3866.425466153625,3860.5734928390207,3854.6905918454513,3848.776599720128,3842.831352146424,3836.8546839392934,3830.8464290406864,3824.8064205149426,3818.7344905441373,3812.630470423438,3806.494190556402,3800.325480450267,3794.1241687112215,3787.8900830396346,3781.6230502252733,3775.3228961424893,3768.9894457453775,3762.6225230629166,3756.2219511940793,3749.787552302914,3743.31914761361,3736.8165574055233,3730.2796010081865,3723.70809679629,3717.1018621846324,3710.460713623054,3703.784466591326,3697.0729355940366,3690.325934155426,3683.5432748142125,3676.7247691183798,3669.870227619945,3662.9794598696913,3656.0522744118775,3649.088478778919,3642.087879486041,3635.0502820258994,3627.9754908631808,3620.863309429167,3613.7135401162755,3606.525984272565,3599.30044219622,3592.0367131300018,3584.734595255669,3577.39388568837,3570.0143804710083,3562.595874568572,3555.1381618624428,3547.6410351446602,3540.104286112174,3532.5277053610516,3524.9110823806604,3517.2542055478166,3509.5568621209113,3501.818838233996,3494.039918890837,3486.2198879589505,3478.358528163588,3470.4556210817077,3462.5109471358983,3454.5242855882866,3446.4954145343954,3438.4241108969845,3430.3101504198507,3422.153307661595,3413.95335598936,3405.7100675725396,3397.4232133764344,3389.0925631559035,3380.7178854489584,3372.2989475703303,3363.8355156050147,3355.3273544017625,3346.774227566551,3338.1758974560153,3329.532125170845,3320.8426705491465,3312.107292159775,3303.325747295614,3294.497791966846,3285.6231808941657,3276.7016675019654,3267.7330039114886,3258.716940933935,3249.6532280635465,3240.541613470636,3231.381843994604,3222.1736651368906,3212.9168210539124,3203.6110545499587,3194.2561070700312,3184.85171869267,3175.397628122735,3165.8935726841387,3156.339288312548,3146.7345095480555,3137.0789695277927,3127.3723999785216,3117.6145312091835,3107.805092103398,3097.9438101119404,3088.030411245157,3078.0646200653614,3068.046159679181,3057.9747517298615,3047.8501163895294,3037.671972351422,3027.440036822075,3017.1540255134537,3006.8136526350672,2996.418630886017,2985.9686714470254,2975.4634839724,2964.902776581968,2954.2862558529773,2943.6136268119353,2932.8845929264126,2922.098856096802,2911.25611664805,2900.356073321307,2889.398423265586,2878.38286202932,2867.3090835519192,2856.176780155266,2844.985642535161,2833.7353597527353,2822.4256192258026,2811.056106720185,2799.626506340976,2788.136500523764,2776.585770025808,2764.973993917168,2753.300849571798,2741.5660126585576,2729.769157132231,2717.909955224452,2705.9880774345884,2694.003192520603,2681.95496748985,2669.8430675898094,2657.6671562987967,2645.42689531661,2633.1219445551337,2620.751962128884,2608.3166043455094,2595.8155256962496,2583.2483788463283,2570.6148146253076,2557.914482017376,2545.1470281516126,2532.312098292169,2519.4093358284167,2506.4383822650484,2493.398877212091,2480.290458374932,2467.1127615442206,2453.8654205857574,2440.5480674303253,2427.160332063469,2413.7018425151996,2400.172224849669,2386.571103154772,2372.8980995317206,2359.1528340845202,2345.334924909435],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Total Mortgage Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Mortgage Payment","line":{"color":"#B6E880","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Total Mortgage Payment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464,4973.70518509464],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Utilities\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Utilities","line":{"color":"#FF97FF","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Utilities","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Downpayment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Downpayment","line":{"color":"#FECB52","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Downpayment","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=PMI\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"PMI","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"PMI","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Sellers Fee\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Sellers Fee","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Sellers Fee","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,293650.4962734258],"yaxis":"y","type":"scattergl"},{"hovertemplate":"variable=Loan Payoff\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Loan Payoff","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Loan Payoff","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,441143.6117472747],"yaxis":"y","type":"scattergl"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"},"type":"log"},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### House Assets
What money will the home bring in? This includes a tax credit for paying interest on a house (limited to 750K over the lifetime of the loan) and the sale of the house in the last month (shown on the log-scale).

In [5]:
render_plotly_html(px.line(pd.DataFrame({"Total Home Assets": df.total_home_assets}), log_y=True, markers=True))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="ab96ca59-c8d6-4476-a42e-5fc4459ab98b" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("ab96ca59-c8d6-4476-a42e-5fc4459ab98b")) {                    Plotly.newPlot(                        "ab96ca59-c8d6-4476-a42e-5fc4459ab98b",                        [{"hovertemplate":"variable=Total Home Assets\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Total Home Assets","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Total Home Assets","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.95208893486053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.08610778693706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.13739075383549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.10131760279104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.97298205914564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.74717344170979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.418357105238048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.980653612252112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.427816551370036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.7532089138939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.9497779346481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.010028296925073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.925993594858495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.689205939581747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.290663588111238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.7207964659921953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4894174.93789043],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"},"type":"log"},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Rent Liabilities
What will renting cost us? Mostly, this is the rent payment. But also includes some other expenses that do not occur when buying a home.

In [6]:
render_plotly_html(
    px.line(
        pd.DataFrame(
            {
                "Rent Payment": df.rent,
                "Renters Insurance": df.renters_insurance,
                "Security Deposit": df.security_deposit_cost,
                "Brokers Fee": df.brokers_fee_cost,
            }
        ),
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="4e6fe5ff-bf84-4c39-9b6c-6906e29a7df4" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("4e6fe5ff-bf84-4c39-9b6c-6906e29a7df4")) {                    Plotly.newPlot(                        "4e6fe5ff-bf84-4c39-9b6c-6906e29a7df4",                        [{"hovertemplate":"variable=Rent Payment\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Rent Payment","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Rent Payment","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3550.0,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3585.5,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3621.355,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3657.5685500000004,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3694.1442355,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3731.0856778550005,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3768.3965346335503,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3806.0804999798856,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3844.1413049796847,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3882.582718029481,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3921.408545209776,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,3960.6226306618737,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4000.228856968493,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4040.2311455381778,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4080.6334569935602,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4121.439791563495,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4162.654189479131,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4204.280731373921,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4246.32353868766,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537,4288.786774074537],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Renters Insurance\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Renters Insurance","line":{"color":"#EF553B","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Renters Insurance","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.5,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,35.855000000000004,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.21355,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.575685500000006,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,36.941442355,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.310856778550004,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,37.683965346335505,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.060804999798854,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.44141304979685,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,38.82582718029481,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.21408545209776,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,39.60622630661874,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.00228856968493,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.40231145538178,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,40.8063345699356,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.214397915634954,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,41.62654189479131,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.04280731373921,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.463235386876605,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537,42.88786774074537],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Security Deposit\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Security Deposit","line":{"color":"#00cc96","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Security Deposit","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[3550.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"},{"hovertemplate":"variable=Brokers Fee\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Brokers Fee","line":{"color":"#ab63fa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Brokers Fee","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Rent Assets
The only positive cash flow in renting is getting back the security deposit in the last month.

In [7]:
render_plotly_html(
    px.line(
        pd.DataFrame({"Rent Assets": df.total_rent_assets}),
        markers=True,
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="7e7e26b4-c8ee-4629-ba87-53287d07343c" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("7e7e26b4-c8ee-4629-ba87-53287d07343c")) {                    Plotly.newPlot(                        "7e7e26b4-c8ee-4629-ba87-53287d07343c",                        [{"hovertemplate":"variable=Rent Assets\u003cbr\u003eindex=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Rent Assets","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"Rent Assets","orientation":"v","showlegend":true,"x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239],"xaxis":"x","y":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3550.0],"yaxis":"y","type":"scatter"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"index"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"value"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"margin":{"t":60}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### Opportunity Costs
If we didn't buy the house, how much could we earn in the market after we pay the rent?

```python
opportunity_cost = (total_home_liability - total_home_assets) - (total_rent_liability - total_rent_assets)
```

We take the future value of each month's cash flow (to the last month's period) and then apply a capital gains tax penalty for selling the asset at the end of the period. Then we compute:

```python
buy_vs_rent = rental_cumulative_opportunity - home_cumulative_opportunity
```

In the last month to see the total advantage of owning a home vs. renting.

In [8]:
df[
    [
        "home_opportunity_cost",
        "rental_opportunity_cost",
        "home_opportunity_cost_fv",
        "rental_opportunity_cost_fv",
        "home_opportunity_cost_fv_post_tax",
        "rental_opportunity_cost_fv_post_tax",
        "home_cumulative_opportunity",
        "rental_cumulative_opportunity",
        "buy_vs_rent",
    ]
]

,home_opportunity_cost,rental_opportunity_cost,home_opportunity_cost_fv,rental_opportunity_cost_fv,home_opportunity_cost_fv_post_tax,rental_opportunity_cost_fv_post_tax,home_cumulative_opportunity,rental_cumulative_opportunity,buy_vs_rent
0,209604.871852,0.000000e+00,1.025843e+06,-0.000000e+00,871966.474075,0.000000e+00,8.719665e+05,0.000000e+00,-8.719665e+05
1,3154.871852,0.000000e+00,1.533824e+04,-0.000000e+00,13037.503844,0.000000e+00,8.850040e+05,0.000000e+00,-8.850040e+05
2,3154.871852,0.000000e+00,1.523666e+04,-0.000000e+00,12951.162759,0.000000e+00,8.979551e+05,0.000000e+00,-8.979551e+05
3,3154.871852,0.000000e+00,1.513576e+04,-0.000000e+00,12865.393470,0.000000e+00,9.108205e+05,0.000000e+00,-9.108205e+05
4,3154.871852,0.000000e+00,1.503552e+04,-0.000000e+00,12780.192188,0.000000e+00,9.236007e+05,0.000000e+00,-9.236007e+05
...,...,...,...,...,...,...,...,...,...
235,7596.642466,0.000000e+00,7.801254e+03,-0.000000e+00,7596.642466,0.000000e+00,2.971966e+06,0.000000e+00,-2.971966e+06
236,7596.642466,0.000000e+00,7.749590e+03,-0.000000e+00,7596.642466,0.000000e+00,2.979562e+06,0.000000e+00,-2.979562e+06
237,7596.642466,0.000000e+00,7.698269e+03,-0.000000e+00,7596.642466,0.000000e+00,2.987159e+06,0.000000e+00,-2.987159e+06
238,7596.642466,0.000000e+00,7.647287e+03,-0.000000e+00,7596.642466,0.000000e+00,2.994755e+06,0.000000e+00,-2.994755e+06


Giving a final opportunity of:

In [9]:
print(df.buy_vs_rent.iloc[-1])

1153478.706195381


## Optimize

In addition to visualizing the cost curves, we can also use `scipy.optimize.minimize_scalar` to answer questions. If we only wanted to stay in the house for 5 years, what would be the optimal mortgage rate to breakeven on our investment?

In [10]:
optimize_kwargs = kwargs.copy()
optimize_kwargs["years"] = 5

In [11]:
minimize_scalar(
    fun=rent_vs_buy_breakeven_objective_closure(
        scalar="mortgage_rate", **optimize_kwargs
    ),
    bounds=[0.0001, 0.1],
)

 message: Solution found.
 success: True
  status: 0
     fun: 8689.864875817322
       x: 0.09999339591456094
     nit: 20
    nfev: 20

We can also maximize the house advantage by optimizing the downpayment for a certain amount of years:

In [12]:
optimization_per_year(
    scalar="downpayment",
    bounds=[0.0, 1.0],
    start_year=1,
    end_year=21,
    kwargs=optimize_kwargs,
)

,fun,status,success,message,x,nfev,nit
1,[23103.309099471895],0,True,Solution found.,[0.9999940391390134],25,25
2,[127799.75501517579],0,True,Solution found.,[0.9999940391390134],25,25
3,[166019.82495555584],0,True,Solution found.,[0.9999940391390134],25,25
4,[195736.4579065186],0,True,Solution found.,[0.9999940391390134],25,25
5,[226646.98079641187],0,True,Solution found.,[0.9999940391390134],25,25
6,[258721.60208122618],0,True,Solution found.,[0.9999940391390134],25,25
7,[291917.17659622687],0,True,Solution found.,[0.9999940391390134],25,25
8,[326175.1932424761],0,True,Solution found.,[0.9999940391390134],25,25
9,[379172.62853051606],0,True,Solution found.,[0.19925364117187222],25,25
10,[438689.434209381],0,True,Solution found.,[0.19925364117187222],25,25


Or, we can see what home price growth rate we would need to breakeven with renting per year:

In [13]:
breakeven_per_year(
    scalar="home_price_growth_rate",
    bounds=[0.0001, 0.1],
    start_year=1,
    end_year=21,
    kwargs=kwargs,
)

,fun,status,success,message,x,nfev,nit
1,8762.269232,0,True,Solution found.,0.099993,20,20
2,2.396666,0,True,Solution found.,0.06651,17,17
3,0.627588,0,True,Solution found.,0.057847,15,15
4,2.871965,0,True,Solution found.,0.054638,13,13
5,7.840696,0,True,Solution found.,0.053182,15,15
6,0.727825,0,True,Solution found.,0.052594,14,14
7,14.16125,0,True,Solution found.,0.052506,13,13
8,3.670759,0,True,Solution found.,0.052725,14,14
9,5.841816,0,True,Solution found.,0.053153,14,14
10,5.531079,0,True,Solution found.,0.053726,13,13


## Vectorize
Since `numpy-financial` is a vectorized library, we can actually see the breakdown for a range of values. For example, we can visualize a 2-d surface of the total opportunity.
### `home_price` and `downpayment`

In [14]:
_x, _y = np.linspace(100_000, 2_000_000, 100), np.linspace(0.00, 1.0, 100)
x, y = np.meshgrid(_x, _y, indexing="ij")
x = x.flatten()
y = y.flatten()
surface_kwargs = kwargs.copy()
surface_kwargs.update({"home_price": x, "downpayment": y})
render_plotly_html(
    px.imshow(
        RentVsBuy().calculate(**surface_kwargs).value.reshape((100, 100)).T,
        aspect="auto",
        labels=dict(x="Home Price", y="Downpayment", color="Opportunity"),
        x=_x,
        y=_y,
        title="Opportunity Surface",
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="187dabbc-29dc-4b42-9f29-85348335ddf2" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("187dabbc-29dc-4b42-9f29-85348335ddf2")) {                    Plotly.newPlot(                        "187dabbc-29dc-4b42-9f29-85348335ddf2",                        [{"coloraxis":"coloraxis","name":"0","x":[100000.0,119191.9191919192,138383.8383838384,157575.75757575757,176767.67676767678,195959.59595959596,215151.51515151514,234343.43434343435,253535.35353535353,272727.2727272727,291919.1919191919,311111.1111111111,330303.0303030303,349494.9494949495,368686.8686868687,387878.78787878784,407070.70707070705,426262.62626262626,445454.5454545454,464646.4646464646,483838.38383838383,503030.303030303,522222.2222222222,541414.1414141414,560606.0606060605,579797.9797979798,598989.898989899,618181.8181818181,637373.7373737374,656565.6565656565,675757.5757575757,694949.494949495,714141.4141414141,733333.3333333333,752525.2525252525,771717.1717171717,790909.0909090908,810101.0101010101,829292.9292929292,848484.8484848484,867676.7676767677,886868.6868686868,906060.606060606,925252.5252525252,944444.4444444444,963636.3636363635,982828.2828282828,1002020.202020202,1021212.1212121211,1040404.0404040404,1059595.9595959596,1078787.878787879,1097979.797979798,1117171.717171717,1136363.6363636362,1155555.5555555555,1174747.4747474748,1193939.3939393938,1213131.313131313,1232323.2323232323,1251515.1515151514,1270707.0707070706,1289898.98989899,1309090.909090909,1328282.8282828282,1347474.7474747475,1366666.6666666665,1385858.5858585858,1405050.505050505,1424242.424242424,1443434.3434343433,1462626.2626262626,1481818.1818181816,1501010.101010101,1520202.0202020202,1539393.9393939392,1558585.8585858585,1577777.7777777778,1596969.6969696968,1616161.616161616,1635353.5353535353,1654545.4545454544,1673737.3737373736,1692929.292929293,1712121.212121212,1731313.1313131312,1750505.0505050505,1769696.9696969695,1788888.8888888888,1808080.808080808,1827272.727272727,1846464.6464646463,1865656.5656565656,1884848.4848484846,1904040.404040404,1923232.3232323232,1942424.2424242422,1961616.1616161615,1980808.0808080807,2000000.0],"y":[0.0,0.010101010101010102,0.020202020202020204,0.030303030303030304,0.04040404040404041,0.05050505050505051,0.06060606060606061,0.07070707070707072,0.08080808080808081,0.09090909090909091,0.10101010101010102,0.11111111111111112,0.12121212121212122,0.13131313131313133,0.14141414141414144,0.15151515151515152,0.16161616161616163,0.17171717171717174,0.18181818181818182,0.19191919191919193,0.20202020202020204,0.21212121212121213,0.22222222222222224,0.23232323232323235,0.24242424242424243,0.25252525252525254,0.26262626262626265,0.27272727272727276,0.2828282828282829,0.29292929292929293,0.30303030303030304,0.31313131313131315,0.32323232323232326,0.33333333333333337,0.3434343434343435,0.3535353535353536,0.36363636363636365,0.37373737373737376,0.38383838383838387,0.393939393939394,0.4040404040404041,0.4141414141414142,0.42424242424242425,0.43434343434343436,0.4444444444444445,0.4545454545454546,0.4646464646464647,0.4747474747474748,0.48484848484848486,0.494949494949495,0.5050505050505051,0.5151515151515152,0.5252525252525253,0.5353535353535354,0.5454545454545455,0.5555555555555556,0.5656565656565657,0.5757575757575758,0.5858585858585859,0.595959595959596,0.6060606060606061,0.6161616161616162,0.6262626262626263,0.6363636363636365,0.6464646464646465,0.6565656565656566,0.6666666666666667,0.6767676767676768,0.686868686868687,0.696969696969697,0.7070707070707072,0.7171717171717172,0.7272727272727273,0.7373737373737375,0.7474747474747475,0.7575757575757577,0.7676767676767677,0.7777777777777778,0.787878787878788,0.797979797979798,0.8080808080808082,0.8181818181818182,0.8282828282828284,0.8383838383838385,0.8484848484848485,0.8585858585858587,0.8686868686868687,0.8787878787878789,0.888888888888889,0.8989898989898991,0.9090909090909092,0.9191919191919192,0.9292929292929294,0.9393939393939394,0.9494949494949496,0.9595959595959597,0.9696969696969697,0.9797979797979799,0.98989898989899,1.0],"z":[[1677570.0126555362,1665012.4677415271,1652454.922827517,1639897.3779135055,1627339.8329994958,1614782.2880854853,1602224.743171474,1589667.1982574647,1577109.6533434538,1564552.108429443,1551994.5635154329,1539437.0186014227,1526879.4736874125,1514321.9287734018,1501764.383859391,1489206.8389453809,1476649.2940313704,1464099.0618160905,1451558.9953064616,1439026.467972908,1426500.3931898568,1413979.8041776377,1401463.842012509,1388951.7447764138,1376442.837741739,1363936.5244956962,1351432.278917486,1338930.6024347323,1326430.6118902497,1313931.5858511652,1301434.8942221487,1288938.5395265957,1276444.0122990515,1263949.615908466,1251456.9797839199,1238964.3436593693,1226472.9248546558,1213981.966715786,1201491.0085769147,1188993.7991611753,1176501.1630366212,1164008.5269120736,1151498.7400236093,1139004.212796065,1126509.6855685161,1113985.1387672867,1101488.4884036295,1088991.838039972,1076450.0911085722,1063951.0650694864,1051452.0390304038,1038953.0129913213,1026388.6903658588,1013887.0138831134,1001385.3374003549,988883.6609175978,976293.1250159782,963788.4992263014,951283.8734366414,938779.247646973,926274.6218573009,913650.5349251823,901142.6348490575,888634.7347729271,876126.834696807,863618.9346206933,851111.0345445676,838603.1344684372,825933.0006236006,813421.4729757672,800909.9453279227,788398.4176800922,775886.8900322439,763375.3623844106,750863.8347365726,738352.3070887215,725840.779440877,713109.1869999794,700593.6478179367,688078.1086358903,675562.5694538383,663047.0302717872,650531.4910897482,638015.9519077064,625500.4127256591,612984.8735436052,600469.3343615653,587661.9211490676,575141.9532244047,562621.9852997279,550102.0173750557,537582.0494503686,525062.0815256946,512542.11360103637,500022.1456763549,487502.1777516706,474982.2098269984,462462.2419023244,449942.2739776457,437422.30605297815],[1677513.4003917812,1664944.99049786,1652376.5806039397,1639808.170710015,1627239.7608160921,1614671.3509221696,1602102.9410282467,1589534.5311343237,1576966.1212404012,1564397.711346478,1551829.3014525548,1539260.8915586327,1526692.4816647095,1514124.0717707868,1501555.6618768636,1488987.2519829413,1476418.8420890183,1463855.0780044545,1451301.7933870465,1438756.739779772,1426218.1554853122,1413685.0787229172,1401156.6547555807,1388632.1251332783,1376110.817961669,1363592.1391018238,1351076.1137956344,1338562.4584247053,1326050.1494552563,1313539.6673365105,1301030.6746906436,1288522.5442506045,1276016.0346045592,1263510.0058548155,1251005.2500435729,1238501.0932947753,1225997.4553741652,1213494.959661695,1200992.463949224,1188489.9682367505,1175980.309550777,1163476.1528019817,1150955.7471071165,1138449.7183573726,1125943.689607629,1113408.7180900695,1100900.5876500322,1088392.4572099955,1075840.5070803985,1063330.0249616518,1050819.5428429046,1038309.0607241695,1025734.8136683158,1013221.7078780611,1000708.6020878083,988195.4962975467,975682.3905072883,963079.3293037373,950563.3039975436,938047.2786913561,925531.2533851666,913015.2280789744,900378.6902379775,887859.4237189274,875340.1571998876,862820.8906808468,850301.6241617994,837782.3576427521,825263.0911237029,812580.6824966203,800057.8250479987,787534.9675993705,775012.1101507507,762489.2527021281,749966.3952535018,737443.5378048867,724920.6803562641,712181.0893301051,699654.260867835,687127.4324055491,674600.6039432678,662073.775480995,649546.9470187007,637020.1185564203,624493.2900941446,611966.4616318587,599439.6331695747,586912.8047073036,574093.8432237953,561562.6307536336,549031.41828351,536500.2058133688,523968.9933432294,511437.78087310493,498906.56840296276,486375.3559328187,473844.1434626933,461312.93099255674,448781.71852241363,436250.50605227426],[1677458.4750543765,1664879.5239340847,1652300.5728137887,1639721.621693499,1627142.6705732048,1614563.7194529115,1601984.7683326185,1589405.8172123246,1576826.86609203,1564247.9149717374,1551668.9638514444,1539090.0127311507,1526511.0616108573,1513932.110490564,1501353.15937027,1488774.2082499769,1476195.2571296834,1463619.2011629846,1451052.5930103878,1438494.8498302787,1425944.0797779527,1413398.8390189228,1400858.2770029223,1388321.6387479524,1375788.25519295,1363257.9459845256,1350730.4334036738,1338204.742686655,1325680.531342441,1313158.8246009175,1300637.8546917408,1288118.5679288602,1275600.103949754,1263082.897431352,1250566.2148499992,1238050.861230488,1225535.5076109853,1213021.618414999,1200507.9088824666,1187994.1993499412,1175474.0931329522,1162958.7395134429,1150443.3858939377,1137910.8322473322,1125393.6257289257,1112876.4192105243,1100329.2664857055,1087809.9797228328,1075290.692959954,1062726.5357037098,1050204.9212588444,1037683.3068139898,1025161.6923691318,1012575.2477370733,1000051.0363928569,987526.8250486478,975002.6137044313,962390.300259388,949863.1991902185,937336.0981210489,924808.9970518798,912281.895982705,899754.7949135322,887106.1960644126,874575.8868559832,862045.5776475612,849515.2684391364,836984.9592307108,824454.6500222851,811924.3408138594,799230.0544423433,786696.1909464793,774162.3274506126,761628.4639547579,749094.600458893,736560.7369630188,724026.8734671641,711493.0099712983,698741.8136203354,686204.0196313541,673666.2256423747,661128.4316534093,648590.637664427,636052.8436754588,623515.0496864915,610977.2556975111,598439.4617085308,585901.6677195514,573363.8737305738,560533.7771991352,547991.6439370727,535449.5106750187,522907.37741295435,510365.2441508956,497823.11088884063,485280.97762678564,472738.8443647269,460196.7111026887,447654.5778406244,435112.4445785731],[1677476.0630779527,1664900.487436934,1652324.911795911,1639749.3361548905,1627173.76051387,1614598.184872849,1602022.609231827,1589447.0335908048,1576871.4579497841,1564295.8823087623,1551720.3066677414,1539144.7310267193,1526569.1553856984,1513993.5797446773,1501418.0041036555,1488842.4284626343,1476266.8528216132,1463692.4216784227,1451127.4410509022,1438570.6200144417,1426020.8515444295,1413477.3634944079,1400938.5801353818,1388403.7499533892,1375872.7971159266,1363344.7715284768,1350819.1007194077,1338295.2913619387,1325773.8911694102,1313254.0796899446,1300735.2496246097,1288218.5228430259,1275702.0212364965,1263187.5536550798,1250673.0860736605,1238160.2272246832,1225647.593440111,1213135.7086380627,1200624.701991117,1188113.695344171,1175602.6886972198,1163084.4245172418,1150571.79073267,1138042.8778409003,1125528.410259481,1113013.9426780636,1100470.627141925,1087954.1007618592,1075437.5743817808,1062877.4783832477,1050358.6483179098,1037839.8182525737,1025320.9881872432,1012738.8863561377,1000217.4861636027,987696.0859710719,975174.6857785401,962653.2855860111,950042.7766711586,937518.5165446438,924994.256418136,912469.9962916309,899945.7361651259,887421.476038618,874774.7990341932,862247.3638420366,849719.928649867,837192.4934577141,824665.0582655538,812137.6230733888,799448.9665926695,786918.0137551986,774387.060917737,761856.1080802735,749325.1552428165,736794.2024053549,724263.2495678915,711732.2967304243,699201.3438929562,686452.5399639364,673917.6971539026,661382.8543438809,648848.0115338536,636313.1687238198,623778.3259137999,611243.4831037708,598708.6402937444,586173.7974837301,573638.9546736935,561104.1118636727,548276.8864594735,535737.7491111215,523198.6117628021,510659.47441445384,498120.33706612885,485581.1997177927,473042.06236946303,460502.9250211334,447963.78767279163,435424.6503244657],[1677425.5670667505,1664840.3002720682,1652255.0334773806,1639669.766682694,1627084.4998880066,1614499.2330933218,1601913.9662986328,1589328.699503948,1576743.432709262,1564158.1659145756,1551572.8991198887,1538987.632325203,1526402.3655305165,1513817.0987358303,1501231.8319411438,1488646.5651464576,1476061.2983517712,1463476.031557085,1450899.0056637714,1438330.755784445,1425769.5314959716,1413214.3138222578,1400664.1965758875,1388118.3750444348,1375576.1357544984,1363036.8472142206,1350499.9515439984,1337965.4657272135,1325433.301239362,1312902.3183890174,1300373.2060702695,1287845.3281013593,1275318.495471153,1262792.9269571023,1250268.1316797298,1237744.1770313003,1225221.1964487236,1212698.2494076781,1200176.8790133782,1187655.5086190784,1175134.1382247768,1162606.2935358407,1150083.3129532607,1137560.3323706836,1125020.17890603,1112495.3836286562,1099970.5883512823,1087416.0060180551,1074889.1733878553,1062362.3407576499,1049790.9596953387,1037261.8473765901,1024732.7350578369,1012203.6227390915,999610.2537209685,987078.5980471848,974546.9423733973,962015.2866996112,949483.6310258289,936861.9195901798,924327.4337733961,911792.9479566179,899258.4621398384,886723.9763230495,874069.3626696551,861531.7348608263,848994.1070519891,836456.4792431528,823918.8514343128,811381.2236254802,798843.5958166448,786144.0215906026,773602.9127786113,761061.8039666135,748520.6951546129,735979.5863426151,723438.4775306117,710897.3687186111,698356.2599066133,685815.1510946248,673055.7539956709,660510.79573166,647965.8374676537,635420.8792036371,622875.9209396243,610330.962675604,597786.0044115912,585241.0461475737,572696.087883559,560151.1296195462,547606.1713555288,534768.8399151722,522219.631847634,509670.42378010415,497121.2157125473,484572.00764500257,472022.79957745597,459473.59150990844,446924.38344236,434375.1753748115],[1677450.3241908187,1664869.8087633816,1652289.2933359395,1639708.7779084966,1627128.2624810555,1614547.7470536153,1601967.2316261732,1589386.7161987335,1576806.2007712915,1564225.6853438504,1551645.169916409,1539064.6544889687,1526484.1390615278,1513903.6236340865,1501323.1082066454,1488742.5927792042,1476162.0773517634,1463581.5619243218,1451006.0988566312,1438440.552587013,1425882.3586867023,1413330.1935633072,1400783.1556009962,1388240.4438797198,1375701.3480301863,1363165.239057991,1350631.8611533924,1338101.2920613757,1325572.3550300715,1313044.8863959417,1300519.691278508,1287995.1046431803,1275472.4082837529,1262950.156584432,1250429.4761320231,1237908.8682050474,1225389.983345375,1212871.0984857054,1200353.1249654996,1187835.8333447585,1175318.541724014,1162801.2501032692,1150276.674187358,1137757.7893276913,1125222.6716078962,1112701.9911554847,1100181.310703067,1087631.9628923098,1075109.2665328868,1062586.5701734559,1050063.8738140278,1037495.6982023013,1024970.7461510506,1012445.7940997998,999920.8420485533,987330.7019086243,974803.2332744966,962275.7646403695,949748.2960062386,937132.6845046896,924602.4155185446,912072.1465323959,899541.8775462443,887011.6085600983,874481.3395739486,861830.1228740495,849296.7449694397,836763.3670648457,824229.9891602509,811696.6112556532,799163.233351049,786629.8554464513,773933.8792802887,761397.05701466,748860.2347490368,736323.4124834277,723786.5902178017,711249.7679521758,698712.9456865527,686176.1234209267,673639.3011553027,660883.83444844,648343.203251359,635802.5720542548,623261.9408571739,610721.3096600669,598180.6784629906,585640.0472659059,573099.4160688072,560558.7848717188,548018.1536746174,535477.5224775393,522644.61699148826,510099.7807256449,497554.9444598006,485010.1081939461,472465.2719280943,459920.4356622482,447375.59939640295,434830.7631305456],[1677555.332117627,1664994.9697266459,1652434.607335661,1639874.2449446756,1627313.8825536927,1614753.5201627102,1602193.1577717266,1589632.7953807425,1577072.4329897596,1564512.0705987767,1551951.708207793,1539391.3458168092,1526830.983425826,1514270.621034843,1501710.2586438595,1489149.8962528764,1476589.5338618928,1464029.1714709094,1451472.0319629177,1438923.7086103028,1426383.4700644214,1413849.7591605573,1401321.2021515165,1388797.00190963,1376276.451169711,1363759.3171196748,1351245.1971036622,1338733.214267013,1326222.9063614667,1313715.0264362071,1301208.4773658596,1288703.0872513214,1276199.4253138874,1263696.2668944483,1251194.7281402738,1238693.5641820312,1226193.481136415,1213694.0936688643,1201194.8851504293,1188697.0739724548,1176199.2627944811,1163701.4516165121,1151197.156331115,1138697.7688635653,1126198.381396017,1113681.9245996051,1101180.760641362,1088679.5966831162,1076148.890602856,1063645.7321834122,1051142.5737639735,1038595.2849605796,1026089.8948460459,1013584.504731508,1001079.114616971,988510.148379493,976002.2684542327,963494.3885289789,950986.5086037158,938478.6286784583,925881.7482191673,913371.0977328671,900860.4472465748,888349.796760276,875839.1462739808,863328.4957876848,850696.1438576737,838182.4175265199,825668.6911953678,813154.9648642195,800641.2385330694,788127.5122019192,775613.7858707672,762936.882717968,750419.7486679358,737902.6146178953,725385.480567866,712868.3465178227,700351.2124677999,687834.0784177687,675316.9443677254,662799.8103176914,650063.7567132208,637542.8542522667,625021.9517913116,612501.0493303519,599980.1468693977,587459.2444084352,574938.3419474866,562417.4394865269,549896.5370255644,537375.6345646121,524854.732103657,512041.74371070415,499516.6809157524,486991.6181208156,474466.5553258732,461941.4925309252,449416.4297359856,436891.3669410469],[1677590.6498892496,1665037.0656564527,1652483.481423663,1639929.8971908689,1627376.312958076,1614822.7287252825,1602269.14449249,1589715.560259697,1577161.9760269038,1564608.3917941097,1552054.8075613172,1539501.2233285238,1526947.6390957308,1514394.0548629372,1501840.470630144,1489286.886397351,1476733.3021645576,1464179.7179317651,1451627.5271052048,1439084.1620438164,1426548.4120894459,1414019.1984707885,1401495.5611678879,1388976.6470226664,1376461.6989811137,1363950.046362913,1351441.0960637885,1338934.324604246,1326429.9177395275,1313927.6048515297,1301426.3269499994,1288927.1238939043,1276428.7560842773,1263931.7607265597,1251435.5441322431,1238940.5217899038,1226445.7931546366,1213952.5282009388,1201459.263247245,1188967.004192546,1176475.298579073,1163983.5929656005,1151491.8873521276,1138992.9384787572,1126499.67352506,1113990.2983665052,1101495.2760241646,1089000.2536818236,1076476.8303607926,1063979.8350030738,1051482.839645356,1038985.8442876367,1026443.8597159311,1013944.6566598397,1001445.4536037426,988946.2505476521,976382.6202957965,963880.9542011423,951379.2881064885,938877.6220118329,926375.9559171726,913784.491203839,901280.0843391214,888775.6774744093,876271.2706096824,863766.8637449751,851262.4568802482,838635.6609890796,826128.2113530999,813620.7617171425,801113.3120811693,788605.8624452055,776098.4128092295,763590.9631732684,750919.8314398462,738409.0107271187,725898.1900144173,713387.3693016851,700876.5485889809,688365.72787626,675854.9071635511,663344.0864508236,650833.2657381175,638103.3313988652,625588.7827957859,613074.2341926796,600559.685589606,588045.1369865034,575530.5883834362,563016.0397803271,550501.4911772506,537986.9425741499,525472.3939710716,512957.8453679783,500151.48865938466,487632.8244570829,475114.1602547886,462595.496052484,450076.831850186,437558.1676478842],[1677710.2680745327,1665179.6408671965,1652649.0136598619,1640118.386452528,1627587.7592451945,1615057.132037859,1602526.5048305239,1589995.8776231897,1577465.2504158558,1564934.6232085214,1552403.996001186,1539873.3687938515,1527342.7415865173,1514812.1143791822,1502281.4871718476,1489750.8599645128,1477220.232757179,1464689.605549844,1452158.9783425094,1439636.3143690897,1427121.7182028196,1414613.6809452646,1402111.2475654646,1389613.5690495952,1377119.891741654,1364629.547701523,1352141.9459865608,1339656.951913043,1327174.5803955146,1314693.6708312952,1302214.397434508,1289737.005835469,1277260.1687050886,1264785.457592194,1252310.8041634643,1239838.040165834,1227365.3383066617,1214893.8812356058,1202422.9176630308,1189952.210649752,1177482.7894680514,1165013.3682863517,1152543.94710465,1140068.0998001467,1127597.1362275728,1115126.1726549943,1102638.319714977,1090165.6178558087,1077692.9159966363,1065191.0027325503,1052716.3493038202,1040241.6958750952,1027723.426097448,1015246.58896707,1002769.7518366887,990292.9147063037,977753.2891180608,965274.015721275,952794.7423244896,940315.4689277061,927836.1955309175,915269.0959918397,902787.1116159773,890305.127240112,877823.1428642422,865341.1584883798,852859.1741125053,840257.188972285,827772.1948987544,815287.2008252367,802802.2067517154,790317.2126781987,777832.2186046746,765347.2245311616,752862.230457644,740213.1222951431,727724.7937870156,715236.4652788835,702748.1367707597,690259.8082626332,677771.4797545057,665283.1512463745,652794.8227382451,640306.4942301223,627598.9390644664,615106.9231865099,602614.9073085627,590122.8914306089,577630.8755526599,565138.8596746996,552646.8437967533,540154.8279187968,527662.8120408468,515170.79616289306,502678.7802849356,490186.7644069893,477399.23685511574,464903.150112723,452407.0633703368,439910.9766279496],[1677838.4833937075,1665332.4631668166,1652826.4429399313,1640320.4227130448,1627814.4024861571,1615308.382259269,1602802.362032382,1590296.3418054965,1577790.3215786077,1565284.3013517212,1552778.2811248335,1540272.2608979463,1527766.240671059,1515260.220444172,1502754.2002172847,1490248.1799903975,1477742.1597635103,1465236.1395366224,1452730.1193097355,1440228.9274250353,1427736.6205498572,1415251.4096083967,1402771.830106683,1390297.0371751543,1377826.2805558187,1365358.895048744,1352894.708700194,1340433.4033168491,1327974.0389250778,1315516.1826396277,1303060.9518956998,1290606.633515576,1278153.812265897,1265702.2001063437,1253251.538561595,1240801.9041002635,1228353.1726792548,1215904.8797387509,1203457.8675022875,1191010.855265818,1178564.8756904774,1166119.3888955312,1153673.9021005915,1141228.415305649,1128775.7940834821,1116328.7818470118,1103865.8584691668,1091417.1270481516,1078968.3956271457,1066491.6153903706,1054040.9538456239,1041590.2923008772,1029139.6307561309,1016644.5660225963,1004191.7447729143,991738.9235232268,979286.1022735406,966769.7218096531,954314.4910657271,941859.260321802,929404.0295778741,916948.7988339486,904405.0332360314,891947.1213039979,879489.2093719598,867031.2974399393,854573.3855078947,842115.4735758686,829536.972517537,817076.083961457,804615.1954053771,792154.3068492943,779693.4182932219,767232.5297371363,754771.6411810583,742149.5776212132,729685.3912726594,717221.2049241196,704757.0185755575,692292.8322270075,679828.6458784575,667364.4595299112,654900.2731813611,642436.0868327944,629971.9004842583,617288.4554882646,604820.6222904492,592352.7890926087,579884.9558947897,567417.122696951,554949.2894991338,542481.4563013008,530013.6231034761,517545.7899056487,505077.9567078091,492610.1235099882,480142.29031214677,467379.4469407927,454907.58761328086,442435.7282857979],[1677975.4263747286,1665495.688134096,1653015.9498934683,1640536.2116528372,1628056.473412206,1615576.7351715758,1603096.9969309454,1590617.2586903144,1578137.520449684,1565657.7822090543,1553178.0439684237,1540698.3057277936,1528218.5674871632,1515738.8292465333,1503259.0910059027,1490779.3527652721,1478299.6145246418,1465819.8762840114,1453340.1380433813,1440863.3198471856,1428394.7414134475,1415933.2615253474,1403477.9904375605,1391027.7580961795,1378581.8011893951,1366139.6216593632,1353700.4760198859,1341263.8075393573,1328829.1252791542,1316396.960796768,1303966.4841930233,1291536.9711077663,1279109.7836744562,1266682.9454740551,1254257.9508190956,1241833.1208998687,1229410.0349228312,1216986.9489457891,1204565.1952241473,1192143.8093295987,1179722.6898071305,1167302.8124047532,1154882.9350023824,1142463.0576000093,1130036.8798568519,1117615.4939623047,1105194.1080677602,1092756.0891753915,1080333.003198353,1067909.9172213152,1055458.0363397477,1043033.0416847868,1030608.0470298245,1018183.0523748603,1005712.9156846874,993285.7853215178,980858.6549583469,968431.5245951791,955940.1178057687,943510.6047205101,931081.0916352505,918651.5785499886,906222.065464735,893703.3683957327,881271.2039133487,868839.0394309647,856406.8749485826,843974.7104661893,831542.54598381,818989.2701603789,806554.1621275488,794119.0540947132,781683.9460618896,769248.8380290456,756813.7299962174,744378.6219633836,731782.0168616772,719343.6476785168,706905.2784953378,694466.9093121784,682028.5401290143,669590.1709458502,657151.8017626889,644713.4325795295,632275.0633963635,619836.694213206,607179.1154337926,594737.1399218962,582295.1644100109,569853.1888980893,557411.2133862004,544969.2378743021,532527.2623624131,520085.2868505139,507643.31133862864,495201.33582672756,482759.36031483766,470317.3848029496,457580.9900874216,445135.0331806382],[1678208.8132605522,1665773.866442451,1653338.919624355,1640903.9728062549,1628469.0259881564,1616034.0791700578,1603599.1323519598,1591164.1855338602,1578729.2387157618,1566294.2918976636,1553859.3450795643,1541424.398261465,1528989.4514433667,1516554.504625268,1504119.5578071687,1491684.6109890705,1479249.6641709716,1466814.7173528732,1454379.7705347743,1441945.8354633579,1429520.1566724202,1417101.5994213324,1404689.1528640226,1392281.917684356,1379879.0949065248,1367479.9757724076,1355083.9325875237,1342690.4104460892,1330299.592372781,1317911.0121572614,1305523.743483395,1293138.4574930766,1280754.5009486119,1268371.570908225,1255990.1278147448,1243609.3094858453,1231229.6807771018,1218850.7496803044,1206472.3254292523,1194095.0753128934,1181717.8251965404,1169341.566706047,1156965.8081325185,1144590.0495589809,1132214.2909854557,1119831.5746147647,1107454.3244984136,1095077.0743820532,1082682.5076155537,1070303.576518761,1057924.645421959,1045516.2162235035,1033135.3978946102,1020754.579565715,1008330.0400601155,995947.1100197332,983564.1799793374,971181.2499389565,958798.3198985714,946350.4497642354,933965.1637739176,921579.8777835993,909194.5917932885,896809.3058029683,884334.2462811219,871946.3386846604,859558.4310881896,847170.523491729,834782.6158952694,822394.7082988108,809885.2332937634,797494.4152204376,785103.5971471351,772712.7790738251,760321.9610005245,747931.1429271959,735540.3248538887,722987.7609308725,710593.7183493711,698199.6757678725,685805.6331863608,673411.5906048734,661017.5480233543,648623.5054418584,636229.4628603542,623835.4202788388,611441.377697356,598828.2556118835,586430.6472221836,574033.0388325118,561635.4304428166,549237.8220531465,536840.213663457,524442.6052737581,512044.99688408803,499647.3884943798,487249.78010472097,474852.17171501834,462454.56332533434,449763.21617147885],[1678548.1532007589,1666178.3322297698,1653808.5112587817,1641438.690287793,1629068.8693168045,1616699.0483458166,1604329.227374826,1591959.40640384,1579589.5854328512,1567219.764461864,1554849.943490875,1542480.1225198857,1530110.3015488978,1517740.4805779096,1505370.659606921,1493000.838635933,1480631.0176649443,1468261.1966939566,1455891.3757229678,1443521.5547519794,1431158.213477351,1418802.9132878534,1406453.7522280249,1394109.8355076788,1381770.3678679275,1369434.643554305,1357102.0372485938,1344772.6556933292,1332445.8021816232,1320120.7148849233,1307797.398598964,1295476.2640947127,1283155.8728636696,1270837.477570647,1258519.6283004486,1246203.3207221946,1233887.3189705906,1221572.8771786168,1209258.4353866465,1196945.3369091526,1184632.5569955679,1172319.9854694456,1160008.6801493363,1147697.3748292332,1135386.0695091207,1123074.7641890086,1110755.8775140685,1098443.0976004861,1086114.0174669148,1073799.575674951,1061485.133882971,1049142.39947013,1036826.091891876,1024509.7843136224,1012193.4767353651,999832.7811241499,987514.3858311293,975195.9905381091,962877.5952450763,950495.9789124853,938175.2544416934,925854.5299708927,913533.8055001022,901213.0810293062,888804.6448763097,876481.3285903493,864158.0123043777,851834.6960184276,839511.3797324672,827188.0634464994,814864.747160539,802419.4735456454,790093.2798564415,777767.0861672265,765440.8924780199,753114.6987888133,740788.5050996104,728462.3114103982,715974.1963749146,703644.8148196572,691315.4332643868,678986.0517091248,666656.6701538628,654327.2885986026,641997.9070433443,629668.5254880814,617339.1439328203,605009.7623775387,592461.5124517009,580128.6056088042,567795.698765899,555462.7919230126,543129.8850801159,530796.978237222,518464.0713943131,506131.1645514164,493798.2577085076,481465.3508656211,469132.4440227188,456799.5371798277],[1678814.4072519273,1666495.6855432857,1654176.9638346408,1641858.2421260006,1629539.5204173569,1617220.7987087134,1604902.077000072,1592583.3552914304,1580264.6335827855,1567945.9118741439,1555627.1901655004,1543308.4684568585,1530989.7467482162,1518671.025039574,1506352.3033309306,1494033.581622288,1481714.8599136458,1469396.1382050028,1457077.41649636,1444758.6947877177,1432443.946835746,1420136.8940928755,1407836.6808381665,1395542.0559923772,1383251.9169059433,1370965.8378344995,1358683.3289471723,1346403.6177029533,1334126.1817280166,1321850.5858381425,1309577.8092051498,1297306.1606796097,1285035.971143571,1272767.4210467974,1260499.5339163747,1248233.315602745,1235967.4921903631,1223702.9351516026,1211438.9560796851,1199175.4758105418,1186913.1395149701,1174650.8032193924,1162389.349717006,1150128.4710655566,1137867.5924141053,1125606.7137626573,1113339.121741517,1101076.7854459449,1088814.449150369,1076535.1862886264,1064271.207216707,1052007.2281447975,1039714.3175923447,1027448.4941799645,1015182.6707675881,1002916.8473552158,990606.0170470513,978338.1299166298,966070.242786211,953802.3556557852,941470.6641345434,929200.4745985139,916930.2850624751,904660.0955264335,892389.9059904022,880031.5045973007,867758.7530370699,855486.0014768532,843213.2499166336,830940.4983563982,818667.7467961786,806275.5584635232,793999.9625736438,781724.3666837774,769448.7707939092,757173.174904041,744897.579014156,732621.9831242915,720346.3872344242,707908.7677859925,695630.0206722058,683351.273558422,671072.526444627,658793.7793308441,646515.0322170574,634236.2851032652,621957.5379894832,609678.7908757031,597400.0437618978,584902.7204520181,572620.4885711335,560338.2566902544,548056.0248093894,535773.7929285141,523491.56104762666,511209.3291667644,498927.0972858863,486644.8654050054,474362.63352413476,462080.4016432604],[1679194.680754612,1666948.9408293136,1654703.2009040134,1642457.460978714,1630211.7210534161,1617965.9811281161,1605720.2412028178,1593474.5012775194,1581228.7613522194,1568983.0214269205,1556737.2815016205,1544491.5415763217,1532245.8016510233,1520000.0617257245,1507754.3218004247,1495508.5818751256,1483262.8419498266,1471017.1020245273,1458771.3620992284,1446525.6221739296,1434281.868886556,1422045.8342345278,1409816.4491132298,1397592.7616480868,1385373.9254595302,1373159.18904577,1360947.886180394,1348739.4272310347,1336533.2913138645,1324330.1757443259,1312128.7148927834,1299928.434866951,1287730.6627866677,1275533.5071673892,1263338.0107205643,1251143.218169729,1238949.7614442261,1226756.665853017,1214565.0320221586,1202373.3981912984,1190183.0423257947,1177993.0321692298,1165803.1046180995,1153614.5351563017,1141425.9656945113,1129237.3962327144,1117048.8267709156,1104852.9712298084,1092662.961073243,1080457.0598827135,1068265.4260518542,1056073.7922209962,1043882.158390135,1031660.9971107054,1019467.5403852109,1007274.0836597034,995037.0888903751,982841.5924435463,970646.0959967184,958450.5995498952,946255.1031030668,933995.2724585067,921797.5003782157,909599.7282979386,897401.9562176596,885204.1841373816,872917.7596071335,860717.4552936405,848517.1509801475,836316.846666663,824116.5423531728,811916.2380396752,799596.2062542159,787393.090684684,775189.9751151428,762986.8595456108,750783.7439760631,738580.6284065247,726377.5128369955,714174.3972674599,701809.2292113611,689602.9990600413,677396.7689087326,665190.5387574229,652984.3086061124,640778.0784547972,628571.8483034782,616365.6181521574,604159.3880008506,591953.1578495353,579528.7247176832,567319.0503198365,555109.3759219637,542899.7015241161,530690.0271262666,518480.3527284097,506270.67833056394,494061.0039326893,481851.329534838,469641.65513697825],[1679701.8129237592,1667553.4013945577,1655404.9898653561,1643256.578336155,1631108.1668069544,1618959.7552777533,1606811.3437485546,1594662.9322193528,1582514.520690153,1570366.1091609513,1558217.6976317496,1546069.2861025503,1533920.8745733502,1521772.4630441498,1509624.0515149485,1497475.6399857476,1485327.2284565475,1473178.8169273469,1461030.4053981458,1448881.9938689456,1436733.5823397455,1424592.9119104943,1412458.9208516167,1400330.661747668,1388207.2922819876,1376088.0642423765,1363972.3140117063,1351859.5437096297,1339750.0873938776,1327642.6760868628,1315536.8776295274,1303433.3671755248,1291331.3047043858,1279230.2966926133,1267131.1310248822,1255032.3718744074,1242935.3674237258,1230838.6239978592,1218743.1379157742,1206648.1959387306,1194553.6683132867,1182460.330908474,1170366.9935036711,1158274.362261976,1146182.4486025893,1134090.5349432034,1121998.621283819,1109900.3064796194,1097806.969074816,1085713.6316699986,1073603.8341912078,1061508.8922141637,1049413.9502371112,1037290.7291839286,1025193.9857580494,1013097.242332181,1001000.4989063074,988859.6705639567,976760.9114134805,964662.1522630067,952563.3931125365,940464.6339620622,928301.0643512392,916200.0563394642,904099.0483276984,891998.0403159279,879897.0323041519,867706.9908314804,855603.4803774739,843499.9699234758,831396.459469472,819292.949015473,807189.4385614758,794965.9760831529,782859.687446706,770753.3988102535,758647.1101738177,746540.8215373689,734434.5329009155,722328.2442644723,710221.9556280291,697953.6588612907,685844.2922852049,673734.9257091191,661625.5591330379,649516.1925569447,637406.8259808524,625297.4594047815,613188.0928286891,601078.7262525959,588969.3596765297,576642.2443765523,564529.4740744745,552416.7037723986,540303.9334703237,528191.1631682394,516078.39286615327,503965.6225640774,491852.8522619968,479740.0819599191],[1680351.7645422039,1668328.0912024016,1656304.4178626002,1644280.744522796,1632257.0711829937,1620233.397843191,1608209.724503389,1596186.0511635882,1584162.377823785,1572138.7044839829,1560115.0311441808,1548091.3578043801,1536067.6844645776,1524044.0111247757,1512020.3377849732,1499996.6644451716,1487972.9911053698,1475949.3177655677,1463925.6444257651,1451901.9710859635,1439878.2977461615,1427859.178949303,1415847.2522575413,1403841.8065424522,1391841.3135929494,1379845.0512211812,1367852.8343215345,1355863.7595051227,1343877.2713697334,1331892.8798844027,1319910.827974343,1307930.84681336,1295951.8794570034,1283975.0453070418,1271999.0037507028,1260024.391689884,1248050.5454113332,1236077.9250783925,1224105.6354041034,1212134.7950741611,1200163.9547442282,1188194.250622326,1176224.9957625782,1164255.7409028327,1152287.7818651763,1140319.9338014992,1128352.0857378235,1116384.2376741488,1104409.4666040875,1092440.2117443383,1080470.956884595,1068484.7111149249,1056513.8707849933,1044543.0304550501,1032543.3794184062,1020570.7590854578,1008598.1387525215,996625.5184195791,984608.3142909487,972633.7022301266,960659.090169304,948684.4781084768,936646.8549581645,924670.0208081938,912693.186658239,900716.3525082804,888739.5183583153,876762.6842083605,864696.4951685071,852717.1883673063,840737.8815660747,828758.574764871,816779.2679636506,804799.9611624507,792700.5439317767,780718.492021719,768736.4401116734,756754.3882016037,744772.3362915423,732790.2843814902,720808.2324714428,708826.1805613749,696682.2381615769,684697.1449540257,672712.0517464764,660726.958538915,648741.8653313434,636756.7721237894,624771.6789162401,612786.5857086815,600801.4925011126,588816.3992935531,576614.0926598851,564625.6362468842,552637.1798338788,540648.7234208705,528660.2670078753,516671.81059487443,504683.35418186337,492694.897768856],[1681048.1249209333,1669158.096502302,1657268.0680836695,1645378.039665036,1633488.0112464044,1621597.982827771,1609707.9544091397,1597817.9259905084,1585927.8975718766,1574037.869153244,1562147.8407346127,1550257.812315981,1538367.7838973498,1526477.7554787176,1514587.7270600866,1502697.698641454,1490807.6702228226,1478917.6418041904,1467027.613385559,1455137.5849669275,1443247.5565482941,1431360.016732867,1419479.699357007,1407605.5976904104,1395736.8158196588,1383872.557523489,1372012.1162101396,1360154.8658187303,1348300.2525954803,1336448.3394882225,1324598.8792117042,1312750.8438964013,1300904.647479751,1289060.1248302213,1277216.371273842,1265374.555877648,1253532.9976382495,1241693.1839947621,1229853.593486519,1218015.1655089809,1206177.333557922,1194339.7550991056,1182503.4895161847,1170667.2239332707,1158831.4200859936,1146996.5443498031,1135161.6686136103,1123326.792877417,1111491.9171412191,1099649.6304357727,1087813.3648528522,1075961.1820963346,1064123.3501452822,1052285.5181942265,1040447.6862431676,1028580.5548464688,1016740.9643382262,1004901.3738299944,993061.7833217499,981177.1581320581,969335.5998926535,957494.0416532569,945652.4834138528,933747.518138825,921903.7645824505,910060.0110260779,898216.2574696885,886372.5039133094,874528.750356934,862595.3800636474,850749.1836469928,838902.9872303326,827056.7908136882,815210.5943970326,803364.3979803845,791397.9988763463,779549.0904244427,767700.1819725428,755851.2735206503,744002.3650687691,732153.4566168636,720304.5481649525,708455.6397130582,696445.031696273,684593.1185890092,672741.2054817481,660889.2923744824,649037.3792672344,637185.4661599761,625333.5530527113,613481.6399454419,601629.7268381976,589777.8137309458,577709.3035364151,565854.0677442588,553998.8319521071,542143.5961599601,530288.3603678001,518433.1245756345,506577.88878348283],[1681915.6773638288,1670192.1489089823,1658468.6204541363,1646745.0919992942,1635021.563544451,1623298.035089606,1611574.5066347637,1599850.9781799188,1588127.4497250754,1576403.9212702324,1564680.3928153869,1552956.8643605434,1541233.3359056998,1529509.8074508556,1517786.278996012,1506062.7505411678,1494339.222086324,1482615.6936314797,1470892.165176636,1459168.6367217926,1447445.1082669483,1435722.0024755676,1424006.1494582272,1412296.547873582,1400592.306246441,1388892.631955407,1377196.8212709702,1365504.2503471843,1353815.090218493,1342128.4891143455,1330443.773791765,1318760.863840647,1307080.308375283,1295400.612823948,1283722.794903767,1272045.838621197,1260370.189245839,1248695.271552151,1237021.5659112353,1225348.0746700452,1213676.106140498,1202004.1376109417,1190333.0871239007,1178662.6658604364,1166992.2445969707,1155322.8238800736,1143653.775513973,1131984.7271478823,1120315.6787817879,1108640.1382796364,1096969.7170161693,1085299.2957527074,1073612.483785851,1061940.5152563048,1050268.546726747,1038568.5700156111,1026894.8643746944,1015221.1587337716,1003547.4530928563,991830.2536125905,980154.6042372333,968478.9548618766,956803.3054865226,945127.6561111631,933388.2572983634,921710.4393781899,910032.6214579986,898354.8035378158,886676.9856176451,874999.1676974539,863231.5307753077,851551.2997858273,839871.0687963627,828190.8378068823,816510.6068174113,804830.3758279383,793029.916040332,781347.0060892198,769664.0961381001,757981.1861869739,746298.2762358589,734615.3662847523,722932.4563336158,711249.5463825073,699405.2010755977,687719.3231112519,676033.4451469099,664347.5671825735,652661.6892182389,640975.8112538802,629289.9332895437,617604.0553252092,605918.1773608681,594232.2993965261,582546.4214321887,570641.3615961848,558952.2014674926,547263.041338807,535573.881210113,523884.7210814217],[1682979.7600646361,1671460.449501863,1659941.1389390945,1648421.8283763258,1636902.5178135566,1625383.207250785,1613863.8966880175,1602344.5861252465,1590825.2755624773,1579305.9649997079,1567786.6544369378,1556267.3438741686,1544748.0333113987,1533228.7227486295,1521709.4121858599,1510190.10162309,1498670.7910603203,1487151.4804975507,1475632.169934781,1464112.8593720105,1452593.5488092427,1441074.2382464712,1429559.5900869542,1418051.6723778844,1406549.5572100081,1395052.3240228707,1383559.2944997794,1372069.7774062962,1360583.1826603415,1349098.988918689,1337616.8640255504,1326137.5429562747,1314659.4732338912,1303182.925599885,1291708.1583533431,1280233.9939976484,1268761.756529653,1257289.733946622,1245819.3498687306,1234349.2470234265,1222880.1261124332,1211411.7389326626,1199943.3679775586,1188476.5089618224,1177009.649946094,1165542.9404411386,1154077.4373734244,1142611.93430572,1131146.4312380143,1119680.928170308,1108208.4958517137,1096741.6368359877,1085274.7778202542,1073791.0927754599,1062322.7055957043,1050854.3184159282,1039357.4986531003,1027887.395807792,1016417.2929624794,1004947.1901171706,993433.21453691,981961.1919538742,970489.1693708412,959017.1467878157,947545.124204793,936009.0633268254,924534.8989711255,913060.7346154256,901586.5702597396,890112.4059040351,878550.6631415784,867074.1155075673,855597.5678735562,844121.0202395506,832644.4726055395,821167.9249715256,809691.3773375247,798094.6409418415,786615.4474198017,775136.253897747,763657.0603756886,752177.8668536339,740698.6733315857,729219.4798095394,717740.2862874875,706099.9949030103,694617.87000986,683135.7451167284,671653.6202236097,660171.4953304622,648689.3704373306,637207.2455441905,625725.1206510486,614242.995757917,602760.8708647853,591278.7459716443,579578.2581484308,568092.891611496,556607.5250745583,545122.1585376104],[1683850.8302147805,1672498.694731328,1661146.5592478814,1649794.4237644325,1638442.2882809832,1627090.152797534,1615738.017314085,1604385.881830638,1593033.7463471883,1581681.6108637408,1570329.4753802908,1558977.339896842,1547625.2044133923,1536273.0689299447,1524920.9334464953,1513568.797963046,1502216.662479598,1490864.5269961488,1479512.3915127,1468160.2560292506,1456808.1205458022,1445455.9850623533,1434106.3672214225,1422763.5170736364,1411426.4870013008,1400094.4340418153,1388766.6093539598,1377442.3486920018,1366121.0637944287,1354802.4480364844,1343486.8199678299,1332172.8823258786,1320860.362420138,1309550.348442431,1298241.0536186374,1286933.5000146772,1275626.7700036753,1264321.3314002212,1253016.494390945,1241712.9515284807,1230409.4210235616,1219107.5723808352,1207805.7237381,1196504.4985641516,1185204.1589834034,1173903.8194026598,1162604.1172955576,1151305.1167134782,1140006.1161314053,1128707.1155493339,1117408.1149672563,1106101.7819181718,1094801.44233743,1083501.1027566805,1072183.5400252938,1060881.6913825665,1049579.8427398284,1038249.1800041986,1026945.6371417423,1015642.0942792795,1004338.5514168157,992990.8049172498,981685.3663137984,970379.9277103357,959074.4891068777,947769.0505034272,936399.3382918555,925091.784687887,913784.231083936,902476.6774799721,891169.1238760073,879773.8789731497,868463.9718818394,857154.0647905413,845844.1576992413,834534.2506079515,823224.3435166478,811914.4364253515,800484.4467560602,789171.9268503124,777859.4069445776,766546.8870388363,755234.3671330959,743921.8472273517,732609.3273216151,721296.8074158775,709984.2875101259,698508.1857906887,687192.7711559907,675877.3565213084,664561.9418866262,653246.527251929,641931.1126172543,630615.6979825674,619300.283347873,607984.8687131861,596669.4540785132,585136.5764964251,573817.9607384931,562499.3449805602],[1683362.224282104,1671916.3159428872,1660470.4076036753,1649024.4992644568,1637578.5909252428,1626132.6825860285,1614686.774246813,1603240.8659075978,1591794.9575683826,1580349.0492291693,1568903.1408899545,1557457.2325507393,1546011.3242115232,1534565.4158723087,1523119.507533094,1511673.599193879,1500227.690854664,1488781.7825154494,1477335.874176234,1465889.9658370186,1454444.0574978034,1442998.1491585886,1431552.613701158,1420113.8831795673,1408681.0148214707,1397253.1696007692,1385829.6018130637,1374409.649647541,1362993.3474211374,1351579.9137492299,1340168.6617746828,1328759.1596249721,1317352.3654004596,1305946.8133442893,1294542.598860613,1283140.152220841,1271738.261433521,1260338.1791727855,1248938.3766138125,1237540.0104823331,1226142.0796676418,1214744.8644076162,1203348.6063668285,1191952.348326041,1180557.2827940765,1169162.514713219,1157767.7466323725,1146374.0700893337,1134980.6240578024,1123587.1780262776,1112193.7319947476,1100793.9062281116,1089399.1381472568,1078004.3700664141,1066593.509877386,1055197.2518365937,1043800.9937958084,1032404.7357550259,1020979.3250031387,1009581.3941884413,998183.4633737495,986785.5325590465,975343.1151985405,963943.312639568,952543.5100806067,941143.7075216314,929743.904962657,918279.6470267139,906877.7562393891,895475.86545208,884073.974664757,872672.0838774396,861182.448481054,849778.2339973757,838374.0195137039,826969.8050300302,815565.5905463602,804161.3760626912,792757.1615790138,781233.0368482042,769826.2426236849,758419.4483991833,747012.6541746585,735605.8599501466,724199.0657256274,712792.2715011071,701385.4772765907,689978.683052076,678408.8277179189,666999.1754066041,655589.5230952948,644179.8707839698,632770.2184726391,621360.5661613205,609950.9138499908,598541.2615386555,587131.6092273332,575721.9569160277,564095.8226810945,552683.009767076],[1682873.618349429,1671333.9371544456,1659794.2559594673,1648254.5747644845,1636714.8935695025,1625175.2123745217,1613635.5311795403,1602095.84998456,1590556.1687895786,1579016.4875945982,1567476.8063996176,1555937.1252046353,1544397.444009654,1532857.7628146738,1521318.0816196925,1509778.4004247119,1498238.7192297308,1486699.0380347488,1475159.356839768,1463619.6756447868,1452079.9944498066,1440540.3132548255,1429000.6320598437,1417465.2482024585,1405936.1745902258,1394412.5151546374,1382893.4734387863,1371378.3425294533,1359866.4959553613,1348357.379461976,1336850.5035815325,1325346.6135935723,1313844.3683807775,1302343.299837482,1290844.7563700979,1279346.8044270044,1267850.5764563342,1256355.0269453493,1244860.8604308679,1233367.069436186,1221874.7506692545,1210382.431902322,1198891.4889955604,1187400.8215567246,1175910.4066047394,1164421.2100237831,1152932.0134428265,1141443.023465192,1129955.1319842027,1118467.2405032222,1106979.3490222455,1095491.4575412567,1083996.8339570956,1072507.6373761361,1061018.4407951809,1049512.812290636,1038022.1448517917,1026531.4774129479,1015040.8099741037,1003520.694097606,992028.3753306838,980536.0565637564,969043.7377968207,957506.6975688059,946012.5310543105,934518.3645398328,923024.1980253533,911530.0315108718,899971.2813948607,888475.0534241898,876978.8254535189,865482.5974828452,853986.3695121752,842402.4032042027,830903.8813281646,819405.3594521172,807906.8375760671,796408.3157000095,784909.7938239602,773411.2719479268,761793.0783028053,750292.0097595183,738790.9412162257,727289.8726729266,715788.8041296415,704287.7355863424,692786.6670430563,681285.5984997777,669784.5299564721,658120.99429191,646617.1043039318,635113.2143159835,623609.3243280342,612105.4343400607,600601.5443521123,589097.6543641565,577593.7643761886,566089.8743882384,554585.9844002621,543082.0944123166],[1682385.012416752,1670751.5583660037,1659118.1043152546,1647484.6502645118,1635851.1962137632,1624217.7421630153,1612584.2881122692,1600950.8340615209,1589317.3800107753,1577683.9259600274,1566050.4719092813,1554417.0178585318,1542783.5638077853,1531150.1097570378,1519516.6557062909,1507883.2016555436,1496249.7476047967,1484616.2935540495,1472982.8395033022,1461349.3854525546,1449715.9314018087,1438082.477351061,1426449.0233003139,1414817.6430054337,1403192.6171066286,1391573.0538371168,1379958.1601747756,1368347.2318801628,1356739.644489587,1345135.03742055,1333533.6379485135,1321934.2177237878,1310336.371361092,1298741.028499027,1287146.9138795778,1275554.084611154,1263962.8914791448,1252372.326325118,1240783.3442479135,1229194.8137779161,1217607.421670876,1206020.714951709,1194434.3716242965,1182849.2947873995,1171264.2179505029,1159679.9053343446,1148096.2802532883,1136512.6551722274,1124929.6399106067,1113347.3029801664,1101764.9660497317,1090182.6291192956,1078600.292188859,1067010.9046858698,1055427.279604807,1043843.6545237461,1032243.2959077624,1020658.2190708732,1009073.1422339864,997459.9940067763,985873.2872876143,974286.5805684524,962699.8738492923,951113.1671301322,939481.5520280376,927893.0215580389,916304.4910880448,904715.9606180508,893127.4301480502,881474.2413962893,869883.6762422724,858293.1110882489,846702.5459342236,835111.9807801973,823433.7431426216,811840.9138741959,800248.0846057683,788655.2553373426,777062.4260689262,765469.5968004959,753876.7675320711,742283.9382636491,730569.2282577921,718973.8853957253,707378.5425336575,695783.1996715898,684187.856809522,672592.513947458,660997.1710853856,649401.8282233169,637644.6855126014,626046.5578480028,614448.4301834069,602850.3025188195,591252.1748542292,579654.047189638,568055.9195250385,556457.7918604352,544859.6641958375,533261.5365312407],[1681896.4064840758,1670169.179577562,1658441.9526710464,1646714.7257645337,1634987.498858023,1623260.2719515082,1611533.0450449944,1599805.818138483,1588078.591231969,1576351.364325456,1564624.1374189411,1552896.9105124297,1541169.6836059154,1529442.4566994032,1517715.2297928894,1505988.0028863763,1494260.775979863,1482533.5490733492,1470806.3221668361,1459079.0952603233,1447351.8683538102,1435624.6414472973,1423897.4145407844,1412170.1876342702,1400449.0596230319,1388733.5925195995,1377022.8469107656,1365316.3175439117,1353613.6632713801,1341914.0335334595,1330216.9065051696,1318521.821854003,1306829.4565127688,1295138.757160567,1283449.1399797278,1271761.9347282625,1260075.206501954,1248390.3041645736,1236705.8280649702,1225022.9336394602,1213340.0926724938,1201658.998001095,1189977.9033296956,1178297.768018073,1166618.281783123,1154938.7955481815,1143260.5470637456,1131582.4934825818,1119904.4399014157,1108227.3654571152,1096550.5830772193,1084873.800697328,1073197.0183174391,1061520.2359375497,1049836.1184144388,1038158.0648332727,1026480.011252109,1014784.9607288013,1003105.4744938528,991425.9882589024,979718.1992445383,968037.1045731464,956356.0099017397,944674.915230359,932993.8205589429,921267.6785762459,909584.7841507317,897901.8897252223,886218.9952997137,874536.1008742051,862788.5270310296,851103.6246936414,839418.7223562626,827733.8200188885,816048.9176815161,804276.4682962755,792589.331635477,780902.1949746748,769215.0583138848,757527.921653077,745840.7849922888,734153.6483314782,722466.5116706891,710657.8981185015,698968.2809376707,687278.6637568139,675589.0465759849,663899.4293951346,652209.8122143075,640520.1950334683,628830.5778526161,616979.9013800425,605287.5360387927,593595.1706975782,581902.8053563321,570210.4400151037,558518.0746738762,546825.7093326394,535133.3439914035,523440.9786501732],[1681407.8005514,1669586.8007891194,1657765.801026841,1645944.8012645603,1634123.8015022809,1622302.8017400021,1610481.8019777224,1598660.802215443,1586839.8024531633,1575018.802690884,1563197.8029286047,1551376.8031663261,1539555.8034040465,1527734.8036417668,1515913.8038794878,1504092.8041172086,1492271.8043549291,1480450.8045926504,1468629.804830371,1456808.8050680915,1444987.8053058116,1433166.8055435326,1421345.8057812531,1409524.8060189732,1397707.2656026683,1385895.6566755753,1374089.1363020157,1362286.95545694,1350488.4495136968,1338693.0296463703,1326900.1750618273,1315110.5735234388,1303322.822966347,1291536.4858221118,1279752.5943224945,1267969.78484537,1256188.3407921933,1244408.282004028,1232629.0424832935,1220851.0535009936,1209073.7951199738,1197297.2810504753,1185521.7984268442,1173746.3158032163,1161972.3456157525,1150198.4499827493,1138424.8138742014,1126652.3317929362,1114879.8497116668,1103107.4279340617,1091336.2001047148,1079564.9722753707,1067793.744446022,1056022.5166166718,1044244.9572240561,1032472.4751427895,1020699.9930615206,1008927.5109802568,997137.8067537262,985363.9111207221,973590.0154877212,961787.6285778433,950012.1459542122,938236.6633305782,926461.180706949,914642.3355944501,902865.0772134233,891087.818832404,879310.5604513837,867533.3020703644,855756.0436893413,843914.1382990442,832134.8987783138,820355.6592575749,808576.4197368436,796797.1802161084,785017.9406953733,773149.1346120043,761367.6905588424,749586.2465056619,737804.8024524879,726023.3583993064,714241.9143461362,702460.4702929566,690558.0193416821,678774.1278420631,666990.2363424487,655206.3448428372,643422.4533432228,631638.561843601,619854.6703439895,608070.7788443714,596286.887344759,584340.0388763212,572553.4358584518,560766.832840587,548980.2298227111,537193.6268048417,525407.0237869741,513620.4207691066],[1680919.194618724,1669004.4220006783,1657089.6493826346,1645174.8767645874,1633260.104146541,1621345.3315284955,1609430.5589104507,1597515.7862924056,1585601.0136743593,1573686.2410563135,1561771.4684382672,1549856.6958202226,1537941.9232021777,1526027.1505841324,1514112.377966086,1502197.6053480408,1490282.8327299955,1478368.0601119504,1466453.2874939046,1454538.5148758595,1442623.742257814,1430708.9696397693,1418794.1970217214,1406879.4244036772,1394965.8087288819,1383058.1756024507,1371155.6858468074,1359257.5933699708,1347363.2357560112,1335472.5715797336,1323584.9614263012,1311699.64492401,1299816.1894199243,1287935.3487254202,1276056.048665259,1264177.8828468693,1252301.883516925,1240426.2598434868,1228552.6831393996,1216679.1733625326,1204807.5510259983,1192935.9286894673,1181065.6935239937,1169195.822948128,1157326.409448374,1145458.104417317,1133589.7993862615,1121722.1701032892,1109855.259521918,1097988.3489405462,1086121.81713221,1074256.1438534074,1062390.470574609,1050524.7972958032,1038659.1240170063,1026786.8854523152,1014919.9748709421,1003053.0642895717,991170.139013601,979301.8339825473,967433.528951495,955565.2239204347,943668.2820066698,931798.4114308041,919928.5408549402,908058.6702790782,896145.3702761149,884273.7479395857,872402.1256030481,860530.5032665152,848658.8809299832,836787.2585934484,824851.0752003631,812977.4984962754,801103.9217921877,789230.3450880991,777356.768384005,765483.1916799098,753520.3228037888,741644.5713582346,729768.8199126916,717893.06846713,706017.3170215907,694141.5655760188,682265.8141304906,670269.5919272993,658391.4261089172,646513.2602905137,634635.0944721382,622756.9286537329,610878.7628353545,599000.5970169492,587122.4311985718,575244.2653801665,563204.0663605714,551323.2256660517,539442.3849715628,527561.5442770412,515680.7035825448,503799.86288803536],[1680430.5886860476,1668422.0432122387,1656413.4977384275,1644404.9522646128,1632396.4067907992,1620387.8613169894,1608379.3158431787,1596370.770369367,1584362.224895554,1572353.6794217436,1560345.1339479308,1548336.5884741198,1536328.0430003086,1524319.4975264962,1512310.9520526847,1500302.406578874,1488293.8611050623,1476285.31563125,1464276.7701574382,1452268.2246836275,1440259.6792098153,1428251.1337360037,1416242.588262192,1404234.0427883798,1392225.4973145677,1380221.4814928626,1368223.0018864076,1356229.2622161596,1344239.5554472555,1332253.2547136557,1320269.805117894,1308288.7163245755,1296310.7204740131,1284334.4775023607,1272359.5030080283,1260387.062870862,1248415.426241654,1236445.3674037284,1224476.323795496,1212508.409908054,1200541.3069320256,1188575.3206399796,1176609.5886211437,1164645.330093042,1152681.0715649407,1140717.7588518853,1128755.0444227788,1116792.3299936694,1104830.6693321685,1092869.330250693,1080907.9911692198,1068947.3154314472,1056987.1967031928,1045027.0779749407,1033066.9592466797,1021106.8405184168,1009139.956680363,997178.6175988824,985217.2785174064,973239.7568443627,961277.042415251,949314.3279861417,937351.613557036,925360.1595310355,913395.9010029342,901431.6424748311,889467.3839467373,877459.6770467628,865493.6907547181,853527.7044626698,841561.7181706261,829595.7318785675,817629.7455865201,805599.3377349572,793631.4238475142,781663.509960073,769695.59607263,757727.6821851861,745759.7682977449,733702.8962108204,721732.8373728944,709762.7785349619,697792.7196970377,685822.6608591173,673852.6020211903,661882.5431832643,649792.6158753838,637820.1757382192,625847.7356010498,613875.2954638833,601902.8553267214,589930.415189553,577957.9750523912,565985.5349152079,554013.094778047,542040.6546408823,529904.5401203968,517929.46174925193,505954.3833781127,493979.3050069604],[1679941.9827533728,1667839.6644237956,1655737.3460942134,1643635.027764637,1631532.7094350609,1619430.391105484,1607328.0727759034,1595225.7544463272,1583123.43611675,1571021.117787172,1558918.799457594,1546816.4811280165,1534714.162798439,1522611.8444688623,1510509.526139283,1498407.207809705,1486304.8894801287,1474202.571150551,1462100.2528209721,1449997.9344913955,1437895.616161816,1425793.29783224,1413690.9795026626,1401588.6611730831,1389486.342843506,1377386.172942316,1365291.5972237536,1353201.822623263,1341116.1445373225,1329033.9378475791,1316955.0674545087,1304879.2796338787,1292805.5947184896,1280733.6062793017,1268664.2902315245,1256596.2428948572,1244529.5284389094,1232464.6027360861,1220400.2365057832,1208337.7133807936,1196275.4623099985,1184214.7125904863,1172154.3623429304,1160094.8372379541,1148036.1907576225,1135977.5442772834,1123920.2894592928,1111863.165632132,1099806.079142422,1087750.3115608417,1075694.543979254,1063638.776397679,1051583.92283177,1039529.3586540595,1027474.7944763498,1015420.2302986402,1003365.6661209352,991304.1709081968,979248.4033266213,967192.6357450364,955120.5558790173,943063.4320518523,931006.3082246901,918949.1843975326,906863.2611509282,894804.614670597,882745.9681902574,870687.3217099262,858585.2559063751,846524.9056588188,834464.555411254,822404.2051637014,810343.8549161386,798283.5046685832,786158.9259028612,774096.6748320591,762034.423761256,749972.1726904707,737909.9216196612,725847.6705488637,713696.8548330963,701632.4886027919,689568.1223724904,677503.7561421786,665439.3899118984,653375.0236815764,641310.6574512897,629246.2912209835,617060.3767299717,604993.6622740161,592926.9478180679,580860.2333621327,568793.5189061826,556726.8044502586,544660.0899943067,532593.375538379,520526.66108241584,508297.37922145706,496228.0631736759,484158.7471259022],[1679453.3768206968,1667257.2856353514,1655061.1944500068,1642865.1032646648,1630669.0120793206,1618472.9208939765,1606276.8297086323,1594080.7385232889,1581884.6473379452,1569688.556152601,1557492.4649672573,1545296.3737819125,1533100.2825965695,1520904.1914112256,1508708.1002258821,1496512.009040538,1484315.9178551943,1472119.8266698504,1459923.7354845074,1447727.644299163,1435531.5531138193,1423335.4619284759,1411139.3707431313,1398943.279557789,1386747.1883724458,1374551.097187101,1362360.2906760152,1350174.652739126,1337993.4315795521,1325815.9601457254,1313641.6474800408,1301469.9709041594,1289300.4689629688,1277134.0788002242,1264969.1813318275,1252805.627607416,1240644.4341441318,1228483.8380684396,1216325.1644457644,1204167.0168535472,1192010.4285993953,1179854.1045410023,1167699.3903379277,1155544.676134856,1143391.309950302,1131238.275517736,1119085.5344958096,1106934.001270594,1094782.468045381,1082631.2928709849,1070481.096789301,1058330.9007076109,1046180.7046259325,1034031.6393331904,1021882.629706033,1009733.6200788678,997584.6104516955,985435.6008245302,973279.5281358254,961129.3320541428,948979.1359724561,936812.5361175621,924661.002892347,912509.4696671311,900357.9364419188,888177.5868663527,876024.5524337869,863871.5180012127,851718.4835686376,839522.1068549696,827367.3926519062,815212.6784488251,803057.9642457562,790903.2500426872,778748.5358396051,766529.8397040423,754373.2514498923,742216.6631957358,730060.0749415746,717903.4866874292,705746.898433269,693502.1986706192,681343.525047943,669184.851425265,657026.1778025879,644867.5041799173,632708.8305572337,620550.1569345593,608391.4833118739,596112.0290841479,583951.0403094385,571790.0515347151,559629.0627600038,547468.0739852898,535307.0852105748,523146.0964358561,510985.1076611504,498824.1188864298,486501.742969241,474338.1892448338],[1678964.7708880198,1666674.9068469072,1654385.0428057993,1642095.1787646895,1629805.314723579,1617515.4506824696,1605225.58664136,1592935.7226002496,1580645.8585591402,1568355.9945180311,1556066.1304769209,1543776.2664358104,1531486.4023947003,1519196.5383535908,1506906.6743124807,1494616.810271371,1482326.946230261,1470037.0821891506,1457747.2181480424,1445457.354106932,1433167.4900658214,1420877.6260247119,1408587.7619836011,1396297.8979424932,1384008.0339013827,1371718.1698602727,1359431.130448711,1347149.324486944,1334872.002848969,1322598.5004128304,1310328.2275055705,1298061.150895718,1285796.988156199,1273534.7558886139,1261274.1685697858,1249016.2810310824,1236759.3398493482,1224504.0767558576,1212250.0923857354,1199997.1113706874,1187745.3948887833,1175494.4694512784,1163244.4183329288,1150995.3401743467,1138746.4291429794,1126499.00675818,1114251.5843733712,1102004.8369090576,1089758.8942857916,1077512.951662518,1065267.6495993412,1053023.0250175544,1040778.4004357671,1028533.9200123232,1016290.4649357004,1004047.0098590837,991803.5547824684,979560.0997058512,967316.6446292261,955066.0283632483,942821.4037814578,930576.7791996626,918315.6975600142,906069.7549367314,893823.81231347,881577.8696901985,869303.1366773043,857055.7142925095,844808.2919077035,832560.8695228938,820313.4471380934,808021.1517339638,795772.0735753682,783522.9954167772,771273.9172582002,759024.8390996158,746712.0791385258,734461.1537010055,722210.2282635001,709959.3028259948,697708.3773884801,685457.451950985,673118.9277233947,660865.9467083337,648612.9656932997,636359.9846782265,624107.0036631916,611854.0226481315,599601.0416330723,587348.0606180243,574975.1328007923,562719.8697073087,550464.6066138279,538209.3435203247,525954.0804268457,513698.81733334344,501443.55423985794,489188.29114636686,476933.02805285994,464677.76495937444],[1678476.164955344,1666092.5280584656,1653708.8911615927,1641325.2542647137,1628941.6173678404,1616557.9804709642,1604174.3435740883,1591790.7066772108,1579407.0697803355,1567023.4328834594,1554639.7959865837,1542256.1590897066,1529872.5221928307,1517488.885295955,1505105.248399079,1492721.6115022027,1480337.9746053272,1467954.3377084516,1455570.7008115747,1443187.0639147,1430803.4270178229,1418419.7901209465,1406036.1532240715,1393652.5163271963,1381268.879430321,1368885.2425334419,1356501.9702214035,1344123.9962347622,1331750.6092337319,1319381.6443921905,1307016.169552351,1294653.6278128538,1282293.527005162,1269935.700565609,1257580.6485020828,1245226.9344547542,1232874.9053770644,1220524.708142295,1208175.1707300306,1195827.7319182986,1183480.4435108611,1171135.0985573144,1158789.8359364541,1146446.00421383,1134102.5620997269,1121759.737998622,1109417.9276615847,1097076.1173245446,1084735.3205261924,1072394.968504867,1060054.6164835491,1047715.1493274923,1035376.0962456022,1023037.0431637038,1010698.3001653766,998360.3996393085,986022.4991132244,973684.5985871572,961346.6980610862,949008.7975350069,936663.671590466,924324.618508569,911985.56542667,899630.0402063401,887289.6881850231,874949.3361636968,862608.984142378,850239.9105837895,837898.1002467582,825556.2899097167,813214.4795726743,800872.6692356439,788486.1829049764,776142.7407908821,763799.2986767804,751455.856562689,739112.4144485891,726705.6442062836,714360.381585421,702015.1189645594,689669.8563436847,677324.5937228315,664979.331101968,652634.0684811119,640199.7535839919,627852.4651765684,615505.1767691327,603157.8883616943,590810.599954267,578463.3115468416,566116.0231394172,553768.7347319899,541300.1504676379,528950.6130553642,516601.0756431045,504251.5382308252,491902.0008185599,479552.4634062955,467202.92599403765,454853.3885817714],[1677987.559022668,1665510.149270024,1653032.7395173842,1640555.3297647408,1628077.9200120985,1615600.5102594579,1603123.1005068137,1590645.690754172,1578168.28100153,1565690.8712488879,1553213.4614962474,1540736.0517436033,1528258.6419909617,1515781.23223832,1503303.8224856774,1490826.4127330356,1478349.002980393,1465871.5932277516,1453394.1834751093,1440916.773722467,1428439.3639698238,1415961.9542171834,1403484.544464541,1391007.1347118996,1378529.7249592573,1366052.3152066155,1353574.9054539716,1341100.6812480618,1328631.3573920217,1316166.2265239204,1303704.660996494,1291246.104729991,1278790.8219033517,1266338.0869932568,1253887.128434385,1241438.1468033758,1228991.3211240964,1216545.3395287427,1204101.5277976915,1191658.3524658955,1179216.7566660945,1166775.7276633517,1154336.1278591338,1141896.6682533203,1129458.8621837045,1117021.0561140939,1104584.2709497907,1092148.0726605211,1079711.874371253,1067276.9853472258,1054842.22392785,1042407.4625084717,1029973.7920554322,1017540.3104734374,1005106.8288914454,992673.7894195188,980241.4434439959,967809.0974684684,955376.7514929371,942944.4055174133,930512.0595418802,918072.4578174753,905638.9762354717,893205.4946534662,880755.5640565678,868320.8026371971,855886.0412178142,843451.2797984304,830987.9085858176,818551.7102965433,806115.5120072635,793679.3137179939,781243.115428729,768762.4861649834,756324.6800953755,743886.8740257667,731449.0679561533,719011.261886538,706510.5349073485,694070.9351031166,681631.3352989042,669191.7354946984,656752.1356904712,644312.5358862607,631872.9360820455,619344.9456748776,606903.3498750878,594461.7540752795,582020.1582754683,569578.5624756524,557136.9666758608,544695.3708760543,532253.7750762543,519691.88259040564,507248.0708593605,494804.2591283219,482360.4473972786,469916.6356662335,457472.82393519673,445029.01220415905],[1677498.9530899911,1664927.7704815825,1652356.587873171,1639785.405264768,1627214.2226563583,1614643.0400479506,1602071.8574395417,1589500.6748311333,1576929.492222725,1564358.3096143173,1551787.127005907,1539215.9443975007,1526644.761789093,1514073.579180684,1501502.3965722756,1488931.2139638672,1476360.0313554604,1463788.8487470513,1451217.6661386425,1438646.4835302355,1426075.3009218266,1413504.1183134196,1400932.9357050112,1388361.7530966029,1375790.570488193,1363219.387879785,1350648.205271374,1338077.669343998,1325512.1055503122,1312950.8713702213,1300393.90065408,1287840.1836671263,1275289.1957120972,1262740.473420909,1250194.342138726,1237650.4295795865,1225107.736871127,1212567.232440167,1200027.8848653645,1187489.798815548,1174953.0698213195,1162417.1666291421,1149882.419781813,1137348.4827942452,1124815.162267676,1112282.9922425533,1099750.8222174277,1087220.0279964996,1074689.4417549944,1062159.0021895827,1049629.8313721563,1037100.6605547178,1024571.4897372928,1012043.5777831711,999515.6677010637,986987.7576189684,974460.387774758,961933.5963497814,949406.8049247959,936880.0134998141,924353.2220748356,911826.4306498617,899292.3870442603,886764.4769621529,874236.5668800585,861692.2691106889,849163.0982932625,836633.9274758212,824104.7566584041,811547.1306833625,799016.5444418518,786485.9582003495,773955.371958836,761424.785717342,748850.0615139706,736317.8914888334,723785.7214637185,711253.5514385868,698721.3814134663,686189.211388344,673592.8142541191,661058.8772665393,648524.9402789706,635991.0032913974,623457.0663038353,610923.1293162648,598301.5229810197,585765.6197888413,573229.7165966658,560693.8134044977,548157.9102123193,535622.0070201242,523086.103827944,510550.2006357601,498014.29744358733,485356.980025813,472818.8939759936,460280.80792617146,447742.7218763614,435204.63582654856],[1677010.3471573158,1664345.39169314,1651680.436228965,1639015.4807647895,1626350.5253006178,1613685.5698364442,1601020.614372269,1588355.658908095,1575690.703443921,1563025.7479797464,1550360.792515571,1537695.8370513988,1525030.8815872238,1512365.9261230486,1499700.970658875,1487036.0151946996,1474371.0597305251,1461706.1042663516,1449041.148802177,1436376.1933380037,1423711.237873829,1411046.282409655,1398381.3269454807,1385716.3714813057,1373051.4160171312,1360386.4605529574,1347721.5050887845,1335056.5496246102,1322394.8253403716,1309737.695938982,1297084.4936481249,1284434.626288309,1271787.6086918386,1259144.080477417,1246502.5761450601,1233862.7123557958,1221225.3364126072,1208589.4105105656,1195954.6684029615,1183321.8815644407,1170689.5211958461,1158059.1723919953,1145428.9618074382,1132800.4375335728,1120172.1633626493,1107544.9283710136,1094918.3943903823,1082291.9833324766,1069667.0091387369,1057042.0349449967,1044417.438816457,1031793.858600969,1019170.2783854879,1006546.8450928945,993924.5065106936,981302.1679284866,968679.8293462833,956058.0952310963,943436.8583566546,930815.6214822168,918194.3846077854,905573.1477333512,892951.9108589105,880323.4592708508,867701.1206886545,855078.7821064442,842440.1553686969,829816.5751532158,817192.9949377375,804569.414722247,791917.5768764447,779292.6026827013,766667.6284889635,754042.6542952266,741417.6801014785,728748.9089519121,716122.3749712761,703495.8409906458,690869.3070100043,678242.773029373,665616.239048739,652926.0190384043,640297.7448674683,627669.4706965499,615041.1965256296,602412.9223547019,589784.6481837649,577156.3740128484,564439.2749178717,551809.0643333066,539178.8537487583,526548.6431642091,513918.43257964496,501288.2219950883,488658.0114105437,476027.80082597677,463277.340554703,450644.9801861197,438012.61981753353,425380.25944892876],[1676521.7412246394,1663763.0129046985,1651004.2845847583,1638245.556264818,1625486.8279448773,1612728.0996249365,1599969.3713049972,1587210.642985057,1574451.9146651153,1561693.1863451744,1548934.4580252338,1536175.7297052937,1523417.0013853554,1510658.273065414,1497899.5447454732,1485140.8164255328,1472382.0881055922,1459623.3597856518,1446864.6314657116,1434105.9031457705,1421347.1748258304,1408588.4465058893,1395829.7181859494,1383070.989866009,1370312.261546068,1357553.5332261291,1344794.8049061892,1332036.076586248,1319277.961759547,1306524.5205077443,1293775.4391485988,1281030.2003652218,1268288.0024900606,1255548.3511777672,1242810.8101513959,1230076.3740265793,1217343.4220327334,1204611.8489334537,1191882.6087967302,1179153.9643133301,1166427.3296259684,1153701.178154842,1140976.6601779182,1128252.3922728999,1115529.7809186224,1102807.1695643421,1090085.9665633244,1077365.0686271787,1064644.576522477,1051925.214376505,1039205.8522305223,1026487.0566472183,1013769.0670336778,1001051.077420142,988333.3453203146,975616.5782380099,962899.8111556973,950183.0440733903,937466.9117885167,924751.2294646278,912035.5471407278,899319.8648168445,886604.1824929565,873888.5001690593,861165.6744972263,848448.9074149178,835732.1403326131,822999.2228306038,810281.2332170615,797563.2436035322,784845.253990002,772099.2471650513,759379.8850190826,746660.5228731055,733941.1607271209,721221.798581156,708459.0284788469,695738.1305426881,683017.2326065339,670296.3346703965,657575.436734247,644854.5387980891,632070.549455977,619347.9381016977,606625.3267474268,593902.7153931465,581180.1040388532,568457.4926845813,555734.8813303057,542924.3152621519,530199.7972851917,517475.2793082716,504750.761331344,492026.2433544211,479301.7253774721,466577.20740054734,453852.68942362536,441128.171446681,428282.51775867864,415555.8830713099],[1676033.135291963,1663180.6341162566,1650328.132940551,1637475.6317648462,1624623.1305891355,1611770.6294134303,1598918.1282377227,1586065.6270620166,1573213.1258863108,1560360.6247106038,1547508.1235348978,1534655.6223591915,1521803.1211834853,1508950.6200077778,1496098.1188320713,1483245.6176563657,1470393.1164806585,1457540.6153049523,1444688.1141292462,1431835.6129535376,1418983.1117778316,1406130.6106021258,1393278.1094264193,1380425.6082507123,1367573.1070750067,1354720.6058992993,1341868.1047235948,1329015.6035478886,1316163.1023721818,1303313.5627256427,1290468.340127833,1277626.804221573,1264788.3962882801,1251953.2810458844,1239120.9266179071,1226290.455611602,1213461.8670654693,1200635.7373974905,1187810.5491904803,1174987.3317296165,1162165.138056079,1149344.2290499373,1136524.3585483911,1123705.5331790857,1110887.3984745983,1098070.4499369608,1085253.5387362745,1072438.276844611,1059623.014952946,1046808.3938080124,1033994.643709797,1021180.8936115964,1008367.8556818715,995555.4566702889,982743.0576586933,969930.9885475268,957119.7929651155,944308.5973826987,931497.4018002916,918686.8374470277,905876.709673686,893066.581900333,880256.4541269839,867446.3263536431,854636.1985803079,841819.0327234035,829007.8371409802,816196.6415585764,803369.4714964041,790557.0724848174,777744.6734732175,764932.2744616373,752119.8754500374,739278.3914509956,726464.6413527802,713650.8912545694,700837.1411563614,688023.391058146,675165.1582030831,662349.8963114247,649534.6344197579,636719.3725280883,623904.1106364299,611088.8487447677,598209.456969209,585392.5084315706,572575.5598939322,559758.6113562919,546941.6628186787,534124.7142810347,521307.7657433916,508401.91545233876,495583.0900830496,482764.26471372973,469945.4393444229,457126.61397511326,444307.78860580735,431488.9632365117,418670.13786719553,405731.5066937022],[1675544.5293592853,1662598.2553278138,1649651.9812963433,1636705.7072648667,1623759.4332333952,1610813.1592019247,1597866.885170451,1584920.6111389794,1571974.3371075063,1559028.063076034,1546081.7890445606,1533135.5150130892,1520189.2409816144,1507242.9669501432,1494296.6929186704,1481350.4188871975,1468404.1448557242,1455457.8708242513,1442511.5967927794,1429565.3227613056,1416619.0487298341,1403672.7746983613,1390726.5006668894,1377780.226635416,1364833.952603944,1351887.6785724717,1338941.4045409986,1325995.1305095246,1313048.8564780538,1300102.8474680574,1287161.460338217,1274224.3125688618,1261290.5435256062,1248359.6260211433,1235431.0942098633,1222505.2958412603,1209581.7962291762,1196659.6258615218,1183739.6584500722,1170821.034446735,1157903.579309225,1144987.7631612816,1132072.579836362,1119158.9241571892,1106245.7913955026,1093333.7303095828,1080422.4445885918,1067511.4850620474,1054601.8592148712,1041692.2333676931,1028783.4351890786,1015875.2971386313,1002967.1590881948,990059.83592042,977153.0275107799,964246.2191011319,951339.7747745318,938434.150692015,925528.5266095009,912622.9025269831,899717.8722066302,886813.2989838235,873908.7257610327,861004.1525382288,848099.5793154249,835195.0060926313,822283.5339493733,809377.909866849,796472.285784333,783566.661701818,770644.092956448,757737.2845468009,744830.4761371594,731923.6677275104,718988.1219784357,706079.9839279912,693171.8458775384,680263.7078270996,667355.5697766598,654403.4579524305,641493.8321052613,628584.2062580967,615674.5804109089,602764.9545637416,589855.3287165677,576882.3014700087,563971.015749013,551059.730028036,538148.4443070507,525237.1585860495,512325.8728650622,499414.5871440815,486415.4188347319,473502.28607304953,460589.1533113709,447676.0205496857,434762.88778800797,421849.75502632465,408936.6222646432,396023.48950295243],[1675055.9234266109,1662015.8765393742,1648975.8296521315,1635935.7827648944,1622895.7358776578,1609855.6889904176,1596815.6421031791,1583775.5952159383,1570735.5483287007,1557695.5014414636,1544655.4545542237,1531615.4076669847,1518575.360779746,1505535.313892507,1492495.2670052685,1479455.220118029,1466415.1732307922,1453375.1263435527,1440335.0794563137,1427295.032569074,1414254.9856818365,1401214.9387945973,1388174.8919073585,1375134.8450201182,1362094.7981328804,1349054.7512456437,1336014.7043584036,1322974.6574711646,1309934.6105839275,1296894.5636966894,1283856.8934038694,1270823.289958565,1257793.158244272,1244766.3045542643,1231742.7581080857,1218721.3603248245,1205701.7253928785,1192684.8320041252,1179669.3497580127,1166655.18519047,1153643.044349716,1140631.3515355675,1127621.8396289297,1114612.3819852406,1101604.874981233,1088597.4348271713,1075591.387777858,1062585.7648735126,1049580.7034767927,1036576.7136741057,1023572.7238714187,1010569.70066567,997567.1746629942,984564.6486603217,971562.9973628605,958561.7795551717,945560.5617474681,932559.704001327,919559.6514187083,906559.5988360904,893559.5462534651,880560.016067314,867560.9973950731,854561.9787228238,841562.9600505643,828563.9413783057,815564.922706048,802559.1781751243,789559.1255925,776559.0730098682,763559.0204272699,750542.2946319813,737541.0768242814,724539.8590165898,711538.6412088871,698509.0766013954,685506.5505987201,672504.0245960467,659501.4985933751,646498.9725906895,633453.029790787,620449.0399880838,607445.0501854047,594441.0603827201,581437.070580028,568433.0807773387,555429.0909746457,542360.848699756,529355.2257954031,516349.60289108846,503343.9799867403,490338.3570823837,477332.73417803925,464327.11127368826,451232.8672783142,438225.4271242656,425217.98697020207,412210.5468161432,399203.10666207504,386195.66650802456],[1674567.3174939337,1661433.4977509328,1648299.6780079242,1635165.8582649208,1622032.038521917,1608898.2187789108,1595764.3990359064,1582630.5792929013,1569496.7595498962,1556362.939806893,1543229.120063887,1530095.3003208812,1516961.4805778759,1503827.6608348722,1490693.841091867,1477560.021348862,1464426.2016058564,1451292.3818628527,1438158.5621198474,1425024.7423768416,1411890.922633838,1398757.1028908328,1385623.2831478282,1372489.4634048236,1359355.6436618178,1346221.8239188127,1333088.004175809,1319954.184432802,1306820.364689798,1293686.544946796,1280552.72520379,1267423.2359861648,1254297.4916310105,1241174.93013524,1228055.0542603845,1214937.7925491054,1201823.420223,1188710.6087465067,1175599.5629364434,1162490.6793423267,1149382.5093901968,1136276.5788994175,1123171.0994214965,1110067.3674590276,1096963.9585669572,1083862.2110205214,1070760.4634740935,1057660.3708794243,1044560.4107917221,1031461.1939805108,1018362.8402223159,1005264.4864641139,992167.1902377969,979070.2762828916,965973.3623279752,952877.3400091915,939781.7128034397,926686.0855976916,913590.7762279231,900496.2951451913,887401.8140624771,874307.3329797452,861213.269029106,848119.8049074076,835026.3407856971,821932.8766639847,808839.4125422724,795745.948420573,782645.9654006697,769551.4843179425,756457.0032352144,743362.5221524946,730251.6775113996,717156.0503056515,704060.4230999015,690964.7958941469,677869.1686883867,664744.3413649974,651647.4274100903,638550.5134551711,625453.5995002547,612356.6855453514,599215.5199598828,586117.1662016809,573018.8124434957,559920.4586852733,546822.104927077,533723.7511688843,520562.0072837919,507462.0471960893,494362.08710840065,481262.12702068035,468162.16693299916,455062.206845304,441962.2467575902,428862.2866698988,415673.0861523906,402571.33860595804,389469.5910595264,376367.843513092],[1674078.7115612582,1660851.1189624907,1647623.5263637186,1634395.9337649462,1621168.3411661745,1607940.7485674052,1594713.155968632,1581485.5633698618,1568257.970771092,1555030.3781723212,1541802.7855735505,1528575.1929747798,1515347.6003760071,1502120.007777236,1488892.4151784652,1475664.822579695,1462437.2299809237,1449209.6373821534,1435982.0447833806,1422754.4521846103,1409526.8595858389,1396299.2669870688,1383071.6743882995,1369844.0817895285,1356616.4891907566,1343388.8965919856,1330161.3039932156,1316933.7113944418,1303706.1187956748,1290478.5261969026,1277250.93359813,1264024.8173750609,1250802.545431186,1237583.8461243995,1224368.7237508341,1211156.0398784443,1197945.3815981746,1184737.2739043888,1171531.187257512,1158326.1734941779,1145123.6146144574,1131921.8062632708,1118721.638095852,1105522.3529328196,1092324.3466515802,1079126.987213877,1065930.932275061,1052734.9768853378,1039540.6796142762,1026346.3823432238,1013152.9565732148,999960.2388594989,986767.5211457787,973575.903905455,960384.6019983045,947193.3000911712,934002.863859429,920812.8272556183,907622.7906518094,894432.991454293,881244.0818714704,868055.1722886413,854866.2627058169,841677.631091998,828489.7215208253,815301.8119496685,802113.9023785004,788925.9928073315,775738.0832361802,762543.8956260094,749354.9860431748,736166.0764603382,722977.1668775193,709772.2415947188,696582.204990916,683392.1683871122,670202.1317833094,657012.0951794926,643793.3562267944,630602.0543196481,617410.7524124896,604219.4505053572,591028.1485982127,577793.2720206715,564600.5543069448,551407.8365932312,538215.1188795185,525022.4011657992,511829.6834520791,498636.96573836636,485380.1942300545,472185.8969590096,458991.59968795814,445797.30241690576,432603.0051458441,419408.70787478983,406214.410603744,392932.1303957766,379736.07545696665,366540.0205181511],[1673590.105628584,1660268.7401740442,1646947.37471951,1633626.0092649707,1620304.6438104347,1606983.278355898,1593661.9129013596,1580340.5474468241,1567019.1819922864,1553697.8165377507,1540376.451083213,1527055.0856286753,1513733.7201741382,1500412.354719601,1487090.9892650642,1473769.6238105271,1460448.2583559905,1447126.8929014537,1433805.5274469166,1420484.161992378,1407162.7965378421,1393841.4310833048,1380520.0656287686,1367198.7001742325,1353877.3347196938,1340555.9692651578,1327234.6038106175,1313913.238356087,1300591.8729015451,1287270.507447007,1273949.1419924719,1260627.776537934,1247309.604927076,1233994.9763350429,1220683.3601941268,1207374.5390910546,1194068.9278505724,1180765.132171162,1167462.9213690017,1154163.460925886,1140864.9116102257,1127568.155574414,1114272.6277830624,1100978.22193901,1087685.057806592,1074392.7772065825,1061101.4010760342,1047811.0387448575,1034520.9484368348,1021232.3139824267,1007943.6795280138,994655.9912548824,981368.909585658,968081.8279164289,954795.8416686505,941510.1518092677,928224.461949884,914939.5689135436,901655.1229116851,888370.6769098248,875086.3496804722,861803.0115975467,848519.6735146018,835236.3354316726,821953.1022559572,808670.7472353419,795388.3922147136,782106.0371940956,768823.6821734961,755541.327152878,742258.9721322525,728969.6307682032,715686.2926852629,702402.9546023356,689119.6165194092,675819.5408800766,662535.0948782004,649250.648876342,635966.2028744789,622653.5951841278,609367.9053247543,596082.2154653678,582796.5256059887,569510.8357466031,556225.145887224,542895.2145011714,529608.1328319507,516321.0511627281,503033.96949349623,489746.88782428484,476459.806155039,463109.6668973183,449821.0324428994,436532.3979884973,423243.76353409886,409955.12907968275,396666.49462527875,383377.86017086357,370089.2257164642,356712.1975232428],[1673101.4996959064,1659686.3613856018,1646271.223075302,1632856.0847649982,1619440.946454695,1606025.8081443924,1592610.669834088,1579195.5315237849,1565780.3932134828,1552365.2549031787,1538950.1165928738,1525534.9782825713,1512119.8399722693,1498704.7016619667,1485289.5633516626,1471874.4250413598,1458459.2867310564,1445044.148420753,1431629.0101104514,1418213.8718001465,1404798.7334898445,1391383.5951795413,1377968.4568692364,1364553.3185589337,1351138.1802486319,1337723.0419383284,1324307.9036280245,1310892.7653177246,1297477.6270074188,1284062.4886971149,1270647.350386813,1257232.2120765108,1243817.3346392068,1230406.6891255707,1216999.5600598445,1203595.1863966535,1190193.1263569528,1176793.7181767868,1163396.6765273763,1150000.9328256445,1136607.4297516188,1123215.255414097,1109824.302222197,1096434.9739495902,1083046.3304289002,1069659.214042821,1056272.6591240247,1042887.200153457,1029502.5304298946,1016118.2456216267,1002735.2739838618,989352.3023460992,975970.2980255196,962588.8524007858,949207.406776051,935827.0035273698,922446.9257157515,909066.8479041392,895687.4551715627,882308.5997716542,868929.7443717364,855550.8889718186,842173.0843233988,828795.3177403631,815417.5511573227,802039.7845742889,788662.8820509389,775286.0815808689,761909.281110798,748532.4806407355,735155.6801706553,721778.8797005899,708395.418493025,695017.6519099837,681639.885326949,668262.1187439049,654868.0579731064,641489.2025731923,628110.3471732847,614731.4917733753,601352.6363734622,587944.9804253764,574564.9026137656,561184.82480215,547804.7469905335,534424.6691789189,521001.14678438473,507619.7011596523,494238.2555349069,480856.80991016794,467475.3642854206,454093.91866068076,440712.47303594183,427267.49356009625,413884.52192234155,400501.5502845794,387118.5786468191,373735.6070090635,360352.6353712948,346969.6637335392],[1672612.893763231,1659103.982597162,1645595.0714310908,1632086.160265026,1618577.249098955,1605068.3379328852,1591559.4267668168,1578050.5156007453,1564541.604434677,1551032.693268607,1537523.7821025369,1524014.8709364694,1510505.9597703996,1496997.0486043321,1483488.13743826,1469979.2262721914,1456470.315106124,1442961.4039400537,1429452.492773985,1415943.5816079145,1402434.6704418454,1388925.759275777,1375416.8481097072,1361907.936943636,1348399.0257775702,1334890.1146115006,1321381.2034454318,1307872.292279363,1294363.3811132922,1280854.4699472215,1267345.5587811526,1253836.6476150844,1240327.7364490132,1226820.567160941,1213316.766680964,1199816.296605593,1186319.0343238045,1172823.8678601943,1159330.4316857648,1145840.014666807,1132350.7492300747,1118863.0937347384,1105377.204905787,1091892.0545577952,1078408.8447628357,1064925.9635655093,1051444.821641489,1037963.9924039282,1024484.3844461176,1011005.40733018,997526.8684397154,984049.5596185937,970572.2507974762,957095.8768851431,943620.0673048864,930144.2577246334,916669.3894816162,903194.9237177735,889720.4579539159,876246.5226334641,862773.2578354878,849299.9930375293,835826.7282395652,822354.3000490488,808882.1049659029,795409.9098827718,781937.7147996221,768466.1259676367,754994.8800481148,741523.6341285836,728052.3882090691,714581.1422895407,701109.8963700151,687632.3492176402,674160.1541344989,660687.9590513492,647215.7639682125,633743.5688850665,620254.4914720878,606781.2266741116,593307.9618761605,579834.6970781842,566333.279621535,552858.8138576876,539384.3480938468,525909.882329979,512435.4165661372,498960.9508022843,485442.54157631565,471966.73199604824,458490.92241581064,445015.11283556465,431539.3032553084,418063.4936750671,404587.684094795,391047.9714894779,377570.66266834456,364093.3538472438,350616.04502610676,337138.7362050079],[1672124.2878305544,1658521.603808719,1644918.9197868826,1631316.2357650478,1617713.5517432124,1604110.867721378,1590508.1836995417,1576905.4996777074,1563302.8156558722,1549700.1316340365,1536097.4476122004,1522494.7635903652,1508892.0795685311,1495289.3955466952,1481686.7115248595,1468084.0275030239,1454481.3434811886,1440878.659459354,1427275.9754375184,1413673.2914156825,1400070.6073938468,1386467.9233720128,1372865.2393501769,1359262.5553283405,1345659.8713065074,1332057.1872846717,1318454.5032628372,1304851.8192410017,1291249.1352191668,1277646.451197329,1264043.767175495,1250441.0831536592,1236838.3991318224,1223235.715109988,1209636.0963291563,1196039.5796371792,1182445.6924917232,1168854.903975585,1155266.511717528,1141679.6667167922,1128095.146589438,1114512.260791866,1100930.452875217,1087350.760085295,1073771.4125812156,1060194.240222814,1046617.121348849,1033041.8806026191,1019466.7770462828,1005892.953954028,992319.6694457168,978746.8168910914,965175.1708866195,951603.5248821429,938032.7278337348,924462.5542979753,910892.3807622055,897322.999531405,883754.1458153212,870185.2920992412,856616.7712992625,843049.0971032353,829481.4229072221,815913.7487111995,802346.6587744886,788780.0351912463,775213.4116080003,761646.7880247524,748080.478985427,734514.7876164457,720949.0962474653,707383.4048784804,693817.7135095047,680252.0221405281,666686.330771545,653113.799358787,639547.1757755512,625980.5521923061,612413.9286090601,598830.9615748832,585263.2873788476,571695.6131828297,558127.9389868118,544560.2647907827,530963.9491971498,517395.09548106976,503826.24176497664,490257.38804889936,476688.5343328165,463119.68061673176,449506.4805461969,435936.3070104299,422366.13347466383,408795.9599389024,395225.7864031382,381655.6128673693,368085.4393316079,354451.1006854195,340879.45468094293,327307.8086764794],[1671635.6818978786,1657939.2250202761,1644242.7681426757,1630546.3112650742,1616849.8543874728,1603153.397509873,1589456.9406322695,1575760.4837546686,1562064.0268770673,1548367.5699994648,1534671.1131218642,1520974.6562442626,1507278.1993666617,1493581.7424890588,1479885.2856114577,1466188.8287338573,1452492.371856256,1438795.9149786541,1425099.4581010519,1411403.001223451,1397706.5443458492,1384010.0874682479,1370313.630590646,1356617.173713047,1342920.7168354425,1329224.2599578418,1315527.803080241,1301831.3462026387,1288134.8893250376,1274438.432447435,1260741.9755698363,1247045.5186922352,1233349.061814631,1219652.604937029,1205956.148059427,1192263.9221438468,1178574.545414344,1164887.5730058076,1151202.7953519868,1137521.057928924,1123840.6955310334,1110162.07229529,1096485.4675110318,1082809.5071730637,1069135.9209526242,1055462.5168801178,1041791.1603295179,1028119.8037789161,1014450.390926192,1000781.0130510726,987112.908677185,973445.3167764992,959778.0909757572,946112.1077879295,932446.1246000985,918780.8508712961,905116.3133800183,891451.7758887438,877787.8336767284,864124.5920083979,850461.3503400935,836798.2011689506,823136.1175748706,809474.0339807961,795811.9503867244,782150.1604997264,768489.1084163748,754828.0563330231,741167.0042496789,727505.9521663236,713845.8042858746,700185.6674674358,686525.5306489952,672865.3938305667,659205.2570121344,645545.1201936966,631878.5875828872,618217.535499528,604556.4834161829,590895.431332822,577234.3792494889,563556.5292874556,549894.4456933923,536232.3620993057,522570.2785052443,508908.19491116516,495217.0669638226,481553.8252954958,467890.5836271858,454227.34195885435,440564.1002905518,426857.5011852952,413192.96369403973,399528.4262027377,385863.88871147484,372199.35122018866,358534.8137289034,344870.27623762656,331142.8643357763,317476.88114795275],[1671147.0759652015,1657356.8462318357,1643566.616498468,1629776.3867651024,1615986.1570317324,1602195.9272983652,1588405.6975649968,1574615.4678316293,1560825.2380982616,1547035.0083648937,1533244.778631527,1519454.54889816,1505664.3191647928,1491874.0894314244,1478083.8596980562,1464293.6299646897,1450503.4002313223,1436713.170497954,1422922.9407645855,1409132.711031219,1395342.4812978525,1381552.2515644846,1367762.0218311157,1353971.7920977497,1340181.5623643799,1326391.3326310148,1312601.1028976436,1298810.8731642796,1285020.643430912,1271230.4136975443,1257440.1839641777,1243649.9542308082,1229859.7244974421,1216069.494764073,1202279.265030707,1188490.0968256998,1174704.044605021,1160921.6640124423,1147141.4752445272,1133363.1099664196,1119587.5064944122,1105813.5181285054,1092040.7917843563,1078270.3693875694,1064500.429324042,1050732.687507785,1036965.1993101873,1023199.6050829459,1009434.0108557148,995670.3526122156,981906.7004183242,968144.248615596,954382.34932253,940620.6906937114,926860.370322532,913100.0499513489,899340.2459978461,885581.3445510464,871822.4431042597,858063.8919175658,844306.2622970091,830548.6326764589,816791.003055905,803034.3192504048,789277.8262582766,775521.3332661502,761764.840274022,748009.3246413022,734253.8440578412,720498.3634743923,706742.8828909434,692987.9300563894,679233.3477885025,665478.7655206015,651724.1832527202,637969.6009848285,624215.0187169313,610460.4364490556,596699.0382233104,582943.5576398559,569188.0770564014,555432.5964729469,541660.9523999691,527904.4594078548,514147.9664157238,500391.4734235918,486634.98043146916,472850.262542109,459092.63292156346,445335.0033010077,431577.3736804528,417819.74405989423,404062.11443934124,390260.8924665749,376501.9910197975,362743.08957299776,348984.18812620547,335225.2866794225,321466.3852326274,307707.4837858472],[1670658.470032526,1656774.467443392,1642890.4648542614,1629006.4622651234,1615122.4596759905,1601238.4570868588,1587354.4544977252,1573470.4519085907,1559586.4493194572,1545702.446730324,1531818.4441411905,1517934.441552055,1504050.4389629234,1490166.4363737903,1476282.4337846546,1462398.4311955222,1448514.4286063872,1434630.4260172541,1420746.423428121,1406862.4208389872,1392978.4182498534,1379094.4156607203,1365210.4130715854,1351326.4104824527,1337442.4078933178,1323558.4053041865,1309674.4027150515,1295790.400125916,1281906.3975367856,1268022.3949476518,1254138.3923585159,1240254.3897693814,1226370.3871802506,1212486.384591116,1198602.3820019825,1184718.3794128513,1170836.3676101165,1156957.0998649728,1143080.9354314837,1129207.3992078644,1115335.50970894,1101465.857413969,1087598.1345091732,1073731.3050566353,1059866.9664214766,1046002.9987231083,1032141.0597507618,1018279.4063869794,1004419.5744831087,990559.7425792394,976701.7656606878,962843.8391480218,948986.9737692568,935130.7670838181,921274.6160449623,907419.9584904285,893565.3009358831,879710.913213361,865857.6478110561,852004.3824087502,838151.1742539518,824299.1566811493,810447.1391083775,796595.1215355797,782743.7021298315,768892.7997396467,755041.8973494675,741190.9949592892,727340.6838660156,713490.7747824639,699640.8656988963,685790.9566153381,671941.164927993,658092.137210655,644243.1094933143,630394.0817759708,616545.0540586188,602696.0263412837,588846.9986239364,574991.683946874,561141.7748633213,547291.865779764,533441.9566962048,519592.0476126345,505725.6543261986,491874.7519360082,478023.8495458374,464172.94715564325,450322.0447654752,436442.6646431321,422590.6470703343,408738.6294975607,394886.611924774,381034.59435199667,367182.5767791914,353286.82792582177,339433.56252352055,325580.2971212212,311727.0317189153,297873.76631660573],[1670169.8640998497,1656192.0886549521,1642214.3132100478,1628236.5377651507,1614258.7623202533,1600280.986875353,1586303.2114304518,1572325.4359855514,1558347.6605406532,1544369.885095753,1530392.1096508533,1516414.3342059534,1502436.5587610528,1488458.7833161536,1474481.007871253,1460503.2324263542,1446525.4569814547,1432547.6815365546,1418569.9060916556,1404592.1306467555,1390614.3552018548,1376636.5797569552,1362658.8043120562,1348681.0288671565,1334703.2534222538,1320725.4779773557,1306747.7025324577,1292769.9270875538,1278792.1516426585,1264814.3761977612,1250836.6007528573,1236858.8253079583,1222881.0498630577,1208903.2744181564,1194925.4989732567,1180947.7235283577,1166969.948083459,1152994.9350410048,1139022.4476944925,1125052.3596714768,1111085.3448958267,1097119.8118055305,1083156.110687601,1069194.54467306,1055233.6762146591,1041275.2586127543,1027317.1738099963,1013361.0376815591,999405.1381105054,985451.0685299877,971496.9989494844,957544.6300716139,943592.4292401704,929641.0841381578,915690.5700603398,901740.0559825334,887790.8722916082,873841.8775537256,859892.8828158369,845945.2231600415,831997.5938022183,818049.9644444035,804103.2751608444,790156.8696358111,776210.4641107917,762264.2662131414,748318.9544249093,734373.6426366763,720428.3308484433,706483.1860905215,692538.8485068558,678594.5109231928,664650.1733395401,650705.8357558772,636762.0357339038,622818.5625670953,608875.0894003017,594931.6162335034,580988.1430667043,567044.6698998986,553101.196733105,539151.1350865746,525206.7975029163,511262.45991924964,497318.1223355988,483373.78475193586,469412.71866020374,455467.40687195957,441522.0950837163,427576.7832954861,413631.4715072531,399657.51493523177,385711.10941020213,371764.7038851874,357818.29836016614,343871.8928351486,329925.48731013294,315935.3075630022,301987.678205193,288040.0488473736],[1669681.2581671732,1655609.7098665105,1641538.161565842,1627466.6132651775,1613395.064964509,1599323.5166638456,1585251.968363181,1571180.4200625133,1557108.8717618475,1543037.3234611838,1528965.7751605166,1514894.2268598513,1500822.678559184,1486751.1302585178,1472679.5819578515,1458608.0336571857,1444536.48535652,1430464.9370558546,1416393.3887551893,1402321.840454523,1388250.2921538574,1374178.743853192,1360107.1955525258,1346035.6472518598,1331964.0989511937,1317892.5506505282,1303821.0023498612,1289749.4540491956,1275677.9057485294,1261606.3574478657,1247534.8091471985,1233463.2608465329,1219391.7125458673,1205320.1642451999,1191248.6159445343,1177177.0676438664,1163105.5193432062,1149033.9710425418,1134965.7991183745,1120899.9483449492,1106836.0465129907,1092775.3123084134,1078716.016256208,1064658.2663153103,1050602.7486201627,1036547.8159846202,1022495.2459614025,1008442.9545847042,994392.6213001837,980342.3944807472,966294.0872236029,952245.7799664633,938198.9458449786,924152.4706947659,910106.5797223174,896061.7582521299,882016.9367819298,867973.1117260838,853929.7798048425,839886.4478835976,825844.0705979783,811802.0772846555,797760.0839713262,783718.6177360332,769677.824258795,755637.0307815261,741596.2373042749,727556.2903140672,713516.5691277683,699476.8479414834,685437.1267551966,671398.0652310569,657359.2991472911,643320.533063516,629281.766979753,615243.0433582375,601205.1247419808,587167.2061257325,573129.2875094749,559091.368893221,545053.4502769662,531015.5316607263,516977.61304446496,502932.87222584244,488894.1061420962,474855.3400583258,460816.57397455256,446761.8665882889,432722.14540197793,418682.42421570513,404642.70302941557,390602.9818431111,376563.26065684203,362494.81341838464,348454.01994113065,334413.2264638813,320372.4329866078,306331.6395093752,292290.846032111,278206.33137814794],[1669192.6522344982,1655027.3310780688,1640862.0099216336,1626696.6887652026,1612531.3676087721,1598366.046452338,1584200.7252959064,1570035.4041394764,1555870.0829830426,1541704.7618266118,1527539.4406701801,1513374.1195137454,1499208.7983573151,1485043.4772008825,1470878.1560444497,1456712.8348880182,1442547.5137315874,1428382.1925751544,1414216.8714187222,1400051.5502622915,1385886.2291058598,1371720.9079494292,1357555.5867929948,1343390.2656365621,1329224.9444801325,1315059.6233237002,1300894.3021672661,1286728.9810108363,1272563.6598544035,1258398.3386979715,1244233.017541539,1230067.6963851058,1215902.375228675,1201737.0540722432,1187571.7329158115,1173406.4117593816,1159241.0906029437,1145075.7694465136,1130910.4482900868,1116748.9598422311,1102589.6018163473,1088432.0247878903,1074277.3014456257,1060124.1230609547,1045972.3242971031,1031822.7463504705,1017673.6509119547,1003526.9284674148,989380.3410472353,975235.8106066249,961091.280166015,946948.6305639381,932806.0856301589,918664.7129808059,904523.9635118167,890383.4605217148,876244.3316591559,862105.2027965859,847966.6767938491,833829.0076892506,819691.3385846568,805554.1901249187,791417.8328560675,777281.4755872302,763145.1844067769,749010.0029772939,734874.8215478035,720739.6401183167,706604.8074071072,692470.6768227704,678336.546238427,664202.415654079,650068.4249550384,635935.230371167,621802.0357872937,607668.8412034279,593535.6466195583,579402.7960179513,565270.4319522474,551138.0678865369,537005.7038208293,522873.3397551235,508740.97568941396,494608.61162371654,480476.24755800795,466336.89536472876,452203.70078085363,438070.5061969822,423937.31161311176,409788.06513591204,395653.9345515752,381519.80396723375,367385.67338289786,353251.5427985452,339117.41221419536,324954.56009259634,310819.37866310775,296684.19723362755,282549.0158041427,268413.8343746457],[1668704.0463018212,1654444.952289628,1640185.858277429,1625926.7642652285,1611667.6702530317,1597408.5762408338,1583149.4822286346,1568890.3882164357,1554631.2942042388,1540372.20019204,1526113.1061798404,1511854.0121676428,1497594.918155445,1483335.824143249,1469076.7301310473,1454817.6361188511,1440558.5421066526,1426299.4480944546,1412040.354082256,1397781.2600700594,1383522.1660578602,1369263.0720456634,1355003.9780334665,1340744.8840212657,1326485.7900090693,1312226.6959968698,1297967.6019846748,1283708.507972476,1269449.4139602752,1255190.319948076,1240931.225935882,1226672.1319236816,1212413.0379114822,1198153.9438992867,1183894.8498870856,1169635.7558748843,1155376.661862691,1141117.567850498,1126858.4738382925,1112599.7278170413,1098344.4172125608,1084091.4081086977,1069840.0247874097,1055591.3123074602,1041344.1322197774,1027098.2846329631,1012854.5378639991,998611.2959778938,984370.3061307967,970129.4312649509,955890.6056008926,941651.8780041886,927414.698551009,913177.915940586,898941.931479082,884706.907691319,870471.8839035397,856238.2902814317,842004.8540264918,827771.5674949028,813539.5612069517,799307.5549190035,785075.5817408087,770844.8605164569,756614.1392921042,742383.4180677421,728153.4057913283,713923.8364096088,699694.2670278791,685464.697646155,671235.9657216668,657007.4257392557,642778.8857568614,628550.3457744587,614322.3045948436,600094.6815108694,585867.0584268915,571639.4353429182,557411.8122589542,543184.7668798538,528957.9573646914,514731.14784953557,500504.3383343704,486277.5288192034,472050.71930405125,457823.90978887957,443590.82758713234,429363.2045031823,415135.58141920157,400907.9583352171,386680.33525125775,372436.62609132566,358208.08610892855,343979.546126551,329751.00614414737,315522.46616173536,301293.9261793569,287036.75495789014,272807.18557616416,258577.61619442515],[1668215.4403691455,1653862.5735011827,1639509.7066332204,1625156.839765255,1610803.9728972896,1596451.106029324,1582098.2391613615,1567745.3722933982,1553392.5054254332,1539039.63855747,1524686.7716895042,1510333.9048215414,1495981.0379535765,1481628.171085612,1467275.3042176466,1452922.437349685,1438569.5704817194,1424216.7036137553,1409863.83674579,1395510.9698778274,1381158.1030098628,1366805.2361418968,1352452.3692739347,1338099.5024059694,1323746.6355380067,1309393.7686700416,1295040.9018020798,1280688.0349341142,1266335.168066149,1251982.3011981845,1237629.4343302213,1223276.5674622562,1208923.7005942944,1194570.8337263288,1180217.9668583658,1165865.099990401,1151512.23312244,1137159.366254468,1122806.4993865085,1108453.6325185387,1094101.3335106121,1079752.17122937,1065405.367221986,1051060.0465115565,1036717.3448939226,1022376.0437326883,1008036.147322908,993698.1231607455,979360.7781662443,965025.3789515574,950690.3516643881,936357.006282982,922024.0815301877,907692.2911848067,893361.2708977489,879030.6013398021,864701.3032332584,850372.0051267194,836043.6341189565,821715.8904716335,807388.1468243189,793061.4403579524,778735.096886646,764408.753415334,750083.1602658136,735758.0750859547,721432.9899060866,707108.0327009028,692784.0753669376,678460.1180329816,664136.1606990173,649812.4358244371,635489.4864439806,621166.5370635362,606843.5876830928,592520.638302628,578198.4702342199,563876.4186501475,549554.3670660714,535232.3154819962,520910.2638979135,506588.9559439346,492267.70097932406,477946.44601470977,463625.19105010293,449303.9360854812,434982.6811208483,420661.42615624797,406333.8512252867,392011.7996412134,377689.74805713724,363367.6964730434,349045.6448889738,334707.54945453256,320384.60007408913,306061.65069363173,291738.7013131827,277415.75193272904,263092.8025522847,248741.39801423997],[1667726.8344364683,1653280.194712738,1638833.5549890113,1624386.9152652794,1609940.2755415516,1595493.6358178204,1581046.9960940897,1566600.356370358,1552153.716646629,1537707.0769228982,1523260.437199167,1508813.7974754376,1494367.157751707,1479920.5180279766,1465473.8783042452,1451027.2385805182,1436580.598856787,1422133.9591330555,1407687.3194093248,1393240.679685596,1378794.0399618654,1364347.4002381347,1349900.7605144044,1335454.120790673,1321007.4810669438,1306560.8413432136,1292114.2016194838,1277667.5618957537,1263220.9221720207,1248774.2824482913,1234327.6427245582,1219881.003000832,1205434.3632771014,1190987.723553369,1176541.08382964,1162094.4441059153,1147647.8043821775,1133201.1646584515,1118754.524934717,1104307.8852109923,1089861.2454872583,1075415.2358506601,1060972.221892661,1046531.4791562171,1032092.0899603274,1017655.3992050062,1003219.8575996626,988785.9123669234,974353.5022407002,959922.054137812,945492.1469296864,931062.9672212056,916635.0126528889,902207.8907440142,887781.4084653179,873356.1505016326,858930.8925379375,844507.1501376564,830083.5777123319,815660.363171733,801238.3121320447,786816.2610923462,772394.6451422228,757973.9644875675,743553.2838329114,729132.7321041487,714713.2829687772,700293.8338333899,685874.3846980212,671455.538419799,657037.1931336084,642618.8478474142,628200.502561219,613782.7283526091,599365.3695740998,584948.0107955988,570530.652017084,556113.4019573824,541696.9218731932,527280.4417890226,512863.9617048381,498447.48162065074,484031.06362427026,469615.3632102115,455199.6627961444,440783.96238206234,426368.2619679924,411952.56155393086,397536.8611398572,383121.16072577983,368699.16086300183,354282.68077883776,339866.2006946476,325449.7206104584,311033.2405262813,296600.8352255272,282183.47644701693,267766.1176685067,253348.75889001414,238931.40011151694],[1667238.228503793,1652697.8159242957,1638157.4033448007,1623616.990765304,1609076.5781858081,1594536.1656063108,1579995.7530268151,1565455.3404473194,1550914.9278678242,1536374.5152883267,1521834.1027088305,1507293.6901293334,1492753.2775498382,1478212.8649703423,1463672.4523908438,1449132.0398113476,1434591.6272318517,1420051.2146523572,1405510.80207286,1390970.389493364,1376429.9769138682,1361889.564334372,1347349.1517548752,1332808.7391753765,1318268.32659588,1303727.9140163844,1289187.5014368892,1274647.0888573916,1260106.6762778945,1245566.2636983984,1231025.8511189027,1216485.4385394074,1201945.025959911,1187404.613380413,1172864.2008009173,1158323.7882214193,1143783.37564193,1129242.963062426,1114702.5504829343,1100162.1379034352,1085621.725323941,1071081.312744439,1056541.4348371918,1042004.5692024291,1027469.7439113958,1012936.1551337219,998405.4752407135,983875.5738207204,969347.5797650176,954820.6751038707,940295.1238925843,925770.6100651822,911247.2779353871,896724.6247106213,882203.3056456572,867682.050392569,853162.5547522362,838643.0591119071,824124.4484045133,809606.6016604193,795088.7549163206,780572.1190076796,766055.7605756167,751539.4021435585,737024.1577217905,722509.1398837725,707994.1220457572,693479.7629405735,678965.9498496819,664452.1367587792,649938.3236678988,635425.4923297651,620912.7590913456,606400.0258529121,591887.2926144917,577375.3832885567,562863.6151120048,548351.8469354408,533840.078758873,519328.56809603795,504817.65951176174,490306.7509274669,475795.842343173,461284.9337588921,446774.134542183,432263.9886786593,417753.84281513,403243.6969516054,388733.55108808074,374223.4052245524,359713.2593610333,345203.1134975068,330686.7565003205,316175.8479160238,301664.9393317355,287154.0307474658,272643.12216317374,258132.21357887518,243604.7152277343,229092.94705118798],[1666749.622571118,1652115.4371358554,1637481.2517005922,1622847.0662653316,1608212.880830069,1593578.695394805,1578944.5099595431,1564310.3245242815,1549676.1390890186,1535041.9536537556,1520407.7682184936,1505773.582783231,1491139.397347969,1476505.211912707,1461871.0264774435,1447236.8410421808,1432602.6556069187,1417968.4701716555,1403334.2847363935,1388700.099301132,1374065.9138658703,1359431.7284306057,1344797.542995344,1330163.3575600826,1315529.1721248173,1300894.9866895557,1286260.8012542923,1271626.6158190293,1256992.4303837682,1242358.2449485057,1227724.0595132436,1213089.87407798,1198455.688642717,1183821.5032074568,1169187.3177721933,1154553.1323369313,1139918.946901665,1125284.7614664086,1110650.5760311396,1096016.3905958813,1081382.2051606183,1066748.0197253604,1052113.8342900989,1037479.930470197,1022849.2131586722,1008220.1614875025,993592.6083377916,978967.5730010448,964343.1923958571,949721.1495171925,935099.6417502565,920479.9874305739,905860.7683580532,891243.2838069373,876625.842456182,862010.3262351183,847394.8100140588,832780.4836495705,818166.7503326265,803553.1980338031,788941.0769709358,774328.9559080601,759717.311098584,745106.6452741483,730495.9794496968,715885.676589299,701276.3215679331,686666.9665465634,672057.6115251975,657449.3379549384,642841.1609085333,628232.9838621216,613625.0156214535,599017.8944307975,584410.7732401378,569803.6520494744,555196.5782068977,540590.4006322864,525984.2230576789,511378.04548308253,496771.86790845916,482166.0202342747,467560.6831498854,452955.3460654989,438350.0089811245,423744.6718967203,409139.4236622611,394534.8323492799,379930.24103631545,365325.6497233361,350721.05841035023,336116.46709736437,321511.8757843999,306907.284471428,292302.69315843564,277691.30105280224,263085.96396841947,248480.62688403204,233875.28979964275,219269.95271525532],[1666261.016638441,1651533.0583474133,1636805.1000563854,1622077.141765359,1607349.183474328,1592621.225183298,1577893.2668922725,1563165.3086012418,1548437.350310214,1533709.3920191864,1518981.4337281568,1504253.4754371285,1489525.517146099,1474797.5588550712,1460069.600564042,1445341.642273013,1430613.6839819853,1415885.7256909562,1401157.7673999267,1386429.8091089001,1371701.8508178708,1356973.89252684,1342245.9342358129,1327517.9759447852,1312790.0176537563,1298062.0593627281,1283334.1010717007,1268606.1427806686,1253878.1844896416,1239150.2261986148,1224422.2679075864,1209694.3096165552,1194966.351325526,1180238.3930345005,1165510.434743469,1150782.4764524363,1136054.5181614165,1121326.5598703832,1106598.6015793625,1091870.643288326,1077142.6849972955,1062414.7267062683,1047686.7684152354,1032958.8101242133,1018230.851833187,1003506.1537614055,988782.7318845675,974061.2551453244,959341.6924860012,944623.0343176033,929906.6216234453,915190.5299218153,900476.6447517788,885762.8873237157,871050.9848358585,856339.3478634306,841628.9525124049,826919.2391352635,812209.9371936414,797501.9662000397,782793.9952064464,768087.0036439048,753380.6082622604,738674.2128806217,723968.9151879111,709263.9419710906,694558.968754272,679854.8288853811,665151.1366806505,650447.4444759274,635743.9981491761,621041.4571472555,606338.9161453312,591636.3751434125,576934.253865785,562232.7447228897,547531.2355799843,532829.7264371039,518128.3673564764,503427.7803838169,488727.1934111556,474026.60643848684,459326.01946581155,444625.75837210845,429925.99278761726,415226.22720311955,400526.46161862,385826.6960341232,371126.93098454457,356427.8942220984,341728.85745966714,327029.82069724705,312330.78393479344,297631.747172378,282932.7104099393,268233.6736474931,253534.63688507117,238829.04018918425,224129.27460468933,209429.50902019627],[1665772.4107057643,1650950.6795589721,1636128.948412179,1621307.2172653805,1606485.4861185886,1591663.7549717922,1576842.0238249993,1562020.2926782044,1547198.5615314078,1532376.8303846144,1517555.0992378194,1502733.368091024,1487911.636944231,1473089.9057974345,1458268.1746506407,1443446.4435038457,1428624.7123570514,1413802.9812102567,1398981.2500634613,1384159.5189166686,1369337.78776987,1354516.0566230768,1339694.3254762846,1324872.5943294885,1310050.8631826935,1295229.1320358994,1280407.4008891054,1265585.66974231,1250763.938595514,1235942.2074487207,1221120.4763019257,1206298.7451551293,1191477.0140083341,1176655.282861542,1161833.5517147477,1147011.8205679506,1132190.0894211512,1117368.3582743648,1102546.627127566,1087724.8959807688,1072903.1648339806,1058081.433687185,1043259.7025403907,1028437.9713935899,1013616.2402467988,998794.5091000008,983975.3910106113,969157.4551025657,954342.0547737977,939527.8336955756,924715.1863895021,909903.9960837695,895093.8555591791,880285.0958561953,865477.2353197434,850670.3810221548,835864.5916284136,821059.1844775164,806255.2739442829,791451.363411041,776648.7067141877,761846.498043959,747044.381679331,732243.7119789114,717443.0422784919,702642.5703169247,687843.2897077389,673044.0090985354,658244.728489344,643446.6324480381,628648.603059954,613850.5736718727,599053.0254749497,584256.1205175165,569459.2155600823,554662.3106026454,539865.9403011808,525070.0432060445,510274.14611092117,495478.24901578855,480682.5189138409,465887.52254312485,451092.5261724163,436297.5298017012,421502.5334309777,406707.7825095188,391913.5884249117,377119.3943403112,362325.2002557097,347531.00617113337,332736.81208652817,317943.1742971074,303149.69208521396,288356.209873328,273562.7276614299,258769.24544955138,243975.76323766913,229182.2810257813,214388.79881389067,199589.06532512605],[1665283.80477309,1650368.3007705305,1635452.796767969,1620537.2927654083,1605621.7887628463,1590706.2847602856,1575790.7807577245,1560875.2767551667,1545959.7727526037,1531044.2687500427,1516128.7647474827,1501213.2607449214,1486297.756742361,1471382.2527398001,1456466.748737239,1441551.2447346793,1426635.7407321185,1411720.236729558,1396804.7327269944,1381889.2287244366,1366973.7247218727,1352058.2207193142,1337142.7167167536,1322227.2127141932,1307311.7087116314,1292396.20470907,1277480.7007065082,1262565.19670395,1247649.692701387,1232734.1886988285,1217818.6846962657,1202903.1806937072,1187987.6766911442,1173072.1726885843,1158156.668686022,1143241.164683463,1128325.6606809022,1113410.1566783371,1098494.6526757777,1083579.1486732215,1068663.644670655,1053748.1406680932,1038832.6366655319,1023917.1326629757,1009001.6286604125,994086.124657853,979170.6206552885,964256.9249063004,949344.6525645042,934435.0072232136,919525.996629782,904619.3601860153,889713.2728981753,874809.0835506236,859905.3407438309,845003.3770989832,830101.4723658133,815201.5305507574,800301.5887357099,785402.9144411143,770504.8067517951,755606.9718750874,740710.5255281972,725814.0791813377,710918.2670580102,696023.323038809,681128.3790196218,666234.0226596277,651340.4346580543,636446.8466564901,621553.4538487159,606661.0872772764,591768.72070585,576876.3541344162,561984.5969786746,547093.328065726,532202.0591527829,517310.7902398277,502420.07492763083,487529.7898802692,472639.5048328936,457749.2197855385,442859.03287898935,427969.6271102205,413080.2213414544,398190.8155726688,383301.4098039018,368412.0926465159,353523.47006179765,338634.8474771092,323746.2248924142,308857.60230769124,293968.9797229981,279080.6725742845,264192.74491293356,249304.81725160033,234416.88959025778,219528.96192890778,204641.03426757827,189753.10660622828],[1664795.198840416,1649785.9219820886,1634776.6451237586,1619767.3682654356,1604758.0914071063,1589748.8145487802,1574739.5376904523,1559730.2608321258,1544720.983973799,1529711.707115473,1514702.4302571472,1499693.1533988179,1484683.8765404923,1469674.5996821653,1454665.3228238388,1439656.0459655107,1424646.7691071827,1409637.4922488562,1394628.21539053,1379618.9385322046,1364609.6616738753,1349600.3848155483,1334591.1079572213,1319581.8310988958,1304572.5542405695,1289563.2773822425,1274554.0005239134,1259544.7236655897,1244535.4468072616,1229526.1699489343,1214516.8930906076,1199507.6162322797,1184498.339373952,1169489.0625156267,1154479.785657297,1139470.508798968,1124461.2319406494,1109451.955082322,1094442.6782239904,1079433.4013656653,1064424.1245073355,1049414.8476490136,1034405.5707906862,1019396.2939323564,1004387.0170740322,989377.7402157001,974368.4633573713,959359.1864990462,944350.7554484764,929344.6314299055,914340.1124935737,899336.8649857421,884335.5557071622,869334.9301257059,854336.2138961456,839337.9757256303,824341.3126614261,809345.2459081728,794350.1646304661,779356.0703941016,764362.1606258005,749369.8557803798,734377.5509349629,719386.0776591776,704395.3936356688,689404.7096121479,674415.0551615823,659425.8368236199,644436.6184856454,629448.2454328192,614460.350038873,599472.4546449296,584484.8971278928,569498.1933730962,554511.4896183088,539524.7858635271,524538.7126603518,509553.07979188766,494567.4469234282,479581.8140549669,464596.65774514247,449611.98474555183,434627.31174594536,419642.63874635845,404657.96574676316,389674.0940851085,374690.27891827933,359706.4637514474,344722.64858462755,329738.83341780026,314755.63769826666,299772.58661346696,284789.53552867007,269806.48444385827,254823.43335905485,239840.38905364554,224858.015942852,209875.64283205476,194893.2697212603,179910.89661045372],[1664306.592907739,1649203.5431936472,1634100.4934795527,1618997.4437654603,1603894.3940513674,1588791.3443372743,1573688.2946231808,1558585.2449090863,1543482.1951949936,1528379.1454809024,1513276.0957668058,1498173.0460527143,1483069.9963386224,1467966.9466245286,1452863.8969104374,1437760.8471963424,1422657.7974822498,1407554.747768157,1392451.6980540631,1377348.6483399724,1362245.5986258779,1347142.5489117827,1332039.4991976914,1316936.4494835993,1301833.3997695067,1286730.3500554124,1271627.3003413195,1256524.250627229,1241421.200913132,1226318.1511990419,1211215.1014849504,1196112.0517708536,1181009.0020567614,1165905.9523426662,1150802.9026285782,1135699.8529144828,1120596.8032003874,1105493.753486292,1090390.703772205,1075287.6540581142,1060184.6043440173,1045081.5546299238,1029978.5049158293,1014875.455201732,999772.4054876459,984669.3557735495,969566.3060594574,954463.2563453699,939360.2066312744,924257.1569171799,909156.9069417943,894057.3705848777,878960.1994875213,863863.7729529184,848769.1582337879,833675.2465957394,818582.9596022684,803491.0420070924,788400.8721454525,773310.7022838145,758222.2472098684,743134.0005521951,728046.5104967728,712960.008495274,697873.5064937649,682788.2327367263,667703.3110365886,652618.3893364379,637534.7459900677,622451.2533333199,607367.7606765768,592285.2386365077,577203.035850198,562120.833063867,547038.9916736521,531957.9507355029,516876.90979735367,501795.8688592268,486715.3725199625,471635.3756959997,456555.3788720323,441475.38204805646,426395.68875373434,411316.62780188676,396237.5668500615,381158.5058982223,366079.44494639523,351000.9234677674,335922.6989028873,320844.47433801927,305766.24977312423,290688.0252082553,275610.09133433737,260532.61174942367,245455.13216449972,230377.6525795944,215300.17299469654,200222.69340977818,185145.5051749386,170068.68661469035],[1663817.9869750643,1648621.1644052044,1633424.3418353442,1618227.519265484,1603030.6966956244,1587833.8741257656,1572637.0515559088,1557440.228986049,1542243.4064161903,1527046.5838463309,1511849.76127647,1496652.938706612,1481456.1161367516,1466259.2935668926,1451062.4709970355,1435865.648427175,1420668.8258573164,1405472.0032874572,1390275.180717597,1375078.3581477401,1359881.5355778805,1344684.7130080205,1329487.890438163,1314291.0678683023,1299094.2452984427,1283897.4227285846,1268700.600158726,1253503.7775888664,1238306.9550190067,1223110.1324491461,1207913.3098792888,1192716.48730943,1177519.6647395708,1162322.842169711,1147126.0195998517,1131929.1970299962,1116732.374460136,1101535.5518902764,1086338.729320414,1071141.906750558,1055945.084180696,1040748.2616108395,1025551.439040977,1010354.616471122,995157.7939012591,979960.9713313975,964764.1487615434,949567.3261916842,934370.5036218222,919173.6810519649,903976.8584821098,888781.4791001612,873587.651728082,858395.6868102355,843204.5921233473,828015.4080664795,812826.8042096561,797639.8458329849,782453.5100165494,767268.2921659509,752084.0191959096,736900.119183016,721717.7201040201,706535.3210250102,691354.0717433933,676173.3725858014,660992.6734282235,645813.437107739,630634.2777309557,615455.1183541715,600277.3395434413,585099.5725679183,569921.8055923972,554745.0022706967,539568.4920919975,524391.9819133068,509215.737485975,494040.3593644947,478864.9812429929,463689.6031214893,448514.57655754406,433340.2157780528,418165.8549985811,402991.4942190964,387817.1679533692,372643.7190492973,357470.2701452328,342296.82124117017,327123.3723371038,311950.1152582178,296777.4812952867,281604.8473323425,266432.2133694133,251259.57940648217,236086.94544354267,220914.92288494948,205743.01479993667,190571.1067149248,175399.1986299008,160227.29054488055],[1663329.3810423864,1648038.785616762,1632748.190191138,1617457.5947655109,1602166.999339886,1586876.4039142597,1571585.8084886353,1556295.2130630114,1541004.6176373847,1525714.0222117594,1510423.4267861336,1495132.8313605087,1479842.235934884,1464551.6405092594,1449261.045083634,1433970.4496580071,1418679.8542323825,1403389.2588067576,1388098.6633811307,1372808.0679555093,1357517.4725298837,1342226.8771042572,1326936.2816786314,1311645.6862530056,1296355.0908273824,1281064.4954017566,1265773.89997613,1250483.3045505062,1235192.7091248804,1219902.113699252,1204611.5182736316,1189320.9228480037,1174030.327422378,1158739.7319967537,1143449.1365711265,1128158.5411455007,1112867.945719875,1097577.3502942473,1082286.754868622,1066996.1594430008,1051705.564017375,1036414.968591752,1021124.3731661271,1005833.7777404999,990543.1823148839,975252.5868892488,959961.9914636249,944671.396038,929380.8006123728,914090.2051867475,898799.6097611259,883509.0143355015,868218.4189098743,852930.668856164,837643.3269538986,822358.3086774843,807073.6796237938,791791.0866292883,776508.6344177686,761228.2497783713,745947.8651389759,730669.1298912168,715390.7538127992,700113.1348132119,684836.5833129035,669560.031812571,654284.8443656815,639009.9480519891,623735.0878256205,608461.6907722019,593188.293718785,577914.8966653617,562642.835821717,547370.7945274264,532098.7532331357,516827.53633538727,501556.71876432374,486285.90119323786,471015.1345649045,455745.4192600604,440475.7039552117,425205.98865034897,409936.32477306575,394667.60003805626,379398.8753030794,364130.15056806523,348861.42583307717,333593.258487761,318325.4216314675,303057.58477515634,287789.74791887496,272521.9110625526,257254.6260954691,241987.5827344805,226720.53937349003,211453.49601251166,196186.4526515277,180919.52002007328,165653.1834349651,150386.846849842],[1662840.7751097116,1647456.4068283197,1632072.0385469277,1616687.6702655354,1601303.3019841444,1585918.9337027532,1570534.5654213612,1555150.19713997,1539765.8288585797,1524381.4605771888,1508997.0922957966,1493612.7240144052,1478228.355733013,1462843.9874516227,1447459.619170232,1432075.2508888408,1416690.88260745,1401306.514326057,1385922.1460446673,1370537.7777632764,1355153.4094818851,1339769.0412004949,1324384.6729190995,1309000.3046377082,1293615.936356317,1278231.568074926,1262847.199793535,1247462.8315121431,1232078.4632307522,1216694.09494936,1201309.7266679693,1185925.358386577,1170540.990105186,1155156.621823796,1139772.2535424037,1124387.885261016,1109003.5169796236,1093619.1486982312,1078234.7804168435,1062850.4121354497,1047466.0438540569,1032081.6755726705,1016697.3072912768,1001312.9390098834,985928.5707284925,970544.2024471024,955159.8341657114,939775.4658843172,924391.0976029267,909006.7293215333,893622.3610401461,878237.9927587495,862853.6244773543,847469.2561959676,832085.9826307236,816704.451905123,801324.1780525539,785945.2361866096,770567.3907735469,755190.8086413369,739814.8918942753,724440.458431921,709066.0343697397,693693.5551829161,678321.0759960879,662949.5407581506,647578.8368365113,632208.1329148784,616838.8283635881,601469.7348938249,586100.6414240552,570732.9937301045,555365.359000056,539997.7242699964,524631.2348249005,509264.91921183467,493898.6035987688,478532.84083058126,463167.71586712915,447802.59090368263,432437.4659402231,417073.13042221125,401709.0779340053,386345.02544579655,370980.97295759246,355617.5284760222,340254.43978550844,324891.3510949956,309528.26240450237,294165.24611730315,278803.02130877227,263440.79650024325,248078.5716917012,232716.34688316286,217354.13330344763,201992.68054440338,186631.227785361,171269.7750263177,155908.32226727903,140546.8695082292],[1662352.1691770349,1646874.0280398754,1631395.8869027179,1615917.7457655605,1600439.6046284037,1584961.4634912463,1569483.322354091,1554005.1812169333,1538527.040079772,1523048.8989426168,1507570.7578054592,1492092.616668303,1476614.4755311445,1461136.3343939884,1445658.193256832,1430180.0521196732,1414701.9109825145,1399223.7698453574,1383745.6287082022,1368267.4875710434,1352789.3464338852,1337311.205296729,1321833.0641595686,1306354.9230224118,1290876.7818852558,1275398.6407480978,1259920.499610939,1244442.3584737824,1228964.217336625,1213486.0761994685,1198007.9350623116,1182529.7939251545,1167051.6527879974,1151573.5116508412,1136095.3705136827,1120617.2293765247,1105139.0882393625,1089660.9471022035,1074182.8059650483,1058704.6648278902,1043226.5236907359,1027748.3825535779,1012270.2414164236,996792.100279267,981313.959142107,965835.818004949,950357.6768677868,934879.5357306339,919401.3945934735,903923.2534563169,888445.1123191598,872966.971182005,857488.830044847,842010.6889076941,826532.5477705318,811054.4066333808,795578.2106494112,780102.828247034,764629.3347931057,749156.4693376208,733685.1453725463,718214.5741026364,702744.9540789444,687276.4717936171,671808.1503362712,656341.568041049,640874.9857458407,625409.3370178249,609944.4806748759,594479.6243319223,579016.0237371614,563552.7331113042,548089.4424854359,532627.3459814703,517165.4735747874,501703.601168097,486242.53655297216,470781.94662112277,455321.3566892948,439860.91575625446,424401.4834004333,408942.0510445973,393482.61868879106,378023.4928509332,362565.1031793868,347106.7135078097,331648.3238362605,316190.00109191425,300732.54844591115,285275.0957998745,269817.643153863,254360.19050782733,238903.06917712558,223446.4564163573,207989.84365558624,192533.2308948217,177076.6181340497,161620.14076210558,146164.27860499732,130708.41644791886],[1661863.5632443589,1646291.649251433,1630719.7352585108,1615147.8212655885,1599575.9072726662,1584003.9932797397,1568432.0792868184,1552860.1652938933,1537288.251300971,1521716.3373080464,1506144.4233151237,1490572.5093221995,1475000.595329275,1459428.6813363521,1443856.7673434298,1428284.8533505038,1412712.939357582,1397141.0253646586,1381569.1113717349,1365997.1973788121,1350425.2833858856,1334853.369392963,1319281.4554000404,1303709.5414071183,1288137.6274141923,1272565.7134212705,1256993.7994283477,1241421.885435422,1225849.9714424976,1210278.057449575,1194706.1434566535,1179134.2294637284,1163562.3154708035,1147990.401477879,1132418.4874849543,1116846.573492032,1101274.6594991111,1085702.7455061902,1070130.8315132656,1054558.9175203443,1038987.0035274187,1023415.0895344955,1007843.1755415667,992271.2615486402,976699.3475557189,961127.4335627975,945555.5195698729,929983.6055769515,914411.691584026,898839.7775911023,883267.8635981772,867695.9496052563,852124.0356123345,836552.1216194117,820980.2076264909,805408.2936335569,789836.37964065,774264.4656477207,758695.0319610536,743125.7978819227,727558.7971755369,711992.0081305392,696426.9434207827,680862.3830131898,665298.8209717525,649736.2898634663,634173.8538692119,618613.1684656041,603052.4830619935,587492.5235922448,571933.5148279788,556374.5060637146,540816.4304863578,525258.9427044187,509701.4549224684,494144.74752629735,478588.63744296227,463032.5273596309,447476.7410059413,431921.876755327,416367.01250471827,400812.1482540993,385258.02136424,369704.281616034,354150.54186783545,338596.80211963877,323043.779691346,307491.05283643026,291938.32598151267,276385.5991265746,260833.20128422882,245281.38468269072,229729.56808115635,214177.75147963222,198625.93487809505,183074.56452354882,167523.5638105385,151972.5630975403,136421.56238454022,120870.56167153269],[1661374.957311682,1645709.2704629914,1630043.5836143035,1614377.8967656156,1598712.2099169225,1583046.5230682339,1567380.836219544,1551715.1493708561,1536049.462522165,1520383.775673476,1504718.0888247872,1489052.4019760957,1473386.7151274062,1457721.0282787168,1442055.3414300275,1426389.6545813372,1410723.9677326472,1395058.280883959,1379392.5940352695,1363726.9071865797,1348061.2203378885,1332395.5334891998,1316729.8466405093,1301064.159791822,1285398.4729431302,1269732.7860944404,1254067.0992457522,1238401.4123970624,1222735.725548371,1207070.038699683,1191404.351850993,1175738.6650023027,1160072.978153611,1144407.291304924,1128741.6044562324,1113075.9176075482,1097410.230758857,1081744.5439101597,1066078.8570614732,1050413.1702127857,1034747.4833640922,1019081.796515408,1003416.1096667103,987750.4228180279,972084.7359693395,956419.0491206441,940753.3622719599,925087.6754232673,909421.988574578,893756.301725891,878090.6148771993,862424.9280285137,846759.2411798234,831093.5543311252,815427.8674824364,799762.1806337498,784096.4937850544,768430.8069363758,752765.1200876785,737099.5323215369,721436.4465656485,705773.3608097602,690112.5651998539,674451.852565323,658792.7849182757,643134.2353729885,627476.4925727248,611819.9126414806,596163.3327102428,580508.3564565843,564853.5679445956,549199.1004814003,533545.9392958106,517892.77811023686,502240.04861120787,486588.3636731813,470936.67873514444,455285.1983645782,439634.85060461145,423984.50284465216,408334.1550846677,392684.70961442497,377035.5710450364,361386.43247564044,345737.3297585454,330089.2826179527,314441.23547739163,298793.1883368036,283145.1411962267,267498.0434316145,251850.9793933276,236203.91535507515,220556.8513167994,204910.21774347685,189264.03718643263,173617.85662937444,157971.67607233673,142325.4955152925,126679.73215655982,111034.34349132888],[1660886.3513790062,1645126.8916745507,1629367.4319700967,1613607.9722656384,1597848.512561184,1582089.052856727,1566329.5931522711,1550570.1334478152,1534810.6737433616,1519051.2140389043,1503291.7543344502,1487532.294629994,1471772.8349255372,1456013.3752210815,1440253.9155166256,1424494.4558121702,1408734.996107715,1392975.5364032593,1377216.0766988029,1361456.6169943481,1345697.1572898915,1329937.6975854374,1314178.2378809787,1298418.7781765258,1282659.3184720697,1266899.858767613,1251140.399063156,1235380.9393587024,1219621.4796542434,1203862.019949792,1188102.5602453353,1172343.1005408785,1156583.6408364233,1140824.1811319687,1125064.721427516,1109305.2617230495,1093545.8020185963,1077786.3423141432,1062026.8826096905,1046267.4229052309,1030507.9632007717,1014748.5034963181,998989.0437918655,983229.5840874072,967470.1243829527,951710.6646785,935951.2049740362,920191.7452695873,904432.2855651276,888672.8258606754,872913.3661562265,857153.9064517603,841394.4467473021,825634.9870428499,809875.5273383968,794116.0676339334,778356.6079294784,762597.1482250253,746837.6885205712,731078.2288161153,715318.7691116603,699559.3918958567,683802.4544632034,668045.5170305679,652290.6388660632,636536.0026419954,620782.6698650001,605030.1311820447,589277.9688818567,573527.3401276572,557776.7113734372,542027.1320140054,526278.2403936116,510529.3487732215,494781.7540783975,479034.44047151506,463287.12686461397,447540.99601758085,431795.1139234593,416049.2318293229,400304.11305969674,384559.5276230974,368814.942186499,353070.44519843254,337327.0323102707,321583.61942210514,305840.20653393306,290097.05405037757,274354.6995174233,258612.34498447366,242869.99045151845,227127.685293369,211386.2840717528,195644.8828501273,179903.48162850272,164162.0804068679,148421.05046964623,132680.5059570819,116939.96144452319,101199.41693195887],[1660397.745446334,1644544.512886109,1628691.2803258887,1612838.047765665,1596984.8152054427,1581131.5826452216,1565278.3500849998,1549425.1175247775,1533571.8849645555,1517718.6524043325,1501865.419844112,1486012.1872838894,1470158.9547236688,1454305.7221634462,1438452.4896032244,1422599.257043003,1406746.0244827813,1390892.7919225586,1375039.5593623365,1359186.326802116,1343333.0942418934,1327479.8616816718,1311626.6291214502,1295773.3965612277,1279920.1640010057,1264066.9314407841,1248213.698880561,1232360.4663203412,1216507.2337601169,1200654.0011998976,1184800.7686396737,1168947.5360794538,1153094.3035192331,1137241.0709590074,1121387.838398783,1105534.6058385675,1089681.373278346,1073828.1407181174,1057974.9081578972,1042121.6755976779,1026268.4430374596,1010415.2104772362,994561.9779170114,978708.7453567917,962855.5127965668,947002.2802363513,931149.0476761241,915295.815115903,899442.5825556796,883589.3499954599,867736.1174352351,851882.8848750135,836029.652314797,820176.4197545755,804323.1871943492,788469.9546341365,772616.7220739052,756763.4895136775,740910.2569534723,725057.0243932381,709203.7918330161,693350.5592727968,677497.326712573,661644.094152349,645793.0556537304,629942.2665443141,614093.0181741416,598244.4583605584,582396.5982609885,566550.070440351,550703.5426196977,534858.5723219691,519013.8947448032,503169.4951378405,487326.50040905643,471483.5056802705,455640.95917572547,439799.49314751197,423958.02711930405,408116.8397376314,392276.76048740465,376436.6812372068,360596.6019869847,344757.60169498995,328918.77858177386,313079.9554685317,297241.3963003913,281403.7090934431,265566.0218864912,249728.3346795682,233890.93398797233,218054.27206264157,202217.61013732478,186380.94821199868,170544.28628667258,154708.50161177386,138872.76320677064,123037.02480180189,107201.286396835,91365.69946272299],[1659909.1395136567,1643962.1340976665,1628015.1286816774,1612068.1232656925,1596121.1178497015,1580174.1124337143,1564227.1070177278,1548280.101601738,1532333.0961857499,1516386.0907697608,1500439.085353775,1484492.0799377873,1468545.074521799,1452598.0691058105,1436651.063689823,1420704.0582738356,1404757.0528578456,1388810.0474418586,1372863.0420258697,1356916.0366098813,1340969.0311938955,1325022.0257779062,1309075.020361919,1293128.014945932,1277181.0095299445,1261234.0041139547,1245286.9986979668,1229339.9932819805,1213392.98786599,1197445.9824500056,1181498.977034015,1165551.971618028,1149604.9662020397,1133657.9607860474,1117710.9553700648,1101763.9499540776,1085816.9445380834,1069869.9391221032,1053922.933706116,1037975.9282901194,1022028.9228741359,1006081.9174581505,990134.9120421619,974187.9066261677,958240.901210187,942293.8957941979,926346.890378207,910399.8849622142,894452.8795462348,878505.8741302504,862558.8687142516,846611.8632982676,830664.8578822785,814717.8524662852,798770.8470503055,782823.841634321,766876.836218331,750929.8308023484,734982.8253863407,719035.81997036,703088.8145543765,687141.8091383874,671194.803722404,655247.7983064102,639300.7928904286,623353.7874744292,607408.2501371959,591463.6093510352,575519.7031241115,559577.2197210044,543634.736317886,527694.0531478953,511753.536189558,495813.60922444053,479874.88282431755,463936.15642418154,447998.3479909329,432061.250153753,416124.15231658705,400187.9361382611,384252.3176887352,368316.6992392037,352381.61842702515,336447.3420207156,320513.06561442465,304578.7892081188,288645.6642704904,272712.60348060913,256779.54269074276,240846.70148968883,224914.73996397853,208982.77843826637,193050.81691253744,177118.96957131103,161188.00025360845,145257.03093591426,129326.0616182182,113395.09230051376,97464.69605164975,81534.6204633303],[1659420.5335809803,1643379.755309225,1627338.9770374694,1611298.198765714,1595257.4204939643,1579216.6422222103,1563175.8639504528,1547135.0856787008,1531094.3074069442,1515053.5291351918,1499012.750863437,1482971.972591684,1466931.1943199295,1450890.4160481761,1434849.6377764218,1418808.8595046652,1402768.0812329138,1386727.3029611588,1370686.524689405,1354645.74641765,1338604.9681458976,1322564.189874143,1306523.4116023874,1290482.633330634,1274441.855058879,1258401.0767871272,1242360.2985153743,1226319.5202436184,1210278.7419718632,1194237.9637001099,1178197.1854283575,1162156.4071566039,1146115.6288848491,1130074.8506130935,1114034.0723413434,1097993.2940695845,1081952.515797833,1065911.7375260754,1049870.9592543235,1033830.1809825711,1017789.4027108168,1001748.6244390612,985707.846167306,969667.0678955531,953626.2896237997,937585.5113520403,921544.733080287,905503.9548085365,889463.1765367831,873422.3982650293,857381.6199932694,841340.8417215231,825300.0634497674,809259.2851780117,793218.5069062607,777177.7286345009,761136.9503627466,745096.1720909942,729055.3938192436,713014.6155474801,696973.8372757407,680933.0590039752,664892.2807322182,648851.5024604835,632810.7241887199,616769.9459169563,600729.1676452002,584688.3893734561,568648.0379136298,552609.5454507126,536571.0529877916,520534.28672333155,504497.87973068655,488462.0793046877,472427.5732086655,456393.0671126433,440359.6756119691,424326.900388889,408294.12516580615,392262.58219365496,376231.38124809135,360200.18030252773,344169.9985729102,328140.2277020719,312110.4568312485,296081.21417987626,280052.7406174764,264024.2670551054,247995.79349270556,231968.30078615993,215941.00231966376,199913.70385314617,183886.40538663045,167860.12492185459,151833.889077371,135807.65323286224,119781.41738837957,103755.88409032486,87730.60738022253,71705.33067015931],[1658931.9276483045,1642797.3765207827,1626662.8253932623,1610528.274265742,1594393.7231382204,1578259.172010703,1562124.6208831812,1545990.06975566,1529855.518628139,1513720.9675006217,1497586.4163731001,1481451.8652455807,1465317.3141180612,1449182.76299054,1433048.2118630197,1416913.660735499,1400779.1096079794,1384644.5584804588,1368510.007352938,1352375.4562254192,1336240.9050978997,1320106.3539703805,1303971.802842859,1287837.251715338,1271702.7005878175,1255568.1494602982,1239433.5983327774,1223299.0472052577,1207164.4960777354,1191029.944950215,1174895.3938226956,1158760.8426951752,1142626.291567653,1126491.7404401386,1110357.189312613,1094222.6381850974,1078088.087057571,1061953.5359300561,1045818.9848025409,1029684.4336750163,1013549.8825474987,997415.3314199713,981280.7802924565,965146.2291649347,949011.6780374143,932877.1269098949,916742.5757823698,900608.0246548494,884473.4735273393,868338.9223998087,852204.3712722939,836069.8201447725,819935.2690172526,803800.7178897331,787666.1667622151,771531.6156346835,755397.0645071641,739262.513379652,723127.9622521382,706993.4111246094,690858.8599970834,674724.3088695705,658589.7577420482,642455.2066145353,626320.6554870084,610186.1043594852,594051.5532319685,577917.0021044556,561782.4509769278,545647.8998494092,529513.3487218833,513380.0748433443,497247.73070366494,481115.65936754644,464985.328785385,448854.99820321053,432725.6536770193,416597.15844329353,400468.6632095929,384341.4490617532,368214.6250156928,352087.8009696463,335962.1977460347,319836.89369207155,303711.5896381093,287587.1464796681,271463.2231875118,255339.29989536386,239215.62699620426,223092.95627772622,206970.28555923328,190847.61484075617,174725.51124201715,158603.9750988772,142482.43895573728,126360.90281256288,110239.86396710202,94119.35380382277,77998.84364054166,61878.33347729221],[1658443.3217156287,1642214.9977323408,1625986.6737490566,1609758.3497657687,1593530.0257824822,1577301.7017991967,1561073.3778159088,1544845.0538326234,1528616.7298493362,1512388.4058660497,1496160.0818827646,1479931.7578994776,1463703.4339161925,1447475.1099329044,1431246.785949618,1415018.4619663323,1398790.1379830453,1382561.8139997595,1366333.4900164737,1350105.1660331876,1333876.842049902,1317648.5180666142,1301420.194083328,1285191.8701000414,1268963.5461167544,1252735.222133468,1236506.8981501823,1220278.5741668951,1204050.2501836084,1187821.9262003212,1171593.6022170386,1155365.2782337535,1139136.954250466,1122908.6302671814,1106680.3062838935,1090451.9823006056,1074223.6583173177,1057995.334334041,1041767.0103507456,1025538.6863674661,1009310.3623841736,993082.0384008931,976853.7144176001,960625.3904343182,944397.0664510257,928168.7424677406,911940.4184844629,895712.0945011713,879483.7705178862,863255.4465345959,847027.1225513173,830798.7985680238,814570.4745847415,798342.1506014499,782113.8266181601,765885.5026348904,749657.1786515927,733428.8546683034,717200.5306850243,700972.2067017434,684743.8827184476,668515.5587351527,652287.2347518727,636058.9107685955,619830.5867853062,603602.2628020104,587373.9388187323,571145.614835456,554917.2908521565,538688.9668688793,522460.6428855844,506232.3189022979,490003.9949190095,473775.6709357314,457549.0017124992,441322.8058960531,425097.08348195255,408872.8293102598,392648.57513854466,376425.293223219,360202.80885183066,343980.3244804293,327758.9295737343,311538.05670472886,295317.18383572716,279097.1946480498,262877.787485688,246658.3803233467,230439.3798585441,214221.30414507538,198003.22843160015,181785.15271814354,165567.98900142778,149351.12112685293,133134.25325228367,116917.38537771441,100701.52181825787,84485.74799847789,68269.97417869978,52054.200358895585],[1657954.7157829523,1641632.6189439,1625310.5221048477,1608988.4252657928,1592666.3284267401,1576344.2315876894,1560022.1347486372,1543700.0379095841,1527377.941070532,1511055.8442314814,1494733.747392429,1478411.6505533736,1462089.553714322,1445767.4568752693,1429445.3600362164,1413123.2631971645,1396801.1663581128,1380479.06951906,1364156.9726800073,1347834.875840955,1331512.7790019028,1315190.6821628485,1298868.5853237968,1282546.4884847444,1266224.391645693,1249902.2948066415,1233580.1979675887,1217258.1011285342,1200936.004289483,1184613.9074504294,1168291.8106113812,1151969.713772324,1135647.6169332778,1119325.5200942196,1103003.423255174,1086681.3264161176,1070359.22957707,1054037.132738011,1037715.0358989658,1021392.9390599076,1005070.8422208559,988748.7453818037,972426.6485427539,956104.5517036999,939782.4548646472,923460.3580255979,907138.2611865439,890816.1643474875,874494.0675084367,858171.9706693804,841849.8738303347,825527.7769912756,809205.6801522304,792883.5833131731,776561.4864741173,760239.3896350674,743917.2927960269,727595.1959569575,711273.0991178993,694951.0022788625,678628.9054398155,662306.8086007433,645984.7117616953,629662.6149226502,613340.518083591,597018.4212445328,580696.3244054997,564374.2275664378,548052.1307273814,531730.0338883344,515407.93704928644,499085.84021022543,482763.743371184,466441.64653212857,450119.5496930769,433797.4528540233,417475.3560149707,401154.81852108426,384834.7710278742,368514.9660591865,352196.7882979596,335878.61053672805,319560.9979432924,303244.5244342098,286928.05092513654,270612.1171479821,254297.1954559991,237982.27376403287,221667.57289971504,205354.06262895092,189040.55235819798,172727.0420874562,156414.47057474498,140102.2424399564,123790.01430517156,107477.83878858946,91166.77375792712,74855.7087272685,58544.64369660616,42233.642477855086],[1657466.1098502763,1641050.2401554578,1624634.3704606348,1608218.5007658168,1591802.6310709994,1575386.7613761823,1558970.891681362,1542555.0219865446,1526139.1522917273,1509723.2825969094,1493307.412902087,1476891.5432072703,1460475.6735124525,1444059.8038176342,1427643.934122815,1411228.064427998,1394812.194733177,1378396.3250383595,1361980.4553435417,1345564.5856487232,1329148.7159539042,1312732.8462590862,1296316.9765642679,1279901.1068694484,1263485.2371746302,1247069.3674798133,1230653.4977849962,1214237.6280901765,1197821.758395354,1181405.8887005378,1164990.019005722,1148574.1493108994,1132158.2796160826,1115742.4099212675,1099326.5402264409,1082910.6705316254,1066494.8008368057,1050078.9311419912,1033663.0614471715,1017247.191752357,1000831.3220575377,984415.4523627148,967999.5826678951,951583.7129730792,935167.8432782604,918751.9735834445,902336.1038886262,885920.2341938051,869504.364498985,853088.4948041663,836672.6251093522,820256.7554145302,803840.8857197096,787425.0160248983,771009.1463300763,754593.2766352538,738177.4069404397,721761.53724562,705345.667550805,688929.7978559807,672513.9281611564,656098.0584663404,639682.1887715356,623266.3190767113,606850.4493818823,590434.5796870701,574018.7099922439,557602.8402974391,541186.9706026213,524771.1009077951,508355.23121298105,491939.3615181623,475523.49182333704,459107.6221285267,442691.7524337033,426275.88273888547,409860.0130440695,393444.1433492331,377028.2736544339,360612.4039596189,344197.5252691135,327783.626099146,311369.7269291729,294957.2057484947,278545.10439774115,262133.00304697547,245722.3051904766,229311.8425437035,212901.37989693973,196492.04126951005,180083.07075456623,163674.10023963638,147265.71912185382,130858.10574272648,114450.49236357585,98042.87898442522,81636.3419204168,65229.96136431396,48823.58080821112,32417.243452453986],[1656977.5039175996,1640467.8613670107,1623958.2188164277,1607448.5762658457,1590938.9337152592,1574429.291164676,1557919.648614091,1541410.0060635041,1524900.363512922,1508390.7209623384,1491881.0784117507,1475371.4358611673,1458861.7933105822,1442352.1507599987,1425842.508209414,1409332.8656588292,1392823.2231082455,1376313.58055766,1359803.9380070749,1343294.2954564912,1326784.6529059054,1310275.0103553222,1293765.3678047373,1277255.7252541522,1260746.0827035683,1244236.4401529834,1227726.7976023997,1211217.1550518167,1194707.5125012281,1178197.8699506428,1161688.2274000589,1145178.5848494759,1128668.9422988882,1112159.2997483108,1095649.6571977194,1079140.0146471323,1062630.3720965548,1046120.7295459686,1029611.0869953879,1013101.4444448012,996591.8018942121,980082.1593436287,963572.516793048,947062.874242459,930553.231691875,914043.5891412892,897533.9465907072,881024.3040401195,864514.661489536,848005.018938953,831495.3763883659,814985.7338377857,798476.091287198,781966.4487366118,765456.8061860306,748947.163635456,732437.5210848553,715927.8785342751,699418.2359836996,682908.5934331054,666398.950882527,649889.3083319422,633379.6657813406,616870.023230765,600360.3806801904,583850.7381295906,567341.095579017,550831.4530284293,534321.810477851,517812.1679272689,501302.52537667844,484792.8828260908,468283.2402755087,451773.597724922,435263.95517434645,418754.3126237476,402244.6700731646,385735.02752258163,369225.38497201167,352715.7424214035,336206.09987083357,319696.4573202608,303186.8147696508,286677.1722190846,270169.37110984325,253661.62026311923,237154.08166184835,220648.05672156904,204142.0317812711,187636.15112061612,171131.69933615252,154627.24755168986,138122.79576722998,121619.57480733842,105116.55546944495,88613.5361315608,72111.04047687538,55609.32398931682,39107.6075018011,22605.891014257446],[1656488.8979849233,1639885.482578569,1623282.0671722202,1606678.6517658709,1590075.236359518,1573471.8209531694,1556868.4055468184,1540264.9901404672,1523661.5747341162,1507058.1593277678,1490454.7439214136,1473851.3285150651,1457247.913108714,1440644.4977023646,1424041.0822960127,1407437.6668896617,1390834.2514833102,1374230.8360769595,1357627.4206706109,1341024.0052642599,1324420.5898579084,1307817.1744515589,1291213.7590452055,1274610.3436388571,1258006.9282325066,1241403.5128261556,1224800.0974198037,1208196.6820134562,1191593.2666071008,1174989.8512007496,1158386.4357944024,1141783.0203880533,1125179.6049816976,1108576.1895753504,1091972.7741690003,1075369.3587626466,1058765.9433562956,1042162.5279499479,1025559.1125435946,1008955.6971372413,992352.2817308903,975748.8663245435,959145.4509181925,942542.0355118429,925938.6201054938,909335.2046991405,892731.7892927905,876128.3738864376,859524.9584800871,842921.5430737385,826318.1276673847,809714.7122610351,793111.2968546911,776507.8814483369,759904.4660419887,743301.0506356359,726697.6352292793,710094.2198229302,693490.8044165745,676887.389010231,660283.9736038791,643680.5581975402,627077.1427911837,610473.7273848327,593870.311978464,577266.8965721317,560663.4811657742,544060.0657594213,527456.6503530778,510853.23494671937,494249.8195403684,477646.4041340314,461042.98872766364,444439.5733213164,427836.1579149645,411232.7425086256,394629.3271022737,378025.9116959227,361422.4962895671,344819.080883218,328215.66547686514,311612.2500705011,295008.8346641697,278405.41925782803,261802.00385146774,245198.5884451149,228595.17303876393,211992.00605998654,195390.40353649203,178788.80101301614,162187.46750823595,145587.5189784132,128987.57044860069,112387.62191878445,95789.18038032018,79190.73945816234,62592.298536023125,45994.71776146628,29397.649600625038,12800.58143977262],[1656000.2920522473,1639303.1037901284,1622605.9155280136,1605908.7272658944,1589211.539003781,1572514.3507416616,1555817.1624795478,1539119.9742174293,1522422.785955311,1505725.5976931965,1489028.409431077,1472331.2211689618,1455634.032906844,1438936.8446447277,1422239.6563826115,1405542.468120494,1388845.2798583778,1372148.0915962607,1355450.9033341436,1338753.7150720283,1322056.5268099103,1305359.3385477914,1288662.1502856775,1271964.962023561,1255267.773761445,1238570.585499327,1221873.3972372075,1205176.2089750944,1188479.0207129759,1171781.832450857,1155084.6441887436,1138387.4559266288,1121690.2676645084,1104993.079402389,1088295.8911402705,1071598.7028781623,1054901.514616041,1038204.3263539211,1021507.1380918045,1004809.9498296948,988112.7615675735,971415.5733054541,954718.3850433403,938021.1967812232,921324.0085191047,904626.8202569876,887929.6319948751,871232.4437327506,854535.2554706419,837838.0672085257,821140.8789464063,804443.6906842864,787746.5024221684,771049.3141600611,754352.12589793,737654.9376358213,720957.7493737154,704260.5611115806,687563.3728494672,670866.1845873604,654168.9963252349,637471.8080631224,620774.6198010053,604077.4315388901,587380.2432767767,570683.0550146587,553985.8667525388,537288.6784904189,520591.4902282916,503894.3019661857,487197.113704063,470499.9254419459,453802.7371798372,437105.548917721,420408.36065560766,403711.17239348684,387013.9841313707,370316.7958692508,353619.60760712996,336922.41934501193,320225.231082893,303528.04282079265,286830.85455866717,270133.6662965454,253436.4780344423,236739.28977231216,220042.10151020065,203344.913248091,186647.72498596087,169950.53672385216,153253.34846173227,136556.16019961052,119860.62254908495,103165.16834883206,86469.71414859407,69775.59251904115,53081.72039969079,36387.84828032553,19694.748322984204,3002.3182631470263],[1655511.6861195713,1638720.725001686,1621929.7638838075,1605138.8027659215,1588347.8416480399,1571556.8805301576,1554765.9194122716,1537974.9582943895,1521183.9971765066,1504393.0360586257,1487602.0749407394,1470811.1138228602,1454020.1527049749,1437229.1915870928,1420438.2304692094,1403647.2693513262,1386856.3082334443,1370065.3471155607,1353274.3859976784,1336483.4248797966,1319692.4637619113,1302901.5026440285,1286110.5415261465,1269319.5804082649,1252528.6192903821,1235737.6581724973,1218946.6970546145,1202155.7359367316,1185364.7748188502,1168573.8137009637,1151782.8525830796,1134991.8914651996,1118200.9303473206,1101409.9692294374,1084619.008111551,1067828.0469936668,1051037.0858757873,1034246.1247579074,1017455.163640019,1000664.2025221381,983873.2414042517,967082.2802863694,950291.3191684922,933500.3580506081,916709.3969327188,899918.4358148389,883127.474696957,866336.5135790706,849545.5524611971,832754.5913433079,815963.6302254265,799172.6691075414,782381.7079896592,765590.7468717704,748799.7857538881,732008.824636003,715217.8635181254,698426.9024002533,681635.9412823664,664844.9801644823,648054.0190466037,631263.0579287065,614472.0968108298,597681.1356929466,580890.1745750625,564099.2134571867,547308.2523393026,530517.2912214054,513726.3301035296,496935.36898565385,480144.4078677725,463353.44674987905,446562.48563200235,429771.52451411635,412980.56339623965,396189.6022783555,379398.64116046764,362607.6800425844,345816.7189247012,329025.75780681707,312234.79668893013,295443.835571059,278652.87445316557,261861.91333529446,245070.95221741218,228279.99109951593,211489.02998163924,194698.0688637523,177907.1077458663,161116.14662798494,144325.18551009614,127534.22439222131,110743.26327434275,93952.30215647165,77161.34103858098,60370.37992068194,43580.02814759128,26790.722270576283,10001.41639356874,-6787.722656559199],[1655023.080186896,1638138.3462132458,1621253.6122395948,1604368.878265949,1587484.1442922996,1570599.410318649,1553714.676344999,1536829.9423713512,1519945.2083977014,1503060.474424054,1486175.740450404,1469291.0064767557,1452406.272503105,1435521.5385294564,1418636.8045558075,1401752.070582161,1384867.3366085088,1367982.6026348618,1351097.8686612123,1334213.1346875639,1317328.4007139138,1300443.6667402636,1283558.9327666152,1266674.1987929677,1249789.4648193172,1232904.730845669,1216019.9968720202,1199135.2628983692,1182250.5289247222,1165365.794951072,1148481.0609774236,1131596.327003776,1114711.5930301263,1097826.8590564816,1080942.12508283,1064057.3911091816,1047172.6571355285,1030287.9231618778,1013403.1891882289,996518.4552145838,979633.7212409358,962748.9872672898,945864.2532936349,928979.519319986,912094.7853463348,895210.0513726887,878325.3173990366,861440.5834253887,844555.8494517342,827671.1154780928,810786.3815044407,793901.6475307955,777016.9135571411,760132.1795834927,743247.4456098452,726362.7116362043,709477.97766254,692593.243688901,675708.5097152432,658823.7757416042,641939.0417679474,625054.3077943008,608169.5738206571,591284.8398469985,574400.1058733603,557515.3718997138,540630.637926056,523745.903952403,506861.16997876484,489976.43600511085,473091.702031455,456206.9680578178,439322.2340841703,422437.5001105191,405552.76613686886,388668.0321632158,371783.2981895646,354898.5642159255,338013.8302422613,321129.09626862314,304244.3622949766,287359.62832131516,270474.8943476649,253590.16037401836,236705.42640038393,219820.69242671598,202935.95845308248,186051.2244794229,169166.49050578196,152281.756532141,135397.0225584805,118512.28858483583,101627.55461118091,84742.82063754741,67858.08666389249,50973.352690238506,34088.618716588244,17203.88474295847,319.15076928399503,-16565.58320436068],[1654534.4742542193,1637555.967424803,1620577.4605953873,1603598.95376597,1586620.446936558,1569641.9401071444,1552663.4332777285,1535684.9264483117,1518706.4196188976,1501727.9127894822,1484749.4059600662,1467770.8991306506,1450792.3923012384,1433813.8854718215,1416835.3786424058,1399856.8718129932,1382878.3649835761,1365899.8581541618,1348921.3513247452,1331942.8444953302,1314964.3376659157,1297985.830836502,1281007.3240070855,1264028.8171776708,1247050.3103482546,1230071.803518841,1213093.2966894247,1196114.789860011,1179136.283030596,1162157.7762011783,1145179.2693717633,1128200.762542347,1111222.2557129315,1094243.7488835156,1077265.2420541025,1060286.7352246935,1043308.2283952776,1026329.7215658631,1009351.2147364467,992372.7079070313,975394.201077613,958415.6942481971,941437.1874187822,924458.6805893653,907480.1737599522,890501.6669305395,873523.1601011213,856544.6532717082,839566.1464422904,822587.6396128726,805609.1327834628,788630.6259540487,771652.1191246384,754673.6122952197,737695.1054657977,720716.5986363823,703738.0918069705,686759.5849775542,669781.0781481406,652802.571318714,635824.0644893032,618845.5576598924,601867.0508304816,584888.5440010661,567910.0371716414,550931.5303422399,533953.023512817,516974.51668340154,499996.00985399075,483017.50302457716,466038.99619515333,449060.48936574254,432081.98253632244,415103.4757069051,398124.96887748875,381146.46204808913,364167.9552186774,347189.4483892666,330210.94155984744,313232.43473043013,296253.9279010091,279275.42107159086,262296.91424218286,245318.4074127674,228339.90058334917,211361.3937539328,194382.88692452107,177404.38009510655,160425.87326569576,143447.36643627286,126468.85960685555,109490.35277744476,92511.84594803676,75533.33911860641,58554.832289192826,41576.325459796935,24597.818630378693,7619.311800967902,-9359.195028452203,-26337.701857864857],[1654045.8683215436,1636973.588636361,1619901.30895118,1602829.0292659993,1585756.7495808166,1568684.469895634,1551612.190210457,1534539.910525275,1517467.6308400927,1500395.3511549109,1483323.0714697305,1466250.791784548,1449178.5120993657,1432106.2324141862,1415033.9527290044,1397961.6730438222,1380889.3933586415,1363817.1136734607,1346744.8339882796,1329672.5543030978,1312600.2746179174,1295527.994932736,1278455.7152475575,1261383.4355623738,1244311.155877193,1227238.8761920112,1210166.5965068291,1193094.3168216506,1176022.037136468,1158949.757451287,1141877.477766106,1124805.1980809234,1107732.9183957423,1090660.6387105621,1073588.3590253792,1056516.0793401967,1039443.7996550137,1022371.5199698354,1005299.2402846552,988226.9605994704,971154.6809142898,954082.4012291157,937010.1215439318,919937.8418587502,902865.5621735663,885793.2824883847,868721.002803205,851648.7231180212,834576.4434328438,817504.1637476604,800431.8840624811,783359.604377301,766287.3246921161,749215.0450069346,732142.7653217651,715070.4856365705,697998.2059513945,680925.9262662055,663853.646581036,646781.3668958396,629709.0872106785,612636.8075254792,595564.5278403107,578492.2481551189,561419.9684699522,544347.6887847641,527275.4090995751,510203.1294144001,493130.8497292213,476058.5700440388,458986.2903588619,441914.01067366265,424841.730988496,407769.45130330883,390697.17161812726,373624.89193295036,356552.6122477744,339480.33256259095,322408.05287740193,305335.77319222596,288263.4935070509,271191.21382185444,254118.93413668312,237046.65445149038,219974.37476633023,202902.09508113656,185829.81539596058,168757.5357107725,151685.25602559838,134612.97634041123,117540.69665523339,100468.41697004251,83396.13728487678,66323.85759968497,49251.57791450247,32179.29822931625,15107.018544137478,-1965.26114102453,-19037.540826205164,-36109.82051139884],[1653557.2623888678,1636391.2098479196,1619225.1573069724,1602059.1047660257,1584893.0522250785,1567726.9996841282,1550560.9471431812,1533394.8946022335,1516228.8420612859,1499062.78952034,1481896.7369793924,1464730.684438445,1447564.6318974993,1430398.5793565523,1413232.5268156026,1396066.474274656,1378900.4217337088,1361734.3691927618,1344568.3166518142,1327402.2641108665,1310236.2115699186,1293070.1590289737,1275904.1064880244,1258738.053947077,1241572.0014061308,1224405.9488651832,1207239.8963242362,1190073.8437832897,1172907.791242342,1155741.7387013924,1138575.686160447,1121409.6336195003,1104243.5810785526,1087077.5285376073,1069911.4759966577,1052745.4234557115,1035579.3709147666,1018413.3183738175,1001247.2658328698,984081.213291923,966915.1607509749,949749.1082100295,932583.0556690795,915417.0031281314,898250.9505871814,881084.8980462328,863918.845505286,846752.7929643393,829586.7404233953,812420.6878824467,795254.6353414962,778088.5828005518,760922.530259599,743756.477718656,726590.4251776999,709424.3726367624,692258.320095812,675092.2675548662,657926.215013924,640760.1624729717,623594.109932024,606428.057391081,589262.0048501352,572095.95230918,554929.8997682445,537763.8472272921,520597.7946863435,503431.7421453912,486265.6896044435,469099.63706350606,451933.58452254906,434767.53198160883,417601.47944066674,400435.42689972,383269.37435876764,366103.32181782555,348937.2692768751,331771.2167359274,314605.16419498343,297439.1116540311,280273.0591130825,263107.0065721385,245940.95403118804,228774.90149023756,211608.8489492992,194442.79640834592,177276.74386740662,160110.69132645335,142944.6387855122,125778.58624456171,108612.53370360844,91446.48116265517,74280.42862172145,57114.37608077377,39948.32353982143,22782.270998883992,5616.218457920477,-11549.834083011374,-28715.886623978615,-45881.93916491233],[1653068.656456191,1635808.8310594787,1618549.005662765,1601289.1802660506,1584029.354869335,1566769.529472623,1549509.7040759097,1532249.878679197,1514990.0532824807,1497730.2278857692,1480470.4024890559,1463210.5770923442,1445950.7516956306,1428690.9262989163,1411431.1009022014,1394171.2755054892,1376911.4501087754,1359651.6247120616,1342391.7993153487,1325131.9739186342,1307872.1485219216,1290612.3231252087,1273352.4977284954,1256092.6723317814,1238832.8469350683,1221573.0215383552,1204313.1961416425,1187053.3707449283,1169793.5453482154,1152533.719951501,1135273.89455479,1118014.069158075,1100754.243761363,1083494.418364645,1066234.592967933,1048974.7675712146,1031714.9421745031,1014455.1167777888,997195.2913810769,979935.4659843645,962675.6405876521,945415.8151909397,928155.989794225,910896.1643975149,893636.3390008016,876376.513604084,859116.6882073693,841856.8628106606,824597.0374139422,807337.212017233,790077.3866205197,772817.5612238054,755557.7358270846,738297.910430382,721038.085033657,703778.2596369581,686518.4342402294,669258.6088435343,651998.7834468065,634738.9580501048,617479.1326533817,600219.3072566679,582959.4818599569,565699.6564632496,548439.8310665293,531180.0056698238,513920.18027310725,496660.3548763897,479400.5294796787,462140.7040829584,444880.87868623994,427621.0532895243,410361.22789281886,393101.4024961125,375841.5770993866,358581.7517026821,341321.92630596645,324062.1009092564,306802.275512537,289542.45011583343,272282.6247191252,255022.79932239745,237762.97392569575,220503.1485289745,203243.32313226722,185983.49773554318,168723.6723388387,151463.84694212023,134204.02154542133,116944.19614869263,99684.37075198162,82424.54535526969,65164.719958549365,47904.89456184395,30645.069165123627,13385.243768399581,-3874.581628292799,-21134.40702500567,-38394.232421724126,-55654.05781846307],[1652580.0505235142,1635226.4522710368,1617872.8540185536,1600519.2557660753,1583165.657513597,1565812.0592611148,1548458.4610086372,1531104.862756158,1513751.264503678,1496397.6662511965,1479044.0679987199,1461690.4697462413,1444336.8714937603,1426983.2732412792,1409629.6749887997,1392276.076736322,1374922.478483841,1357568.8802313635,1340215.2819788824,1322861.6837264022,1305508.0854739242,1288154.4872214447,1270800.8889689646,1253447.2907164844,1236093.6924640066,1218740.0942115246,1201386.495959044,1184032.897706565,1166679.299454087,1149325.7012016084,1131972.1029491303,1114618.504696649,1097264.90644417,1079911.3081916869,1062557.7099392074,1045204.1116867289,1027850.5134342499,1010496.9151817728,993143.3169292947,975789.7186768125,958436.120424333,941082.5221718485,923728.9239193713,906375.3256668923,889021.7274144138,871668.1291619376,854314.5309094512,836960.9326569708,819607.3344044979,802253.7361520189,784900.1378995399,767546.5396470577,750192.9413945749,732839.343142096,715485.7448896207,698132.1466371398,680778.5483846627,663424.9501321809,646071.3518797047,628717.7536272276,611364.1553747449,594010.5571222724,576656.9588697888,559303.3606173005,541949.7623648271,524596.1641123444,507242.56585986447,489888.9676073836,472535.3693548972,455181.77110242285,437828.1728499355,420474.5745974714,403120.9763449868,385767.37809251156,368413.7798400307,351060.18158753496,333706.5833350625,316352.9850825779,298999.3868301045,281645.7885776209,264292.19032515027,246938.59207267407,229584.99382019602,212231.39556771703,194877.79731523525,177524.19906275813,160170.6008102782,142817.00255780295,125463.4043053044,108109.80605282076,90756.20780034829,73402.6095478693,56049.01129539404,38695.41304292157,21341.814790455624,3988.2165379561484,-13365.381714504212,-30718.979967001826,-48072.57821948826,-65426.176471948624],[1652091.4445908389,1634644.0734825947,1617196.702374347,1599749.331266102,1582301.9601578559,1564854.5890496084,1547407.2179413647,1529959.8468331196,1512512.4757248727,1495065.1046166273,1477617.7335083825,1460170.362400136,1442722.9912918902,1425275.6201836462,1407828.2490753978,1390380.8779671548,1372933.506858908,1355486.135750662,1338038.7646424165,1320591.3935341705,1303144.022425926,1285696.6513176805,1268249.2802094338,1250801.9091011893,1233354.5379929435,1215907.1668846963,1198459.7957764533,1181012.4246682045,1163565.0535599617,1146117.6824517166,1128670.3113434692,1111222.9402352227,1093775.5691269757,1076328.1980187315,1058880.8269104878,1041433.4558022446,1023986.0846939976,1006538.7135857446,989091.3424775028,971643.9713692591,954196.6002610154,936749.229152766,919301.8580445219,901854.4869362707,884407.1158280293,866959.7447197833,849512.3736115363,832065.0025032917,814617.6313950489,797170.2602868024,779722.8891785559,762275.5180703104,744828.1469620587,727380.775853809,709933.404745575,692486.0336373216,675038.6625290867,657591.2914208332,640143.9203125862,622696.5492043421,605249.1780960923,587801.8069878593,570354.435879603,552907.0647713616,535459.6936631175,518012.32255486865,500564.95144662354,483117.58033836074,465670.20923013054,448222.8381218873,430775.4670136487,413328.0959053943,395880.7247971473,378433.3536888985,360985.9825806599,343538.6114724139,326091.24036417063,308643.8692559209,291196.4981476711,273749.12703944184,256301.75593118556,238854.38482294139,221407.01371469814,203959.64260643348,186512.27149820887,169064.9003899591,151617.52928173076,134170.15817346983,116722.78706522193,99275.41595698055,81828.044848728,64380.67374049593,46933.30263223499,29485.931523988023,12038.560415742919,-5408.810692479834,-22856.181800743565,-40303.552908999845,-57750.92401723191,-75198.29512548633],[1651602.8386581624,1634061.69469415,1616520.5507301376,1598979.4067661255,1581438.262802115,1563897.118838102,1546355.9748740904,1528814.8309100792,1511273.6869460694,1493732.5429820553,1476191.3990180455,1458650.2550540308,1441109.1110900221,1423567.967126011,1406026.8231619964,1388485.679197985,1370944.535233974,1353403.3912699628,1335862.2473059492,1318321.1033419392,1300779.959377926,1283238.815413916,1265697.6714499043,1248156.527485891,1230615.3835218786,1213074.2395578665,1195533.095593858,1177991.951629843,1160450.8076658328,1142909.663701823,1125368.5197378104,1107827.375773795,1090286.2318097851,1072745.0878457772,1055203.9438817636,1037662.7999177473,1020121.6559537388,1002580.5119897267,985039.3680257201,967498.224061701,949957.0800976884,932415.9361336818,914874.7921696738,897333.6482056584,879792.5042416425,862251.3602776327,844710.2163136182,827169.072349607,809627.9283855995,792086.7844215874,774545.6404575687,757004.496493563,739463.3525295537,721922.2085655453,704381.064601534,686839.9206375182,669298.776673507,651757.6327094892,634216.4887454743,616675.3447814696,599134.200817462,581593.0568534434,564051.912889435,546510.7689254228,528969.6249614051,511428.48099739477,493887.3370333826,476346.1930693686,458805.0491053546,441263.9051413508,423722.7611773396,406181.6172133228,388640.47324931435,371099.3292853022,353558.18532130495,336017.0413572863,318475.8973932713,300934.7534292508,283393.6094652461,265852.46550122276,248311.3215372134,230770.17757320497,213229.03360920027,195687.88964518812,178146.74568118993,160605.60171717033,143064.45775315445,125523.3137891423,107982.16982512455,90441.02586111333,72899.88189710397,55358.737933089025,37817.59396908432,20276.4500050731,2735.3060410562903,-14805.837922966108,-32346.981886960566,-49888.1258509811,-67429.26981501095,-84970.41377900355],[1651114.2327254869,1633479.3159057065,1615844.3990859315,1598209.48226615,1580574.5654463754,1562939.6486265957,1545304.7318068189,1527669.814987041,1510034.8981672633,1492399.981347484,1474765.0645277095,1457130.1477079284,1439495.2308881518,1421860.3140683738,1404225.3972485957,1386590.4804288165,1368955.5636090403,1351320.6467892628,1333685.729969484,1316050.813149706,1298415.8963299294,1280780.9795101515,1263146.0626903719,1245511.145870595,1227876.229050819,1210241.312231041,1192606.3954112614,1174971.4785914873,1157336.5617717071,1139701.6449519256,1122066.7281321534,1104431.8113123719,1086796.8944925983,1069161.9776728172,1051527.0608530384,1033892.1440332662,1016257.2272134884,998622.3103937004,980987.3935739286,963352.4767541518,945717.5599343712,928082.6431145924,910447.7262948165,892812.8094750387,875177.8926552646,857542.9758354793,839908.0590157071,822273.1421959256,804638.2253761552,787003.3085563718,769368.3917365898,751733.4749168083,734098.5580970412,716463.6412772592,698828.7244574893,681193.8076377045,663558.8908179253,645923.9739981517,628289.0571783688,610654.1403585887,593019.2235388234,575384.3067190414,557749.3898992566,540114.4730794914,522479.5562597038,504844.63943991903,487209.72262015566,469574.8058003681,451939.8889805898,434304.9721608106,416670.0553410286,399035.1385212578,381400.2217014814,363765.3048817003,346130.3880619239,328495.47124214657,310860.5544223711,293225.63760257885,275590.72078280896,257955.8039630372,240320.88714325428,222685.97032348998,205051.0535037052,187416.13668391947,169781.21986415423,152146.30304436292,134511.38622458372,116876.4694048129,99241.55258503184,81606.63576525543,63971.71894548461,46336.80212569516,28701.885305928066,11066.968486133963,-6567.948333630338,-24202.86515340209,-41837.781973203644,-59472.69879297353,-77107.61561273783,-94742.5324325189],[1650625.62679281,1632896.9371172644,1615168.2474417244,1597439.5577661796,1579710.8680906338,1561982.1784150908,1544253.488739547,1526524.799064004,1508796.1093884578,1491067.4197129128,1473338.7300373719,1455610.040361827,1437881.350686281,1420152.661010738,1402423.9713351952,1384695.28165965,1366966.5919841072,1349237.902308562,1331509.2126330195,1313780.522957475,1296051.8332819308,1278323.1436063878,1260594.4539308422,1242865.7642553,1225137.0745797555,1207408.384904211,1189679.6952286651,1171951.0055531254,1154222.31587758,1136493.6262020352,1118764.9365264876,1101036.246850947,1083307.5571754035,1065578.8674998572,1047850.1778243165,1030121.4881487754,1012392.7984732226,994664.1087976848,976935.4191221441,959206.7294465946,941478.0397710525,923749.3500955063,906020.6604199642,888291.9707444217,870563.2810688759,852834.5913933306,835105.9017177895,817377.2120422437,799648.5223667021,781919.8326911535,764191.1430156128,746462.4533400694,728733.7636645334,711005.0739889638,693276.3843134223,675547.6946378937,657819.0049623502,640090.3152868142,622361.625611267,604632.9359357115,586904.2462601624,569175.5565846246,551446.8669090876,533718.1772335423,515989.48755799886,498260.79788245354,480532.1082068933,462803.4185313545,445074.72885582875,427346.03918028716,409617.349504726,391888.6598291863,374159.97015364934,356431.2804781003,338702.590802555,320973.901127005,303245.21145146806,285516.5217759302,267787.8321003886,250059.1424248293,232330.45274929702,214601.7630737396,196873.0733982073,179144.38372266572,161415.6940471232,143687.00437157042,125958.31469602324,108229.62502047606,90500.93534493726,72772.24566940591,55043.555993859656,37314.866318299435,19586.17664275877,1857.4869672264904,-15871.202708330005,-33599.89238388464,-51328.58205940947,-69057.27173496969,-86785.96141049266,-104514.65108604729],[1650137.0208601342,1632314.5583288218,1614492.0957975117,1596669.633266206,1578847.1707348945,1561024.7082035835,1543202.2456722753,1525379.783140963,1507557.3206096534,1489734.8580783408,1471912.3955470356,1454089.9330157246,1436267.4704844125,1418445.0079531015,1400622.5454217927,1382800.0828904829,1364977.620359172,1347155.1578278611,1329332.695296554,1311510.2327652427,1293687.770233932,1275865.3077026233,1258042.8451713128,1240220.3826400016,1222397.9201086918,1204575.4575773817,1186752.9950460684,1168930.532514764,1151108.069983452,1133285.6074521416,1115463.1449208297,1097640.6823895248,1079818.2198582129,1061995.757326902,1044173.2947955956,1026350.8322642795,1008528.369732975,990705.9072016557,972883.4446703508,955060.9821390468,937238.5196077288,919416.0570764234,901593.5945451125,883771.1320137982,865948.6694824891,848126.2069511823,830303.7444198714,812481.281888559,794658.8193572508,776836.3568259347,759013.8942946317,741191.4317633156,723368.9692320125,705546.5067007048,687724.044169385,669901.5816380698,652079.1191067724,634256.6565754572,616434.1940441411,598611.731512852,580789.2689815387,562966.8064502096,545144.3439189084,527321.8813876007,509499.41885629017,491676.95632498246,473854.4937936729,456032.03126235306,438209.5687310556,420387.1061997339,402564.6436684262,384742.18113712594,366919.7186058061,349097.2560745068,331274.7935431842,313452.33101187926,295629.8684805585,277807.4059492536,259984.94341795146,242162.48088663258,224340.0183553258,206517.5558240246,188695.09329270292,170872.63076139428,153050.16823009588,135227.70569877326,117405.24316746369,99582.78063616529,81760.31810484827,63937.855573528446,46115.393042235635,28292.930510914885,10470.467979608104,-7351.994551692158,-25174.45708301477,-42996.919614316896,-60819.38214563206,-78641.84467695281,-96464.30720825866,-114286.7697395552],[1649648.4149274565,1631732.179540381,1613815.9441533028,1595899.7087662288,1577983.4733791528,1560067.237992079,1542151.002605,1524234.7672179225,1506318.5318308463,1488402.2964437732,1470486.0610566982,1452569.8256696216,1434653.5902825466,1416737.3548954686,1398821.1195083917,1380904.8841213153,1362988.6487342387,1345072.4133471637,1327156.1779600878,1309239.942573011,1291323.7071859336,1273407.4717988588,1255491.2364117817,1237575.001024706,1219658.7656376269,1201742.530250555,1183826.2948634792,1165910.059476403,1147993.824089326,1130077.5887022493,1112161.3533151713,1094245.1179280952,1076328.88254102,1058412.647153947,1040496.4117668639,1022580.1763797929,1004663.940992712,986747.705605641,968831.4702185625,950915.2348314868,932998.9994444158,915082.7640573331,897166.528670263,879250.2932831822,861334.0578961088,843417.8225090308,825501.5871219523,807585.3517348771,789669.1163478056,771752.8809607234,753836.6455736523,735920.4101865795,718004.174799487,700087.9394124234,682171.7040253505,664255.4686382748,646339.2332511954,628422.9978641123,610506.7624770375,592590.5270899711,574674.2917028842,556758.0563158048,538841.8209287338,520925.5855416665,503009.3501545908,485093.1147675123,467176.87938042823,449260.64399335627,431344.40860627964,413428.17321919743,395511.9378321301,377595.70244504977,359679.4670579778,341763.23167089466,323846.9962838162,305930.7608967526,288014.52550967317,270098.2901225835,252182.05473551527,234265.81934844796,216349.5839613611,198433.34857427888,180517.1131872069,162600.87780014146,144684.64241307415,126768.40702598356,108852.17163889855,90935.93625182007,73019.70086475648,55103.46547769103,37187.23009059485,19270.994703533128,1354.7593164574355,-16561.47607062012,-34477.71145769581,-52393.94684476592,-70310.18223184347,-88226.41761894152,-106142.6530060079,-124058.88839309663],[1649159.8089947817,1631149.800751939,1613139.7925090971,1595129.7842662546,1577119.7760234135,1559109.7677805698,1541099.7595377276,1523089.7512948858,1505079.7430520437,1487069.7348092026,1469059.7265663606,1451049.7183235171,1433039.7100806746,1415029.7018378319,1397019.6935949903,1379009.6853521478,1360999.6771093057,1342989.6688664625,1324979.660623621,1306969.65238078,1288959.6441379366,1270949.6358950958,1252939.6276522523,1234929.6194094077,1216919.6111665685,1198909.6029237243,1180899.594680883,1162889.5864380416,1144879.5781951994,1126869.5699523566,1108859.561709513,1090849.553466672,1072839.5452238303,1054829.536980988,1036819.5287381453,1018809.520495302,1000799.5122524602,982789.5040096114,964779.4957667757,946769.4875239339,928759.4792810911,910749.4710382489,892739.4627954066,874729.4545525671,856719.4463097248,838709.4380668821,820699.4298240356,802689.4215811943,784679.4133383567,766669.4050955055,748659.3968526712,730649.3886098294,712639.3803669838,694629.372124128,676619.3638813021,658609.3556384612,640599.3473956082,622589.3391527766,604579.3309099376,586569.3226670921,568559.31442424,550549.3061814066,532539.2979385685,514529.2896957202,496519.28145286813,478509.2732100291,460499.26496719755,442489.256724352,424479.24848149903,406469.24023867026,388459.23199582193,370449.2237529801,352439.2155101346,334429.20726730116,316419.1990244482,298409.1907816101,280399.18253877107,262389.17429592926,244379.16605308466,226369.15781024564,208359.14956740662,190349.14132456202,172339.13308170997,154329.1248388728,136319.11659603938,118309.10835319012,100299.10011034645,82289.0918674944,64279.08362467028,46269.07538181916,28259.067138976417,10249.058896133676,-7760.949346698821,-25770.95758955367,-43780.9658323843,-61790.9740752466,-79800.98231807724,-97810.99056091718,-115820.99880375527,-133831.00704658963],[1648671.2030621055,1630567.421963499,1612463.6408648917,1594359.8597662821,1576256.078667671,1558152.2975690635,1540048.516470458,1521944.7353718483,1503840.9542732397,1485737.1731746336,1467633.3920760187,1449529.610977411,1431425.8298788043,1413322.048780197,1395218.2676815873,1377114.4865829803,1359010.7054843712,1340906.9243857644,1322803.1432871548,1304699.3621885474,1286595.5810899392,1268491.79999133,1250388.0188927217,1232284.2377941143,1214180.4566955054,1196076.6755968952,1177972.8944982868,1159869.1133996798,1141765.332301071,1123661.551202464,1105557.7701038579,1087453.989005242,1069350.2079066401,1051246.4268080276,1033142.6457094247,1015038.8646108122,996935.0835122098,978831.3024135968,960727.5213149837,942623.7402163809,924519.9591177665,906416.178019166,888312.3969205515,870208.6158219432,852104.8347233357,834001.0536247264,815897.2725261208,797793.4914275119,779689.7103289077,761585.9292302979,743482.14813169,725378.367033069,707274.5859344704,689170.804835869,671067.0237372713,652963.242638641,634859.4615400368,616755.6804414336,598651.8993428089,580548.1182442103,562444.3371456098,544340.5560469897,526236.7749483865,508132.9938497711,490029.2127511753,471925.4316525571,453821.6505539464,435717.8694553459,417614.08835672867,399510.30725813564,381406.5261595128,363302.74506091606,345198.96396229975,327095.18286369834,308991.4017650839,290887.6206664769,272783.83956787735,254680.05846925732,236576.27737065312,218472.49627204426,200368.71517344099,182264.93407483306,164161.15297621023,146057.37187761068,127953.59077900182,109849.80968038552,91746.0285817841,73642.24748317804,55538.46638458688,37434.68528595101,19330.904187343083,1227.123088747263,-16876.658009858802,-34980.439108477905,-53084.22020707652,-71188.00130568258,-89291.78240429424,-107395.56350289844,-125499.34460151754,-143603.12570012175],[1648182.5971294285,1629985.043175056,1611787.4892206818,1593589.9352663069,1575392.3813119289,1557194.8273575567,1538997.2734031836,1520799.7194488093,1502602.165494435,1484404.6115400614,1466207.057585684,1448009.5036313091,1429811.9496769367,1411614.3957225622,1393416.8417681872,1375219.2878138137,1357021.7338594368,1338824.1799050642,1320626.6259506887,1302429.071996316,1284231.5180419409,1266033.9640875666,1247836.4101331912,1229638.8561788178,1211441.302224442,1193243.748270066,1175046.1943156966,1156848.6403613188,1138651.0864069448,1120453.5324525712,1102255.9784981976,1084058.424543819,1065860.8705894467,1047663.3166350727,1029465.7626806949,1011268.2087263251,993070.6547719482,974873.1008175742,956675.5468632011,938477.9929088228,920280.4389544544,902082.8850000771,883885.3310457016,865687.777091328,847490.2231369494,829292.669182573,811095.1152282013,792897.5612738263,774700.0073194532,756502.453365068,738304.8994107153,720107.3454563385,701909.791501957,683712.2375475783,665514.6835931968,647317.1296388479,629119.5756844673,610922.0217300821,592724.4677757081,574526.9138213247,556329.3598669618,538131.8059125906,519934.2519582156,501736.6980038313,483539.144049461,465341.5900950851,447144.0361407213,428946.4821863454,410748.9282319583,392551.3742775833,374353.8203232186,356156.2663688343,337958.7124144677,319761.15846009087,301563.6045057243,283366.05055134837,265168.496596965,246970.94264258258,228773.3886882197,210575.83473385777,192378.28077947255,174180.7268250892,155983.17287071608,137785.61891635042,119588.0649619773,101390.51100759581,83192.95705322642,64995.403098838404,46797.84914446622,28600.295190107077,10402.741235725582,-7794.812718642876,-25992.36667301692,-44189.92062741332,-62387.474581781775,-80585.02853615582,-98782.58249052614,-116980.13644487783,-135177.6903992761,-153375.24435363337],[1647693.9911967549,1629402.6643866133,1611111.337576471,1592820.010766331,1574528.683956192,1556237.3571460517,1537946.0303359113,1519654.7035257714,1501363.37671563,1483072.0499054904,1464780.7230953455,1446489.3962852068,1428198.0694750692,1409906.7426649262,1391615.4158547854,1373324.0890446426,1355032.7622345046,1336741.4354243623,1318450.1086142238,1300158.781804083,1281867.4549939425,1263576.1281838012,1245284.8013736617,1226993.4745635213,1208702.1477533798,1190410.8209432405,1172119.4941330976,1153828.1673229565,1135536.8405128168,1117245.5137026794,1098954.1868925346,1080662.8600823954,1062371.5332722533,1044080.2064621183,1025788.8796519735,1007497.552841831,989206.2260316955,970914.8992215507,952623.5724114068,934332.2456012745,916040.9187911302,897749.5919809905,879458.2651708522,861166.9383607064,842875.6115505677,824584.2847404238,806292.9579302841,788001.6311201449,769710.3043100033,751418.9774998706,733127.6506897276,714836.323879581,696544.9970694454,678253.6702592922,659962.3434491539,641671.0166390222,623379.689828882,605088.3630187353,586797.0362086035,568505.7093984578,550214.3825883139,531923.055778184,513631.72896804195,495340.4021578962,477049.0753477579,458757.7485376168,440466.4217274729,422175.09491734114,403883.76810718514,385592.4412970524,367301.1144869141,349009.7876767712,330718.46086662635,312427.13405649364,294135.8072463479,275844.4804362105,257553.153626075,239261.8268159218,220970.5000057984,202679.17319563963,184387.84638550598,166096.51957537234,147805.19276521914,129513.8659550855,111222.53914494906,92931.21233480051,74639.88552466873,56348.55871452391,38057.23190438282,19765.905094244517,1474.5782841173932,-16816.748526040465,-35108.07533616759,-53399.40214632824,-71690.72895645723,-89982.05576660112,-108273.38257674687,-126564.70938686095,-144856.03619701602,-163147.36300719157],[1647205.3852640782,1628820.2855981744,1610435.1859322626,1592050.0862663602,1573664.986600452,1555279.8869345454,1536894.7872686367,1518509.6876027319,1500124.5879368237,1481739.4882709188,1463354.3886050102,1444969.2889391058,1426584.1892731984,1408199.0896072914,1389813.989941383,1371428.890275477,1353043.7906095705,1334658.6909436649,1316273.5912777593,1297888.491611852,1279503.3919459446,1261118.2922800367,1242733.1926141311,1224348.0929482253,1205962.993282318,1187577.8936164118,1169192.7939505037,1150807.694284595,1132422.5946186897,1114037.4949527862,1095652.3952868772,1077267.2956209723,1058882.1959550628,1040497.0962891583,1022111.996623253,1003726.8969573448,985341.7972914325,966956.697625536,948571.5979596269,930186.4982937151,911801.3986278106,893416.2989619067,875031.1992959972,856646.0996300876,838260.9999641813,819875.9002982774,801490.8006323688,783105.7009664644,764720.6013005534,746335.5016346471,727950.4019687399,709565.3023028346,691180.2026369385,672795.1029710341,654410.0033051223,636024.9036392067,617639.8039732985,599254.7043073876,580869.6046414897,562484.5049755899,544099.4053096836,525714.3056437746,507329.20597786363,488944.10631195083,470559.0066460511,452173.90698013734,433788.8073142357,415403.70764833223,397018.6079824269,378633.508316515,360248.40865060873,341863.30898470804,323478.20931878965,305093.10965288524,286708.00998697523,268322.9103210736,249937.81065516733,231552.71098926477,213167.61132335756,194782.5116574457,176397.411991545,158012.3123256294,139627.21265973896,121242.1129938243,102857.0133279236,84471.91366201732,66086.81399610825,47701.714330192655,29316.614664286375,10931.514998380095,-7453.584667517804,-25838.68433343433,-44223.783999344334,-62608.883665245026,-80993.98333114572,-99379.08299704269,-117764.18266295455,-136149.28232887015,-154534.38199477643,-172919.48166067153],[1646716.7793314033,1628237.9068097305,1609759.0342880564,1591280.1617663833,1572801.289244711,1554322.4167230385,1535843.544201366,1517364.6716796923,1498885.7991580176,1480406.926636348,1461928.0541146721,1443449.1815930025,1424970.309071327,1406491.4365496559,1388012.5640279825,1369533.6915063104,1351054.8189846366,1332575.9464629649,1314097.0739412892,1295618.2014196194,1277139.3288979472,1258660.4563762732,1240181.5838546008,1221702.7113329275,1203223.838811255,1184744.9662895831,1166266.093767907,1147787.2212462388,1129308.3487245636,1110829.4762028884,1092350.6036812188,1073871.7311595473,1055392.8586378735,1036913.9861161988,1018435.1135945227,999956.2410728578,981477.3685511821,962998.4960295069,944519.6235078322,926040.7509861626,907561.8784644911,889083.0059428192,870604.1334211417,852125.2608994707,833646.3883777973,815167.5158561259,796688.6433344516,778209.7708127806,759730.8982911091,741252.0257694395,722773.1532477513,704294.2807260891,685815.4082044158,667336.5356827453,648857.6631610785,630378.7906393912,611899.9181177272,593421.0455960631,574942.1730743786,556463.3005527109,537984.428031032,519505.5555093661,501026.6829876993,482547.81046601105,464068.93794434424,445590.06542266905,427111.1929009985,408632.3203793345,390153.44785765,371674.57533597853,353195.7028143024,334716.8302926319,316237.9577709669,297759.08524928987,279280.2127276156,260801.34020593576,242322.4676842671,223843.5951626012,205364.7226409223,186885.85011924896,168406.97759757098,149928.10507591534,131449.23255423456,112970.36003255378,94491.48751089163,76012.61498919968,57533.74246754497,39054.869945880026,20575.99742419552,2097.124902524054,-16381.747619157657,-34860.620140822604,-53339.49266249314,-71818.36518414877,-90297.23770584352,-108776.11022752151,-127254.98274918087,-145733.85527086444,-164212.72779252566,-182691.60031421855],[1646228.1733987243,1627655.5280212876,1609082.8826438494,1590510.2372664073,1571937.5918889714,1553364.946511532,1534792.3011340946,1516219.6557566521,1497647.0103792152,1479074.3650017774,1460501.7196243356,1441929.0742468967,1423356.4288694605,1404783.7834920192,1386211.1381145814,1367638.4927371435,1349065.8473597053,1330493.201982264,1311920.5566048264,1293347.9112273876,1274775.2658499484,1256202.6204725082,1237629.975095069,1219057.3297176315,1200484.6843401934,1181912.0389627544,1163339.3935853122,1144766.7482078774,1126194.102830436,1107621.4574529957,1089048.81207556,1070476.1666981196,1051903.5213206839,1033330.8759432435,1014758.2305658017,996185.5851883623,977612.9398109228,959040.2944334913,940467.6490560514,921895.0036786105,903322.3583011683,884749.7129237293,866177.0675462908,847604.4221688514,829031.7767914142,810459.1314139757,791886.4860365358,773313.8406590996,754741.1952816611,736168.549904218,717595.9045267766,699023.2591493372,680450.6137718894,661877.9683944592,643305.3230170384,624732.6776395952,606160.0322621511,587587.3868847145,569014.7415072732,550442.0961298309,531869.4507523887,513296.8053749474,494724.1599975135,476151.5146200685,457578.8692426402,439006.2238651961,420433.578487752,401860.9331103284,383288.28773288336,364715.6423554411,346142.99697800074,327570.3516005641,308997.7062231209,290425.0608456936,271852.41546825226,253279.77009081002,234707.12471337337,216134.4793359274,197561.83395849075,178989.18858104944,160416.54320360348,141843.89782617614,123271.25244873948,104698.60707130097,86125.96169386711,67553.31631642208,48980.6709389817,30408.02556154877,11835.380184096284,-6737.265193337575,-25309.910570777953,-43882.555948199704,-62455.201325649396,-81027.84670308232,-99600.49208053015,-118173.13745796122,-136745.78283540905,-155318.42821283452,-173891.07359026745,-192463.71896771714],[1645739.5674660502,1627073.149232843,1608406.7309996416,1589740.312766435,1571073.8945332295,1552407.4763000256,1533741.0580668189,1515074.6398336156,1496408.2216004126,1477741.8033672064,1459075.3851339982,1440408.966900793,1421742.5486675915,1403076.1304343857,1384409.7122011792,1365743.2939679762,1347076.8757347686,1328410.4575015625,1309744.0392683612,1291077.6210351554,1272411.2028019505,1253744.7845687447,1235078.3663355415,1216411.9481023345,1197745.5298691308,1179079.1116359243,1160412.6934027178,1141746.2751695178,1123079.8569363104,1104413.438703103,1085747.0204698965,1067080.6022366951,1048414.1840034905,1029747.7657702891,1011081.3475370822,992414.9293038761,973748.5110706692,955082.0928374622,936415.6746042566,917749.2563710501,899082.8381378502,880416.4199046423,861750.0016714428,843083.5834382339,824417.1652050312,805750.7469718195,787084.328738621,768417.9105054149,749751.4922722094,731085.074039001,712418.6558057973,693752.2375726011,675085.8193393806,656419.4011061918,637752.9828729779,619086.564639777,600420.1464065686,581753.7281733667,563087.3099401733,544420.8917069556,525754.4734737538,507088.055240538,488421.6370073417,469755.21877413243,451088.80054092593,432422.38230772223,413755.96407451946,395089.5458413102,376423.12760810833,357756.7093749065,339090.29114169814,320423.8729084963,301757.4546752935,283091.03644208424,264424.6182088796,245758.1999756666,227091.78174246196,208425.3635092657,189758.94527605362,171092.52704285737,152426.1088096602,133759.69057644252,115093.2723432444,96426.85411003511,77760.4358768398,59094.01764361281,40427.59941041935,21761.181177202612,3094.7629440128803,-15571.655289191753,-34238.07352239266,-52904.491755619645,-71570.90998879261,-90237.32822200842,-108903.74645521119,-127570.16468844563,-146236.58292162046,-164903.00115483813,-183569.41938803717,-202235.83762124926],[1645250.9615333735,1626490.7704444013,1607730.5793554292,1588970.3882664624,1570210.1971774898,1551450.00608852,1532689.8149995469,1513929.6239105747,1495169.4328216058,1476409.2417326346,1457649.0506436632,1438888.8595546898,1420128.668465718,1401368.4773767493,1382608.286287778,1363848.0951988061,1345087.9041098366,1326327.7130208656,1307567.5219318948,1288807.330842924,1270047.1397539526,1251286.94866498,1232526.7575760102,1213766.566487038,1195006.3753980668,1176246.1843090965,1157485.9932201281,1138725.8021311564,1119965.6110421834,1101205.4199532103,1082445.228864239,1063685.0377752674,1044924.8466862943,1026164.6555973236,1007404.4645083556,988644.2734193848,969884.0823304183,951123.8912414461,932363.700152474,913603.5090635056,894843.3179745311,876083.126885558,857322.9357965868,838562.7447076165,819802.5536186425,801042.3625296741,782282.1714407057,763521.9803517321,744761.7892627614,726001.5981737878,707241.4070848152,688481.2159958435,669721.0249068756,650960.8338178992,632200.6427289331,613440.4516399615,594680.2605509814,575920.0694620153,557159.8783730445,538399.6872840729,519639.49619510025,500879.3051061379,482119.1140171671,463358.92292819545,444598.7318392191,425838.54075025115,407078.3496612832,388318.15857230686,369557.96748333424,350797.77639436815,332037.5853053881,313277.3942164229,294517.2031274559,275757.0120384898,256996.82094951533,238236.62986053992,219476.4387715701,200716.247682604,181956.05659362115,163195.86550465878,144435.6744156871,125675.48332671169,106915.29223775398,88155.10114877205,69394.91005980596,50634.718970832415,31874.52788186632,13114.336792885326,-5645.854296080768,-24406.04538505338,-43166.236474025995,-61926.42756299302,-80686.61865197122,-99446.80974094197,-118207.00082993135,-136967.19191888534,-155727.3830078654,-174487.57409682497,-193247.76518579386,-212007.95627475716]],"type":"heatmap","xaxis":"x","yaxis":"y","hovertemplate":"Home Price: %{x}\u003cbr\u003eDownpayment: %{y}\u003cbr\u003eOpportunity: %{z}\u003cextra\u003e\u003c\u002fextra\u003e"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Home Price"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"autorange":"reversed","title":{"text":"Downpayment"}},"coloraxis":{"colorbar":{"title":{"text":"Opportunity"}},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"title":{"text":"Opportunity Surface"}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

### `home_price_growth_rate` and `investment_return_rate`

In [15]:
_x, _y = np.linspace(0.02, 0.1, 100), np.linspace(0.02, 0.1, 100)
x, y = np.meshgrid(_x, _y, indexing="ij")
x = x.flatten()
y = y.flatten()
surface_kwargs = kwargs.copy()
surface_kwargs.update({"home_price_growth_rate": x, "investment_return_rate": y})
render_plotly_html(
    px.imshow(
        RentVsBuy().calculate(**surface_kwargs).value.reshape((100, 100)).T,
        aspect="auto",
        labels=dict(x="Home Price Growth", y="Investment Growth", color="Opportunity"),
        x=_x,
        y=_y,
        title="Opportunity Surface",
    )
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="e8077885-5dd2-47ff-a2ed-d8aaa7559120" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("e8077885-5dd2-47ff-a2ed-d8aaa7559120")) {                    Plotly.newPlot(                        "e8077885-5dd2-47ff-a2ed-d8aaa7559120",                        [{"coloraxis":"coloraxis","name":"0","x":[0.02,0.02080808080808081,0.021616161616161617,0.022424242424242423,0.023232323232323233,0.024040404040404043,0.02484848484848485,0.025656565656565655,0.026464646464646465,0.027272727272727275,0.02808080808080808,0.028888888888888888,0.029696969696969697,0.030505050505050507,0.031313131313131314,0.03212121212121212,0.03292929292929293,0.03373737373737374,0.034545454545454546,0.03535353535353535,0.03616161616161616,0.03696969696969697,0.03777777777777778,0.038585858585858585,0.0393939393939394,0.0402020202020202,0.04101010101010101,0.041818181818181824,0.04262626262626262,0.043434343434343436,0.04424242424242424,0.04505050505050505,0.04585858585858586,0.04666666666666666,0.047474747474747475,0.04828282828282829,0.04909090909090909,0.0498989898989899,0.05070707070707071,0.051515151515151514,0.05232323232323233,0.053131313131313126,0.05393939393939394,0.05474747474747475,0.05555555555555555,0.056363636363636366,0.05717171717171718,0.05797979797979798,0.05878787878787879,0.05959595959595959,0.060404040404040404,0.06121212121212122,0.06202020202020202,0.06282828282828283,0.06363636363636364,0.06444444444444444,0.06525252525252526,0.06606060606060606,0.06686868686868687,0.06767676767676768,0.06848484848484848,0.0692929292929293,0.07010101010101011,0.07090909090909091,0.07171717171717172,0.07252525252525252,0.07333333333333333,0.07414141414141415,0.07494949494949495,0.07575757575757576,0.07656565656565657,0.07737373737373737,0.07818181818181819,0.07898989898989899,0.0797979797979798,0.08060606060606061,0.08141414141414141,0.08222222222222222,0.08303030303030304,0.08383838383838384,0.08464646464646465,0.08545454545454546,0.08626262626262626,0.08707070707070708,0.08787878787878788,0.08868686868686869,0.0894949494949495,0.0903030303030303,0.09111111111111111,0.09191919191919193,0.09272727272727273,0.09353535353535354,0.09434343434343435,0.09515151515151515,0.09595959595959597,0.09676767676767677,0.09757575757575758,0.09838383838383839,0.09919191919191919,0.1],"y":[0.02,0.02080808080808081,0.021616161616161617,0.022424242424242423,0.023232323232323233,0.024040404040404043,0.02484848484848485,0.025656565656565655,0.026464646464646465,0.027272727272727275,0.02808080808080808,0.028888888888888888,0.029696969696969697,0.030505050505050507,0.031313131313131314,0.03212121212121212,0.03292929292929293,0.03373737373737374,0.034545454545454546,0.03535353535353535,0.03616161616161616,0.03696969696969697,0.03777777777777778,0.038585858585858585,0.0393939393939394,0.0402020202020202,0.04101010101010101,0.041818181818181824,0.04262626262626262,0.043434343434343436,0.04424242424242424,0.04505050505050505,0.04585858585858586,0.04666666666666666,0.047474747474747475,0.04828282828282829,0.04909090909090909,0.0498989898989899,0.05070707070707071,0.051515151515151514,0.05232323232323233,0.053131313131313126,0.05393939393939394,0.05474747474747475,0.05555555555555555,0.056363636363636366,0.05717171717171718,0.05797979797979798,0.05878787878787879,0.05959595959595959,0.060404040404040404,0.06121212121212122,0.06202020202020202,0.06282828282828283,0.06363636363636364,0.06444444444444444,0.06525252525252526,0.06606060606060606,0.06686868686868687,0.06767676767676768,0.06848484848484848,0.0692929292929293,0.07010101010101011,0.07090909090909091,0.07171717171717172,0.07252525252525252,0.07333333333333333,0.07414141414141415,0.07494949494949495,0.07575757575757576,0.07656565656565657,0.07737373737373737,0.07818181818181819,0.07898989898989899,0.0797979797979798,0.08060606060606061,0.08141414141414141,0.08222222222222222,0.08303030303030304,0.08383838383838384,0.08464646464646465,0.08545454545454546,0.08626262626262626,0.08707070707070708,0.08787878787878788,0.08868686868686869,0.0894949494949495,0.0903030303030303,0.09111111111111111,0.09191919191919193,0.09272727272727273,0.09353535353535354,0.09434343434343435,0.09515151515151515,0.09595959595959597,0.09676767676767677,0.09757575757575758,0.09838383838383839,0.09919191919191919,0.1],"z":[[-125546.27668532706,-106368.492240789,-86862.39774120797,-67022.56601090136,-46843.48158944596,-26319.539309679996,-5445.0428533048835,15785.796716659097,37378.86244679056,59340.13300836948,81675.68423595838,104391.69069130882,127494.42725182814,150990.2707245706,174885.70148582617,199187.3051473801,223901.77424852597,249035.909975881,274596.6239103081,300590.93980173836,327025.9953722821,353909.04414853477,381247.45732221566,409048.7256414287,437320.4613315989,466070.4000471898,495306.40285440884,525036.4582460604,555268.6841875187,586011.330196349,617272.7794545477,649061.5509546062,681386.3016796093,714255.828818711,747679.0720167537,781665.1156609138,816223.1912031376,851362.6795197788,887093.1133086705,923424.1795249877,960365.7218547203,997927.7432287277,1036120.4083762569,1074954.0464192117,1114439.153507638,1154586.3954977824,1195406.6106715186,1236910.8125003562,1279110.1924527592,1322016.1228463345,1365640.159745235,1409994.0459043665,1455089.71375905,1500939.2884636512,1547555.09097784,1594949.6412020954,1643135.6611629033,1692126.0782494484,1741934.028500119,1792572.859942925,1844056.1359881922,1896397.6388753813,1949611.3731744946,2003711.5693440493,2058712.6873437983,2114629.4203066677,2171476.698268148,2229269.69195512,2288023.8166346555,2347754.7360249897,2408478.366266628,2470210.879958485,2532968.7102571735,2596768.55504155,2661627.381143217,2727562.4286452276,2794591.2152469316,2862731.5407002443,2932001.4913152307,3002419.44453747,3074004.073597815,3146774.3522370635,3220749.559503343,3295949.2846278725,3372393.43197698,3450102.2260825727,3529096.2167537836,3609396.2842670316,3691023.644641013,3773999.8549940567,3858346.818986679,3944086.7923501213,4031242.3885040213,4119836.5842602104,4209892.725619841,4301434.533660928,4394486.110519501,4489071.945465215,4585216.921074817,4682946.319500266],[-135672.04553100653,-116505.12971093913,-97009.96513974585,-77181.12503918714,-57013.09434905683,-36500.26830520318,-15636.950995154679,5582.6461094066035,27164.405643542996,49114.30586413713,71438.42218846106,94142.92875805474,117234.10002717073,140718.31237675133,164602.04575398774,188891.8853385616,213594.52323463582,238716.76019065012,264265.50734621217,290247.78800691036,316670.73944739066,343541.61474364717,370867.78463363345,398656.73940850305,426916.0908335224,455653.57409976213,484877.04980678204,514594.50597745646,544814.0601039294,575543.9612272053,606792.5920493682,638568.4710796161,670880.2548143461,703736.7399525899,737146.8656456203,771119.7157835646,805664.5213178173,840790.6626206527,876507.6718822734,912825.235546635,949753.1967849098,987301.5580094999,1025480.4834275395,1064300.3016351303,1103771.5082528042,1143904.7686035514,1184710.92043222,1226200.9766694992,1268386.128239208,1311277.7469104552,1354887.3881950143,1399226.7942915051,1444307.8970750214,1490142.821135739,1536743.8868651355,1584123.613591479,1632294.722764986,1681270.1411944847,1731063.004333895,1781686.6596226115,1833154.6698781706,1885480.8167430342,1938679.1041859682,1992763.7620599905,2047749.2497150437,2103650.2596699186,2160481.721341609,2218258.804834094,2276996.9247871214,2336711.7442871407,2397419.178838373,2459135.4003989277,2521876.8414800465,2585660.1993106203,2650502.440067664,2716420.803174969,2783432.8056679377,2851556.2466298016,2920809.2116971714,2991210.077637377,3062777.5169981807,3135530.5028324146,3209488.313495338,3284670.5375203486,3361097.078570978,3438788.1604713094,3517764.332317602,3598046.473668308,3679655.7998190233,3762613.867159806,3846942.5786176967,3932664.1891852175,4019801.311537994,4108376.9217385272,4198414.365033277,4289937.361740156,4382970.013229657,4477536.808000405,4573662.627852604,4671372.754156092],[-146011.258934631,-126855.71994587581,-107371.99787912669,-87554.66639312205,-67398.21086776734,-46897.02698219335,-26045.419270382263,-4837.599653688259,16732.31405004859,38670.29964155215,60982.432078652084,83674.88504014118,106753.9325141944,130225.95041231625,154097.41820888245,178374.92060735147,203065.1492322269,228174.90434883465,253711.0966101801,279680.74883174314,306090.99779451685,332949.09607728804,360262.4139172039,388038.44110099645,416284.78888586373,445009.19195113797,474219.5103809375,503923.7316789685,534129.9728144482,564846.482301671,596081.6423121968,627843.9708208551,660142.1237858054,692984.89736394,726381.2301604648,760340.2055154741,794871.053826344,829983.1549073115,865686.040386487,901989.3961416562,938903.0647737074,976437.0481206176,1014601.5098109194,1053406.7778579078,1092863.3472950687,1132981.8828540773,1173773.221684165,1215248.3761160672,1257418.5364692823,1300295.073904196,1343889.543319427,1388213.6862959682,1433279.434086789,1479098.9106554007,1525684.435762046,1573048.528099157,1621203.9084764689,1670163.5030576545,1719940.4466467705,1770548.0860285952,1821999.9833612705,1874309.9196230427,1927491.898113609,1981560.1480120197,2036529.1279893254,2092413.5298804562,2149228.2824135358,2206988.5549986353,2265709.761576508,2325407.5645294935,2386097.878652536,2447796.8751892773,2510520.9859312484,2574286.9073823513,2639111.604989294,2705012.3174402025,2772006.5610294156,2840112.1340936525,2909347.1215175404,2979729.8993108915,3051279.1392583894,3124013.8136441754,3197953.2000491684,3273116.8862267286,3349524.775054613,3427197.0895653493,3506154.378057811,3586417.519287197,3668007.727739932,3750946.558990944,3835255.915146136,3920958.0503708366,4008075.5765073854,4096631.468778848,4186649.071586058,4278152.104395059,4371164.667718193,4465711.249189593,4561816.729738589,4659506.389857717],[-156557.4900702678,-137413.79696904856,-117941.99049484276,-98136.64478005853,-77992.24568174256,-57503.189359677024,-36663.78083202173,-15468.2325078703,6089.337303617969,28014.905908227898,50314.54776521679,72994.4360511289,96060.8442481982,119520.14775826177,143378.82554227416,167643.46178648784,192320.7475943889,217417.48270644643,242940.57724693534,268897.0534987019,295294.0477061805,322138.8119076316,349438.71579568484,377201.2486085363,405434.0210507908,434144.7672451015,463341.3467147895,493031.7463986201,523224.08269668533,553926.6035499431,585147.6905523685,616895.8610969302,649179.7705556152,682008.2144947986,715390.1309247888,749334.6025863716,783850.8592731676,818948.2801911796,854636.3963557654,890924.893027409,927823.6121851106,965342.555040335,1003491.8845904535,1042281.9282128967,1081723.1803005624,1121826.3049397718,1162602.1386295937,1204061.693045766,1246216.1578479097,1289076.903531614,1332655.4843257526,1376963.6411365739,1422013.3045372781,1467816.597806533,1514385.8400146286,1561733.5491588933,1609872.4453487748,1658815.45404242,1708575.7093330945,1759166.5572894735,1810601.5593482414,1862894.4957607938,1916059.3690945366,1970110.4077907575,2025062.0697772158,2080929.0461399893,2137726.264852741,2195468.8945654295,2254172.348453002,2313852.288126246,2374524.6276027784,2436205.5373430746,2498911.448349628,2562659.056331387,2627465.325934127,2693347.4950390384,2760323.0791274575,2828409.8757169964,2897625.9688670174,2967989.733755864,3039519.8413305017,3112235.2630310496,3186155.275588054,3261299.465898106,3337687.7359757395,3415340.30798376,3494277.7293447694,3574520.877931092,3656090.967339621,3739009.5522490516,3823298.533862292,3908980.1654348653,3996077.0578924417,4084612.1855344973,4174608.8918313053,4266090.895311322,4359082.295542217,4453607.579206318,4549691.626273948,4647359.716271392],[-167307.36636437895,-148175.95956240164,-128716.51284184109,-108923.60084273235,-88791.70993353997,-68315.23678928567,-47488.48694721679,-26305.67333939788,-4760.914801866515,17151.76544033899,39438.4413116856,62105.28544991976,85158.57079444476,108604.67220019945,132450.06807714468,156701.34205640806,181365.1846821853,206448.39513146388,231957.8829608066,257900.66988109075,284283.8915604786,311114.7994566378,338400.7626772409,366149.2698711294,394367.93114912114,423064.4800356212,452246.77545119426,481922.8037273183,512100.68065222516,542788.6535504065,573995.1033947214,605728.546952344,637997.6389647557,670811.174363059,704178.0905175109,738107.4695240059,772608.5405264099,807690.6820760637,843363.4245287203,879636.4524802745,916519.6072401109,954022.8893450231,992156.461112597,1030930.649235351,1070355.9474160736,1110443.0190457604,1151202.699922884,1192646.0010172853,1234784.1112773342,1277628.4004819805,1321190.4221380223,1365481.9164241592,1410514.8131805225,1456301.234947144,1502853.500050073,1550184.1257367488,1598305.8313610274,1647231.5416197253,1696974.389838993,1747547.7213145534,1798965.0967042658,1851240.295474771,1904387.3194027506,1958420.3961327442,2013353.982789701,2069202.7696507731,2125981.6838745303,2183705.8932896303,2242390.810243453,2302052.0955129103,2362705.6622753832,2424367.680144699,2487054.579270238,2550783.054501321,2615570.069617538,2681432.8616272653,2748388.945132363,2816456.1167642423,2885652.459689287,2955996.348186029,3027506.452294744,3100201.742541918,3174101.4947374696,3249225.294850315,3325593.0439602043,3403224.9632879975,3482141.599307134,3562363.8289334932,3643912.8648001957,3726810.2606147667,3811077.9166014907,3896738.08502976,3983813.375831541,4072326.7623049766,4162301.5869113104,4253761.567162219,4346730.801600778,4441233.775876837,4537295.368920314,4634940.859209072],[-178257.45636795717,-159138.75699913513,-139692.09480542142,-119912.04496822145,-99793.09440147865,-79329.64032983943,-58515.988844350446,-37346.353435087716,-15814.85350034223,6084.487168318359,28357.741924519418,51011.08283081744,74050.78224704997,97483.21444417187,121314.85724371206,145552.29368387721,170202.2137114245,195271.41590134217,220766.8092036005,246695.41471785237,273064.3674963745,299880.9183762348,327152.4358397664,354886.4079056962,383090.4440499041,411772.27715699887,440939.7655028589,470600.8947693324,500763.78009005566,531436.6681299042,562627.9391970693,594346.1093889605,626599.8327721618,659397.9035977,692749.2585505352,726662.9790360115,761148.2935021373,796214.5797990509,831871.3675759183,868128.3407166132,904995.3398130273,942482.3646789447,980599.5769033753,1019357.3024446554,1058766.0342657415,1098836.4350120986,1139579.3397309245,1181005.7586349973,1223126.879909787,1265954.072565462,1309498.889334109,1353773.0696137382,1398788.5424577657,1444557.429613436,1491092.048607881,1538404.9158834603,1586508.7499827216,1635416.4747849023,1685141.2227922534,1735696.3384702231,1787095.3816399712,1839352.1309249643,1892480.587252177,1946494.9774098669,2001409.7576600623,2057239.617410307,2113999.4829428224,2171704.521203122,2230370.143648587,2290012.0101592317,2350646.0330085694,2412288.380899546,2474955.483063586,2538664.0334249223,2603430.9948308636,2669273.603350265,2736209.372638151,2804256.0983717395,2873431.8627557894,2943755.039099731,3015244.296467186,3087918.6044003842,3161797.2377173197,3236899.7813872527,3313246.1354824873,3390856.520208581,3469751.481015752,3549951.8937886767,3631478.9701212207,3714354.2626735475,3798599.6706143967,3884237.4451493723,3971290.1951383417,4059780.8927989686,4149732.8795035696,4241169.871666381,4334115.96672445,4428595.649212949,4524633.79693839,4622255.687246434],[-189404.47401256207,-170298.89201946824,-150865.42797309963,-131098.6576319189,-110993.06849139638,-90543.05836220155,-69742.93392595276,-48586.909267864656,-27069.104385985993,-5183.54367629136,17075.845605750335,39715.23490887252,62740.895974327344,86159.20244967635,109976.63152821781,134199.76561503857,158835.29401885928,183890.0146716684,209370.83587542805,235284.7780767486,261638.97566977236,288440.6788283102,315697.2553662681,343416.19262873917,371605.0994127139,400271.7079186174,429423.8757328051,459069.58784222184,489216.9586801722,519874.23420572956,551049.7940157701,582752.1534908258,614989.965974998,647772.024991178,681107.2664904972,715004.7711387211,749473.7666384878,784523.6300887335,820163.8903815262,856404.2306377154,893254.4906801784,930724.6695476549,968824.9280480319,1007565.5913523857,1046957.1516302298,1087010.2707273627,1127735.782885022,1169144.6975036943,1211248.2019501708,1254057.6644095017,1297584.6367821628,1341840.8576280028,1386838.2551556833,1432588.9502610366,1479105.2596130772,1526399.698789278,1574484.985460486,1623374.0426273562,1673080.0019066322,1723616.2068712618,1774996.216442859,1827233.808338241,1880342.9825705665,1934337.9650070593,1989233.2109814275,2045043.4089655627,2101783.4842986222,2159468.6029755943,2218114.1754958173,2277735.860773667,2338349.570109376,2399971.4712248947,2462617.9923628774,2526305.8264509453,2591051.935331893,2656873.554062088,2723788.195276037,2791813.6536223334,2860968.0102689434,2931269.6374802734,3002737.203266607,3075389.676108472,3149246.3297537128,3224326.748092929,3300650.830111185,3378238.7949181385,3457111.186859387,3537288.8807061883,3618793.0869301204,3701645.3570601107,3785867.589124652,3871482.0331800054,3958511.2969275247,4046978.3514171196,4136906.5368440375,4228319.568436038,4321241.542434221,4415696.942168261,4511710.644229529,4609307.924738845],[-200745.65105654253,-181653.56788370514,-162233.68680234498,-142480.58418047975,-122388.74812792602,-101952.57707452867,-81166.37832594337,-60024.36659633997,-38520.66251766123,-16649.291124770883,5595.819682941306,28220.840705220355,51232.04302921309,74635.79964321782,98438.58707607398,122646.9870631632,147267.6882382182,172307.48785291007,197773.29352351115,223672.12500552018,250011.11599651352,276797.51596821565,304038.6920269055,331742.1308044391,359915.4403789332,388566.35222623753,417702.72320239944,447332.53755826317,477463.9089851987,508105.0826944632,539264.437529179,570950.4881101456,603171.8870157006,635937.4269968846,669256.0432268512,703136.8155872023,737588.9709901791,772621.8857380301,808245.0879198043,844468.2598469302,881301.2405264273,918754.0281746781,956836.7827706551,995559.8286499162,1034933.6571397965,1074968.9292371897,1115676.478327668,1157067.3129492095,1199152.6195992199,1241943.7655864097,1285452.3019279225,1329689.966293207,1374668.6859934076,1420400.5810196556,1466897.9671290345,1514173.3589797786,1562239.4733161412,1611109.232204752,1660795.7663208582,1711312.4182883531,1762672.7460721948,1814890.526424866,1867979.7583874539,1921954.6668473103,1976829.7061504035,2032619.563772948,2089339.164050429,2147003.6719660945,2205628.4969994063,2265229.2970366627,2325821.982341746,2387422.7195919156,2450047.9359767265,2513714.3233622173,2578438.842521048,2644238.7274308284,2711131.4896386107,2779134.9226967627,2848267.106668187,2918546.412703301,2989991.5076894164,3062621.358975014,3136455.2391667524,3211512.731004824,3287813.732314581,3365378.461036605,3444227.4603379527,3524381.6038018074,3605862.1007020613,3688690.5013602595,3772888.7025877303,3858478.953213716,3945483.8597026113,4033926.3918573298,4123829.888616019,4215218.063939145,4308115.012790227,4402545.217210992,4498533.5524944095,4596105.293452349],[-212281.08447655966,-193202.87023967097,-173796.94548639574,-154057.88722546946,-133980.18421252898,-113558.2355283713,-92786.3491347949,-71658.74040733348,-50169.53064456396,-28312.745553298853,-6082.313710234361,16527.93500166596,39524.270981391426,62913.06652358663,86700.79745790223,110894.04481494497,135499.49651803263,160523.94910274725,185974.30946354452,211857.59662832925,238180.9435612578,264951.59899476497,292176.92928991164,319864.4203273738,348021.6794280568,376656.43730452727,405776.55004342995,435390.00112002785,465504.90344388597,496129.5014381688,527272.1731515825,558941.4324041202,591145.9309668757,623894.4607771514,657195.9561878138,691059.4962535768,725494.3070531308,760509.7640484625,796115.3944815821,832320.8798100597,869136.0581801601,906570.9269405727,944635.645195574,983340.5363989628,1022696.0909891895,1062712.9690670923,1103402.0031149322,1144774.2007600826,1186840.7475819776,1229613.0099639252,1273102.5379901782,1317321.068389743,1362280.527525705,1407993.0344334508,1454470.9039065738,1501726.6496320115,1549772.9873748324,1598622.8382145474,1648289.3318312648,1698785.8098456631,1750125.829211334,1802323.1656611813,1855391.8172084193,1909346.007704177,1964200.1904497663,2019969.0518682492,2076667.5152333833,2134310.7444580486,2192914.147942617,2252493.382485508,2313064.3572538598,2374643.2378192423,2437246.450256506,2500890.68530789,2565592.9026130857,2631370.3350074785,2698240.492886577,2766221.1686417907,2835330.4411655804,2905586.6804283555,2977008.5521277627,3049615.022412896,3123425.362681207,3198459.1544537796,3274736.2943268865,3352276.9990019533,3431101.81039673,3511231.6008348563,3592687.5783203514,3675491.291894472,3759664.63707776,3845229.86139807,3932209.5700077293,4020626.731386813,4110504.683139758,4201867.137882364,4294738.189222432,4389142.317834815,4485104.397634374,4582649.702043536],[-224008.7721317811,-204944.7727151023,-185553.1531605752,-165828.49114608264,-145765.27610177558,-125357.90778838983,-104600.69485313445,-83487.85336246248,-62013.50531143369,-40171.67710895208,-17956.29803945939,4638.80130063137,27619.890591111965,50993.341401626356,74765.62883097585,98943.33317301795,123533.14160838048,148541.849923949,173976.3642594337,199843.70288189943,226150.99798850296,252905.49753847858,280114.5671134321,307785.6918082668,335926.47815175983,364544.65605792706,393648.0808083913,423244.7350668695,453342.7309247905,483950.31198055623,515075.8554514039,546727.8743191136,578915.0195097527,611646.0821087342,644929.9956101077,678775.8382027855,713192.8350926132,748190.360861632,783777.941864741,819965.2586651728,856762.1485075697,894178.6078316511,932224.7948253099,970911.0320184887,1010247.8089182456,1050245.78468642,1090915.7908586129,1132268.8341078006,1174316.0990512215,1217068.9511021285,1260538.939366782,1304737.7995881988,1349677.457135405,1395370.0300415924,1441827.832089953,1489063.3759487474,1537089.3763560161,1585918.7533558297,1635564.6355843598,1686040.3636097864,1737359.4933245678,1789535.799391763,1842583.2787459579,1896516.1541507898,1951348.8778111457,2007096.1350446483,2063772.8480105312,2121394.179497974,2179975.536774408,2239532.5754959863,2300081.2036781744,2361637.5857313825,2424218.1465597358,2487839.575725097,2552518.8316770466,2618273.146051051,2685120.028032791,2753077.268793866,2822162.945996859,2892395.42837214,2963793.380367073,3036375.7668701364,3110161.8580077407,3185171.234019428,3261423.7902093185,3338939.741975991,3417739.62992357,3497844.3250511894,3579275.03402739,3662053.3045468866,3746201.0307725137,3831740.458863168,3918694.1925908616,4007085.199043898,4096936.814423397,4188272.74993019,4281117.097745354,4375494.337105176,4471429.340473987,4568947.379811628],[-235927.898871599,-216878.45162876055,-197501.47759508726,-177791.5551466957,-157743.17441764835,-137350.73587830737,-116608.54889128497,-95510.83024425618,-74051.70265937364,-52225.19327855483,-30025.23212522082,-7445.650540790521,15519.820403447142,38877.55151993525,62634.017144093756,86795.796800168,111369.57689288654,136362.1524269029,161780.4287533022,187631.42334407545,213922.26759480266,240660.20865658415,267852.6112962775,295506.959787369,323630.8598304866,352232.0405047124,381318.3562498947,410897.788881063,440978.44963400904,471568.58124446473,502676.5600599204,534310.8981852648,566480.2456624662,599193.3926855747,632459.2718499473,666286.9604384073,700685.682743262,735664.8124254865,771233.8749113313,807402.549827734,844180.6734753156,881578.2413420014,919605.4106560363,958272.5029797952,997590.006844782,1037568.5804292301,1078219.0542770098,1119552.4340611673,1161579.90339075,1204312.8266624815,1247762.751957683,1291941.4139859504,1336860.7370743393,1382532.8382054323,1428970.0301031175,1476184.8243675667,1524189.9346598843,1572998.2799382643,1622622.9877439972,1673077.3975412974,1724375.0641094907,1776529.7609892574,1829555.4839834813,1883466.4547146894,1938277.1242371753,1994002.1767083972,2050656.5331177681,2108255.355074917,2166814.048657899,2226348.268323588,2286873.920878173,2348407.1695127194,2410964.437901831,2474562.4143676013,2539218.0561094927,2604948.5935024265,2671771.5344610047,2739704.6688751434,2808766.0731150256,2878974.114607823,2950347.456486808,3022905.0623153737,3096666.200883765,3171650.451084169,3247877.7068620734,3325368.182246057,3404142.416458751,3484221.2791062295,3565625.975452292,3648378.051775132,3732499.400809186,3818012.2672729585,3904939.2534859823,3993303.3250718866,4083127.8167548124,4174436.438246205,4267253.280225234,4361602.820413653,4457509.92974852,4554999.878649553],[-248038.36558956024,-229003.80372708163,-209641.81142244197,-189946.96778035385,-169913.7636694688,-149536.60030078562,-128809.78778364835,-107727.54365861486,-86283.99140692875,-64473.15893585165,-42288.977040464524,-19725.27784021315,3224.2068091700785,26565.84692919557,50306.116057828534,74451.59291526093,99008.96309552388,123985.02078586258,149386.670513198,175220.92891852488,201494.92655955208,228215.90974256094,255391.24238259182,283028.4078942486,311135.0111121542,339718.7802421951,368787.5688437759,398349.3578441723,428412.257584044,458984.50989654986,490074.4902191006,521690.7097389393,553841.8175727467,586536.6029815862,619783.9976200522,653593.0778223716,687973.0669243457,722933.3376224695,758483.4143704665,794632.9758146296,831391.8572667539,868770.053217672,906777.7198902112,945425.1778329278,984722.914555023,1024681.587203878,1065312.0252838873,1106625.2334199199,1148632.3941640668,1191344.8708472368,1234774.2104760113,1278932.146676232,1323830.6026821055,1369481.694374209,1415897.7333651627,1463091.2301345458,1511074.897213453,1559861.65242058,1609464.6221481357,1659897.1447015905,1711172.773691775,1763305.2814810458,1816308.6626840427,1870197.137725041,1924985.1564499848,1980687.4017977994,2037318.7935290975,2094894.4920143336,2153429.9020819417,2212940.6769286236,2273442.7220897586,2334952.1994748614,2397485.5314661656,2461059.4050824647,2525690.776208908,2591396.873894975,2658195.204718624,2726103.5572218,2795140.0064153103,2865322.918355431,2936670.954792924,3009203.0778969545,3082938.555051712,3157896.963731407,3234098.196451516,3311562.46579846,3390310.3095404813,3470362.595816899,3551740.528412299,3634465.6521130856,3718559.858149219,3804045.3897219333,3890944.8476205794,3979281.1959255785,4069077.7678047237,4160358.271399853,4253146.795807157,4347467.817151918,4443346.20476111,4540807.227430631],[-260340.027672305,-241320.66941624123,-221973.97989071335,-202294.53895736195,-182276.83824805787,-161915.27974336874,-141204.1743286047,-120137.74032675405,-98710.10200800141,-76915.28807512322,-54747.23012534203,-32199.76108693541,-9266.613631211454,14058.581440957496,37782.29683828936,61911.110444803024,86451.7070113679,111410.87987503014,136795.53270538687,162612.68127895193,188869.45528170583,215573.10014091176,242730.97888522502,270350.57403543894,298439.4895248718,327005.4526505461,356056.316055377,385600.0597424593,415644.793120513,446198.7570829247,477270.3261194376,508868.0104616571,541000.4582625951,573676.4578115281,606904.9397830914,640694.9795232986,675055.7993714081,709996.7710189796,745527.4179063218,781657.4176577723,818396.6045545416,855754.9720481818,893742.6753134606,932370.0338420176,971647.5340772138,1011585.8320915711,1052195.7563055356,1093488.3102508564,1135474.675377248,1178166.2139039144,1221574.4717163073,1265711.1813096406,1310588.2647779074,1356217.8368517996,1402612.2079842975,1449783.8874854934,1497745.58670707,1546510.2222782883,1596090.919391815,1646501.015143384,1697754.0619237984,1749863.8308650097,1802844.3153407776,1856709.7345239343,1911474.5369983255,1967153.4044300194,2023761.25529592,2081313.2486718374,2139824.788080507,2199311.5254017925,2259789.3648429993,2321274.466974222,2383783.2528268304,2447332.4080572184,2511938.887176474,2577619.917848279,2644393.0052529243,2712275.936522739,2781286.7852468346,2851443.9160476215,2922765.989229707,2995271.9655037103,3068981.1107827635,3143913.0010574088,3220087.5273467167,3297524.900727867,3376245.657446895,3456270.6641078396,3537621.1229468123,3620318.5771884355,3704384.9164874614,3789842.3824563595,3876713.574282049,3965021.454428726,4054789.3544340413,4146040.980795649,4238800.420951387,4333092.14935389,4428941.033643065,4526372.340913184],[-272832.8626968025,-253829.01189766463,-234497.93165486644,-214834.20261336525,-194832.31719490665,-174486.6781765623,-153791.5972468322,-132741.2935386293,-111329.89213887439,-89551.42257393803,-67399.8172715588,-44868.90999751631,-21952.434267669218,1355.9782654575538,25062.7994517521,49174.60630916804,73698.08271521423,98640.02112621046,124007.32432359876,149807.00718824216,176046.19850291032,202732.14278402342,229872.2021417017,257473.8581704381,285544.7138694131,314092.49559361325,343125.05503594177,372650.37124143494,402676.5526526426,433211.8391885904,464264.60435640183,495843.3573967293,527956.7454632125,560613.5558372883,593822.7181771831,627593.3068038675,661934.5430228489,696855.7974831385,732366.5925736371,768476.6048583188,805195.6675490106,842533.7730187683,880501.0753546744,919107.8929514177,958364.7111460469,998282.1848953369,1038871.141494469,1080142.5833403142,1122107.690738019,1164777.8247524416,1208164.5301048192,1252279.5381161938,1297134.769696345,1342742.3383816152,1389114.5534204117,1436263.922907936,1484203.1569705708,1532945.171001773,1582503.088947812,1632890.24664733,1684120.1952232542,1736206.704528768,1789163.7666478925,1843005.5994526313,1897746.6502148244,1953401.5992772542,2009985.3637821593,2067513.1014591916,2126000.2144733523,2185462.353335073,2245915.420870431,2307375.576256389,2369859.2391191768,2433383.093697926,2497964.093074245,2563619.4634699877,2630366.7086111754,2698223.614163305,2767208.2522359816,2837338.985959322,2908634.474132737,2981113.6759486273,3054795.855788773,3129700.5880991053,3205847.7623407133,3283257.588019292,3361950.599795781,3441947.6626753774,3523269.977281493,3605939.085212053,3689976.874480996,3775405.585045739,3862247.8144237734,3950526.5233953614,4040265.041799588,4131487.0744207716,4224216.706968523,4318478.412152204,4414297.05585328,4511697.90339228],[-285519.4785297585,-266531.4420091902,-247216.28054870444,-227568.57560319698,-207582.8204111975,-187253.41857346613,-166574.68260916462,-145540.83248888934,-124145.99414434168,-102384.19795382791,-80249.37720424356,-57735.366527805105,-34835.90031415946,-11544.611097036395,12144.972084771376,36239.4253529869,60745.431681272574,85669.78261441621,111019.38001458207,136801.23783552833,163022.48392503336,189690.3618565651,216812.23278925195,244395.5773584938,272447.99759620265,300977.218881842,329991.0919244706,359497.59477688116,389504.8348808994,420021.05114628305,451054.61606225837,482614.0378428721,514707.9626063814,547345.1765899716,580534.608398682,614285.3312912732,648606.5655019365,683507.6805991847,718998.1978821561,755087.7928157048,791786.2975031061,829103.7031993463,867050.1628638324,905635.9937538803,944871.6800593967,984767.8755801385,1025335.4064443142,1066585.2738717776,1108528.6569805266,1151176.91563803,1194541.5933578254,1238634.4202428397,1283467.3159742288,1329052.3928491084,1375401.9588659632,1422528.520859275,1470444.7876838143,1519163.673450439,1568698.3008117238,1619062.00430143,1670268.333726308,1722331.0576119765,1775264.1667033888,1829081.877521882,1883798.635976925,1939429.1210371128,1995988.2484585806,2053491.174572851,2111953.300134655,2171390.27423191,2231817.99825582,2293252.629936008,2355710.5874387817,2419208.5535306474,2483763.4798077815,2549392.590993676,2616113.389302942,2683943.6588765047,2752901.4702861137,2823005.1851106216,2894273.4605846466,2966725.2543221544,3040379.8291127095,3115256.757796131,3191375.9282133654,3268757.548235817,3347422.15087585,3427390.5994756706,3508684.092981158,3591324.171298028,3675332.7207331993,3760731.979522123,3847544.543445239,3935793.3715305477,4025501.7918495033,4116693.5074033057,4209392.6021027975,4303623.546842793,4399411.205674284,4496780.842071251],[-298397.8453934025,-279425.90176429914,-260126.9400496782,-240495.54253728956,-220526.20330560568,-200213.32680247352,-179551.2264013479,-158534.12293439615,-137156.1432022287,-115411.318459464,-93293.58287678589,-70796.77197773173,-47914.62105087959,-24640.763536566636,-968.7293879331555,23108.056594581343,47596.276454594685,72502.72079889197,97834.29054361326,123597.99868839234,149800.97211871087,176450.45343749458,203553.80282501644,231118.49992941786,259152.1457868917,287662.46477263,316657.306582802,346144.64824860403,376132.59618149046,406629.3882519873,437643.39590115217,469183.1262858417,501257.2244580239,533874.4755794124,567043.8071703047,600774.2913953725,635075.1473852855,669955.7435955135,705425.6002025439,741494.3915388898,778171.9485657052,815468.2613859812,853393.4817971778,891957.9258846093,931172.0766560442,971046.5867188678,1011592.2809985906,1052820.1595019577,1094741.4001233308,1137367.3614959503,1180709.5858884107,1224779.8021478981,1269589.9286889217,1315152.0765309604,1361478.5523837546,1408581.861781848,1456474.7122687662,1505170.0166327239,1554680.8961921348,1605020.684134997,1656202.9289105686,1708241.397675155,1761150.079792465,1814943.190390547,1869635.1739734258,1925240.7080919964,1981774.7070723162,2039252.3258033374,2097688.96358463,2157100.268036225,2217502.1390686138,2278910.732917754,2341342.466243217,2404814.0202915957,2469342.345125836,2534944.663922771,2601638.4773368086,2669441.5679349843,2738372.0047013615,2808448.1476131817,2879688.6522893966,2952112.4747141185,3025738.8760327445,3100587.4274264844,3176678.0150631014,3254030.8451261045,3332666.4489251105,3412605.6880846075,3493869.7598176273,3576480.20228179,3660458.900020519,3745828.0894902265,3832610.3646766385,3920828.6827972103,4010506.3700968884,4101667.127734239,4194335.0377612095,4288534.56919731,4384290.584201651,4481628.344339618],[-311470.57712491136,-292515.00764048647,-273232.5292964807,-253617.72523662436,-233665.09040268965,-213369.0301132081,-192723.85861977492,-171723.79764021817,-150362.97486839443,-128635.4224598282,-106535.07549383095,-84055.77041036868,-61191.24342231336,-37935.128902226454,-14280.957743490813,9777.844305138336,34247.95833080239,59136.173975585494,84449.39118262054,110194.62197016319,136378.99223385495,163009.74357825867,190094.23517666943,217639.9456615732,245654.475044735,274145.54666808364,303121.00918559264,332588.8385772626,362557.14019426866,393034.15083767846,424028.24086984666,455547.9163595892,487601.82126141083,520198.73963005166,553347.5978692491,587057.4670174303,621337.5650692505,656197.2593343067,691646.0688332715,727693.6667327986,764349.882818056,801624.7060058292,839528.2868970493,878070.9403701075,917263.1482153307,957115.5618120711,997639.0048471054,1038844.4760776544,1080743.1521376625,1123346.3903889577,1166665.7318176348,1210712.9039771783,1255499.8239771088,1301038.6015205139,1347341.541989238,1394421.1495783187,1442290.1304800662,1490961.39611965,1540448.066440518,1590763.4732436503,1641921.1635791492,1693934.9031918962,1746818.6800218064,1800586.7077606427,1855253.4294635474,1910833.521219816,1967341.8958810696,2024793.7068488759,2083204.3519223481,2142589.477207872,2202964.981088981,2264347.0182612417,2326752.0038302764,2390196.6174750496,2454697.807677068,2520272.796017796,2586939.081542204,2654714.4451937056,2723616.9543184144,2793664.9672411727,2864877.137913935,2937272.420639108,3010870.0748655205,3085689.6700628176,3161751.090672069,3239074.5411348147,3317680.551003295,3397589.9801290627,3478824.023936517,3561404.218778804,3645352.447378882,3730690.944356564,3817442.3018446844,3905629.4751913785,3995275.7887556814,4086404.941793506,4179041.014437248,4273208.4737698175,4368932.179996516,4466237.392711546],[-324737.5943776311,-305798.67138903437,-286532.95109424705,-266935.01751636073,-246999.36648408195,-226720.40421051532,-206092.44584953832,-185109.71402904205,-163766.33736079582,-142056.34892616002,-119973.68473827653,-97512.18217900558,-74665.57841125573,-51427.508765838575,-27791.505102674942,-3750.994145426899,20700.704209702788,45570.3786129395,70864.92800695566,96591.36340087256,122756.80967245251,149368.5073995369,176433.81471979222,203960.20922107622,231955.28986145277,260426.77891998715,289382.5239785451,318830.49993569404,348778.8110517557,379235.6930274565,410209.51511521055,441708.7822642233,473742.13729963196,506318.3631369467,539446.3850307392,573135.2728602355,607394.243450759,642232.662932365,677660.0491358633,713686.0740276421,750320.5661821035,787573.513294664,825455.064734187,863975.5341362071,903145.4020372995,942975.3185520656,983476.1060914453,1024658.7621256143,1066534.461990182,1109114.5617372305,1152410.601031602,1196434.3060939119,1241197.5926890946,1286712.5691638126,1332991.5395315553,1380047.0066069476,1427891.6751897319,1476538.45530024,1526000.4654646884,1576291.0360543323,1627423.7126769319,1679412.259622294,1732270.6633624092,1786013.1361081505,1840654.11942068,1896208.2878821031,1952690.5528235226,2010116.0661125316,2068500.2240006907,2127858.671033126,2188207.304018285,2249562.276062684,2311940.0006688097,2375357.15589827,2439830.688600863,2505377.8187118666,2572016.0436154353,2639763.1425794256,2708637.181259494,2778656.516275,2849839.7998572504,2922205.98457269,2995774.3281187355,3070564.398198031,3146596.0774688926,3223889.5685742376,3302465.3992516357,3382344.427521794,3463547.8469619346,3546097.192061533,3630014.3436632473,3715321.534489792,3802041.3547599553,3890196.75789072,3979811.0662926966,4070907.9772559544,4163511.5689294487,4257646.306394897,4353337.0478384895,4450609.050817232],[-338202.0959644271,-319280.10232631664,-300031.42541662056,-280450.65016340336,-260532.27330819226,-240270.70198484464,-219660.25227598194,-198695.14774631895,-177369.5179525877,-155677.39692933555,-133612.72165118298,-111169.33046984789,-88340.96152653452,-65121.2511388869,-41503.73216226045,-17481.83232443058,6951.127465517726,31801.93483608216,57077.48769928818,82784.79602459748,108930.98364101979,135523.29006847437,162569.0723774454,190075.80707926163,218051.09204602824,246502.64846133394,275438.32280196785,304866.08885171753,334794.04974634876,365230.44005215494,396183.627877153,427662.11701609613,459674.5491295124,492229.70595807116,525336.5115711542,559004.034652364,593241.490820878,628058.2449899705,663463.8137629458,699467.86786786,736080.2346298576,773310.9004840676,811170.0135279442,849667.886114367,888814.9974859315,928621.9964518116,969099.7041059704,1010259.1165899541,1052111.4078989765,1094667.9327328529,1137940.2293921704,1181940.0227212089,1226679.2270963357,1272169.9494633498,1318424.492422424,1365455.3573623297,1413275.2476442775,1461897.0718372855,1511333.9470033506,1561599.2020364855,1612706.381054059,1664669.2468422186,1717501.7843559012,1771218.204275412,1825832.946617698,1881360.6844068829,1937816.3274021796,1995215.0258852732,2053572.1745076485,2112903.416200064,2173224.6461421754,2234552.015797143,2296901.9370094053,2360291.0861676806,2424736.4084339123,2490255.122040401,2556864.7226530807,2624582.987806171,2693427.981406172,2763418.05830758,2834571.868961015,2906908.3641362498,2980446.7997178948,3055206.7415795033,3131208.070533868,3208470.987361783,3287016.0179219563,3366864.0183393266,3448036.1802782826,3530554.0362982494,3614439.465294441,3699714.698024585,3786402.3227247717,3874525.2908114027,3964106.9226764594,4055170.9135731636,4147741.3395952373,4241842.663750601,4337499.74213291,4434737.830187724],[-351863.37127669854,-332958.56754844345,-313727.19678409805,-294163.84483889176,-274263.0093896069,-254019.0985135301,-233426.42924495763,-212479.22610859224,-191171.61962955957,-169497.64481927105,-147451.23963776627,-125026.24343081447,-102216.39534238796,-79015.33270168351,-55416.5893844855,-31413.594147964148,-6999.668939623982,17831.9728214785,43088.22799061448,68776.10547119193,94902.72801679373,121475.33406244311,148501.27958417404,175988.03998922743,203943.21203588252,232374.51578408387,261289.7965770564,290697.0270550307,320604.309200105,351019.8764147167,381952.0956327412,413409.46946440544,445400.63837523316,477934.3829003086,511019.62589275464,544665.4348091455,578881.0240307467,613675.7572219465,649059.1497260858,685040.8710000745,721630.7470866237,758838.7631270329,796675.0659134388,835149.9664817865,874273.9427460302,914057.6421748749,954511.8845098836,995647.6645281559,1037476.1548482974,1080008.708781248,1123256.863226333,1167232.3416140606,1211947.0568944183,1257413.1145740733,1303642.815801222,1350648.6604996738,1398443.350552581,1447039.7930376818,1496451.103512343,1546690.6093524816,1597771.8531437656,1649708.5961269243,1702514.8216976186,1756204.7389628887,1810792.7863523078,1866293.6352883715,1922722.1939142873,1980093.6108812136,2038423.2791954554,2097726.8401278006,2158020.1871829974,2219319.470134234,2281641.099120761,2345001.7488107444,2409418.362630088,2474908.1570593882,2541488.62599709,2609177.5451939814,2677992.9767570253,2747953.273724943,2819077.0847161794,2891383.3586517717,2964891.3495508875,3039620.621404771,3115591.0531268707,3192822.84358145,3271336.5166933425,3351152.9266360793,3432293.263104977,3514779.0566725302,3598632.184229007,3683874.874509004,3770529.7137071164,3858619.651179714,3948168.0052400436,4039198.469043699,4131735.1165677123,4225802.408684062,4321425.199331038,4418628.741779214],[-365723.4189366719,-346836.0628679114,-327622.2581548904,-308076.59160160937,-288193.56184196554,-267967.57791875256,-247392.95784026268,-226463.9271137626,-205174.617255572,-183519.06427703053,-161491.2071469028,-139084.88622857002,-116293.8416925848,-93111.71190377465,-69532.03178269416,-45548.23114049644,-21153.63298798632,3658.548183097737,28895.208145851037,54563.35471194913,80670.10953357676,107222.7099346118,134228.5107701032,161694.98631637357,189629.7321907645,218040.4673021615,246935.03583251848,276321.40925048664,306207.68835618324,336602.1053595601,367513.0259914079,398948.95164817036,430918.52157079056,463430.51505886763,496493.85371904215,530117.6037502934,564310.9782650908,599083.339647705,634444.2019499433,670403.2333256474,706970.2585028308,744155.2612963493,781968.3871600307,820419.945779558,859520.4137065336,899280.4370351231,939710.8341200335,980822.5983390964,1022626.900899102,1065135.09368652,1108358.71216342,1152309.478310164,1196999.3036135712,1242440.2921039967,1288644.743440065,1335625.1560426117,1383394.2302783057,1431964.871694753,1481350.1943054167,1531563.5239284015,1582618.4015775386,1634528.5869075384,1687308.061713753,1740971.033488466,1795531.9390319313,1851005.4481226061,1907406.4672448165,1964750.1433758398,2023051.867832975,2082327.2801827192,2142592.27221012,2203862.9919530926,2266155.847799893,2329487.512651846,2393874.9281519805,2459335.308981873,2525886.1472246093,2593545.216799161,2662330.577964034,2732260.5818927116,2803353.875321443,2875629.405271974,2949106.4238468977,3023804.4931034553,3099743.490003491,3176943.6114418944,3255425.3793561803,3335209.645914469,3416317.5987883676,3498770.7665082174,3582591.0239035003,3667800.597629217,3754422.07178138,3842478.3935986077,3931992.8792570466,4022989.219755653,4115491.4868951016,4209524.139351094,4305112.028845561,4402280.406412402],[-379785.90981890913,-360916.268274546,-341720.2987337577,-322192.58897210064,-302327.6386035938,-282119.8576598163,-261563.5651465787,-240652.98757748236,-219382.25748408958,-197745.41190196038,-175736.3908331527,-153349.03568348754,-130577.08767520008,-107414.1862341296,-83853.86735124653,-59889.561917628394,-35514.594033586094,-10722.179290048545,14494.576977117453,40142.681460764725,66229.25468430505,92761.53283278923,119746.86961230193,147192.73813996022,175106.73286356707,203496.57151203952,232370.09707683674,261735.27982547856,291600.21934623225,321973.14662636607,352862.42616305966,384276.55810810486,416224.18044666364,448714.0712113066,481755.1507302965,515356.48391278205,549527.2825698357,584276.9077726624,619614.8722482133,655550.842813588,692094.6428480428,729256.2548055646,767045.8227668717,805473.6550321928,844550.2267552032,884286.1826195559,924692.3395567506,965779.6895085764,1007559.4022328388,1050042.8281539571,1093241.5012587563,1137167.1420390394,1181831.6604796173,1227247.1590952687,1273425.9360153435,1320380.4881175722,1368123.5142115587,1416667.9182737453,1466026.81273221,1516213.5218053125,1567241.5848926287,1619124.7600199862,1671877.027339073,1725512.592683601,1780045.891180182,1835491.5909184415,1891864.5966785112,1949180.0537179976,2007453.3516188827,2066700.128196553,2126936.273469012,2188177.9336910374,2250441.515451499,2313743.6898359386,2378101.3966550454,2443531.8487413432,2510052.536311986,2577681.231402978,2646435.9923726646,2716335.168477005,2787397.4045171943,2859641.6455622218,2933087.1417440753,3007753.4531313367,3083660.4546789927,3160828.3412566846,3239277.6327581257,3319029.1792888977,3400104.16643915,3482524.1206386695,3566310.914597101,3651486.7728301454,3738074.277274859,3826096.3729910683,3915576.373956094,4006537.968949841,4099005.227533512,4193002.606122737,4288554.954158541,4385687.520372927],[-394050.9592394647,-375199.28202455805,-356021.399499765,-336511.90043230576,-316665.2854366539,-296475.96555371815,-275938.2608075957,-255046.39873921522,-233794.51291659125,-212176.64142094832,-190186.7253093084,-167818.60705184448,-145066.02894461364,-121922.63149684062,-98381.95179253654,-74437.42182556028,-50082.36680884636,-25310.00345586799,-113.43823506124318,25514.334403738612,51580.43383104727,78092.09506870992,105056.67064919858,132481.63250551792,160374.57389075332,188743.2113284003,217595.38659369713,246939.0687270325,276782.35607853415,307133.478386221,338000.7988867988,369392.81646026904,401318.1678085639,433785.6296694875,466804.12106489134,500382.70558574516,534530.5937130766,569257.1451760605,604571.8713475203,640484.437678219,677004.6661687468,714142.5378819744,751908.1954949517,790311.9458915438,829364.2627962718,869075.7894506976,909457.3413311937,950519.9089112396,992274.6604670491,1034732.9449280074,1077906.2947723693,1121806.4289696782,1166445.255968679,1211834.8767341445,1257987.5878313382,1304915.8845596958,1352632.4641361812,1401150.2289301222,1450482.2897478528,1500641.96917123,1551642.8049484345,1603498.5534388474,1656223.1931125287,1709830.9281062272,1764336.191834115,1819753.6506577523,1876098.207613428,1933385.006198981,1991629.4342205434,2050847.1277014222,2111053.97485115,2172266.120099484,2234499.9681935823,2297772.188360396,2362099.718535007,2427499.769657132,2493989.830033781,2561587.6697732736,2630311.345288595,2700179.203872476,2771209.88834486,2843422.3417752795,2916835.812277877,2991469.857884847,3067344.351496049,3144479.4859070824,3222895.7789185233,3302614.078523534,3383655.5681803785,3466041.772167312,3549794.5610225946,3634936.1570705175,3721489.1400365,3809476.452748304,3898921.406930563,3989847.689089693,4082279.366492378,4176240.893238507,4271757.11643194,4368853.282445887],[-408520.55281486805,-389687.0860348671,-370527.53861452045,-351036.50033210835,-331208.4728221715,-311037.8681547572,-290519.00739227515,-269646.11912322533,-248413.33797252947,-226814.70308775338,-204844.1566017759,-182495.54207023792,-159762.6028843678,-136638.98065834446,-113118.21359102591,-89193.73480109149,-64858.87063636421,-40106.83895537956,-14930.747381896013,10676.408468571957,36721.74679039675,63212.50141872582,90156.02368867677,117559.78432514053,145431.3753622251,173778.51209349278,202609.03505317448,231930.91202948033,261752.24010908464,292081.247755161,322926.2969180825,354295.8851799101,386198.6479329262,418643.3605934577,451638.9408499424,485194.45094788517,519319.100010684,554022.2463976033,589313.4000991555,625202.2251712803,661698.5422071053,698812.3308492841,736553.7323417682,774933.0521223291,813960.7624562657,853647.5051126734,894004.0940820742,935041.518338595,976770.9446454651,1019203.7204053246,1062351.3765557795,1106225.6305117048,1150838.3891530125,1196201.751861336,1242328.0136043795,1289229.6680694353,1336919.4108466152,1385410.142663541,1434714.9726698468,1484847.2217755446,1535820.426041689,1587648.340125108,1640344.940777718,1693924.4304024035,1748401.2406635897,1803790.0361570532,1860105.71813712,1917363.4283033197,1975578.5526469748,2034766.7253599262,2094943.832803419,2156126.017541948,2218329.682440259,2281571.4948255876,2345868.39071582,2411237.5791158387,2477696.546379997,2545263.0606459826,2613955.1763379825,2683791.238741598,2754789.8886511307,2826970.0670917816,2900351.0201144973,2974952.303669212,3050793.788554296,3127895.66544444,3206278.4499996966,3285962.988052906,3366970.4608820197,3449322.3905647947,3533040.645418615,3618147.4455263214,3704665.3683510944,3792617.354437493,3882026.7132057687,3972917.1288365554,4065312.666249171,4159237.7771743396,4254717.306324735,4351776.497660156],[-423196.8286688959,-404381.81556788424,-385240.84843338944,-365768.5180735589,-345959.32716199884,-325807.6888171118,-305307.92515902873,-284454.2658434324,-263240.84657197935,-241661.7075786125,-219710.7920923161,-197381.94477466634,-174668.91013273527,-151565.33090656856,-128064.74643099122,-104160.59097084543,-79846.19203042914,-55114.768635140965,-29959.42958610435,-4373.171686784364,21651.122058534995,48120.68427554914,75042.86507902946,102425.13396250829,130275.08171762899,158600.42238427978,187408.99523168616,216708.76677163295,246507.8328028242,276814.4204888209,307636.8904686186,338983.73900103546,370863.60014312016,403285.2479638688,436257.59879217646,469789.71350167086,503890.79983142647,538570.2147438107,573837.466819762,609702.218692821,646174.289520795,683263.6574979585,720980.4624067042,759335.0082099359,798337.7656846493,837999.375098076,878330.6489251778,919342.5746107008,961046.3173745289,1003453.2230618752,1046574.8210386871,1090422.8271338148,1135009.1466266506,1180345.8772836667,1226445.312442618,1273319.9441459377,1320982.4663237992,1369445.7780286798,1418722.9867197112,1468827.4116008985,1519772.5870116337,1571572.2658712706,1624240.423178306,1677791.2595660791,1732239.2049132132,1787598.9220132527,1843885.3103017136,1901113.5096425654,1959298.904174677,2018457.126220359,2078604.060254105,2139755.8469362473,2201928.8872098103,2265139.84646257,2329405.658755047,2394743.5311166737,2461170.9479081007,2528705.675254876,2597365.7655504416,2667169.5620308565,2738135.703421912,2810283.128661136,2883631.0816924646,2958199.1163392877,3034007.101253714,3111075.2249442614,3189424.0008847043,3269074.272701302,3350047.2194449347,3432364.36094559,3516047.563251989,3601119.0441572024,3687601.378813346,3775517.5054323957,3864890.7310802983,3955744.737561456,4048103.5873968084,4141991.7298963354,4237434.007329392,4334455.661189656],[-438081.9621973408,-419285.6431523921,-400163.49857226654,-380710.12031319423,-360920.01210624806,-340787.5881367759,-320307.17160142283,-299472.9932420526,-278279.18985624146,-256719.80278370064,-234788.77636912419,-212479.95639982447,-189787.08851873945,-166703.81661200197,-143223.6811708454,-119340.1176269413,-95046.45466193277,-70335.91248918301,-45201.601108495845,-19636.518532847753,6366.451013050508,32814.53892343654,59715.09407048253,87075.5846938903,114903.60032012477,143206.8537124258,171993.18285182305,201270.55295021553,231047.0584946426,261330.92532509216,292130.5127449762,323454.3156653985,355310.96678344905,387709.23879580153,420658.0466465452,454166.44981190306,488243.65462082066,522899.0166126988,558142.0429325306,593982.3947648301,630429.8898051574,667494.5047721944,705186.3779592696,743515.8118266191,782493.2756348422,822129.4081209023,862435.0202155046,903421.0978049992,945098.8045366071,987479.4846684728,1030574.6659649247,1074396.06263852,1118955.5783375197,1164265.3091823054,1210337.546849421,1257184.7817048302,1304819.7059868306,1353255.2170404743,1402504.4206017433,1452580.6341356635,1503497.3902266095,1555268.440022771,1607907.7567351523,1661429.5391931187,1715848.215454684,1771178.446475983,1827435.1298381558,1884633.4035336787,1942788.6498126222,2001916.4990910473,2062032.8339195624,2123153.793016821,2185295.7753661936,2248475.4443776514,2312709.732115589,2378015.8435948156,2444411.261142672,2511913.7488325387,2580541.356986641,2650312.426750596,2721245.594740331,2793359.7977639055,2866674.2776159914,2941208.585950726,3016982.589230776,3094016.4737548414,3172330.750766298,3251946.261640232,3332884.1831553676,3415166.032848343,3498813.674453143,3583849.323426502,3670295.5525623853,3758175.2976925913,3847511.863480659,3938328.9293061164,4030650.5552423536,4124501.188128908,4219905.66774157,4316889.233057107],[-453178.1648085718,-434400.777326306,-415297.6946525733,-395863.5097092936,-376092.727302804,-355979.7627033787,-335518.9402023533,-314704.4916461194,-293530.5549467404,-271991.17256844183,-250080.2899905725,-227791.75414532213,-205119.311830848,-182056.60809891485,-158597.18461692566,-134734.47800334264,-110461.81813733233,-85772.42644060403,-60659.41413225187,-35115.78045557905,-9134.410876763519,17291.92474570847,44170.57401993219,71509.00391005632,99314.8026554226,127595.68172045494,156359.47777556814,185614.1547101338,215367.80567662907,245628.65516834683,276405.06112973765,307705.51710056653,339538.65439408645,371913.24431052804,404838.20038478216,438322.5806710257,472375.59006317356,507006.5826524808,542225.0641225665,578040.6941832032,614463.2890417196,651502.8239149491,689169.4355806145,727473.424969461,766425.259798584,806035.5772472834,846315.1866743127,887275.0723796394,928926.3964095409,971280.5014064994,1014348.9135043428,1058143.3452701233,1102675.698691461,1147958.0682128142,1194002.7438193662,1240822.2141701395,1288429.169780763,1336836.506257718,1386057.3275823987,1436104.9494490253,1486992.902654834,1538734.9365443555,1591345.0225082615,1644837.3575387627,1699226.3678397187,1754526.7124959466,1810753.2871999433,1867921.2280380465,1926045.9153365237,1985142.9775697961,2045228.2953288215,2106318.0053544147,2168428.5046337508,2231576.454562057,2295778.78517027,2361052.6994208326,2427415.6775696296,2494885.481599319,2563480.1597219324,2633218.050953253,2704117.7897595223,2776198.3107790784,2849478.8536166423,2923978.967715975,2999718.517308731,3076717.68644177,3154996.9840855952,3234577.2493211823,3315479.6566116903,3397725.721156551,3481337.304330661,3566336.6192095866,3652746.236183837,3740589.088659227,3829888.478850576,3920668.0836657397,4012951.960683251,4106764.554224373,4202130.701522983,4299075.638990063],[-468487.6940928523,-449729.47483965557,-430645.6925828849,-411230.94132796477,-391479.72697449883,-371386.4658958884,-350945.483496543,-330151.01274598157,-308997.19268952636,-287478.0669349164,-265587.58211533935,-243319.58632726828,-220667.82754367287,-197625.95200177422,-174187.50256516458,-150345.9170593461,-126094.52658145875,-101426.5537822533,-76335.11112100491,-50813.19909246988,-24853.704425655305,1550.6017466501798,28407.06574664265,55723.15324087115,83506.45115925814,111764.66964488756,140505.6440347829,169737.33687276556,199467.83995348774,229705.37640002393,260458.30277408706,291735.11122007295,323544.43164311326,355895.0339224236,388795.8301588907,422255.87695955415,456284.37775793043,490890.68517149333,526084.3033965724,561874.8906420018,598272.2616003994,635286.3899599744,672927.4109557876,711205.6239617402,750131.4951237657,789715.6600355508,829968.9264556335,870902.2770690431,912526.8722922215,954854.0531228066,997895.344034607,1041662.4559193382,1086167.289073824,1131421.9362361054,1177438.6856691856,1224230.0242939878,1271808.6408719637,1320187.4292391883,1369379.4915902675,1419398.1418160764,1470256.9088938008,1521969.5403310517,1574550.005664548,1628012.5000153407,1682371.4476987594,1737641.5058935618,1793837.568368474,1850974.769268175,1909068.4869592283,1968134.3479381092,2028188.230799423,2089246.270269048,2151324.8613004424,2214440.6632361733,2278610.604035369,2343851.8845693385,2410181.982983322,2477618.6591296233,2546179.9590700213,2615884.219649953,2686750.0731450226,2758796.4519824497,2832042.5935351327,2906508.044994128,2982212.6683172886,3059176.6452563675,3137420.4824652467,3216965.0166865615,3297831.420023189,3380041.205292129,3463616.23146348,3548578.7091854187,3634951.2063982505,3722756.6540345605,3812018.351812672,3902759.97412045,3995005.5759927277,4088779.599183103,4184106.8783336068,4281012.647238945],[-484012.8573738588,-465274.04382778273,-446209.80134541716,-426814.7250343567,-407083.32190635044,-387010.0094570103,-366589.1142231168,-345814.8703168214,-324681.41793644195,-303182.80185316387,-281312.9698741841,-259065.77128064516,-236434.95524094766,-213414.16919861105,-189996.95723449555,-166176.7584024507,-141946.90503914212,-117300.62104613986,-92231.02014493989,-66731.10410402925,-40793.76093775593,-14411.763075996423,12422.23449452105,39715.696119457716,67476.20739577198,95711.4771213932,124429.33927587792,153637.75503315474,183344.81480543152,213558.74032066623,244287.88673266373,275540.7447649967,307325.9428889265,339652.24953663326,372528.5753486734,405963.9754583235,439967.6518117897,474548.95552554843,509717.3892811036,545482.6097585196,581854.43010755,618842.822459317,656457.9204774238,694710.0219498209,733609.5914218312,773167.2628717546,813393.8424278114,854300.3111296257,895897.8277330205,938197.7315596433,981211.5453917813,1024950.9784139667,1069427.9292000069,1114654.48874895,1160642.9435686693,1207405.7788086645,1254955.6814425113,1303305.5435017985,1352468.4653598648,1402457.759069384,1453286.9517522326,1504969.7890434403,1557520.2385897066,1610952.4936044489,1665280.9764775776,1720520.342444477,1776685.483312361,1833791.5312460898,1891853.8626139106,1950888.1018953172,2010910.125649074,2071936.0665461773,2133982.317465974,2197065.53565751,2261202.64696678,2326410.8501321627,2392707.6211459776,2460110.717687406,2528638.1836247337,2598308.3535892977,2669139.8576218197,2741151.6258936357,2814362.8935005683,2888793.205335172,2964462.421035183,3041390.7200104017,3119598.6065507075,3199106.9150124565,3279936.8150897725,3362109.8171681836,3445647.7777633863,3530572.9050459974,3616907.764455362,3704675.284399503,3793898.7620483134,3884601.8692171546,3976808.6583440118,4070543.5685610855,4165831.431864187,4262697.479376754],[-499755.98679378955,-481036.8131666413,-461992.34649464546,-442617.18300547625,-422905.83084112743,-402852.70863772184,-382452.1440829204,-361698.37245021085,-340585.53510981356,-319107.67801548215,-297258.75016775215,-275032.60205198056,-252422.98405175377,-229423.54483686597,-206027.82972564548,-182229.27902070037,-158021.22631887067,-133396.8967934195,-108349.40544916107,-82871.75534964679,-56956.83581614867,-30597.42059743381,-3786.1660112098325,23484.39094499848,51221.834512641886,79433.87212170195,108128.33637122181,137313.187041929,166996.5131400309,197186.5349745385,227891.60626726644,259120.21629659832,290880.99207530054,323182.70056361286,356034.2509165774,389444.6967682552,423423.23855178733,457979.2258566194,493122.1598231108,528861.6955759327,565207.6446950596,602169.9777273063,639758.8267373024,677984.4878992026,716857.4241295946,756388.267762925,796587.823268312,837467.0700108886,879037.1650564233,921309.446020812,964295.4339647379,1008006.836335111,1052455.5499519533,1097653.6640442011,1143613.4633331276,1190347.4311649813,1237868.2526932824,1286188.818112587,1335322.2259420548,1385281.7863628678,1436081.0246079394,1487733.6844056747,1540253.7314783253,1593655.3570968616,1647952.9816905714,1703161.2585158485,1759295.0773823934,1816369.5684388233,1874400.1060182417,1933402.312545908,1993392.062507073,2054385.4864797576,2116398.975230667,2179449.183876349,2243553.0361102605,2308727.7284979858,2374990.734838608,2442359.8105974263,2510852.9974079775,2580488.6276458073,2651285.329074572,2723262.029567064,2796437.961898874,2870832.6686204234,2946466.0070051914,3023358.1540763834,3101529.6117147207,3181001.211844624,3261794.1217052666,3343929.849203988,3427430.2483548312,3512317.5248030564,3598614.241438722,3686343.3240963807,3775528.0673480574,3866192.1403865726,3958359.593002488,4052054.861655422,4147302.77564321,4244128.563365663],[-515721.3599448933,-497022.0722586905,-477997.62938267866,-458642.6286842573,-438951.5794555353,-418918.90149323037,-398538.9236561905,-377805.88239977136,-356713.92028686614,-335257.0844747897,-313429.32517864276,-291224.49410946085,-268636.34288772685,-245658.5214314554,-222284.5763186249,-198507.9491230538,-174321.9747244462,-149719.87959069805,-124694.78003314836,-99239.68043387751,-73347.4714448019,-47010.92815756891,-20222.708245130256,7024.6499272736255,34738.72922078031,62927.235672333045,91598.00047509186,120758.98199093784,150418.26779412315,180584.0767484887,211264.7611173184,242468.80870698672,274204.84504462755,306481.63559111,339308.08798820805,372693.25434270385,406646.3335463065,441176.6736327219,476293.7741721482,512007.2887045101,548327.0272103213,585262.9586220675,622825.2133750436,661024.0859989193,699870.0377504737,739373.6992889079,779545.8733924816,820397.5377197016,861939.847613808,904184.1389520878,947141.9310403864,990824.9295543907,1035245.0295263445,1080414.3183806946,1126345.079017349,1173049.7929441421,1220541.1434589766,1268832.0188833957,1317935.5158460017,1367864.942619697,1418633.8225112064,1470255.8973046744,1522745.1307598308,1576115.7121666924,1630382.0599549704,1685558.82536269,1741660.8961621772,1798703.4004455127,1856701.7104698936,1915671.446565118,1975628.4811012216,2036588.942521059,2098569.219436033,2161585.9647870474,2225656.100071362,2290796.8196376157,2357025.595046998,2424360.1795057496,2492818.612366974,2562419.2237041984,2633180.6389572676,2705121.783653139,2778261.888199341,2852620.4927557735,2928217.4521827204,3005072.9410672793,3083207.4588309443,3162641.8349155337,3243397.2340540355,3325495.161623766,3408957.4690846796,3493806.35950365,3580064.39316779,3667754.493283917,3756899.951771277,3847524.4351446307,3939651.9904909236,4033307.0515403715,4128514.4448353443,4225299.395993875],[-531910.1284266848,-513230.95943303534,-494226.7748834024,-474892.17330213205,-455221.66514884925,-435209.67139846133,-414850.5220987813,-394138.4549050196,-373067.6135909043,-351632.0465356887,-329825.7051876164,-307642.4425021666,-285076.0113556958,-262120.06293362705,-238768.14509299165,-215013.700698419,-190850.06593229994,-166270.4685772017,-141268.0262712536,-115835.74473553104,-89966.51597330626,-63653.11644004518,-36888.205185099505,-9664.321962817805,18026.114685995504,46190.80938337045,74837.59189448948,103974.41914002085,133609.3772396769,163750.68358927662,194406.68897047848,225585.8796942928,257296.87977861287,289548.4531610571,322349.50594600826,355709.08868856286,389636.398714317,424140.78247631015,459231.73794933944,494918.9170630921,531212.1281728342,568121.3385706556,605656.6770361573,643828.4364278617,682647.0763158067,722123.2256566926,762267.6855103667,803091.4318008644,844605.6181207167,886821.5785801241,929750.8307012997,973405.0783595834,1017796.2147699995,1062936.3255227294,1108837.6916662105,1155512.792839425,1202974.3104538622,1251235.13092694,1300308.3489652248,1350207.2709015086,1400945.4180841427,1452536.5303204674,1504994.5693747855,1558333.7225228706,1612568.406161215,1667713.2694754351,1723783.1981660814,1780793.3182338937,1838758.9998249724,1897695.8611380595,1957619.7723919966,2018546.8598580814,2080493.5099556085,2143476.3734126203,2207512.369492541,2272618.690289012,2338812.805086826,2406112.464794244,2474535.7064445727,2544100.8577695154,2614826.5418448313,2686731.6818109457,2759835.505666134,2834157.5511381375,2909717.6706319298,2986536.0362559147,3064633.1449292754,3144029.8235676745,3224747.2343538515,3306806.880090547,3390230.60963857,3475040.623440817,3561259.4791353755,3648910.0972547107,3738015.767018144,3828600.1522146873,3920687.2971794503,4014301.632864467,4109467.983007314,4206211.570394351],[-548324.5408443965,-529665.7183736084,-510682.02168237895,-491368.05046817183,-471718.31637446943,-451727.2415708923,-431389.15731091495,-410698.3024664812,-389648.8220392149,-368234.7656475466,-346450.08599028224,-324288.6372849692,-301744.17368164146,-278810.34765112586,-255480.70834769378,-231748.6999451723,-207607.6599472277,-183050.8174698865,-158071.29149704357,-132662.08910798468,-106816.10367673077,-80526.11304217228,-53784.7776498911,-26584.638663386693,1081.88395433384,29222.49338992522,57845.017959871795,86957.41312272241,116567.76352248364,146684.28506557457,177315.3270304117,208469.3742107849,240155.04909326485,272381.11406988604,305156.4736850613,338490.17691938207,372391.4195092672,406869.54630374955,441934.0536586691,477594.59186962666,513860.9676425399,550743.1466047454,588251.2558555198,626395.5865573399,665186.5965683316,704634.913117242,744751.3355197764,785546.8379394838,827032.5721918934,869219.8705935341,912120.2488560947,955745.4090273988,1000107.2424777667,1045217.8329353447,1091089.4595690027,1137734.6001204753,1185165.9340861342,1233396.3459502333,1282438.9284679594,1332306.9860023127,1383014.0379132784,1434573.8220010444,1487000.2980038167,1540307.6511521249,1594510.295777871,1649622.878982562,1705660.2843629138,1762637.635795911,1820570.3012837858,1879473.8968610873,1939364.2905619484,2000257.6064522315,2062170.2287248708,2125118.80586039,2189120.254853364,2254191.7655070117,2320350.804793925,2387615.121288161,2456002.749666617,2525532.015282134,2596221.538808965,2668090.2409631154,2741157.347295341,2815442.3930625087,2890965.228175126,2967746.022223318,3045805.2695839284,3125163.7946059857,3205842.7568810424,3287863.656595868,3371248.3399702664,3456019.0047808364,3542198.2059738194,3629808.8613640424,3718874.257427127,3809418.055182072,3901464.2961673797,3995037.4085116317,4090162.213101829,4186863.929846355],[-564967.1007951547,-546328.8494337691,-527365.8668336506,-508072.75388078054,-488444.0234180627,-468474.09882556414,-448157.3125783289,-427487.9047810859,-406460.0216796007,-385067.71414788696,-363304.93615190336,-341165.54318802315,-318643.290696898,-295731.8324518716,-272424.7189217708,-248715.395607105,-224597.20135047846,-200063.36661925004,-175107.01176113728,-149721.1452318721,-123898.66179467714,-97632.3406905036,-70914.84378000256,-43738.71365486877,-16096.371719555696,12019.883757756092,40617.87962520216,69705.56985932356,99291.03760834504,129382.49726897618,159988.29659587052,191116.9188448575,222776.98495020554,254977.25573718036,287726.6341687951,321034.1676294897,354909.0502446089,389360.62523705745,424398.3873213469,460031.98513638205,496271.2237158902,533126.0669993486,570606.6403823523,608723.2333077379,647486.301897839,686906.4716293388,726994.5400494239,767761.4795364933,809218.4401041674,851376.7522501005,894247.9298500095,937843.6730984447,982175.871495012,1027256.6068795004,1073098.1565146246,1119712.9962179868,1167113.8035436566,1215313.4610152512,1264325.0594087895,1314161.9010894082,1364837.5034003197,1416365.6021058734,1468760.1548891454,1522035.3449060882,1576205.5843943588,1631285.5183413643,1687290.0282096711,1744234.2357218626,1802133.5067053034,1861003.4549990369,1920859.946420802,1981719.102799008,2043597.3060678425,2106511.202427589,2170477.706570869,2235514.005977015,2301637.565272588,2368866.1306632375,2437217.734434856,2506710.6995264683,2577363.6441754457,2649195.4866376435,2722225.4499801425,2796473.066952386,2871958.1849334715,2948700.9709578836,3026721.916822344,3106041.8442710433,3186681.9102657335,3268663.612338158,3352008.7940276195,3436739.650404495,3522878.733682814,3610448.958918933,3699473.6098035006,3789976.344543752,3881981.2018393883,3975512.606952854,4070595.377877417,4167254.731599856],[-581842.3842012933,-563224.9413528143,-544282.9120453092,-525010.8983705764,-505403.41438844043,-485454.8847070816,-465159.64304097043,-444511.9307456985,-423505.8953294193,-402135.5889401918,-380394.9668297777,-358277.8857922333,-335778.1025778912,-312889.2722818919,-289604.9467070815,-265918.5727003489,-241823.4904631537,-217312.93183429958,-192380.0185456709,-167017.76045001787,-141219.05372054433,-114976.67902129097,-88283.2996492181,-61131.45964569552,-33513.58187834127,-5421.966092121089,23151.213070553727,52213.9080817888,81774.20057133306,111840.30340325949,142420.5627852711,173523.46041179937,205157.61564111803,237331.78770775744,270054.87796911807,303335.9321889868,337184.1428568866,371608.8515445611,406619.5512998793,442225.88907947065,478437.668218995,515264.85094390786,552717.5609196881,590806.0858427833,629540.8800727122,668932.5673067479,708991.9432959191,749729.9786055472,791157.8214190688,833286.8003867115,876128.427519321,919694.4011289801,963996.6088150381,1009047.1304990731,1054858.2415074566,1101442.4157031176,1148812.3286669597,1196980.8609307506,1245961.1012597838,1295766.3499894093,1346410.1224138057,1397906.152228843,1450268.3950294852,1503511.0318637297,1557648.472841256,1612695.3608012598,1668666.575037652,1725577.2350837323,1783442.7045567376,1842278.595064507,1902100.770172336,1962925.3494346933,2024768.7124901363,2087647.5032214127,2151578.6339814644,2216579.2898875945,2282666.9331817618,2349859.307662194,2418174.4431843436,2487630.660233515,2558246.5745698647,2630041.1019482967,2703033.462910964,2777243.187658166,2852690.1209953865,2929394.4273588015,3007376.5959218657,3086657.4457802866,3167258.13122188,3249200.1470787292,3332505.3341645026,3417195.8847976965,3503294.34841396,3590823.6372644906,3679807.0322077456,3770268.1885914393,3862231.142228165,3955720.3154653884,4050760.5233532405,4147376.979906932],[-598952.9774535086,-580356.5772349538,-561435.7370786654,-542185.0602987813,-522599.06218872836,-502672.1686016824,-482398.7145086017,-461772.94253316964,-440789.00146335457,-419440.94473886583,-397722.72891507065,-375628.2121017012,-353151.1523769554,-330285.2061761413,-307023.9266547144,-283360.7620247153,-259289.05386544135,-234802.0354063243,-209892.82978281425,-184554.4482642773,-158779.7884537161,-132561.6324582845,-105892.64503151062,-78765.37168489536,-51172.2367699102,-23105.541529222857,5442.537883074256,34479.9524132777,64014.78215106507,94055.23840603512,124609.66581686353,155686.54449427035,187294.49219797808,219442.26654897933,252138.76727600605,285393.0384988964,319214.2710477803,353611.80481943465,388595.1311710016,424173.8953525035,460357.89897691924,497157.10253081727,534581.6279244018,572641.7610823051,611347.9545755363,650710.8302959732,690741.1821722123,731449.9789299285,772848.3668955348,814947.6728446789,857759.4068958932,901295.2654510424,945567.1341811586,990587.0910612561,1036367.4094526768,1082920.5612346856,1130259.2199857126,1178396.2642160284,1227344.780650237,1277118.0675636106,1327729.6381706567,1379193.2240677965,1431522.778730561,1484732.4810673394,1538836.739027815,1593850.1932705988,1649787.7208882289,1706664.4391916224,1764495.7095544206,1823297.1413194467,1883084.5957653269,1943874.1901380096,2005682.3017454455,2068525.57211748,2132420.9112316174,2197385.5018069814,2263436.8036643527,2330592.5581575856,2398870.7926743,2468289.8252082807,2538868.2690042527,2610625.037277515,2683579.348006211,2757750.7288019694,2833159.021856715,2909824.388967891,2987767.3166448213,3067008.6212934116,3147569.454485755,3229471.30831202,3312736.020817505,3397385.7815256286,3483443.137049952,3570930.996792337,3659872.6387343523,3750291.7153190114,3842212.2594261,3935658.6904418976,4030655.8204266676,4127228.860376781],[-616299.9285763283,-597724.7892814704,-578825.358093174,-559596.2395615651,-540031.9502274714,-520126.9172029947,-499875.47672965075,-479271.8727134436,-458310.25523652066,-436984.6790447603,-415289.10201179585,-393217.3835778523,-370763.2831639529,-347920.4585607124,-324682.4642914601,-301042.749948832,-276994.6585055506,-252531.4245974454,-227646.1727794467,-202331.9157536202,-176581.55256902333,-150387.86679234775,-123743.52465026709,-96641.07314119069,-69072.93811737979,-41031.42233631085,-12508.703481059521,16503.16785139474,46012.270192997996,76026.81328103668,106555.14016713924,137605.72936009173,169187.19700266374,201308.29908371344,233977.9336845132,267205.14326195675,300999.1169676066,335369.1930038673,370324.86101757595,405875.76453233464,442031.70341843995,478802.6364033518,516198.6836215765,554230.1292052853,592907.4239161054,632241.187819438,672242.2130001397,712921.4663227131,754290.0922347887,796359.4156154369,839140.9446686588,882646.3738636384,926887.5869204137,971876.6598444753,1017625.8640089624,1064147.669286059,1111454.7472280343,1159559.9742997428,1208476.4351609217,1258217.4260023143,1308796.4579340676,1360227.2604281872,1412523.7848155473,1465700.2078394261,1519770.9352637385,1574750.6055404744,1630654.0935344675,1687496.5143076514,1745293.226963176,1804059.8385516724,1863812.2080376307,1924566.4503307075,1986338.9403801933,2049146.3173346578,2113005.48876751,2177933.634970704,2243948.2133145696,2311066.9626789754,2379307.907953782,2448689.364611022,2519229.943349411,2590948.5548137375,2663864.4143869085,2737997.047060314,2813366.2923803907,2889992.3094736105,2967895.5821525482,3047096.9241003576,3127617.484140082,3209478.751586305,3292702.561681913,3377311.101120826,3463326.913659743,3550772.9058160186,3639672.352658791,3730048.9036904443,3821926.588821637,3915329.8244407335,4010283.419581015,4106812.582182505],[-633888.1132509708,-615334.4672925504,-596456.6791761698,-577249.3547036961,-557707.0116792893,-537824.0784900603,-517594.8926643566,-497013.6994069237,-476074.65011070506,-454771.8008445273,-433099.11081725475,-411050.44081674586,-388619.55162418354,-365800.10240298137,-342585.6490620503,-318969.64259250555,-294945.4273785667,-270506.2394807243,-245645.20489185164,-220355.33776537539,-194629.53861526633,-168460.59248681087,-141841.16709909774,-114763.81095690769,-87220.95143297082,-59204.89281946933,-30707.814348554006,-1721.768180812709,27761.322637449717,57749.66625043377,88251.60410174611,119275.61307703634,150830.30768067949,182924.44224774162,215566.9131901299,248766.7612796626,282533.17396695353,316875.487737424,351803.19050474325,387325.92404299043,423453.4864564203,460195.83468977246,497563.0870779855,535565.5259366494,574213.6001936169,613517.9280631694,653489.2997615021,694138.680266751,735477.2121223004,777516.2182849084,820267.2050180233,863741.8648318409,907952.0794688072,952909.9229380165,998627.664597238,1045117.77228409,1092392.915496916,1140465.9686270824,1189350.014241071,1239058.3464164112,1289604.4741298798,1341002.124699742,1393265.247282573,1446408.0164265651,1500444.8356795784,1555390.3412563298,1611259.4057629942,1668067.1419812043,1725828.9067119826,1784560.3046817519,1844277.1925085126,1904995.6827329292,1966732.1479125533,2029503.2247812536,2093325.8184745286,2158217.106822999,2224194.5447119833,2291275.8685124246,2359479.100581115,2428822.5538326087,2499324.8363835094,2571004.856271596,2643881.8262476167,2717975.2686454044,2793305.0203281846,2869891.2377132885,2947754.4018779774,3026915.3237436395,3107395.1493448415,3189215.365180698,3272397.803651368,3356964.6485805004,3442938.440826714,3530342.0839811713,3619198.8501584455,3709532.385877662,3801366.7180372654,3894726.2599841426,3989635.817680528,4086120.59596555],[-651721.0140866702,-633189.0965990605,-614333.1884008868,-595147.8965617301,-575627.7401652063,-555767.1488897507,-535560.4615670124,-515001.9247171404,-494085.69106069347,-472805.8180064601,-451156.26611572574,-429130.8975413486,-406723.47444223124,-383927.65737232775,-360737.0036440622,-337144.9656651474,-313144.8892496247,-288730.011901153,-263893.4610692584,-238628.25237764,-212927.28782429057,-186783.3539524076,-160189.11999301706,-133137.13597700885,-105619.83081751992,-77629.51036159601,-49158.35541084129,-20198.41971003497,9258.372095440282,39220.22653567302,69695.48342497298,100692.61800436443,132220.24311810895,164287.1114255092,196902.11764691328,230074.3008466349,263812.8467516531,298127.0901074931,333026.5170714748,368520.76764473435,404619.6381418088,441333.08370078565,478671.2208328452,516644.33001253544,555262.8583092245,594537.4220610745,634478.809590366,675097.9839633307,716406.0857932968,758414.4360886116,801134.5391457919,844578.0854893855,888756.9548572875,933683.2192349497,979369.145937206,1025827.2007392768,1073070.0510574216,1121110.569181052,1169961.8355546244,1219637.1421133673,1270149.9956712776,1321514.1213631534,1373743.4661412106,1426852.2023281776,1480854.731225125,1535765.6867784627,1591599.9393042903,1648372.5992722062,1706099.021149002,1764794.807304452,1824475.811977273,1885158.1453059572,1946858.1774227936,2009592.5426130432,2073378.1435400262,2138232.155538371,2204172.030973348,2271215.503671554,2339380.5934208906,2408685.610542232,2479149.1605334436,2550790.148788282,2623627.7853879053,2697681.589970752,2772971.3966785534,2849517.359180831,2927339.9557804004,3006459.994597337,3086898.6188377324,3168677.3121447936,3251817.9040350644,3336342.575420586,3422273.864220082,3509634.671056264,3598448.265046383,3688738.2896831003,3780528.7688089404,3873844.112685125,3968709.1241581608,4065149.0049210954],[-669798.9086344328,-651288.9317247005,-632455.1169515526,-613292.0726641765,-593794.3192376537,-573956.2876538755,-553772.3180600409,-533236.6583040659,-512343.4624466221,-491086.78924907,-469460.6006378485,-447458.760143687,-425075.0313161856,-402303.07611296535,-379136.45326321013,-355568.6166046022,-331592.9133945191,-307202.58259343053,-282390.75312129664,-257150.44208600116,-231474.55298361438,-205355.87386944052,-178787.07550077583,-151760.70944909472,-124269.20618256275,-96304.87311785715,-67859.89264094969,-38926.32009587437,-9496.081742308335,20439.02732036612,50887.345261646435,81857.34566224646,113357.63969048299,145396.97831391264,177984.25454517687,211128.5057247593,244838.9158395403,279124.8178784943,313995.6962257917,349461.1890926403,385531.090986717,422215.35522212577,459524.0964687932,497467.59334255755,536056.2910364387,575300.8039944512,615211.9186267424,655800.596069244,697077.9749866317,739055.374420075,781744.2966801862,825156.4302867074,869303.6529536233,914198.0346231994,959851.8405475868,1006277.5344196477,1053487.7815534081,1101495.4521159618,1150313.624409171,1199955.5882051922,1250434.84813424,1301765.1271264483,1353960.3699082437,1407034.7465552536,1461002.6560998985,1515878.7301981824,1571677.836853822,1628415.0842018332,1686105.8243520036,1744765.657294446,1804410.43486534,1865056.2647775305,1926719.514714305,1989416.81648835,2053165.0702665797,2117981.4488631627,2183883.402098591,2250888.661230143,2319015.243451584,2388281.456464608,2458705.903122585,2530307.486149211,2603105.4129297533,2677119.200380677,2752368.67989538,2828874.002368429,2906655.6433007927,2985734.4079835294,3066131.436766271,3147868.210408041,3230966.5555131948,3315448.6500532907,3401337.0289779836,3488654.5899120593,3577424.598945665,3667670.6965149,3759416.9033759497,3852687.6266736407,3947507.666107705,4043902.22019372],[-688129.5579483795,-669641.7695661805,-650830.2980158698,-631689.7529438374,-612214.656034432,-592399.4395909696,-572238.4450943752,-551725.9217387065,-530856.0249433103,-509622.81484088604,-488020.2547419998,-466042.20957442047,-443682.4442978373,-420934.6222931256,-397792.3037260263,-374248.9438842216,-350297.89148864965,-325932.3869770486,-301145.56076048873,-275930.43145194114,-250279.90406667534,-224186.7681934603,-197643.69613745832,-170643.241032538,-143177.83492397354,-115239.78682036093,-86821.28071458917,-57914.373572738376,-28510.99329181481,1397.0633760292549,31818.132931932807,62760.68727250816,94233.33586605662,126244.82796406024,158804.05484685977,191920.05210625636,225602.00196388457,259859.23562676413,294701.2356802253,330137.6385195777,366178.23681936716,402832.98204318737,440111.9869928616,478025.52839838155,516584.0495489973,555798.1629668348,595678.6531218644,636236.4791913745,677482.7778627218,719428.8661809138,762086.2444413642,805466.5991293823,849581.8059051028,894443.932637299,940065.2424848394,986458.1970272625,1033635.4594450577,1081609.8977513611,1130394.5880734385,1180002.8179880055,1230448.0899088015,1281744.1245282046,1333904.8643134143,1386944.4770591287,1440877.359494931,1495718.1409518395,1551481.6870862325,1608183.1036631921,1665837.7403997516,1724461.194870226,1784069.3164717057,1844678.2104544293,1906304.2420153222,1968964.0404566994,2032674.503410872,2097452.8011329,2163316.380859437,2230282.9712389275,2298370.586831056,2367597.532677932,2437982.408947581,2509544.1156523274,2582301.8574397797,2656275.148462175,2731483.817321863,2807948.0120952623,2885688.2054378036,2964725.1997673544,3045080.13253241,3126774.4815626377,3209830.0705045094,3294269.074342904,3380114.02501169,3467387.817090466,3556113.713594496,3646315.351854984,3738016.749492924,3831242.3104873165,3926016.831341134,4022365.5073419455],[-706712.1882695588,-688246.8042495373,-669457.8931749245,-650340.0660013297,-630887.8457344872,-611095.6660113973,-590957.8696590778,-570468.7072302194,-549622.33551546,-528412.8160315701,-506834.1134861107,-484880.094216882,-462544.52460676595,-439821.0694731502,-416703.2904317032,-393184.64423361956,-369258.4810770347,-344918.04289073823,-320156.46159081464,-294966.7573093772,-269341.8365950859,-243274.4905845034,-216757.3931451235,-189783.09898784105,-162344.04174977867,-134432.53204637463,-106040.75549248606,-77160.77069144393,-47784.50719294627,-17903.763417418813,12489.795452250633,43404.63961274875,74849.37681638217,106832.7545824193,139363.66244260315,172451.134223575,206104.35036510578,240332.64027545997,275145.4847241577,310552.51827347535,346563.5317475314,383188.4747419094,420437.45817268034,458320.75686615356,496848.81218978553,536032.2347256201,575881.8069850716,616408.4861682002,657623.4069663072,699537.8844092858,742163.4167582125,785511.688444634,829594.5730552941,874424.1363657713,920012.6394216996,966372.5416691785,1013516.5041348264,1061457.3926572714,1110208.2811684366,1159782.4550286445,1210193.414413966,1261454.8777576392,1313580.7852460255,1366585.3023710619,1420482.823537436,1475287.9757289046,1531015.622231989,1587680.8664191016,1645299.0555915476,1703885.7848846319,1763456.9012329096,1824028.5074003262,1885616.9660735037,1948238.904020214,2011911.2163137305,2076651.0706253285,2142475.9115828853,2209403.4652008144,2277451.7433792725,2346639.048475067,2416983.9779448938,2488505.4290634603,2561222.6037141844,2635155.0132582877,2710322.483479989,2786745.159610204,2864443.5114311804,2943438.3384596696,3023750.7752148174,3105402.296568407,3188414.723180235,3272810.227019382,3358611.336974515,3445840.9445503345,3534522.309657184,3624679.0664910404,3716335.2295070305,3809515.199487356,3904243.7697069207,4000546.1321936515],[-725552.298412011,-707109.5517448277,-688343.4357613699,-669248.5627410809,-649819.4570258384,-630050.5536012431,-609936.1966554702,-589470.6381150591,-568648.0361573151,-547462.4536986265,-525907.8568592358,-503978.11340283393,-481666.9911515303,-458968.1563753977,-435875.17215640564,-412381.49672577064,-388480.48177554016,-364165.3707424151,-339429.29706455045,-314265.2824104198,-288666.2348794821,-262624.9471736718,-236134.09474059357,-209186.23388611246,-181773.79985734797,-153889.10489490815,-125524.33625414199,-96671.5541943456,-67322.68993682112,-37469.543589381734,-7103.782038252102,23783.0631938132,55199.598123416305,87154.56851767446,119656.86213960289,152715.5110303387,186339.69382722303,220538.73811898823,255322.12283834978,290699.480693331,326680.60063617793,363275.4303727944,400494.07891160995,438346.81915310724,476844.0905205924,515996.5016334285,555814.833021625,596310.0398849461,637493.254895292,679375.7910438753,721969.1445336062,765284.9977181859,809335.2220866398,854131.8812967457,899687.2342560582,946013.7382521066,993124.0521322491,1041031.0395349495,1089747.7721708482,1139287.5331576562,1189663.8204072965,1240890.3500670826,1292981.060015472,1345950.1134142985,1399811.9023156932,1454581.0513282153,1510272.4213402893,1566901.113303105,1624482.4720733915,1683032.090318276,1742565.812480302,1803099.7388073257,1864650.229445559,1927233.908597779,1990867.6687474563,2055568.6749509815,2121354.369196017,2188242.4748311774,2256251.0010649646,2325398.2475364152,2395702.8089580843,2467183.57983388,2539859.7592494977,2613750.855741252,2688876.6922409637,2765257.4110994036,2842913.479190631,2921865.693094886,3002135.1843661824,3083743.4248822606,3166712.2322796774,3251063.7754747523,3336820.5802735966,3424005.535068237,3512641.8966259225,3602753.295968797,3694363.744347109,3787497.639306809,3882179.7708548666,3978435.3277192484],[-744652.909656686,-726233.0304769232,-707489.9412918178,-688418.2557185121,-669012.4994490632,-649267.1088318164,-629176.4294304154,-608734.7145596968,-587936.1237982346,-566774.7214767674,-545244.4751431283,-523339.2540019385,-501052.827329732,-478378.8628646317,-455310.9251704116,-431842.4739739862,-407966.8624771419,-383677.33564048726,-358967.028440404,-333828.9640980428,-308256.0522801562,-282241.08727071877,-255776.74611425493,-228855.58672861988,-201470.04598810477,-173612.43777584075,-145274.95100520225,-116449.64760917309,-87128.46049854928,-57303.19148661685,-26965.509181201458,3893.0531570347957,35281.099790334934,67207.374714843,99680.76390578668,132710.29759957618,166305.15261167916,200474.65469166706,235228.28091560001,270575.66211721,306526.5853566055,343090.9964295586,380279.00241614785,418100.8742701607,456567.0494496261,495688.1345898914,535474.9082180089,575938.3235116522,617089.5111013174,658939.7819172884,701500.630081838,744783.7358481099,788800.9685844593,833564.3898076748,879086.2562637916,925379.0230580973,972455.3468347504,1020328.0890078736,1069010.319042387,1118515.3177886978,1168856.5808696141,1220047.822121319,1272102.9770888782,1325036.2065782573,1378861.9002630203,1433594.6803502152,1489249.4053035902,1545841.1736262538,1603385.3277032045,1661897.4577059592,1721393.405557326,1781889.268961044,1843401.4054945954,1905946.4367671446,1969541.2526434287,2034203.0155357295,2099949.1647619377,2166797.420975012,2234765.7906616195,2303872.5707125315,2374136.3530653245,2445576.0294219553,2518210.79603891,2592060.158595761,2667143.937139764,2743482.2711090394,2821095.624436589,2900004.790732932,2980230.898553376,3061795.4167476925,3144720.1598948906,3229027.2938238806,3314739.3412231673,3401879.1873366963,3490470.085752819,3580535.6642836714,3672099.9309380474,3765187.2799886833,3859822.4981372426,3956030.7707739654],[-764016.7932013292,-745620.0064047802,-726900.1704032664,-707851.9001643927,-688469.7227432508,-668748.0758639411,-648681.3064787013,-628263.6693039299,-607489.3253328521,-586352.3403240915,-564846.6832667142,-542966.2248200763,-520704.73572906107,-498055.88521391223,-475013.239334411,-451570.2593275318,-427720.29991928395,-403456.6076088189,-378772.3189255323,-353660.45865819464,-328113.9380559197,-302125.5529999519,-275687.982147136,-248793.78504284122,-221435.40020423126,-193605.14317281428,-165295.20453600027,-136497.64791662712,-107204.40793132759,-77407.28811536194,-47097.95881482423,-16267.955045108916,15091.325684661046,46988.625582468,79432.82881841552,112432.96380665526,145998.20552308555,180137.87786022061,214861.45601941808,250178.56894185394,286099.0017770801,322632.6983921046,359789.7639198662,397580.4673484387,436015.24415138736,475104.69896064955,514859.6082807286,555290.9232474328,596409.772429853,638227.4646771592,680755.4920105767,724005.5325620859,767989.4535585532,812719.3143547352,858207.369513913,904466.0719376446,951508.0760452158,999346.2410044856,1047993.6340125201,1097463.5336300563,1147769.4331682003,1198925.044129164,1250944.2997015615,1303841.358312184,1357630.6072324894,1412326.666244225,1467944.391362432,1524498.8786178506,1582005.4678992205,1640479.7468576604,1699937.5548711913,1760394.9870741447,1821868.3984496882,1884374.4079875536,1947929.902907637,2012552.0429517287,2078258.2647413178,2145066.2862067632,2212994.1110856994,2282060.0334931365,2352282.6425638963,2423680.8271699194,2496273.780710113,2570081.0059786523,2645122.3201092733,2721417.8595981766,2798988.0854077414,2877853.788148884,2958036.093348018,3039556.466796433,3122436.719984764,3206699.0156233464,3292365.8732516114,3379460.174933594,3468005.171046613,3558024.4861603007,3649542.125009165,3742582.478559557,3837170.330174297,3933330.8618720267],[-783649.0599519501,-765275.6064175054,-746579.2661923276,-727554.6556105037,-708196.3031063668,-688498.6477961617,-668456.0380372673,-648062.7299643799,-627312.8860023043,-606200.5733546463,-584719.7624689993,-562864.3254769274,-540628.0346093457,-518004.5605864648,-494987.47098214785,-471570.228561671,-447746.18959372607,-423508.60213467595,-398850.60428578756,-373765.2224225139,-348245.3693956204,-322283.8427031059,-295873.32263384666,-269006.37038065004,-241675.42612369382,-213872.8070832251,-185590.70554128918,-156821.1868313963,-127556.18729706272,-97787.51221679989,-67506.83369649295,-36705.688528030645,-5375.476013893727,26492.544243491488,58907.25458662212,91877.68158463365,125412.99835087522,159522.5268975529,194215.74052780634,229502.26626649336,265391.8873285621,301894.5456279537,339020.3443258833,376779.5504198638,415182.5973738618,454240.08779098885,493962.7961275114,534361.6714513693,575447.8402439756,617232.6092468067,659727.4683531662,702944.0935467023,746894.3498852919,791590.2945338837,837044.1798448749,883268.456487692,930275.7766279813,978078.9971582762,1026691.1829783847,1076125.6103296392,1126395.7701813644,1177515.371671395,1229498.3456011298,1282358.8479870888,1336111.2636671462,1390770.209965941,1446350.5404176158,1502867.3485479974,1560335.9717166466,1618771.995020986,1678191.2552605774,1738609.8449662724,1800044.116492482,1862510.6861746619,1926026.4385526204,1990608.5306620058,2056274.3963918532,2123041.7509134533,2190928.595178514,2259953.220488973,2330134.2131391615,2401490.4591328185,2474041.1489726626,2547805.7825284144,2622804.173980776,2699056.4568440625,2776583.089069612,2855404.8582278327,2935542.886774885,3017018.637401728,3099853.918468294,3184070.8895235322,3269692.0669144765,3356740.329481449,3445238.9243463967,3535211.472791614,3626681.976231966,3719674.82228153,3814214.7909178655,3910327.0607410534],[-803551.4722986151,-785201.5775421298,-766528.960094482,-747528.2376690783,-728193.9400924685,-708520.5078861215,-688502.2908257989,-668133.5464778615,-647408.4387121825,-626321.0361909957,-604865.3108341964,-583035.1362594808,-560824.2861978351,-538226.4328836652,-515235.1454192416,-491843.888112647,-468046.01878990466,-443834.78707937384,-419203.3326691361,-394144.68353643245,-368651.75414892496,-342717.3436367931,-316334.1339364969,-289494.6879040259,-262191.44739845674,-234416.73133484018,-206162.73370606918,-177421.5215727263,-148185.0330217632,-118445.0750916833,-88193.32166506792,-57421.31132737687,-26120.445191715844,5718.015311653726,38102.95067997929,71043.3856196052,104548.4913633191,138627.58802484744,173290.14699074207,208545.79335098062,244404.30836715433,280875.63198118773,317969.8653634223,355697.2735014539,394068.2878300969,433093.5089038806,472783.7091108544,513149.83543091966,554203.0122374156,595954.5441435203,638415.9188937931,681598.8103024887,725515.0812372472,770176.7866516991,815596.1766656493,861785.6996944214,908758.0056278608,956525.9490607376,1005102.5925729482,1054501.2100635143,1104735.2901368407,1155818.5395429889,1207764.8866725047,1260588.4851077348,1314303.717228829,1368925.1978789,1424467.7780865305,1480946.5488476967,1538376.84496758,1596774.2489644326,1656154.5950336116,1716533.973076465,1777928.7327923556,1840355.4878358562,1903831.1200398314,1968372.783706626,2033997.9099653475,2100724.211200481,2168569.6855497533,2237552.621473672,2307691.6023974083,2379005.5114275375,2451513.536141295,2525235.173454341,2600190.2345644156,2676398.8499736954,2753881.4745918936,2832658.892917997,2912752.2243066556,2994182.9283169126,3076972.810146062,3161144.026149371,3246719.0894488394,3333720.8756280765,3422172.628520308,3512097.966086761,3603520.886388571,3696465.773653065,3790957.4044377077,3887020.953888782],[-823729.4967031209,-805403.4021519365,-786754.7505955045,-767778.1611420632,-748468.1650260523,-728819.2041900218,-708825.629844131,-688481.701002585,-667781.5829966976,-646719.3459638646,-625288.9633130049,-603484.3101648269,-581299.1617674893,-558727.1918868308,-535761.9711710035,-512396.9654885207,-488625.53424056084,-464440.9286455093,-439836.28999647824,-414804.64789090236,-389338.91843197285,-363431.9024008459,-337076.2834006024,-310264.6259696279,-282989.3736653479,-255242.8471172601,-227017.24204897066,-198304.62726821075,-169096.94262567908,-139385.99694038485,-109163.4658923531,-78420.88988159248,-47149.671853031265,-15341.075086218305,17013.779049190925,49923.91337482142,83398.49722075579,117446.84877984412,152078.43749940395,187302.88651164365,223129.97510165325,259569.64121592813,296631.98401028384,334327.2664384828,372665.91788199334,411658.53682230366,451315.8935545306,491648.9329455297,532668.7772352917,574386.7288831284,616814.2734590156,659963.0825816891,703845.0169021178,748472.129135903,793856.667143238,840011.0770580356,886948.0064666988,934680.3076383197,983221.0408046255,1032583.4774937774,1082781.1039163684,1133827.624405488,1185736.9649112932,1238523.2765520886,1292200.939220103,1346784.5652463818,1402289.0031230608,1458729.3412850308,1516120.9119515126,1574479.295029644,1633820.3220782727,1694160.0803365814,1755514.9168158402,1817901.4424563367,1881336.536350194,1945837.3500322797,2011421.311837228,2078106.131327752,2145909.8037922545,2214850.6148140603,2284947.1449130285,2356218.2742619882,2428683.187475727,2502361.3784784144,2577272.6554470123,2653437.1458332893,2730875.30146663,2809607.903735461,2889656.0688533243,2971041.253207302,3053785.2587915487,3137910.2387266853,3223438.70286824,3310393.523501197,3398797.94112766,3488675.570344925,3580050.4058170104,3672946.828340626,3767389.6110087684,3863403.9254690306],[-844185.6162641598,-825883.5532345906,-807259.1004152867,-788306.8783226194,-769021.4196123545,-749397.1676616834,-729428.4751288756,-709109.6024898372,-688434.7165513062,-667397.8889399767,-645993.0945680633,-624214.2100737256,-602055.0122368569,-579509.1763694636,-556570.2746804387,-533231.7746137662,-509487.03716096305,-485329.3151457738,-460751.7514818397,-435747.3774024602,-410309.11066215555,-384429.75370906014,-358101.9918290174,-331318.3912591296,-304071.3972716576,-276353.3322272047,-248156.3935969225,-219472.65195269813,-190294.0489261623,-160612.39513419103,-130419.3680718008,-99706.50997127709,-68465.22562729195,-36686.78018677258,-4362.29690457345,28517.24513772549,61961.013348824345,95978.32398182177,130578.64452573331,165771.59613571642,201566.95610088645,237974.66035264288,275004.8060123641,312667.65397988143,350973.63156301226,389933.3351496749,429557.53292130167,469857.16761074006,510843.35930344835,552527.408283452,594920.7979245123,638035.1976279633,681882.4658060097,726474.6529138759,771824.0045295712,817942.9644827717,864844.1780333887,912540.4951015068,961044.973547142,1010370.8825037745,1060531.7057641256,1111541.1452199803,1163413.124356519,1216161.7918031686,1269801.5249391012,1324346.933557916,1379812.863589636,1436214.4008821337,1493566.8750424404,1551885.8633400844,1611187.1946706176,1671486.9535839753,1732801.4843759625,1795147.3952449174,1858541.5625142492,1923001.134923085,1988543.5379829947,2055186.4784060381,2122947.9486020864,2191846.2312477664,2261899.9039277732,2333127.8438510327,2405549.2326393668,2479183.5611946457,2554050.6346418737,2630170.5773509005,2707563.8380389395,2786251.194951639,2866253.7611288936,2947592.9897529054,3030290.679581406,3114368.9804667407,3199850.398963952,3286757.80402502,3375114.432786168,3464943.896445597,3556270.1862346944,3649117.679483626,3743511.1457845853,3839475.75324977],[-864923.9399816503,-846646.1431362136,-828046.1252741065,-809118.5083334422,-789857.8264050487,-770258.5243146596,-750314.9561826827,-730021.3839608927,-709371.9759457794,-688360.8052677955,-666981.848357076,-645228.9833839973,-623095.9886751012,-600576.5411036378,-577664.2144544735,-554352.4777624495,-530634.6936249633,-506504.11648683064,-481953.8908980957,-456977.0497439485,-431566.51244645147,-405715.0831370803,-379415.44880098803,-352660.17739070975,-325441.7159102205,-297752.38846829673,-269584.3943008871,-240929.80576145486,-211780.56628015917,-182128.48828954995,-151965.25111760618,-121282.39884705492,-90071.33814066835,-58323.33603130677,-26029.517677770462,6819.1359162156,40231.79021882173,74217.7595240525,108786.50934306625,143947.65883422154,179710.9832706824,216086.4165485648,253084.05373444315,290714.15365359373,328987.14151935466,367913.61160503025,407504.3299570901,447770.2371528775,488722.45110158017,530372.2698899934,572731.1746734469,615810.8326134523,659623.0998607725,704180.0245873462,749493.8500658241,795577.0177982473,842442.1706943591,890102.1563013424,938570.0300833103,987859.0587546346,1037982.723665467,1088954.724241329,1140788.9814771959,1193499.6414880943,1247101.0791143826,1301607.9015861377,1357034.9522449384,1413397.3143250067,1470710.3147942717,1528989.5282574464,1588250.780919277,1648510.1546126585,1709783.9908898212,1772088.8951787446,1835441.7410054053,1899859.6742841569,1965360.1176741575,2031960.775007144,2099679.6357844616,2168534.9797457056,2238545.38150977,2309729.715290672,2382107.159685959,2455697.202543511,2530519.64590436,2606594.611024067,2683942.5434749303,2762584.218326755,2842540.7454123166,2923833.574675122,3006484.501602288,3090515.6727433074,3175949.5913177924,3262809.1229093643,3351117.501252643,3440898.334110614,3532175.6092454996,3624973.700483989,3719317.373880141,3815231.793972991],[-885949.3984839276,-867696.1123913539,-849120.7757306014,-830218.0118766703,-810982.3563707623,-791408.2555026175,-771490.0648704064,-751222.0479175798,-730598.3744463362,-709613.1191070271,-688260.2598640064,-666533.6764363435,-644427.1487138828,-621934.3551479569,-599048.871116407,-575764.1672621204,-552073.6078057594,-527970.4488307603,-503447.83654132276,-478498.805492471,-453116.2767919323,-427293.0562728485,-401021.83263820363,-374295.17557468545,-347105.5338369068,-319445.2333009457,-291306.474986871,-262681.33304926264,-233561.7527365666,-203939.5483169579,-173806.4009715633,-143153.85665395018,-111973.32391560916,-80256.071696172,-47993.22707945481,-15175.77301262482,18205.454010433983,52159.766302868025,86696.62737598037,121825.65436908719,157556.6205169293,193899.4576576131,230864.25877992297,268461.28061136603,306700.94624731666,345593.84782273206,385150.7492251231,425382.5888520675,466300.48241195874,507915.7257695608,550239.7978367149,593284.3635097635,637061.2766524004,681582.5831273757,726860.5238757986,772907.5380455842,819736.2661695541,867359.5533949011,915790.4527624841,965042.2285398925,1015128.3596067335,1066062.5428939727,1117858.6968777878,1170530.965129889,1224093.7199225705,1278561.5658928333,1333949.343763914,1390272.1341261622,1447545.26127784,1505784.2971279412,1565005.0651591602,1625223.6444557454,1686456.3737944127,1748719.855800468,1812030.961169789,1876406.8329589167,1941864.8909411943,2008422.836034249,2076098.6547967433,2144910.623996705,2214877.3152522794,2286017.599747264,2358350.653019191,2431895.9598258403,2506673.3190877293,2582702.8489091923,2660004.9916802463,2738600.519257068,2818510.538227103,2899756.4952565185,2982360.182522739,3066343.7432328765,3151729.6772311367,3238540.8466923665,3326800.4819086907,3416532.187166531,3507759.9467171277,3600508.1308413954,3694801.5020124842,3790665.2211530027],[-907263.0245561879,-889034.4698132764,-870484.036278298,-851606.3487734423,-832395.9443007101,-812847.2706243473,-792954.6848308889,-772712.4518661592,-752114.7430488649,-731155.6345601398,-709829.1059095629,-688129.0383759795,-666049.2134237527,-643583.3110935688,-620724.908367662,-597467.4775084734,-573804.384371537,-549728.8866906366,-525234.1323359509,-500313.1575442718,-474958.8851210396,-449164.1226132305,-422921.5604539213,-396223.7700763375,-369063.20199823193,-341432.1838755973,-313322.9185253545,-284727.48191604996,-255637.82112739142,-226045.7522762695,-195942.95841016364,-165320.98736682837,-134171.24959992152,-102485.01596943964,-70253.41549793328,-37467.43308984488,-4117.907215089537,29804.472444518236,64309.16738619562,99405.79271584423,135104.11961528147,171414.0778499588,208345.75831609126,245909.41562850494,284115.4707495528,322974.51366063347,362497.3060749639,402694.7841948634,443578.0615122956,485158.43165420834,527447.3712730156,570456.5429838062,614197.7983469716,658683.1808996731,703924.9292349298,749935.4801298319,796727.4717233977,844313.7467458108,892707.3557974868,941921.5606818721,991969.837790512,1042865.8815421266,1094623.607876232,1147257.1578031965,1200780.9010090479,1255209.4395193313,1310557.6114203832,1366840.4946399415,1424073.4107876993,1482271.929057823,1541451.8701916803,1601629.3105054,1662820.5859805276,1725042.2964198454,1788311.3096690993,1852644.7659067758,1918060.0819999785,1984574.9559315983,2052207.3712967788,2120975.6018709503,2190898.2162502734,2261994.0825668797,2334282.373276638,2407782.5700253583,2482514.4685909688,2558498.1839042753,2635754.155150506,2714303.1509494744,2794166.274620341,2875364.969528749,2957921.0245190226,3041856.5794322575,3127194.130713372,3213956.5371043384,3302167.0254304386,3391849.196476948,3483027.0309592863,3575724.8955875365,3669967.5492286095,3765780.1491631214],[-928873.9210559989,-910670.3629291764,-892145.0997091081,-873292.7576835706,-854107.8753339265,-834584.9019177218,-814718.1960288554,-794502.0241347214,-773930.5590899568,-752997.8786261166,-731697.9638178423,-710024.6975238444,-687971.8628032797,-665533.14130673,-642702.1116415635,-619472.2477107511,-595836.9170259221,-571789.3789926553,-547322.7831688032,-522430.16749485955,-497104.4564961635,-471338.45945594227,-445124.8685600583,-418456.25701120123,-391325.07711346564,-363723.65832620393,-335644.2052869119,-307078.7958020882,-278019.37880695215,-248457.77229164005,-218385.66119479365,-187794.59526342922,-156675.9868787583,-125021.10884678969,-92821.09215474222,-60066.9236905938,-26749.443926815176,7140.655432975851,41612.833844761364,76676.70435358421,112342.03606056212,148618.75663042488,185516.95483841887,223046.88315794803,261218.9603893049,300043.7743309494,339532.0844920804,379694.8248496973,420543.1066489364,462088.2212481778,504341.64300931245,547315.0322347065,591020.2381496336,635469.3019334809,680674.4597986243,726648.1461183969,773402.9966047104,820951.8515370479,869307.7590412996,918483.9784223023,968493.9835486594,1019351.4662915841,1071070.34001825,1123664.7431415915,1177149.0427248543,1231537.8381452067,1286845.9648147197,1343088.4979607342,1400280.7564661335,1458438.3067715946,1517576.966838026,1577712.81017387,1638862.1699254555,1701041.643032547,1764268.094449778,1828558.6614361359,1893930.7579105399,1960402.0788787296,2027990.6049294076,2096714.60680199,2166592.6500267303,2237643.5996396514,2309886.6249699956,2383341.2045061146,2458027.1308373325,2533964.51567437,2611173.79495057,2689675.734001711,2769491.432830465,2850642.3314531674,2933150.2153316843,3017037.220891156,3102325.841126703,3189038.931296291,3277199.714706652,3366831.788589578,3457959.1300717285,3550606.1022387776,3644797.4602971985,3740558.357830767],[-950780.3678173181,-932602.0230228234,-914102.148070681,-895275.3707231738,-876116.2309500355,-856619.1795111774,-836778.5765170171,-816588.6899657419,-796043.6942572142,-775137.6686827985,-753864.5958916601,-732218.3603318986,-710192.746667088,-687781.438167403,-664978.0150751187,-641775.9529436072,-618168.6209505177,-594149.2801832631,-569711.0818974744,-544847.0657475414,-519550.15798897995,-493813.1696516415,-467628.7946846187,-440989.60807062057,-413888.0639106999,-386316.4934782982,-358267.10324229184,-329731.9728580401,-300703.0531272574,-271172.1639244186,-241130.9920905172,-210571.08929309784,-179483.86985229375,-147860.60853159637,-115692.43829446868,-82970.34802406793,-49685.180207203375,-15827.628581149736,18611.764256884344,53642.60927899927,89274.67349346308,125517.88245199667,162382.32279650634,199878.24484649627,238016.06522760028,276806.36954264296,316259.91508400114,356387.6335904421,397200.6340472433,438710.2055310807,480927.82010005694,523865.1357304733,567533.9992989893,611946.4496136298,657114.7204924067,703051.2438910645,749768.6530804583,797279.7858753055,845597.6879127673,894735.6159847532,944707.0414225026,995525.6535351411,1047205.3631028095,1099760.3059261898,1153204.8464308134,1207553.5813303739,1262821.3433474754,1319023.204993729,1376174.4824097282,1434290.7392670503,1493387.7907304047,1553481.7074846625,1614588.819824919,1676725.7218117565,1739909.2754923962,1804156.6151898787,1869485.151858361,1935912.5775096891,2003456.869709251,2072136.296143415,2141969.419259354,2212975.1009796453,2285172.5074894344,2358581.114101982,2433220.710200216,2509111.4042568253,2586273.6289351564,2664728.1462687063,2744496.052925218,2825598.7855531373,2908058.1262131203,2991896.207895436,3077135.520126324,3163798.9146604915,3251909.6112666796,3341491.2036036164,3432567.6651894324,3525163.3554654405,3619303.025957555,3715011.8265324086],[-972991.244145399,-954838.3711446862,-936364.1447131959,-917563.1941041412,-898430.0607922318,-878959.1970565307,-859144.96454093,-838981.6327915585,-818463.37777084,-797584.2803474534,-776338.3247627916,-754719.3970722437,-732721.28356189,-710337.6691397657,-687562.135701546,-664388.1604696712,-640809.1143067069,-616818.2600009714,-592408.7505251735,-567573.6272670974,-542305.8182321531,-516598.13621673686,-490443.27695332374,-463833.81722499174,-436762.2129503535,-409220.7972377357,-381201.77840844006,-352697.2379879416,-323699.1286659646,-294199.2722230458,-264189.3574244953,-233660.93788061407,-202605.42987290258,-171014.1101450408,-138878.11365968618,-106188.43131839158,-72935.90764573682,-39111.238436394604,-4704.96836474305,30292.5114441514,65890.96588117117,102100.31836026022,138930.6533649303,176392.219035442,214495.42979716416,253250.86903138924,292669.2917874763,332761.627539495,373538.98298608745,415012.64489515685,457194.08299366385,500094.952904148,543727.0991266817,588102.5580696347,633233.5611280431,679132.5378111382,725812.1189194582,773285.1397733688,821564.6434914018,870663.8843222912,920596.3310293122,971375.6703285887,1023015.8103819452,1075530.8843461908,1128935.253977138,1183243.513292679,1238470.4922932438,1294631.2607416231,1351741.1320026838,1409815.666945062,1468870.677903071,1528922.232703406,1589986.6587549378,1652080.547203693,1715220.7571536857,1779424.4199558254,1844708.9435628974,1911092.0169558409,1978591.6146392939,2047226.001208717,2117013.7359898933,2187973.677753248,2260124.9895006376,2333487.143330629,2408079.925379672,2483923.4408419044,2561038.119069691,2639444.7187527707,2719164.3331820476,2800218.3955956534,2882628.6846101396,2966417.3297375157,3051606.816991245,3138219.9945783885,3226280.078684808,3315810.659350741,3406835.7064398485,3499379.575702615,3593467.014937399,3689123.170246142],[-995508.2969447011,-977381.1343262655,-958932.7965028303,-940157.9142309042,-921051.0305026396,-901606.5991288528,-881818.9832996235,-861682.4541218742,-841191.1891335482,-820339.2707937388,-799120.6849492881,-777529.3192762276,-755558.9616966178,-733203.2987699853,-710455.9140591389,-687310.2864694807,-663759.7885624976,-639797.6848415865,-615417.1300108198,-590611.1672058173,-565372.7261964623,-539694.6215604232,-513569.55082841986,-486990.09259892534,-459948.70462325914,-432437.72185996943,-404449.3544982404,-375975.6859492862,-347008.6708066156,-317540.1327727614,-287561.7625534469,-257065.1157179703,-226041.61052562157,-194482.52571685286,-162378.9982702483,-129722.02112267166,-96502.44085357012,-62710.95533224731,-28338.11132768262,6625.697920275386,42190.23516737204,78365.42167179985,115161.33974053292,152588.23531628354,190656.52060563676,229376.77674966957,268759.7565358388,308816.38715443667,349557.77299821004,390995.198506834,433140.1310564508,476004.2238959996,519599.3191288924,563937.4507435802,609030.8476916575,654891.9370151535,701533.3470234061,748967.9105213247,797208.6680874834,846268.8714059116,896161.9866501587,946901.6979213608,998501.9107408244,1050976.7555990377,1104340.5915594446,1158608.0099212276,1213793.8379395097,1269913.1426048838,1326981.2344828718,1385013.6716153286,1444026.2634820445,1504035.0750271948,1565056.4307488212,1627106.9188535265,1690203.3954770043,1754362.9889726462,1819603.1042662328,1885941.4272818854,1953395.9294373034,2021984.8722105855,2091726.811779417,2162640.6037350385,2234745.4078687523,2308060.693036838,2382606.2421014253,2458402.1569499355,2535468.8635953036,2613827.1173547744,2693498.008113351,2774502.965669504,2856863.765166021,2940602.5326066767,3025741.7504618727,3112304.2633604137,3200313.2838743473,3289792.398394174,3380765.5730974884,3473257.1600120375,3567291.903176352,3662894.9448951026],[-1018334.4162022129,-1000233.1919502117,-981810.9720526774,-963062.3887794233,-943981.9866501616,-924564.2210176433,-904803.4566284395,-884693.9661606222,-864229.9287381333,-843405.4284210273,-822214.4526722592,-800650.8907992393,-778708.5323708532,-756381.065609046,-733662.0757548022,-710545.0434076034,-687023.3428391044,-663090.2402790925,-638738.8921744318,-613962.3434201165,-588753.5255621285,-563105.2549711443,-537010.2309879691,-510461.03403841937,-483450.12371857837,-455969.8368493628,-428012.3855001095,-399569.85498016,-370634.20179928397,-341197.25159462076,-311250.6970250085,-280786.0956315864,-249794.8676643665,-218268.29387363326,-186197.51326707914,-153573.5208301698,-120387.16521063028,-86629.1463659408,-52290.01317333663,-17360.161001222674,18170.17075708788,54310.9011900085,91072.11041382654,128464.0421594074,166497.10640013823,205181.88202232216,244529.11953684734,284549.74383540684,325254.8569899071,366655.7410966973,408763.86116589047,451590.86805745214,495148.60146264406,539449.0929343388,584504.5689649102,630327.4541132529,676930.3741814201,724326.1594426446,772527.8479191614,821548.688713768,871402.1453936188,922101.8994280207,973661.8536807764,1026096.1359589165,1079419.1026162049,1133645.34221566,1188789.6792494627,1244867.1779182367,1301893.1459702002,1359883.1386022987,1418852.9624215257,1478818.6794710816,1539796.611319542,1601803.343215289,1664855.7283067228,1728970.891930567,1794166.2359662494,1860459.4432615698,1927868.4821276027,1996411.6109052016,2066107.38260388,2136974.6496154577,2209032.5685002306,2282300.6048515583,2356798.538236421,2432546.4672145033,2509564.8144380795,2587874.3318304955,2667496.1058492153,2748451.5628312323,2830762.4744235356,2914450.9630994163,2999539.5077637704,3086050.9494444863,3174008.497076921,3263435.7333787256,3354356.620818137,3446795.507676614,3540777.1342091192,3636326.6388990427],[-1041479.4071053674,-1023404.3973090368,-1005008.5734015311,-986286.5691846792,-967232.9307246568,-947842.1149352822,-928108.4881389346,-908026.3246043918,-887589.805061341,-866793.0151907874,-845629.9440919808,-824094.482724139,-802180.4223236195,-779881.4527956594,-757191.1610805229,-734103.0294931175,-710610.4340368316,-686706.6426896674,-662384.8136633479,-637637.9936345357,-612459.115947864,-586840.998789828,-560776.3433343761,-534257.7318579794,-507277.62582506356,-479828.3639427626,-451902.1601846658,-423491.10178262275,-394587.14718731283,-365182.12399540516,-335267.72684406955,-304835.51527177077,-273876.9115450233,-242383.19845000468,-210345.51704988233,-177754.86440537777,-144602.0912594609,-110877.89968504407,-76572.84069519071,-41677.31181474868,-6181.554614278022,29924.347796330694,66650.47331383312,104007.06342808483,142004.5258497768,180653.43718060758,219964.54562471807,259948.7737446865,300617.2212607162,341981.1678946186,384052.0762588992,426841.59479262726,470361.5607426078,514624.0031934888,559641.1461453675,605425.4116405924,651989.4229401373,699346.0077513666,747508.2015056107,796489.2506894483,846302.6162282652,896961.9769237633,948481.2329460424,1000874.5093820668,1054156.1598388935,1108340.7701059044,1163443.1618744694,1219478.3965168972,1276461.7789253453,1334408.8614126318,1393335.447673284,1453257.5968093807,1514191.6274194638,1576154.1217526197,1639161.9299284066,1703232.174224868,1768382.2534325798,1834629.8472800078,1901992.9209280931,1970489.7295364523,2040138.8229019018,2110959.0501717934,2182969.5646298626,2256189.82856046,2330639.618188776,2406339.02869959,2483308.4793367973,2561568.718581546,2641140.829414927,2722046.234663031,2804306.7024270524,2887944.351599222,2972981.6574677457,3059441.4574078233,3147346.9566657366,3236721.7342333044,3327589.748815766,3419975.3448940567,3513903.258884671,3609398.6253942004],[-1064941.8901700207,-1046893.3226665565,-1028524.1238782093,-1009828.9291481219,-990802.2860982994,-971438.6532130691,-951732.3984001409,-931677.7975285992,-911269.0329435656,-890500.1919567522,-869365.2653135487,-847858.1456349085,-825972.6258346704,-803702.3975114501,-781041.049314976,-757982.0652858368,-734518.8231695055,-710644.5927026316,-686352.5338723187,-661635.6951475048,-636487.0116821819,-610899.3034894441,-584865.2735872685,-558377.5061137488,-531428.4644127155,-504010.48908867245,-476115.79603072326,-447736.4744045576,-418864.4846132181,-389491.65622443706,-359609.68586534867,-329210.1350834798,-298284.42817367846,-266823.8499699519,-234819.54360297485,-202262.50822086865,-169143.5966740772,-135453.5131632944,-101182.81084986497,-66321.8894276265,-30860.992657064926,5209.794140875805,41900.54663052689,79221.50404775003,117183.07182744099,155795.82427347545,195070.50726992218,235018.04103681166,275649.5229290826,316976.2302803369,359009.6232916992,401761.34796741,445243.2390958159,489467.32327917404,534445.8220110419,580191.1548028355,626715.942359928,674033.0098091774,722155.3899762356,771096.3267165795,820869.2782987864,871487.9208418066,922966.1518067517,975318.0935451007,1028558.0969016468,1082700.7448764644,1137760.8563442682,1193753.4898331105,1250693.9473629738,1308597.778346315,1367480.7835488124,1427359.0191149092,1488248.8006564379,1550166.7074063653,1613129.58643847,1677154.5569549892,1742259.0146403923,1808460.6360863997,1875777.383286269,1944227.5082006496,2013829.5573957944,2084602.3767565647,2156565.1162719387,2229737.234898923,2304138.5055024032,2379789.0198735804,2456709.193829143,2534919.7723890278,2614441.8350388003,2695296.801074301,2777506.435031411,2861092.852201636,2946078.524236632,3032486.2848388758,3120339.335545361,3209661.251601687,3300475.987929534,3392807.88518856,3486681.675935851,3582122.490880082],[-1088728.285847835,-1070706.4064949253,-1052364.080202763,-1033695.943868111,-1014696.5466812487,-995360.3487095623,-975681.7194587742,-955654.9364110716,-935274.1835399235,-914533.5498007997,-893427.0275984018,-871948.5112287102,-850091.7952964681,-827850.5731072358,-805218.4350338739,-782188.866856484,-758755.2480765688,-734910.8502035129,-710648.8350140157,-685962.2527836713,-660844.0404903067,-635287.0199882763,-609283.896154355,-582827.255003195,-555909.5617731127,-528523.1589811917,-500660.26444742153,-472312.96928685624,-443473.2358705669,-414132.8957531885,-384283.64756778814,-353917.05488709686,-323024.5440505962,-291597.4019565778,-259626.7738198454,-227103.6608926966,-194018.91815000772,-160363.25193736586,-126127.2175816861,-91301.2169632488,-55875.49605009705,-19840.142392034642,16814.917424866464,54099.92036431143,92025.26956705842,130601.53702064045,169839.46627034806,209749.97517476277,250344.15870447643,291633.29178561317,333628.8321884363,376342.4234627206,419785.8979184679,463971.2796554975,508910.7876405823,554616.8388337209,601102.0513640293,648379.2477569771,696461.4582114513,745361.923930522,795094.1005044598,845671.6613477166,897108.501190465,949418.7396265268,1002616.7247160347,1056717.036647126,1111734.4914550097,1167684.144800385,1224581.2958077434,1282441.490965621,1341280.5280870427,1401114.460334783,1461959.6003096527,1523832.524204,1586750.0760210063,1650729.3718620762,1715787.804280274,1781943.0467050225,1849213.057936069,1917616.0867090155,1987170.6763332034,2057895.6694043805,2129810.2125898753,2202933.761492167,2277286.0855883947,2352887.273248439,2429757.7368337554,2507918.2178747733,2587389.792332903,2668193.875944847,2750352.229651982,2833886.9651155625,2918820.5503208954,3005175.815267615,3092975.957753007,3182244.549245682,3273005.540852703,3365283.26938105,3459102.4634966794,3554488.2499783062],[-1112845.797520616,-1094850.8763471916,-1076535.6944111516,-1057894.8901793757,-1038923.0144270832,-1019614.5288215836,-999963.804483624,-979965.1205257087,-959612.6625670451,-938900.5212244324,-917822.6905796428,-896373.0666216405,-874545.4456642072,-852333.5227381785,-829730.8899580899,-806731.0348622713,-783327.3387271939,-759513.0748540938,-735281.4068286174,-710625.3867525321,-685537.9534472737,-660011.9306284005,-634040.025051682,-607614.8246287087,-580728.7965128964,-553374.2851547489,-525543.5103262046,-497228.5651129605,-468421.41387564107,-439113.89017752185,-409297.6946796053,-378964.3930020286,-348105.41355138924,-316712.0453129597,-284775.4356086247,-252286.58781806938,-219236.35906405188,-185615.4578607676,-151414.44172465848,-116623.71474669455,-81233.52512695594,-45233.96266888315,-8614.956233969424,28633.728844021913,66522.4933861238,105061.90703767305,144262.71097902162,184135.8206808772,224692.32870276365,265943.50753630185,307900.8124935804,350575.88464228576,393980.5537861665,438126.8414943749,483026.96417834004,528693.3362178095,575138.5731364563,622375.4948288668,670417.1288373466,719276.7136824019,768967.702245492,819503.7652057642,870898.7945312741,923166.9070266769,976322.4479355686,1030379.9946019291,1085354.3601888856,1141260.5974568417,1198114.0026014852,1255930.1191537455,1314724.7419399414,1374513.9211067292,1435313.9662090847,1497141.4503635024,1560013.2144669755,1623946.3714840608,1688958.3107999968,1755066.7026450871,1822289.502588336,1890644.9561026366,1960151.6032023462,2030828.2831555866,2102694.139269093,2175768.6237514312,2250071.502652166,2325622.8608795935,2402443.107299187,2480552.979910683,2559973.551109673,2640726.2330316002,2722832.782980716,2806315.308944944,2891196.2751996447,2977498.507997548,3065245.201351631,3154459.9229084565,3245166.6199148567,3337389.6252790177,3431153.6637291415,3526483.858066773],[-1137294.7563075111,-1119327.028075357,-1101039.2265829793,-1082425.9918768923,-1063481.8763268464,-1044201.3432097274,-1024578.7652710711,-1004608.4232635142,-984284.5044619231,-963601.1011544203,-942552.2091099096,-921131.726020518,-899333.4499192911,-877151.0775726892,-854578.2028473162,-831608.3150502474,-808234.797243508,-784450.9245308705,-760249.8623176811,-735624.6645427567,-710568.2718820863,-685073.509923476,-659133.0873128118,-632739.5938698605,-605885.4986744572,-578563.1481219472,-550764.7639477204,-522482.44121973915,-493708.14629988535,-464433.71477188496,-434650.8493365715,-404351.1176735135,-373525.9502685168,-342166.6382060549,-310264.3309274155,-277810.03395207785,-244794.6065632142,-211208.75945625524,-177043.05234991992,-142287.89155870164,-106933.5275276876,-70970.05232700007,-34387.397106758785,2824.6704885242507,40676.54894676525,79178.80555756344,118342.17912271479,158177.58271129522,198696.10645788908,239909.0204056278,281827.77739432454,324464.01599533716,367829.5634918199,411936.4389077765,456796.85608470114,502423.2268073559,548828.1639791285,596024.4848487349,644025.2142867898,692843.5881159967,742493.0564936465,792987.2873480534,844340.1698695421,896565.8180578095,949678.574324016,1003693.0131519302,1058623.944816412,1114486.4191612415,1171295.7294368302,1229067.4161998942,1287817.2712732772,1347561.3417706192,1408315.9341840171,1470097.618536903,1532923.2326027318,1596809.88619171,1661774.9655036037,1727836.1375517817,1795011.3546565385,1863318.8590099467,1932777.1873130864,2003405.1754880114,2075221.9634622629,2148247.0000317334,2222500.0477994755,2298001.188193094,2374770.826562833,2452829.697358281,2532198.869389622,2612899.7511712383,2694954.096350308,2778384.009221288,2863211.9503293056,2949460.742159663,3037153.574920388,3126314.0124151227,3216965.9980094316,3309133.860691489,3402842.3212303105,3498116.498428701],[-1162081.7903246684,-1144141.5079002073,-1125881.3412715206,-1107295.9320772956,-1088379.8342944826,-1069127.5128223205,-1049533.3420440305,-1029591.6043654236,-1009296.488730167,-988642.0891109772,-967622.4029773229,-946231.3297379918,-924462.6691589898,-902310.1197561736,-879767.2771621624,-856827.6324668268,-833484.5705319499,-809731.3682781721,-785561.1929450186,-760967.1003229658,-735942.032957312,-710478.8183230099,-684570.1669711072,-658208.6706447427,-631386.8003654962,-604096.9044890439,-576331.2067298729,-548081.8041539923,-519340.665140487,-490099.627309632,-460350.39541836083,-430084.53922208026,-399293.49130238267,-367968.5448596887,-336100.8514715845,-303681.4188144244,-270701.1083490369,-237150.6329694849,-203020.55461434787,-168301.2818394052,-132983.0673526693,-97056.00550906779,-60510.02976561664,-23334.910096046515,14479.749635782093,52944.51434489945,92070.12043552333,131867.47855588933,172347.67639623722,213521.98153160885,255401.8443097463,297998.9007857032,341324.9757018322,385392.08551660506,430212.44148098724,475798.45276395604,522162.72962758644,569318.0866535255,617277.5460192482,666054.3408280136,715661.918491058,766113.9441638007,817424.3042365015,869607.109881362,922676.7006543553,976647.6481560403,1031534.7597497809,1087353.0823392645,1144117.906205941,1201844.7689083545,1260549.459241704,1320248.0212621745,1380956.7583743045,1442692.2374835405,1505471.2932146061,1569311.0321979215,1634228.8374220743,1700242.3726575435,1767369.5869496786,1835628.7191832345,1905038.3027192396,1975617.1701066494,2047384.4578664768,2120359.6113543413,2194562.3896989054,2270012.8708189023,2346731.4565208843,2424738.8776755286,2504056.199478569,2584704.826793965,2666706.509582179,2750083.3484142255,2834857.800074718,2921052.6832509628,3008691.184315119,3097796.8631966845,3188393.659348419,3280505.897806597,3374158.2953488417,3469375.9667466637],[-1187214.0934497658,-1169301.5320686586,-1151069.2773791268,-1132511.9726284076,-1113624.1734171775,-1094400.346283754,-1074834.8672659395,-1054922.0204397694,-1034655.9964349773,-1014030.8909262929,-993040.7031013521,-971679.3341034164,-949940.585449422,-927818.1574228059,-905305.6474405632,-882396.5483939203,-859084.2469631699,-835362.0219047822,-811223.0423116165,-786660.3658451668,-761666.9369396011,-736235.5849767902,-710359.0224329352,-684029.8429947279,-657240.5196459193,-629983.4027231329,-602250.7179408104,-574034.564384073,-545326.91247051,-516119.6018784675,-486404.33944273414,-456172.69701654185,-425416.1092995028,-394125.8716304437,-362293.1377459604,-329908.91750222025,-296964.07456090255,-263449.3240381719,-229355.2301161578,-194672.20361587824,-159390.49953248538,-123500.21453019883,-86991.28439765936,-49853.48146284232,-12076.411966664717,26350.486605789047,65437.94823531527,105196.88112319261,145638.3704889114,186773.68141274992,228614.2617233917,271171.7449323111,314457.95321349613,358484.90043200785,403264.7952201036,448810.0441024909,495133.254671148,542247.2388115176,590165.0159785091,638899.8165261727,688465.08508963,738874.4840209587,790141.8968796218,842281.4319792404,895307.4259891547,949234.4475949402,1004077.3012163318,1059851.03078445,1116570.9235789198,1174252.5141269173,1232911.5881624129,1292564.186650191,1353226.6098729074,1414915.4215833098,1477647.4532223078,1541439.8082050434,1606309.8662730474,1672275.2879176293,1739354.0188725158,1807564.294678046,1876924.6453176988,1947453.899929395,2019171.1915892772,2092095.9621739085,2166247.967298347,2241647.2813328113,2318314.3025000426,2396269.758051292,2475534.709526813,2556130.558098695,2638079.049998739,2721402.282032128,2806122.707180076,2892263.140288531,2979846.763849931,3068897.1338752853,3159438.185859694,3251494.2408421803,3345090.0115631092,3440250.6087163077],[-1212692.42639578,-1194807.8279601121,-1176603.728472224,-1158074.7727976376,-1139215.518170562,-1120020.4327782963,-1100483.8943231918,-1080600.188561578,-1060363.5078193806,-1039767.9494835844,-1018807.5144703027,-997476.1056676537,-975767.5263540011,-953675.4785909245,-931193.5615904548,-908315.270055929,-885033.9924969794,-861343.009516834,-837235.4920727082,-812704.4997082169,-787742.978757615,-762343.7605209851,-736499.5594110664,-710202.9710695988,-683446.47045409,-656222.4098937975,-628523.0171148779,-600340.3932334683,-571666.5107176306,-542493.2113158661,-512812.2039529965,-482615.06259237416,-451893.22406401206,-420637.9858576348,-388840.5038814491,-356491.7901841528,-323582.71064108657,-290103.98260342726,-256046.17250989424,-221399.69345988845,-186154.80274896557,-150301.59936398827,-113830.02143872483,-76729.84366897168,-38990.67468642676,-601.9543901160359,38447.04876339296,78167.24051434314,118569.70359729929,159665.70058345422,201466.6767680184,243984.26310431957,287230.27918329975,331216.7362618088,375955.8403384662,421459.99527864065,467741.80598901166,514814.08164351666,562689.838959035,611382.305524827,660904.9231841583,711271.3514699326,762495.4710947955,814591.3874976756,867573.4344449951,921456.177690933,976254.418695041,1031983.1983991689,1088657.8010642729,1146293.7581691765,1204906.8523694738,1264513.1215212657,1325128.8627678761,1386770.6366917752,1449455.2715323018,1513199.8674714314,1578021.8009855757,1643938.7292686673,1710968.5947244302,1779129.629530259,1848440.3602733798,1918919.612661826,1990586.516307848,2063460.509589753,2137561.344589635,2212909.09210966,2289524.1467690794,2367427.2321798126,2446639.406206599,2527182.0663094227,2609076.9549710173,2692346.1652101553,2777012.146183873,2863097.7088758075,2950626.031877545,3039620.667260283,3130105.546539937,3222104.986736582,3315643.69653143,3410746.782518527],[-1238522.3419398316,-1220665.9554709112,-1202490.2616425478,-1183989.9069492365,-1165159.4502695175,-1145993.3614505248,-1126486.0198701418,-1106631.71297606,-1086424.6348015512,-1065858.8844571144,-1044928.4645987034,-1023627.2798707595,-1001949.1353246204,-979887.734811676,-957436.6793507545,-934589.4654691622,-911339.4835178447,-887680.0159588705,-863604.2356259886,-839105.2039572282,-814175.8691993158,-788809.064583069,-762997.5064703806,-736733.7924707688,-710010.3995283295,-682819.6819779242,-655153.8695705028,-627005.0654663849,-598365.2441974275,-569226.2495957087,-539579.7926896266,-509417.44956631213,-478730.6591999703,-447510.7212451403,-415748.7937956669,-383435.89110693755,-350562.88128224295,-317120.48392218887,-283099.2677366333,-248489.64811802516,-213281.8846771354,-177466.07873841142,-141032.1707958202,-103969.93792820163,-66268.99117336096,-27918.772859772667,11091.446103002876,50772.5689784172,91135.67600019835,132192.02721439535,173953.06536653265,216430.4188354551,259635.90461256402,303581.5313298479,348279.50233547995,393742.21881854534,439982.2829833324,487012.50127498293,534845.8876549499,583495.6669301088,632975.2781341425,683298.3779628566,734478.8442640426,786530.7795836693,839468.5147668589,893306.6126178326,948059.871617205,1003743.3296986409,1060372.2680853466,1117962.215188521,1176528.9505659537,1236088.508945426,1296657.1843111394,1358251.5340552982,1420888.3831954994,1484584.8286601827,1549358.24364011,1615226.2820110884,1682206.8828259124,1750318.2748779026,1819578.9813367217,1890007.8244589847,1961623.9303713292,2034446.7339318865,2108495.983667654,2183791.746790417,2260354.414293377,2338204.706126381,2417363.6764556705,2497852.71900595,2579693.5724874865,2662908.3261090033,2747519.425179531,2833549.6767963283,2921022.2556258147,3009960.7097748606,3100388.966755455,3192331.339543716,3285812.5327364854,3380857.6488025626],[-1264713.6166639584,-1246885.7338778297,-1228738.7394263512,-1210267.2814512947,-1191465.920494053,-1172329.1280803278,-1152851.2852824468,-1133026.681258574,-1112849.5117686759,-1092313.8776662934,-1071413.783366963,-1050143.135291408,-1028495.7402841293,-1006465.3040067044,-984045.4293053856,-961229.6145522806,-938011.2519607176,-914383.6258728902,-890339.9110206037,-865873.1707580637,-840976.3552664684,-815642.2997295891,-789863.7224809544,-763633.2231205849,-736943.2806021373,-709786.2512892841,-682154.3669812472,-654039.7329062703,-625434.3256839861,-596329.9912543027,-566718.4427737142,-536591.2584779281,-505939.8795103971,-474755.60771581344,-443029.60339927603,-410752.8830487621,-377916.3170216973,-344510.6271945869,-310526.3845751928,-275954.0068760845,-240783.75605060672,-205005.7357884487,-168609.88897175575,-131585.99509068253,-93923.66761775967,-55612.35133980401,-16641.31964860717,23000.32821292663,63323.66994779697,104339.96304668719,146060.64767487627,188497.34960582713,231661.88320014533,275566.2544333623,320222.66397126066,365643.51029433403,411841.39287180174,458829.1153869848,506619.6890124874,555226.3357390263,604662.49175653,654941.810889177,706078.1680849441,758085.6629615328,810978.6234070281,864771.6092395447,919479.4159242385,975117.0783496303,1031699.874663814,1089243.330172561,1147763.22129763,1207275.5795998443,1267796.6958651743,1329343.1242560134,1391931.6865282455,1455579.4763163333,1520303.8634844813,1586122.498548993,1653053.3171699154,1721114.544714172,1790324.7008910598,1860702.6044624913,1932267.378025706,2005038.4528743797,2079035.5739356247,2154278.8047855473,2230788.532745475,2308585.4740568376,2387690.6791405096,2468125.53793849,2549911.785340622,2633071.5066970726,2717627.143419794,2803601.4986700322,2891017.7431388474,2979899.420917993,3070270.455464142,3162155.1556575256,3255578.221958033,3350564.752656056],[-1291266.1604209216,-1273467.0305834967,-1255348.9861707008,-1236906.676981566,-1218134.6652303594,-1199027.4241314854,-1179579.3364619163,-1159784.6931005858,-1139637.69154453,-1119132.4344007971,-1098262.927855062,-1077023.0801149665,-1055406.699828888,-1033407.4944794278,-1011019.0687511486,-988234.9228719717,-965048.4509286862,-941452.9391547942,-917441.5641914334,-893007.3913203485,-868143.3726686758,-842842.3453847019,-817097.0297852436,-790900.0274725768,-764243.8194217675,-737120.7640372643,-709523.0951785978,-681442.9201540998,-652872.2176834175,-623802.8358266661,-594226.4898808897,-564134.7602429441,-533519.0902381986,-502370.7839142792,-470681.00380043336,-438440.7686302215,-405640.95102831395,-372272.2751603408,-338325.3143452876,-303790.48862929223,-268658.0623218226,-232918.14149150578,-196560.67142243544,-159575.43403000245,-121952.04523547506,-83679.95229816437,-44748.43110635644,-5146.583423700649,35136.66590755433,76112.57180810673,117792.57184822345,160188.28918109555,203311.53552152496,247174.31417344417,291788.8231049734,337167.45807259995,383322.8157949508,430267.69717784366,478015.1105892365,526578.2751877476,575970.6243034769,626205.8088727472,677297.7009273688,729260.3971402775,782108.2224258771,835855.7335993978,890517.7230936051,946109.2227347936,1002645.5075787217,1060142.0998083558,1118614.772691904,1178079.5546055413,1238552.7331191842,1300050.8591474341,1362590.751166307,1426189.4994979887,1490864.4706616607,1556633.311795523,1623513.9551480585,1691524.6226408565,1760683.8305037334,1831010.393984608,1902523.4321318846,1975242.3726551565,2049186.956861856,2124377.2446724265,2200833.6197161977,2278576.7945058476,2357627.815696387,2438008.0694264346,2519739.286744539,2602843.549121268,2687343.294050263,2773261.3207353307,2860620.795870572,2949445.2595108063,3039758.631035424,3131585.215206556,3224949.708324769,3319877.2044794564],[-1318185.2008039085,-1300415.0756594124,-1282326.2344370726,-1263913.3286031988,-1245170.9220550694,-1226093.4897059652,-1206675.4160477936,-1186910.993690634,-1166794.4218790284,-1146319.8049840813,-1125481.150972219,-1104272.3698487284,-1082687.2720766652,-1060719.5669705367,-1038362.8610642098,-1015610.6564524879,-992456.3491067868,-968893.2271631842,-944914.4691834955,-920513.1423884553,-895682.2008626817,-870414.483730614,-844702.7133040994,-818539.4931995065,-791917.3064252406,-764828.513438558,-737265.3501714526,-709219.9260245534,-680684.22182989,-651650.0877802181,-622109.2413256872,-592053.2650369387,-561473.6044340422,-530361.5657804175,-498708.3138424379,-466504.8696123352,-433742.10799522186,-400410.75545916543,-366501.3876478048,-332004.42695438163,-296910.14005813235,-261208.63542038715,-224889.86074108072,-187943.60037484113,-150359.47270578938,-112126.92747998517,-73235.24309656536,-33673.523854416795,6569.302844368853,47504.48933575163,89143.47058014013,131497.86709545506,174579.48793558404,218400.33371760882,262972.5996966115,308308.678889574,354421.16524887737,401322.8568870714,449026.75935155526,497546.0889527905,546894.276144865,597084.9689599178,648132.0364971347,700049.572468061,752851.8987966767,806553.5692784307,861169.3732966399,916714.3395982021,973203.7401291798,1030653.093932251,1089078.1711044312,1148494.996819451,1208919.8554132232,1270369.294534442,1332860.1293609971,1396409.4468843723,1461034.6102601471,1526753.2632296532,1593583.3346109209,1661543.042861117,1730650.9007113385,1800925.7198761078,1872386.615835396,1945053.0126949698,2018944.6481226552,2094081.5783631396,2170484.183333472,2248173.1717971237,2327169.586622607,2407494.8101243456,2489170.5694885906,2572218.942285132,2656662.3620679234,2742523.624061773,2829825.890942068,2918592.6987047605,3008847.962629812,3100615.983338923,3193921.452950809,3288789.461331132],[-1345479.5937908897,-1327738.758008366,-1309679.4064828008,-1291296.1923635933,-1272583.681247002,-1253536.3497613308,-1234148.5841297403,-1214414.678709904,-1194328.834510521,-1173885.1576835604,-1153077.6579931912,-1131900.2472594862,-1110346.737777522,-1088410.8407112195,-1066086.1644614453,-1043366.2130077428,-1020244.3842242253,-996713.9681677723,-972768.1453393281,-948399.9849172339,-923602.4429623892,-898368.3605944116,-872690.4621393706,-846561.3532471375,-819973.5189790723,-792919.3218650345,-765390.9999294986,-737380.6646856344,-708880.2990982675,-679881.755513445,-650376.7535552909,-620356.8779893266,-589813.5765516004,-558738.1577428663,-527121.7885883774,-494955.4923610147,-462230.1462685405,-428936.4791039049,-395065.06885807216,-360606.3402943085,-325550.5624847999,-289887.84630696056,-253608.1419002344,-216701.23608237598,-179156.74972453015,-140964.13508389937,-102112.67309515178,-62591.4706173297,-22389.457637583837,18504.61556986021,60102.18132980494,102414.85749890516,145454.45044384012,189232.9580682777,233762.57288741274,279055.68515159097,325124.8860195009,371982.9707827261,419642.94214006653,468118.0135255349,517421.61248856597,567567.3841282004,618569.194581748,670441.1345698182,723197.5229960862,776852.9106060304,831422.0837029968,886920.067923638,943362.1320731258,1000763.7920223577,1059140.8146652686,1118509.2219409454,1178885.2949187434,1240285.5779485484,1302726.8828768427,1366226.2933307942,1430801.1690683737,1496469.150399698,1563248.1626775912,1631156.4208596935,1700212.4341428746,1770435.0106723816,1841843.2623234647,1914456.6095613977,1988294.7863773163,2063377.8453026707,2139726.162504326,2217360.442958257,2296301.7257077317,2376571.389203823,2458191.1567309042,2541183.1019179593,2625569.654338795,2711373.605198316,2798618.1131118266,2887326.7099746005,2977523.306924893,3069232.200401227,3162478.078297243,3257286.026211185],[-1373153.2123787617,-1355441.940216857,-1337412.3543180116,-1319059.1095281797,-1300376.773156188,-1281359.8235590926,-1262002.6487051158,-1242299.5447135472,-1222244.7143713394,-1201832.2656255795,-1181056.2100525529,-1159910.461301659,-1138388.8335146506,-1116485.0397196717,-1094192.6901995263,-1071505.290833608,-1048416.2414139579,-1024918.8339336484,-1001006.2508482698,-976671.563309463,-951907.7293702613,-926707.5921614321,-901063.8780394306,-874969.1947039182,-848416.0292856793,-821396.7464038066,-793903.5861920302,-765928.6622930146,-737463.9598215015,-708501.3332941057,-679032.5045263781,-649049.0604963377,-618542.4511738312,-587503.9873148948,-555924.838221811,-523796.0294664423,-491108.44057771284,-457852.8026921351,-424019.696166893,-389599.54815432243,-354582.6301388093,-318959.0554333008,-282718.7766363714,-245851.58304873947,-208347.09804860223,-170194.7764245239,-131383.9016671353,-91903.58321626438,-51742.75366494944,-10890.1659186529,30665.60969057679,72936.1883366569,115933.3736773734,159669.16088109463,204155.73970099213,249405.49759843294,295431.0229158718,342245.10810111463,389860.75298135774,438291.16809088085,487549.77805100894,537650.2250039387,588606.3721011458,640432.3070480986,693142.3457037164,746751.0357387969,801273.1603517774,856723.7420438388,913118.0464538359,970471.5862551341,1028800.1251126179,1088119.6817044918,1148446.5338070309,1209797.222444526,1272188.5561050023,1335637.615023973,1400161.7555341842,1465778.6144866375,1532506.1137407715,1600362.4647262292,1669366.173076883,1739536.043339639,1810891.1837556828,1883451.0111201098,1957235.255717413,2032263.9663355285,2108557.515360551,2186136.603950028,2265022.267290782,2345235.8799389773,2426799.161245237,2509734.18086553,2594063.36436097,2679809.498883687,2766995.7389556803,2855645.6123379744,2945783.025993209,3037432.2721424545,3130618.034419649,3225365.394120633],[-1401209.4894141671,-1383528.040293701,-1365528.4808866493,-1347205.4677457786,-1328553.569902829,-1309567.2674540335,-1290240.9501232237,-1270568.9158018802,-1250545.3690659245,-1230164.4196683264,-1209420.0810083759,-1188306.2685757445,-1166816.7983699257,-1144945.3852944167,-1122685.6415252124,-1100031.074852878,-1076975.0869987924,-1053510.9719037542,-1029631.9139896177,-1005330.9863930272,-980601.1491709563,-955435.2474772388,-929826.0097107121,-903766.0456329456,-877247.844456346,-850263.7729015586,-822806.0732239794,-794866.8612082768,-766438.1241317492,-737511.7186942981,-708079.3689157418,-678132.6639995375,-647663.0561623727,-616661.858428746,-585120.2423912424,-553029.2359340885,-520379.7209208617,-487162.4308452669,-453367.94844440743,-418986.70327356784,-384008.96924326755,-348424.8621160849,-312224.33696392365,-275397.1855847854,-237933.03387834178,-199821.33917910838,-161051.38754835865,-121612.2910215226,-81492.98481245479,-40682.22447293624,831.4169929418713,43059.552061029244,86013.98166477168,129706.69822162436,174149.888707045,219355.9377775453,265337.4309433601,312107.1577923768,359678.115263917,408063.5109761418,457276.76660568453,507331.5213212264,558241.6352715627,610021.1931300354,662684.5076936875,716246.1235413468,770720.8207491315,826123.6186651713,882469.779744261,939774.8134443518,998054.4801832656,1057324.795360105,1117602.0334397242,1178902.7321022958,1241243.696458737,1304642.0033340477,1369115.0056167664,1434680.3366796058,1501355.9148693043,1569159.948067984,1638110.9383268682,1708227.6865746626,1779529.2973984862,1852035.1839030702,1925765.0726458756,2000739.0086507895,2076977.3605024042,2154500.8255189527,2233330.4350096984,2313487.5596146234,2394993.9147291337,2477871.566014532,2562142.9349973993,2647830.804755047,2734958.3256939407,2823549.021418435,2913626.7946929093,3005215.933498179,3098341.1171854343,3193027.422724892],[-1429653.9695656868,-1412002.6054945984,-1394033.336043608,-1375740.8194826422,-1357119.6265767803,-1338164.239171934,-1318869.0487581387,-1299228.3550097512,-1279236.3643024215,-1258887.1882058766,-1238174.8419533486,-1217093.242885831,-1195636.2088716717,-1173797.4567009674,-1151570.6004542008,-1128949.1498445135,-1105926.508534152,-1082495.9724232221,-1058650.7279115447,-1034383.8501325569,-1009688.3011590724,-984556.9281799721,-958982.461648579,-932957.5134005563,-906474.5747422299,-879526.0145081931,-852104.0770880131,-824200.8804209637,-795808.4139596096,-766918.5365999592,-737522.9745790241,-707613.319338737,-677181.0253557749,-646217.4079363458,-614713.6409766946,-582660.7546869074,-550049.6332788086,-516871.0126169843,-483115.47783230804,-448773.4608969181,-413835.23816156154,-378290.92785260733,-342130.4875295856,-305343.7115022051,-267920.2282061572,-229849.49753657356,-191120.80814014282,-151723.27466282854,-111645.83495435165,-70877.24722796539,-29406.087175111752,12779.254966557492,55690.577390585095,99339.86974783335,143739.3162201927,188901.29864337156,234838.39967926685,281563.40603961004,329089.3117594817,377429.3215243821,426596.85404961,476605.5455135121,527469.2530452535,579202.0582689638,631818.2709025429,685332.4324154868,739759.3197440142,795113.9490655358,851411.5796329263,908667.717670735,966898.1203315421,1026118.7997170426,1086346.026962188,1147596.3363844235,1209886.5296987216,1273233.680300631,1337655.1376153585,1403168.5315180393,1469791.776823226,1537543.0778459148,1606440.9330348661,1676504.1396806356,1747751.798696096,1820203.3194752932,1893878.4248281764,1968797.155993882,2044979.877734648,2122447.2835083315,2201220.400725412,2281320.596088281,2362769.5810155147,2445589.417151958,2529802.521967671,2615431.674442952,2702500.0208463687,2791031.0806030305,2881048.7522562817,2972577.319523699,3065641.4574505733,3160266.2386580775],[-1458497.1074468642,-1440876.1344060334,-1422937.4629287892,-1404675.7530203834,-1386085.5771976053,-1367161.4190746227,-1347897.6719264223,-1328288.6372292025,-1308328.5231774962,-1288011.4431771527,-1267331.4143149124,-1246282.355802859,-1224858.0873981894,-1203052.3277977786,-1180858.6930070177,-1158270.6946822528,-1135281.7384474333,-1111885.1221830535,-1088074.034288178,-1063841.551914571,-1039180.6391726292,-1014084.1453082685,-988544.8028515026,-962555.2257345587,-936107.9073803758,-909195.2187604222,-881809.4064216369,-853942.5904813344,-825586.7625910279,-796733.7838667817,-767375.382786999,-737503.1530565536,-707108.551436827,-676182.8955407431,-644717.3615934795,-612702.9821564956,-580130.6438156869,-546991.0848326341,-513274.8927583359,-478972.50200847164,-444074.19140094845,-408570.08165318985,-372450.13283989485,-335704.141810318,-298321.739564341,-260292.38858614117,-221605.38013661932,-182249.83150134282,-142214.683195299,-101488.69612294715,-60060.44869312411,-17918.333887360524,24949.44371725805,68554.87097394979,112910.12923964811,158027.59749751212,203919.85552901123,250599.6871372927,298080.0834203642,346374.2460979228,395495.59089038894,445457.7509518671,496274.5803576256,547960.1576479096,600528.789426446,653995.01401794,708373.605182915,763679.5758918109,819928.1821589721,877134.9269384961,935315.5640802667,994486.1023507188,1054662.8095166069,1115862.2164939055,1178101.1215625228,1241396.5946489526,1305765.981675034,1371226.908977833,1437797.2877987954,1505495.3188443976,1574339.496919062,1644348.6156328549,1715541.772181613,1787938.3722054036,1861558.1347228824,1936421.0971441604,2012547.6203643451,2089958.3939356487,2168674.44132399,2248717.125247835,2330108.153102116,2412869.5824678596,2497023.8267106875,2582593.6606654827,2669602.226413893,2758073.0391522422,2848029.9931527367,2939497.3678190294,3032499.833839245,3127062.4594336986],[-1487740.3346141893,-1470150.0245714248,-1452242.2245811792,-1434011.5963954092,-1415452.714294089,-1396560.0636711933,-1377328.0395983395,-1357750.9453653304,-1337822.9909974649,-1317538.291748665,-1296890.8665712716,-1275874.6365606312,-1254483.423375076,-1232710.9476306613,-1210550.8272701812,-1187996.5759058057,-1165041.6011359054,-1141679.2028341887,-1117902.5714119177,-1093704.7860522422,-1069078.8129162944,-1044017.5033203352,-1018513.5918844759,-992559.6946510102,-966148.3071731345,-939271.8025729728,-911922.4295687007,-884092.3104696814,-855773.4391404947,-826957.6789315357,-797636.7605769597,-767802.2800590945,-737445.6964386636,-706558.3296500645,-675131.3582623086,-643155.8172032558,-610622.5954480437,-577522.4336705159,-543845.9218572569,-509583.49688304216,-474725.44004865736,-439261.87457842706,-403182.7630782011,-366477.904952927,-329136.93378294073,-291149.31465790747,-252504.3414695002,-213191.13415961014,-173198.635925367,-132515.61037945375,-91130.63866529148,-49032.11652560718,-6208.251325602643,37352.940972652286,81663.63888332015,126736.21851905715,172583.25676277233,219217.5344907255,266652.03984541725,314899.9715620931,363974.7423475045,413889.98231256194,464659.54245949024,516297.4982253327,568818.1530801514,622236.042184215,676565.9361024904,731822.8445784166,788022.0203676028,845178.9631332918,903309.4234021045,962429.4065844268,1022555.1770578632,1083703.2623157706,1145890.4571816213,1209133.8280913294,1273450.71744161,1338858.7480095294,1405375.8274412672,1473020.1528124218,1541810.2152605373,1611764.8046924197,1682903.0145638888,1755244.2467378373,1828808.2164182225,1903614.9571625204,1979684.8259748626,2057038.5084777465,2135697.024168226,2215681.7317563593,2297014.334588687,2379716.886157414,2463811.7956985645,2549321.8338761176,2636270.1385591524,2724680.220689272,2814575.970241398,2905981.6622788366,2998921.9631058923,3093421.936515114],[-1517388.906093961,-1499829.5290614674,-1481952.872064733,-1463753.5986129954,-1445226.284760063,-1426365.4176904818,-1407165.3942833378,-1387620.5196529606,-1367725.0056664278,-1347472.9694368553,-1326858.4317933794,-1305875.3157259023,-1284517.4448052754,-1262778.5415782232,-1240652.2259365122,-1218132.0134598042,-1195211.313732665,-1171883.428633875,-1148141.5505988726,-1123978.7608542356,-1099388.0276240252,-1074362.2043070893,-1048894.0276260006,-1022976.1157456066,-996600.9663619355,-969760.9547604334,-942448.3318432986,-914655.2221248494,-886373.6216957348,-857595.396153762,-828312.2785020866,-798515.8670138088,-768197.6230624388,-737348.8689174089,-705960.7855052177,-674024.4101339318,-641530.6341817975,-608470.2007489775,-574833.7022717516,-540611.578098265,-505794.11202654196,-470371.42980230646,-434333.4965771972,-397670.11432658555,-360370.91922612814,-322425.3789859754,-283822.7901437008,-244552.27531276504,-204602.78038782394,-163963.07170528965,-122621.73315881379,-80567.1632681014,-37787.57220237842,5729.021244939882,49994.79272908205,95022.1154758567,140823.56345316302,187411.91459375992,234800.15406789584,283001.47760948585,332029.2948945728,381897.2329736869,432619.1397586847,484209.08756591985,536681.3767141877,590050.53918151,644331.3423193726,699538.7926261364,755688.1395804188,812794.8795362194,870874.7596783168,929943.7820423376,990018.2075978415,1051114.5603965246,1113249.631786229,1176440.4846928804,1240704.457968533,1306059.1708105342,1372522.5272499179,1440112.72071133,1508848.238645194,1578747.8672346552,1649830.696174906,1722116.1235309145,1795623.8606709703,1870373.937278775,1946386.706446162,2023682.849844418,2102283.382980032,2182209.6605327567,2263483.3817786486,2346126.596098853,2430161.708577297,2515611.4856844037,2602499.061053809,2690847.941349333,2780682.0122253154,2872025.544381273,2964903.199713998,3059340.0375643633],[-1547448.7031193143,-1529920.5318102203,-1512075.292026978,-1493907.6490467545,-1475412.180707939,-1456583.3759964781,-1437415.6336098297,-1417903.2604978706,-1398040.470380548,-1377821.382241363,-1357240.0187975161,-1336290.304944855,-1314966.0661782196,-1293261.0269865475,-1271168.8092222407,-1248682.9304442245,-1225796.8022350906,-1202503.7284906546,-1178796.9036825707,-1154669.4110930033,-1130114.2210211891,-1105124.1889609036,-1079692.0537496584,-1053810.4356874006,-1027471.8346256542,-1000668.6280259478,-973393.0689874005,-945637.2842422454,-917393.2721203051,-888652.9004800068,-859407.9046068033,-829649.8850779925,-799370.3055934203,-768560.4907711912,-737211.6239090583,-705314.7447091723,-672860.7469669082,-639840.3762228324,-606244.2273771758,-572062.7422657884,-537286.2071984308,-501904.75045680255,-465908.33975306805,-429286.77964785835,-392029.70892713405,-354126.5979366214,-315566.7458750182,-276339.2780427197,-236433.14304739004,-195837.10996479122,-154539.76545455027,-112529.51082924102,-69794.55907813087,-26322.93184213573,17897.543658729177,62879.237747007515,108634.72145899711,155176.76976726577,202518.3648529225,250672.6994315558,299653.18013135483,349473.4309251611,400147.29661701806,451688.84638501797,504112.377378908,557432.4183765813,611663.7334979768,666821.3259781878,722920.4420004464,779976.5745909936,838005.4675740851,897023.1195917358,957045.7881864468,1018089.9939490925,1080172.524732511,1143310.4399331608,1207521.0748387459,1272822.04504704,1339231.2509539141,1406766.8823128575,1475447.422866818,1545291.6550547066,1616318.6647903812,1688547.8463199567,1761998.907154975,1836691.8730840776,1912647.0932653449,1989885.2453971868,2068427.3409736995,2148294.7306222976,2229509.1095263287,2312092.522933455,2396067.3717528805,2481456.4182386636,2568282.791765986,2656569.994697652,2746341.9083440206,2837622.799017149,2930437.3241825076,3024810.5387052405],[-1577930.074164697,-1560433.4215442892,-1542619.9139748905,-1524484.2185193817,-1506020.914818694,-1487224.4936783027,-1468089.3556323925,-1448609.8094849044,-1428780.070827372,-1408594.2605325964,-1388046.403224951,-1367130.4257255404,-1345840.155472685,-1324169.3189172654,-1302111.5398922425,-1279660.3379559068,-1256809.1267092302,-1233551.2120856128,-1209879.7906136897,-1185787.947652199,-1161268.655596737,-1136314.7720574646,-1110919.0380084629,-1085074.0759066592,-1058772.3877811905,-1032006.353292024,-1004768.2277577629,-977050.1401513871,-948844.091064946,-920141.950640793,-890935.4564702385,-861216.2114585852,-830975.6816560552,-800205.1940537379,-768895.9343452367,-737038.9446515911,-704625.1212104172,-671645.2120280468,-638089.8144942955,-603949.3729586415,-569214.1762687494,-533874.3552687955,-497919.8802581909,-461340.5584098785,-424126.03114745626,-386265.77147985715,-347749.08129480993,-308565.08860787563,-268702.74476824794,-228150.82161988737,-186897.90861753654,-144932.4098961004,-102242.54129466088,-58816.32733175438,-14641.598133054562,30294.013689956628,76003.07621088019,122498.36141064065,169792.84844946628,217899.72699248232,266832.40058755456,316604.49009702215,367229.83718391974,418722.5078545371,471096.79605566384,524367.2273307913,578548.5625336501,633655.801601008,689704.1873853183,746709.2095492352,804686.6085202959,863652.3795102965,923622.7765976894,984614.3168750782,1046643.7846625191,1109728.23578873,1173885.001938411,1239131.6950706672,1305486.2119066813,1372966.7384888413,1441591.7548121917,1511380.0395305748,1582350.674735196,1654523.0508115683,1727916.8713722508,1802552.1582681355,1878449.2566803712,1955628.8402908426,2034111.9165371135,2113919.8319496284,2195074.2775739105,2277597.2944784784,2361511.279351701,2446838.990184605,2533603.5520467213,2621828.462952152,2711537.5998190083,2802755.2245231355,2895505.9900493072,2989814.9467370864],[-1608835.4757362632,-1591370.626683107,-1573589.1378301405,-1555485.6780380192,-1537054.828762374,-1518291.0826404968,-1499188.8420556707,-1479742.4176783843,-1459946.0269843708,-1439793.7927484296,-1419279.7415149363,-1398397.8020431616,-1377141.8037279968,-1355505.4749953954,-1333482.441672162,-1311066.2253293584,-1288250.2415998527,-1265027.7984682654,-1241392.0945339531,-1217336.2172461164,-1192853.1411107245,-1167935.7258684372,-1142576.7146441608,-1116768.7320661882,-1090504.2823557707,-1063775.747385958,-1036575.3847095894,-1008895.325555305,-980727.5727924162,-952063.9988623699,-922896.3436776358,-893216.2124869619,-863015.0737065324,-832284.256716154,-801014.9496211202,-769198.1969773881,-736824.89748094,-703885.8016201982,-670371.5092909997,-636272.4673730214,-601578.9672686057,-566281.1424012752,-530368.965674757,-493832.2468915065,-456660.6301301066,-418843.5910802148,-380370.4343362842,-341230.2906468753,-301412.11412071297,-260904.67938811285,-219696.57871726993,-177776.21908392292,-135131.81919565052,-91751.40646742098,-47622.81394960638,-2733.6772069055587,42928.568852239754,89376.69319756282,136623.67194867972,184682.69170035003,233567.15290002292,283290.67327926075,333867.0913396464,385310.4698949866,437635.0996682849,490855.50294759125,544986.4372991966,600042.899340116,656040.1285703955,712993.6112672719,770919.0844395286,829832.539846506,889750.2280801553,950688.6627121773,1012664.6245069443,1075695.1657023826,1139797.6143568675,1204989.578767341,1271288.9519565431,1338713.916231804,1407282.9478160911,1477014.821553755,1547928.6156887226,1620043.7167210113,1693379.8243390946,1767956.9564307667,1843795.4541746723,1920915.9872103278,1999339.5588927106,2079087.5116289617,2160181.532300203,2242643.657769027,2326496.2804758437,2411762.1541213538,2498464.3994418303,2586626.5100748125,2676272.358518058,2767426.2021827907,2860112.6895444565,2954356.8663880755],[-1640170.2837050236,-1622737.5192798143,-1604988.3317501177,-1586917.3917852575,-1568519.2826665514,-1549788.4988741914,-1530719.4446517143,-1511306.4325474289,-1491543.6819326053,-1471425.3174954911,-1450945.367712003,-1430097.763291232,-1408876.3355963246,-1387274.815040178,-1365286.8294553503,-1342905.9024376853,-1320125.4516640808,-1296938.7871825926,-1273339.1096756405,-1249319.5086952925,-1224872.9608704352,-1199992.3280848481,-1174670.355627003,-1148899.6703094065,-1122672.7785583746,-1095982.0644730835,-1068819.7878538042,-1041178.082198103,-1013048.9526659257,-984424.2740113311,-955295.7884815517,-925655.1036825115,-895493.6904101996,-864802.8804470953,-833573.8643242533,-801797.6890467447,-769465.2557832422,-736567.3175186722,-703094.4766694438,-669037.182660141,-634385.7294625598,-599130.2530944995,-563260.72907905,-526766.9698633617,-489638.6221963172,-451865.1644637268,-413435.9039823469,-374339.9742494095,-334566.33214902785,-294103.75511382753,-252940.8382415753,-211065.99136506533,-168467.43607675284,-125133.20270456234,-81051.12724021962,-36208.848218438216,9406.196453351062,55806.7727166065,103005.85363308853,151016.62270980934,199852.4772762875,249527.03191561298,300054.1219500811,351447.80698304623,403722.37449557055,456892.3435019711,510972.46826264635,565977.7420562422,621923.4010116365,678824.9280017,736698.0565973707,795558.7750862734,855423.3305544206,916308.2330330187,978230.259710995,1041206.4592155186,1105254.1559585715,1170390.9545546146,1236634.7443074952,1304003.7037688447,1372516.3053687336,1442191.3201210215,1513047.822401166,1585105.194802334,1658383.133067357,1732901.65109919,1808681.0860520294,1885742.1035009213,1964105.7026959122,2043793.221898349,2124826.34380224,2207227.1010413268,2291017.8817849876,2376221.4354202384,2462860.8783266037,2550959.6997413356,2640541.7677178965,2731631.3351787566,2824253.046065663,2918431.941584541],[-1671940.7365651992,-1654540.340648627,-1636823.739882057,-1618785.6067544478,-1600420.5263838912,-1581722.9951046107,-1562687.4190316608,-1543308.1126025075,-1523579.2970953868,-1503495.0991235299,-1483049.5491060077,-1462236.5797134468,-1441050.024289088,-1419483.6152446403,-1397530.982430448,-1375185.651479266,-1352441.0421242006,-1329290.4664890564,-1305727.1273517332,-1281744.116379693,-1257334.4123373271,-1232490.8792642874,-1207206.2646254224,-1181473.1974303925,-1155284.1863236122,-1128631.6176435577,-1101507.7534511702,-1073904.7295263144,-1045814.5533330676,-1017229.1019516774,-988140.119977864,-958539.2173885456,-928417.8673734455,-897767.4041317757,-866579.0206345397,-834843.7663501999,-802552.5449345326,-769696.1118835069,-736265.0721487622,-702249.8777145483,-667640.8251370457,-632428.0530433524,-596601.5395910637,-560151.0998873264,-523066.3833667226,-485336.87112676445,-446951.873222176,-407900.5259146928,-368171.7888797363,-327754.4423683374,-286637.08432401903,-244808.12745301006,-202255.79624912236,-158968.1239698748,-114932.94956509722,-70137.91455645254,-24570.459867423866,21782.177397987805,68932.96722631296,116895.09001934063,165681.93997097714,215307.12849806948,265784.48772564204,317128.07402851153,369352.1716275378,422471.2962448625,476500.19881642936,531453.8692637896,587347.540325705,644196.6914516794,702017.0527555388,760824.6090338072,820635.6038470147,881466.5436661183,943334.2020847071,1006255.6240991512,1070248.1304547521,1135329.3220630833,1201517.0844885097,1268829.592506181,1337285.3147323108,1406903.018329125,1477701.7737822933,1549700.9597566202,1622920.2680275957,1697379.708491461,1773099.6142558646,1850100.6468091384,1928403.801273916,2008030.4117430951,2089002.1567007136,2171341.0645285957,2255069.5191018214,2340210.2654702733,2426786.4156330484,2514821.454403222,2604339.2453658837,2695364.036930441,2787920.4684804357,2882033.576617945],[-1704155.3796826932,-1686787.6594561338,-1669103.954504089,-1651098.9391505388,-1632767.2003658372,-1614103.2363539457,-1595101.4551172599,-1575756.1729983843,-1556061.613198677,-1536011.904272658,-1515601.07859903,-1494823.0708265987,-1473671.7162954805,-1452140.74943321,-1430223.8021250393,-1407914.4020579343,-1385205.9710387331,-1362091.8232846458,-1338565.1636868909,-1314619.0860463523,-1290246.571281148,-1265440.4856051772,-1240193.578678265,-1214498.4817259794,-1188347.705629815,-1161733.6389866744,-1134648.5461375255,-1107084.5651640813,-1079033.7058542997,-1050487.847634567,-1021438.7374692312,-991877.9877265645,-961797.0740106106,-931187.3329580808,-900039.9600009064,-868346.0070921783,-836096.3803961934,-803281.8379416082,-769892.987237175,-735920.2828489221,-701354.0239397013,-666184.3517684969,-630401.2471502214,-593994.5278750286,-556953.8460864727,-519268.68561729323,-480928.3592839376,-441922.0061366507,-402238.5886664316,-361866.8899672176,-320795.5108530717,-279012.8669286808,-236507.18561450252,-193266.50312321773,-149278.6613886035,-104531.30494534504,-59011.87775935978,-12707.620006798767,34394.435196744744,82307.46512058936,131044.86079553142,180620.2304443526,231047.4029667345,282340.43148052273,334513.59691769443,387581.41167922225,441558.6233472689,496460.2184566329,552301.426326077,609097.7229514085,666864.8349587843,725618.7436227109,785375.6889469652,846152.1738106878,907964.9681802047,970831.1133887912,1034767.9264824586,1099793.0046369522,1165924.22964379,1233179.7724679704,1301578.0978777888,1371137.9691494303,1441878.4528439776,1513818.92366269,1586979.0693781697,1661378.895843991,1737038.7320849523,1813979.2354658926,1892221.3969449387,1971786.5464089843,2052696.3580941614,2134972.856092004,2218638.4199445248,2303715.7903252435,2390228.0748132,2478198.753757228,2567651.6862335093,2658611.1160974405,2751101.678132936,2845148.4042963716],[-1736821.1608904223,-1719486.431243096,-1701835.9389483589,-1683864.3601816618,-1665566.2837822814,-1646936.2098407068,-1627968.548263645,-1608657.6173160027,-1588997.642139583,-1568982.7532476953,-1548606.9849963665,-1527864.2740304207,-1506748.457704929,-1485253.272481466,-1463372.352298619,-1441099.2269161844,-1418427.3202335224,-1395349.9485802727,-1371860.318980147,-1347951.527386864,-1323616.5568918607,-1298848.275903077,-1273639.4362953294,-1247982.6715303229,-1221870.4947470336,-1195295.296821445,-1168249.3443954324,-1140724.777873605,-1112713.6093891005,-1084207.7207359727,-1055198.8612689446,-1025678.645769618,-995638.5522785624,-965069.9198924331,-933963.9465267833,-902311.6866422202,-870104.04893474,-837331.7939891049,-803985.5318948212,-770055.7198236021,-735532.6595691536,-700406.4950467525,-664667.2097533019,-628304.6241869065,-591308.3932253174,-553668.0034619682,-515372.7705007782,-476411.83620655024,-436774.16591214295,-396448.5455810125,-355423.5789246247,-313687.684473298,-271229.0926015987,-228035.8425051272,-184095.77912963415,-139396.55005122907,-93925.60230691079,-47670.17917394079,-617.3168993494473,47246.15862409491,95933.63523416314,145458.71792931762,195835.23235329427,247077.2283360646,299198.9834894482,352215.00686164666,406140.04264908796,460989.07396756625,516777.32668313663,573520.2733049304,631233.6369380662,689933.3953012708,749635.7848074725,810357.3047095151,872114.7213115706,934925.0722485813,998805.6708316449,1063774.1104645892,1129848.2691296935,1197046.313944939,1265386.7057934697,1334888.2040277594,1405569.8712462056,1477451.0781480297,1550551.508464009,1624891.1639656764,1700490.3695551585,1777369.7784335623,1855550.3773537986,1935053.491955583,2015900.7921855184,2098114.297802772,2181716.3839737,2266729.786952434,2353177.609854429,2441083.328520247,2530470.7974726553,2621364.2559680175,2713788.334145096,2807768.0592685207],[-1769941.8715277845,-1752640.4250526433,-1735023.4396081185,-1717085.5932323686,-1698821.4766449146,-1680225.5918341817,-1661292.350622717,-1642016.0732093023,-1622390.9866879093,-1602411.2235424079,-1582070.8201180475,-1561363.7150676663,-1540283.7477733723,-1518824.6567429525,-1496980.0779806327,-1474743.5433314207,-1452108.4787996914,-1429068.2028400882,-1405615.9246215404,-1381744.7422633737,-1357447.6410432726,-1332717.491576236,-1307547.04796518,-1281928.9459211722,-1255855.7008540237,-1229319.7059322584,-1202313.2301121878,-1174828.416135027,-1146857.278492872,-1118391.7013613451,-1089423.4364995775,-1059944.1011166368,-1029945.1757038604,-999418.001832231,-968353.7799154173,-936743.566936174,-904578.2741369242,-871848.6646733948,-838545.3512308486,-804658.7936017876,-770179.2962260186,-735097.0056904792,-699401.9081895519,-663083.8269449384,-626132.4195843427,-588537.1754777827,-550287.4130326975,-511372.27694458,-471780.73540447606,-431501.57726178225,-390523.409141975,-348834.65251769405,-306423.54073449597,-263278.1159878783,-219386.2262527533,-174735.5221639201,-129313.45384691749,-83107.26769770915,-36104.00311250193,11709.510835972149,60346.65877439873,109821.04245870467,160146.4842583444,211337.03069689963,263406.95604735287,316370.7659863061,370243.2013054737,425039.2416824498,480774.1095112944,537463.2737949649,595122.45409788,653767.6245631669,713415.0179929142,774081.1299934797,835782.7231865632,898536.8314882391,962360.7644539955,1027272.111694877,1093288.747362859,1160428.8347076713,1228710.8307058807,1298153.4907646244,1368775.8734978423,1440597.345580725,1513637.586680042,1587916.5944629046,1663454.6896862197,1740272.521364592,1818391.0720227365,1897831.6630300577,1978615.9600202464,2060765.9783965289,2144304.0889258403,2229253.023418941,2315635.880503497,2403476.1314873947,2492797.6263153707,2583624.599619914,2675981.676869582,2769893.880611974],[-1803524.1293288297,-1786256.2615633993,-1768673.0801200955,-1750769.264911103,-1732539.4085475723,-1713978.0149273744,-1695079.4978004752,-1675838.1793112422,-1656248.2885175613,-1636303.9598857833,-1615999.231762321,-1595328.0448201194,-1574284.240480481,-1552861.5593096975,-1531053.6393899892,-1508854.0146640593,-1486256.113253878,-1463253.255751772,-1439838.6534846711,-1416005.4067504124,-1391746.5030259162,-1367054.8151463398,-1341923.09945593,-1316343.993928437,-1290310.0162579378,-1263813.56191905,-1236846.9021961747,-1209402.1821809104,-1181471.4187382085,-1153046.498439244,-1124119.1754616802,-1094681.0694562825,-1064723.6633795383,-1034238.301291211,-1003216.1861176991,-971648.3773786086,-939525.7888776409,-906839.1863564576,-873579.1851111641,-839736.2475702683,-805300.6808350342,-770262.6341795325,-734612.096511242,-698338.8937911922,-661432.6864129007,-623882.9665389955,-585679.0553965583,-546810.1005280586,-507265.0729991463,-467032.76456170995,-426101.7847719132,-384460.5580615769,-342097.3207642082,-299000.1180923367,-255156.80106729642,-210555.02339999191,-165182.2383220601,-119025.69536588667,-72072.43709470937,-24309.295779248234,24277.109977920074,73700.37867188407,123974.32937932294,175113.0052987882,227130.67734562932,280041.8478058912,333861.25404743943,388603.8722903924,444284.92143736593,500919.8669655118,558524.4248787593,617114.5657246984,676706.5186744179,737316.7756674914,798962.095622682,861659.5087165344,925426.3207280738,990280.1174545661,1056238.7691964763,1123320.4353139023,1191543.5688552577,1260926.9212606149,1331489.5471374895,1403250.809114921,1476230.382773323,1550448.2616529036,1625924.7623425922,1702680.529647598,1780736.5418413347,1860114.115999564,1940834.9134195368,2022920.9451248422,2106394.5774590364,2191278.537765377,2277595.9201593767,2365370.1913916958,2454625.196804244,2545385.1663806345,2637674.720894005,2731518.878149477],[-1837574.650427017,-1820340.6598854312,-1802791.582584606,-1784922.100322145,-1766726.807612334,-1748200.2102740526,-1729336.7239963713,-1710130.6728809986,-1690576.2879616185,-1670667.7056990082,-1650398.966452821,-1629764.0129282565,-1608756.6885980836,-1587370.7360994618,-1565599.795605052,-1543437.4031678375,-1520876.9890400334,-1497911.875964435,-1474535.2774388513,-1450740.2959526076,-1426519.9211949504,-1401867.0282344145,-1376774.3756698712,-1351234.603751166,-1325240.232470196,-1298783.6596212818,-1271857.1588307105,-1244452.877554304,-1216562.8350438464,-1188178.9202802056,-1159292.8898737687,-1129896.3659313545,-1099980.833889034,-1069537.6403099606,-1038557.9906479262,-1007032.9469742402,-974953.4256688175,-942310.1950743285,-909093.8731129216,-875294.9248644449,-840903.6601070585,-805910.2308175568,-770304.6286322414,-734076.6822672901,-697216.0548980557,-659712.2414958575,-621554.5661236886,-582732.1791873858,-543234.0546437185,-503048.9871637481,-462165.58925113175,-420572.28831381863,-378257.32369035995,-335208.74362756824,-291414.40221058065,-246861.95624392573,-201538.86208304856,-155432.37241456425,-108529.53298675176,-60817.17928644689,-12281.933163766284,37089.800597062334,87311.83776135044,138398.21818384668,190363.2094033528,243221.3102962575,296987.25478732074,351676.015619779,407302.80818520347,463883.09441520646,521432.5867332951,579967.2520714151,639503.3159494102,700057.2666196306,761645.8592772563,824286.1203385247,887995.3517850274,952791.1355790854,1018691.3381483126,1085714.1149416561,1153877.9150576652,1223201.48594744,1293703.87819,1365404.4503459204,1438322.8738868255,1512479.1382033029,1587893.5556934816,1664586.7669300907,1742579.7459119651,1821893.805397755,1902550.6023246213,1984572.143312599,2067980.7902577957,2152799.2660116525,2239050.6601530337,2326758.4348506103,2415946.43081855,2506638.8733664127,2598860.3785465267,2692635.9593959367],[-1872103.2945338008,-1854903.5108507234,-1837388.8693772028,-1819554.0538136573,-1801393.6605950836,-1782902.1974791617,-1764074.0821119875,-1744903.640570772,-1725385.1058833045,-1705512.6165232991,-1685280.2148823629,-1664681.845716844,-1643711.3545700668,-1622362.486169338,-1600628.882797288,-1578504.082636839,-1555981.518090378,-1533054.5140712599,-1509716.2862685036,-1485959.939383458,-1461778.4653384371,-1437164.7414563193,-1412111.5286117855,-1386611.4693522004,-1360657.0859888892,-1334240.7786577682,-1307354.823349078,-1279991.3699052013,-1252142.4399873426,-1223799.9250088232,-1194955.5840358282,-1165601.0416545128,-1135727.7858040682,-1105327.165574822,-1074390.3889720098,-1042908.5206429418,-1010872.479568305,-978273.0367165946,-945100.8126610867,-911346.2751583322,-876999.7366890172,-842051.3519585594,-806491.1153582535,-770308.8583859303,-733494.2470254744,-696036.7790839775,-657925.7814876479,-619150.4075333294,-579699.6340968395,-539562.2587966272,-498726.89711239794,-457181.9794570501,-414915.74820335675,-371916.25466188276,-328171.35601135856,-283668.71218009293,-238395.78267774917,-192339.82337599015,-145487.8832392376,-97826.8010019171,-49343.201793498825,-23.493709666654468,50146.13567099534,101179.7228265605,153091.5318860514,205896.05828259746,259608.03246390168,314242.4236619598,369814.44372256333,426339.55099667935,483833.454291909,542312.1168886316,601791.760619178,662288.8700119625,723820.1965014837,786402.7627061498,850053.8667721716,914791.086788578,980632.2852713335,1047595.6137190112,1115699.5172406323,1184962.739258213,1255404.32628171,1327043.6327622393,1399900.3260211414,1473994.3912574574,1549346.1366360835,1625976.1984543665,1703905.5463932073,1783155.4888502713,1863747.678358254,1945704.1170887663,2029047.1624451084,2113799.5327409925,2199984.3129722304,2287624.960678577,2376745.311898966,2467369.5872208965,2559522.3979273275,2653228.7522381153],[-1907115.6267796475,-1889950.3701419677,-1872470.486564883,-1854670.661666478,-1836545.4938174686,-1818089.4927294462,-1799297.0780208453,-1780162.5777597895,-1760680.2269837651,-1740844.1661951488,-1720648.4398333721,-1700086.9947219712,-1679153.6784910036,-1657842.237974284,-1636146.3175808887,-1614059.4576403482,-1591575.0927220332,-1568686.5499268675,-1545387.0471521972,-1521669.6913287167,-1497527.4766292993,-1472953.2826488086,-1447939.8725555593,-1422479.8912124399,-1396565.8632683775,-1370190.1912192286,-1343345.153437725,-1316022.9021715312,-1288215.461510138,-1259914.725318442,-1231112.4551376696,-1201800.2780527798,-1171969.6845257417,-1141612.0261938856,-1110718.5136339606,-1079280.2140894812,-1047288.0491623776,-1014732.7924676014,-981605.0672504054,-947895.3439651029,-913593.9378161626,-878691.0062591098,-843176.5464619296,-807040.3927259878,-770272.2138658618,-732861.510546736,-694797.6125806295,-656069.676178176,-616666.6811572779,-576577.4281070367,-535790.535506675,-494294.43679780187,-452077.3774113762,-409127.411745911,-365432.40009820927,-320980.0055450401,-275757.69077526825,-229752.71487079747,-182952.1300376542,-135342.77828358952,-86911.28804341378,-37644.07075051544,12472.682646109723,63453.00522107119,115311.15766654629,168061.63194506709,221719.15499964403,276298.692523133,331815.4527873723,388284.8905341765,445722.71092646476,504144.8735640114,563567.5965622156,624007.3606959186,685480.9136089282,748005.2740915432,811597.7364240037,876275.8747910811,942057.5477658836,1008960.9028650159,1077004.381176082,1146206.7220597821,1216586.967924436,1288164.4690788044,1360958.8886606796,1434990.2076439597,1510278.729926352,1586845.0874955333,1664710.2456797785,1743895.5084807426,1824422.5239912504,1906313.2898987355,1989590.1590775093,2074275.8452670518,2160393.4288431797,2247966.3626794373,2337018.4781018137,2427573.9909376577,2519657.5076620304,2613294.031638691],[-1942616.982689092,-1925486.5607979782,-1908041.7444882211,-1890277.2213072595,-1872187.5915732556,-1853767.366963593,-1835010.9690809625,-1815912.727996446,-1796466.8807693683,-1776667.56994301,-1756508.8420169672,-1735984.6458944033,-1715088.8313046556,-1693815.1472006699,-1672157.2401307349,-1650108.6525838508,-1627662.8213093122,-1604813.0756086812,-1581552.635600806,-1557874.6104590122,-1533771.996620118,-1509237.6759644735,-1484264.4139676553,-1458844.8578217735,-1432971.5345272191,-1406636.848953726,-1379833.0818705994,-1352552.387944962,-1324786.7937089058,-1296528.195493307,-1267768.3573289937,-1238498.9088143758,-1208711.342948995,-1178397.0139321084,-1147547.1349269722,-1116152.7757885135,-1084204.8607551432,-1051694.166103705,-1018611.3177669784,-984946.7889126921,-950690.897484919,-915833.8037052178,-880365.5075343284,-844275.846093378,-807554.4910439518,-770190.9459258192,-732174.5434534326,-693494.4427679647,-654139.6266463008,-614098.8986652391,-573360.8803207092,-531914.0081003644,-489746.53051081067,-446846.5050561549,-403201.79516903264,-358800.0670926091,-313628.78671302367,-267675.21634064894,-220926.41144152405,-173369.21731524635,-124990.26572067058,-75775.97144775558,-25712.528834934812,25214.091769763734,77018.14760231366,129714.12713513477,183316.75378687866,237840.98969139904,293302.0395265571,349715.35440484993,407096.6358241243,465461.8396830186,524827.1803592918,585209.1348532937,646624.4469971,709090.1317316275,772623.4794496875,837242.0604102211,902963.7292216136,969806.6293965331,1037789.1979789739,1106930.1702459245,1177248.5844815169,1248763.7868293514,1321495.4362207232,1395463.509381217,1470688.3059179867,1547190.453485454,1624990.91303549,1704110.9841497568,1784572.310457022,1866396.8851361172,1949607.0565077579,2034225.533712362,2120275.39248069,2207780.080994809,2296763.4258422772,2387249.6380645554,2479263.319302899,2572829.4680387764],[-1978614.4870338873,-1961519.2106989464,-1944109.7741500596,-1926380.8668757244,-1908327.0911534126,-1889942.9606382127,-1871222.8989291124,-1852161.2381122888,-1832752.2172811255,-1812989.9810321305,-1792868.5779374994,-1772381.9589925425,-1751523.9760385116,-1730288.3801602412,-1708668.8200580429,-1686658.84039334,-1664251.8801084184,-1641441.2707186139,-1618220.2345775648,-1594581.8831145929,-1570519.2150439539,-1546025.1145450717,-1521092.3494144941,-1495713.5691873936,-1469881.303229552,-1443587.9587986378,-1416825.8190746822,-1389587.0411585653,-1361863.6540393948,-1333647.5565285352,-1304930.5151610686,-1275704.162063622,-1245959.9927882017,-1215689.3641109904,-1184883.4917969147,-1153533.4483275567,-1121630.160593221,-1089164.4075481568,-1056126.8178283125,-1022507.8673306219,-988297.8767546723,-953487.0091041108,-918065.2671486107,-882022.4908453682,-845348.3547194516,-808032.3652018057,-770063.8579260209,-731431.9949807203,-692125.7621187661,-652133.9659218164,-611445.2309198263,-570047.9966639248,-527930.514753974,-485080.84581742436,-441486.85644063493,-397136.2160512237,-352016.3937508003,-306114.6550965747,-259418.05883309804,-211913.45357048837,-163587.4744104785,-114426.53951858776,-64416.84664183948,-13544.369570215698,38205.14545657905,90846.18340120837,144393.46413813718,198861.94622224662,254266.83071726048,310623.5650860192,367947.8471408081,426255.6290584509,485563.12145820167,545886.7975448961,607243.397317653,669649.9318465996,733123.6876155571,797682.2309358367,863343.4124291651,930125.3715821304,998046.5413727523,1067125.6529718009,1137381.740516474,1208834.145962311,1281502.524010966,1355406.8471164056,1430567.4105716795,1507004.8376742369,1584740.0849756165,1663794.4476133361,1744189.564727739,1825947.4249644987,1909090.3720659665,1993641.1105484609,2079622.711472509,2167058.6183032375,2255972.652864118,2346389.0213848557,2438332.3206467694,2531827.544222703],[-2015115.3706155894,-1998055.5537888021,-1980681.8126509679,-1962988.8386434931,-1944971.236015088,-1926623.5204105375,-1907940.1174372768,-1888915.3612087753,-1869543.4928647876,-1849818.6590674792,-1829734.910474147,-1809286.2001848903,-1788466.3821655929,-1767269.2096457758,-1745688.3334907538,-1723717.300547378,-1701349.5519640804,-1678578.4214832305,-1655397.1337066104,-1631798.802333011,-1607776.4283676988,-1583322.8983029008,-1558430.9822699132,-1533093.332160895,-1507302.4797210528,-1481050.8346101653,-1454330.6824332497,-1427134.1827393193,-1399453.3669890019,-1371280.1364888023,-1342606.2602927787,-1313423.3730706745,-1283722.972941923,-1253496.419274781,-1222734.9304510814,-1191429.5815944276,-1159571.3022625237,-1127150.874102656,-1094158.9284696989,-1060585.9440056938,-1026422.2441817522,-991657.9947997788,-956283.2014546813,-920287.7069561514,-883661.1887093056,-846393.1560529671,-808472.9475567429,-769889.7282736646,-730632.4869497279,-690690.0331887319,-650050.9945720597,-608703.8137319041,-566636.7453791033,-523837.85328230914,-480295.00719965156,-435995.87976135174,-390927.943302826,-345078.46664660564,-298434.5118343858,-250982.93080561608,-202710.3620238686,-153603.22704933863,-103647.72705690842,-52829.83929797774,-1135.3135075457394,51450.33174752258,104941.81277755974,159354.08453834476,214702.3444591784,271002.0363327628,328268.8542652107,386518.7466906756,445767.9204489463,506032.84492813516,567330.256273054,629677.1616615043,693090.8436465818,757588.86457005,823189.0710448804,889909.598509267,957768.8758527911,1026785.6301173079,1096978.8912701928,1168367.9970558533,1240972.5979230353,1314812.6620305846,1389908.4803337865,1466280.6717492254,1543950.1884040046,1622938.320967183,1703266.7040661834,1784957.3217887827,1868032.513273995,1952514.97838892,2038427.7834984334,2125794.36732514,2214638.54690258,2304984.523622642,2396856.8893804336,2490280.632813676],[-2052128.6867333832,-2035104.6623818916,-2017766.9515654906,-2000110.2476937561,-1982129.1570019978,-1963818.1971402995,-1945171.7957401893,-1926184.28895829,-1906849.9199967352,-1887162.8375994705,-1867117.0945252206,-1846706.6459953033,-1825925.3481168647,-1804766.956280912,-1783225.1235346466,-1761293.3989274849,-1738965.2258312586,-1716233.9402328052,-1693092.7689996378,-1669534.8281178002,-1645553.120901515,-1621140.5361738915,-1596289.846419309,-1570993.705905372,-1545244.6487753529,-1519035.0871099506,-1492357.308958214,-1465203.4763364852,-1437565.6231962938,-1409435.6533588506,-1380805.3384169894,-1351666.315603496,-1322010.0856254273,-1291828.0104634413,-1261111.311136843,-1229851.0654320251,-1198038.205595064,-1165663.5159874407,-1132717.6307043494,-1099191.0311545278,-1065074.0436024345,-1030356.8366702325,-995029.4188003261,-959081.6356773628,-922503.167609198,-885283.5268654614,-847412.0549749047,-808877.9199783592,-769670.1136385542,-729777.448605265,-689188.5555354194,-647891.8801665935,-605875.6803451674,-563128.0230058278,-519636.78110352205,-475389.63049642276,-430374.04677933315,-384577.30206593033,-337986.46172117535,-290588.381040202,-242369.70187505567,-193316.8492075042,-143416.0276674945,-92653.21799527388,-41014.17344886856,11515.58384730434,64950.766611124855,119306.32617068989,174597.45629198942,230839.5970684234,288048.43887040764,346239.9263596656,405430.26256641885,465635.91303166933,526873.610015193,589160.3567714421,652513.4318914246,716950.3937156796,782489.0848164279,849147.6365511157,916944.4736882173,985898.3191076657,1056028.1985736564,1127353.4455857677,1199893.7063057688,1273668.9445629856,1348699.4469401725,1425005.8279378954,1502609.035223327,1581530.3549612295,1661791.4172298103,1743414.2015223433,1826421.0423374963,1910834.6348557249,1996678.0407084702,2083974.6938376082,2172748.406448148,2263023.375055173,2354824.186628143,2448175.8248298024],[-2089662.9696855028,-2072675.084023811,-2055373.7518472727,-2037753.6685510646,-2019809.442374757,-2001535.5929915213,-1982926.5500749943,-1963976.6518431103,-1944680.1435787412,-1925031.1761261735,-1905023.8043642652,-1884651.9856545038,-1863909.5782643931,-1842790.3397657333,-1821287.9254071524,-1799395.8864603152,-1777107.6685403623,-1754416.6098987085,-1731315.939688945,-1707798.7762048643,-1683858.1250904005,-1659486.8775205098,-1634677.808353796,-1609423.5742547035,-1583716.711786194,-1557549.6354717228,-1530914.6358264002,-1503803.8773562005,-1476209.3965260785,-1448123.0996947046,-1419536.7610166627,-1390442.020311024,-1360830.380895936,-1330693.2073881766,-1300021.7234684986,-1268807.0096102804,-1237040.0007724431,-1204711.4840554039,-1171812.096319654,-1138332.321765874,-1104262.4894773844,-1069592.770922414,-1034313.1774168885,-998413.5575467665,-961883.594549248,-924712.8036516542,-886890.5293690688,-848405.9427575837,-809248.0386244259,-769405.6326934136,-728867.3587253541,-687621.6655918607,-645656.8143038349,-602960.8749912581,-559521.7238354911,-515327.0399525557,-470364.302226861,-424620.78609378496,-378083.5602724198,-330739.48344480665,-282575.20088299084,-233577.14102222864,-183731.5119797252,-133024.29801717494,-81441.25594857009,-28967.91148916725,24410.444452843163,78708.7595438296,133942.22385209752,190126.27373697003,247276.59579806542,305409.1308891326,364540.07819490926,424685.8993730019,485863.32276145136,548089.3476542518,611381.2486428199,675756.5800285684,741233.1803045995,807829.1767088906,875562.9898496089,944453.3384051379,1014519.2438964732,1085780.03553783,1158255.3551630904,1231965.1622306579,1306929.7389088804,1383169.6952400208,1460705.9743885305,1539559.8579715784,1619752.9714744492,1701307.2897516578,1784245.1426168163,1868589.2205185122,1954362.580309019,2041588.6511032484,2130291.240230934,2220494.5392830516,2312223.1302556167,2405501.9917880567],[-2127722.9825794618,-2110771.559249661,-2093506.9310978297,-2075923.7955168523,-2058016.7627627584,-2039780.354544089,-2021209.002588994,-2002297.0471892275,-1983038.735720981,-1963428.2211415928,-1943459.5604628634,-1923126.713199346,-1902423.539791934,-1881343.8000063312,-1859881.1513058117,-1838029.1471976407,-1815781.2355537121,-1793130.7569035667,-1770070.9427005358,-1746594.9135599572,-1722695.6774693492,-1698366.127969533,-1673599.0423074586,-1648387.0795586498,-1622722.778720101,-1596598.556772499,-1570006.706711632,-1542939.3955477974,-1515388.662274224,-1487346.4158019875,-1458804.432862508,-1429754.355876362,-1400187.6907881047,-1370095.8048661049,-1339469.924468138,-1308301.1327703134,-1276580.3674602294,-1244298.4183932163,-1211445.9252111884,-1178013.3749230187,-1143991.0994472797,-1109369.2731147725,-1074137.9101316156,-1038286.8620018815,-1001805.8149090474,-964684.2870552177,-926911.6259590304,-888477.0057092081,-849369.424174997,-809577.7001719195,-769090.4705824791,-727896.1874302877,-685983.1149088866,-643339.3263618187,-599952.701215284,-555810.9218617161,-510901.47049387917,-465211.6258877455,-418728.46013560146,-371438.83532559266,-323329.4001691588,-274386.58657457447,-224596.6061660326,-173945.44674654212,-122418.8687060955,-70002.40137101617,-16681.339295976795,37559.26150313113,92734.58737616986,148860.07092748722,205951.39496476902,264024.49651254434,323095.57088861195,383181.0758455121,444297.7357777348,506462.5459968075,569692.777072344,634005.9792442219,699419.9869038528,765952.9231469356,833623.2043983713,902449.5451118159,972450.9625416603,1043646.7815932161,1116056.6397486455,1189700.4920714549,1264598.6162914098,1340771.6179680289,1418240.4357384294,1497026.3466473543,1577150.971562108,1658636.2806731923,1741504.5990836653,1825778.6124845338,1911481.3729229672,1998636.304660758,2087267.2101259641,2177398.2759588393,2269054.0791551033,2362259.5933037745],[-2166316.3965482395,-2149401.76247436,-2132174.16702886,-2114628.3096145433,-2096758.802516167,-2078560.1694900522,-2060026.8443313309,-2041153.1694181592,-2021933.3942327606,-2002361.6738582917,-1982432.0674524147,-1962138.536695741,-1941474.9442156649,-1920435.0519850121,-1899012.5196950233,-1877200.9031019995,-1854993.6523481198,-1832384.1102547336,-1809365.5105886383,-1785930.9763006212,-1762073.5177357702,-1737786.0308148724,-1713061.2951874498,-1687891.9723544505,-1662270.6037613612,-1636189.608860674,-1609641.283143542,-1582617.7961394303,-1555111.1893847494,-1527113.3743580696,-1498616.1303828345,-1469611.1024964242,-1440089.79928526,-1410043.5906848712,-1379463.705745756,-1348341.2303625736,-1316667.1049675718,-1284432.1221870917,-1251626.9244606895,-1218242.0016217981,-1184267.68844074,-1149694.1621275316,-1114511.4397952766,-1078709.3758830419,-1042277.6595376763,-1005205.811953262,-967483.1836693222,-929098.9518247237,-890042.1173683633,-850301.5022252486,-809865.746417522,-768723.3051388967,-726862.4457837599,-684271.2449276438,-640937.5852601202,-596849.1524687591,-551993.4320735298,-506357.7062100321,-459929.0503629041,-412694.3300457918,-364640.19742901623,-315753.0879135425,-266019.2166503966,-215424.57500392757,-163954.92696031975,-111595.8054773002,-58332.50877656182,-4150.096576963551,50966.61373203853,107033.05097873881,164064.89415768255,222078.07644273108,281088.7892631367,341113.4864446046,402168.8884160924,464271.98648446705,527440.0471752035,591690.6166441008,657041.5251581324,723510.8916477896,791117.1283315443,859878.9454150111,929815.3558624377,1000945.6802464649,1073289.5516736312,1146866.9207883775,1221698.0608575041,1297803.5729332361,1375204.391100559,1453921.7878068024,1533977.3792760684,1615393.1310093915,1698191.3633736242,1782394.7572763148,1868026.3599334313,1955109.590727291,2043668.2471577246,2133726.5108874715,2225308.953884905,2318440.5446613873],[-2205450.996980545,-2188573.482404431,-2171383.2516797087,-2153875.006231368,-2136043.360385284,-2117882.83995803,-2099387.8808242856,-2080552.8274613165,-2061371.93147013,-2041839.3500726142,-2021949.1445852735,-2001695.2788678699,-1981071.6177475045,-1960071.9254174454,-1938689.8638103076,-1916918.9909449646,-1894752.7592475205,-1872184.5138448656,-1849207.4908311649,-1825814.815506609,-1801999.500587953,-1777754.4443901542,-1753072.428979643,-1727946.118297221,-1702368.0562514311,-1676330.664781262,-1649826.2418880204,-1622846.9596352684,-1595384.8621176502,-1567431.8633963964,-1538979.7454022793,-1510020.1558049107,-1480544.605848107,-1450544.4681502618,-1420010.9744703912,-1388935.2134376606,-1357308.1282450315,-1325120.5143060691,-1292363.0168743148,-1259026.128624239,-1225100.1871945295,-1190575.3726911778,-1155441.705151158,-1119689.0419655899,-1083307.0752618536,-1046285.3292433289,-1008613.1574879135,-970279.7402022146,-931274.0814325521,-891585.0062313345,-851201.1577783888,-810110.994455744,-768302.7868770366,-725764.6148682646,-682484.364401075,-638449.7244770038,-593648.1839622282,-548067.0283711189,-501693.3366000145,-454513.97760748584,-406515.6070424351,-357684.66381831,-308007.3666329649,-257469.71043222584,-206057.4628187837,-153756.16040229192,-100551.10509210825,-46427.360330919735,8630.252731467132,64637.15912611177,121609.03401301056,179561.80669381283,238511.6646874561,298475.05787087884,359468.70268535754,421509.58641082793,484614.97150608804,548802.4000201477,614089.6980726612,680494.980405814,748036.6550083691,816733.4278143891,886604.3074742942,957668.6102041481,1029945.9647107683,1103456.317195191,1178219.9364367644,1254257.4189556506,1331589.6942597725,1410238.0301738568,1490224.0382534242,1571569.679284452,1654297.2688717432,1738429.4831133317,1823989.3643677062,1911000.3271112018,1999486.1638886826,2089471.0513583883,2180979.556434155,2274036.6425222177],[-2245134.6852415404,-2228294.6237591277,-2211142.0931388726,-2193671.7968402957,-2175878.351242867,-2157756.2842359515,-2139300.0337864906,-2120503.946483597,-2101362.27605996,-2081869.1818891668,-2062018.72745964,-2041804.878823513,-2021221.5030208726,-2000262.3664788285,-1978921.1333849228,-1957191.3640341822,-1935066.5131504158,-1912539.9281798874,-1889604.8475580916,-1866254.398948691,-1842481.597454319,-1818279.3437984216,-1793640.4224787357,-1768557.4998904883,-1743023.1224199373,-1717029.7145073477,-1690569.5766790968,-1663634.8835478644,-1636217.681781687,-1608309.8880397533,-1579903.2868755343,-1550989.528606432,-1521560.1271493384,-1491606.4578212565,-1461119.7551056796,-1430091.1103823124,-1398511.4696210385,-1366371.6310389473,-1333662.2427200517,-1300373.8001964367,-1266496.6439918894,-1232020.9571252586,-1196936.762574391,-1161233.9206996271,-1124902.1266261782,-1087930.907584154,-1050309.6202074303,-1012027.4477880844,-973073.3974877442,-933436.2975042514,-893104.7941933321,-852067.3491436578,-810312.2362065874,-767827.5384772718,-724601.145228222,-680620.7487939075,-635873.8414058387,-590347.7119764541,-544029.4428332285,-496905.90639931103,-448963.76182192285,-400189.4515469768,-350569.1978392536,-300088.9992463505,-248734.62700794404,-196491.6214062241,-143345.288059053,-89280.69415391935,-34282.664622139186,21664.22174874693,78575.63626266224,136467.5043273149,195356.00952945603,255257.59777514124,316188.9814957185,378167.14392168866,441209.3434225363,505333.11791763315,570556.2893562457,636896.9682690063,704373.5583915301,773004.7613626611,842809.5814951458,913807.3306244188,986017.63303329,1059460.4304549354,1134155.9871565318,1210124.8951013023,1287388.0791949653,1365966.8026143261,1445882.6722207735,1527157.6440594243,1609814.0289470484,1693874.4981459402,1779362.0891306382,1866300.2114448198,1954712.6526514613,2044623.584377143,2136057.568453793,2229039.5631548814],[-2285378.006938401,-2268575.7588713644,-2251461.2908199662,-2234029.308295638,-2216274.429749002,-2198191.185159958,-2179774.01460558,-2161017.2668049177,-2141915.1976405936,-2122461.968656338,-2102651.6455311207,-2082478.196528238,-2061935.490919771,-2041017.297385852,-2019717.282388312,-1998029.0085179296,-1975945.9328159606,-1953461.4050680103,-1930568.6660710808,-1907260.8458726804,-1883530.961981878,-1859371.9175513827,-1834776.4995312595,-1809737.376792332,-1784247.0982200021,-1758298.0907774556,-1731882.6575379872,-1704992.9756854693,-1677621.0944836556,-1649758.933212176,-1621398.2790700118,-1592530.7850453085,-1563147.967751252,-1533241.2052269676,-1502801.734704083,-1471820.650336823,-1440288.9008961595,-1408197.2874271765,-1375536.460869019,-1342296.9196363646,-1308469.0071632997,-1274042.9094069903,-1239008.6523118983,-1203356.0992335677,-1167074.9483213043,-1130154.7298585172,-1092584.8035618598,-1054354.355835971,-1015452.3969851625,-975867.7583803902,-935589.089581206,-894604.8554111817,-852903.3329879497,-810472.608704634,-767300.5751637593,-723374.9280621889,-678683.1630265042,-633212.5723972982,-586950.2419635747,-539883.0476437495,-491997.65211441973,-443280.50138530275,-393717.821319825,-343295.6140993796,-291999.6546329898,-239815.48690810427,-186728.42028414411,-132723.52572682826,-77785.6319827442,-21899.321692174766,34951.07255825028,92781.47224743571,151608.05699467612,211447.2686995552,272315.81574748224,334230.6772831436,397209.1075499221,461268.6403003847,526427.0932758404,592702.5727573871,660113.478189087,728678.506875793,798416.6587533229,869347.2412368469,941489.8741450766,1014864.4947028235,1089491.3626241335,1165391.0652738875,1242584.522913746,1321092.994030287,1400938.0807479834,1482141.7343278802,1564726.2607549778,1648714.3264116524,1734128.9638438202,1820993.5776173347,1909331.9502676162,1999168.2483434468,2090527.0285481364,2183433.243975241],[-2326188.348137528,-2309424.270285241,-2292348.2236590157,-2274954.9158382476,-2257238.967360823,-2239194.9103135066,-2220817.186899958,-2202100.147985791,-2183038.0516203437,-2163625.0615344048,-2143855.245614604,-2123722.5743526584,-2103220.919270058,-2082344.0513175668,-2061085.6392490503,-2039439.2479689738,-2017398.336854132,-1994956.2580477714,-1972106.2547268171,-1948841.45934126,-1925154.891825427,-1901039.4577802739,-1876487.9466273522,-1851493.0297324515,-1826047.258499664,-1800143.0624348114,-1773772.7471780598,-1746928.4925045744,-1719602.3502941255,-1691786.2424673322,-1663471.9588893657,-1634651.1552400426,-1605315.3508499824,-1575455.9265017132,-1545064.122196581,-1514131.0348850284,-1482647.6161611076,-1450604.6699200573,-1417992.8499785573,-1384802.6576564861,-1351024.4393210658,-1316648.383890843,-1281664.5203001867,-1246062.714923372,-1209832.6689575594,-1172963.9157634275,-1135445.818164628,-1097267.5657028696,-1058418.171849844,-1018886.4711745731,-978661.1164656584,-937730.5758070191,-896083.1296082875,-853706.8675865387,-810589.6857005376,-766719.2830360304,-722083.1586414715,-676668.6083126073,-630462.7213272867,-583452.3771267827,-535624.2419449165,-486964.7653833744,-437460.1769326292,-387096.48243660526,-335859.46050267154,-283734.6588528445,-230707.39061769983,-176762.73057107953,-121885.5113050756,-66060.31934319716,-9271.491193474736,48496.89066304872,107261.00184677215,167037.28021900868,227842.4300871105,289693.42647752585,352607.519474769,416602.23863146827,481695.3974475325,547905.0979207302,615249.7351694265,683748.002129972,753418.8943264093,824281.7147184219,896356.0786250364,969661.9187267539,1044219.490148155,1120049.375619072,1197172.4907200364,1275610.089209904,1355383.768438376,1436515.4748441447,1519027.5095418086,1602942.5339947306,1688283.5757807344,1775074.0344479713,1863337.6874639834,1953098.6962590124,2044381.612366599,2137211.383658774],[-2367572.2001804803,-2350846.63647362,-2333809.357037993,-2316455.0715335836,-2298778.402598319,-2280773.884438674,-2262435.96139786,-2243758.986501119,-2224737.2199776643,-2205364.827758643,-2185635.8799517127,-2165544.3492905134,-2145084.1095595444,-2124248.933993914,-2103032.4936533477,-2081428.355769941,-2059429.9820700795,-2037030.727068796,-2014223.8363372555,-1991002.4447423148,-1967359.5746580658,-1943288.1341483511,-1918780.915120966,-1893830.591451546,-1868429.7170778613,-1842570.7240635187,-1816245.920630751,-1789447.4891614201,-1762167.4841668184,-1734397.8302241634,-1706130.3198805628,-1677356.6115234145,-1648068.227216757,-1618256.5505026923,-1587912.8241685408,-1557028.1479774094,-1525593.4763629632,-1493599.6160872828,-1461037.2238614187,-1427896.8039274118,-1394168.7056027376,-1359843.1207845723,-1324910.0814145813,-1289359.4569033124,-1253180.9515134576,-1216364.1017008182,-1178898.2734140307,-1140772.6593499691,-1101976.2761660246,-1062497.9616477098,-1022326.3718312699,-981449.978079733,-939857.0641136151,-897535.7229930162,-854473.8540521865,-810659.1597851207,-766079.1426816876,-720721.1020125239,-674572.1305642053,-627619.1113208979,-579848.7140938756,-531247.3920971821,-481801.37846889533,-431496.68273621006,-380319.0872257636,-328254.1434152229,-275287.16822758457,-221403.240266277,-166587.19599050656,-110823.6258288417,-54096.87023272645,3608.9843346779235,62310.1094724834,122022.93898116006,182764.1730672503,244550.78261598805,307400.0135298688,371329.3911382342,436356.72467603907,502500.1118339617,569777.9433807218,638208.9078599783,707811.9963596398,778606.5073593166,850612.0516535644,923848.5573535315,998336.2749691433,1074095.7825697246,1151147.9910290055,1229514.1493522525,1309215.850088275,1390275.0348270894,1472713.9997863057,1556555.4014835078,1641822.2625013832,1728537.9773431104,1816726.31838087,1906411.441898616,1997617.8942321455,2090370.61800374]],"type":"heatmap","xaxis":"x","yaxis":"y","hovertemplate":"Home Price Growth: %{x}\u003cbr\u003eInvestment Growth: %{y}\u003cbr\u003eOpportunity: %{z}\u003cextra\u003e\u003c\u002fextra\u003e"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Home Price Growth"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"autorange":"reversed","title":{"text":"Investment Growth"}},"coloraxis":{"colorbar":{"title":{"text":"Opportunity"}},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]},"title":{"text":"Opportunity Surface"}},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

## Simulate
We can also simulate market conditions.

In [16]:
n = 10_000

In [17]:
simulated_investment_rate = np.random.beta(10, 5, n) * (0.1 - 0.03) + 0.03
render_plotly_html(px.histogram(simulated_investment_rate, title="Investment Rate"))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="233e3181-886a-49c8-badc-b636332e3a6a" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("233e3181-886a-49c8-badc-b636332e3a6a")) {                    Plotly.newPlot(                        "233e3181-886a-49c8-badc-b636332e3a6a",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[0.08403961380397129,0.06912098147371099,0.07407565405082284,0.06833511427561381,0.06794519537826227,0.06791091719728659,0.07580138702399468,0.07644440839315386,0.07716668925094508,0.07617228897982535,0.0617773650412136,0.0701928442731273,0.07120790265448887,0.06730843132343312,0.06360064161554899,0.08177185682609381,0.07540628414042955,0.08088250155302251,0.07931904044994173,0.08146109730733013,0.06411714278758562,0.07127904250527532,0.06942753610579139,0.08217584808348127,0.0792086826212707,0.07460593781861163,0.06632251037060188,0.08265644820114491,0.08542085358186177,0.07632244670030663,0.07899690760655367,0.08318687390797064,0.0679530140629925,0.07430135033200966,0.07922267551189406,0.08790340648788943,0.07288118533763885,0.08316390347792717,0.07381535170980663,0.08929078502901006,0.06820055804254191,0.06713904654228983,0.07737570748391828,0.07343011488222763,0.0835777367390223,0.07063524275262405,0.07285926432337145,0.08098064301695615,0.07096605746115293,0.08278721465335584,0.07735215912705873,0.06693767652025862,0.07711090656636009,0.07423602132932866,0.06721719109055899,0.0764966673644485,0.08179809130472482,0.07174186958159515,0.08470088674979462,0.059286037183110124,0.06780006369139789,0.07657506392792157,0.07169227066117846,0.05981014394974726,0.07782575115552096,0.08064499883979731,0.08186461373187126,0.07393863129534856,0.075297017951883,0.07155137629167681,0.0689653449229162,0.07268780067885239,0.09262680305214925,0.06912762494198779,0.08721820307271652,0.06867505902596671,0.05838304638189249,0.07527166378331153,0.06593289202732486,0.07107062415848625,0.07941438003836992,0.07292218549331703,0.08680684868601862,0.07375189712419178,0.07428458881292688,0.08361278045489538,0.07756246087449398,0.07999126818509959,0.08356518668538378,0.08582290608587122,0.0861581331528469,0.07450161381402798,0.08033026914872524,0.08206128006074467,0.07910138438299752,0.07779640246658077,0.06958555371896107,0.08020770054203541,0.08476746787060493,0.07449217242134731,0.07217412763091725,0.07916247093378886,0.07438809411768668,0.07085130607771983,0.06426836630332732,0.08011875507856764,0.07720212547096551,0.08116775332107733,0.07257501646665865,0.08180499290335946,0.07931937111169614,0.07799132397976644,0.08835610127587615,0.06683903936434973,0.061875592777064205,0.08127918111725621,0.07490456501180823,0.059259395727386005,0.07842323659395455,0.08112912729521923,0.0790289756838837,0.06779907219059475,0.07409205947538976,0.07139686755008681,0.07515611849492518,0.07670978067032791,0.08095534795869999,0.0690904609819045,0.0757724974288393,0.0761199231404768,0.07995617958505416,0.07076374965444021,0.07761687485122772,0.09505395828980752,0.08685646900755956,0.0613843422938049,0.08687450944454905,0.0878192605140955,0.08537033182243359,0.05924387971393881,0.07029935537498985,0.08301956340834124,0.0678491814264579,0.07581504666886547,0.06361072588135588,0.07955082184357311,0.0689810884710017,0.06461241186416455,0.08124874125406062,0.08588741316063234,0.07542561106388025,0.07274349179613473,0.08910118876946646,0.06676571366044427,0.08023539998382788,0.07062537252705531,0.07237169967465505,0.07758947185850668,0.0858807048685753,0.0628863537304814,0.05922271784284944,0.0793656405504416,0.07150662503430408,0.07488416883251867,0.07164564880916242,0.08525408841666302,0.07812406335577861,0.06667761810790752,0.07351941961867953,0.07017547177293654,0.06402304155892292,0.08022778860905588,0.06474254247380647,0.06974800462932558,0.068427695279488,0.07529924468234342,0.07360938735775213,0.07290153768037058,0.0749247315399533,0.0732059227495373,0.089277578444193,0.08115897419101856,0.07204657457628878,0.08198311650960582,0.08814536116923022,0.08094688439573952,0.0676164486928519,0.08567682419295988,0.07088896842744546,0.07729814275708896,0.08873283868918655,0.07998993798131301,0.07005532744536627,0.09113120043343409,0.07499783968443943,0.06394287577026858,0.08427081934851265,0.06727345071498711,0.08143611761925586,0.06970084125514837,0.0912388244947511,0.07325736808964767,0.06985568044017691,0.07822576937168647,0.0723909661926922,0.06182246597135413,0.0750566111089446,0.07661472100905765,0.08450039687640977,0.08480273772130392,0.08943691383590541,0.09171168123325431,0.07662399307774534,0.07185316082301951,0.07335777336560427,0.08529901018334791,0.08645957041585553,0.07186444326118628,0.0679784857996251,0.08037503509652974,0.07867312941281449,0.0902753040258791,0.09007114122544237,0.06856059096465852,0.06533875527008479,0.08721458654270188,0.07058816142749243,0.08735962331873776,0.08554789615928787,0.07339033497581054,0.08850179705949071,0.07563424934368329,0.06992942705936057,0.0721209723648834,0.07473294867390413,0.07092812771542621,0.07968528383666165,0.07724595046569874,0.06469090453713328,0.06515150512568002,0.07742670356585102,0.08078262037151214,0.07253460225365616,0.06785786782577438,0.07963973372516397,0.07879039901228749,0.08460950092636205,0.06563693672437798,0.0783996501702083,0.07518477863744907,0.07863729903376909,0.07901449413637499,0.07127230456996864,0.07216066185825423,0.07582040911948892,0.08878253512204987,0.07142683881137414,0.08693444766168867,0.0677885374610078,0.07455532066635859,0.0775219847372342,0.07450673432086147,0.07612280103347599,0.0758589239317234,0.08414217111110053,0.0737424955501034,0.06547238423349241,0.08912190875334991,0.06789980364256283,0.08702517560008455,0.0736360144115834,0.08241023302537301,0.07171675120117381,0.07209247906216137,0.08513246199827568,0.07949956534113232,0.07070650431072124,0.07462493693438191,0.07248738783475042,0.07570227043809485,0.07219780126771529,0.07005892431594674,0.07893448959281174,0.06635280858420858,0.06505098835256021,0.08956311483061327,0.07321973906837476,0.05933205447386167,0.07500802294897659,0.07345592094104575,0.07764372453829763,0.07384977324861206,0.08615822371345008,0.08648815842569768,0.07273554152717156,0.07912307361749871,0.07665466762235795,0.06869083694215597,0.08319976544265584,0.07445129636276762,0.08535976104602276,0.05944540723676736,0.0625187150599453,0.07334024625979217,0.09260799961259886,0.07926987101076402,0.0816409051766096,0.0792482655756849,0.06946312689946532,0.07464782887589436,0.07433764218536593,0.08272337431698566,0.08006860657482454,0.07892921114626963,0.07802705125206388,0.06508838641927567,0.09276083963211157,0.07914538728615873,0.07537678155773883,0.059021310982853153,0.08023193851004642,0.07822687652641916,0.08086234274026455,0.07565997267508794,0.0817078627734748,0.0804932737235894,0.08865700849375092,0.07606543044077833,0.06684478708426778,0.06718593499234177,0.06975794628218007,0.07532650771408214,0.07647088625850793,0.07632845837594324,0.08872442429329899,0.08212204855619223,0.08221465496639219,0.08136169516988695,0.07567543366780426,0.07725049082760152,0.07578109405575327,0.06582148046871908,0.08684771980414133,0.08334134959567999,0.09119990576071818,0.06684297591337052,0.06811018040523377,0.0760574954315289,0.06723006950944492,0.08962047076610957,0.07435926555728048,0.0791632182827376,0.07170818138283741,0.08630063826880127,0.08286181507453835,0.07932442520683916,0.05240323040149841,0.07805438893768443,0.08026634518611436,0.07648609259961821,0.08023197321293826,0.08240144608987132,0.07455545634121441,0.0680544080473636,0.0946465080711908,0.08032610197620385,0.06678323353672791,0.08011753192494597,0.0638285928772618,0.0829959986757858,0.07464071574252258,0.06747430789639049,0.08658055414209553,0.0707315399947927,0.06635161806636727,0.07596725792035286,0.06664260351325357,0.06835217270508517,0.0735528262511855,0.07063490425282293,0.08238821541527278,0.06536559113441104,0.07825172414394868,0.0840015599914195,0.08250177964529853,0.06102457921101507,0.07282579331125921,0.0746060734338912,0.07274980667073175,0.07167713064522804,0.06758553015542665,0.07915009517882664,0.07426498764594955,0.08542240192465606,0.07766953710876905,0.0719828398549116,0.07169081035049926,0.090334987532658,0.07558107712954423,0.08064384926951063,0.07483709023622281,0.0740847388628362,0.06901107233132553,0.0854342757941189,0.07434580570609084,0.06642218961945445,0.07211741380427217,0.07488430233953214,0.06159935223522077,0.06347105273647174,0.078309451244133,0.08203634770177842,0.07473780398043114,0.0704196882627418,0.06874371741184246,0.07528580443778174,0.08839402130476215,0.07521191784449922,0.09173652861421469,0.0760877886533746,0.08396076341186139,0.06855606084192922,0.07731829534371126,0.061392588657922416,0.08325669243042094,0.078098827775473,0.07568538901578123,0.08428119284853533,0.07425352227372342,0.08253601627319929,0.06851265527580391,0.07778879058827692,0.06341837743450005,0.07216550033354557,0.08070859136849229,0.0735365639746858,0.07328081784160562,0.0689328844652714,0.0551173657034846,0.04762841856067396,0.07739067991391488,0.08204822352552926,0.08892713559911022,0.06918274079222456,0.07825594591776247,0.07522657325165245,0.06788855103651958,0.08269471512232894,0.06207638752114135,0.0713876529204039,0.07269805183856806,0.0651591863580396,0.06363871548033116,0.07033026136818202,0.07494135641615479,0.07933880936091825,0.08068489788723332,0.07395833155228859,0.06455106510248262,0.07471155578221102,0.07933539934108075,0.06064619061796682,0.08201681550388679,0.06900235961851811,0.07494367211566237,0.08528665436339335,0.0714321027400794,0.06439537717101065,0.05898729117640805,0.07277482105546643,0.07607226380322354,0.0773563171062798,0.07379644578178776,0.06948687013889543,0.05401288508892382,0.07382006982976397,0.07265234691530836,0.08403003477470318,0.06948444504047013,0.06616195983098574,0.07191711570287523,0.0800115708141918,0.08761292708526291,0.08162319834872231,0.08051565571119951,0.06635550744725213,0.07897175203305618,0.06710426702411132,0.09235625510382406,0.07683556606261553,0.07040435452190608,0.07861952090231912,0.07961314139932935,0.07677249923263658,0.08357716377008967,0.07654086349334238,0.08101509885453877,0.08709529817996875,0.08524402093636774,0.0781311797777029,0.07573904848944976,0.0664956238713933,0.05965369027490568,0.0826076845022086,0.08698029048982023,0.0924835349900873,0.08413159924724642,0.08315721843020937,0.08349035714180628,0.07389241496633653,0.07579503036859596,0.08550922458682683,0.0699998464028917,0.06589909664228719,0.08398095590960072,0.07893921485670291,0.07716295023332023,0.0781454159594001,0.0815830431249373,0.06979099791608745,0.08213925287156534,0.06603013800597779,0.0670414823820262,0.0798734111215551,0.08009045375592919,0.06845183249300274,0.06042545548033884,0.08691047160390661,0.0716854713548834,0.08853017226911504,0.09032632478826161,0.08256823789432349,0.05946421500388509,0.06311720539392512,0.06683611136773204,0.06457312704006019,0.07384603355089615,0.0769818840247975,0.07351905649600675,0.07367064383932455,0.07873155120842498,0.08020875839498319,0.07650842144779302,0.07464362666471479,0.09116876300076625,0.08133097824681366,0.08556296730803911,0.07865031521506757,0.06037242211249067,0.0724509085195987,0.07219744662921909,0.07510962755838019,0.08621315515760845,0.0856728362896543,0.07731902999354584,0.08229972193341784,0.0713931266106107,0.08685715807475439,0.0661740898092811,0.06705372264715942,0.07314270276397289,0.08397859691487497,0.08289946943677712,0.08843812792969293,0.0767579837088893,0.07790858903309945,0.08410372162367308,0.08514229757549474,0.07071169064648969,0.08461060945440248,0.08790740500810991,0.057477928420627766,0.09425341745780803,0.09333653888317751,0.08078013856342335,0.0848306644318885,0.08870414080771558,0.0904006916198228,0.06025494327065406,0.07959801542707698,0.0768666635723167,0.09236223234518037,0.07647743260291122,0.0840986179092473,0.05910193274341838,0.0676168369225622,0.07430164961531416,0.07525900676627191,0.08714937890784724,0.07036444794662522,0.07901709648218086,0.09214893509744229,0.0751889891510129,0.07626802932268524,0.07644793749863088,0.07376323719317229,0.0757633628031518,0.05662219254816255,0.08930568737505487,0.08782899911816439,0.08719256806764618,0.08216827202130271,0.06081631306759353,0.07321360231688898,0.06907916019724991,0.08137015487849067,0.06686172916211255,0.05516793187802328,0.08491444035490756,0.06817994287364026,0.0767992123728507,0.06390565727483012,0.08615316960400546,0.09242148128544828,0.07761229318216172,0.07796814493921007,0.07748327916619666,0.07467232495015985,0.0777403429138796,0.07377717018165939,0.07827111099926312,0.08095558223942033,0.07983151094110279,0.0835562004655513,0.0807356048075154,0.07301169929244719,0.07412149547223038,0.07780345446746444,0.07441268248990905,0.07331309775776748,0.06246128143122218,0.07935361629053742,0.06657674481404546,0.07317023053390377,0.07553394698954946,0.08576229671496002,0.0759442106444882,0.07806322963211343,0.0772244780196178,0.08386836735954077,0.07956357851578125,0.07170764177912423,0.06720764706741492,0.0853009454351879,0.07471363862625371,0.08151933983173573,0.0886754753630487,0.08090528879943312,0.0790484972010324,0.08114434818664798,0.08253733066674773,0.0687724130460954,0.07685213434138459,0.08637015458477723,0.08618366463407928,0.08002268246083794,0.07974696916305427,0.07543627838874123,0.09448357884221566,0.0892249189417318,0.07772497073615647,0.093264920484931,0.08268549590736712,0.07817920176484926,0.09305819615479863,0.07669130127334492,0.07318483856178767,0.07244410920134273,0.08869990719784228,0.05805855052268114,0.09378632090732433,0.0752525424527124,0.08391094800031673,0.07402902066196457,0.06833661374688602,0.05763047096751635,0.08118815940868764,0.08234371611381636,0.06415318232782125,0.06220115678299932,0.07046254828446366,0.07327811386943142,0.0716218453491663,0.09100720824378938,0.07337703131871424,0.08111091424468304,0.08140909097648152,0.06614730597864774,0.0782836935054658,0.07347032932717325,0.08078825289151718,0.07001359711717599,0.07980840565711989,0.08338525732801036,0.07835132083540312,0.08392514642047733,0.0774754071519357,0.08985518113671241,0.0636258843238614,0.08003977782805899,0.07053515632292899,0.07486912616273997,0.0678027883890562,0.08420020078299817,0.0826815149833375,0.07597229553755803,0.07798056826597335,0.07583912682247311,0.08038124263711724,0.07429038733474436,0.07515117065062843,0.062292759209944495,0.06390254394623127,0.08144878258029697,0.08057619297887622,0.07168605436584032,0.08039380394885767,0.06974169055352543,0.06251055465002986,0.08085229875898747,0.06967701124201867,0.06912719123377022,0.06551496212539132,0.08088914093987527,0.08134036729841387,0.07298209947521252,0.07150582781935068,0.07528401441810653,0.08343653005514859,0.054469447205153725,0.08067820412004682,0.07671853795377151,0.08107956313506609,0.07685991172973122,0.07904737868184529,0.07479593501869819,0.08494403521565902,0.07483036316031796,0.05790896160074374,0.08106445814809651,0.07572686160749087,0.08822606755943486,0.08829956541461684,0.07445537620832446,0.05991258791097312,0.07888634455503771,0.0851789561272854,0.06393252210976719,0.07260681627199062,0.07199423556308948,0.08438882528111255,0.0791961340247353,0.061966907224521334,0.07197669389924917,0.07798240986309204,0.07739165783811508,0.08354554208564145,0.06715986985988069,0.08286783887198285,0.07767804868618944,0.06631479975796342,0.08740359848603763,0.07656815828154441,0.07679607599710953,0.06462445910822524,0.07788340631474348,0.0807226776512878,0.07943622585982558,0.07538104477520802,0.07865484183929503,0.07517568120588142,0.07795685628747567,0.06285055318380983,0.0879679678858552,0.08273733095528102,0.07397299822973313,0.08071752343625295,0.08041891775459122,0.09250817533222341,0.08699979427426373,0.07329737250439847,0.08497992296405037,0.07174056037480159,0.08384320669078427,0.08006816067786217,0.07275727633370904,0.0793049126860028,0.08406059926607795,0.0850903146571575,0.0756167706009107,0.06249385420430028,0.05851238596475404,0.0781730650988037,0.08342756683418241,0.07490319627621522,0.06951572230371889,0.0697523440354854,0.07538058592042471,0.0867687899128288,0.07703742070685773,0.06289144847817564,0.07942155180843691,0.07954482974586283,0.08459034664602808,0.058733090170968213,0.07449890851559078,0.08823958106786964,0.08556545972824359,0.0871144033653879,0.07746053880968036,0.06300916544938522,0.08469848936934504,0.07350047059938183,0.06200839098046335,0.0774325468742085,0.06561280972204794,0.07169148261200683,0.07594586628180477,0.07750869913638986,0.06996235288014943,0.08780807934045068,0.08750124728029217,0.07020181908242963,0.07276203178468826,0.065852825722699,0.08986289050749671,0.0783879059430254,0.08050609343323054,0.07264748438330476,0.05534222702297784,0.06378543296127899,0.07508376521840374,0.07300119783637296,0.06833101311368747,0.06959523057234747,0.08751849956610668,0.07537895714365203,0.0899791958174864,0.08534566614585579,0.06549704156273406,0.05519223282887525,0.08496008336137277,0.07424230537218951,0.08281763476135345,0.07536927946972308,0.07542130126610885,0.08374268188487652,0.07123804700771547,0.08237594029591255,0.0877480427956147,0.07956437157597335,0.06054785437679307,0.08418153386454252,0.08725835609670432,0.07318847704503784,0.08506653498453005,0.08695003265745394,0.08421992627585098,0.07986862150028282,0.08768331705254762,0.07541442353311437,0.06950548697338409,0.07860421009882756,0.08165715827065567,0.06533297857042347,0.07344022275376881,0.06094328640821055,0.08223133120143196,0.08388356457306294,0.0702455149117302,0.07132850596876432,0.07578318042149264,0.0688879111896093,0.05750570308146133,0.07973533556898856,0.08545820660590855,0.07095120955293155,0.08598169682166176,0.07776697354979062,0.07553201825616598,0.08454311161419957,0.07323245916482746,0.06066522166240115,0.06512319325026647,0.07445912769098409,0.08156161264628456,0.0625078355840436,0.06965364538078746,0.07774802057917951,0.07847040139387645,0.08745670795438662,0.0845264915551949,0.07563774088886585,0.08363014144081124,0.08006587316061611,0.071881742207519,0.07059893183535876,0.06714621074788019,0.09091395713437375,0.06516546948907882,0.08879386650782686,0.0896318628435909,0.07262944931734644,0.0834049892740249,0.06283091938183957,0.08440751789191994,0.07954322709218216,0.0827942152799251,0.0705728859675458,0.07828472839678322,0.08599858444461686,0.07327079689889318,0.0664351304167013,0.08332498605591108,0.08012667249646514,0.08705928149492258,0.08166427576800933,0.07823180052847303,0.07627277392668003,0.08737832008187688,0.07085961590547213,0.0796707913299804,0.07662326400987349,0.070806059022852,0.08689491473894556,0.08096763778420372,0.07031588388803658,0.08402643550840322,0.08262624209947521,0.07085305252523047,0.07977023650347877,0.08233107459070012,0.08611528497549681,0.07354101190145809,0.06958780992438954,0.07212325774122108,0.07589172222283072,0.07892769879851735,0.0840873681079182,0.07398204966286857,0.08244951106927592,0.07420565746987143,0.07418002325280793,0.07278314972829247,0.0841657927587172,0.078140101883718,0.07447702992289579,0.07040104066847444,0.08168499862768983,0.09076098599877414,0.07369225601275592,0.07760629050669518,0.08395858263596881,0.0825776868180536,0.08155819820457666,0.07203708051658719,0.07861394953946363,0.0729119552208082,0.08027185403605817,0.06690668524936069,0.07095694428340081,0.07623657263865832,0.07993445572619738,0.07425365689497246,0.06792173977340439,0.07361935453849894,0.08592754996036231,0.09220588435363242,0.07365608814479924,0.0758317954851821,0.07119233971700138,0.061004333404900836,0.08225557821158468,0.08613204536461419,0.07587064832944632,0.07440377754322271,0.07249557869426076,0.08258304598493857,0.08072289426866144,0.0671214161855351,0.0906687414427862,0.08023745027761818,0.06835603545136458,0.07571755650010398,0.08558849361460694,0.08006357292583144,0.07926172062132092,0.07830336629198087,0.08830710216679405,0.07794341045906353,0.07376554001698415,0.08474122451381623,0.09505629345396314,0.08821398179553282,0.09197679392608274,0.077004672287573,0.08574391836222053,0.07017958573259739,0.060938773144106034,0.07851850030056955,0.08210870777771392,0.07148382441248265,0.08320742319622085,0.08894985116789478,0.06410412317928504,0.08879782109572787,0.07275642775597055,0.09092737844131066,0.07525862475759387,0.07691285404517498,0.08184686508875269,0.08558519461241226,0.07199555096764797,0.0850744641646836,0.05968441945315746,0.07025315184172455,0.08250569194999574,0.08200240781076291,0.07452287428884222,0.07672532677260538,0.0782107579528517,0.07187425261303834,0.08205464289156805,0.06809905656353506,0.06181312853382659,0.0811396739341944,0.0780389037489718,0.07506571775893392,0.08168447977949443,0.09180998459758272,0.0680250426414091,0.0687613082343867,0.07703365428034809,0.08357015709022811,0.07569956195671995,0.07827355745433376,0.07818262167001741,0.07291865004210933,0.07615189861021426,0.0664363602062276,0.08770967519340903,0.07432555779938556,0.08025771020269518,0.08318697138876538,0.06637727411333065,0.08089929955561692,0.07352567582778928,0.07315426521650367,0.05372856384813251,0.07233909585460593,0.07932787578458536,0.07277809315260465,0.08316870431459497,0.07011665524216448,0.07433274642247896,0.07932415242994159,0.08172673234536101,0.0882603791061046,0.07823930790102104,0.06544178029164899,0.07618592808683042,0.05023824964784393,0.07426737962477076,0.07697382832395452,0.07334604081979018,0.06803022614944926,0.07273730193089932,0.0745697018090763,0.05948278340139761,0.07345713743677891,0.06465595921742301,0.05898184620342602,0.08842179150294706,0.07985526151817617,0.09153458626862104,0.08343946979441147,0.06601664591140113,0.07796741776836812,0.07579268476386913,0.06793120805159461,0.08068245481999647,0.06848696463408585,0.08117392645098132,0.08564275995734605,0.0879250925734278,0.07522521882170072,0.06951326576437572,0.08390721269065596,0.08141566384182541,0.07051519192398453,0.08847275270511284,0.08556056871393425,0.07860305593049366,0.08333562869389591,0.08208370401063436,0.08422580384120171,0.05895830562947779,0.0667150521468013,0.07113624647363985,0.08465129233766969,0.07525862199280728,0.06864054989094913,0.07083118705204108,0.06953841383059212,0.08124496002803333,0.07827443170688325,0.0722670746464049,0.09742439947383412,0.0740530802383556,0.07914647472057038,0.0792548300384866,0.08064372011453151,0.07425844047640394,0.07810955851206323,0.07562522023503035,0.08139919428246446,0.08481250988479327,0.08412204240089222,0.06523256842615344,0.07105173612438262,0.0796123166775377,0.07082871405385768,0.06866847748170088,0.06974669499090555,0.06892533635791666,0.08465427558030106,0.07220356917897511,0.07319453425879247,0.06897653376920301,0.07467872816743731,0.07461982562589042,0.06697364786234192,0.06883152003693774,0.07753211268071306,0.07011763206156907,0.0833506256893107,0.07849392959153859,0.08643866421943133,0.07926247965299033,0.07110516211398828,0.07929867659833705,0.05809980516824806,0.07706117317879382,0.08532989338971064,0.07129089838239028,0.07051352559565385,0.08203929957427304,0.06714644876405527,0.0891128738529329,0.07086448126842368,0.08779798243814096,0.07436458410620395,0.06429936611748585,0.07553137540078518,0.08042748372758421,0.06655450722505377,0.07323245154506483,0.08809402258739346,0.08515860713825654,0.05678265585337215,0.07687851876669537,0.08527434270330778,0.06947735529315303,0.06884210017506398,0.08617319488179107,0.07376289756615519,0.08986557732591427,0.07099416864872535,0.07223215703531746,0.06782268507201931,0.07519223405604393,0.08913272503466241,0.08147108637429518,0.07383367242763161,0.06628084180402645,0.08239770449482123,0.07020825817065593,0.08131986420207625,0.07583015100194199,0.08186455736673554,0.06820297518600885,0.07287503153957209,0.07260405266373945,0.0840285511866953,0.0650873188187241,0.07490663979146721,0.06963828950991721,0.06914790527223019,0.08488453280563688,0.07650638391926394,0.05922744908137971,0.07246145742994274,0.08307692323829069,0.07235534920540365,0.07141511978529579,0.08148257472337722,0.084536079807022,0.07839633015155342,0.0783121550568676,0.07060727568634553,0.07025286370738967,0.06786651558314294,0.08478576221874684,0.08522969657359902,0.0732890556608353,0.0788155677092062,0.06651538898968798,0.07685678225789998,0.08215339030640165,0.08979997394246088,0.08503778309254922,0.06479904156024921,0.06964997474709218,0.08171793029525337,0.08961251067401134,0.06863896301731146,0.08390815403006677,0.0830630148505016,0.07824483502150836,0.08435226845452423,0.07257129635275378,0.0740659683454889,0.07723768707734985,0.07909035615065987,0.07773996240786267,0.06493160921706066,0.08754783914844597,0.08236674549809783,0.0923236454523779,0.06179646467831232,0.06523062373897473,0.08411416989994838,0.0845282919279513,0.07835062039151497,0.08612402395963814,0.0838872656569233,0.06880305844385476,0.07819603204070341,0.08587172746661587,0.08609022593513263,0.08295760535792336,0.07283911312074862,0.08922448076764124,0.07842997889606312,0.08682092747244685,0.07011439496834354,0.07856740845311731,0.06921863698622928,0.08891055427013908,0.08330405912320621,0.0698168211186151,0.07410070981063516,0.07019425048243462,0.06598198864981529,0.08604602023600225,0.08050938737860001,0.07648065731326456,0.06386161215337972,0.0834066058284693,0.07521235622937333,0.059109240270046104,0.08560813140030848,0.07564096794937665,0.07348664307282565,0.06789331709651134,0.06446737811136088,0.07127881709083023,0.0674150747986824,0.0652653307269655,0.08223858743587037,0.06382172810728637,0.07126682760955086,0.08025122813414653,0.07229310832735067,0.07691464629121172,0.08960797469541294,0.08510632720453015,0.07973733983548276,0.06650381920371379,0.08048701731474057,0.08344819546777478,0.06750255365908092,0.08064096126532028,0.07923269880281411,0.0605385346799317,0.0715054611354844,0.06120927183566825,0.0827720377502406,0.08725249718963908,0.06841410241220654,0.0728846059195753,0.0786619974284956,0.06281805203255975,0.0807137005179086,0.08021701056951783,0.07728128842836313,0.07933751723974276,0.09186228478227065,0.08462129244605952,0.07821070671211927,0.08458760767204201,0.0750743733397743,0.08327192757772582,0.07838570508645304,0.0756811934798034,0.0678436140421638,0.06641790632167405,0.07150507001650669,0.07129117171289734,0.09360505907007462,0.07510078417326457,0.0706093873010421,0.09300998715594823,0.06797667821382172,0.059566329481276326,0.07975864034505017,0.05633472398882254,0.0709771221831693,0.07769773232271665,0.07016190343611092,0.07861825173680465,0.08272113732378933,0.07573432472908023,0.07806966177786553,0.07943944702843905,0.08875508802272239,0.07861824193372413,0.08179450533984231,0.07035216602658212,0.06523276712247228,0.0684393019396372,0.08118140189236078,0.06988273752049562,0.0792161184942351,0.0650174508534654,0.06696680988291992,0.0810899799341116,0.06424535138118892,0.07368100265738631,0.08808989784996679,0.0737549064475564,0.06678063704391161,0.08189072754101565,0.07113292842248423,0.06415818185434971,0.07144962160560828,0.06810522786329842,0.06976885196700623,0.07270483723755612,0.07976857829431144,0.08909070431158428,0.05884821775043375,0.08215640351616608,0.07927732391144376,0.06262671468158287,0.08224552941503606,0.07862381258163609,0.08755644699056826,0.091759377775686,0.07677084185752456,0.06417494213715394,0.08594989773924522,0.07515555906675911,0.08343237185285099,0.07745870802909299,0.060000171566785776,0.08383254044383665,0.06503013807427793,0.08219876546863336,0.08125959810405411,0.07238507649735362,0.06845828499017445,0.08586471140890584,0.08705280156373044,0.07243725429151604,0.08196557079472375,0.07035082698616091,0.08456798134281923,0.07575532227159357,0.074053371960862,0.08085103299067256,0.06334108382679052,0.08116584266899297,0.07666685746924647,0.07800542278042098,0.09005358183137549,0.0877012319103333,0.0665628397981898,0.0708604034428254,0.0831088762517314,0.08085422373453832,0.08169582200834852,0.0745676879940323,0.07109020697397883,0.07111048398445513,0.08951530133036056,0.08432322944480006,0.06613414734627945,0.06236762327514533,0.07129693949038274,0.07860109746200117,0.05760777336899919,0.08655078567804875,0.08608661680988097,0.09242921007201525,0.07423695584088331,0.07918849765687538,0.07818412944511564,0.07568451078845684,0.08398953586233096,0.08259251738370173,0.07699498714411604,0.06360924456455418,0.08696077631506804,0.08391402702137593,0.08540236587079408,0.07275281896475745,0.07499283831387413,0.0909142409098948,0.07280203975560379,0.08042736340930533,0.07979290709030698,0.07636997334906878,0.07889478706405036,0.08424565083905587,0.07563286956319826,0.060320570996780926,0.07399888474284241,0.06760312380149958,0.06472450668514687,0.08788651156469657,0.0666863101800727,0.07430685457149395,0.0799276030087574,0.06950832453027739,0.0800367312749947,0.07739100741062002,0.09188020253798493,0.08229732925174912,0.08073065659813501,0.0808034269353257,0.08057915336333751,0.07256376894452307,0.08468726549073582,0.0773025770228788,0.06739719854277135,0.09005307416455935,0.06877002888476053,0.08353123453296679,0.08276270174406591,0.08643787027506443,0.0843308596998006,0.06160892830928861,0.0777303121899131,0.06434465989285028,0.07321712499419142,0.05938979064334121,0.07251313452362784,0.07736155372867151,0.08308313834665673,0.08049036247661312,0.05922815854131066,0.08527956678703366,0.08566740247671784,0.0831777773106436,0.07689895800260044,0.06543546574875336,0.056381339855864666,0.07901085037180744,0.07684376531344117,0.07256769995146833,0.08703787947642552,0.05938153217923397,0.06612968232560708,0.08768897967078987,0.07318605820138208,0.08897605983452002,0.07812330622983948,0.07114966441455839,0.08108696071491558,0.057010890109040625,0.08358573042952598,0.0820907996876015,0.06207136500762378,0.08094078097080482,0.06641750354060859,0.08485849966033351,0.07450783407532859,0.06819181968709864,0.0885147124194171,0.07543418935935844,0.059453670462413656,0.08708769882241475,0.08455876083579633,0.0861989179988244,0.0938112022187696,0.06955267620388055,0.07421044634554963,0.08493506649982475,0.08365372109584457,0.057692497776304016,0.08188406782575755,0.07477352630633616,0.0763339704226397,0.09294491690193259,0.07708100411158361,0.0842767858945728,0.08307394787542016,0.08406353146090884,0.07867959008452288,0.07251465255808962,0.08129907603643606,0.09016200512782127,0.08076138052972598,0.06878680626296196,0.07216656694835324,0.07057774314036111,0.07382922635719663,0.07193816877775171,0.07833410764091507,0.07698806220138996,0.07070794130741377,0.0845795448588639,0.0857073382546909,0.06886221926874861,0.0662711977351533,0.06979651929085197,0.07859611350599364,0.09355816557204441,0.0866668688634612,0.0661842399699209,0.06660155131393836,0.06609686741744786,0.08155460203720936,0.08745612239997673,0.06696259359591836,0.07939124733640764,0.05842947430827961,0.07516708868707744,0.0806844442743958,0.07803769778180457,0.08184943218490949,0.0790526811364497,0.07261493131534132,0.08168966800706501,0.08352859305887678,0.08791411002904961,0.04688326104556724,0.0664126256156698,0.07723122332243512,0.07689485179910668,0.07817196164897469,0.08690086463659552,0.07618044519297451,0.08359535109241278,0.07231583654481846,0.08448401674065205,0.0805376738619206,0.0712406593653552,0.07472260585704744,0.06623455540835095,0.06971340092216294,0.08321998275493567,0.08729375593618646,0.060001330139024533,0.0736079467924481,0.08662556895292114,0.08672755458604905,0.07914999712247309,0.06395481923436061,0.07737404012546145,0.07786786690024045,0.08945143236617084,0.08021365676724863,0.06646679216320447,0.08212050172040382,0.06271808123413722,0.0856311239954165,0.07586484184975803,0.08289251476239538,0.06776831234679748,0.07760469860690569,0.07915324419760694,0.08563525158663055,0.07517631781131744,0.08345640292380231,0.07816631616979292,0.08638570145191765,0.0758267901720528,0.06792696246512785,0.07432075254326878,0.06712263567964966,0.08055765572647028,0.08805865975103275,0.06991018435110302,0.06156219488887682,0.08580129472943276,0.09449951498484792,0.08542837362204729,0.08837292071024319,0.07607806286869404,0.07778207615546834,0.08491057095138786,0.08503519487024447,0.07458781467593903,0.07641138244159663,0.07588085407329978,0.08049786355784203,0.06896734079167226,0.0828646886221576,0.06905879843162177,0.08293648219465552,0.07481777210980718,0.07641302613450607,0.07488477910307482,0.07769146247842457,0.07497821773088734,0.06607057346480363,0.07532551492450287,0.08439407263852025,0.07304715739261548,0.06944401562083649,0.07493520698274381,0.062414229527045874,0.07724895197648077,0.07469458739117288,0.08194643924462383,0.06185339096288721,0.0763458583679173,0.0760363129351172,0.05434868276798459,0.06731490007307162,0.07007000115218166,0.08003614711987872,0.06753342190993182,0.05836453980352209,0.06460737577113967,0.07101569890789511,0.06894045580731323,0.08008773590976322,0.08124577478132523,0.08599887607233017,0.07698272267229314,0.0652141793338687,0.07323428986054666,0.07187089246489783,0.056341925419460884,0.09029108498381586,0.07625675454519369,0.08101505958483302,0.07001805196311539,0.09290822160714535,0.08085692142033277,0.07712256110276755,0.0801135695228764,0.05824579411702543,0.08417795689522753,0.08558801378331318,0.07785302874994876,0.08074207433262862,0.062167375134201695,0.07124707575888939,0.059512694824255134,0.0782410066204622,0.06509413437786288,0.07348631877716266,0.08267771306485544,0.07393516222808083,0.07936748914274896,0.086342773869035,0.08591123213933656,0.07351625704430725,0.0726903203453812,0.07470168159242918,0.07135172524628022,0.07772412177418425,0.07515485309815875,0.0604806108872544,0.07248014108925346,0.0810105456700592,0.084087933644696,0.0900591286234455,0.0736113315977838,0.08102815413905402,0.08429868836395576,0.07332844519479004,0.0699400695910075,0.0741785104503578,0.07756014588842158,0.07191091059648028,0.0633155308326874,0.08078981263112195,0.07449803325740484,0.07592053436981558,0.08006034079428473,0.08145980908050286,0.0809617794464437,0.08633553046332729,0.07928092975739512,0.06795960982432771,0.06442789529409303,0.07126936624677428,0.07872535373731432,0.0796197314886797,0.0848990309140531,0.07255951260075369,0.0684440811737059,0.07082227475438815,0.06981833406688895,0.0803047501454636,0.08651123110474043,0.08265117760298854,0.0813185989577361,0.07213212188791239,0.060528042062217985,0.07160821281664574,0.08069600912170595,0.07371997004788342,0.07916977824852955,0.08098342950717707,0.07667869646220105,0.07734817657918588,0.07212720970969441,0.05907012886009144,0.08810921270959804,0.07968799604101692,0.05564226281325656,0.07313639158648491,0.07590235636017484,0.0683718235929027,0.07691956769020203,0.0874954424052787,0.07589071454632472,0.06333213417755827,0.07266505896264575,0.06285358826684273,0.06927395488555713,0.06056990591444792,0.07443362589772523,0.0780076886718125,0.06752434794716396,0.0771613130845567,0.0790486273356371,0.08097279755944271,0.08586029286593888,0.08475609801585873,0.062005848617143026,0.08233127010728816,0.0619092742627629,0.09070055018793943,0.08101905352125541,0.08243968115597297,0.06909969359993726,0.066074936977442,0.07150047058215286,0.06769195119343573,0.09099287009171414,0.07381909327340083,0.08554322085476423,0.08937264716455039,0.08423406485307591,0.072559379925455,0.08453797032674812,0.07413653456636349,0.07950329440682663,0.0803186346528453,0.08794649876156659,0.07943560926203092,0.06893729336606355,0.08087881692690613,0.06678439094519903,0.08154105405663771,0.08875557605519364,0.08184745817739772,0.06831657989737033,0.08179597565520683,0.07339552852015882,0.08975706873429298,0.08752924764317338,0.076120992181471,0.08790502253138206,0.07950888220304536,0.07581197861075273,0.061665857857649574,0.07484045783256144,0.07874347880856974,0.07754463604700149,0.08555993609443602,0.07208044640278925,0.07070589121782236,0.08083513643863408,0.0778760523036916,0.08473668618108943,0.07795758102636896,0.07794835901985708,0.06939057457973666,0.07371923789922874,0.07685911952208518,0.07635373405560261,0.0739032968072436,0.0685145260302849,0.08755013276008591,0.08320218162367475,0.08402180263382808,0.07208543244684074,0.067865176289403,0.073082576900103,0.07745772410428431,0.07222860114930597,0.06981798274026525,0.07991910054534937,0.08370618183469564,0.08510462630174306,0.07856897584370089,0.08071809449764919,0.08856180073924114,0.08588414636972588,0.07002069552233145,0.06124758224761048,0.07653844070737341,0.09235834079824357,0.06535358667883134,0.06452728109527107,0.06762786654025418,0.0840731535700217,0.0815805969451012,0.07462562520935337,0.05707858881498927,0.0722702022618113,0.07396156890849147,0.07894459361263531,0.05826667668814835,0.08446915029197144,0.08040222980253188,0.06615551772760792,0.06775255496757734,0.07731384946926825,0.07358937373721308,0.08103576238846583,0.09029187735069905,0.07430344035084413,0.06814437293323009,0.08739757349427005,0.07808027580262139,0.07908056181657261,0.07298514124468557,0.067531679151967,0.07519826135357162,0.08313431176345601,0.05716526741790736,0.06335563686051407,0.07242013730627964,0.08206321026094882,0.07369556421394445,0.0759698057931516,0.08253014255327434,0.07227270890876086,0.057352351459918696,0.07217403121689694,0.06974298218258142,0.07354691808176812,0.09534840926796688,0.07158133311871252,0.083054818668574,0.05393246838211117,0.08252449641304227,0.07003071120361633,0.06112128890685033,0.059944464338233826,0.08384235763864019,0.07956992938923954,0.07785470648433873,0.07122205332810914,0.0783149365608837,0.08560187911721268,0.07965383037620223,0.07720996734850762,0.07786791215838465,0.062269377931940866,0.08901948431634857,0.0611917804966043,0.07015274375868458,0.07739850256455394,0.07247859674507678,0.07864303670918626,0.06228846356208354,0.06958672237653296,0.08296602756356553,0.0727328629080067,0.09114188770712973,0.08367787223130244,0.07635762307155278,0.08135847193272859,0.06960615691571234,0.07436443153516192,0.08195495225730452,0.080039581958925,0.07205687980411014,0.08130285854465628,0.07683615051437649,0.06973262184625262,0.0734318925546198,0.07474860699408487,0.07509542886135243,0.07768022736961933,0.07293467317700061,0.06866063664693633,0.07619330222235765,0.05834600446144156,0.08451838576244552,0.0835371225644304,0.06705900514536892,0.06994960221378504,0.08003414361561448,0.06072868964429946,0.0878308740528528,0.07856211176723613,0.07158315985145552,0.07750930969937872,0.07146127183614785,0.0731010767526594,0.07452776505967304,0.08918015343389604,0.07011282351319767,0.07585961134360623,0.07301042749278627,0.07235023507483693,0.08472625638367587,0.06947206803815108,0.09143599243382575,0.07288595466529517,0.08719306867313692,0.07266532325575295,0.07909692564114704,0.07041993049006905,0.08161227290499487,0.06367423977432883,0.07299533132797059,0.08490614714924853,0.0760674343230611,0.07278339123517664,0.06497528178015935,0.08783654157223397,0.08461315744476494,0.07721456632668179,0.07371090221079854,0.07378839665476461,0.07286645740919195,0.07049654302566291,0.07169478299904011,0.08918793891769662,0.08504032300554118,0.0838596593223221,0.07067164589114389,0.06347567541196592,0.06943093419902162,0.07900878233889694,0.05863139858352416,0.07106860380077525,0.0510076652894714,0.07309187360525102,0.07318144816544848,0.07611056701937875,0.07134714249336013,0.07900179143515133,0.08138581263192454,0.08213576605827994,0.07767064764756398,0.07193743063956279,0.07686129432664984,0.08331368657584018,0.08803696334710379,0.07695425741272194,0.07169946919599639,0.08439264594232075,0.075506773733032,0.06715082526141217,0.08290047644792148,0.07246205421581065,0.0702064683936873,0.07077538580632069,0.07150330714738373,0.08235623213087329,0.07001871933922726,0.07836564013169522,0.06959138725356098,0.0813990825705167,0.0814633655819639,0.07116603113358781,0.08135461646506623,0.05132015812263604,0.07948140894689068,0.07930460707722481,0.06388299498498881,0.0716966911301027,0.0735877474428995,0.07258519711196927,0.09043040781741962,0.06833252248756738,0.06182651211176901,0.07796190659730629,0.07067716653445272,0.07553072265053544,0.06524535984227917,0.08438921991368314,0.0629254659743502,0.07022985514908034,0.06763420979077694,0.08135783480663202,0.06814123485271052,0.06927079249639084,0.08536616666730634,0.0804060425378953,0.08380855325087219,0.0808653972109418,0.08191031987423572,0.07795515375356601,0.07916183814255309,0.07780736599582808,0.07875773818254386,0.08220703786625105,0.07151715738179217,0.08056576819339065,0.06820834704911505,0.08933919010913617,0.06700149178824522,0.07565975688466484,0.08350441340986796,0.08472845337213045,0.06916579640651196,0.07493170636826087,0.06442749166226217,0.08015221788362294,0.06955440116179545,0.06456991811006094,0.09265279854634209,0.07848991914962514,0.07024051468577999,0.08004824019582757,0.08801548453414557,0.08641080394322263,0.07795314589446245,0.08299186705301012,0.08877457295261018,0.06445299384924053,0.0956000401601969,0.07543492071342306,0.08557008363126953,0.08969571141067545,0.0746720549606679,0.07044700289573283,0.06712470480936542,0.08438785651327012,0.08090240152379646,0.08122645076723906,0.09181072868202106,0.08032454773782091,0.06488673485677604,0.06229990682110192,0.08843554936851096,0.07095619508871452,0.06706514392091559,0.07467121334703275,0.06728192651520415,0.08007421649467286,0.09084041025152839,0.07453968942645706,0.0801430570559373,0.073129225293242,0.08066531897765655,0.07016049594515603,0.07959509839127094,0.07340364409049266,0.07765670521681428,0.06856992190682713,0.07268268377135687,0.06741173129915931,0.08162114940971052,0.07791297010367243,0.053980024489622636,0.08201626159372521,0.0710607303883988,0.07483157418970765,0.06219178039176836,0.06000252796785674,0.09121151556425303,0.07471784342882974,0.08026716986132806,0.07824004642524013,0.08062844506353958,0.05732569824437646,0.0772238103944824,0.0808980596507983,0.07464198776880238,0.07899459267441244,0.07343618559581212,0.0665330307334917,0.06720994043698056,0.07242274304396372,0.08106637151609715,0.0803671621008556,0.06355254011864411,0.06453985344734597,0.07679724326653406,0.07568344769356294,0.08675625734841333,0.08202485044832433,0.07934343544926886,0.0682355604016964,0.0874181791139787,0.0875094186932617,0.0786310750447003,0.08734022488306034,0.0824330987453506,0.08584911789010644,0.08268922619014116,0.0736346176583405,0.08202275237067787,0.07701072107467284,0.09315312864233642,0.0714293301224237,0.07403248694444797,0.08510594291815705,0.07384800226465342,0.07668972225148474,0.07052006711605785,0.07972395211835923,0.08276861379159814,0.07719606745123339,0.07400057981323822,0.08377550608783578,0.07343795851518434,0.07369125728241593,0.08341586951574922,0.08011156622770389,0.07541658395477537,0.08348320369993484,0.07385754619388696,0.08363640026246524,0.06828140322069692,0.07885866578282001,0.06666213609008523,0.08502677216230872,0.07679489387749108,0.06314943577741258,0.06857035101858805,0.05584983569894092,0.0777473755775929,0.09008391247862135,0.06728791254420481,0.08162000675920955,0.0778639262006014,0.07187530788202821,0.07570552620225873,0.08442496960729498,0.07184353560421836,0.08913212146878145,0.06646090739091501,0.07739441592108741,0.07826103651458623,0.0744412861354917,0.06956598158971525,0.0842645100368726,0.07541316777255591,0.0763575307106475,0.08885320664023864,0.06806092933039849,0.0646089379634022,0.06845871694209911,0.07302392227104748,0.07615519447260234,0.08459232741578318,0.08072950709779692,0.07694148085923723,0.07878132716142693,0.07746085717871828,0.06815648003293373,0.06648091720312385,0.07216161072274883,0.08038518489820996,0.07322793224688154,0.07756210636800523,0.07622411706138812,0.07614448772308763,0.07489272799584512,0.08178832495234362,0.08735230549845467,0.07822224748595624,0.07505177705102607,0.060645618104078806,0.06838859361158675,0.061731359904387294,0.07620935906110532,0.07982389392226849,0.08624594271849323,0.07445670339462526,0.07788872019618923,0.09071216333194207,0.08492591920392922,0.07423486815849184,0.08522509568190462,0.08358091090779116,0.07997809108112675,0.08151245287030048,0.08475785464446298,0.07408285625842323,0.07451099884998116,0.07781242464835043,0.07063407934661783,0.07491544893509358,0.0727930566127325,0.0843387830952058,0.08417032822671637,0.08299029457524548,0.07709159202343989,0.07589514229759786,0.07369452044148098,0.08316475277066879,0.08063113945002119,0.08238958551834949,0.08549272723519268,0.07161437483523157,0.07365076077560254,0.07693968858386876,0.08269275334753737,0.07332707145131268,0.07195156975257498,0.08076223809618889,0.07701313680835964,0.08452284899356985,0.08088502246032131,0.06326169307750226,0.08379266675709057,0.08527745155437508,0.0688505136639995,0.07028156951237645,0.08577798171992004,0.08057481761245708,0.0717865184560667,0.08865958821373578,0.07133240330299537,0.08033934904495364,0.08727681154621446,0.07312944900238516,0.08044682048732273,0.07257458202914596,0.06334908191742486,0.0842034215851442,0.0766985500554266,0.08721364067468029,0.0733846191866501,0.08279012186841242,0.07857970890429294,0.07920666156991517,0.08041322770287043,0.07877757181765344,0.07336213880914831,0.07542810635439631,0.06585662016890612,0.08654395565253695,0.07961666725635921,0.07238075833506549,0.06302706351943246,0.08077802501544427,0.07902259770731968,0.07332656699904336,0.07849450222228713,0.07962878507267807,0.08577677525101582,0.08274530868671183,0.09100693762668466,0.0836411538997559,0.0771690167788421,0.0734238686504418,0.06766096554242709,0.07669890334073651,0.06936441118263145,0.061490889033151724,0.09451420953939835,0.07096982888946014,0.06977798005748069,0.06938254383533812,0.07989008090658271,0.07561180109984453,0.07677334063631591,0.08877266392265934,0.07576233551915477,0.0760722332162427,0.07698731015169977,0.08116272240077291,0.06808892570701458,0.09238340236091801,0.06865012733520995,0.08176812811164211,0.0769399711597745,0.07282706674272899,0.07750963549038362,0.08518475122916108,0.07997696277873075,0.08710776828013093,0.07577773881891077,0.08437174872919706,0.06978132978331546,0.0667832569886317,0.08998704679781741,0.06894559090544836,0.07471371471868539,0.08560880001635762,0.07477735467809193,0.07827303708979075,0.09334208224422122,0.08611401221917833,0.08033140379322182,0.07255481499243308,0.07013007782368337,0.0774694954580924,0.08568181508195284,0.07992680034818503,0.0824631233759326,0.08669950359247965,0.08711887353038866,0.07350925868548697,0.07291661353017728,0.07538392952876624,0.07891069208116754,0.06451807832327908,0.07736381711836013,0.08300813137957685,0.08308805044717664,0.08709685559600061,0.08005030560224982,0.08342237342862178,0.07808896650387892,0.07091541584647348,0.07935943438985113,0.06940514406180635,0.08000953816273426,0.09207922052187235,0.08182305264240126,0.07809584375509238,0.07507478194407072,0.06768411497945043,0.07598497392180918,0.08501610143083646,0.0889204025171392,0.07477374452734617,0.05972138440240653,0.07308815970848391,0.077138825453625,0.07276845267301948,0.07104051310218121,0.0817020362439696,0.07658323191019191,0.08211078637831831,0.08879362358722122,0.09238646365288122,0.07942309605047665,0.08368360942621145,0.08025700653814236,0.08284853330426356,0.07945074940547235,0.0628899622918826,0.07596718605189615,0.07484327635107388,0.07178269083538791,0.07488113951223693,0.07302501559872182,0.08682043367131072,0.07267478218576037,0.0805922083032532,0.07859692904624324,0.08454287795764244,0.08173058434713706,0.07417897140111057,0.06682735851714977,0.08251519404132046,0.08375405167183195,0.08414045159694618,0.06463129931565018,0.07500986010569244,0.07632662044886429,0.08742376012706392,0.08687578204748235,0.07316914546691795,0.07537278647983428,0.08594816894230099,0.0800973638328275,0.07439995477743695,0.07047240053196241,0.08540983935248575,0.08214160322567711,0.0943029761822048,0.08426475157643981,0.07733603125876466,0.08329558354662778,0.0736823783090077,0.06111518869779992,0.08255435687565768,0.07837793067893531,0.07702062435018314,0.07896518779835696,0.08431916693419939,0.08002625829802099,0.05500179824968644,0.06856267365549618,0.06627854008025098,0.07413104996530342,0.07166804545017509,0.07527282271459963,0.08348428121970236,0.06815123472426576,0.083882189529978,0.07801047925460594,0.06704621297231206,0.08096489218036498,0.08467133071002891,0.0704028087003291,0.07617811166317312,0.05437103870116003,0.08121567681436354,0.07658926535158167,0.07754952868678723,0.07179659029038848,0.0793312624223726,0.07478743244107523,0.08215542129670872,0.0775900293742703,0.09090481774917247,0.0789558420873185,0.07530231841741572,0.07828666096029907,0.06428824066562536,0.07317651495179728,0.08186940915483304,0.08195931189284547,0.07112707044644585,0.08308923098624701,0.07192724272590445,0.08847034000214116,0.06749469560251048,0.08000531639451156,0.07378500450149109,0.07515221790874582,0.07827899234439542,0.0799898841385989,0.08575369699265195,0.07957527431671202,0.0763866394355364,0.08261735580526963,0.06889968741465065,0.07846704300437823,0.05914129367978829,0.08498745895878769,0.08238162804441207,0.07767245969368936,0.07564048678570809,0.07935061957483891,0.07896072375911715,0.0662639714966865,0.08685552496078677,0.08739115837039434,0.07427174155001397,0.08458911595102866,0.08658458853365816,0.08595458926790044,0.08890376320774498,0.07953655365124185,0.08385865804783546,0.08993582790688355,0.06709241566374684,0.08007685673412163,0.06873386449797947,0.06848722971468335,0.08117636352157856,0.0868899182616185,0.07256261744631103,0.08242242918005405,0.07400814930046148,0.07998259728149396,0.08357566801183697,0.056168705443292735,0.06848805421883726,0.08905699713674753,0.06850463033354268,0.06713528860548834,0.08249610300733946,0.08743731936751734,0.07475659206339974,0.0822681797345757,0.061183890194805565,0.07096245012821714,0.07076922602880852,0.07453889002385444,0.076796354534392,0.06633663748635468,0.09345293494310046,0.08658787119352415,0.06116215416772155,0.08087932267499143,0.08405839108752271,0.07576622486927989,0.08133496661980288,0.088910942329095,0.08912629667454627,0.06793808575647903,0.08326907567282338,0.0724466035745212,0.07741592771898259,0.07678693270018391,0.0747602987322454,0.06685481380451828,0.08488130451619366,0.0716737606933987,0.08943557831232976,0.07085991035593125,0.08209426564468479,0.07711984573170852,0.08661821388068314,0.08136399888464936,0.08049420574079982,0.06984063358465953,0.06723200177879105,0.07293434573999666,0.0776018649033452,0.08671211281105723,0.0737027047498082,0.08584042721012072,0.07460830422130282,0.06934789068868233,0.06192241716849457,0.06638047535601421,0.09322799652613234,0.0891379620862463,0.08284240315072,0.085443812350675,0.0832205298569319,0.07830318343016188,0.09103847378035862,0.06665572134420214,0.08719421319892035,0.07617515342829162,0.06833606010060672,0.08660563685849598,0.0627036342986331,0.08280493748127365,0.07689396632388275,0.0782027772254027,0.07756039410446841,0.06152439506443174,0.06082860779294569,0.07904913530504737,0.052694458684461626,0.06472238202582471,0.06914750525268067,0.07978668495938818,0.07624758545763646,0.0801261871753638,0.08237029436383816,0.06859635539958771,0.08144321094784512,0.08510253429692982,0.07241508041407399,0.06327458149084822,0.05884406450456564,0.0778595076616436,0.08115048783604316,0.09057145968089257,0.07606572291824412,0.07310208051891501,0.07383891888810508,0.07597604807094437,0.07619759104023255,0.07831762003511364,0.07337434855983371,0.07144396724697236,0.07950716656535764,0.07801426539112102,0.08739372519699966,0.08295168353394625,0.08459552255793326,0.07799132014214444,0.06690155488017083,0.0891480743711521,0.09082158305304458,0.07367915166253207,0.07306330509315329,0.08679481852203268,0.0801139666507648,0.07033278579617691,0.08502874202700207,0.05416260346801173,0.08060621759566028,0.0728094872602885,0.07175021045187237,0.0772741699270466,0.07534730949890289,0.07703330008064814,0.067776262602844,0.06630969307930734,0.07326363821960208,0.0833178693579193,0.07775731541760303,0.08627191083407285,0.08750282152774506,0.08502916292644769,0.08818244221403532,0.07858694913839809,0.07668623587779239,0.07045584231593366,0.08042955663760498,0.07108891751956783,0.06969853909497038,0.06527400093977818,0.06478305122611074,0.07171618550835188,0.08533520348168479,0.08415274009993333,0.08717831604656398,0.08381087586812118,0.06903189687498992,0.07277607145055744,0.08320304739939516,0.07501378779052109,0.05634145384570841,0.08830900513107884,0.06359663284824234,0.07529924222278431,0.07313725382058955,0.07075458700672922,0.07890784863909367,0.07449401598471848,0.0790874298453012,0.07566578222349019,0.07592294891587378,0.07154526832963867,0.08161523595330228,0.08469018201380006,0.07987446070727394,0.06898910353587015,0.06508085110520123,0.06675492212245122,0.07143845858560319,0.09730511829657232,0.06965486923592315,0.058187186640708606,0.08237073832255304,0.07149667739436852,0.07759433063687725,0.07342028149767776,0.06882388325665839,0.07683624161293647,0.059763589924203546,0.08145078450027335,0.07990323119541778,0.09054803318067065,0.08124840089622697,0.06985757721021482,0.06729940814448031,0.08166392337903872,0.07714062214678866,0.08277927786736708,0.07506339029234332,0.07387896172510341,0.07645558080059181,0.09290273524721436,0.06011674952999185,0.08503285130388116,0.08350882838679763,0.080999176254271,0.07756840732778716,0.07839626588512542,0.08548983925009093,0.08923047862534403,0.07119513949967174,0.06620945990725705,0.0659678505119537,0.08006094732582537,0.06400764578598832,0.08142489601807001,0.08479872646067999,0.07479297722608663,0.07869060125624719,0.08317071746966348,0.08093314290750267,0.06506465002656175,0.08521073956582856,0.09005825465467132,0.08018321965995259,0.0803520693361224,0.08613676689948874,0.06965002723025959,0.07316156562038989,0.0902961124098749,0.07010147893722043,0.07881448859088769,0.08045018906309451,0.07445466262313927,0.06086446627300618,0.08479888865294952,0.08907186917165721,0.06812337656656503,0.07358225260516893,0.07226024941833163,0.07494681442950854,0.0730730364606631,0.07908849945624008,0.08844993138168013,0.05374706271812854,0.07270182859632555,0.090141769281183,0.0753889976847897,0.0730154436734956,0.08023028192058948,0.06401082960754362,0.06764926721203593,0.08156854572745519,0.07697916681564117,0.07164026245966218,0.06289461573030253,0.07303489687543258,0.06361826691326153,0.07442195605584256,0.0682607271080646,0.07627441356147699,0.08711740091394246,0.07631450274644308,0.08633623602725896,0.07480648276666824,0.07043086013323266,0.0715568992021316,0.08713290508659105,0.06168563643600288,0.0679137055837872,0.08434673346141189,0.08053667895422353,0.07192723608238816,0.07143021402174274,0.08154850272410541,0.07186249404540285,0.06922008860728043,0.07867397909714702,0.09035544356900457,0.07444460106610862,0.07983099495463244,0.07786397124594222,0.08663288134498243,0.06280286279185648,0.07859587600901227,0.07966549320815285,0.08217462848094421,0.065732210533473,0.08987337410049297,0.0748851123015804,0.07689496887634353,0.06305887689417132,0.07900478022368895,0.0678694827123686,0.07537568379601708,0.07674323036722663,0.08364013813700805,0.08545929222752024,0.08221535424659432,0.07241130043462238,0.0718105163191016,0.07277849464039571,0.06885972732848522,0.07809223191272091,0.07352005207398166,0.06376700561203053,0.07883961530200567,0.0750589943848046,0.0778917189888188,0.07390575953236983,0.08258706568216659,0.08537201607541531,0.07132532939020797,0.08094444777811305,0.059802060992686444,0.08418366622408752,0.061845233801051024,0.07346548536691355,0.0842734138112263,0.0751747681562438,0.09070116739340404,0.0839506851804432,0.07142236675638246,0.0806237678661213,0.07555797235421358,0.08412863756989598,0.08475253953122547,0.08780574500749432,0.08506774288954355,0.08071618482033452,0.08002903416679154,0.0683213824141358,0.09101874675198846,0.07171303717466707,0.0912460252943082,0.0890748709182421,0.07793189702168389,0.07506799056034089,0.08059931378160892,0.07286004893105838,0.08122875131034507,0.07751402940015459,0.07913087447731929,0.08626185188885657,0.09582349359483648,0.06916833024579841,0.07324508905472776,0.08058133330440129,0.07523265506507043,0.06284153771526334,0.07933289146616787,0.0852833852670773,0.07988412737241551,0.08659325321398714,0.08075317960572567,0.07635766733615118,0.06556994518047088,0.07528717766119764,0.07640973870667794,0.0736568026089368,0.08581479786455873,0.07022494707829849,0.0727246692209044,0.06375423723068285,0.062488431538025926,0.0924655210945235,0.08297264403340271,0.08462297706881988,0.06668429953217675,0.06963701329498252,0.08596069778593779,0.07496497785993311,0.07388020810614701,0.07483327313689986,0.08181556126540016,0.0709485871101197,0.0681139698711575,0.06601604929458521,0.08607102878643955,0.07436833277725755,0.08421747262335494,0.07297834342305268,0.08070136613172639,0.07326279491999226,0.06953805701729789,0.07617743323017451,0.07396460247440831,0.08119020590508816,0.07811609769125406,0.0791776644838022,0.08116647803082279,0.0825705582368936,0.07869131651085778,0.06286776087182655,0.07169136415695768,0.061804824170749464,0.085898823344811,0.06972227296393582,0.07389167766643853,0.08353187611295312,0.062707742241213,0.07325871392510648,0.08443846632500462,0.0659360252230276,0.0806226014572303,0.08253745372690811,0.0711496177065718,0.07553753593348284,0.08519154568869705,0.07035041541145659,0.06730613894348028,0.08029999190338981,0.08121618901043257,0.07350789183838982,0.060573006646605226,0.07993229634310894,0.07346055303252436,0.056261388825319295,0.06903885783174923,0.043015275359065866,0.08210791637757853,0.08459633199583994,0.07648953006879902,0.06223741904191964,0.08032536196072773,0.07832836227949104,0.07967346934493788,0.06342750487832964,0.0728321934551911,0.0735147167091173,0.08429849989055463,0.07277824278662293,0.07257427677218571,0.06950819538122328,0.07072062849347684,0.07146283630600678,0.07794770156280648,0.06445454305068896,0.07262460576931579,0.08491259797180806,0.08032609251384634,0.07598578430304465,0.07445782639516185,0.08014074593046655,0.07470271703461423,0.06974299135279086,0.07613697452816567,0.07791133699587648,0.08378478488777225,0.08504619520489481,0.07722961218821216,0.07860049736544496,0.07565390736791006,0.06910630264716491,0.08205042856441389,0.07488335444556354,0.07108722598460948,0.06159287344813072,0.08085576471936247,0.08617301154566806,0.08175599565950725,0.06745698806125056,0.07245731278095466,0.07891110544589727,0.06810338453970874,0.08849790485906532,0.07331711953296277,0.08211842482641607,0.0853007386312917,0.0639130828822841,0.0806506885096162,0.08181655437499265,0.0770454768427149,0.08647908637718026,0.09319607860839776,0.07987152195983484,0.06389200606311904,0.0686080176202575,0.08919174076047545,0.07716035396269275,0.07716082051476651,0.08903833996438673,0.07872955319771435,0.0747799833074359,0.05524121840403254,0.07750286099730208,0.07828693175633544,0.07461039617162671,0.08640293694466324,0.08206202727836413,0.08615161879485979,0.07650674512074063,0.08387174458310134,0.07354680346562147,0.07030391606130001,0.07841865338131823,0.09088087058802644,0.0821988784826354,0.08244711724278267,0.08052071929574467,0.08339422501802639,0.08315803921576331,0.0852928903591948,0.07515328049292158,0.0647638289983016,0.08541956665778512,0.08108318077800131,0.06764805581444473,0.06292456689015716,0.09194724261619855,0.06445224795284739,0.07515923153972995,0.08133350656124282,0.06005507021696681,0.09262258253686527,0.087730742979512,0.07794435863538043,0.073065383855042,0.08307984695964012,0.0789546607548397,0.08024379104603718,0.08027829600246575,0.07109255839819592,0.07754565307021861,0.06601836866617883,0.08248351559159396,0.07405682600219105,0.07499474503120324,0.0808182041296576,0.09121801429348558,0.06856284343527311,0.07484370174615285,0.0733631218780257,0.08372025312939171,0.06446898615509257,0.0824381267309337,0.08039185446969938,0.08622201173587188,0.08317611007264009,0.08694131365262958,0.07091789020877298,0.0905023499523965,0.08498837180384294,0.07372411252987912,0.07153574987262269,0.07331175249309058,0.07276480389522497,0.0861731193275194,0.08194551533288406,0.06249062655876017,0.08966820217477055,0.0891983586555016,0.07383428011076346,0.07339216835705949,0.08196677626819772,0.0722610715626909,0.07816542038901647,0.08278632568220395,0.07843693213123865,0.09093292394968265,0.08833767934912803,0.08593219811127634,0.07225096923069368,0.07208673573144994,0.0808097326989008,0.07644484487269307,0.0656424289777419,0.08440381196507021,0.07250407177385282,0.06163371894983172,0.07728162151264441,0.08625228440376047,0.06260661715696478,0.07998439423550141,0.08822546826787941,0.09367281931665794,0.06619897475223094,0.08514829259029144,0.0755582808037068,0.0684655268686734,0.07090004092743607,0.08466076651208801,0.07666248254816856,0.07182881346687066,0.06371830209728935,0.06968375760677138,0.06877221379256214,0.0773425243384061,0.09154750533666857,0.08632661975157022,0.08701990405458279,0.07621987254181428,0.0958887862117426,0.07346834325103954,0.09061849423124266,0.0756031812834512,0.09126235475757596,0.07938713043943757,0.08345830417514757,0.07862524391061171,0.0771810266358838,0.06607002110980599,0.06264482315634463,0.07050562965526595,0.08545252868489076,0.07002639348548162,0.06744976415861781,0.07121544527313312,0.08894060677626475,0.07995061511216328,0.06468673533781119,0.07835322530541027,0.06678971729941244,0.08449361465288802,0.06557846911373595,0.07756311113787484,0.08041917891347045,0.06781422833411485,0.06842479417620748,0.05345066236948333,0.06920592730297916,0.08020949286907471,0.07462652076199178,0.08111101760706686,0.07365996992412474,0.06185440534111706,0.07311325068581626,0.07816798387609515,0.0744263582178431,0.09483220355739137,0.07532584918585361,0.07685332480576434,0.06978387703425125,0.08113735340802287,0.07569713341414616,0.07366302207135167,0.08885573694366514,0.08482231640749993,0.06965567386297535,0.0707228892148078,0.06312074975521456,0.07439023965756897,0.08503811047295673,0.08336678040261691,0.07687076793376205,0.06369614353587824,0.07890283042168919,0.08721432063537285,0.07017739035989941,0.0778173693189696,0.0755786188883974,0.081725875962463,0.057412759421345175,0.07203689235818277,0.08338217551178193,0.07926755721295606,0.08196723522782742,0.08319550126811785,0.08808000923001476,0.07624173506675003,0.08747344853285849,0.08690173854526276,0.07342069414382597,0.07015670771161382,0.07333295318305516,0.07082659976693481,0.06484090795852168,0.05055179945446549,0.07210825504180834,0.07870791142262011,0.06896082731922819,0.0587220408206501,0.06897226046886026,0.08802169531526151,0.08853194778575404,0.07148716368604,0.08374082383424855,0.0665325485603235,0.0831783234887294,0.07793241920825464,0.07476602656083867,0.08474660633254646,0.0885323728225341,0.06768135490219268,0.07321656381811428,0.06244556132998816,0.07045698998239797,0.06908370184698677,0.0663879681237658,0.07725399099774105,0.08563910448667053,0.07313153820112517,0.0761209707128679,0.06591613995638075,0.07621956734056953,0.08699892650781621,0.07585303161865767,0.07551897928712348,0.07655556091142507,0.08920238508954673,0.09142500080869731,0.07064592850888728,0.07247424770378003,0.08525668121844499,0.07523171340827733,0.08911230508200832,0.08816305982403315,0.0760159779769572,0.07100507414620941,0.0871135704852595,0.07354579218385478,0.07971378962129155,0.07113138263353797,0.06770726092935525,0.052723356131316707,0.07153156332437946,0.09094617619274327,0.07294713636149315,0.07276267559242307,0.08021546501971896,0.07384555900721623,0.08259498296908932,0.06335739061032285,0.07423694649366797,0.08285734990887263,0.0768101231198573,0.06887964069662672,0.07082188120578709,0.07139302066583778,0.07068998508301735,0.07791025277539279,0.08730072564470842,0.08833965095457735,0.08089979176604033,0.06867783414542913,0.07022702585113437,0.0727448052930193,0.07768437794535638,0.06939099375369231,0.07918254466636696,0.06873789811071954,0.07316688447651884,0.0826695109640308,0.0755222209252497,0.0798847066151584,0.08838239910753652,0.0785384928903558,0.07027764259922041,0.0747296508157532,0.08426568247468907,0.08466472323447746,0.07734452071029438,0.06816349730199402,0.08394623431916037,0.07090061558001595,0.08271110620769194,0.06892574665663434,0.0882872990943076,0.08261893154754668,0.08142098518174247,0.0797110658561091,0.08025526606993022,0.08398210704243905,0.08088056694870865,0.07191489023582798,0.0708204801393516,0.07371030881688154,0.08398255622463277,0.06923169647708585,0.08282119761558937,0.08970653734912398,0.06282122154373357,0.0811257898902572,0.09225751577430655,0.09334263763236442,0.08356795843421334,0.07523024833441833,0.07988720377283345,0.07186524261118866,0.07536776487054943,0.06061179648576271,0.08228214860968587,0.07112254199263912,0.0827741594106107,0.07755330061237589,0.08520982311202234,0.08456616902904121,0.08752231574060826,0.07406702543917384,0.0830986258462281,0.07045927215748521,0.07784066053509425,0.08215962781865271,0.08352550950781956,0.07057589034262834,0.08767829551272788,0.08361091615459808,0.08002844483750617,0.07693802073618825,0.08311653869470399,0.07139415293777719,0.07588095733747136,0.07733699026106798,0.07994232324017786,0.076481970682547,0.061751648316835196,0.08458376681938387,0.08275431368881177,0.07546689006420704,0.060037670798204545,0.07221685263152963,0.07275350230748447,0.07772764615650547,0.06328566681510789,0.06578442783904204,0.07997833512995836,0.06975106844908231,0.06393714259926325,0.08632032609756252,0.06903328666386072,0.08017520544262925,0.06200061965738361,0.07672750957678325,0.07822714566909714,0.08173358636443294,0.0815934521407771,0.09226059136645295,0.055769498898646366,0.07908362575228198,0.07055041212008686,0.07289160096733001,0.07292510492239934,0.07738329131190602,0.06096680418696376,0.07328485932536144,0.07629503960970632,0.08829486952790394,0.06587633320639813,0.0771440298074938,0.0865415594916828,0.0804847015190993,0.08335331510291832,0.07843078038609089,0.07451122266990168,0.08340317167923778,0.07766263773253643,0.08887105239219967,0.07617909628860503,0.0636747665246491,0.07350442601890689,0.0797033964675126,0.07226295329709816,0.08219708058644656,0.08180685820382452,0.08876089757150826,0.08431608120125045,0.08369583549376497,0.06435018580687615,0.08845071790982872,0.07640230475547748,0.07078200155595044,0.0828965023769803,0.06941884255271843,0.07667913751389965,0.090973604277204,0.07625979613802777,0.09238593677658746,0.06847463136190637,0.07826791572156135,0.08551225322484371,0.06771587231792757,0.08677032858545439,0.07223921435037174,0.07553140855194646,0.0832869226625862,0.08082903079604953,0.078148914080683,0.07714833469802024,0.08135663208306047,0.07476038108323443,0.07879626136087585,0.07299635575395808,0.06607498492923637,0.08252202977370848,0.09149086692755005,0.06784950447278973,0.07199818322639043,0.06492738672995689,0.0870780414524703,0.06993005942955827,0.08092346043189041,0.08109754917637525,0.08723862766773001,0.07130286792752591,0.08366349588347499,0.07700141955374207,0.06340603679683265,0.0671188410387693,0.07552946787459766,0.08100143166328881,0.07646649037472825,0.07213821879886241,0.06005212433232185,0.0717429904432988,0.07894526295879764,0.0887059722231461,0.06483115094955627,0.07585955016282978,0.08064868068898398,0.07953235258158578,0.07985591837132974,0.08901963691790615,0.08058044718133876,0.0750368378928605,0.07854481691584334,0.08220066871156237,0.06924596162742791,0.06873356769133696,0.08067413696788733,0.07547938982365732,0.07996900764079234,0.07965500548152019,0.08783431596992708,0.06351329568517748,0.08367128605244553,0.08700353497682523,0.07301064647827403,0.0714240220277845,0.06357653812835079,0.06654750985417579,0.07713302343327272,0.07006690381100086,0.07611713645395496,0.06877480433267402,0.0709376385876562,0.07552017341137615,0.09109755391591877,0.0836546933202252,0.06247871112847612,0.08200342741216651,0.08560360075692663,0.06560376626671481,0.07784825015950089,0.09318296941019863,0.0748918155306729,0.07224556100709417,0.0844444852395311,0.08459263571017835,0.08663098703643667,0.08002096138154681,0.06783825827625098,0.08479188446844049,0.0797843774840811,0.07983651680785792,0.06923461343570736,0.06388204350633575,0.07766074809192253,0.06453260585625423,0.06827105377171014,0.07202050393132417,0.08066604244623929,0.0779227341605995,0.0838378522378051,0.09136032206791331,0.07517571606400365,0.08200704814236803,0.07051108917329467,0.06848967736630746,0.07728607677372937,0.08090135954228432,0.07852334848060512,0.06427459813432024,0.07204184480428391,0.08245088295184366,0.06562715279028436,0.08081940177865307,0.07338570416462935,0.07857011198418523,0.09289616073696094,0.0822041654517513,0.07317287390781421,0.08685990187965356,0.0661573193663305,0.0711831490790506,0.0832604713838184,0.08180313878847831,0.08484697121945645,0.08401691444871487,0.07485017543503067,0.0790986916620019,0.08513115676017163,0.0797293553504815,0.07737833242026498,0.0855055903492945,0.08886665448541248,0.07460843104999035,0.0720382916273418,0.0745282572499702,0.08426734624922214,0.08599737414924888,0.06915610105111282,0.08077491235720949,0.08508332925648149,0.07087502650104574,0.07473072529164537,0.0839612550745885,0.07511995294467819,0.07162082619634874,0.06988055350762487,0.07147061189586124,0.08277899457283258,0.07222147294434364,0.0871968926602594,0.08487059057516498,0.07317592423652064,0.07257902174787781,0.07314483110523058,0.07375397765570077,0.07127720474393764,0.07540878758909429,0.06071756704190855,0.07894324475895018,0.0811186976699526,0.07647818800066898,0.07849765903163991,0.07595512451331055,0.06756965628366311,0.07423360492654335,0.08326998267864462,0.08062950826209096,0.07929610866816827,0.08015106108935782,0.07470723573828626,0.0686337764178282,0.07013323062762028,0.07667014153486855,0.07269683045056627,0.06291800750041196,0.08385887753006657,0.07382028453915207,0.07900375762964879,0.06849994630111733,0.07879867825866657,0.06698660996650632,0.07697739904956875,0.06900530536552182,0.08093874150972771,0.06585614554733471,0.07749966877996564,0.07661373001339607,0.06907998695677847,0.08045005099478794,0.07289095366591954,0.07265735464528086,0.08521695201484951,0.07624513499742827,0.0808812254917306,0.08567298455007633,0.07567653069352383,0.08424065418033344,0.07776038187750847,0.07311522584986552,0.068082209993728,0.07316169373934608,0.07299333417631546,0.06811579876380147,0.08373630642632271,0.07422795238783343,0.07585442291756242,0.08525869116021956,0.08281318033740011,0.07231663744131839,0.07683958670969447,0.07174693572346402,0.0826314916765937,0.08813190883555008,0.07931350594377157,0.06144936077314171,0.08354964864279471,0.07607060580036384,0.07077049061865426,0.07040594560422156,0.06971369033916129,0.0876326079373376,0.0842639674942583,0.06961333877954991,0.07033194440434928,0.06316108238114539,0.06856352623515721,0.060845778049179886,0.07439992999306105,0.09209834866175286,0.08045774242122022,0.08731354988813886,0.06787961140201873,0.0759682238668882,0.08598028889896848,0.06658763015255173,0.06844155987344999,0.06466349125685314,0.08024797923440094,0.08268872005490518,0.07977646836679102,0.07184055198092917,0.08534901373483078,0.08284032817297085,0.08354544648845139,0.07592800007043365,0.09073121565815856,0.08866168142018858,0.07947027211556618,0.08230366524196289,0.07187601030759565,0.08113271445155154,0.08186498119278103,0.07476263358361593,0.08241589059338772,0.08877186185993198,0.07818047528004804,0.08453883510354984,0.06722501685475854,0.07818380173916797,0.08200519805619282,0.06352770780748207,0.07121415453127117,0.08524972548093189,0.07942035015925071,0.0679876860817023,0.07662401943025154,0.0833380461442455,0.08629374927079164,0.0836845269046998,0.0823827882846319,0.0925816956670875,0.06067276654297536,0.0765731821919714,0.07220451024939002,0.0828060387413598,0.07938727738749338,0.07626010183230125,0.08755350397838219,0.06339696870695286,0.08351446416323212,0.08043605970948633,0.08754857184221324,0.06747935056605334,0.08179584511739027,0.08479414924760917,0.08486765377359504,0.06976344273283638,0.06144675528041312,0.08439169551081396,0.07507598660887231,0.07641275583017564,0.07324836742618443,0.08016562424089571,0.09383110554288145,0.0762403963733788,0.0816152776813033,0.08637489955274463,0.0801844702527456,0.06155576212990866,0.08116355887853346,0.08374696284955307,0.09049870774802966,0.07169407857313667,0.05948114958598116,0.06536861144268843,0.07787768850918193,0.06736616640647622,0.08291051778804048,0.07587460591784839,0.07585598375242072,0.06606024429325781,0.06822224534542287,0.08194915338760686,0.08984922099027334,0.08124963190820972,0.07931215449552437,0.05368665855226735,0.08956400307652239,0.08431165654497477,0.08294807510769918,0.07959749812952979,0.06935063915724077,0.07497557398035226,0.08482508301068553,0.08336785717220085,0.061628085170194614,0.08436274420133019,0.0697336437231341,0.07304687488676857,0.0761286037955588,0.08687935938229054,0.06820706217903837,0.07811252308934537,0.06915162906303052,0.07226831248100077,0.08338651995880197,0.06715563512943332,0.07499292440505997,0.08621976044089238,0.08455990391815502,0.08573010194339253,0.07154421669450022,0.08770046535342217,0.07282824164768467,0.08118305297838668,0.07336121345935867,0.08155261720795218,0.08898925945643439,0.06803957612678681,0.07135931449958821,0.06806284236417812,0.08562693775909669,0.08148646846007725,0.08082682983998954,0.08031857710003981,0.06145022902480324,0.08093154087422058,0.074640767372255,0.08365714220481474,0.09234714305082292,0.09334647538780963,0.07655623311169102,0.07228978925996077,0.07847650635600728,0.08520575151939011,0.08176365093102245,0.08281751875075949,0.08690477633608873,0.08230427333357816,0.07048106648830138,0.06797126954807564,0.07669308297962191,0.07455345274609865,0.07657584822078856,0.07912427637485116,0.08259084614119763,0.0663655422698674,0.07898889411165375,0.07784378324895955,0.07083596766485853,0.06476731385388923,0.0836500518625805,0.06470637136694646,0.06335116572051518,0.08397717123045058,0.07992761185306507,0.07896329989050117,0.06744615781372632,0.0772110158257252,0.07979298442864477,0.08738255727041505,0.08876005102310017,0.06693899756238197,0.0727116817568319,0.084298733586352,0.08539232345680614,0.08100649251488971,0.07668661919649175,0.07168218749558436,0.07409390240825153,0.0841741315257743,0.08332553291212086,0.08225439764213616,0.07647576673048179,0.07360367930690916,0.07771837122299627,0.07920924304674759,0.0782412575286951,0.08201162960438835,0.06941958760791311,0.07019058800369213,0.06830663123795971,0.07258816457433545,0.0588564521962761,0.08134175223234086,0.07679696473918429,0.08325064336762167,0.07554361536828938,0.07414621686292569,0.07266127220348095,0.08344691954665873,0.08225765173391493,0.08039985244018952,0.08479958921949966,0.08006851548759553,0.08931008290486195,0.08617462204412629,0.081741817739385,0.08287006286755533,0.08701824399329239,0.0738601354265759,0.08305137713779834,0.0621272103794837,0.07159673383456043,0.09593890322146198,0.08030313778694761,0.07267343796982169,0.054045791064950596,0.07557811235026377,0.07005772983408007,0.0944363846972238,0.06075296083615024,0.08214304471104125,0.08108060923717275,0.08098643642907341,0.07224306530909913,0.07977188255043127,0.07214942713780655,0.07690323242508705,0.07567297658479777,0.07891354628183067,0.06684833988059852,0.08063630775720484,0.0790857204095061,0.06717997004811896,0.07442900511460898,0.05921035047957679,0.08377225863364195,0.0806727147193496,0.05913921786161683,0.07164102903029293,0.07754649467351879,0.050372700601136425,0.07369437807805712,0.0664355931809314,0.06679423731380948,0.07733076351287371,0.07503783385845035,0.08717095340312356,0.06786020189992403,0.07825733894114241,0.0804679640198529,0.08070891245099257,0.07680334304160258,0.0836999482126658,0.0469503057997725,0.08997563795975057,0.07699647218998437,0.06506907100384587,0.06618703153939934,0.08517699490303636,0.0879302599804287,0.07478665957988029,0.06669063797094948,0.07976350620104786,0.08235913573090531,0.07526990628270466,0.07210363042830775,0.0892980634258374,0.09227734341373611,0.07341894795873655,0.08789091231393895,0.07978931893556934,0.05787764071921263,0.07824588822515921,0.07683459592130873,0.06553609584604356,0.07362308028315706,0.0827019628629432,0.0736769391899845,0.0764660344474098,0.07939103135156322,0.08173411801123573,0.0798872087023417,0.08098324410431706,0.0847319947401588,0.08994174183487402,0.06932793413581456,0.0806797422522352,0.08391098601956243,0.07198982848949874,0.08205833696575665,0.07716745573945222,0.08696502820353245,0.09323552384133661,0.08776523762492887,0.08409038908641361,0.07997711911819019,0.07856723494875972,0.07436194143873595,0.09294352247438804,0.054568613181108394,0.07225788363958324,0.08330338309528836,0.07184428114616884,0.08833892718426027,0.07566612317307686,0.08272976139414015,0.08271641410146559,0.08871035433310934,0.07826232162934299,0.0662537125463255,0.05864999159730315,0.0889247960449134,0.08308294979343206,0.07547934796026212,0.07800059450641336,0.08408142799685837,0.07463322296604555,0.08102870512643226,0.07930362832704825,0.07929863613985097,0.07310543891232602,0.07507721526104741,0.06665722942903636,0.07518161375667562,0.059348931089944476,0.08740495346792662,0.06594856598456011,0.07771906035611917,0.0745962062130518,0.08565161654315291,0.0842610846971977,0.08311631828863207,0.07796762806894872,0.07394258350591815,0.08316567996709036,0.08093131540103249,0.07950526445392639,0.08157365441665221,0.0718594670342769,0.09479946179160002,0.07905978594958027,0.07183399580899913,0.07973025844902619,0.08304393501204062,0.0733192695863103,0.07981216859597773,0.0801439825605775,0.07430958731902536,0.07571097237000805,0.06092833240396288,0.08880567612969036,0.07988974761803545,0.07462108678044838,0.08145303114520355,0.08839096256193993,0.07889228557108245,0.0801592563502775,0.05627486170237286,0.0487571888469708,0.08363694305884417,0.0823090254895519,0.07827661008347006,0.08337215474905621,0.06805460025928106,0.09076126932753575,0.07482356689864908,0.07521369200265221,0.08606816505697502,0.08157424024573151,0.06936409118525091,0.07850949751151748,0.07854546218774919,0.07842304317782003,0.08299630610399175,0.07642504504531795,0.08409358431182208,0.08173756147617448,0.07152594088777607,0.08555010770745847,0.0835383803052786,0.08648653081287613,0.06698876906501783,0.07224193283292434,0.07387656936493883,0.08327130520460771,0.08353882920989139,0.06254233432484815,0.060672972975637146,0.08068971089437302,0.07962755239406422,0.07787576900537882,0.08557029943530448,0.075951421629402,0.08258006788663366,0.08807481992281804,0.07524446170603852,0.07338422878972845,0.08229723297025257,0.061683189918686716,0.08034082882157359,0.09187279881573618,0.0813884951340326,0.07117038359660362,0.08185010885509454,0.0867802674249197,0.06611412587347743,0.08987427477221582,0.08912960399909756,0.08368154672543926,0.066913430041902,0.06555202870824584,0.07265293366336714,0.06055375813357665,0.07412489518514279,0.09081215350748609,0.08872054974119709,0.07490838743316577,0.07663479943131396,0.08429777877626601,0.07552203931780972,0.07646712135335287,0.09024825648308654,0.09008441354404825,0.07383609648767406,0.0797159389299733,0.0770585949511296,0.08115337268764795,0.09034017618628093,0.06808895707571383,0.07179820872524839,0.08747722117419558,0.08067165290450598,0.055177151346042,0.06205142279793485,0.07314563841080052,0.07698570712945865,0.06667974878407373,0.07011849343898063,0.07986657498665176,0.0893812256551916,0.08064227160631571,0.07816045003262448,0.08558093288464858,0.07906903085231476,0.0720049012437662,0.07651165052039352,0.06819406462936892,0.08780914934142395,0.05633361121336598,0.08064297321583275,0.08197534551203546,0.08243432705490572,0.08345175487511866,0.07277570558290675,0.0726604100762012,0.07235208953571531,0.06683634400062859,0.07456217853804967,0.06868069171710718,0.07751718065796928,0.07223471127381867,0.08057816100677037,0.06202026105406376,0.07966121264468981,0.07013195092703323,0.08651259793256605,0.08073800278405409,0.08616436565537113,0.08192496678698719,0.0736044199486144,0.0862851069751802,0.06933506711161551,0.07460999314157624,0.077108972871759,0.0836701822719971,0.0807522225325994,0.07410632651464089,0.08482260663720428,0.07909601497116134,0.07017296163891734,0.07732348972652048,0.08044191890026625,0.08197946100602452,0.06785348720054421,0.07325579503783824,0.07325906414672909,0.07282877554351919,0.0766145041181481,0.0660023730363723,0.08341101351880806,0.07343432561097271,0.0871775381939622,0.07337990241866243,0.08390707309559306,0.08003749875044466,0.07803541636492112,0.07518446056982167,0.07806180419153802,0.07873100341100772,0.072293326118694,0.07875297755538754,0.06524834323915871,0.06557995063021896,0.09626073806534562,0.06282011741447774,0.07129213538818449,0.07724143352782539,0.06785631342682338,0.08271098300623515,0.0701898997890844,0.0719481897131627,0.0724659710306507,0.07023172805793022,0.08758518517192783,0.07452349566097734,0.09146222862585629,0.07473495782408546,0.07716200088334259,0.07560977614773706,0.06366124864212355,0.08822918815392977,0.07495694164687228,0.08223612881560369,0.06322782501137877,0.0706985892917921,0.07324558112785463,0.08425547454462365,0.0811110171256437,0.07166270919007689,0.06221480555938342,0.08665524908691004,0.06385597944247831,0.07903979816894141,0.06626671900077441,0.07876796276153913,0.07564836736401834,0.07405011068623372,0.07820819767281965,0.08733996824083244,0.08683513233314634,0.08344561516679785,0.07216837762753174,0.07954586198590938,0.08159079934148299,0.07072064673769533,0.08488397973732457,0.07163923914804209,0.06388988468719732,0.08610846966755012,0.08469341627994495,0.07417334871670211,0.08286955206360241,0.08670794148763068,0.09349179506121374,0.08092630073729423,0.08303610113156205,0.06250445271017485,0.07718575556059355,0.06450948142471194,0.08824525298252303,0.07890055370311387,0.07044897772060926,0.05640506854510595,0.08624709411052468,0.07638139012135467,0.07791086885226572,0.07000456201556299,0.08381636361335179,0.08240192688535766,0.0754740807381726,0.08146251535605524,0.07207030379295234,0.07459465723597994,0.07874256327774687,0.06638886351021342,0.07660384817333599,0.08216827371183918,0.09482169636930997,0.07530883241510229,0.06409049408498144,0.07475536868995586,0.07413901829579006,0.07229362007107487,0.07886671373572549,0.08169330257223711,0.05828279567386907,0.08260002094678641,0.05908298438863681,0.07601802588661136,0.08385549707709361,0.07389726565726527,0.07969141402251148,0.06368559094222227,0.0797991009507985,0.09024026266036927,0.07720038533603965,0.08618828539558404,0.07901853084362517,0.07666327272454701,0.08221096113023973,0.07715198952440142,0.06342040217790093,0.07629142503410477,0.0718185027698251,0.07337346105546527,0.06740186726095074,0.07252268095221658,0.06332232845872006,0.08195523035027938,0.0695346794127607,0.0655534230120391,0.0828247989774483,0.07169174483902087,0.0858482063667623,0.062339483667730254,0.08630108873655255,0.07633781255670385,0.07060896488786993,0.08369652472752603,0.07988181913251394,0.08376021670149693,0.07959910352043491,0.07306108216857411,0.08442748810304257,0.08022171947839718,0.0717080182533374,0.07362727309669127,0.06662562750074322,0.08948462673392546,0.08216689958854664,0.06624936025466877,0.07390111112157854,0.055731446585901415,0.07691103180252717,0.08515161510150074,0.07432302619103019,0.07977563833326021,0.06059883745631571,0.08239558642233753,0.07560221981803567,0.09214875926284866,0.05811496928474001,0.07796402263461663,0.08421030162369667,0.07057699076583583,0.08011304430665941,0.0836560981135845,0.07891698036605774,0.07436698458956859,0.06896189466267046,0.08787896533487513,0.07545762178625304,0.0856738099764074,0.0686777283518343,0.07272703035245628,0.08234024990813396,0.07915700323340708,0.07926210526722766,0.06736378205880367,0.07220714277177206,0.08769314369085404,0.08584433578166062,0.08475019749828921,0.08105025933141263,0.06595643236470541,0.08155181337561801,0.09033012989210798,0.0855236661918482,0.07450425940167507,0.08937871990400106,0.0685136081189125,0.07509005031126015,0.0719721005273084,0.07923179972191333,0.0655466932586007,0.06730424908517328,0.07345107825920211,0.0881558646049434,0.06474921732605236,0.08426863572143026,0.08675633312042669,0.08239173318644616,0.08208604490824734,0.051681536818537294,0.07930106994125602,0.08599706751954983,0.08127572460549101,0.08259078360616565,0.06488642679891679,0.07348845759642936,0.0586112106217358,0.07691060036619168,0.08109326338095488,0.06737086329918057,0.07047981493059759,0.08354726711043237,0.08484171908992655,0.07001001427355824,0.07205779578578692,0.07475757343777273,0.09351893611660407,0.0810641989520957,0.07793009281048205,0.0775181103836009,0.06411466332124155,0.07197789259085663,0.06551711219346595,0.08380789503805372,0.07647337967096582,0.07379524400221954,0.08270265710322772,0.07166852916834608,0.08170733407135738,0.07190299000255707,0.08752267048932108,0.07249144809636876,0.07398089517150533,0.06462104696801765,0.07628003304785248,0.08896285442882415,0.0785961619372951,0.08363866702814766,0.07363010458212935,0.07770892460658896,0.07035988714469749,0.06007398758171235,0.08107899140903795,0.08328227208592001,0.09422542727594642,0.07397783515746571,0.06693711917782948,0.08819485780134574,0.08165733700039313,0.07717542396374272,0.0902797380394728,0.08515491850109777,0.07250617042341947,0.06165358025913818,0.07437482807984333,0.07234211820030113,0.08663946171568998,0.08717670867387617,0.07693038255566667,0.08377589876923253,0.07195315436640326,0.08461907995619404,0.08435558958720218,0.0580557754061388,0.07990052738403954,0.07962977258995019,0.06870098307499718,0.08784321150715353,0.0832555484749136,0.08625936981570154,0.07415746356601954,0.07611357118405282,0.08227019011085557,0.07247777625129417,0.0716676323808205,0.08521707560960395,0.06404061548707926,0.07762924328859835,0.08830506906214283,0.07821328898360452,0.07201557339823557,0.06317123718506726,0.07510270445343115,0.06906126785773388,0.0948730256266365,0.08453802774360239,0.09166719748710252,0.08034035188470626,0.08849708771453416,0.08873540644194566,0.08543218013164522,0.06888211638288386,0.06316555936781382,0.07415729494362999,0.08763340453366594,0.07766408610272943,0.06648898227620054,0.07592451267935152,0.0797732172970361,0.08083520902917044,0.08181521763923189,0.07898669844890277,0.07556393455733743,0.06779002417092189,0.08389051163276147,0.0723712385134885,0.09272682678052564,0.07263483130814197,0.06501742715299395,0.07133652904465956,0.06929999814887289,0.06737474888385796,0.06414004487993126,0.08915256266454771,0.08089223293060321,0.08780278593201808,0.067678513966475,0.08657758113909547,0.07916639213126178,0.07272697683979107,0.07042916217342843,0.08584558904333417,0.059628636763128795,0.08406130096041681,0.07003746624805693,0.07478455296141784,0.07959661132333845,0.07211499020221199,0.07855777864314015,0.08566253288765169,0.07305692877468001,0.0685271602406923,0.0702531893845964,0.07983606630547344,0.0623355738067285,0.08836260892098083,0.08047707951437896,0.0928560231380412,0.08585599765201057,0.0851751103270238,0.0757127333833211,0.07912223837407839,0.08819284427791332,0.06731429042665907,0.08303278281909884,0.08806241402118448,0.08576609762978579,0.09007516465464196,0.08650116611294582,0.08574987551954506,0.09244046111170186,0.09033673263989045,0.08811964107074005,0.07905288965740101,0.07951807874181438,0.09051240944164109,0.08020945511331776,0.07453039390696928,0.08526661786413409,0.07606130613428719,0.06958973909947744,0.08236277048161386,0.0692048794722604,0.07036933995122674,0.07245616312301731,0.07432947666903872,0.08111486013583197,0.07490332214087148,0.07417490406647229,0.09185250223379396,0.06013798016511622,0.08130258058489541,0.07948114745640453,0.08392496161768673,0.08003112738567618,0.0647388319536706,0.0821039141879834,0.07053722255378886,0.082483920441737,0.06023552801149863,0.05572324273416337,0.08709744343392549,0.06949802212291281,0.08047111768464905,0.08752980617284509,0.07666348859598246,0.0834074451707853,0.06947561411169803,0.0772763672585203,0.06702582874050633,0.06859514777439688,0.08565378238455486,0.07986342477258415,0.06515602299074538,0.07171527562958718,0.07844447321910705,0.07826535096935355,0.07378901050297647,0.08441506911778779,0.08704644436213463,0.0710707809468823,0.07468995938767589,0.0639104743092731,0.07693157646014194,0.08603721050600503,0.0820925922095686,0.07753439383833508,0.09310555945698014,0.07677570834797848,0.07684626019632187,0.09026857088949845,0.08382651407769988,0.06377458079077217,0.07160556011294233,0.08231998175098941,0.0738970940038529,0.08114643185762793,0.08009411677712092,0.09450156144012986,0.07553230799643788,0.06601609642251183,0.07804707730357832,0.07315077581032489,0.06924111760057403,0.08890036257907902,0.0778841686332759,0.0717201468051532,0.0809979493499727,0.08708681883940744,0.06279935799450895,0.07638965152605326,0.08828229464358993,0.07437784892533034,0.07348516007542252,0.07620749831233929,0.0799768340939429,0.07911455313335587,0.08966493479680146,0.09125528055272836,0.08905408026199547,0.07168858429509668,0.06971391763383863,0.07973622997274249,0.09479679457015124,0.07348748856998129,0.058241801355304504,0.08431624074967455,0.08555078511763539,0.06553210308845736,0.0954352437927524,0.07184122309500335,0.06824437395103422,0.08272625246382073,0.08068994969527643,0.05378833127316723,0.06978337005396838,0.08307398611616387,0.0847879268213858,0.08484259182454217,0.07840873225609896,0.07342068174153178,0.0860491295242739,0.07549896642199477,0.06915558966194205,0.08029112029116214,0.08088455765783767,0.08586380737596724,0.07191519390351087,0.08723288055515364,0.07511657771889525,0.07751910591763403,0.06544113234534608,0.06797938149743553,0.06921380050940774,0.07743862159368814,0.049278433211094694,0.07928165866136619,0.07291699867176399,0.08593443670426489,0.08208607927009977,0.07329959553467993,0.06288074459810786,0.06917231343254662,0.0776308510602409,0.08080902828144201,0.0768115625787044,0.07908353381574873,0.07871020022123465,0.0760415944256598,0.0747000880207982,0.07879857313257899,0.06657879812999588,0.07863749822825392,0.07600714752241838,0.06268033435412332,0.08263612835953912,0.07400241042979114,0.09036516298259176,0.07484239198173234,0.08196035565560389,0.07765952394881251,0.07118386971386254,0.08163944651611366,0.06977472420946426,0.08784184022038763,0.08319691976990622,0.06962759360614264,0.08237473514113777,0.06926169413850328,0.08120160514208652,0.08078442467905027,0.08569783888841456,0.07596834381563154,0.08322232454116217,0.07490544107595681,0.06154664479739498,0.07569268015727623,0.07836540098432648,0.07175304888172694,0.07115605859105834,0.09299162687435183,0.07193781312617403,0.07044211136164445,0.0714385297318127,0.06690488069298169,0.061043922412869545,0.09597065263298542,0.08153309233815842,0.08834114651800992,0.07843466077378125,0.07591086902664869,0.08065953234100698,0.06728253885294663,0.07829888053564021,0.06975398409622938,0.0846080565437615,0.06895804386940294,0.07929072180515837,0.07125647258904023,0.07989680774687669,0.08449420548963274,0.0764988383350326,0.07378958592676474,0.07871749422648663,0.07447804615364981,0.06865592254846035,0.07201027370551902,0.08077371550961313,0.08528557582636373,0.07655715240383396,0.08117135788860508,0.053831160291498176,0.0728759019935625,0.07412217999636304,0.0821185377661505,0.06338524041395219,0.07156945734504333,0.0830010770860497,0.07708145075412748,0.08624224105404327,0.08403734774133598,0.08103143167377183,0.06114702016578475,0.07782148708595617,0.08556210463369351,0.08881288697121009,0.08229480357964097,0.07379996062454314,0.0744769080717437,0.08452495458673973,0.07339948330479285,0.07747345529335328,0.08304170932461202,0.06934565889133143,0.06287085459269016,0.07767702078068203,0.07607686538648595,0.06180600727919999,0.08291578182414884,0.07452855738327781,0.06493966081372338,0.06766778994159701,0.07859515848129131,0.0709579941075207,0.06570214292575069,0.06831226617959374,0.07889166549944629,0.08755365650670083,0.07945519078229182,0.08086794704518668,0.07089389258475502,0.079906043016113,0.08298753584923792,0.06598140968173544,0.07773026554314658,0.08443597864391515,0.07624365357331825,0.08593819519246966,0.08079951611170015,0.07872453995825009,0.08066567449575937,0.07371311020651039,0.06840448813267082,0.0892642017145352,0.06758500814948963,0.07490935923915487,0.0540201207756554,0.07825333289117836,0.08258036190096893,0.06822246885405467,0.07513706987436584,0.07108328501724356,0.061377908657116044,0.09636089549024779,0.07486675182451585,0.058136377289381036,0.08673021935602179,0.05193491776806887,0.07867234108839852,0.06973848851681083,0.07807093131060905,0.07844718533345235,0.07795681408240726,0.07338329920458,0.08446048620374524,0.07869874617040969,0.07830983841522604,0.07874676403115471,0.08799916708505952,0.06613101028877186,0.06773626192303855,0.08176306910476364,0.07029442813118053,0.06672623023598992,0.07254948577840273,0.07535875243645973,0.0816512909666477,0.05784008834351191,0.06365697238878873,0.074007758431174,0.07299415585126721,0.0880715441914986,0.0766410725942927,0.08116524467536232,0.06500227731761357,0.08119361250023899,0.0710806345833783,0.06932554842298616,0.0768348837655056,0.08072100938098199,0.07244554915202975,0.07346072056091374,0.05814781921176188,0.08172826993813126,0.08296046441321096,0.07085187672829969,0.07672629575933385,0.0789350610883785,0.08146761430634315,0.07819049984097734,0.07913575533969214,0.0775463846142122,0.06709609912230435,0.06851822102896454,0.08133063845412716,0.08386682583978441,0.07014297263931335,0.06586214247989625,0.08222733616767446,0.07765258546545067,0.07591324516430313,0.08523580365529128,0.07784393143894475,0.0832898304704656,0.07410335258311013,0.07289133604011896,0.0809659758569222,0.07835223540470793,0.07873314207968558,0.0689623486662557,0.07064918017414495,0.07057970312825901,0.07025656946627429,0.0629966033816381,0.07570382333040099,0.09192261104387536,0.08266188256729218,0.07315379129824598,0.08267423101702667,0.05882733124298349,0.07573226721631514,0.08198603461276562,0.05887294098590794,0.0877155807922069,0.08373478331181285,0.0723891131828111,0.07538762879891667,0.07299377143212014,0.07657170503112215,0.06815117925149285,0.08623318346082184,0.08387182549774302,0.08064184836558713,0.08442480810022136,0.06401627353640343,0.07488880605523734,0.08794252073613097,0.08260087588987428,0.07679306751684312,0.08718540291095324,0.06813395124059457,0.07952953523892779,0.07077671048939685,0.07433860085331206,0.06710679740585605,0.06426314553357831,0.0752443613550654,0.07720275297456075,0.07470535516237253,0.0794839089821211,0.06140286057492722,0.07799690846024518,0.0651912348360291,0.06626431232257818,0.07704481282931185,0.060612820837997,0.0663884650808375,0.08689685702499321,0.05942324967110515,0.07832302586505974,0.09378094361041478,0.08537662668057132,0.08427302920855448,0.0696948268486883,0.06593447437097455,0.0819257595448659,0.07337153319911292,0.08752679053915485,0.07227420344167362,0.08607047850594593,0.060881871525826275,0.08011664515634373,0.07786758893171969,0.0588394989440376,0.06952973796385309,0.08029358802715353,0.07518942859490138,0.07587029996328563,0.08587483795690923,0.06233271308338371,0.070199564066567,0.0702865079920975,0.05983985905011472,0.07723028350848332,0.08187598998907225,0.08336329060843267,0.073334897999345,0.07313480511235373,0.07957905536165244,0.06793539153990644,0.08216771971781613,0.0752331498141655,0.0703090128783509,0.06297099156645092,0.07830599510695024,0.0781642751690212,0.08744141550049735,0.08524446324653816,0.07781957894740753,0.07646872012982522,0.09323425558452823,0.08390243444046874,0.08424530946272599,0.07315607221209224,0.08638638347788635,0.07771225261815061,0.07711885688498782,0.08150417791614842,0.0890118548507427,0.06354044660123889,0.06639847541932643,0.08028935519143642,0.06983341781435404,0.07416588056383809,0.08420656977471933,0.07778919341770209,0.06194226758179456,0.08111378045677813,0.07755223462075453,0.08596885494368461,0.06790540723248967,0.07587400606039844,0.08514058765574475,0.08738605575705558,0.08855140016955954,0.07507356769480879,0.07820684820576779,0.07655822292201364,0.09188762046760239,0.07935464277387308,0.06652648957512462,0.0872614478668016,0.07221836552449121,0.0813005982467324,0.08832630867296434,0.08880832678803639,0.07003573128142704,0.0884248267659008,0.07210380545130615,0.08089984242509729,0.08417707619151027,0.07343695472382805,0.08045375395863172,0.09089771996740095,0.08874587977756491,0.0797901269549155,0.08587223100352925,0.07252128071065031,0.07860160314833742,0.08402304289366347,0.06527689373045395,0.08447888669867895,0.07376834399656856,0.07843744818531101,0.09382703376613764,0.07443038819599133,0.07902592738524969,0.07370969550351299,0.08480014356846183,0.06147224475774938,0.07095557344423964,0.08013163238434812,0.07210961438544448,0.08599043494373708,0.06397357556666067,0.07995816442997822,0.08843823016654079,0.07871593414035481,0.07471059265968788,0.0696110237647235,0.08081184812307762,0.06844214083165112,0.07102405316308943,0.07897305252721823,0.08287718279558168,0.0699583286900777,0.08729419482878017,0.08419571414436934,0.07422771211165291,0.08542890853497587,0.0767749915437333,0.057573083421820936,0.0782466308404321,0.08698195085193655,0.08017006535127041,0.07154158316869506,0.06399234850194427,0.07994787846739458,0.07154792924910945,0.08715314213752232,0.08911267352705923,0.08221248259613681,0.07310994713035457,0.09016239366739509,0.08336919490758926,0.07580775714237391,0.09094611972703034,0.08699897654944377,0.0630083278988405,0.06386092420450652,0.06687824948356391,0.08320522145615246,0.08021054344819324,0.07167243514401646,0.06907865144176942,0.08281362852339658,0.07428067704642852,0.07358025811952394,0.06684365789365299,0.08037379594277311,0.09055341572085109,0.05916259676519285,0.08723599645731889,0.0720266450124684,0.07549802002474268,0.06656058137748633,0.08183382963987726,0.08345008253141137,0.06996798535285106,0.06972031755158349,0.06270181374821748,0.07985870729827135,0.07049496597253851,0.08084764684864355,0.07027993624578965,0.07774393847403466,0.08391208270126918,0.0634673220857864,0.08088896861869715,0.08137342199241748,0.08801894407109775,0.08272284001456011,0.07014860906042339,0.07223632100961908,0.08227334535984648,0.06244466304050108,0.07650495020440462,0.08074294637728749,0.07446711500335307,0.069908285406967,0.07772731689423772,0.07410249642732428,0.06794799768452761,0.07664163066617333,0.07602772567840949,0.07844448294313558,0.08134852529675207,0.08128993341675862,0.07559622087090848,0.06779771310340599,0.07380484760535047,0.07931716002547995,0.07386672158484534,0.0767113422144188,0.08146235976906327,0.08191455597640503,0.0767505542398041,0.0813424002671718,0.07923309117842663,0.09002869334287933,0.07356305140325634,0.07575856974913107,0.0711202669633337,0.06800616672526667,0.0778951895247108,0.07973100201235314,0.08233069263299836,0.08371579441105056,0.06475390320718459,0.07906257277505502,0.08276659099257622,0.06454013533025615,0.08310522027040326,0.08221100258468089,0.07809617514638073,0.08644451509431789,0.08279468542450821,0.08538861374079687,0.05770067227521669,0.08038137427043124,0.06682158093832871,0.06681930377390474,0.08117695958980606,0.0827841356933809,0.08971817224298513,0.08196165493689218,0.06714072664588874,0.07037127524102138,0.07773951699238016,0.06473774568983426,0.06613216552569956,0.09429537653758086,0.07881967390112021,0.07020950314959792,0.06767817526477854,0.0806458291695533,0.07363678422345278,0.08121011012701992,0.07286158976391147,0.07674430363027207,0.06721785803128486,0.07224381774775188,0.06665066622566168,0.06625487133583839,0.06994866941309724,0.07705925332152773,0.09052232352470388,0.07822303617436462,0.06505716256395222,0.06683821175324448,0.07222290185873187,0.08450931789712099,0.07072662547871303,0.07846093751082195,0.08794285174957481,0.07776399373475075,0.07374113203732335,0.08517784141713175,0.075014583262692,0.08146606002766783,0.07908127159410178,0.07548271098167827,0.06038231496208006,0.08439436265640711,0.08428934624771531,0.07332649018408381,0.07846812666406167,0.07646544400027748,0.07416435640471258,0.07285113670067668,0.07911216053386008,0.06862427207941948,0.07788187292974513,0.07613659238708553,0.061555717091229034,0.07555773687857524,0.0814475791840659,0.08980302651784076,0.07049636238504481,0.08638636102853814,0.0896077978931512,0.08203599317606819,0.05718149083307407,0.06650581180219584,0.0743268438729373,0.07376536703608934,0.06661825923273712,0.07973246533130929,0.07516979041611233,0.06682112539544902,0.08398301162052989,0.08395505722137195,0.07464001113275148,0.0764199224389201,0.08224812561563792,0.06265866473410898,0.08843423980479714,0.06361143628824942,0.07944850451546849,0.06494930298080362,0.07973857652395482,0.07603341481693238,0.07312554108432523,0.09063978626879002,0.07904284354077395,0.08515996965272332,0.07279246146402424,0.0740752513350608,0.06276136265087101,0.06924003013271593,0.08574471537370534,0.08279412293647032,0.07703915423130749,0.07774240972857861,0.07786430807482002,0.09300804266925343,0.0832715328099939,0.08006615632114938,0.07567623067656098,0.07388822010917413,0.07021830113993205,0.06805653940664236,0.07910753504818878,0.06908089032563378,0.08605758220506877,0.07785610763113957,0.061955623602920326,0.0720204553200483,0.08070635281716004,0.08278641237382553,0.07855663949767866,0.05092225051027438,0.06531826405381666,0.0680346312414733,0.07265110513979531,0.0760943024461048,0.06750226971711698,0.08232355572912883,0.08251283134948635,0.07661586043940075,0.0727684673436188,0.07194238991982486,0.06956677631786064,0.07990197511797192,0.08850939043126799,0.07722167645978595,0.06151736089028866,0.07043126833197624,0.08521103646246256,0.06394080792104097,0.06640477654228441,0.07942483866820159,0.07932554874850933,0.07734466702617782,0.08123464741938591,0.07221670145663903,0.08808102690000669,0.08618211473879307,0.07600499756764184,0.08286502783134297,0.0672279215886968,0.075118601615531,0.06108071357648695,0.08345487914165894,0.061915099112691094,0.08870537295324182,0.0780392334389837,0.08515456530135442,0.08975195158463535,0.06937364643622439,0.06667717569405798,0.06678487957352572,0.09083861406624662,0.0824996237857944,0.08084565996304936,0.08169755019527278,0.07358718770617592,0.08118889435044699,0.08050431283858502,0.09242184124526859,0.07105795976581804,0.07832725459137424,0.0554952898903268,0.06943301431818366,0.06007661023093465,0.0821432964545168,0.06620043647302504,0.06800319960027168,0.06966875528427983,0.07418932738599941,0.08363734511530804,0.06898555489958771,0.07540588983756583,0.0745804317031209,0.08770513789834872,0.08822430604493162,0.06773297524941299,0.08587984949595973,0.06531184004819082,0.07879510703101358,0.054498976072303425,0.09010448754954464,0.08586062133590813,0.08370840046963823,0.060152712888984244,0.07753863078811146,0.06959375363674346,0.07252467572323117,0.08152829468908289,0.08471407741113418,0.09036490110506347,0.08506589634827494,0.08795730470560084,0.08523896210618857,0.07235859764204987,0.07207676857770237,0.06821024299026439,0.06703136189641283,0.0821602696198163,0.07186450374893921,0.07612937760370778,0.082099911118821,0.09014725181630892,0.06376305998541741,0.07713230850740713,0.08565102063164401,0.07741807036349574,0.08625348635133602,0.06104358944229753,0.06591631707249385,0.08277783373664285,0.08090142880676998,0.08294421916151909,0.07631410459786259,0.08360199540400529,0.07690453197672703,0.08911802156087861,0.07912908500701249,0.09342779769983714,0.0918369329496053,0.08145379546967543,0.07421299030663261,0.07438900994178804,0.07915216978631864,0.07989078839784489,0.05829592255612115,0.08207799768772582,0.09084485653489632,0.0798258177142467,0.06758495087887084,0.0842705158098062,0.0740576873903793,0.07915837492467237,0.08675147570719097,0.08536009053575336,0.07806017511111912,0.0766853014675494,0.0686216431514525,0.08707897452022087,0.09126529669412499,0.06964548960578677,0.07238694384276909,0.08960583152842853,0.08461243588344504,0.07066897624438112,0.08017444318292033,0.0780166270355018,0.06262070641032841,0.07721274409485385,0.0712281251304464,0.07341599132521734,0.0763539351852921,0.08007210780032575,0.08600286526347098,0.07531813052347533,0.08398401279366402,0.06999144565940146,0.07716775446309374,0.08871316339361221,0.09207557736629973,0.07599766630877064,0.08913766717322921,0.07609014372929933,0.08398423516748069,0.07389823946510837,0.09012100086196093,0.07299500178538579,0.08168076623224194,0.08424203217640909,0.057208842083727514,0.08378517195235582,0.06398112044464815,0.08748335308602426,0.07425793784978774,0.07338452336967816,0.09154813750411184,0.07156847798595176,0.07978430538378325,0.07360393377629171,0.0847340478883429,0.06849123718970387,0.08402944475580293,0.0713465116822612,0.08323274517832374,0.054526106713125744,0.08170653668942772,0.07232068014285606,0.08752773840489414,0.08049219779424241,0.08254389969835137,0.0859684625418261,0.05624312564254448,0.07143376424440798,0.07053712180608476,0.0816341777057545,0.09020853889592734,0.06983855472930893,0.07976792295936884,0.05927316859195883,0.0744253318263025,0.0743839271529422,0.07374953070118437,0.0793007896294935,0.07630521577142108,0.07870966280126027,0.06328928091136449,0.08441987826590563,0.0709892386066609,0.08009210419330895,0.08277132682317179,0.08296403721665754,0.0932452968161634,0.085697038801816,0.08610105307628667,0.06817288842734798,0.08583955725526939,0.08101262740331888,0.07576591665176481,0.09422437693387593,0.08099874203720456,0.08286444583197311,0.07450688974733552,0.07834041710055262,0.0708743578951057,0.08815181071008696,0.06856924663267144,0.05429398212888395,0.08407008815304085,0.08167740048954097,0.09144150831694076,0.0775753583872345,0.08383409283127821,0.08797441706321718,0.08246865266766117,0.08062267812065557,0.07186427707813023,0.07766765053819341,0.06702832809029372,0.07427669268826459,0.06888088394292438,0.07884361904019607,0.06680193707206578,0.07708216884787106,0.07421386697192225,0.08117617579320965,0.08619812029404425,0.06835712812414656,0.09133148818737608,0.07508009967707965,0.06848131888863523,0.08621357372023444,0.08309763858649685,0.07016720344289155,0.07161339059157112,0.06215031497218287,0.06018381325911418,0.07770997219020233,0.0826240091128799,0.07975882676556903,0.08337069040296255,0.08298466880100747,0.08076493547912311,0.0806955122116057,0.08398330352324669,0.07773429314039951,0.07096635679192309,0.06116008165792862,0.08809609920988959,0.09140956642606177,0.08155269911997728,0.08247201511044844,0.08152580451660581,0.07745348966924478,0.07757906880093261,0.07987872651890769,0.06249627578587312,0.06763714352544392,0.08653527663660243,0.07329604145258675,0.08281217071289797,0.0607765780777374,0.08152381541255437,0.0886372972224452,0.07974849827114902,0.0831222168293313,0.07665399899629641,0.08483966830685141,0.08080887592148622,0.07703160213511526,0.0927032480364294,0.08980542453155987,0.07352533822369475,0.07616087717341213,0.07256482703216088,0.062207452864948186,0.0848386342159512,0.07835231272996784,0.0834010266221764,0.09366185133137232,0.07517262834037092,0.06762923931840209,0.06985855328637708,0.07898250853360995,0.08974560876412613,0.07073049272369253,0.08224858913652794,0.08537918295324207,0.05624240565309018,0.06833532264576625,0.08879276752179777,0.07831487401025129,0.07754941754028763,0.08238358180395396,0.08950671137403858,0.08117191491058821,0.07221342406396684,0.06994429424401082,0.0759273131085583,0.06577437218122845,0.07917245268371113,0.08591372136595657,0.07833798124883538,0.07784878706681739,0.08368926015863373,0.07796109308064231,0.07916462858291548,0.07146159490426462,0.0774099638790499,0.061017285299773696,0.08663014169721586,0.07223264483504624,0.08559041502773063,0.07540670358525547,0.09182529954351772,0.07932424678958871,0.07757134880482416,0.07155814133223341,0.06963180154278475,0.08380524854778733,0.0795170463434847,0.08984283936015158,0.07274672212245747,0.07447119031369279,0.08411958953970342,0.06331313387576878,0.06849355055059356,0.08702676538792628,0.08455246751338416,0.08895260449474304,0.08520046304108383,0.0764625941894101,0.07682012750719527,0.08358391216519734,0.08401501390195801,0.08373305007053647,0.08131013352992911,0.07892362941982237,0.08701148891325396,0.06153161157485911,0.07256575736756543,0.07348887069012017,0.0744913374069279,0.09388439933096235,0.06600158225423611,0.06686592216134989,0.07663441101872687,0.07386661066405326,0.0631939680392798,0.08207391755752642,0.08264065903871204,0.08942792662534269,0.07425650837852202,0.07489010625087913,0.06820083876516263,0.078921106668077,0.07212279248120586,0.08436900116408336,0.07991302871476585,0.08251971047507863,0.07652936592634264,0.08663463929908906,0.06593208358268779,0.08881293606367013,0.08573752833872546,0.06734896248426687,0.05997941389909298,0.07098129688419605,0.09066322979456465,0.07932754949631776,0.08476895336460855,0.08175127155600788,0.07041550106600475,0.06873162715945474,0.08158754596990231,0.08383504424019095,0.07710585078349724,0.07780250891042653,0.07386444378804692,0.0768329898449211,0.08092590990347612,0.06581225288761415,0.07476753415952009,0.08267241840545836,0.08863575078457253,0.06872795616116881,0.08792520503733936,0.07948048303850061,0.06926804309458254,0.0820167336885653,0.06753867838650118,0.07251656487494162,0.08673794448220837,0.07369583991717271,0.07239686069246005,0.07653998364635314,0.07438186834342764,0.07396096940324197,0.07662783234469189,0.0627924959180456,0.07230264936503644,0.06874490462992702,0.08538351463479586,0.08734522307937348,0.07426797846719177,0.07938291175586665,0.08355191596557815,0.07849523062068793,0.07458609601199524,0.08519700150914995,0.08798840806801297,0.06605774784558427,0.07752359108882137,0.07679541823488024,0.08047695400267504,0.0719364584817181,0.0813748712772034,0.07588454737155895,0.08886930225747185,0.09096886912163576,0.09356515673788353,0.09207678867958835,0.07758621904393065,0.06872115821963284,0.07362533041748875,0.08657969967229141,0.07265833959703297,0.06437211125684061,0.08791238537750652,0.07505254965582743,0.07165041198348657,0.065411813189625,0.08265732140504767,0.0763879140399767,0.07486239507659653,0.07847603400794731,0.08389872752631625,0.07907099189098116,0.07287636209494333,0.08680673664945301,0.07749220246574315,0.06747599732995528,0.07759046397386184,0.08616201228214035,0.07749741768826304,0.08113368895970177,0.07666691355307828,0.06373252716555444,0.08967287614309813,0.084518250099997,0.08122540340224324,0.08683638683195012,0.07365198412514934,0.06716026906244096,0.07403788037489034,0.08448837420685543,0.07143133468864554,0.0771799261689465,0.07907639616969969,0.07515238163097226,0.07933156910013053,0.08915979592798409,0.07674937963288395,0.08751942513228483,0.08076415433963208,0.07146923154425017,0.08584985989156156,0.07010144489529752,0.07578672972445444,0.0695781044949889,0.08414695132663666,0.08042844386569473,0.07815577114681849,0.06902366318568781,0.07783367755143814,0.07284360901584877,0.07482020635310056,0.07303821361205079,0.06116789582306863,0.05993779029599119,0.08406902173691574,0.08428635318623043,0.057111519904092786,0.0660410663760698,0.0830858199250605,0.06279512914180183,0.08891682622913155,0.08371471673070405,0.08232322531723515,0.07154538365647384,0.08819710734153045,0.08534817697533564,0.08250224017252464,0.06956464890582595,0.0701677203249606,0.08310908762346866,0.07380326057298879,0.07638968721355519,0.07737828585944209,0.07670276116702202,0.07028646441496095,0.06996598778738489,0.07355729069004256,0.0651664277820403,0.09356286402902983,0.07283209214559327,0.07856103740826932,0.08010366848241421,0.06988038055555583,0.08881246515456306,0.071728012362957,0.08395388368846002,0.06639789128653624,0.07842694277353875,0.0859286685610926,0.06566828327938189,0.0862299444106058,0.08867751128555076,0.0705202685277336,0.0792522439275169,0.08044326271572247,0.06408391836749305,0.08128023966989228,0.07432049439007617,0.08489260361742679,0.059269355709155494,0.07300690752136402,0.08468530963590618,0.08580058578131974,0.05921448613197772,0.0858651063883367,0.06942623808264818,0.07616004671789332,0.07950884932025953,0.09023621879583243,0.08470607784817727,0.07702972444982972,0.0768435167040191,0.0746783761569387,0.0765474137799438,0.07520990487750548,0.06926052292569196,0.07155609381661357,0.06421984950749388,0.0709164200832943,0.09184393613719707,0.07870697545339445,0.0777327475428221,0.05915126537833357,0.07531321569010731,0.07527157725746636,0.08444170989597873,0.07684290715144218,0.061525581288596407,0.0890247170533573,0.05951861453057018,0.07391081843230093,0.08311728420829426,0.07259585177273756,0.07300225465365487,0.07146354233640247,0.08140669456564777,0.06908952261644838,0.06265143642217116,0.08749408179575227,0.07632015514193935,0.0747912750815499,0.07945644998137144,0.06947986457240296,0.07430452873113631,0.08680089181212447,0.06777299544588854,0.06617370473722586,0.0759225261277926,0.06759817783171608,0.07713500401076724,0.07072335844849117,0.0626962580944598,0.0740626298011762,0.09232505943547384,0.08284700191803708,0.079317626264193,0.08522599395142914,0.08238286785968338,0.07760529612811057,0.08680561502005618,0.0800700723899361,0.06633102484720146,0.08410098377748365,0.07275480812643521,0.05940567148147318,0.0768691756847765,0.07315079137640769,0.07552194167143532,0.06962671412823315,0.08332310764423831,0.08609971922459822,0.0722110525103404,0.06963333428847548,0.07249053102774172,0.0787824871384001,0.07369628814759761,0.06553984141234875,0.07889342791126189,0.05939913495373467,0.08538768354291912,0.07966079139560701,0.08849361420034847,0.0710768647914529,0.06970818587008987,0.07696165550463671,0.08528445290130732,0.06616835646013419,0.07529809164658843,0.08323548062250993,0.07110325658654422,0.0787014084643731,0.07060234152039549,0.08346236034044671,0.08383588771253628,0.05547267167260381,0.08253826556007669,0.08754079399673648,0.078113744221413,0.07502168767563183,0.08528796589178886,0.0689535801522661,0.07694564748807173,0.0772889364386227,0.057421388524871886,0.06625879127116752,0.07024109954400823,0.08322454295504736,0.08442907022592826,0.0805720391114943,0.07908807013793978,0.0740385751594205,0.08204580325970015,0.09099384254449945,0.07885585878639692,0.08197614805005271,0.07089265852675114,0.0870216249790224,0.07698912879562739,0.07715359727717236,0.0651014920031405,0.0940788418906851,0.0795228047771185,0.08655342519283893,0.06038753920531917,0.05834690764925879,0.07241842269843948,0.09011005621844215,0.07555914313186143,0.0809464516568405,0.07865683414100794,0.07651358047836992,0.06930427996412511,0.07182669601897218,0.06808684927670991,0.0802924313804766,0.055498434447994,0.08101038705499261,0.07428970660346468,0.07830536164140217,0.07732705800425288,0.08059766867868035,0.07512791053193385,0.08304005826890883,0.08236993571049284,0.06281019810428093,0.08800975674076453,0.05729282601545939,0.07131744082976592,0.08275331916390799,0.06000497544174829,0.08855694442851017,0.06835851802469256,0.07807522090063032,0.0681110294598453,0.08374310248831184,0.08834924412466355,0.07386343752226368,0.08728132617568055,0.07741461143556065,0.07325662562273696,0.07747499404832581,0.08224953961075573,0.07938424929785845,0.06065476947729534,0.07596196099892154,0.0703976455454923,0.07226602329138782,0.08581114946155954,0.0719466064547036,0.08629979032163962,0.0782499322528503,0.082755368367568,0.08590860174653661,0.07061589661300997,0.08142249310661942,0.06380399892087071,0.06657608625304898,0.06738089577107322,0.0834180752895324,0.08263454487410359,0.07529992297984398,0.07072901088719527,0.07365902465463495,0.0762128993581139,0.08094433078310588,0.08415378991449245,0.07076655679316189,0.08646844275108947,0.0819656827179859,0.06475576331780813,0.06468307127273858,0.0889927036681919,0.09365881343604539,0.08013767895933906,0.07270035985555826,0.08194888715011259,0.0709705765672276,0.08007497763848104,0.06020461589510463,0.08037480733859045,0.07096425221899152,0.07285145682307712,0.0803942173772682,0.07307409199999446,0.07329541664697448,0.0691814160753137,0.0731847719975755,0.06361352500810927,0.06256969018664277,0.08371484314854274,0.07399483797600857,0.06371538540586974,0.07110160659851822,0.07879810099330037,0.05080118074474106,0.07308723852003698,0.06743001849686414,0.08159248034150529,0.07441596380533451,0.06917709950558171,0.0788231429327578,0.09239869957813218,0.08026866918235073,0.0665805616691241,0.06682142972538861,0.06596219131968972,0.07705994172880522,0.06833421161807118,0.0789620556699919,0.08086880558121681,0.08193384652015086,0.06903556344015291,0.07546290612539214,0.06930437661619007,0.07720084979731559,0.08514498223393549,0.07849660343555354,0.07867709974086613,0.08216668504537446,0.0652218753590457,0.08811854273430522,0.0873411507109359,0.0886019670137907,0.0659992811128284,0.08553027867412659,0.07543776073107075,0.07042661784182896,0.07011207402494896,0.061069068727466064,0.07613181853416592,0.07528830603774604,0.08503381540914663,0.07126578431982475,0.0655826581754606,0.09022859312038858,0.08125326291690133,0.0733693009242538,0.07189381353702126,0.08479384624117862,0.09312001521557738,0.057656855552506575,0.07475000334869132,0.06155137674155093,0.08204292393777413,0.06804992350446334,0.09230636366781823,0.07222070775610215,0.06947813430597183,0.0655429606314396,0.06569528593255858,0.09041691802243562,0.08640776962722223,0.07532895484669967,0.06969020866727185,0.07242007110141421,0.07066679523143872,0.07786604843408097,0.07563774873000845,0.07785226631967257,0.05483399720854634,0.08022561527079707,0.08108759764197657,0.07739741599745059,0.07471456327923676,0.06883199843449385,0.07564601785264824,0.07497612846182361,0.061326401627395964,0.07748167793542428,0.08777674770230762,0.06865737176054554,0.07822719265160893,0.07222725916990526,0.08312628504666737,0.08342926432610859,0.06800348652146952,0.08358804847708629,0.08506975414038048,0.07659483152762214,0.08483396893728395,0.08245040177849583,0.09078545003694535,0.07910922984933763,0.07278727099405066,0.06389268631491743,0.08366884238299195,0.07691975708110546,0.07174543273814873,0.06643832490689569,0.08508765639953858,0.07050986273403204,0.09394339160853282,0.08225404818202539,0.08309446063594077,0.08131093795784232,0.08159441845298751,0.07594065626004955,0.07430687969146083,0.07423764479809195,0.07729797569425312,0.06721225700328523,0.07043890008576198,0.08647568339982313,0.06844944217134186,0.05182642401349424,0.08872304124741762,0.07612862789181124,0.06838099017447577,0.07779591071290372,0.07694597647426468,0.07619091424787167,0.08170361037512741,0.08403524280586935,0.05949010152483808,0.06957009945543151,0.07430479169076505,0.08519984906445283,0.08034238668422886,0.07769367957349338,0.07937095771535009,0.06700799859636003,0.08319957378760053,0.07369243150491087,0.09082031901581211,0.08764850168487429,0.076900375917617,0.07831541653837634,0.06620433023931735,0.07823053986504092,0.08710081494929899,0.0856116480179373,0.07751532743276096,0.07913185098642486,0.07970725455071961,0.06789733677411611,0.09076257725719382,0.07236099313176568,0.08928105129629595,0.08931049799082214,0.08739263082078094,0.07472391129669362,0.09230261452834099,0.08576818000733119,0.06190217600198982,0.07796240455491493,0.08186620451509506,0.06684826744747871,0.06920862048563979,0.07794477428271278,0.06667692278477154,0.084937104525494,0.08576308899850696,0.07523821257391639,0.08244814243520218,0.07466411721572706,0.08194081128357836,0.0786131750747523,0.06857793297889125,0.08177918813431065,0.06651507592499428,0.0631161232321363,0.077606109272894,0.08430801371289132,0.07313993028694707,0.08904789340449043,0.08637715298388587,0.07489010329430854,0.0711233474328592,0.07603876133829322,0.07744458860405923,0.07553881747941257,0.07128297339598813,0.08125032918970568,0.07117018433948269,0.06862945085571767,0.0800788465598098,0.07389004662827166,0.07126981651801248,0.06839631087004233,0.05743731110662108,0.06764147871023671,0.07235108276640026,0.07593405666795834,0.06839775535352147,0.0696287452628215,0.07612582033143168,0.07619149145180631,0.07713043630178597,0.08151623362996958,0.07277030143226504,0.09164060074316902,0.0801154554625017,0.0790703685217892,0.07499826689525156,0.06272604059893878,0.08569109678235677,0.06867347318113781,0.08185248421455121,0.06248659902755834,0.07738382587882395,0.07457852076817922,0.07987519316815506,0.05721720007453525,0.07321222564331353,0.06998054202131124,0.08597410031615209,0.0793433366939368,0.08873171145254724,0.06535481707964239,0.06921675864542405,0.08611657514254512,0.08388022593593143,0.06893123113923687,0.08148986131032562,0.09002744146635433,0.0850461289454384,0.08079370837574323,0.07546941548285185,0.08748959187816646,0.0770931063763996,0.07066737679566398,0.08563577407472911,0.06855122796993893,0.061336241357672516,0.07786261204749204,0.06742601356576296,0.0739774971895667,0.08059192517796993,0.08369061141091563,0.08501061527768733,0.08154407417828513,0.06718144188734644,0.08012907625859998,0.08148256796296938,0.07853955356188475,0.07537266618198138,0.074726888032389,0.07113288688000755,0.09176306942812987,0.0792909708940753,0.0856241193032857,0.08048340795550163,0.07572589443235042,0.07546775914207607,0.07621995796825679,0.08927940946646123,0.08151444159383085,0.07398216323309663,0.06982746107927236,0.08915225350176822,0.08009940972182128,0.08232062151829249,0.07663104958009356,0.08384390414642033,0.06403790113251082,0.05685648581410329,0.08577021938462537,0.06939542707904886,0.07718389255533449,0.06897234880089932,0.08485219108912159,0.08374975445891196,0.07114087343709924,0.0703381879959187,0.06879247630540636,0.07251756266830592,0.08411298962622066,0.06616889986020918,0.08822966492688442,0.0644200490426057,0.07729016484323956,0.08275021700634133,0.07858148999359685,0.05796477484970457,0.06600879270285626,0.08713333949570393,0.08673148369292051,0.05838828651285549,0.07475222417414376,0.08352880895296541,0.08332202609362899,0.06538240206197499,0.06641546595870261,0.07296889873811195,0.08453369175918751,0.06745488403614822,0.08088847033538495,0.07316015622947086,0.07542743327632029,0.07651962146245876,0.08695929892090556,0.0920124515086726,0.08404543310439262,0.07070035094781607,0.07196951883883843,0.0756548345609391,0.07692642541798742,0.08498107096688887,0.0796694056662274,0.06725175128119998,0.07936364063655874,0.07044124363108092,0.07113051546684065,0.0725849872894346,0.07562654042665745,0.07181053604594029,0.06417518648451259,0.0666835605609198,0.08380491397649978,0.06569912580112308,0.08619581751221356,0.05999567427130915,0.08277131980173515,0.07715830129550305,0.06068518380543517,0.08718462073314773,0.0799726598829846,0.08082193887826528,0.09078763021458965,0.06991341307175247,0.06318906438495253,0.08881754171305126,0.06700649891019016,0.06826930050236067,0.07162586733839325,0.07955022581816804,0.07568493932922697,0.08960901987800896,0.09092219786019215,0.08256745281976799,0.07899995750295485,0.06889688207915629,0.09011022132360849,0.0844830473669634,0.08542442880609608,0.07430042471308618,0.07903446287489485,0.0888704389472324,0.062395773156688025,0.07655358784784153,0.07006519622437667,0.07736456308609305,0.07147937623415392,0.08668917239621618,0.0797073089113556,0.07171784407026913,0.08098197936648671,0.06136441475306619,0.06981699112400379,0.07748925766236198,0.06889497036690201,0.07256261334528924,0.0791797926686878,0.0745571849924389,0.06483253809439402,0.08911420349472021,0.06659919206403145,0.07756936830761377,0.085353609045658,0.06194715843858175,0.0756964662903814,0.0816661693713647,0.06892294573899696,0.0742137116595034,0.08771440497499666,0.06726298834819373,0.07922892037982185,0.07467922527994805,0.05699102460556092,0.07053725202515557,0.08206013560435729,0.07653378326315081,0.05785700341671694,0.0637210042739651,0.0758736265031122,0.07066906728340183,0.07700982028183245,0.07483657248495112,0.07086374750486338,0.07298293299181775,0.07136251615923712,0.08640964503232987,0.08200018302291281,0.07273325111108077,0.0729219577003613,0.08874367006481776,0.07498707527389621,0.05698498718324359,0.09104584250618467,0.0860459409962091,0.07243558658672683,0.04495570545653131,0.0713533817042271,0.07855686480332336,0.08762650890917334,0.08380102955248256,0.07372460716040116,0.08302457528320267,0.0790086602624923,0.06686987612659612,0.08647339261906331,0.08159007804540439,0.05777459928661259,0.07911301842895918,0.07942832539023242,0.0899307355602011,0.07793075720260023,0.06874458253830415,0.08252436021143497,0.08733250351826663,0.07639453795601925,0.07784601389148908,0.06774495260157702,0.0871603058956065,0.06960522047689635,0.06844430807478094,0.0685939219974198,0.09201754418022598,0.08713813505658075,0.07586377605937306,0.08214425327464998,0.0834415325579697,0.0793797010867397,0.09466686553194319,0.06422627500359941,0.08154815039859796,0.09037328104882872,0.09153891090838387,0.07667260786756161,0.077250866033077,0.09360746317345842,0.071219781092979,0.06913550945278821,0.09333916207564126,0.06716403311976823,0.07325138381172223,0.08271698661566876,0.08365239626162269,0.07047826405061004,0.08483832378044617,0.09196635073506268,0.060562355996698305,0.0715090344170369,0.05978966384962664,0.06364742289029363,0.08962080205204537,0.0768375072369521,0.08253778593269753,0.07983013796619022,0.07541558340886668,0.0766231635997951,0.07873587618526737,0.07838843747248592,0.06734378328946859,0.06802299342077724,0.08590870165298534,0.05839890484156199,0.05786698747365892,0.06952896972024558,0.06991660687398416,0.07254697249268283,0.0841250150908937,0.08739594234687284,0.08675071236206,0.07110151595354841,0.07856188868296554,0.08669095194840912,0.08142182549907184,0.08317638383104098,0.0860643269556111,0.08001774529543296,0.07026049151312574,0.07625475484491848,0.07552686492808969,0.0696514099420967,0.057600432252870606,0.08265444253038703,0.07734441089265987,0.07325757576716319,0.08907337188637016,0.06831880118307215,0.09020707167140624,0.07732161178922561,0.08265249133924164,0.07479018601379793,0.08220565102132582,0.08138508243629977,0.06820500894299389,0.08345269939669323,0.07995107724064827,0.0769816417720091,0.08216273894746622,0.07519804222479809,0.0788207951005881,0.08954743695543445,0.08664680725602254,0.06487923233249465,0.08296201651166929,0.0798641497695267,0.0759511496477212,0.08271610887494167,0.0711647298171725,0.07894621644636626,0.07781633976406785,0.0618790791637303,0.06501103501383998,0.07469935928001831,0.06326063372605917,0.06717979121044779,0.0727512463510396,0.06716188614525505,0.07411451909680097,0.0678902481172526,0.08605157510244782,0.0669995466623085,0.06447515251297282,0.07146705153812732,0.088591870859966,0.06763296653550274,0.07885802975405043,0.068620235670833,0.07393148317576856,0.07785678501716256,0.0838562592174915,0.06943032509002309,0.07241889780515398,0.0695390324483808,0.08019443658117517,0.05249072397411039,0.08423181885289273,0.06774256226526441,0.08066350808656401,0.07962729156651642,0.0741329509144057,0.06326035217640293,0.0747110906061968,0.07814678935053868,0.07942640400713225,0.08336980771007779,0.0734450545267273,0.08007667758005876,0.08027934918895147,0.0843070144490376,0.07688097324519483,0.0715884556722032,0.08211905153630109,0.08011802520889678,0.07443254812805003,0.060966129875702296,0.08861032716666509,0.08081273438152435,0.08333561710800891,0.07877486909665897,0.07647730830375919,0.06947373206398086,0.07571878686418414,0.060689090226558634,0.07281175998414596,0.08116623425135443,0.08328584257037122,0.06303610824867933,0.060004984702871834,0.08398072330480003,0.058075897474788106,0.08037466155293803,0.09179490394500033,0.06945269632102649,0.08285446307459132,0.07574463918786473,0.07614276523420777,0.08493299914451996,0.08287188387331824,0.08430748167699162,0.0701031293082391,0.07183578464193635,0.08191616951769187,0.06825125813277202,0.08464041498426919,0.07919324205661743,0.08380484885354003,0.07978436120444699,0.07859782962044831,0.07197934855083594,0.08260791459568725,0.07052791047806725,0.08349779724090134,0.06871865501140167,0.08412767548902231,0.07400032578603691,0.07825168400320529,0.07539799816205883,0.07827832928239006,0.07746762565229397,0.07363254005590317,0.0824840439122711,0.07693542077942778,0.07293296405663391,0.0640037695550097,0.0782615664887632,0.0738101362333142,0.08631217052047982,0.0837144978505508,0.08497081320264376,0.0598467680832261,0.06823107153189276,0.07617685632705051,0.08036464006821152,0.0712242900929707,0.07774488117824235,0.08511078614872972,0.07910975902587053,0.07710767024889834,0.09119692488878431,0.06618997924764222,0.08271957294820115,0.08232726119335276,0.08044394200392299,0.07288350646350708,0.08004543638566422,0.08045334437881717,0.06442074257293273,0.08310718299459063,0.06664531777263778,0.07472881258500752,0.08072401135588161,0.07236680110319407,0.0782686320378185,0.08898764543228785,0.0753522068035262,0.08727478388689125,0.08079823262133831,0.07802168768963216,0.06483417198215667,0.07831995851329716,0.06124750266582493,0.0701561884313277,0.07652457602713736,0.06801805920016235,0.053364143731581305,0.08519153114728384,0.07327344889446266,0.07502703954022258,0.08399534986408275,0.07347727195391635,0.07871294444228286,0.06014522298341582,0.06385968199962075,0.05707155562947275,0.07869983122276086,0.07807898027181612,0.0793132811469035,0.07314502963755531,0.08290034942755509,0.08635697280396167,0.08185303737976163,0.0746973375201507,0.07185186167165651,0.07427833873110225,0.07312041122628046,0.0794042534900612,0.07498903499844128,0.08341446255499482,0.07459927480113368,0.077276173775306,0.06826675077917296,0.08862388430010748,0.0800680857701998,0.08650954808392976,0.08132299681612373,0.07424108628444415,0.07940586876351953,0.06369783589865577,0.07470429973422664,0.0728213351256744,0.08669197667623782,0.08027995531181112,0.0812933588523845,0.06589287498128839,0.06838367082401618,0.06826045802276337,0.06473287310540833,0.07733733852400024,0.0809470840298001,0.061142055197275925,0.07486769852696362,0.07146954241619075,0.053408798072989316,0.07253847062945057,0.06483958604161308,0.0694689971948912,0.0863653201859714,0.08963452343509785,0.07925862174417453,0.0757518611456517,0.07786300240307262,0.08059670138570804,0.07178255768243394,0.07921177064324599,0.06435562260879782,0.08437296871856163,0.08297699867363978,0.06898113211363555,0.07230134123910044,0.0957954330612902,0.0798130336661724,0.07017377275829445,0.08869852796658516,0.08400013173941423,0.06997599865045427,0.06838436964900732,0.0672486406025431,0.0821536176249292,0.07156648360535497,0.07224203906521964,0.07540120255080474,0.06346407556266975,0.07753646371003689,0.0817151983675964,0.0574393237785749,0.06938212102929314,0.08891615158025118,0.07267836053342286,0.05944322346061304,0.0740252711827816,0.07971741195934062,0.0751088056032326,0.06526815489717537,0.0795113130157802,0.06830071731660466,0.07823138901802783,0.07601229456842151,0.09040175527185568,0.08599239509273902,0.0865982901904114,0.07775516348946249,0.08916331340427552,0.07925853008718704,0.07883124981822315,0.08280324994629057,0.07590640759195716,0.08923430865705007,0.08469554194965738,0.08559217309268083,0.06270398654720388,0.06457590804312424,0.08645936612307364,0.07784615068596104,0.07350159087243063,0.05901976919942404,0.07657355909866177,0.060094970083037574,0.06813942465745787,0.0910028686483965,0.08471671427128646,0.08050776037206812,0.0759505022269835,0.07759003011821738,0.0700617879112326,0.07489349460321268,0.07566452909823246,0.08335511321800979,0.06825050656484588,0.07517208945732451,0.05371588144743885,0.07237187662250028,0.07023506115525194,0.06355428241181021,0.0703290772118913,0.06718268126851958,0.07201705845519435,0.08652976762307468,0.07620713773797252,0.08699847184475593,0.07487719976337333,0.06975760656136433,0.0729735370985049,0.08523378108526969,0.08705349154286246,0.07889861228960968,0.07285571544283558,0.06918588899368346,0.06198825860454372,0.08408795191809902,0.0803629808533778,0.059184283327712736,0.08307721218108328,0.0893996524560263,0.06892792039439988,0.07455767082225367,0.07359971433060364,0.09206551042839231,0.064763002660856,0.06596671593213865,0.08119732285858225,0.08567535875102734,0.07445174574125168,0.06901811658388285,0.0895635091777429,0.08308151483403298,0.07585728571784311,0.08021369104587799,0.07397217657127725,0.08293463759049804,0.06600810583085104,0.09291416995476506,0.07553256330230754,0.07333156640264976,0.08270577659595618,0.07753418081401232,0.07854136471596374,0.06336860818601749,0.0715237203815178,0.07866155901111463,0.07347157837999962,0.08595934955255335,0.07651063703725239,0.08706958361857077,0.06945450367938368,0.07795566315389235,0.07668462449097886,0.08833138354927815,0.07519480718453246,0.07293787116007792,0.08663057472688578,0.07134173998811336,0.07344305524705685,0.08536935916999353,0.061444226528280056,0.07723079078692643,0.071781459245599,0.08953180552865342,0.07068216048104459,0.08785513668607683,0.07739275305965379,0.08852182115702592,0.08436237829375189,0.08301909220136143,0.08231105232676345,0.08605978984172744,0.06726889066707115,0.0858824362369203,0.07616757041775514,0.07913096054783458,0.08565867833168074,0.08613846389037054,0.09463444393455916,0.07270140058478396,0.08049219817482513,0.08021996745228058,0.07913096706912681,0.09074006638150554,0.0704303768680239,0.0739328596307201,0.0864334428459258,0.071214153458845,0.06062046301817825,0.08610984536961541,0.09395860395694515,0.08332739776780335,0.061443310562964,0.07790618137748767,0.06896037553596004,0.07718258350502465,0.08809420240452742,0.08492307392724632,0.07667160747479414,0.08526214209744012,0.06847682136957303,0.08621731084000037,0.07957440134482188,0.07212390681560761,0.07789212217627971,0.06660831493598454,0.06412164043004989,0.07898667506943066,0.08098944371744929,0.07296716875981178,0.07016833298296521,0.08051466932585499,0.07521617295779087,0.0939061259108743,0.06826933710934464,0.055400068576689374,0.08452498167950104,0.07312420855595213,0.08934694967089138,0.06736971832524824,0.07742681713314653,0.06800416170789352,0.06850115759810974,0.07879145219539752,0.07706040662317273,0.08416925789803514,0.07636896872277218,0.08233479413171801,0.08841983247880292,0.08486460153459159,0.08639823671083316,0.06925031635881201,0.08200399376827038,0.07328572465232239,0.07519933271760201,0.059447848900376965,0.08908610058622171,0.09320688513590007,0.08412803110058434,0.0813795347866677,0.08318425151391384,0.07707630585991891,0.07082729233013217,0.08575399894606764,0.07989928080001987,0.06421115565829799,0.08693689512041455,0.08243424533042895,0.07952816480617664,0.08776401828258841,0.08292763018541788,0.06870473308005268,0.0768945128673241,0.07201167339927353,0.06808308691089632,0.07436354854603486,0.06230045321544681,0.08921735936323452,0.07693877711620054,0.07625893394298286,0.07810144813814526,0.0955643922224918,0.08288079934443915,0.08305177845586237,0.06537710765037476,0.07490895316869588,0.08634444552208903,0.05843597715012393,0.08442403740315307,0.06495750709258782,0.08637684816521296,0.08921306348579792,0.06997141239638996,0.08189232599881965,0.08314963872805131,0.08306681767382462,0.08820774380604315,0.07635215225363834,0.09433638006030157,0.07209902686106859,0.07988736227640506,0.08130794172990266,0.07861516423140115,0.07342802768069978,0.08652838710478639,0.08355743809996545,0.07300358204071454,0.0858771004953417,0.07516119739466381,0.08018979831070125,0.07828604758543466,0.06977757939357201,0.08979806446117736,0.08143200071295832,0.0750071362365472,0.0640699927574946,0.06789668804500971,0.07986876966475982,0.07522098883517456,0.08029284162968779,0.06490714538019335,0.08261208839523687,0.06615347696111695,0.07553606635597923,0.07726630612846988,0.08631979565474249,0.07565472240363125,0.07517968901408553,0.08290628800679925,0.058313980350648414,0.0765777472997192,0.0808925601805865,0.08047887396651775,0.06569928685334467,0.08275218118815625,0.08054690753192822,0.06849831054065861,0.058219296467521026,0.07911624978525178,0.08420050369649679,0.07082562396566183,0.07996419423355061,0.08458754460229649,0.07736967213188858,0.07928788694508945,0.07920026184904061,0.0751553373638627,0.07377080420287382,0.06767670417668281,0.08115435609011139,0.058086290574417554,0.07605369161056622,0.07872074043693791,0.08305038587026733,0.07840891431903997,0.06455693723088954,0.06476028314444825,0.08047716610858754,0.08090538442216921,0.06956513992216118,0.0748023590714603,0.08380534304776872,0.08448025392697385,0.08017410529007386,0.08154784985070718,0.08650354979518565,0.07259244377458424,0.0659524256034503,0.07785655674137217,0.08058796113796185,0.0683162785433919,0.08375769669410861,0.06357894668922681,0.06554059922140751,0.0828545038270526,0.08430783591512458,0.08219175151889499,0.08054086494238914,0.06945266939308062,0.06738959312109571,0.09254622134871529,0.0857887607677433,0.06946916343775722,0.06481204391475001,0.07573287830248027,0.07421454597627089,0.07205748866734837,0.08249565956291097,0.08188400491480932,0.08361416883559449,0.0740058869215342,0.07216103519298134,0.07606832088527996,0.08104162230367012,0.07676099746453999,0.07676265462477518,0.07383736736512392,0.08363253444978105,0.08230623293970193,0.08014095345634063,0.07986450425881313,0.07581557312668248,0.07144876871343554,0.07741200727703879,0.07480907707140236,0.08302275916437668,0.07541657901552037,0.08170905135535704,0.07863212460453359,0.07416560557492399,0.0681043854485372,0.08085870719646425,0.07903230295532523,0.08695142090953631,0.08779225064479101,0.06480190352795473,0.07129572349985289,0.07307111065888888,0.08171970119257752,0.06121712215456722,0.07800412177042738,0.06352557766369551,0.06386057008408232,0.0708727454549393,0.0769369784278229,0.07527971122167365,0.05797066201234456,0.06447963924870163,0.07782192472381158,0.0802666221111803,0.080185102770717,0.07073773098911082,0.07581431216000904,0.06792702857851832,0.07738413832552522,0.08074948839324378,0.08933233459032808,0.09111535556932088,0.06382970845097823,0.08526313965695925,0.07837241999037672,0.06589017287302672,0.08308609130387506,0.08096162601356999,0.0751760324548021,0.06356687458436089,0.07038733388622612,0.08032754631788254,0.0625975495197795,0.07400403829767585,0.08626499341816299,0.0630866296890478,0.08206737691447119,0.07085377432547477,0.06882379720614101,0.07083427358465916,0.07005555366546534,0.08329437501627604,0.061298131598124236,0.05983188408457557,0.08282549725080365,0.06232117597570606,0.07388511217426594,0.07560571777784382,0.06986809503316437,0.08009096068375521,0.07901260276591693,0.0702227924789848,0.07415248132114952,0.08718019689041684,0.0588018595575508,0.06837761396475499,0.07741714687628497,0.07556806299070104,0.07263086967453258,0.05995270229750053,0.06569889759581367,0.07879579139742049,0.06982745360990719,0.08368104568204818,0.08276974317134625,0.06333231404404305,0.06740470480250699,0.07560757032124871,0.08612012712525671,0.06173387616827354,0.060315737656480656,0.07878806162255908,0.0718676745342966,0.08229072155545608,0.07248612300383483,0.0877907142080527,0.09061644618981327,0.0708651896964095,0.07821169212542617,0.08177703649863612,0.07805191942889461,0.0782238365458636,0.0825823350232651,0.07997593539770144,0.07145140017414678,0.08307306651805674,0.07889794244330074,0.08829338673924506,0.07585100172275591,0.08333306563616626,0.07388926723556052,0.07891531636473079,0.08437633504312278,0.08437868165157142,0.06735335314086707,0.08268141911832602,0.08091971565435299,0.08344287447631374,0.055060453459514436,0.08453109737345982,0.08002895270338911,0.08731185992329771,0.07564630009837163,0.07825186319073743,0.07437111373820283,0.0733980161745467,0.07874227469038428,0.07736962135018413,0.07459636136643504,0.06669272642890048,0.06733588267527753,0.07456436298618985,0.06858040868465069,0.08741035590713281,0.08040274012077361,0.08376502410093126,0.06969097159054147,0.08223298218612418,0.08862123676615391,0.08329700977657435,0.08524060104596075,0.0660024781286258,0.09231511053081365,0.0712386769364208,0.08411508667458514,0.07336955898407492,0.08652699855988802,0.06291217227407962,0.08229589686055011,0.08864653022344285,0.08479351506604307,0.07287806652509737,0.0792533074805554,0.07616639752928721,0.09394860215565984,0.07918268297210594,0.07440290180432224,0.08037716720870103,0.06342027624692151,0.06771908691530087,0.0718306522654632,0.08030735914652454,0.07570424903294817,0.08472910139772417,0.08915727829684407,0.06345115918276814,0.08109910387499951,0.07806924640691762,0.06039623657555479,0.07744140960349809,0.0621769427580406,0.08804290883280007,0.07265924331004225,0.07870686402799458,0.0814309580324087,0.07812190294330186,0.08724963523995712,0.08425275173066721,0.08595066573809347,0.09095556413919495,0.07492012525021326,0.07140994900292155,0.06313577386734992,0.06000693371906473,0.06928902159113562,0.05942184853965776,0.0861667761212569,0.0869524472476669,0.07165746314436552,0.09460939533800007,0.07613241641749884,0.085971885064848,0.0835198307031619,0.07548187317462368,0.08646428589512498,0.07965928108205662,0.08483805343398668,0.06569451797469997,0.074801785503055,0.0731587708954532,0.07330015756784357,0.09276047836069891,0.09058685815704173,0.06313452402927232,0.06989876246166077,0.08166733047736716,0.06384479480240865,0.07877720988747217,0.07706235804252887,0.0814166815518556,0.08207938048943698,0.0701481043887465,0.07467636731486407,0.08469870214011928,0.08775711266401756,0.08186212922849097,0.09009385395201466,0.057807415295789255,0.09079070080930238,0.07954454496982019,0.07846352030361323,0.06162955890175088,0.08714884957589629,0.08118966285633193,0.06992649893936059,0.07400941880447229,0.07201763290865268,0.07791977240761551,0.08046237330161088,0.07712733872048505,0.07489360630271148,0.0784151026014512,0.08224930210407153,0.0810569137522416,0.06806122288723225,0.07216926763727502,0.0890186577665869,0.06768647329594119,0.0780271477450703,0.07903638447739746,0.08143965811744079,0.08986073084937096,0.06978306922985653,0.07565143355434185,0.06797557670858695,0.07275038394148516,0.09151960068336223,0.06070730775931148,0.0694186936109306,0.0746010382610261,0.08780560969923273,0.07367518606765514,0.07420324508693224,0.07216263602047597,0.08917210367145045,0.08220071863094866,0.08144872046564196,0.07027335141132932,0.0708898421987156,0.07355528971983961,0.08638769924891987,0.07909032758184953,0.08075401419643122,0.06158142466383396,0.0812133229462327,0.0567279267891309,0.07761938009549718,0.08684634439533029,0.05867492340208841,0.0770777201173874,0.0793941858307397,0.07486977585870544,0.07993551715802177,0.08751930813685208,0.06498124077476883,0.08098507757700894,0.07795382332278367,0.06953662514225667,0.07433159088849836,0.08043308055274544,0.08120030140124182,0.07188269972840478,0.08438315187608675,0.08129421969972447,0.08049401023705699,0.05926574620317812,0.0891917738110985,0.08538499849575212,0.06802443223702964,0.08813319737656047,0.08219228161784503,0.07708138809818278,0.07909812854769166,0.07687465772543672,0.08463035645064784,0.08492019357762587,0.07875388247648674,0.07001211303542249,0.07514604422904012,0.08208533345275881,0.0791045690241556,0.06523077957569356,0.08115839822164667,0.07090834019763413,0.06696274377282158,0.06820103632653153,0.07259483635027707,0.052204831095006794,0.08011708407157478,0.08002655588945196,0.0666411300037078,0.08240891250342376,0.06912888706835456,0.07475534089359347,0.08072549113091518,0.0711526853883993,0.07839308986862542,0.078499301541397,0.07694533064722087,0.08279367229637415,0.07988457032999938,0.06569937730946965,0.07503515819329987,0.06143960462515383,0.07371295118826789,0.08436060396223155,0.07783817991020106,0.08178317388652273,0.08450045928940887,0.07600834444825955,0.07979412272476241,0.07771881696232123,0.07994436110696712,0.08593740881418324,0.0680877030148749,0.07414108997948585,0.0794170824672088,0.07344380500346212,0.06455721414613699,0.0781636862355759,0.058606489191422806,0.08862040158452653,0.06357151006025705,0.0792923859918383,0.08498044413638231,0.06886153592028885,0.09212678812406774,0.05876701736412117,0.07851408380079883,0.08317972615376024,0.07627385925223132,0.0908985190409119,0.07790702773730988,0.0835110223206578,0.07656504819123355,0.0826983629131366,0.08493081336780499,0.07648841560528116,0.08516941054618289,0.06903147451667449,0.0808666748269114,0.08308954183986014,0.069127659978113,0.07162581275513685,0.0791807864369257,0.07275977914808684,0.08901115097328817,0.06851890366344093,0.06384444830147089,0.07536583588001276,0.07262123156146813,0.08960441946471864,0.07711472798630162,0.07668121023097857,0.0667299237100823,0.07270677715884764,0.07980622515600991,0.08502778022774019,0.08090351811545615,0.070923414711729,0.07752095218810118,0.07908820457800839,0.06745301776478663,0.06904466614613886,0.0890666641833536,0.06471283895823379,0.07169142374371301,0.06868929376447958,0.08073723263472724,0.0730145400379445,0.08645130812281401,0.07251651206595772,0.07383910539080715,0.08727892060366005,0.06977643436967229,0.0858864087767908,0.08047186612909071,0.09016246058770631,0.07597866538409556,0.07479632685894422,0.07475616823621839,0.0647782382655414,0.08078160764394977,0.08440242203806055,0.07581555650724858,0.08360359354473253,0.08417505117664688,0.07493822191777615,0.08300700763493402,0.08064290641324125,0.08373892663902288,0.06739402623717951,0.07562276707247151,0.07553873655934017,0.06128547834660142,0.06323291524964286,0.07083923404684249,0.06707456594509267,0.07896610086398889,0.07813656167790162,0.07286894613951395,0.07603080495617014,0.0757679329931887,0.07196633928669786,0.0752279210683971,0.06860001463213246,0.07507729735786373,0.08912760247612571,0.07879701586444512,0.08739842929739156,0.0598358955989554,0.08511026789973346,0.07444408265496438,0.08230410704350233,0.08615816006840224,0.071127030386375,0.06797145953514168,0.08381320445445745,0.06405550249924297,0.074860753408998,0.07874141827201878,0.07587348357422594,0.07635727642886495,0.06806766625211136,0.07367567343812348,0.06606981084397864,0.06830478364602682,0.08086450411643109,0.07483229640794134,0.09031799022150058,0.07313195468133836,0.07102219353060951,0.05980258419451837,0.08297234836603692,0.07668576053014786,0.0809700426112154,0.07778162077513442,0.08051517234032181,0.08188214665336542,0.08658251544291604,0.08238401669477258,0.08297639560548395,0.08064279791829825,0.07060022122590753,0.07181839855346908,0.08289111386261772,0.08180229298012784,0.07886459574120364,0.0839326621591999,0.07862673819700391,0.06439189838836387,0.07847427352347011,0.07658408971443464,0.08995426670314202,0.08261392124811565,0.07855684814379489,0.06746526075585701,0.08576904665939378,0.07540320358328127,0.059784907826264166,0.09375303972621073,0.08878833014477486,0.07464869038083932,0.06665573717200356,0.0733879842181,0.07411644645170216,0.06795155849096046,0.08366942668017921,0.06903539153131201,0.05250075888137623,0.060908947729630444,0.07635937659777337,0.07628775473359747,0.08078948308360176,0.08675148201232896,0.0823064451597513,0.07675291147723783,0.08377295577190236,0.08092404914318081,0.08772676026159301,0.08137419433534135,0.07443323488430825,0.08201829997659418,0.07760491242155759,0.06192699677349755,0.0767478754022575,0.08532095967734916,0.07464347842950267,0.06551728178746627,0.08044978497886857,0.080437823573817,0.08531319700053716,0.08523572255355352,0.07306140322741299,0.08389832292826044,0.07724707569541822,0.0730274465763891,0.08131850840522356,0.08929059694149855,0.07168943238714236,0.07694334333409071,0.07212703083469141,0.057524064148660695,0.07451648111834117,0.07323419342224624,0.08819635413786323,0.07169520244028554,0.08093925831633653,0.08082430548111838,0.08600948556950708,0.0730666179109977,0.08642261021709167,0.0819260028186251,0.06552775466801276,0.08304161882673904,0.08136829226411038,0.06733955988991219,0.0800541999390241,0.07027296205668412,0.06571629076100799,0.07696812681365575,0.0844674998021089,0.0812459192047817,0.07455969308806598,0.08312354108209194,0.07825997745511958,0.07035095620839146,0.0907321427300944,0.06801828010892474,0.08433516500354926,0.06573546832173754,0.06898573586289661,0.07806338001679464,0.06916770477415647,0.0766191291769841,0.0761577901960699,0.07776846941702717,0.07187964749339427,0.07933051225693005,0.08090874147018216,0.08494150704561484,0.07967255762513548,0.06990266569215739,0.06804358067207103,0.08750745664285242,0.06262401414252214,0.08425845090413858,0.0779220200683528,0.07309628470320224,0.08434454778822514,0.07952430570506627,0.06572745085417048,0.0719620988963134,0.0873046111330574,0.07548490643093594,0.09213047050810369,0.08709574864359004,0.06559048348586527,0.08830730973289644,0.06532139022577929,0.0773054641005862,0.08940509899256896,0.07153187752938608,0.07650211705184623,0.0650184633302765,0.07666109619830769,0.09293384079660101,0.07995869012997259,0.06430629483152821,0.07804665726764914,0.07530746614383677,0.05049437730792738,0.08805968151259205,0.0793856361293748,0.08510248724276637,0.07588511594757326,0.07909587735992527,0.07493176757454983,0.07253103600287114,0.07983752146935173,0.08065687593268533,0.07736888384903703,0.06605983886765321,0.07811020523973802,0.0913712953837062,0.070957546485949,0.07701473281463304,0.07889517507115673,0.0666049688199933,0.07535079677243592,0.07190122874030408,0.06663213114389016,0.07075616041573378,0.09314235621343422,0.07936822486480499,0.08231324557065536,0.06717700817363148,0.08024579116767822,0.07132135175404414,0.07693840148746478,0.077235819889711,0.08355888751005569,0.087949127985611,0.08261220287968984,0.08211583220174466,0.08441039635969494,0.07004386660388995,0.08216211374502813,0.08854759214022456,0.06485086642802755,0.08630354843233166,0.055635221711124214,0.07624010740109563,0.08579143259174293,0.0805390187479166,0.09052599481375263,0.07557410670743218,0.08243062755010112,0.06815083589207313,0.09010141410160766,0.07414653671597676,0.08232360346625311,0.08627784620480058,0.07614639645321256,0.07016386112758496,0.09558660110444703,0.07817652294076241,0.08428190001005204,0.0734042054070011,0.07153518739407481,0.0858594270713818,0.07546094471505893,0.06805912726408442,0.07248181981632891,0.06922692523100607,0.059521084656386655,0.07265802156614759,0.06880051883660726,0.08111590426536758,0.0653983369753717,0.07401086375379044,0.07362718233190624,0.05990600469745644,0.09391821264886639,0.08287092273366663,0.07781223128901946,0.06678149762016865,0.08087945262452985,0.06906852046802087,0.07736989210386741,0.060445208316628876,0.08257234475925136,0.08198227537181182,0.06654035145263074,0.08858663060019902,0.06909034162777725,0.08376158696998709,0.08590499746701852,0.06675768339955908,0.07679349671898356,0.07417482181188924,0.08933200882925835,0.08768542760604339,0.0820827465136986,0.05447078036384756,0.06450537699988036,0.06780058127774008,0.09072265368058671,0.0756288142159976,0.07314619513811918,0.07756400634818225,0.07298875775438601,0.0811122166091221,0.06821570423111775,0.07414464325986014,0.07468315508338805,0.07761203180681943,0.07080959957602037,0.08580691646480687,0.06634145242012518,0.0670637888535198,0.0750496936605125,0.06668892318487912,0.07358082079279676,0.08558452496823318,0.07328883231385375,0.0625740064189436,0.08547899124141249,0.0590062205453475,0.06946127226721503,0.0807065364870098,0.08090867221666773,0.07930927160720146,0.07999613507744255,0.07909802392564393,0.07940531836512801,0.07827663681124733,0.06781103351381124,0.07286942757599663,0.08411531546151675,0.07142212289814641,0.09194224858512084,0.07874236261849213,0.08213689776331966,0.06279887170585106,0.0776818099655866,0.07334519752919491,0.08467684556672839,0.0844369249724327,0.0658153580230387,0.06895093161500057,0.07526878904300194,0.0858411917411143,0.0711130587549222,0.06525578414691977,0.08159738950472747,0.07925236630851919,0.07261678311113368,0.0805909659822246,0.08481098892320282,0.08127085915580254,0.06542805257686012,0.07865245603047796,0.08943959185417519,0.0793347039956883,0.08365294352430049,0.08593104063760304,0.07719100570788709,0.08149445783873172,0.08542482820547811,0.07423205163990992,0.08741561487173019,0.07761158998726228,0.07866546245259459,0.07252704400515145,0.06875357176833091,0.07764632584708712,0.059287760764289765,0.08133035993253547,0.08735678220496847,0.06606590350237643,0.07124914964369156,0.0889761541757291,0.08078261981607338,0.06308072037584575,0.07421460632086503,0.06701235621035376,0.06242723447163697,0.07667101900554744,0.06575504101250022,0.08416194354659799,0.0926991642075182,0.0872215385635769,0.07006174041130775,0.08356114906948933,0.07220078800761794,0.07960128024920082,0.08929737250580358,0.07834929402120833,0.0669180723407832,0.06281980099221665,0.09192628853012669,0.08950143541326877,0.08574078076608925,0.07099292156206508,0.06559073632602909,0.07756045594188479,0.08360039517336236,0.07321555799405197,0.08695739419141625,0.08494659605355837,0.057423972421982175,0.06706735067820568,0.08628246212751334,0.0491942026907811,0.06906744330462458,0.07547266691412746,0.08503460225358092,0.0802270969388445,0.08087058716647674,0.08995445414584577,0.07871503864602619,0.08903912016013632,0.07992515110807119,0.0821300148608565,0.07329616503645349,0.08273610084405994,0.07539527596431067,0.060239612086673305,0.08120809863210893,0.07941711797230802,0.06831799058336298,0.07623525269597128,0.08813056681560728,0.08404736191514829,0.08970682469591162,0.08786410523580727,0.06535359749836231,0.08170443055592752,0.07453159274337337,0.08439188636202921,0.08347053324263853,0.06489013151186343,0.09418757778177207,0.07643089139153389,0.07372512422547789,0.07996076742676234,0.07533442818526087,0.0847048856744567,0.08626281104905849,0.08353357899229324,0.05479402582779451,0.08009875781431197,0.06610474187585727,0.08013675471619873,0.06444872709232183,0.0820793012015952,0.07972777501478223,0.08037123822944744,0.076737331447688,0.08288382260628754,0.06913047147608893,0.07302499626643821,0.06764332617486074,0.07112910154418622,0.06558723425741805,0.08607969752981962,0.07924072407298835,0.06447947418635169,0.08252783200697225,0.08352279610182035,0.08111325084395085,0.07918530737996099,0.07513708699537733,0.08343761453366527,0.07607650233342308,0.05662379090074053,0.07051426309674691,0.08125506155341487,0.08545656923978878,0.06940397141237808,0.08352262604821163,0.08062069830454746,0.06717044738510655,0.07671014064214392,0.07915365506379193,0.0684470379466707,0.0918319291449281,0.08750852926505384,0.08773515193848028,0.08461680464864468,0.06438597475311927,0.07098756406966616,0.06584178814025685,0.06371432370480254,0.0792979339504839,0.07710233912452508,0.08553222591560633,0.06837801145077602,0.06295123959742356,0.0851962499681563,0.08209983385664334,0.0818327624936983,0.08685496700449209,0.07626532857544346,0.08418760489292734,0.07670969118115298,0.08464099404926512,0.08880717500288071,0.07576778336703145,0.08167565909238529,0.07918880437794244,0.08564250453518205,0.07398868024572824,0.06841998298993646,0.058467119706234125,0.06391766654472367,0.0870213187194763,0.06937007842919105,0.07374163327202649,0.06814168162038131,0.07572565456502084,0.0665953733917449,0.0829373206057144,0.08389102498417284,0.06424574993256993,0.07506960849632162,0.0863278717708108,0.07664365642807106,0.0878282831514128,0.08219001491848932,0.05916452967219659,0.08953072674077209,0.07183119238274072,0.0697595127698553,0.07990239655973208,0.07254568888875634,0.07394803304765596,0.07696300643093901,0.0884521004005043,0.06967824192269338,0.06344312399551097,0.07721468368461958,0.06708247445294557,0.07658354830476391,0.05732712439632092,0.06586415953591518,0.08017856855828392,0.07591771369146863,0.062259366911602144,0.06186733447642989,0.07477111172182142,0.08810481862076304,0.07098445826850217,0.08124751002923192,0.08731758179888888,0.08100035289173829,0.07147010651323851,0.07927290910475282,0.07638510053269551,0.06023878619379669,0.07080487502751577,0.08141504697914162,0.07502993590662088,0.07599583080375535,0.07415662775961417,0.07469463262821105,0.0689943318399652,0.07475068875900132,0.07877769667707074,0.072433914814211,0.0729619179951994,0.07066879620298977,0.06793607712693103,0.08199244477746696,0.08255426300497234,0.07984043421900935,0.07890322910026222,0.07791995755309446,0.07823648871402736,0.08146629627178929,0.0791289250737316,0.06617508969310022,0.0903468847286564,0.07751611128234265,0.06942260389050084,0.0668000144894614,0.08680296993427696,0.07379471718422251,0.06525812690983779,0.08669967808749682,0.06993402196730904,0.0706110235848371,0.07267259681470153,0.07284650590321344,0.060816313159276464,0.08397078173241182,0.08426047212064387,0.0767882873901593,0.06999416699254093,0.07834263188510315,0.07035367745314558,0.08093606305302786,0.057603656567764616,0.07542817072681968,0.07893461415672164,0.07991662298414298,0.05665311716433656,0.09061585359119523,0.084429446941922,0.09574004991695959,0.08423626420733629,0.08456624532464152,0.08162434413588614,0.076672942153269,0.06871493857031595,0.07098670850767805,0.08413778742463854,0.0722329038493806,0.08093484683283904,0.08859750166594131,0.07104477470280723,0.07391304872269178,0.07385832061634492,0.07248380809515304,0.07730344779632409,0.06485176249580457,0.07438096639779745,0.07735702051499982,0.08194747139255565,0.0844229419917713,0.06315440410365868,0.07517503731028116,0.08814070370173283,0.08277681521768371,0.08040713491148449,0.07930469502500509,0.07134769882509356,0.07283293232897436,0.0814226881734752,0.07811417980185381,0.06796227579669956,0.08177220880140731,0.08463920732742297,0.07089332339418397,0.0682994380774771,0.07527250243394995,0.08400945077308883,0.0787664370598957,0.08670608291650625,0.06503983540745378,0.08672653839040492,0.07908452498841612,0.06836908420890495,0.0782758194837133,0.06518216550207852,0.07899810350500266,0.07650395020523282,0.07504605197924447,0.09343823610580876,0.07319613790729315,0.060440415368283706,0.07552499079434274,0.08434551445042701,0.07681993587691141,0.07408439220594971,0.08054685189779674,0.06379821628667229,0.07774514608591956,0.061999201827220636,0.06975262900395932,0.08212515673869898,0.08390908682258431,0.08231908163491167,0.07330827023466532,0.06265352026089402,0.0771701168720953,0.06904673985378446,0.07434332078703557,0.07913998753812768,0.07575931213579606,0.059442102814354386,0.08547371418229008,0.07645491270464785,0.07551387576405799,0.0706083596772785,0.05736268965886226,0.0949010344555219,0.0793448053659937,0.06864346441077934,0.07166405157434304,0.0674148412191545,0.09404898929540492,0.06645455497415201,0.07322262592558143,0.07398680019036746,0.0714652307150152,0.08105363196183935,0.0740346406687728,0.08325193942533202,0.06785487206109247,0.07169331238785255,0.08170274406709273,0.06839132377376445,0.0803431976836928,0.0815526742294833,0.07951299915914156,0.07392058271765023,0.07954521360346842,0.08119121964947851,0.08304048819639873,0.07793930069674344,0.06436947662348247,0.07508173788166352,0.06606156323297732,0.0631734382067822,0.08079044752200187,0.08760816511031683,0.07573585385103687,0.07673318984717672,0.0842100360473195,0.0758310900694937,0.08954891089819961,0.07584435318271623,0.08698527090410205,0.06315749981959146,0.08733276766311888,0.07041398373971086,0.07466916936562247,0.075146380241291,0.07064078929730336,0.07935625661761855,0.07006102657500443,0.0623672051642555,0.07452384661764098,0.06796474075173708,0.07676936670555767,0.08204539432646854,0.0733985949736447,0.07994797049751518,0.07993045594548567,0.07679945516707133,0.0661812543082223,0.06697047110988064,0.07524723603786532,0.07970149302743873,0.07335244694815228,0.08945828844212722,0.07036791503824028,0.08295554812594812,0.073926895150003,0.0919334933664414,0.0861369313083521,0.07561476584656462,0.0868600864856921,0.08813076185468674,0.06850843976687815,0.07440256354976255,0.08671403515221289,0.07784029165029391,0.08243932364447884,0.09063428679711655,0.08184321605045247,0.08429194646772181,0.06959332496881235,0.08173429857222395,0.07194356649916217,0.0788475790927689,0.08551071050931566,0.0878213672621008,0.06912949057512338,0.07705469467309428,0.08579318840060145,0.07838705846567026,0.07625555322127303,0.07838008573943366,0.07228782248462001,0.07518545709027183,0.07177694064121136,0.07398784776631928,0.08111179938314567,0.07442675270587035,0.06028353582231398,0.08020059202222124,0.06495575540879997,0.06265777814655672,0.08450756125761025,0.06461928799326308,0.07436401643556799,0.07434291249279096,0.07603244314727103,0.08728023706505016,0.08195222065187391,0.07290145683762335,0.08996686572113777,0.05872354238300874,0.07428644206387991,0.07132489737508728,0.072989200905579,0.05897074936053433,0.07995105103364497,0.08141155780949845,0.07860141799311268,0.06760270615233561,0.06039672140737097,0.08898628313116388,0.08325383636342969,0.08925143661508497,0.08545409518046336,0.07780401565957444,0.08522056760013479,0.07407530195164405,0.07879851633538543,0.07199996490738905,0.07390255030717409,0.07349066195066843,0.07418625856878197,0.07522861327347545,0.07833770156842013,0.07827439021348209,0.07012135631522555,0.07714198953259238,0.07426196828796788,0.07831215260024213,0.05558126683193368,0.07665527076000908,0.0683802198007637,0.0836827424986866,0.06693711027216087,0.06779242809144587,0.0712080624483559,0.06887867664121314,0.07558426821293635,0.081687392726632,0.07294728419446089,0.08983843328336351,0.08826187378230373,0.07640333362765232,0.08289538773578674,0.0678971200514954,0.08765751963557072,0.07388418544822262,0.0760110840220374,0.08344872666056413,0.06399792935308056,0.08204323038362604,0.07657121033933509,0.07818635211143285,0.08590458467570475,0.08811536685488006,0.059149704769848564,0.07933812256839426,0.0786770321802708,0.07866123794802196,0.07341815947585226,0.08284647686041871,0.07936378996584177,0.07581819270596149,0.06800492256765209,0.0878612775426505,0.07334820783368551,0.07282880291408411,0.060504112222782305,0.08077033786261867,0.08035663591972307,0.06427945457992973,0.08246138160808683,0.06963225663498501,0.09124549102606469,0.08826767808876101,0.08443103132776564,0.07508492311230996,0.09293659676020233,0.09064036054018806,0.07839640933552869,0.08945401133303021,0.07513632420774081,0.08021119529094395,0.08121132136680641,0.07125037606753692,0.07403559148532687,0.06271334694062833,0.06899307613353185,0.08546007641177655,0.09125136134132969,0.07813329212734754,0.07592699943108006,0.07421947202273788,0.07935183570475067,0.07723023204789686,0.07522263036835655,0.08944774941396237,0.08833857662968952,0.08071434488179635,0.08549483102602992,0.07240136889970655,0.06276327461792901,0.07181838209231262,0.08208483417931917,0.08131233614682007,0.08605837798597647,0.0785435568190892,0.06943432916387317,0.0687207868918764,0.09139778333627968,0.09314017011820698,0.07191990363998609,0.08901625001576578,0.07754571786800947,0.08136798737320802,0.08417679286660314,0.07777454655520316,0.08053931309933218,0.07682369311856246,0.07997497023781205,0.06994618631973121,0.08787885525066375,0.07057501344033962,0.09678968148970164,0.08262628484966075,0.07430972630276353,0.07198375094623152,0.06995645623226913,0.08090118939650104,0.08464340232173101,0.08234149627651258,0.08024269130340494,0.08863418055714717,0.08068665611276768,0.07485225190551828,0.08322398952317811,0.07274731465725282,0.08072214079968228,0.06986051980881418,0.07495896987491707,0.06937326388000738,0.0815063711165497,0.06733346919229516,0.08621715512966444,0.07905535147786762,0.08026122915931318,0.08716334865437875,0.0853805441558278,0.07933212693078762,0.06137084643464631,0.06871638997159099,0.07831840920295897,0.07736341191096656,0.08713606698795548,0.08658413512044247,0.07163760183166243,0.08921022145157609,0.08434432267854489,0.0729640561486337,0.0649864046512395,0.07954629753433239,0.07601027341882831,0.07276407220531139,0.06841796184254081,0.08956702387296259,0.0767406224992331,0.0816083655266746,0.06758215811371045,0.08153803287823773,0.08717459984179324,0.06147497942649839,0.0883481767197563,0.07846463911365817,0.07464451804161482,0.08225023610620269,0.07825024481261432,0.0739605850095921,0.07286460169667636,0.08566433768331969,0.048799553521398836,0.07687463088750787,0.06594279192972277,0.07983060524367369,0.08336537536336695,0.0752420764516264,0.09105671552522672,0.07872145421035895,0.08426531090409438,0.07005381666440376,0.0668422603881692,0.09249287263153504,0.07255441991697109,0.08596494520180367,0.08078048491447015,0.06600825930382871,0.09583137077909962,0.07017837075589922,0.07935568140741886,0.06443693504706086,0.0741322542351625,0.08256963094914832,0.08414891215808441,0.07997935110634984,0.06742211582610733,0.08549651419636511,0.06936486732120206,0.08391203870511052,0.06870009820636994,0.07876997302077768,0.06992266180893178,0.08200019205300757,0.08246170750309145,0.07895051130322336,0.08087231095881178,0.0616014774149501,0.08141281267148198,0.07747675678358065,0.09391237246960935,0.0701037704382247,0.08606925873370036,0.08901272662552451,0.078666021083972,0.08233400774350133,0.07138063186836283,0.06855471776164238,0.07441630500256183,0.07368604307359641,0.06498415860665291,0.07539431942291,0.08138908718499488,0.07562444662066206,0.07331218772685276,0.06381877461105835,0.08479929229462718,0.0776450498312867,0.07366427382977084,0.07278974768823343,0.08680550579593636,0.07406715324796624,0.0731476745917364,0.07734915816655502,0.08201145810036323,0.0675791043646029,0.08978673285142334,0.07903469737338198,0.0624414369177726,0.06452072343583143,0.06884683027621227,0.06808640009313027,0.09180739531725252,0.08219869088684426,0.07726856264495058,0.07612351724007871,0.08144649972465144,0.07138379402274733,0.0862629799346788,0.08685960259984546,0.06129474370742089,0.07473998878737657,0.08035120057797354,0.07028363962314335,0.08077058801756104,0.09004981595134617,0.07333174668011902,0.08358575482022611,0.08573163749094456,0.07577123780419112,0.08597704584570987,0.07944774270168534,0.07052038258937819,0.0723580870655948,0.07890571131047433,0.07849186892605067,0.07552307501447603,0.07615174938875792,0.06886526415320317,0.0787470294904189,0.07443931723081863,0.09176179993967729,0.07384171646590716,0.07777288083961115,0.06916067570626966,0.07668432646070458,0.05522061217670341,0.06694474870102732,0.05862838504503108,0.07357033140418867,0.0744718033411341,0.07914221343010949,0.07791608750642118,0.08186387946862919,0.09205040372407414,0.07963074653751935,0.09036151016792676,0.08653225771936529,0.07468476747843791,0.0887383137150176,0.08390468271536913,0.08664002064266103,0.05504177852662047,0.07147874594627673,0.08487925732458508,0.09013542183135141,0.08121267524765372,0.08017291495935278,0.08909452337102992,0.07787520353957963,0.0730974178854081,0.06972752653623632,0.07477381656054408,0.09439919056524539,0.073931224912678,0.07109830691066624,0.073590829538903,0.07504581449908895,0.08546208567673673,0.07185461925750498,0.06915232262287987,0.06332745933292971,0.08087639326272619,0.08673028724342055,0.07403764996431292,0.09016799364046474,0.07551112755999001,0.06465721164693115,0.08542650501433718,0.08019797650922836,0.06366952027896346,0.08728869735685503,0.08315679615915289,0.07425645185269097,0.08813358333099139,0.08302888828861359,0.08052610749239819,0.07621454325017119,0.08292976564517737,0.08672029572548054,0.07945659479913658,0.07419335534024593,0.0755080090994617,0.08447663943152645,0.07877265562676107,0.07648174654625664,0.08609027479973697,0.06944622891262356,0.09248879651952047,0.06312175750682794,0.06827890326922027,0.07506789853703669,0.07764666975639034,0.07475092836805752,0.08502561165732565,0.08499236120424078,0.07497408647575818,0.07890112196488003,0.07314503597697004,0.07631437635514857,0.07197739460482533,0.0746584735556681,0.0917035096888334,0.07967754990196929,0.06741611570616288,0.07818405900314454,0.08620788429834092,0.06206569341264795,0.06970986254382211,0.08692010737593386,0.08008491062765916,0.05899953257423904,0.07288976512464174,0.07145164992289399,0.06994649029206676,0.08168156158976836,0.056343751264710015,0.07883231506039554,0.07209861071227446,0.07529334757527376,0.06614549959621438,0.08259375809194311,0.0799779779486322,0.06334938307687041,0.06118126927561775,0.07837780210473697,0.07467509015637128,0.06479272389154216,0.06465686347842575,0.05680122022920294,0.06777599699789323,0.08078155537894759,0.06331758191535365,0.07553577742935197,0.08604683518029749,0.0852786375687391,0.0730942376420535,0.07766132561065205,0.08765916837080365,0.06612411145587697,0.07584041180685659,0.06980082644441993,0.0802630355321282,0.08581185229230795,0.06326205533043569,0.07510277035855467,0.08979960298305076,0.061705398779321956,0.07589638672327036,0.0861666340422747,0.0749341277592274,0.08428112839232801,0.0696870544205569,0.08197295717641695,0.06701667511681139,0.07300429248415015,0.08487055173320338,0.07605639223248327,0.08252072283995814,0.07379212506798725,0.07019038325236518,0.0745094922034665,0.06640527832901442,0.06980879914565138,0.06657201708021192,0.0762079826194339,0.08766029217593499,0.07451202860667899,0.09589739033609516,0.07902191100077215,0.07196761452435219,0.08346343193319371,0.07870505780483977,0.07621691441538304,0.08332450944790255,0.08671956918407765,0.07647067808028923,0.08002092100670632,0.07170512437036528,0.07715334089206007,0.07761034131014644,0.07958025109978746,0.08950408004309686,0.0792569958001404,0.08724424374719486,0.08190598411064715,0.07121870774629321,0.0856768591535873,0.08226886902035377,0.0763940438311656,0.0809422622063257,0.07604983706836455,0.07315403335670377,0.08381102759458402,0.07914409883560206,0.08030964676861915,0.08581150403803912,0.07015964206486827,0.07822570606292092,0.08602309337067843,0.08062964908668874,0.0739155652068458,0.08309047744197585,0.06944761374612704,0.08909361713369747,0.07138795188717821,0.07619076016838472,0.0824908616734206,0.07721163246344505,0.08664736183360178,0.05704854797717514,0.08652671229241991,0.08744114190063246,0.08368500308105681,0.07138884216150793,0.06732103444697235,0.06540280147123123,0.07161284258268552,0.07691581956017765,0.07603101267960964,0.07463915211088129,0.07257939396539023,0.07279488529926062,0.06984229534374548,0.08952419432356465,0.08733720712419853,0.08091823081798069,0.08239458463969793,0.08344971307074239,0.07446247181060849,0.07083659080996582,0.0713622416643952,0.07258392805831876,0.0727109851039536,0.054297783482325265,0.08097431688960091,0.06791766087532658,0.07331131462090461,0.07223186819672545,0.07057789457734068,0.07679940958443293,0.05987907534539106,0.08845217646973266,0.09041642125871435,0.06742016889951682,0.06917402793048988,0.07130927669279204,0.08819832199771357,0.07858309449769549,0.07952580338693027,0.08525785202084768,0.06521147980350624,0.08330942296394289,0.07496846081332313,0.08613773535037506,0.08038324610466493,0.07269259712266782,0.08208595502473127,0.07317150006838008,0.0680472778687876,0.08099555372378867,0.06663169521458315,0.0817014270766302,0.07597110415270891,0.08626625629835413,0.08267021274150732,0.08430155936017392,0.07520372622187457,0.07225990106332271,0.072027725606832,0.06523711828342306,0.08571613541000045,0.063338363032933,0.08048154433442628,0.08900322048336495,0.07544129566664223,0.07307741493294964,0.08318759821600441,0.06708028935232127,0.07809774583327134,0.07480776939910991,0.08128044855975773,0.08253510637299587,0.0787862047268383,0.07060360475749519,0.07857783810854303,0.07026349712488117,0.07302163302966841,0.06841650017145418,0.089108728360379,0.08544456300447548,0.08442686412986673,0.0799702748369494,0.07223157280799986,0.08724300671104487,0.07192443586223193,0.0825952511731725,0.06283047033545999,0.06406578475337138,0.09041739023052156,0.06988205318065659,0.07928988386447819,0.08467853926107236,0.08564845320317319,0.08304630897114257,0.0790680985798887,0.07009014313313983,0.07498318788799886,0.06288400201656921,0.08774755775151938,0.07681351872349121,0.07171529466705197,0.08307769442974822,0.08090336531294628,0.07798519201288276,0.08317737532378913,0.08646970072871646,0.07646179400881019,0.07280495578108745,0.06543512625061859,0.07773695440837233,0.06876189713042061,0.07694736338772398,0.06945813215773432,0.0837217315123504,0.07030919446334648,0.07647752750443937,0.07785865585004692,0.08587307101915903,0.07968776163039099,0.08560051697707136,0.08632599306928558,0.08368505052471925,0.08249003321868068,0.07572786721096497,0.08227624547077789,0.07987887296554169,0.06719266685224734,0.07047307965766236,0.07060666466330029,0.07054803124337483,0.0858349313336906,0.08096483056870046,0.07533243616026306,0.09114954863854995,0.07497317031564052,0.06760719561715098,0.0879716788792632,0.07243653590876215,0.069394663651516,0.07927812203199769,0.08033048996742338,0.05583716366481824,0.0717630420023252,0.07934000593334276,0.08568059690394643,0.07991096986874366,0.07299921569084666,0.08084880904711969,0.06856180058229686,0.0648076537946441,0.07430786032303031,0.08601273024987205,0.07462541171419063,0.08137819594482802,0.07685277994168085,0.07362869884865392,0.06633079148986729,0.07661729022912495,0.07689998131198406,0.07116552982708135,0.08256173249393005,0.06889266919690223,0.06963159848358033,0.07300633447861056,0.08059972434692472,0.07918797184681614,0.0748851074730521,0.06988663454897492,0.07707922591242368,0.08718523816981345,0.06720133228417541,0.07381392889022456,0.07750162440379667,0.07313271635959816,0.08463833731401132,0.08108573069764699,0.08449117271427156,0.080753910286893,0.07099807957418508,0.06768812595660248,0.08062177071890289,0.07447851715228641,0.06775636817243375,0.08252785021902573,0.06330970708089495,0.07587730425047291,0.0707038295353963,0.07166256441511608,0.0707845752083818,0.0584000782898315,0.08702258803728141,0.07673267955879683,0.09271613831469418,0.07897076392866004,0.07800831431546229,0.08466226939003813,0.08570053423832304,0.06976121805013841,0.08595081020763765,0.08257164443650823,0.05921156607112969,0.06473224865563822,0.061664192683755364,0.06440506608261562,0.0886892317744358,0.07495263362200577,0.08218707885458543,0.08849735636863637,0.08773370692843412,0.08591097613927373,0.05896873111989222,0.08028660879062087,0.07572285410426961,0.07697048596227357,0.08137675289176176,0.06117378243110529,0.07008123691192356,0.06972247103547233,0.08094213894789024,0.06998633311874362,0.08811663336973402,0.06928510191930859,0.08115000374154743,0.07276613709614146,0.0756666973877222,0.0899429963663902,0.08744263990300624,0.05875571715798422,0.07485351016154798,0.0786779507934472,0.07202032036702397,0.07738279733831827,0.0763640812374711,0.07080460700335461,0.08131576995557016,0.07250715639448206,0.08946883115431843,0.0672601983633976,0.07668392009025214,0.08577039080207212,0.07714413362150108,0.0757619557060458,0.08667018267534574,0.07152116112088427,0.07740980519399818,0.08100968031281987,0.0791827662199022,0.06556773768721241,0.08301230715478602,0.08881593283707728,0.06763052498153682,0.07956690593509913,0.06248458600832002,0.07410843849672646,0.067847120287642,0.0817279488540581,0.08556942642062439,0.09229819475362702,0.06181745398758176,0.08575433445911207,0.08733626083053736,0.07103481206779573,0.08900682881954344,0.08262452233437594,0.07374686303984626,0.08325461265879666,0.06680162768691694,0.08201743040635756,0.0728370684423554,0.08805847067031075,0.083504966407317,0.06345381121231489,0.07228699266017061,0.06789100125795708,0.08608063323741452,0.08536351177399137,0.08529522578755198,0.0808499372512657,0.06610962491393013,0.07172858996347951,0.0878207374519031,0.09187431277834024,0.08758910279055894,0.0700766495412299,0.08223753315499643,0.08113988075390717,0.0757701269223445,0.07705129860369093,0.08177637847869365,0.0898339963971656,0.06306734287262586,0.08155811993056417,0.08679738366489456,0.06464038163788659,0.06039738167316863,0.08467360556457235,0.06859221307491814,0.0681062190406118,0.08365954782692442,0.08694132452598435,0.08933646356220953,0.08368087238334454,0.0771237353199528,0.07534471576650799,0.09103411679360596,0.08201881005860089,0.0853108143095124,0.0798899111155851,0.08705355339797213,0.07618359199384528,0.08106545317978432,0.07623490746862999,0.07587627459299665,0.07217120856592699,0.06765117663161747,0.07306142363336307,0.07577348048219897,0.09059010076485666,0.07523420248075158,0.06828791678128437,0.08037649044844192,0.07158241914256744,0.0745017779775536,0.07759542872791589,0.07949159321324842,0.09442510014206669,0.07288279026584443,0.08434352444430002,0.08977483243175932,0.06593854075063296,0.09247029969486675,0.08675484024806823,0.07779548825967356,0.0858855083795943,0.06869267390241449,0.07718288030779293,0.07744380482683728,0.07112253872332863,0.08093061402800844,0.07589114582348018,0.08045330600830475,0.07711379828317323,0.08307629711668034,0.06710778323537936,0.08057886808001419,0.06589236421790538,0.08158036834225385,0.08859174458345292,0.06538734014650902,0.0823486842301599,0.0693136288271409,0.07180306283539113,0.07621811100776942,0.08710136592720617,0.06010820311034685,0.08505042411693937,0.08776396618515497,0.08616831921686921,0.08322289972331738,0.06418561340611348,0.06978701082871719,0.07973072610234608,0.07584100827464232,0.06263213602623816,0.06047641595353545,0.07756313419145495,0.08625267503667682,0.06604557361935023,0.07141343507753972,0.08989703101355277,0.062107639642426674,0.06604861926518726,0.0741206199251738,0.07725699901282386,0.08429334854235326,0.0732420351272225,0.06734830010333931,0.07477417632256389,0.08451704566930193,0.07835119927739398,0.0902008373952139,0.07557426057122646,0.07973942867164337,0.07139485495420747,0.08063488643790162,0.08278228413644607,0.0788869405060964,0.06546071575921011,0.05808677277172247,0.08156336425440151,0.07686308798993277,0.07375618795544843,0.07118705468768213,0.08250860146069815,0.0694482541242605,0.0757402589268723,0.07962873096097697,0.08668855887547874,0.07924889385592493,0.07583933320109523,0.06981714924351096,0.06303813643784845,0.07546969947047769,0.0707970174652911,0.08125223669858658,0.08118312212230955,0.06326227673607765,0.06971300074422809,0.056874478334134004,0.08528377565236536,0.09122734807085593,0.08751503885753625,0.07192826529772513,0.08131523932467188,0.0543688137911256,0.07782866858591958,0.07187191455084643,0.06344382077663721,0.07338482341937588,0.06362490237329946,0.07009292511009538,0.08136202265647122,0.08390945796677547,0.08850418976948728,0.08241953366096563,0.071527466981861,0.08036612374171634,0.08670793670173321,0.0782075183695027,0.06878729458810641,0.07895029706475834,0.06525044055939877,0.06788181557258036,0.0720386691851644,0.0721204311078273,0.0749599313758571,0.07925966915198261,0.08621989715364128,0.07589304665364599,0.06770446693607518,0.0633026293556022,0.07437572227621887,0.08211707762100792,0.08857070120949931,0.07747121405497744,0.06612794537156945,0.08439102946056529,0.0790375862962579,0.07778376112812674,0.08482613609445845,0.08019611835961213,0.07323900187532603,0.08487192686434386,0.0666217169452356,0.08422058069461134,0.09225647150993663,0.09187429373684716,0.08623174319082207,0.07807495824551453,0.08862973679487474,0.0772654466410809,0.07754841560152254,0.07444889872687936,0.07733983172410062,0.08181268852659332,0.07515379069894429,0.057468731724204594,0.08275399422461585,0.06680597510222261,0.07255032450177351,0.08843235684332329,0.07312953822045598,0.08015482225298468,0.08433496131720863,0.08314734514596497,0.05566223985868901,0.08275928234829469,0.07263988888144346,0.07449376487548799,0.07424624547703,0.06723314336780685,0.05873274200921679,0.07596220652343043,0.06835468643754604,0.0736807120557519,0.07721374984736029,0.06928757815982545,0.078501115289073,0.0827758816989093,0.08010167497670453,0.06325359605065495,0.08115245653824879,0.08109166904330382,0.08187924183991967,0.08037404747452719,0.08044039914263425,0.07828133986307162,0.07213441298640574,0.07442146272648797,0.075434596440437,0.07831502808483108,0.064602037225191,0.0689179266770866,0.07192327599422735,0.070957275543722,0.07512762048753696,0.08434290519123372,0.07866566006572409,0.08598983755159506,0.07550061792275639,0.08048574334684525,0.08797025527753573,0.08493636502183877,0.075489627186305,0.07720635814228373,0.06813815758851743,0.08095980084955202,0.0836082137346238,0.07257560402217064,0.0712858505271562,0.0755915829046687,0.07666009939876273,0.07013081802466728,0.08474154301866126,0.06991942090005399,0.07579991938487513,0.07519982636213085,0.07885966814662446,0.08550491150189554,0.08818728247400334,0.06514024726550437,0.07318028734113609,0.07835868616685189,0.08029891900471876,0.07758962785760234,0.0799918700090615,0.08241415493963931,0.07567958935800784,0.07826966987797104,0.07255284632889299,0.0712388808971509,0.07619179077851404,0.08031489322607743,0.08337201870995353,0.06894476973239153,0.07156165882384498,0.08444937864558782,0.09414332892093552,0.0855067166776734,0.0672711608077006,0.06924885909496128,0.07973127740916912,0.08688770074033145,0.06375589963685419,0.0705051788368597,0.07286730451390866,0.08975391109904882,0.062338667248141015,0.06756752310847317,0.07883456906233735,0.07998551259084924,0.0759076443254044,0.0738992906182549,0.08888845299921216,0.08399999566043714,0.061235691899547175,0.07780420710452966,0.08887077489654407,0.07843627996157895,0.08858833830322589,0.06848011726189679,0.08046443331530397,0.08096206801753628,0.06912326133719217,0.0687437352788628,0.05972756265494362,0.07573693894730268,0.08453464034719253,0.08547397646662266,0.07439840252408095,0.07941596648031521,0.08582924668292968,0.08818833754703234,0.08007299886205888,0.07458194947327651,0.07267047031664331,0.08592018379778779,0.07399318806770724,0.08565276115316343,0.08128586121164036,0.07011578258606378,0.08522433022207429,0.08335242143310853,0.07636505145866779,0.08211626806200223,0.0792910060868047,0.07451912337958055,0.08248832284735716,0.07490051765706329,0.06933047732689576,0.06290582849517609,0.08183105969408647,0.06731766283206866,0.08086547978371741,0.08400209799368755,0.07283870454394106,0.0829422520952176,0.07525098247559067,0.08585293514143937,0.08452468472149693,0.07910889466425909,0.07163240596536899,0.07427985079327637,0.07887636704135009,0.08948512806166242,0.06523715783218406,0.06336246098375889,0.07920148016936596,0.07008503075126762,0.06970753692429196,0.07766944904892621,0.07313681286389342,0.07509918687561173,0.08316554765652825,0.0771711304101356,0.053054383883376785,0.07883844401429385,0.0681717365426869,0.07962821423580826,0.07602078962615683,0.0827823033482179,0.08111572702766634,0.08804801613556085,0.08678001652128131,0.08555927320877127,0.07150626090040979,0.08497822710972128,0.07656594381420563,0.07108473494493069,0.07623135222943829,0.07856527632993654,0.06693593191109865,0.07534568973038726,0.06955050905442128,0.06292656944758963,0.07031095480975508,0.07708166220115242,0.07396533225703746,0.08529651269412242,0.0741132938557916,0.07458907768126656,0.07102519634748974,0.06678573072382289,0.06740510136775092,0.09259498491567413,0.07257296489631737,0.08672886280927788,0.08535708407912423,0.0801074897772526,0.0807664669485717,0.07903240184049169,0.06674509655070941,0.0829387434718671,0.07782238670820385,0.08168992020451252,0.0816573046252752,0.07957705924875635,0.07764437579517439,0.08297208948892568,0.08340892956457943,0.08395606806306499,0.08374804323528962,0.07509350682646626,0.07744685236265225,0.07904271886460353,0.05124336262760461,0.06527101500087244,0.06123916603008009,0.07192776955273034,0.08967263977010617,0.07208118751946174,0.07159070079397864,0.07033364084105524,0.07318500032106645,0.08822988496308354,0.06096940144042848,0.07238214938957252,0.08305158843350141,0.06734060373396791,0.07375928091221289,0.08753182626365216,0.07766297711033424,0.09396329508315507,0.07932614299647897,0.06678068453983746,0.08219647973353977,0.06513505610157394,0.06906288884753475,0.08528126508768288,0.08363807292518875,0.07455417855046376,0.08690725451923992,0.08528812739268296,0.06547016498116687,0.08437143438178013,0.09207679546282901,0.06234531115612176,0.07823104260797328,0.0826500715909311,0.07540219154898617,0.07035025135581767,0.09008663354548177,0.07883824292438336,0.0789818394230336,0.07968043758255869,0.07275248204584273,0.08409998613614995,0.08487048356012752,0.09063323682155089,0.08271409565879409,0.06989332086102701,0.07043962876739565,0.08803212248517113,0.0916755412162553,0.08354117384810031,0.08313337634813704,0.06664236170604444,0.06827598500101573,0.07053203448076203,0.06672258040924642,0.0704173372073931,0.08222715348804947,0.08627051891867316,0.06816424291846619,0.08453709446370603,0.07424571246926392,0.0847701851794182,0.0802991410391508,0.08881412497041949,0.08702867454056382,0.08190157129611027,0.07944223609667903,0.08739371012490943,0.08098277882493851,0.06551613448662061,0.07018307477790729,0.07474314195367782,0.07619757728773802,0.07746061098571727,0.07783712454316763,0.07401659757162891,0.08862767496772905,0.08250619296103406,0.05899163164259766,0.0797337769085588,0.07260404636462958,0.07512645774008156,0.08957787023339786,0.08615870679939211,0.08488771176965508,0.08598251014803394,0.06237513804031987,0.06888104740992232,0.07573090039577728,0.07188803612348721,0.07323027170755289,0.07157339551438292,0.07840857279623312,0.07183889501076017,0.09210028404781986,0.08327369111401484,0.08234669631401918,0.08131447550060913,0.07934508668690157,0.06510501404669669,0.0811866619977123,0.07662528490932122,0.07572395768782914,0.06653808328674007,0.06512968389467733,0.0905890288757454,0.08305649425676026,0.06772702613161141,0.06728998689651428,0.08406455946321247,0.08469126252874587,0.06563524320227085,0.07728120099598096,0.07511684343338579,0.07904491946975695,0.0814756892296907,0.06611894273179011,0.07983556076236056,0.08063074197500819,0.06848456526646901,0.07813605812474383,0.0833631883344805,0.07529676222981854,0.07531010063570653,0.08774069101727808,0.07323255706171987,0.07363358431834008,0.07829999678071936,0.08955575145302314,0.06399914972874443,0.07781613237380941,0.07703035587961361,0.07765876158523646,0.06464480240513922,0.07388186192039198,0.07647133396868966,0.07699694693120132,0.08119394165893751,0.07853643288581008,0.08140773767840717,0.08007643309589066,0.062244248981666694,0.056934894022482974,0.08339282272449343,0.08430165217684388,0.07486348950671273,0.06794828984546095,0.09369099042856516,0.06925765152239785,0.08220372665670198,0.08467350861196316,0.07524971055759153,0.07817533036119828,0.07280217711972327,0.07244168762511,0.07194991300237721,0.06618888540502683,0.08810385120838096,0.08042902792293688,0.05773243660610752,0.07048660450146074,0.0740447526587911,0.08067960241328946,0.08509961503352577,0.07436427341086552,0.08326457470896026,0.08726242607190245,0.06682743192522582,0.0656356908989083,0.0814496569984397,0.07858567407039849,0.08069309474948508,0.07598347396687749,0.08381589768804626,0.058439901440636,0.07473351667045039,0.07089554610918691,0.083603631324697,0.08812356590188009,0.06008478398568067,0.0853335775691573,0.07551007507213692,0.08311690143103959,0.08146030751035022,0.07318025190399736,0.0889443867143356,0.0705698237968794,0.08433180672491752,0.07734065537942247,0.08439867727212283,0.08370115148996074,0.064468820427379,0.06722009296946524,0.07966749092500952,0.06567119358115273,0.06516783103703312,0.07817110696333307,0.07732907035051123,0.06377677000381557,0.062005520380309806,0.08160810794154533,0.06896647376184592,0.07193691850395119,0.08742555016353146,0.07449752397602398,0.07138000413858572,0.06731756185469438,0.07041725768938091,0.06559234343053004,0.0912590876821511,0.0833104792848875,0.08339586695387473,0.09050783023041095,0.06826395525090274,0.057654728977349995,0.0811314977732301,0.09256941884516032,0.07583587652697792,0.07501266201327186,0.08520363525927846,0.0823070176980201,0.08907150843620132,0.0721709158333768,0.06854902044810533,0.07247874727992912,0.07069515355124867,0.08000220341750436,0.07399125164209272,0.0847894239769307,0.07704290115175383,0.06887694722527479,0.07766374191422322,0.08604515046969403,0.07167172811019956,0.0871915131805881,0.0744399364881928,0.0727192437279992,0.06526065137592733,0.0737006016915363,0.08363324607532599,0.07871168802262136,0.07278201398849737,0.0854897415637855,0.08379130975148864,0.083532654619395,0.07591779328427736,0.06932582306871889,0.07785400792015806,0.07417933039160665,0.06616156352166938,0.07933929393290204,0.07527648987364181,0.06865027891591877,0.06941782568081523,0.07300812481929231,0.08773140259726575,0.07549058693466043,0.07766879649666983,0.07267493092890394,0.06333022502825608,0.07750015979061106,0.060309200922495726,0.07959758354883137,0.09152804527495831,0.08519723792645692,0.0820532478111072,0.08541580673490151,0.06538498757417041,0.07893373646510876,0.07284782769195278,0.08630835891030161,0.08688137990535356,0.06703971176698989,0.08072983656965742,0.07209820222198593,0.07858564241437821,0.08507222082508578,0.08860008962587546,0.07187218451185495,0.07688744237897516,0.07518467187306513,0.07385133018783788,0.06637577165710445,0.0799061430203351,0.06878849936219913,0.07841083722265627,0.08459340174611536,0.08386612022353432,0.08965647958638004,0.07366451622543818,0.07418839016257074,0.07340591540685433,0.0806254879786362,0.0741486515408252,0.07355980648039144,0.06981279472376231,0.07434900479328178,0.09556615466342532,0.08287105538733563,0.07941059731264327,0.0913137552914538,0.08168484891552019,0.08030186806342307,0.06966335990962996,0.09006601733826047,0.07688369460794722,0.06666637692137214,0.0778278881731174,0.07249470829002416,0.0711842553581501,0.0841902285779911,0.07152110861880939,0.0839851115982998,0.08337646843394572,0.07005269850417364,0.07930964361581924,0.08205329665109953,0.07052555413252395,0.08053705452809351,0.08011517391792686,0.0734197894935672,0.08269504066247103,0.0818133597606732,0.068876180029474,0.06906796437188281,0.0801560704164779,0.07658012493350547,0.08822504498151433,0.07473132868394994,0.08843577451408524,0.08193534391655434,0.058329682456976395,0.08049051473599916,0.08217311182894359,0.08615020924119023,0.09125806417972085,0.07875662744147405,0.06893994921611885,0.06555411598639321,0.06238944060436802,0.06761548427087824,0.06674608179781619,0.08265844664166583,0.08303803292916315,0.0896348880991493,0.07957034261201774,0.07738159403610481,0.07322220195157234,0.07315178393319305,0.08578431850834636,0.08057906921825947,0.08806821933536062,0.08604530520354053,0.07133605794004919,0.06672008256737884,0.09191762365175016,0.06815383504444841,0.08318050104379178,0.07361893368795758,0.07663978792298413,0.08349444885861781,0.09051706075707552,0.06930468532188716,0.08469656549462201,0.06763575333116703,0.07087267553510714,0.06280336998823316,0.08326479588111654,0.07168231811681006,0.08480725213965487,0.07560437808933337,0.08315022669799291,0.05845512375811264,0.08361879746824491,0.08312324704291102,0.07373972656366111,0.06212713885578624,0.08361359065481974,0.06869104065586662,0.08814569710174466,0.06688549375506861,0.07911660631844192,0.0800382755368223,0.07729238295498278,0.08401526682560771,0.07758858231959742,0.07582598150731193,0.06739004398429571,0.076970854277388,0.07013132873587463,0.07679936684582034,0.08995283625352737,0.08854348815330315,0.08083283023747072,0.06624147899819499,0.0820722238369617,0.07973789850169984,0.06450826461722124,0.07103724006642514,0.08560444165526798,0.0846152998783942,0.07351972943583769,0.07031033649839749,0.07782656687074632,0.07168123793145709,0.07571962485913866,0.08589632566084589,0.07878077222633598,0.09418593302394865,0.07804887838664329,0.08666811047859158,0.07939269230773632,0.06817484632142569,0.08391285138073375,0.058276079909906034,0.06425647508789456,0.08840681856677013,0.07051582718771074,0.07186176301754632,0.08170492660780296,0.07174346266787093,0.08469595580246211,0.07378557694779446,0.08143808142194155,0.08655244545265774,0.07570772079547167,0.07932048010188633,0.0708015933411313,0.07404764105328979,0.0682851987911506,0.05924843562665223,0.05744913250239517,0.08399350301055336,0.08250423040443866,0.06988538393823998,0.08073885443831419,0.0859217914911325,0.07764407936892351,0.07714253789758257,0.07251014781818789,0.07563775445746163,0.08382282931496196,0.0672786854919073,0.0644426920324132,0.0836298236802297,0.07852737954232267,0.07488959847830648,0.07563746623162418,0.0818637417400683,0.0719511128387876,0.07978739789350886,0.0851806518630444,0.07871988653597566,0.0881769786314824,0.08934920148361156,0.08652363559959506,0.07492358840954727,0.07349416383069635,0.07202367468653528,0.09204638316366894,0.06824573750136571,0.08384951487426567,0.07051688994697165,0.06975333617839652,0.07112294704121386,0.08657840408942387,0.07827414238311138,0.07236378543498584,0.08294132884347807,0.07115247255069643,0.08533438619947384,0.08535433485567928,0.07376776213051289,0.07308649407768586,0.07687290439205,0.0619432532129908,0.08473129848501743,0.09170498476221092,0.0766673739050358,0.0802842247123094,0.0783421656494263,0.07794183290121583,0.07818781251602881,0.07881296503140936,0.07186941117148482,0.07727541511425251,0.07879460725660686,0.08636661212047919,0.07814908426032416,0.07322575718739774,0.0840861760385112,0.06923586899255015,0.07271292849031219,0.07999964915658025,0.07388913223834268,0.0725454145112844,0.08507554824173566,0.06016579157214558,0.07491032883610563,0.0661957177275308,0.06519599242040056,0.08835345020191784,0.06088973089675774,0.07414226787233352,0.0806827152554867,0.08375995973926342,0.0696818844587987,0.07294299016048827,0.08089633293505401,0.0660615925103335,0.08368724617736396,0.08034227457964825,0.0857195141028359,0.07957438041808655,0.09125106109001714,0.07375825180030682,0.0804031427215163,0.08532757725208101,0.0836310245986566,0.07831964923630005,0.08356084404313083,0.06854725091880849,0.07383136589981748,0.08041713340464182,0.07743411483702278,0.06966963945211316,0.07195989699063529,0.06978285228047737,0.0853653392971645,0.06355118189350531,0.06958104134087352,0.07165506190136042,0.06223624718676204,0.06694982788509661,0.08757919161533624,0.08253588723544773,0.08437681809424052,0.0894392855288086,0.08566803213953786,0.07903253530364988,0.08978225922543571,0.07760222488406843,0.08594093736254782,0.0765566340528135,0.07122138624600641,0.09269906537947466,0.08501700807697796,0.07949969770166126,0.08553263819214144,0.07117520162492567,0.07119362293308645,0.07522160336854872,0.0763379786329361,0.07295383053550215,0.07646839582380162,0.07105815919627012,0.07817497403162693,0.08320690582155595,0.07657791567282383,0.07312323532821575,0.0799856987380063,0.06919357641609272,0.0839928504679896,0.06604330561895469,0.08741690555043903,0.09047303638476875,0.07516927404381676,0.08146713586528877,0.06279435178051385,0.08540544041378043,0.08322154839249143,0.09353353803270374,0.07231312303102855,0.06581010827647911,0.09313960328216836,0.07613606081334168,0.07412261997942424,0.06428975176817975,0.0872821322400365,0.076150883932116,0.06804571317046475,0.08346143430998595,0.0695136505506075,0.0809632935242254,0.06529693904289251,0.06996163384120993,0.08191367240330447,0.0884024642415921,0.06992103701717557,0.06548820826578008,0.07978004174315087,0.0696663036192611,0.0886283516481382,0.06746875020384194,0.0710188583235094,0.08077890158913975,0.07099145114571509,0.07767308029208536,0.08458063774217221,0.0620994429787136,0.07631069289123255,0.07695015878287581,0.0875958935395719,0.08082349642670092,0.07546128325860535,0.09066898407804568,0.06989893505356085,0.09190935502783859,0.07496063524658902,0.08699742864285878,0.0795600130279263,0.08135270562154381,0.08167414945769345,0.07135757051030107,0.07679127098595039,0.07689763756104046,0.09187919277736388,0.0633982298188275,0.07865446940411015,0.09105747590492408,0.08502350906200329,0.08101668787251817,0.07945718756612773,0.08803779208229376,0.06339654954058131,0.07277818395869892,0.08185936137628402,0.07695302080777841,0.07589757942630546,0.07832872466546614,0.07936512688482905,0.08772640124034091,0.08475083305605588,0.07273169605210925,0.072825335903757,0.08802611303384957,0.08753360013242625,0.07551323782239244,0.07954477038016305,0.08489127625792485,0.07582202176155657,0.06840436501602085,0.08232767732655313,0.07935968773880242,0.08328799549538562,0.08652557705664923,0.07800499220577035,0.06892815265949825,0.08038903514938645,0.07895447919959783,0.07635775504632925,0.07790629967754845,0.07647055933315763,0.06993539209326398,0.064538034401582,0.07464376198112846,0.07680151745293377,0.07494260360429886,0.0725912341079726,0.07909810871936879,0.06666727151860694,0.07312847584473874,0.08526696924351788,0.08093469884725413,0.0796514900365593,0.0861496646287839,0.08044590980493495,0.06318021157625087,0.08173316314899577,0.07681581098694332,0.08211033914088578,0.08684203984971105,0.08285046366900026,0.063869543644587,0.07848728980922315,0.06797555280193177,0.07209932542695657,0.08058747653524263,0.08702459178191024,0.0936656123200779,0.07299061791389587,0.07849336676819385,0.09476057710885336,0.05953771803170649,0.08497818490616621,0.08442857038785015,0.07377927934389146,0.07214174100647877,0.0947752792026987,0.07887992645112887,0.08403934608863989,0.08285294420147768,0.08149750914905673,0.07161506218866583,0.07386988886972548,0.0671872467147463,0.07278587909861745,0.08413051366061419,0.05605932070704367,0.08349005968630178,0.07668949989535691,0.08319620158814883,0.08161636046504057,0.0742652674945821,0.06960918285609682,0.09347723921107466,0.07865347216247745,0.07007064118653486,0.08039184414390982,0.08798997801043354,0.07110687464301804,0.07468472109077232,0.07528614580261644,0.054698638415287895,0.07904955971697611,0.08057121854952998,0.08521566514211851,0.06246061664270385,0.09058904530438533,0.07205845798370117,0.07530920678472594,0.07938848003828802,0.06858229630633222,0.07422896140689869,0.07406359283757694,0.06413077964932945,0.09016795855908667,0.07786104982467161,0.08143142248311515,0.0767307916996762,0.09127645538530768,0.08230413239866685,0.07172378889825523,0.06703145727156427,0.07707372037043775,0.06895690473586696,0.07849029889173428,0.07716464394757694,0.07119436826175563,0.059042929250903195,0.08400643731593094,0.08345806852833179,0.06673083436483682,0.07802020710367505,0.06663838978472568,0.08291931965177318,0.06599656265503287,0.07322386030585645,0.07129829090671266,0.09422140675803149,0.07723285296522327,0.07196996265339793,0.06944951059218596,0.080092745071563,0.07349893433511717,0.06203715288646641,0.0807164494364302,0.06983825608042199,0.07512009969981379,0.08370465783611686,0.05893785907393676,0.07668820202207483,0.06677622430868399,0.08200441034954743,0.08248675546080886,0.08346411728806828,0.07690120525416225,0.06845884887361389,0.06315677215447899,0.06882469369138186,0.0650821369874075,0.06688244324665629,0.09589733370423774,0.08611747834554973,0.08549521402311541,0.0587784192557028,0.08970095039251455,0.06614961932751054,0.07890875086394693,0.06657596617925826,0.07208513952434809,0.06382991986854965,0.06855404884007524,0.0738181469582084,0.06241435803291752,0.07239501009411381,0.08191679969617977,0.06548683801306376,0.08846123584150228,0.07025827327771517,0.08690476402319351,0.08772420794290968],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Investment Rate"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [18]:
simulated_home_growth_rates = np.random.beta(10, 5, n) * (0.08 - 0.03) + 0.03
render_plotly_html(px.histogram(simulated_home_growth_rates, title="Home Growth Rate"))

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="30f8b25c-2528-4e71-9c71-348f60ed5a92" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("30f8b25c-2528-4e71-9c71-348f60ed5a92")) {                    Plotly.newPlot(                        "30f8b25c-2528-4e71-9c71-348f60ed5a92",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[0.06226575344330246,0.068610633554376,0.06428007804853741,0.06502956903165086,0.06468703605801152,0.061794295426729526,0.059904790164949914,0.06738979034650977,0.06005552759864709,0.058631452324100886,0.058139960136885924,0.05592674847502666,0.06777607118347106,0.054376838296055,0.05947186464430998,0.06478332647975535,0.058452316176508326,0.061653601158219586,0.06954270277397848,0.06718828578357137,0.05531006375894107,0.067566653011137,0.06173285214093212,0.06855848105046979,0.07047165759647603,0.06421487701654599,0.053142508219438625,0.06141832460872127,0.06662595924647444,0.06037247224384208,0.06164973031695125,0.06008745124735874,0.05804433944860111,0.0641764254813825,0.06484257544339378,0.06207678805383597,0.05153410086670157,0.06031481388368376,0.06266687076381669,0.06951453793481355,0.06858278995666775,0.054042444135742385,0.07192446991083504,0.07049682791770431,0.06854663727718689,0.07071454975855121,0.06532336166895902,0.058927600648574566,0.07567130387374203,0.06916708961383314,0.05324468340021713,0.0686623294740172,0.06103627487577257,0.059069012924803155,0.05553158188665003,0.0696560173808512,0.06103752537955215,0.05283855458372814,0.06029249704227503,0.0658955642096149,0.059655168104692666,0.058605026740733035,0.06426433653388186,0.07027799865441506,0.05855375015947068,0.07046059535630896,0.050098144661731264,0.06763055582981228,0.06743708325427025,0.061552087106182624,0.06534421439930742,0.06393520520890814,0.059067406791808004,0.06519524155040812,0.06259002289099905,0.06498538681235605,0.05589460901287854,0.06639393667173572,0.06313117651839889,0.0469021478010307,0.07332845427817056,0.06913685012646323,0.06313901263311317,0.06382100432978799,0.06998859598639244,0.06156292704455945,0.059846076384394514,0.05956718539398216,0.06507340195087896,0.06616269833968733,0.06014090739937342,0.06266405891142474,0.07354370391410478,0.053555257037913334,0.06150178422111387,0.07129539639781282,0.05386798292341943,0.05795014063726282,0.06526281028505197,0.06159262964846759,0.07164852954202686,0.05959873136891237,0.058162724970843155,0.07362631191337865,0.05749593106413654,0.06864640407303413,0.0550391165421206,0.06718836515520982,0.06510398688373983,0.06566248732699134,0.05520883015827768,0.06402364587254955,0.061318814242628,0.05473303421501323,0.06145468920212484,0.061870221463764224,0.06383458675761428,0.05875382608691167,0.06354751873328532,0.06438147462752555,0.06273414589611678,0.07562599118223062,0.06711776508939564,0.06719248594016553,0.0681409111811299,0.06937181445258828,0.07480722809898072,0.05944828388293297,0.07661727418378295,0.062101005562844286,0.0674245644388437,0.06722948577023559,0.07119910478272541,0.05865210472222666,0.06721779232579633,0.05628500890564804,0.06551045196495062,0.06607768386228649,0.07188376892988976,0.07284482889612279,0.06592181020142185,0.06809531609553462,0.06748820283555687,0.06246814377017557,0.05791612823705333,0.05826447538658766,0.07224073791301247,0.07041369194766063,0.06718531048330728,0.059611853002695114,0.07365044375550173,0.061895144264257726,0.053721478259525286,0.055660032199180226,0.05392187117746878,0.05732831703276765,0.0620906445139735,0.06227479492187059,0.07014052459604636,0.07209477616314847,0.06686139138839561,0.07305786822326846,0.06460429760273088,0.04444250515926203,0.06399162664910171,0.06680508330858143,0.05168406300524962,0.06665657834246513,0.05731620481152994,0.058077115804842164,0.06954594266543013,0.06269915408381507,0.062321582323136254,0.056971355610636105,0.07033964387681452,0.05665844709961005,0.053585593476236704,0.06443991658336545,0.06664874422468009,0.05218928964328588,0.057276592454843234,0.06149422961658279,0.05550768064286771,0.055115930385284964,0.0761860952948257,0.055444843463054616,0.0566966313583888,0.0566552067151813,0.07040077430472533,0.06659911474793309,0.06441667133337894,0.07228328331197559,0.07333789479991823,0.06286897309491024,0.06160915126501531,0.06371439583299758,0.07300417367123788,0.06802241422061674,0.06340918433460649,0.0587365297830572,0.06938505610538603,0.06702147199823724,0.07254080073202157,0.065764159768871,0.0687456437601941,0.05701340886021464,0.07474844473568909,0.06823401563478335,0.06248290193966596,0.07274443932692345,0.06143248322554452,0.06947875835143008,0.0639736905244496,0.06483486543092562,0.06888557122487811,0.06271607891744937,0.05604326128649205,0.05729733162460528,0.06706826227325853,0.07298110317023394,0.07079386129296389,0.060524655284876924,0.0643527447342374,0.06695308757722812,0.06525327077885378,0.0646660899531662,0.06318682884829188,0.06486049150563516,0.07168293492780983,0.058804719325389776,0.06253431081495012,0.06007830399254328,0.06080645004747226,0.06010244624804159,0.06683801106167772,0.07166291265977096,0.05351198146984723,0.06497759280811921,0.06553274707332446,0.06531453632097446,0.06250159054089913,0.06799205886361426,0.05906177289658629,0.06547391544018388,0.0635696949961948,0.06730765485608406,0.05863431344063916,0.05878210593010817,0.05732373469571975,0.07046212655235826,0.06531449864046716,0.06835411170982722,0.05852898792605181,0.0628312290355669,0.06444591728085558,0.06603695946062929,0.06878700080102032,0.06254930396840577,0.06250102483342013,0.0702823777741072,0.05605000605589232,0.06486466414237584,0.060468908276640435,0.05916577511338343,0.0655075919315907,0.06305269197688634,0.05516430372058649,0.06378278833111492,0.05821663570767675,0.06951875234879397,0.059317431239147934,0.05639269916459958,0.0662250838582765,0.0702098404155165,0.06828455090573665,0.0509758206834711,0.06312444939345097,0.06568391601208926,0.061870584333223726,0.0693086210228693,0.05964287821168156,0.055106212313644695,0.06202756869957795,0.0632685926964097,0.0699029596264096,0.06261595744353915,0.06090937996193782,0.06221567727512336,0.06732332930963933,0.07004156978106518,0.06946704665412894,0.06776405806684657,0.06308696110086046,0.05832912263146287,0.06264895540977587,0.05629720672239122,0.0692703487494083,0.061705723000874076,0.05298903386214587,0.07014320109745667,0.06607994226683936,0.062120042830643346,0.06584497023937674,0.06035260663489128,0.0670687312799938,0.0691079732493717,0.07226340168103895,0.06791537623751545,0.07147608426949656,0.06754811668431866,0.07233291705160852,0.06507818505831453,0.06083493156419742,0.07120036712174815,0.0630933314560104,0.05254349204869503,0.0635632963140178,0.061685973028970865,0.06563245738098095,0.059380091811325295,0.05547076724268746,0.06400839305625915,0.06626766180045965,0.0629978973034844,0.05644281668716434,0.0609325622916322,0.06481505250338282,0.06459267631522944,0.06404052336066332,0.059819074681951245,0.052229731586284446,0.06405523744494548,0.06006903531518264,0.06803508623063566,0.06118285481529898,0.06627192986700775,0.0663314900152094,0.06138556192416609,0.06012883137964277,0.06434169002011428,0.06890383795354249,0.06394448491446605,0.06019651903246401,0.06552748893850206,0.05631706525000334,0.06116022069111812,0.0609640411169778,0.0622314274765166,0.06531098298979167,0.05908442164553045,0.07519940991999993,0.05836492388445872,0.06507249880634998,0.06831617932214278,0.05544985341078241,0.04915198943214015,0.06525350080036015,0.06836645861547515,0.06519528702608282,0.07277539577189959,0.06423821668551603,0.07170948705374389,0.0671533704482373,0.07126118943555554,0.06579515118698986,0.059647743404944595,0.06843500509672983,0.06770137037762101,0.06838032787377715,0.06490664897973242,0.06259239256557617,0.06471218499789808,0.06939559816291596,0.06877684389306447,0.0641325649602493,0.05270427823401713,0.07368368950188417,0.05973680221607023,0.054932010086161664,0.05116952043052898,0.05990179997669441,0.06666387985916508,0.050848903916518,0.06689906211168453,0.06292590505028114,0.07208948590158876,0.06639645981266001,0.06152448215734021,0.06105782443766768,0.06517231649405612,0.05330450119776734,0.060538462918467595,0.06725276507826397,0.0578187281538356,0.0668886506661755,0.050235292048033064,0.058253413217604244,0.06625849914247725,0.05812516847414288,0.06478060436530916,0.0629334477144533,0.07295066326769963,0.05976281626082333,0.06171880266464551,0.0611071602798049,0.07088366935946346,0.06208189209918404,0.06450117407144335,0.06132341996128306,0.049174997422587866,0.07723993813977056,0.058970473905348514,0.06302826691289408,0.061366509483451716,0.0706849781131409,0.06402336103950915,0.06253220871943865,0.05084110275717456,0.06069698631429955,0.06753707652619723,0.0658776999968298,0.05909005946479971,0.05872615001450772,0.05582625897354278,0.054375421885691264,0.06901012342203554,0.07197063729691877,0.05829394259312684,0.06434049823281632,0.06935022611702538,0.07096831144883656,0.05979899804790604,0.07174547341118657,0.05825839701026056,0.06322689634338109,0.05570489927234974,0.07047104632247538,0.06307468052992521,0.059471075480257554,0.058194473460580376,0.07072083004607904,0.07359454450358169,0.06784710880349634,0.07077853826205957,0.07213003444050366,0.051182752468349746,0.06656023406084341,0.053816943751433075,0.06696571067048926,0.0583452895816283,0.05449976289356959,0.0512157405701284,0.059766089646218074,0.06304296125023484,0.06637585315933645,0.06377634186468732,0.05851118124037972,0.06615879073217751,0.06101944823895576,0.06548540857647808,0.06776162750036974,0.06612812373051141,0.0692510648217236,0.06495981707214138,0.06969198010385658,0.05057797759285953,0.06435294554828304,0.06669571433726681,0.052985401135381874,0.07229114544980131,0.0651773097192179,0.05688148504002252,0.05438435270400539,0.05745855768102165,0.05923819131231303,0.06303936403130547,0.05666958367702943,0.0624466873026851,0.07219481128470373,0.06868433744208985,0.06536235771767732,0.062211460675490146,0.051575845453404795,0.06498597957469915,0.060286912154528735,0.06933583189919479,0.06446353439358919,0.06324963182288554,0.06562786407905674,0.06395883411629896,0.06156216858015357,0.053157308385621375,0.0652779009810782,0.06119989432220633,0.062206502205099184,0.06624720970302031,0.06062549224676189,0.060436824221274874,0.06417564617590818,0.07350215363855794,0.06765420899569435,0.06527429666587423,0.06851360087039071,0.06765930309947071,0.06743993559319378,0.0626109649066598,0.05760655844500996,0.05545868870600423,0.06682674978792995,0.06582349746528246,0.06901750145297561,0.06509708966550225,0.06737655847207724,0.06983404313661394,0.05845273387452677,0.06916511598144542,0.056430610857027975,0.0688917237226675,0.05804940494863293,0.05997974403418153,0.05839244107666125,0.060068345019814146,0.07362424750118407,0.06421978265556205,0.06762395410629103,0.052726385657302796,0.06662272324624227,0.06974077974992737,0.06610471320127755,0.06259719395038005,0.06410994318245253,0.06767637297381082,0.06327663690191246,0.06041746126344685,0.06705851064250434,0.07253462761310202,0.06874526987262186,0.06402510848188385,0.06687579502532401,0.05563018659469712,0.06950453553722952,0.06573102245454715,0.06050046892793892,0.05953513293966668,0.059394448755601,0.06973150159320488,0.059215609838727715,0.06238423536218471,0.0642972596719881,0.06398873694281429,0.056010448946236,0.07517403222073302,0.06293337605578798,0.07566289189764805,0.07013903745899638,0.0594608618694161,0.060406516733887174,0.07202752249851493,0.06930197547370975,0.075508366652078,0.06917210891859404,0.07006663686852943,0.05731997088228452,0.05902822086144489,0.05753780428893504,0.07086159068870432,0.0728758640463405,0.05052784095612708,0.07320676097095555,0.06064717951809738,0.045669895300089386,0.06257079783808006,0.06597419682978026,0.05972621518520087,0.06700806784511804,0.05959558287548853,0.06874768672792644,0.06408616566446751,0.052056488591080974,0.05165536112025623,0.05937598335796762,0.0699061272865303,0.06718060461022302,0.05954754600437494,0.06479615334346679,0.058850525768748475,0.07056791546534871,0.06810014240482205,0.0688800046673565,0.055910040721304524,0.05925870891791099,0.07405757552262326,0.060704769473798895,0.07469344137323403,0.07714194179509198,0.056609723630205,0.06498617629980659,0.06514280047800391,0.056070358487774204,0.05928803491016131,0.06104651599217496,0.061720768479793436,0.05734417629065776,0.05590603283428923,0.05611424113377168,0.06704448801847154,0.052467598241826514,0.06488807357729295,0.05660657035721921,0.06502179869968683,0.07145830986611046,0.06808033345505263,0.06315675896978504,0.059525010490497,0.07107431783516793,0.06233763227680415,0.050679830453654554,0.06511350757221351,0.05851941531057231,0.06129452968690407,0.06437514572908125,0.07402104127055277,0.05372740584002034,0.07030722994085004,0.06550917490131224,0.06299555469428725,0.05946821292303948,0.06211927208152593,0.05965099543412318,0.05257433648128025,0.05086437052379711,0.0722494975284022,0.06854752552102522,0.06107277803477274,0.05064805053483733,0.06256762853648574,0.0583247912165205,0.07346079225091429,0.06336651741579874,0.05694529449483643,0.06611201588641874,0.06262857227960139,0.0649746111331636,0.06378638662638131,0.07448550454502006,0.06648907966990163,0.0616159037124977,0.0645691860526395,0.06795259136968287,0.06791717961206273,0.06101743753860919,0.06524589966746239,0.06480344238550989,0.06878214903290228,0.05139649337304518,0.06103094665806888,0.06535668548722792,0.07277060114115147,0.06394378989917365,0.060532276477137784,0.06482230538844688,0.06628511818080854,0.05768121635910614,0.0665050423352147,0.066415461158436,0.057463308405915556,0.05321877456056124,0.0636390491389401,0.05680527240008659,0.051033652741577895,0.0533555947918018,0.05411037312352223,0.05610181020260958,0.06733786910154431,0.052588999804830144,0.06864766167847661,0.06091107431583113,0.061180671646871805,0.06676535574418568,0.07059768697705496,0.06853945853067131,0.05876563124514114,0.06377341474731563,0.06275839554811574,0.06817634739857165,0.07283543900664532,0.06489619305167485,0.06458005870320514,0.06457263194299531,0.06128122958900992,0.06280558344503906,0.05142620674595032,0.06163763029877293,0.0592361821292616,0.0666454199907418,0.06703545832615705,0.06171494233675877,0.0722977707517044,0.05738638684949206,0.06478066219038803,0.052772205664340405,0.06810486408601824,0.0659464306877337,0.05456518205569018,0.05573925692892999,0.06051550060486116,0.06094207622466217,0.059405387918103264,0.05536519599989333,0.06237144498845242,0.07320182159062336,0.06934091029861401,0.06011375767569571,0.06230090732379572,0.06158899165341902,0.05097104583886709,0.06638488706190242,0.06825145953755082,0.07359767473886274,0.06460676977996503,0.06377147869235322,0.060638833081700774,0.0720906048911149,0.05726867110522639,0.06030401039691275,0.060245244192667975,0.06283381795011925,0.06512875555918315,0.06603391535900344,0.06092524681333236,0.05757914318992167,0.07156498643119412,0.052928962521078135,0.05820947322520073,0.06711125278644181,0.05550158205516441,0.06621381286303718,0.06873097054720029,0.06625807765584305,0.06137653850045612,0.06427011004447927,0.058994081139367216,0.06787819169116363,0.06398533795585666,0.05893060794922034,0.05875306801748668,0.056782380116629214,0.0625931942912292,0.07061279496249101,0.06509767772558461,0.05520982853765845,0.068038911332726,0.06289780364997113,0.04647757879350718,0.07060944103709735,0.05474148297058877,0.05258841695484942,0.06973676051455203,0.0730903578987107,0.06375780844004073,0.05652298399775879,0.06256955350403598,0.055860781097959514,0.05101037482176527,0.06287235967978763,0.06860997247937559,0.0666007260086852,0.0665561026295145,0.059037737371796524,0.06193706453027968,0.05695689557087663,0.06908431162953305,0.059879222517543365,0.07004534235193044,0.06348868777384015,0.05911740973285852,0.0691879843339886,0.0669017967048775,0.061321559435741434,0.061645528170630945,0.05432258196369547,0.06778072397462377,0.05892824914073952,0.05833177079840306,0.0694063636134792,0.05129700778074163,0.055805853160394256,0.06954527555532217,0.06523786914926813,0.060925873462355375,0.05274511621496861,0.06758991476055398,0.06394868495352257,0.05363957323192266,0.07016220745543914,0.05205130132798039,0.05654341406057893,0.06907268731287416,0.06737596274555141,0.07282483846479688,0.05411030650291948,0.0691799585220951,0.06609066491724075,0.05561731270524784,0.06648231598157361,0.05919468561714085,0.07517830722416433,0.05883946920182166,0.06522486327708252,0.06814937994283037,0.06068128165878442,0.06441127708352457,0.061749554756579735,0.07792469094506407,0.06054749761401054,0.0670064588110989,0.06390024168481878,0.06546863369699402,0.07272214979881007,0.06725327325173361,0.0590345653396977,0.07192282157517985,0.057235088663910944,0.06731596900932318,0.06088630181454263,0.06132368249314433,0.053846916675295134,0.054428734163739584,0.0668828167428463,0.06794498031480048,0.07072180527028596,0.06685443718904463,0.06379138367691364,0.07308045219216014,0.06264821547511447,0.07133464321084512,0.06117549954810296,0.06293913905801658,0.07170835049827975,0.0652151053965245,0.06654807374859248,0.07554362321952432,0.06684496584831627,0.07117230231789101,0.05533669616382098,0.06948556203236744,0.05899558244539782,0.06129418833782518,0.07399209789135634,0.06528297332938376,0.05859458351694531,0.06379680395693094,0.04995253482552811,0.06267250107018003,0.06157494540856303,0.05828912150295943,0.06276811424697762,0.06036730157227366,0.06731869628806289,0.06783739657195052,0.07164776294957903,0.05499958762738977,0.06735235613316745,0.058205957608218624,0.07384475081872605,0.056384109260197476,0.06300248741517617,0.06367900085705927,0.06682873972202819,0.06688727781700718,0.04803738999894116,0.05793682971241161,0.06444570016995044,0.06542908569323132,0.0649596149063682,0.06986130133940285,0.06883725770896504,0.05023926944077518,0.0641123824701228,0.06110567506645407,0.06488737757890659,0.07211498049575399,0.06830877027836812,0.05838489696965708,0.06859545821133475,0.05774695421993535,0.06581279973805232,0.05812695026754007,0.07443170637005578,0.06453222481755808,0.0688954468996573,0.06100996264258206,0.061405148909796405,0.06012246204551564,0.06175411773513539,0.07446648638911474,0.060937376884919806,0.06165492618647092,0.05412994651530896,0.0711873765147877,0.06877056118915376,0.05485815448409191,0.055777485531998856,0.06373098783263856,0.0638494384660761,0.06186662557885116,0.06765095984940259,0.05524737990458253,0.06602612735122779,0.06770601699341364,0.061231121773780546,0.06898893456103447,0.07320610669316881,0.06345318689732699,0.05766733308961679,0.06877458986592165,0.06790437826435236,0.06360279610035183,0.0658642900645042,0.07142487213897386,0.06652482081251487,0.06968368040658915,0.07098086040382796,0.07516331857542416,0.05719048461798993,0.063959629084972,0.05264381159726912,0.06586030277872135,0.06661312818508952,0.05822412105624415,0.06346091143424162,0.06558900574654745,0.06510215569610928,0.06028980493408038,0.05891088981205562,0.06200011393366707,0.06463533903490644,0.06435752699776093,0.06718425610810702,0.05706875818748367,0.05720623441976132,0.06607523009656338,0.058113101586858854,0.06265122114284856,0.06195292156274048,0.066003731919938,0.06426586893366498,0.0691967693355253,0.06682874377261214,0.066568405231859,0.06427858016441884,0.054090633925345105,0.050746737120542854,0.0579368339963902,0.05209105730885735,0.05711738380219569,0.06386380256642446,0.06053871031807091,0.043416098040761596,0.07202809599704335,0.0697163510791575,0.05394437783136097,0.06357591193299894,0.0668780271459842,0.06163532278340045,0.06249818370958369,0.06183986264070448,0.061508646610988746,0.05214325193824003,0.06093489789712961,0.06516125528895288,0.053254178214972483,0.05595499389675873,0.06717715508495084,0.06054446254162208,0.060354324820890565,0.06292144546274278,0.06956755116788307,0.06602975284470552,0.058325650702102286,0.06282721556745535,0.06379481977857476,0.05999515463920928,0.06308309338215279,0.06215049925684995,0.06883038999254323,0.06943957743814676,0.06708895196450898,0.057242459730376394,0.056613767178514046,0.0730446179831164,0.06262426367295225,0.06707608428199074,0.06536306491444438,0.04948576908752443,0.060412550837458945,0.06563700604298658,0.061145240561414495,0.06801628156944695,0.06281828173266829,0.05600710838081822,0.05800108030693511,0.05890751103298919,0.06587610073192114,0.07727738137264797,0.05702242029329219,0.06406840216236068,0.07291477330344118,0.06104810438704383,0.06108374510219059,0.06148969789867603,0.06362005153622083,0.05834088666196284,0.06595139146097151,0.05602197931204915,0.07296593452724393,0.06935034976713106,0.06682710925113763,0.06627316678677439,0.06490071940883568,0.06685342412777315,0.06411571766384142,0.05775189847884023,0.0678622742700258,0.06544478379506273,0.06329551887965754,0.06947449913085005,0.05973679738549624,0.06397425295461567,0.055177824480941204,0.0637884867608774,0.06365509793363311,0.07050246812659583,0.06875864940826731,0.06496835525105168,0.06282901419474915,0.06879059564133477,0.06467955204216404,0.06153338351034369,0.06887690123210682,0.06812388396593405,0.05626979753993358,0.07058464598459827,0.07245977344951687,0.06266508390333811,0.06557108685352737,0.05902552116254077,0.06444527278353207,0.06832921851005141,0.06206780575471678,0.07275010676393608,0.06383520099341251,0.0594656334134373,0.06925705689841793,0.04447762295102431,0.06687748438874377,0.06892195801228024,0.06374258074908859,0.06470868694718215,0.06746657758895491,0.06298405246760067,0.0647032435555414,0.060300304372593566,0.07453804299007842,0.05791621154977904,0.07603645750150961,0.061881290359468204,0.06617853030815163,0.06718871403728996,0.06185540380678869,0.05761508962453668,0.06397400362072518,0.06868970806683566,0.06840350099889952,0.05983611947080521,0.06463108806244747,0.07279724765225801,0.06501147415553835,0.05683462577809255,0.06257902149461005,0.06904063737648714,0.05981032925802009,0.0635477376393562,0.0562605505660696,0.06421128859290132,0.06262757905639432,0.05980398392015022,0.06637577043165963,0.061412205800991236,0.06620870994921768,0.06575906134068413,0.06582178451149047,0.06291100855218484,0.05668307672289956,0.054336801173179294,0.0653082131424782,0.0572894642474515,0.06437431052157093,0.07295997302463861,0.06006045887788336,0.06724380070117052,0.06361160852135453,0.05465595542841412,0.07078200662300801,0.06411605853191543,0.05894738729813629,0.06320920246388373,0.06292976549347061,0.06603767342670339,0.06513389976103268,0.05946452975273114,0.07287606223802898,0.0515276818920127,0.06618688241195414,0.06370250541381017,0.06021098488844917,0.05442378981838002,0.05639479478133658,0.06736678579494626,0.05991773383244044,0.05917904976522688,0.06775106273394585,0.06218797883444152,0.052004238072843066,0.0655778209148847,0.05990451844531286,0.05134417281878921,0.07246689742271277,0.05433114446526202,0.06462090879973685,0.05908342819056581,0.061726979245932914,0.06774816836673883,0.06771327946000212,0.06253682889363926,0.06818259111323938,0.05922551972474201,0.0714486813517386,0.06262344364052613,0.05885860483085135,0.06710056730593233,0.06265387978803516,0.061791279178012776,0.07044531543356294,0.06860145717884947,0.0445848343733474,0.053601000758955006,0.06908024917853033,0.0642832882785869,0.06604226806125285,0.06381381227891939,0.0647812778802894,0.05982502026560531,0.06113360108980108,0.06695352084831835,0.0615485624935631,0.07370545101880446,0.06267270368338138,0.06036005035771292,0.06530569203273281,0.06930986525377317,0.062282936642410316,0.06594940715640188,0.06334627990494313,0.0639989862015229,0.06332425196728392,0.05684349128204064,0.06701344295348399,0.05971429124407859,0.05970121705768537,0.07091764018383431,0.06476687236518985,0.06593300662447131,0.06260373943825737,0.06533607900090577,0.06039903514076001,0.06156208268805104,0.06485937428532412,0.06405775700467475,0.06978321156786037,0.06427524037343788,0.06463931325590888,0.0641957680609394,0.0519669794440003,0.06699459625851789,0.06409794491530535,0.06345825135650118,0.06373870608063954,0.0711326990483663,0.061387805727604536,0.0584512901193326,0.06419661954437983,0.05586762243291946,0.0675353073692938,0.07335918982140746,0.0725615075410414,0.044881939533268314,0.06309671626692429,0.06745351785502571,0.054850924551349864,0.0695368412833685,0.050833962127532856,0.06117777889042411,0.0722419780374893,0.07029248632931566,0.05762836623488008,0.06430839217085332,0.06443284399254387,0.07417158370714437,0.06047372916778638,0.06491339202214418,0.07174827208286502,0.06127870167073893,0.06034447094975294,0.06432955476057367,0.06292097442249936,0.07099848077598428,0.07379654817453334,0.05918420790152315,0.06908178031298512,0.06723693449164839,0.059293391911036365,0.06894766233016097,0.05999692872832045,0.06230424404850676,0.06374521069756425,0.04994353293354786,0.06919041660824814,0.06453280911624631,0.07146242418867244,0.0632909832379111,0.06130235240244681,0.06729651161954078,0.06360745293733266,0.06975708032075568,0.06385178516251128,0.06444870724970803,0.06040549879632335,0.06647705185327737,0.06387772230381417,0.0703769553115583,0.051497406272262805,0.06800189690443761,0.06446742290553233,0.06643471815323883,0.057675756068650025,0.06669148118023246,0.06811040563797639,0.057066828753493754,0.05652952644727838,0.07121395044163858,0.07350648245510727,0.06824755473996794,0.06518272163090413,0.07132343255762894,0.057853603043049234,0.06417044169120208,0.05376186181172981,0.06571846877972606,0.05271378036701085,0.056973473778946265,0.06138790309450425,0.059165297986266296,0.06529769473543406,0.06983260687340594,0.05812707199111278,0.07069754507410836,0.06538270191105794,0.04891370910917435,0.07292616739337021,0.06772536661336429,0.06729490166014865,0.06510083468877503,0.07183810406859795,0.06734797444125021,0.05778322242688295,0.07247579676059898,0.06192662699821926,0.05507828240411149,0.07019483299586562,0.054760858149166675,0.06343376206485815,0.06878605396060967,0.06660600682331128,0.06493479492577367,0.05089648616832691,0.05676567261907445,0.06209070917510155,0.06597697329608651,0.0651369648110923,0.05776215134069832,0.06382926065647834,0.060088188486725654,0.055779912056154214,0.04930974108968102,0.06879370466777546,0.06857859292204485,0.060187392158424655,0.0650386923535762,0.05183928937979537,0.07087419707713866,0.05692065330804799,0.07102729221720591,0.06582147288237788,0.06238935465487372,0.06224166788790648,0.05764932105024388,0.06572868483934804,0.05387561411392816,0.05413086348357728,0.06188704427719729,0.06839145899943667,0.060432072240170456,0.06629884623688315,0.06614691435845899,0.06827505493372582,0.06380417864363506,0.06765851155830319,0.06752629686197503,0.06128373280427154,0.0643299917162708,0.05547368093221312,0.06452399628554989,0.05937877577854074,0.06377319084391164,0.060478871089356324,0.066791408563323,0.05645559986817312,0.06850815306078531,0.07012980208749958,0.07431330255384028,0.06526725471657283,0.06570482221465,0.06338620574542042,0.05236985934285303,0.0568040260600997,0.07549227031765388,0.057515035183044884,0.06997229232809583,0.06639388904601054,0.07296385475958464,0.06887562568607188,0.06385712556458847,0.06455717163450841,0.06966097247216596,0.05862535257196452,0.060869983020609544,0.07287918474926458,0.06063639738431065,0.05871703024588548,0.05709530734026753,0.06293898802215817,0.05794137313867592,0.07209265843765998,0.05842068839845854,0.061696823142785955,0.062126962144773985,0.06495951538377173,0.05487600854416369,0.06186872061916876,0.05616476239276266,0.05374523363332863,0.05187071410044519,0.06085876960237538,0.06013864660097602,0.06902695231150473,0.06414723535879274,0.061162451503372794,0.05786189416546786,0.06313192273750518,0.05718711324021501,0.05586819135356272,0.06335375037939855,0.061999901680919484,0.06329940149673412,0.06888200043766506,0.06655259312940903,0.06525854067287581,0.05467945367296735,0.05843796287124568,0.07034527775125352,0.07271802647624662,0.057305876045728885,0.058690133734516634,0.062440505380807004,0.07406895941621487,0.06004189694206662,0.06473904572603074,0.05800406732317615,0.0670782340278987,0.07210789643545568,0.049904596848154945,0.06253184498389483,0.07217376686337171,0.05693609878911527,0.06564973578785209,0.060295679682725936,0.06899558221258878,0.05743564750633191,0.0722894326349269,0.059637927398163315,0.058578983051188124,0.06638652052324456,0.07304518314825745,0.05958884806779202,0.056599733080902555,0.06794132961574954,0.05552965265641774,0.06944212231599231,0.0639466307896836,0.06660121391101616,0.06332500607143245,0.06416748093651402,0.06495897820760821,0.07137812563310128,0.0610646082600724,0.05965155759037659,0.059347827018268226,0.06545204609701383,0.05813953008698208,0.047712698847187665,0.06393515249718648,0.06905706723453033,0.06378421492296356,0.06972079355723323,0.06007718600999824,0.06650919264080199,0.07600983919740391,0.0705611076372393,0.06913740944050475,0.06613846848545234,0.06240945361764817,0.05666038494682887,0.06430484792316402,0.060568511707162195,0.05513713086274348,0.06265643653323696,0.07136455473198618,0.0626411258473628,0.061444752100649994,0.05277521629273922,0.06675431034431137,0.06417923301281434,0.05469251969175569,0.06723394562105264,0.062494264526847774,0.06279040304898145,0.06251867643750632,0.053480917505383765,0.063463535537645,0.05534092335816802,0.06884514394602187,0.06896873817025864,0.0574035535511924,0.07277942113526377,0.05155476348099336,0.06532403307040147,0.06846024899085612,0.07010717511076267,0.0678540273600115,0.05941435321728407,0.06748799301898295,0.06550128745993633,0.06675473814527923,0.05568478236142521,0.06963468517918701,0.06257588591039673,0.05942918938402274,0.060835523397839625,0.06282844791730377,0.060916112058345635,0.06930865139953855,0.056460386892162054,0.06603795505318172,0.059038889378977374,0.06283739233428134,0.06223899947910724,0.05979573130021691,0.060605141753015734,0.06822075247083842,0.059410856748913804,0.06439935591295673,0.05872337485766159,0.05780557416452529,0.06722707408582507,0.07615661989175054,0.049921103598520505,0.06626506547899241,0.05319867533719207,0.06367782152552784,0.06885264860202397,0.059929490129469035,0.06796760961340481,0.06354249845349885,0.06998999642555948,0.06063376848277223,0.0721696699827137,0.05849888439243259,0.061207162968540604,0.061084359206959926,0.057105967517859964,0.053404848379066616,0.06132237881028261,0.05545785691644377,0.06606086415311582,0.058239290355043896,0.05933271513487935,0.06655727471516074,0.06772360991293375,0.06514536880905109,0.052155842332538505,0.06865972825279626,0.06724588650563956,0.06739787509527587,0.06937911138497785,0.061340981204113486,0.06424850026917214,0.07350052148621984,0.068125324279074,0.058032702809872806,0.06505857170185195,0.046517663547272665,0.07436986819933494,0.07381940154580971,0.06505283881671234,0.06301394699306181,0.06556786623638503,0.06097991717598439,0.06087495154945752,0.06978204548884448,0.06355104426151706,0.056717211944634235,0.06140479351811461,0.05980258027619151,0.050922584432640014,0.06295420039302924,0.0648439238198214,0.07143708506929973,0.07381748396263406,0.06610785650493015,0.07361241869517818,0.05681972099026979,0.05737816049044257,0.06558270562444117,0.05392550066590732,0.06631509499917512,0.06073554364138998,0.06385656387451455,0.06874891142987587,0.065578292675893,0.06757252980823569,0.07299552569063567,0.07150526821929082,0.0629064198375312,0.06837958955557628,0.06406608695726704,0.06189053597704156,0.05918942500964084,0.06032029450739702,0.062337855386944495,0.06034931880745903,0.06331278002505908,0.05917364035819404,0.06544371858162021,0.0659272489969635,0.07270503705513827,0.060900781089077816,0.06541933707239615,0.05445711082331428,0.07472483679470479,0.0731071879715931,0.060748384035164096,0.0666635738265032,0.05430322408321438,0.07254042346869494,0.06759251939001285,0.06543756898814858,0.06447044006992729,0.05971912175532133,0.06357310132257127,0.057936145039278436,0.06745119986705206,0.06769964939163736,0.06777713776337235,0.06057003711577964,0.06221439541705198,0.060093154033341964,0.06732293159946423,0.05364904752331332,0.06222988668286869,0.0600118274883118,0.054710429101547595,0.06133540782940658,0.06123642938184557,0.06520700953621038,0.06038082492145555,0.06328549240248929,0.060397527518155,0.05520294988220788,0.056744345235975324,0.070847348549356,0.06090481635663717,0.06442306438032595,0.047420214100279454,0.06454399922134801,0.051426856266345494,0.06537213643810537,0.07099408797572748,0.06134168286539412,0.06855738507084841,0.06259516938827928,0.06301958896569229,0.06364338141244028,0.07011594657108192,0.06375182526084794,0.0527056225597293,0.06418427956008979,0.058962908547775195,0.05048478721192445,0.06573842457037948,0.06949663453796194,0.0647121577230372,0.063847474000503,0.06094657383386762,0.0688688664263146,0.0681956273314076,0.0645650447973836,0.0692756237717694,0.06752148935088864,0.049144376000692325,0.06370345521104961,0.06868994392075692,0.06696334020468991,0.06597708575419947,0.06285938913317696,0.06405055157263037,0.06167864392868252,0.054629099189178174,0.062103096415162755,0.05224792390674221,0.05703265036332007,0.06585043328451665,0.05361822746510456,0.05816548061925717,0.06297007001779922,0.051248630480487306,0.07178079020986441,0.061039795385558926,0.05404174827190314,0.06353087390862255,0.056401705602131966,0.06258340789690159,0.06595701689218536,0.06339602208724865,0.060745111100468624,0.05714463762440295,0.06782960911532451,0.06506331026421369,0.06756087280564207,0.06373941025893178,0.06928436189992573,0.06215001029295979,0.05835419040234911,0.07464951141884818,0.06958207575750808,0.05321488091964968,0.06489306584049431,0.05916286609032723,0.05424444169507513,0.06178665238794206,0.06954112269589244,0.06108336110016671,0.06137662767873106,0.058792826537863815,0.05371192637317915,0.05367535088460671,0.06127772809122561,0.06583616586751206,0.05834711305821197,0.05978707637426324,0.0688307946665564,0.06611595299240289,0.05825861091454535,0.05246608149545333,0.064084243737691,0.061363071767477996,0.06101592893208139,0.06483238608144595,0.06114750876737596,0.06421267025888938,0.07019056785300962,0.07076417875211705,0.05255738031551395,0.05981049137332093,0.06080422312995537,0.06126738906292618,0.0652615013970162,0.06671141320801807,0.06455630841757506,0.06774365788242365,0.055635892173404686,0.06575271383368989,0.0715737848191775,0.07046266304363696,0.06399780655043136,0.06072995861204786,0.06457517934793836,0.06599330314259527,0.06289962792019607,0.05907526792359401,0.06494285002053965,0.06546932073464416,0.06015573236232813,0.06987499905705194,0.06450242345840189,0.06934966542995334,0.06349787032363854,0.07216269864514281,0.0723801154973128,0.0656057886306754,0.06731310283228632,0.061641162295392966,0.05173545779128551,0.06568967930124772,0.06473817829055692,0.06491295585640827,0.06458214045588938,0.07455109321707154,0.0736489339851989,0.07261630865161416,0.06636114046088826,0.060319054091096155,0.06694576551550505,0.06934152996479837,0.05974183944894243,0.06689407830864967,0.05871217032321914,0.06471308808307244,0.06468989811614313,0.058485636483824446,0.057897334947569615,0.07115863141590123,0.06982531392611382,0.068934365696104,0.06279856932229724,0.07429104010286583,0.06205066434343983,0.07540591609603342,0.054688594898409874,0.05659021201979114,0.05733274322799717,0.06347467544028228,0.05814993061507154,0.06808656497570795,0.06651739140626756,0.062381637312831265,0.06684968814103331,0.061788543634385004,0.052713247141759234,0.06329841976666015,0.059236981836971014,0.06723706332331839,0.059117139447664065,0.07502671332911537,0.07147995779131597,0.06138055814784013,0.07278323491128577,0.05547297763320498,0.06913899405070964,0.055420980490077945,0.06764653792555395,0.05975237237680947,0.05941242708425607,0.06513132974229296,0.06987697670718523,0.06666597996306912,0.06230722497744322,0.07122481747817574,0.0731050587010386,0.06338073215987178,0.06197649056478088,0.06869967002963154,0.07396632994380758,0.06512219676801162,0.06240633397425307,0.06557112398466258,0.059754162929856,0.061581868843956285,0.060781286049538294,0.06091998588312995,0.07261893024970362,0.05504186692080211,0.0691569439892683,0.06608048036942235,0.056093435848899004,0.06456357592959686,0.06151063202885766,0.0708774109653003,0.056403997426177084,0.07413223603833777,0.058421871998921626,0.06973820896684975,0.05854224589642801,0.07047141865945772,0.06872403265786833,0.06374331331090863,0.06535926440512087,0.05136437547296839,0.06918986527157223,0.05995566911608444,0.06466231987409282,0.0657699988732669,0.06012545043702777,0.07099602764110607,0.06189192840361557,0.053947407275340115,0.06264391831799797,0.07022774769710863,0.06907679303771957,0.06018292567426203,0.05034659891489418,0.055408221350780024,0.06921817263248649,0.048481779266607364,0.06846570018869028,0.054510884709089505,0.07311577350451598,0.06697138913607585,0.05994926978990263,0.06295320579457972,0.05703304551930098,0.06910978056937234,0.06257538720511387,0.06812222106217267,0.06017593604840347,0.06945808073869261,0.06898990840939029,0.053750509795370646,0.06908911923883315,0.07167270181194393,0.05723300275680836,0.058548568013648744,0.06868996171453959,0.06543764026049698,0.059339957647953814,0.06313477713133661,0.0668305704301411,0.06416891868892757,0.06369927034362671,0.06557133254763681,0.06463659588266646,0.05548863354056076,0.06445408464748095,0.05561387603209218,0.06430253867941901,0.06356266576276773,0.06399378348104606,0.07135061688142458,0.07091302608310268,0.049823283574518935,0.05937216724690962,0.0579454023180309,0.05798611005581855,0.052844108829441745,0.05979509489940944,0.061015493781727534,0.06579829911872628,0.05600606591897919,0.06619148829033342,0.06311109489966601,0.06717942174754697,0.055570466089815286,0.07055241290239225,0.055227675987580306,0.06957052085990623,0.05921903641664343,0.06294222508441621,0.06913509710035377,0.0692925758911804,0.07611242589478193,0.061559749156277745,0.0622703799100158,0.05667275487278578,0.06548220254580694,0.05940828954600018,0.07255049765795728,0.06871302054544381,0.06889107730907149,0.06582301075141184,0.0626174897381718,0.07495348025655109,0.07204223294937545,0.07136984681867409,0.05063686652762415,0.06916000789367194,0.05452574469071442,0.06501624783143922,0.06819961078517073,0.070050364961186,0.06498306774034035,0.05962060626439203,0.06024077037243829,0.07214099941420206,0.06592015371166485,0.05803870248884582,0.06717939696752452,0.060997644674771354,0.06507673235597732,0.06735878967247187,0.06923493971723124,0.06887599438786893,0.05303526330992173,0.055879850933153466,0.06612501060344544,0.05766445679443414,0.071053697911859,0.06868023570218722,0.054904654131857425,0.05868920984402999,0.06941030534628556,0.07358796342817034,0.06031102363186333,0.06969026247773541,0.06439720594089876,0.06027303321059296,0.06011892186935647,0.06423774466297119,0.06316928903834954,0.07252708043438237,0.06095429862363948,0.06791182029191445,0.06200812079321496,0.07238822075775297,0.057883364628771264,0.0571046588151401,0.05959686403021492,0.06313804209348031,0.06546316165002065,0.06034810816438164,0.06807798618232744,0.07288870458317559,0.06919179182718446,0.06137743881091847,0.05368833536326962,0.07081494500009358,0.05425403971678501,0.06799436839434203,0.07371044167385629,0.05468880297474636,0.061570240065859094,0.06128695307555005,0.06279430136884509,0.05651419074359723,0.061547675211733405,0.052509277254906686,0.0562422218390143,0.05435291684625948,0.06586467882298343,0.06343359349139926,0.0664084053458966,0.055473360709580866,0.05940959132743995,0.06138308224166656,0.07444795415820885,0.059405714374561366,0.0641192118643768,0.068579362236869,0.062295094101222886,0.05527212090799351,0.06882663014374696,0.06439965575513366,0.06151982716887547,0.06834193535621186,0.05563812835372184,0.05460314494239038,0.058221719239512804,0.065440941292838,0.06345215827407341,0.062063592159494176,0.04680133257557652,0.05720577381190402,0.06690187068805145,0.054306226865469515,0.07115868444528547,0.06380491604297428,0.0680698619001926,0.0687681467932357,0.05636135130563656,0.04960920034412036,0.06640576795080111,0.06200418383122079,0.05679312170499136,0.05177287644645178,0.05989696341317689,0.0637939795754055,0.05045772342615014,0.062016757072803234,0.0638589518649572,0.06003481863451557,0.0639178149184167,0.06663111165292523,0.06841830358303624,0.07049049476920748,0.05424187565165298,0.05932584051859868,0.06004715927970573,0.06319850250605898,0.06139837823395228,0.05724605650576079,0.06240768462583972,0.062452075716497704,0.06461792106166121,0.06610168352507365,0.07553110898456986,0.07338579951826454,0.07087850887510072,0.056065643654119215,0.07080881486645307,0.06734561046636417,0.0707269975039668,0.05992529727788669,0.060028227350271815,0.06278067524287924,0.06485503401785675,0.06727211556072303,0.06859202190184735,0.06835668231676376,0.07263299280741214,0.05057055029133391,0.05793867859325896,0.06022056339568989,0.06119643004524837,0.05355039670163678,0.06096248399246882,0.06624991135554076,0.06544383765650644,0.05317434929176755,0.0561037487112025,0.06271562662462918,0.06384126078599488,0.06510500827054111,0.06956178603791753,0.07379098451856583,0.0657958422418499,0.06767398584052796,0.06406776629433164,0.05832611624326854,0.07395914469783363,0.06445829470147688,0.061125789808829795,0.05790891825902948,0.0690568049830651,0.06849117698238047,0.07623353664837232,0.06591680651352991,0.06597770747976484,0.06305667923976532,0.06772510857159872,0.06590634543796456,0.06084308654020332,0.07005855015194956,0.06661241500793477,0.0656929944526123,0.06892674909472855,0.05795582541154244,0.0563826185267509,0.07075920558846721,0.06257350888437713,0.06442263217823928,0.06039537791946899,0.05149784251046988,0.05978748462287477,0.07354030439953493,0.0637611076737479,0.06794018744876013,0.06552428938519908,0.06994963712223257,0.059770549442518445,0.06395199773216392,0.0597280630994263,0.06765424323034629,0.06789357359233916,0.0636958577369506,0.07064678083312848,0.05162588792347077,0.06622531721226839,0.07005624050494985,0.062369018137723435,0.0671429769497486,0.05897055064784609,0.07102577246756603,0.07304489553503836,0.054772512148253336,0.05285593925987238,0.06759908183650273,0.07142263614232247,0.06085340894406881,0.05649937123673057,0.0729922208496509,0.05770290540504126,0.06695229793597839,0.06465175508533397,0.07174555791401686,0.06161587258834772,0.0657704127281242,0.057323815077592696,0.0716793829905848,0.05950427382952897,0.06328118935686369,0.06020938733548463,0.07115479732074359,0.06253075586355533,0.06434316709315246,0.0604554919786265,0.06512823182059928,0.07103510649815145,0.058370459794772026,0.06679434577035309,0.063300439310703,0.06683126498404271,0.05915218605505773,0.07642555719557215,0.06056491842636014,0.054718579700785516,0.0726495005808753,0.07016151446115186,0.05955987059045094,0.05454359404096003,0.07029248396244578,0.07067129890270182,0.06814921830974474,0.0634573369416733,0.057101578383604204,0.05601721627160218,0.06177702697001865,0.05781431479876689,0.07009875312540614,0.06456694411691215,0.06968357287925694,0.05889575546319116,0.07245959855953557,0.07358045867765484,0.06535343916750398,0.06570152791641592,0.06296557882461606,0.061760401892274816,0.06360765651682429,0.06476482005173424,0.05542165113315109,0.06016293630473726,0.06105066666190807,0.059874639934622755,0.06051615242923961,0.05864285139436427,0.05377567157828775,0.06568028947259993,0.0710771930795921,0.06770746202295702,0.05435472166967993,0.05864940470940408,0.0631994142811397,0.06818545552944516,0.07515362022981707,0.07127955621369357,0.061800601906223836,0.07141767873036309,0.046335118588845675,0.053159409787437745,0.05441535035938825,0.06568537287173298,0.06487156858731619,0.07092733016545943,0.05441477119417738,0.06287630747335932,0.056248760828014624,0.0699198196791162,0.0555993700278337,0.0714684144650467,0.06164037125463856,0.06673064819032246,0.060439813617673206,0.06615531384840459,0.06668038691987101,0.07186539661749736,0.06876352365931786,0.06467355495498836,0.06168117145839521,0.06954281369027043,0.05998304472622683,0.06510270686735378,0.06048533828907467,0.06042065886576911,0.05971859963491931,0.05054942826706019,0.06693341591906954,0.05718753169934228,0.06874105544651937,0.06867384727845446,0.06308616267169996,0.06635655398519974,0.0448900541452146,0.06301688291168875,0.05665263843676813,0.06857748253867232,0.06698699772485975,0.06939569782274954,0.06220407046744899,0.0652560194503676,0.06010875556712858,0.06856583805769237,0.057798098118928144,0.05967780728677988,0.06175652026754788,0.07177293180708638,0.06666285254506935,0.06347738621081114,0.0630214477495843,0.06181896532202963,0.05871645492810693,0.06641372860370784,0.06696342900350877,0.060679355780132734,0.06466730198729106,0.06956882210265082,0.0610144683117609,0.064456021104327,0.06084782059397445,0.0627034797484502,0.060490291708442126,0.07173879583678602,0.05282065273968775,0.06951751732271674,0.06960925181350228,0.06726668828808734,0.05741866519394351,0.05486688433382958,0.06607315485715959,0.06770106502609177,0.06605000483227792,0.061505962414199616,0.05924816114249605,0.06069485787579641,0.058506384582590595,0.06462888936491099,0.05500231579784862,0.05926765920137134,0.05488561935971595,0.06544730803117438,0.05247932230450483,0.06959339153440636,0.06285210681737577,0.05701110584140952,0.0547360177168939,0.06692208590779875,0.06386373219513644,0.06477967180691382,0.0651852378625048,0.04996153569534611,0.06231721045380345,0.06279088619058443,0.062481298119247186,0.058518808857019385,0.06337516714440389,0.059843424076138754,0.07194538572137898,0.06735295905771546,0.06441784845033328,0.07256212176546495,0.06693714903805667,0.06723030347891193,0.07051877557525836,0.059352394028117636,0.056573515973458555,0.07077908566567226,0.06440170244099536,0.06125552999096847,0.06799755146385555,0.057251444716066524,0.07017624110583978,0.06762129053900523,0.05575970660919839,0.06277966248713354,0.05771938908180475,0.06948852261355701,0.06090230101728862,0.05294292687060823,0.06999159865469883,0.06960244558691292,0.05777736917031731,0.06571402916679625,0.07158900278326336,0.07049872004410339,0.06650318073751674,0.0654500403461263,0.06422369834338043,0.07100163522781208,0.06606852333918259,0.07030058572354841,0.06421942924762791,0.06682314271319875,0.06868563438588343,0.06839575003150583,0.0590461171364241,0.06064848297647711,0.0699618276948119,0.06800863041745939,0.06449868448595894,0.06971914673353577,0.06490025761018087,0.044888762992722764,0.06838070305637185,0.06137559028888494,0.05944067714588067,0.05619672069223217,0.06305838556408272,0.053874253149171256,0.04833212518855917,0.06890394705298132,0.07236208439231742,0.06478345371219155,0.06016116648459899,0.07025522146961279,0.06484840608660532,0.0728111002117994,0.07118983077634836,0.06294384429552072,0.061023192201342905,0.06684005639385557,0.06443450029103186,0.06368078393862571,0.07266696638470814,0.07051898752801025,0.0607463514403035,0.0548281113744639,0.05895160109296027,0.05972124517332181,0.06510460711171667,0.06900182709256183,0.05683943994566085,0.06188211263461698,0.04776043087705273,0.06435842467644581,0.05960066833163936,0.06837583382890083,0.06014027728344505,0.0615292703012846,0.06650777433686833,0.061037309798285096,0.05752374617300973,0.06059040547527589,0.05632377363804004,0.053294617136386925,0.06705700910166165,0.06741748702922024,0.056752706507232825,0.06296171001013885,0.06448506292619502,0.05858724194170659,0.06412081551149558,0.06681637547736949,0.07216898395256147,0.06081527754855866,0.050230267252327446,0.06957117265881815,0.05770562965924987,0.060621373498999334,0.0777962356468753,0.0647087129318253,0.06771740358751044,0.06534474865200673,0.06372554464003458,0.06203934738382061,0.061359574846562626,0.06321971468478202,0.06849615662800154,0.05609440399890499,0.06090984959707963,0.05246519631313672,0.06781216298832649,0.06567187938853915,0.06616358834305253,0.046835909290582914,0.06346327586380542,0.06747092975775242,0.06783488871612861,0.05130535578961247,0.05706022396918559,0.048039003739148795,0.07040410142494793,0.06340619951391774,0.07251230163339449,0.06621610853971427,0.07109285212791408,0.06580107936762078,0.05645520102699378,0.07163559822948917,0.06324666973652593,0.0519660842825741,0.05923377389454969,0.059655413361485915,0.06687279951698694,0.06483128642384847,0.0701008002960426,0.06533275759157306,0.07104091048084518,0.05752582731099719,0.06673916408443031,0.06432358944005526,0.06035484892245635,0.07273314323992011,0.06477998653358921,0.06267154321944951,0.06778840922173548,0.06434071461756571,0.06174238431313916,0.06331653293288761,0.06448589892547432,0.05932286068757528,0.06192396992193403,0.07061344133144587,0.06605860709774056,0.06649335019860492,0.06344512249487921,0.07062034503319595,0.06469732177748454,0.06920444834806284,0.04606110507388023,0.06546556153984287,0.059619152018886845,0.05828954125927607,0.062157166245366983,0.07379712210435636,0.06174541116820267,0.059462419176132104,0.06820363687955197,0.06705964836361797,0.06725327330539016,0.06073788473469933,0.07120881044509428,0.06152219955109865,0.059940265428667885,0.0645410482499871,0.07076244940709979,0.06566584761150929,0.06203021994342358,0.05887468124517094,0.06708227875859635,0.05369030717757947,0.0651054987945267,0.06397663389580657,0.06573798199223183,0.06403588689444742,0.06294870877871203,0.06988815178855774,0.06630487984696509,0.06193669763310427,0.06036676301554332,0.07216154222353112,0.07477003912131405,0.0645691998048043,0.06899964998630764,0.05865210360022925,0.07115585925817353,0.06809343532555728,0.060548773701893197,0.07134496164336071,0.06808425050910909,0.06493413821735675,0.06110019880715835,0.05372721734816277,0.06326394056904036,0.06333834656024695,0.06796521712217873,0.06991482598015905,0.066864247076617,0.06615718402990095,0.06665672029944641,0.05907128480138667,0.06045308729059856,0.06277706162600774,0.07315105455614436,0.06729808177574317,0.0562992529236568,0.06482029545821041,0.06215270353276752,0.06569527102256087,0.07198979274817435,0.06475723948119338,0.06843120991101784,0.06859863022189538,0.056982361789821315,0.0658039495191862,0.06367290544813545,0.06044086680003552,0.06524519319129721,0.07109959569925209,0.06749989467219819,0.055713338520145955,0.05280911192759436,0.07170495055447582,0.0628202419589835,0.06412472964465127,0.061463154635762775,0.07306678406692452,0.06451136993385174,0.07165874337153827,0.06693279126649235,0.06636417138723816,0.05931006772236415,0.07172135261681384,0.06103598106904568,0.0640383590529717,0.04331565266313553,0.06051705259134661,0.056303707618466155,0.06709247752904571,0.06651039593539135,0.06852378805725856,0.06281943259945819,0.05605220955874242,0.0655902381415236,0.06253807219972882,0.057201114779218076,0.0718861478924234,0.05559273904285604,0.05783313996269482,0.07149797827296275,0.07130793799710547,0.06141423422666267,0.05536239840793867,0.05498214781077211,0.0664350313044402,0.06983360850755746,0.05680755817732292,0.0632167345819134,0.054969139224323324,0.07217300264747811,0.06680372785578216,0.055450617218912375,0.06817280182610125,0.05666892841647636,0.06861184672986512,0.0779394922930991,0.07025653100297144,0.06104336401493082,0.05756261955169688,0.06660881569743485,0.06707624515972738,0.05606349201449662,0.05674237642576588,0.06538233199379276,0.06842552805386046,0.0697798492748854,0.0670405378518375,0.060995516503138794,0.06713414118097433,0.06275863496702253,0.06590954548856659,0.06832270196864375,0.06896798741393559,0.06963375098937905,0.05968683154953096,0.06424905987783569,0.05490666977090131,0.0616844923600854,0.06501323139164933,0.06464424448688796,0.0714426000731585,0.06144598770291591,0.06175705763099864,0.06814782832138003,0.05725102640849311,0.06941424494365453,0.05886411391375797,0.05985886911911387,0.05951199470097488,0.0704544419514462,0.061543746039219326,0.06573715148655203,0.06573329917830693,0.06459436805032363,0.06256457553097386,0.06382780239451738,0.056533223148047554,0.06375152115091332,0.056896121505490616,0.06495605794372833,0.059279874853633674,0.05932658749402511,0.05527085274047375,0.06832093991366539,0.06379227533582832,0.06488400319560311,0.0598743276067822,0.06661246692703252,0.06012782553443248,0.06253632997139111,0.05965459733669199,0.0644230769205418,0.07022166434565624,0.06429136936666818,0.06367120500544526,0.06289396738975797,0.06135881867517524,0.0576896866185959,0.06400844237533422,0.06486122423131524,0.06511220324547513,0.06199031183141915,0.07380409592883293,0.06746819323812583,0.061226739233166086,0.05734854202469214,0.06939545260373628,0.06265990234131517,0.04662443137237962,0.07375311163100204,0.05756884863897928,0.06082688401384218,0.06507403817485599,0.061246841067256974,0.05755981170883526,0.07078033232373257,0.06640030562426982,0.06715165977278689,0.0678345522746227,0.07353295011997182,0.07264843797041581,0.0671272620050042,0.07140847741440329,0.06142873444450944,0.06113440223473354,0.06062304022316012,0.059545607322516264,0.05807144338455986,0.07156183503974332,0.0639015316077777,0.06446888667649081,0.07514471757183741,0.06894241764783166,0.06026273250557674,0.06857361045282694,0.05805888902875529,0.060591091756998436,0.061743147297149095,0.07058949968269233,0.05758941749898847,0.06372345771647556,0.07068055177325037,0.06236513097044374,0.0740536584630831,0.06322089482782361,0.05556521940572654,0.05755319349423034,0.06706044846338563,0.048435414342644936,0.07385625341009674,0.06228301775777844,0.06150728416597065,0.06818830275019674,0.062996466060603,0.058031616820752516,0.05491678803954063,0.07041267021302784,0.0588322449695205,0.06000612384070626,0.07344315021178752,0.07214838200592988,0.06553968126129243,0.06741807097312023,0.06398819604053382,0.061256664671316435,0.06485732782641107,0.06860353734556524,0.06081428196145666,0.0565052262927936,0.056553596080017074,0.07022311831637751,0.05445187155112705,0.0735129234104408,0.07135951165567896,0.0677580331612544,0.05985207840997898,0.06240249130490644,0.06799321312160678,0.06914239231506572,0.0711554900641698,0.060008668213460764,0.06179520850537462,0.06446851080898239,0.0536395502851572,0.04751711688490626,0.05595327361363282,0.07400618026081227,0.05752479918025842,0.06802428081392084,0.06821217855509681,0.0638201945966704,0.06555595024916705,0.07038452507106222,0.0668312638761151,0.06077782769404003,0.06718139353149932,0.059472116010817345,0.06332250105508111,0.05822459179310201,0.07474122538438463,0.0631460298477314,0.05837258605367033,0.06534078044451916,0.07197681034999412,0.06635185135207666,0.06329977209317225,0.07292353295257828,0.06434300978439787,0.06570059426531738,0.06471330728680269,0.06776056200010169,0.07100235803115351,0.06225050349472784,0.06694458240440193,0.06439089072710283,0.0691701661303855,0.06114478068844739,0.06235631558198342,0.06934686529659559,0.06966971779820101,0.056576937750324,0.06968229954104682,0.05749594175639239,0.05366352166361507,0.05765015768956201,0.06680532010648571,0.0553445105296265,0.05379599204331029,0.07202330031011064,0.06815002216706476,0.05929951734524773,0.047484609591731036,0.06643622195626218,0.059897447586931385,0.05206256796249739,0.06860866943102534,0.0629396141922497,0.06570500252717822,0.05836302971799802,0.05573202488909565,0.06228935236534939,0.06805954492160571,0.056754194009822,0.07235347438588785,0.05652402277357313,0.0489972638074668,0.06797847248272176,0.06985169170152485,0.06739699550951617,0.06323247876623259,0.060392154719163885,0.07315624546927263,0.06636513729696022,0.06689838612012382,0.07120903481475273,0.05770114260985722,0.06471745715634052,0.05014856067205098,0.06288862039563739,0.06396979330784425,0.060263118278856456,0.06217415874708789,0.06623528085806799,0.06965014712559675,0.06875861957467785,0.06810252023593319,0.06930046904243505,0.05959688683380095,0.05468825242539974,0.06230891543922852,0.056207728978193816,0.06393346297467842,0.051539899190109054,0.07495131611928828,0.07373955481965107,0.0622107464118081,0.05955097628405451,0.052951446190137766,0.06709254616312456,0.052739431577167525,0.06488099677776266,0.06305244952306382,0.06740439834880022,0.06281313032965949,0.05421212648503204,0.06440580439314411,0.06699006887754132,0.06348824080679075,0.06762601531706519,0.06654407938578248,0.06350229792946419,0.04321598371337544,0.05684337624548946,0.07021174112149314,0.06829616964382224,0.06509755861393512,0.05972687919856709,0.06145498203768912,0.06695850738466387,0.06782368176176942,0.057996934553782395,0.06713826305660914,0.06393691544196045,0.06964949665288006,0.056787260731661654,0.06954687416618512,0.05740916567768079,0.056286321743789516,0.0599216886305718,0.06736400974661275,0.056037064266591,0.061063842558643194,0.06096042179438275,0.06863971708416283,0.05344540318540436,0.06854227610374036,0.0607200878025516,0.05960580253097742,0.05459306064529925,0.05532148224287882,0.06708204034698675,0.06886022468900224,0.06293258733189905,0.051813240154271384,0.06636330238427109,0.05796648461076975,0.05374666040995224,0.06080607822864535,0.0684481077374427,0.062457212434912,0.05969793330972979,0.06263582554917488,0.06747772591917897,0.05586604302365889,0.052675757149925104,0.06612960398549649,0.07140766908495082,0.05961431353969325,0.06769784903930409,0.06491178385200524,0.06599378969191888,0.06792813783843824,0.06967062628130163,0.06499272028561862,0.06742666203305044,0.07178701522672293,0.07002732008049375,0.07436798823263419,0.0577785646472168,0.07181470532619671,0.06497176958515619,0.06042260875140683,0.06703577434069188,0.06632626948822858,0.05867619739238919,0.06182159025792266,0.06761352021498937,0.06779086617255461,0.05560576899804012,0.06344894187169764,0.06307317489937302,0.05771399441074836,0.07426311661534019,0.06142812336205116,0.06495694074329594,0.06183831201683061,0.06468711796846374,0.0678801739121486,0.06242184678576861,0.06752834610535044,0.06848382136331896,0.05917269285249417,0.06789935284427023,0.05903482698983492,0.058916625709842525,0.07298017218885411,0.06675437836184242,0.05736546399102305,0.06032726175499409,0.06225538469415143,0.06616228085252429,0.0684310999194773,0.06628317984389584,0.06367763673723532,0.06185746023552026,0.0688200402449651,0.05530682696070808,0.0678736932963378,0.0609834469849972,0.0672678398381448,0.06693660690106576,0.061968252856869056,0.06643859783646068,0.04911517423266461,0.049851605254019674,0.051950018425583416,0.07278584335206202,0.06602717886882195,0.06439026536486875,0.06540784437262837,0.05747587637275932,0.06730527181650295,0.0686705792450525,0.055661377302869804,0.06052827833077866,0.0733808308927246,0.062318704819822095,0.07507423675794175,0.0603338738815994,0.050298105371902296,0.06346324798202865,0.06606867899593805,0.06493215499657012,0.06582813561118447,0.0683501541764096,0.06996599810006274,0.059248743133905664,0.06433130477094208,0.06726383803155148,0.05733523431129672,0.06525040548493491,0.05797716387908948,0.06278555135143413,0.06349409492847383,0.06609742937239045,0.06852619041690504,0.06967980095682687,0.06386281045678308,0.0668637462167887,0.06654252880748732,0.06390365160220629,0.06346549115175387,0.062458214515939095,0.06495358392259412,0.05237345850964065,0.0603975581043463,0.06915980608418987,0.06956346068670652,0.06481490511714683,0.06653046569237836,0.06705852471147616,0.06891008349897715,0.06818929274826138,0.05876564969539136,0.06729419193591699,0.06177331169072601,0.06064258389585878,0.06954984639433025,0.06525080086565846,0.05883630033941273,0.06546968612291126,0.06040499110838009,0.05445230307490153,0.07064152349384013,0.0666591459453118,0.06463130634762701,0.07432755882225071,0.07449757524895058,0.06536737798317999,0.06142844032961646,0.06428488549418492,0.07393959339374644,0.06885948570711362,0.05878996054314253,0.065432465749729,0.06154511391385341,0.06358868321560637,0.059353903200364044,0.06920641433109315,0.06586510296728673,0.06929964274786085,0.060984108645462595,0.059337368661784795,0.0578405924081795,0.05299465122007062,0.06504309269951748,0.07470333598571935,0.06820482295721828,0.06301797197352228,0.06367827427347747,0.0636439596368586,0.06883735007424033,0.05823886989948883,0.06604317956086234,0.0659000109541451,0.06652027511113581,0.07116911351504548,0.06844648552194807,0.05875570800176569,0.062019893384184954,0.06367768314383171,0.05798088815817329,0.062049203791304124,0.05115532895508608,0.0666216819333103,0.05381490795373069,0.050573386402657045,0.057769739868494166,0.06475048235067421,0.06527632966397558,0.05791290642796422,0.0591037608071773,0.06745518117437085,0.062011644720497076,0.05935002747713231,0.06486241761582559,0.07202462942330222,0.06820079821493988,0.04362043420736043,0.06585534270149482,0.0473091200257739,0.06991364043099804,0.06286686180841072,0.06468053835711171,0.062059985276431244,0.06851283960202899,0.07359721798484403,0.06368008035677403,0.06520539442138548,0.06623122399909695,0.0624430985457563,0.05681103288074657,0.05402305060897468,0.058928976168446445,0.049633886499659,0.06951009588903363,0.05318911905236694,0.0681770997830952,0.054913843402894055,0.05799068186654677,0.06867185202928108,0.07234651503954267,0.06203784863334159,0.07073314757013295,0.05504772640902725,0.07289438854025357,0.055217851257662875,0.07400978322331532,0.05643492801374642,0.06639908949678425,0.061499746874558704,0.06156527703690553,0.06669515003428877,0.06800437630261161,0.06516370284139764,0.06401898423186335,0.06733734026129135,0.06837757482446856,0.0671326215420989,0.06266798688032751,0.05994466030492458,0.05835936469324677,0.06481873881344498,0.06439044264015764,0.07495410312903816,0.06326140215233905,0.06382380216471331,0.053867830976660876,0.059046603110505386,0.06525507510394729,0.0690798382227889,0.06401320958798729,0.06547602228480578,0.06725171720624434,0.0614168405028353,0.06346376849053861,0.06770019364104948,0.05620509888062536,0.053493751961349055,0.06670450785832005,0.05584643510703781,0.05425466425620308,0.0673329233353076,0.06681232545799312,0.06335510407991077,0.07249307317498346,0.06912966378921889,0.0686736516140754,0.06771720790196473,0.0626022682188355,0.07075214178825275,0.06524566703565035,0.05953868939197075,0.06532885630691329,0.07080334167278449,0.05915043641970778,0.055675403642270124,0.06323298424960727,0.06779095979225605,0.06438630770082848,0.06995067008336078,0.06244870581109624,0.06385843137007394,0.06498955122525313,0.057629283435499626,0.06267307372866307,0.06490572561558772,0.06534535019337034,0.07127685995867813,0.056664930344986375,0.05885223053769184,0.06789527289275993,0.06584457759715077,0.07134274734252224,0.073827416214449,0.05978418209574574,0.07493519571670401,0.06672521389559424,0.06807170499361881,0.06981779826871351,0.06657859763927382,0.0660441687370133,0.06627475929147157,0.06728091551346344,0.06609841059272334,0.0737547875386953,0.05595659416223463,0.06623001661136046,0.059924125990050064,0.07609495895061985,0.0513068782048917,0.0645870043488033,0.060767357417621515,0.06856743520367378,0.06692021582495432,0.0519440207508143,0.07125714960350163,0.06325662588686115,0.06129541263211441,0.05792135674956357,0.0657040084583064,0.06216635220129816,0.0622985842728316,0.07518871802399424,0.063481327864275,0.054913770016155446,0.06488895307380385,0.06570959923564981,0.06457925828983069,0.0523759186697994,0.0589196944105361,0.0626436297396587,0.07225570139440338,0.07080161778200475,0.05929626822259105,0.06233089676419906,0.052360447264149505,0.06592930084959239,0.05706281942297847,0.06443636217283094,0.06109848626589341,0.06803556667346045,0.0646321783517006,0.06495852141250694,0.0696210538718886,0.05214946217149269,0.05478961627712848,0.06510333357887535,0.06097283769179208,0.06298565888641786,0.05593658710802754,0.06355108799331824,0.053159514959717935,0.06718114922481977,0.07081653043783812,0.06389520541458993,0.052947640294074894,0.0705493449508087,0.049941242583854926,0.06878681658291247,0.06945624374296758,0.06791137091874641,0.06339232651748146,0.06169220732903622,0.06020769428808127,0.05900286793150307,0.063573177035716,0.05147337571404137,0.0628893480018465,0.07314403678362749,0.06482273199662018,0.07449611422542426,0.06342197653736095,0.0700162660283894,0.06077339041058059,0.06778000567396078,0.056625180260230196,0.06806817205082771,0.06485792061329797,0.05780307599521707,0.06748633437089421,0.05511043152562639,0.05900177169917645,0.06039535691398616,0.06254798534141515,0.057148180636541965,0.06613652362607231,0.054854588579531766,0.06451836302773342,0.0664950268608214,0.06165576587304172,0.0673556828650581,0.058505710153795354,0.07180591263259763,0.06413371891921119,0.06904000979073358,0.07129305174773598,0.06687977356015855,0.062192438931819825,0.07104592097623413,0.06880753199740117,0.06483141016086116,0.0634977125484957,0.06104707875314844,0.06862513290211249,0.061939575576691025,0.06716809225342275,0.056361428416352444,0.07326152082603465,0.06350243999495209,0.0684338298653846,0.05940419458788125,0.053123112471120545,0.05772496862341109,0.0707292265115044,0.06687872745756598,0.07342251239853809,0.054532797633013524,0.07052469739255657,0.06269278206667692,0.05242362851738136,0.07108391438832626,0.06914800927251855,0.07067936658913954,0.06804761869030398,0.05967165372958108,0.06510876818260947,0.06304440339075328,0.059039045031755125,0.05953020124206352,0.06202668935516334,0.061849606872825987,0.05956987117197385,0.05785912270359414,0.06329404807884284,0.05609170264908593,0.06932024705209197,0.050802297378431754,0.06138633699422457,0.07180301160145292,0.06284662499725344,0.06908544696336355,0.06346787613517352,0.054912531156467395,0.06386982700167858,0.062003913057211923,0.06565668256166643,0.07190157482496265,0.06696313569088308,0.06224464649146028,0.06378695588779656,0.06931044578899459,0.06096819027127069,0.05972170906338076,0.04916834891818039,0.06571064283801314,0.06518074128050157,0.054708447841475126,0.052873565404389485,0.07022855754146384,0.06942471032451708,0.05897046174722445,0.07184915162321989,0.06144713988655845,0.05455806647396202,0.06382075052002514,0.06032043491306481,0.07355716575779972,0.06895223432368364,0.06357778650853285,0.06772548913663304,0.06225427880589664,0.06692603141112838,0.07175781690001462,0.07178043891195016,0.06488004829820071,0.0612303514586984,0.06669485520655392,0.05503056513025362,0.06574981002721648,0.0703249716803228,0.06005205081641628,0.0606549255513145,0.0679450228978555,0.06361460153479624,0.06503839565833902,0.0637136756798222,0.07168215678817794,0.06339893891259965,0.06011363962032708,0.06569063869063413,0.05889045983150548,0.06532348927508949,0.058727127806945535,0.06838001621400237,0.0752525234449964,0.06616944600477775,0.05600536995527578,0.06908804456418007,0.06085441919159973,0.06340501344982144,0.06700343361551853,0.07076241005272901,0.0684635324335162,0.06155935053910048,0.06218378115318132,0.07385631269468527,0.07239856384720131,0.06636854790542043,0.06993323327291956,0.06049848814087329,0.06688873685702909,0.06384026679639743,0.07429500694091434,0.06696904682723104,0.07136493948778067,0.05581210217377725,0.05767060361598712,0.06289626927132197,0.06883900666022072,0.06117260173643117,0.06696316359762719,0.06411696927114234,0.06445996823031676,0.06599806274625905,0.06820507529695169,0.05418976733982422,0.057700795060869545,0.06900029668337687,0.05863967442077068,0.05603304246842173,0.062378923783095674,0.06788702284973605,0.07080849856214848,0.07335213791930706,0.06362899900927185,0.05814671347884234,0.07127279227198027,0.06607841882045362,0.05370630439962856,0.06286610029191231,0.0684435591376168,0.05826836531474627,0.05613364597294873,0.07183529550309017,0.05639706896032279,0.06845918936268636,0.0655210747825602,0.06440375084611008,0.06780602609304026,0.04768969678860998,0.06352118860893245,0.05790088255797393,0.06250286095263713,0.05974836949783659,0.05543675813684927,0.059809147701226514,0.06912391487247443,0.0652428346172991,0.060404681632097376,0.0591946861782916,0.0692877342642374,0.06461541111031627,0.06218657837471544,0.06329997044432037,0.058878703858268946,0.06507553685902416,0.07430704359629958,0.05751545482943696,0.06482320186521606,0.07415388487899788,0.06707344875247016,0.06741817477771142,0.061983875590770836,0.062013024960648554,0.05739380254379334,0.06668810086146906,0.058615762994433454,0.066584550937351,0.06756300110614846,0.06585071888551422,0.06325457247847535,0.07262577389487188,0.05841581021124864,0.067712751273593,0.06360229987786906,0.057071637167708175,0.07037224011078541,0.060343932413777016,0.06668240872707415,0.05739242377334909,0.05363271092212171,0.07191184953286424,0.059416964818010655,0.04609765938265853,0.06069409840267799,0.06331751196279711,0.05064803352761015,0.05909312695738579,0.06644509876594201,0.0708875318201505,0.07204370449732622,0.06003312856345394,0.06384737827647335,0.06878355189866926,0.05931268819790179,0.05693785150952161,0.05439234972946441,0.06985413964504772,0.05632430889724141,0.06245368996593978,0.06949367640109569,0.06898730688524848,0.06362753780416355,0.06520516946621177,0.05755396961574402,0.06434666535458933,0.0680693799622254,0.062468718642034037,0.061700010522336556,0.055513190947213756,0.06046939511132947,0.0680184480783114,0.056566713715652195,0.06087848410389474,0.05954912695944573,0.06262682782110147,0.055327130220977586,0.06785475278521816,0.07142885102274615,0.06726210335923727,0.05440192874561287,0.07084822197937284,0.07571579225120112,0.058534416585575205,0.0688837115616539,0.0647712361178317,0.06766337208774065,0.05747870200623992,0.07358454336916767,0.049449998780659546,0.06947412011302348,0.06889464349375256,0.07285605210633231,0.06471196137750268,0.06296605374244435,0.05973807965205714,0.05417836121224012,0.0712270071162901,0.06452555483429333,0.06405532078250945,0.06299166012104618,0.057453643746647944,0.06684089111338144,0.04962597566034289,0.06571900138036223,0.06306981893249988,0.05468312279576392,0.047541720686941846,0.051951307921532214,0.06700989724516093,0.0672840138809252,0.06968735104771265,0.06546694849028092,0.06289637984488977,0.05404802848295455,0.05174833967570455,0.0577062493354704,0.0614891016584593,0.06727696907085316,0.05925584278777697,0.059910824074481735,0.05355858987811049,0.07528320444321931,0.07044149108277806,0.06162856460888282,0.06526352445146848,0.07259781333124125,0.0745856584599097,0.05807459790269073,0.0662002556639244,0.0653419200277677,0.05354786223278292,0.06399177049217875,0.05321722877220325,0.07272090528818684,0.0649604286459406,0.06515749595963974,0.06714887874509745,0.05962784710928856,0.06273453607541729,0.058932074424052475,0.05183817558133812,0.060936392241212264,0.062090818219502665,0.06033988232612132,0.0582464185740263,0.07463611688405011,0.061923528908399084,0.07207847629064199,0.06906051063421004,0.06896414243082793,0.05731165838648265,0.06228993411756778,0.06538429262738304,0.06289100911675893,0.05969296012278619,0.06488286115592085,0.06242418279733577,0.0582941820887886,0.05947662333577855,0.06688622803156713,0.07046332382248693,0.07166315809709092,0.06099682319566099,0.06932976781788337,0.07005598141384915,0.06000917143287293,0.0680857641825654,0.06400952235048657,0.07041425579337782,0.06799531227169858,0.06485739622119595,0.048100146668983476,0.06790372867168681,0.06670873793794524,0.06651277324876788,0.07018710681553225,0.06453317271457458,0.07283473314182902,0.05929100112107469,0.05971373729930955,0.056103500413452606,0.0643320748089648,0.05623475487914774,0.057480590678587073,0.06506704387338394,0.05510151004143744,0.06580735085069511,0.06371381783603897,0.06635294463908804,0.06206214713137919,0.06983109519676048,0.06534961086603988,0.06642093564549725,0.06507091760919584,0.05535347708025995,0.0623402988885886,0.07263048696521213,0.06086911324431575,0.07013767307187624,0.06614920397623589,0.06911141661216942,0.06531231802711893,0.06788018262689283,0.05102048756830785,0.06713954883946899,0.06746264497289245,0.05488388448331897,0.07094580052286323,0.06396872691606448,0.07300574770667649,0.061681968364402866,0.061963494485582284,0.06029534562361338,0.06553714637132119,0.06942603021677651,0.06776807550731308,0.07208218511886061,0.06455547164216671,0.06551238569217062,0.062249666252578044,0.0660499694001349,0.057775188196991734,0.07407246627463382,0.056211601116345544,0.06420870969702874,0.06650613549432352,0.07100061916500344,0.06595602540325443,0.06778766009222705,0.06823938201195626,0.0607447292545186,0.07063156154757061,0.05753993923144034,0.0629630863651919,0.065598620127926,0.06178463447174742,0.06198664757856828,0.06190702086864708,0.06035963071373321,0.0530328981895653,0.060208614451556654,0.0600576019131538,0.06820079543737467,0.06480215635759817,0.06348136333899991,0.06945822900991666,0.0518474706207606,0.0660999715483257,0.05746952887944323,0.07195293592214444,0.07189574523696579,0.06628651926204743,0.052920991383604295,0.06819237313830312,0.07036568770537541,0.05722072147161134,0.0506640175949241,0.05877539990800583,0.06568548875746348,0.06388969299253654,0.05488186006854551,0.0629753576780262,0.07348192372659207,0.070286002493237,0.07285272608986709,0.061246028780030284,0.06973993067936315,0.06534842209174008,0.06851748294110063,0.07683712636501813,0.06619863946870343,0.06294548930485128,0.06357415947927178,0.07333968112683462,0.06511458731264937,0.059980439707282236,0.06484213761863164,0.06285268453798673,0.06995316349793693,0.06886889905316454,0.07245344203072632,0.06753214942785934,0.06304493505518494,0.06947476274899188,0.07105289120299463,0.0662771473048116,0.07037496961793453,0.060584544072481636,0.06121429804173387,0.06299913930908896,0.06798190200446039,0.05832001005279987,0.06419451059449396,0.06870628239060217,0.06560212507087546,0.05765841319167132,0.06505334125109713,0.06221994009362668,0.06132057965445906,0.07315737056911478,0.06796078357134389,0.06659778630148733,0.06798914475171763,0.06675415463676615,0.058786891061536596,0.06242906335212435,0.06222494815125959,0.07078333051205585,0.07294545587690028,0.0678325640019421,0.059308480996596476,0.06747538649673977,0.06223270014667104,0.06041707810270628,0.06333267144614402,0.058947475955691615,0.0567973268914733,0.059690706024063095,0.06720787054977923,0.06700211610927183,0.062454353717071524,0.055802765971824826,0.06997435933476939,0.06328478840742773,0.0617368187317741,0.06584070690810198,0.06653849784426355,0.06382095633712895,0.06840627261597049,0.06642291367649211,0.07026014902488059,0.06257024108650647,0.06765443537467933,0.06141863276643323,0.06598373227711035,0.05815525177115408,0.05885823953719778,0.0726898442724716,0.06288980343871238,0.06292702004623518,0.06632796691609244,0.05785441632119446,0.06387423060018826,0.05874929902941571,0.05757327005776817,0.05242656105607559,0.055553238243112535,0.0646164007508113,0.046770782747296735,0.060533752547744665,0.05452440102872845,0.05725345408523223,0.06789095883894118,0.06013862989039492,0.06270346500298535,0.06716598972927204,0.06151123576859556,0.058372456467785216,0.06756797956450958,0.07745296058165761,0.07087058399328725,0.06218321285275549,0.0499894105787157,0.06727240520128175,0.05678496157549506,0.05886728181989219,0.07294065212375059,0.061286298368177776,0.06374100432875562,0.0731932830114991,0.07015143977769561,0.051049116331812125,0.061448063061771974,0.04849113390383003,0.06391527365192187,0.0674487483426718,0.06752688646153275,0.06945255491200364,0.06945000400541249,0.07005358940043158,0.07143182995917635,0.06040119033116556,0.06366225721170646,0.06833797360675084,0.05439850010877687,0.06817243214639293,0.054214860946672175,0.07212905664639294,0.06971147912165922,0.053352254337544286,0.04964535636936962,0.061913177951802456,0.06679300428854032,0.048471134554268616,0.06571791506720176,0.05922140169611802,0.060500169871175076,0.06819059602545025,0.06183642335443933,0.06621006186146075,0.06825647998901077,0.06551927220004289,0.07131596222059494,0.06597816072185558,0.06120226959918919,0.07086414753457354,0.065536895746793,0.06358508035179991,0.06656746519824125,0.057198341904246885,0.05482550684712499,0.06267754363118336,0.059217125938963416,0.0655661923906189,0.06893211313328496,0.06573314064765573,0.061355481637864126,0.06654066204647384,0.06933332031116157,0.05337496826907957,0.07161023898617674,0.06124495561733016,0.05480989475471658,0.0629577980191556,0.06850150649008924,0.05601341152440152,0.04853346361900267,0.07127947554878222,0.06664909975486277,0.06574317606309321,0.059815802168150675,0.06935724462599321,0.055952684840039346,0.06036735307095588,0.06873159258530642,0.054974526212248825,0.06486729080824312,0.06168419206365313,0.06891977652137393,0.057015780950034484,0.06468022691257647,0.057697719542322924,0.07151940146171247,0.05378986550391597,0.05686617779602142,0.05738162091878013,0.06024261628200597,0.059171496856782024,0.05735251642939253,0.06031159516359118,0.06630936404207052,0.06781491256305097,0.055874542776245574,0.059913915813357546,0.05156624379778907,0.05889986191383702,0.06479342700247967,0.0718590939579396,0.06262183743532107,0.06907583773596795,0.07156152866322836,0.04984585777868971,0.06610869298422065,0.06478386907452927,0.06845763326198465,0.05807073799405424,0.059938146340876464,0.06407833108854874,0.06749191652209074,0.060024138857862924,0.06254914640126821,0.056765378065103234,0.06913948372197831,0.0666241292873013,0.06935421721405838,0.06118429230124634,0.07434579764595749,0.07204074055251228,0.06595506808690704,0.0553713843592264,0.06376751245679019,0.06729695760613699,0.06739204884745531,0.0593016609511068,0.062313840670585985,0.07011677030408747,0.04998976120372149,0.06425564840551194,0.06376193789957804,0.05854355678333527,0.06801738285143494,0.07383652643631808,0.06679891178148503,0.05677935136240363,0.05906253294395618,0.06468239442553927,0.07037517579737665,0.06839279886618618,0.05465923012093103,0.07237656162200146,0.06706362256132173,0.06713956634058581,0.06419914411286978,0.066666027956235,0.07036953584830882,0.05639304670995131,0.05471900755275569,0.07114485726908235,0.06921011740839825,0.07212625733995442,0.07454736769048989,0.059133192658779615,0.060735435648081065,0.06847364526083108,0.07266081158524934,0.06359371103555019,0.05911606950328538,0.0564603030406942,0.06436066785977168,0.059255865815358,0.06824935329340184,0.06274510232620853,0.07193843974792964,0.05751390586292503,0.061500493132055266,0.06032230764836029,0.06945879726129742,0.06541587263951851,0.05769399570119274,0.05514973279661828,0.06655427383139656,0.062121592616097274,0.06224887234630951,0.05792476951143591,0.06855271153502862,0.056467417846992385,0.06905495323898939,0.07271971616646186,0.05556392512670964,0.06369325620321253,0.0557543561147445,0.055040587154391626,0.07247316096764489,0.06497635805694127,0.07647625913591558,0.0687970505939317,0.06186404959595012,0.07425263511876345,0.06469507874604645,0.056004134693113675,0.05564657096636745,0.07149925563753301,0.07282508002971765,0.06898309241085052,0.06452381421124109,0.050098215357907164,0.05507046558086892,0.0698575923673515,0.0696107436517468,0.05758921777745887,0.07474576140239811,0.0716668792284321,0.06722542326939082,0.06438098439216859,0.055930132750791645,0.05482551464609186,0.06240672952467934,0.064312760717172,0.06124052606699876,0.0656805397426514,0.06205462520877245,0.06370348189751232,0.06994868764847688,0.06542524642819122,0.06842924865486019,0.06055096157584004,0.05887481257954709,0.052223889885893444,0.07333177353584062,0.06832213947519461,0.06421413429775882,0.07382676647888757,0.059124903041271666,0.05865052654495442,0.06331610129591343,0.06756239327904633,0.07045584280508053,0.05671065338439302,0.06706379594088877,0.06426682555967475,0.07368093143698909,0.07269092747272382,0.06592082443296704,0.060037280576014845,0.056988788435681594,0.06951281550096275,0.055052629129241096,0.07107922817966145,0.05928554407460014,0.06124834328429815,0.0707057330259972,0.06785061919299226,0.07006140050106663,0.06086670928391044,0.06308277879108404,0.05282573227788572,0.06782684093560476,0.05340734364939122,0.056831217887898536,0.06957273218193302,0.06929862320019811,0.07033106637846243,0.06104204259485267,0.06406923774193213,0.06946423420504524,0.06413207629731785,0.06364529408184821,0.06718844087962918,0.06320081045443607,0.059115286287890895,0.06049920935916737,0.06298479516841038,0.06702333281215561,0.053873728933987716,0.04907301893486144,0.05573963811324603,0.05808531201442858,0.06366399341557666,0.06802820151425651,0.06858578819001615,0.048100540790541615,0.06681118247085535,0.061081181391038714,0.05949511497552659,0.06480503295204729,0.07655349319022747,0.07376373605609957,0.06893373916841337,0.06466958607328924,0.06838311527927537,0.05802810997773561,0.06601793244824955,0.06255367168775744,0.05887539026457654,0.05717062513985566,0.06591134247011066,0.061053576578673344,0.0647817237299686,0.06825702163405054,0.05940344854215693,0.06812655554569289,0.07567774905492297,0.0616632076012457,0.06196951845332922,0.062017824848307576,0.06954190530053997,0.05909521783622253,0.05295266650874951,0.05508248123019811,0.06649101349656686,0.06477464873116963,0.057074022214524386,0.06626016173544572,0.054564972816680726,0.06385276493965437,0.06888050281192237,0.04942993968379314,0.05904024339480194,0.061502333066875,0.05495350599232149,0.05779848382599063,0.0585571770472825,0.06951982956195363,0.060470892811878146,0.0684911464677711,0.06705991916677673,0.057492810952852615,0.06299744590373277,0.05683020327013899,0.06434869570784586,0.06327625760933395,0.05976464458625511,0.05478566187227853,0.050940581754625625,0.05158746685800468,0.05304263903499483,0.05957227158688214,0.06888317941216882,0.06986480671962575,0.05833008888219014,0.06202473019416387,0.06938622607591227,0.06018717944862109,0.06713704712163587,0.061352190763479995,0.07055216218709462,0.06648455640723115,0.07166361816739328,0.0667126838384296,0.06352142360713098,0.06443460993030423,0.06822049012300038,0.06254649903611093,0.07330311395404013,0.07079233783759495,0.06490580487047659,0.06971395141954262,0.05719337499000111,0.06828492486187915,0.06040066172318538,0.06294835449918057,0.06952783880317497,0.06986635037902003,0.05219098868253671,0.06946927734524294,0.05451516748272714,0.06661602687081145,0.0633266940203786,0.05709382436219984,0.05815817902859469,0.0611297690142911,0.06561448326648597,0.051257732047127004,0.053354401002364535,0.058409960772020694,0.06851103830358225,0.06673542310581329,0.047569374308339726,0.0699190892464061,0.06766593025691241,0.0484703527175075,0.06551581267922364,0.05645179793732905,0.05314531057921398,0.06764248162562259,0.06385018790750199,0.06385685090717769,0.06763617004695596,0.06664434446127673,0.046027770518526856,0.07161276684018414,0.056934794798530704,0.05726120616376657,0.07207275341443518,0.05642403903179714,0.07125100235765974,0.07247586167623135,0.06368152182769168,0.0620416235881805,0.06479495723595073,0.07163751925598247,0.07198766560218695,0.05767859010253202,0.07294551236427171,0.05862425191212726,0.0704883749427826,0.0640583953541437,0.06185857001006637,0.06156396804371838,0.060581829525287345,0.051136668002760434,0.058312677656429626,0.06767018369360447,0.06329684679132447,0.06167718297066125,0.054182478591707814,0.0764415655134128,0.057511399413456066,0.05829923367242412,0.06356875065273379,0.07276785384354126,0.06834442819102754,0.06256091361475338,0.06955509778907298,0.06309140292745347,0.06287343638986613,0.06899494948152518,0.06688937831388062,0.06321285871786406,0.06340096332106421,0.06335475690041517,0.05502144465370734,0.06497707168751288,0.056292867085183586,0.06609125716984007,0.07156755059475055,0.06446724422869918,0.06912473257478453,0.07120078732264915,0.056934894798705935,0.05601068992216913,0.06447881624782995,0.06413783399935813,0.05761844920096346,0.068544299562493,0.06262053799994768,0.06906336366997581,0.07280633524544719,0.07105383443372025,0.06694692091319063,0.06733133298769425,0.06874774849210089,0.05088174446006764,0.07091352252358385,0.07311554936171427,0.06405609971492815,0.06760076851650645,0.059721180678939424,0.06944942498849385,0.06838552017466784,0.06002983754489609,0.07206223572822001,0.06456123240702545,0.05219702608876305,0.06972447027848433,0.06115587668922012,0.05895242711913335,0.07303971361151851,0.056846815565832584,0.06630896523949006,0.059216011179440975,0.06359551288038451,0.05764886760439718,0.07256843881952629,0.06451910175314948,0.05985963983597795,0.062190530245033286,0.049173543419603755,0.06533850577916153,0.060233056598594154,0.04688713004736631,0.06384511430640219,0.0621633518662779,0.06259666472667053,0.060144013277991196,0.06161877760258621,0.05253169384197772,0.0584824242343132,0.06956069671105594,0.06191106456159691,0.05110399525332003,0.06752670405676969,0.06574823343261202,0.06361464866545631,0.05778831347662641,0.0673666408493423,0.06844202901543173,0.05543612738218899,0.05216512129527949,0.05385041054630425,0.04988442346602789,0.06894886297976316,0.060917986624231785,0.05980512236936987,0.07092383398018044,0.059401856992169134,0.0709703260935569,0.055771217981209195,0.06761696027078257,0.055847253600984734,0.0648240669632672,0.07299247436881445,0.06956404208730183,0.05878826104258074,0.051995670773608414,0.06669779885280223,0.053000694845626264,0.058608680745867975,0.06544125458130218,0.06665698637037598,0.05683199133241907,0.05938654293206071,0.06328614193315715,0.06716587264252125,0.06334930084896308,0.0595504580014692,0.05741013119931353,0.06317665178216704,0.06903681603138223,0.050321572923662766,0.06449590708002503,0.06184223235445505,0.07232599336412274,0.06574058148256769,0.06916526252202183,0.067307528435908,0.05423414407480494,0.054935775720503774,0.060895753900289874,0.06029531394905805,0.06842525291385179,0.059594016058520806,0.05219327821911462,0.06252968620669627,0.062341675888399055,0.061916783356342035,0.060779991723021516,0.06966167988450504,0.06993931113376209,0.06135964912577291,0.06447915729561943,0.06563448811676403,0.05928978391664431,0.06286448994992125,0.05459246991356838,0.06464352173951299,0.0704196352205749,0.05329916521028859,0.06526714829108246,0.0589061589753672,0.06746729988557054,0.06466849895471499,0.06322331499182583,0.0517420491824836,0.05598221792700618,0.06382544335445517,0.059977437638466205,0.061211493014824495,0.06452590363005911,0.06263786686992484,0.05994922997021815,0.056880803525857615,0.06712907311186855,0.06338655844550814,0.06723760159154424,0.059248075334865646,0.05963399601653921,0.07197145541377689,0.06395627250139442,0.06682322942290889,0.06167134963477893,0.06804141562586677,0.06696200844899336,0.06418514953394316,0.06310609698141481,0.053187810214538306,0.06760933038512656,0.054244865713757515,0.06517987652430682,0.06200427486166029,0.06417710230980261,0.07001817309185246,0.07208661732283203,0.07111880553392272,0.05952786355176545,0.04776985325389917,0.053626783012554496,0.06834946016795204,0.05738295833914106,0.06539193164094281,0.06814569642152096,0.0638994519410265,0.06112685292478324,0.07539399062861782,0.06198725166851335,0.0701126503009229,0.06380632880740328,0.05979702902149202,0.07001634126693923,0.04866732420298625,0.06958900283512574,0.06719820232252513,0.06771278497145007,0.06670544614131854,0.06305336267457945,0.06253681539452305,0.05641659685969458,0.06152316763491539,0.0602621550683413,0.0638738626897093,0.0610949143154676,0.06883351532893622,0.07019915091993326,0.06108726189199984,0.06414998279760162,0.058128048514358384,0.06576856816119374,0.06542877868923105,0.06401780959767639,0.055907958967769705,0.06867948713228939,0.06747309892489403,0.06625455190197593,0.06512582514754389,0.06091309439540238,0.07716292036518976,0.062073802128377785,0.06738445797274922,0.05508896377593765,0.06613138448089023,0.06375352154961658,0.058521335148782355,0.07224554877322147,0.06262132613936255,0.06925992750584525,0.06683980902099709,0.06146145825421636,0.05340473019179588,0.06802310479910931,0.07004935902091261,0.057882973234047105,0.06555737107997611,0.059810577812623625,0.06513962712843266,0.06407624956497898,0.04803025875634721,0.049374452970352437,0.06540447178770901,0.0570249183139961,0.068478553270927,0.065838167818377,0.06455277957377858,0.055342219523801556,0.058675638005554864,0.06289166520575304,0.05931690972798365,0.06065511312518203,0.06229764560837808,0.049434631392823,0.06647662970251209,0.06078988763174617,0.06688545348108779,0.0667118409787319,0.06428753468966289,0.06319219890343496,0.05994073969970671,0.06184811579519226,0.05896726646760959,0.0606173881118369,0.07121403969219509,0.05612579967134862,0.06316905287335638,0.06656090352036448,0.06380593079901462,0.07725588976337011,0.05512970449173947,0.06908065785917092,0.05139224296396458,0.061664242812139676,0.06941952603733788,0.06255362059577485,0.0562811352230206,0.0706362005202125,0.06658931414960303,0.060254378937113426,0.06292261527214177,0.06937712599273244,0.05311447384278351,0.06383957149733734,0.06824001721276456,0.05948696216710993,0.049629093820450214,0.06579807180335706,0.06677443930753856,0.06855761860823298,0.06012677237564188,0.059176336796952785,0.07201438700379739,0.05953628792842196,0.05459809658294654,0.059488883654857924,0.07072202279580876,0.07448629628292125,0.06406059379655757,0.06069731942907864,0.06456163806384692,0.05523219228970257,0.06865719895154004,0.06679621355466044,0.058124215366588364,0.061999165630993315,0.06239102286001567,0.05571657510412871,0.058604652762808115,0.06275600041149755,0.06905284537477624,0.06469521832184724,0.050529739586762996,0.057373089426592624,0.05134192841266039,0.05759604347172374,0.06632129854193777,0.06557599530469341,0.05362762076775916,0.058807488410659414,0.06417375796534913,0.06804575254031356,0.06649455541425794,0.06298873478991317,0.07202442642281917,0.05492847024383644,0.05962797269791224,0.049318372313300024,0.06523831799669234,0.05965543450104148,0.06159001675704907,0.06555646121658854,0.07150673839928487,0.04932947279492625,0.060827687576289244,0.058853468631747265,0.07483888740319211,0.06046399419839096,0.06266912888502008,0.06112726569310527,0.06959312405042098,0.06629992311730898,0.06881496778753665,0.07126722850284714,0.06478387117494723,0.057559463620040205,0.055437254108761665,0.06321957192164697,0.06406286376472214,0.06065260513501447,0.06212112047289541,0.059162179868530645,0.06223642892276523,0.05165336769701086,0.06163456726957814,0.0696865322973492,0.07334158967424231,0.06250366013267211,0.06800070117061457,0.06769242974077146,0.057227804419297645,0.0663781245491735,0.07023412717249958,0.06828144082061968,0.052324906099642174,0.06688266829505196,0.06717676139084039,0.07020302481729106,0.07456689185027696,0.07270267069148259,0.05472716732153922,0.051336536880045705,0.050293018212750895,0.07173849655895981,0.06906958984630196,0.06622400050519561,0.06840345670543616,0.06420847844330238,0.05615234936493077,0.0608919216334593,0.06443797880554725,0.0661933563869993,0.0625743957194215,0.06409037013410845,0.061452832426078255,0.0671331064333148,0.06771592998404416,0.06772552757579087,0.055942809534731915,0.06318573996727907,0.05593875347684185,0.06103910152558615,0.0697058648581051,0.06198145696160117,0.058597066836152525,0.05776421864592508,0.06543510923028986,0.04927675914005752,0.06120622858630864,0.0580850959866058,0.06132648454620391,0.06127134721585172,0.06243475839217676,0.06925151482957598,0.05921025217978424,0.0501093445149373,0.06258969829766402,0.06528302698272867,0.06395141057530095,0.06397808617360015,0.06442803372352936,0.06546365471980164,0.0698045810194784,0.05644567862295914,0.07660593998953091,0.050206846001192657,0.06971352936601441,0.06540091720943769,0.06043576121536387,0.06829144040342672,0.054172875198550005,0.05260372573611967,0.057605118331229097,0.06835278337675296,0.05884416013788131,0.05335629937809343,0.06375555800129704,0.06305347092688546,0.06455204433758485,0.07444792669310041,0.06701520157606719,0.06059854377454371,0.0608777911430327,0.07026246325790143,0.06202186724167975,0.06949650670273846,0.05710460798971753,0.06307844323244902,0.07344096566289607,0.05714133178378969,0.058769197863372126,0.07623252585743187,0.07562876684647152,0.05282401586266,0.07184432251266135,0.06449640257590898,0.06825750444386511,0.07280948544982443,0.06493642518456622,0.0685428214764144,0.05744214469551388,0.06483585372421073,0.06700320925943154,0.06872167822943923,0.06572507482751827,0.07217176182943472,0.0685139434505361,0.05682262339558952,0.06384267979780558,0.07594160883549594,0.0656379262339532,0.05538830376858683,0.05676733619422765,0.0613372123407652,0.06647702076426862,0.05291050046330096,0.06436922741130532,0.063944280959843,0.06291483880592533,0.06723633128739229,0.061618217305353766,0.06682610434787123,0.05701035170066121,0.0598244283902334,0.0645573175247618,0.05745196017654294,0.06180992574700272,0.05409166910352741,0.071132468981611,0.06774793891135347,0.06424274838103705,0.06006795445305867,0.07184312874848976,0.06132813773293124,0.06472268944080375,0.06283641455378104,0.0469485180104746,0.06132401288794253,0.07169532858977204,0.05590881475241239,0.06629421467186328,0.07434974797362875,0.058995575388656636,0.0635316761430767,0.07506655776235704,0.05627027572939078,0.05729149396777432,0.06241759289755755,0.07548267712105772,0.06804089422284451,0.05815290351474538,0.05747443008904163,0.0658080382749232,0.055909339583587944,0.07092420923222326,0.07157573960753419,0.05885895356519567,0.06315142305060063,0.07279679358121022,0.058852007283851884,0.05628222019796778,0.05801370718996758,0.06506228153068258,0.0667512900739409,0.051982344954029785,0.06310759004648395,0.05418675763684805,0.05775685999327203,0.0720471660125533,0.07277410760566747,0.06407432959266363,0.06992850663297676,0.07246644927137205,0.0669592685374106,0.06705073958613932,0.06738156591064695,0.07634223174268248,0.06888739320968229,0.06541202148391287,0.06070625652458099,0.06082368952074414,0.07305449142940174,0.06845196965144096,0.062681514323253,0.07190214147087792,0.0604955461468146,0.05867713221459135,0.052670683040955166,0.05831008488496814,0.05948987088437652,0.06145713327940765,0.06224521350188911,0.06457610514600824,0.0617677165347148,0.05298923793547802,0.06722844372122425,0.057892489130920474,0.06139690885632554,0.061297505456161416,0.07215764176753987,0.06859147160958047,0.06452598683623612,0.06001126084931214,0.06783037655810284,0.05462521911107168,0.061737060643732855,0.06305678395909742,0.06870138642964288,0.06530862279062113,0.06566533277040595,0.056859644263437634,0.0588011316069076,0.05204629656636673,0.0699894865251405,0.07288063619950293,0.07146124929253805,0.05762705087167675,0.0681272381921182,0.05775311356733232,0.06485107872725317,0.06885894524529183,0.0712188662355551,0.06711529286414192,0.06410481222312137,0.06872623697678992,0.06276705523713216,0.06131835574412753,0.06738498807799775,0.06191757316173403,0.0649431478648376,0.054338338701881615,0.06808813149487808,0.06541262295304026,0.06562152482740563,0.05463223338865585,0.06427835076080979,0.06874555098800057,0.06408955208681846,0.06856892136319669,0.07124047120142521,0.06221051101198027,0.06761593137810931,0.07016826928328918,0.06321006460702089,0.06357656977024394,0.07092253351974209,0.06706157064997736,0.06397016899811427,0.06572809353527914,0.07068440525348518,0.06308528206168074,0.07214767013692291,0.06682933601832027,0.054491296578762796,0.06575071127488599,0.06180494247844095,0.06307611362174402,0.052684175153177915,0.0653681666419981,0.053510478807719065,0.059899448070683134,0.05999699068603257,0.06087872572216955,0.06851082563475659,0.06946839759253928,0.053837886408460364,0.07000110196311023,0.06676040669280173,0.06144514691710309,0.06189135487725637,0.06345214494899173,0.07393685724792926,0.06068982674431683,0.06481793767242813,0.04980422934111296,0.06440388583169632,0.05774661511382318,0.05511770555288498,0.05967735886521345,0.053986000815922114,0.05140479897074392,0.05529862503041809,0.07382462840029153,0.06090707823678561,0.060175362377104365,0.0688428584378182,0.06913137811671552,0.06183665769857143,0.06925718136126427,0.057839717739915406,0.049607946917325375,0.06627757074606352,0.050737731636860406,0.06838952278034838,0.058309207672132796,0.06435158114366514,0.05933995345015605,0.04782724810574479,0.0639917488543904,0.05969145645400732,0.07121803818616079,0.05090415332849109,0.06437701096599002,0.06776423808630161,0.06604608415611879,0.07283405406043103,0.06332647951369146,0.05192293271578344,0.0696204579136552,0.06242567154739975,0.062205762206209235,0.06402388779458759,0.06307435315671056,0.06033521481689488,0.06157247687316736,0.06309425443495591,0.05352667870919367,0.07091257957905725,0.06822274636120751,0.05993466296925091,0.06219773168409457,0.05945813548342799,0.05427225451901434,0.05220524192269467,0.06920872882770748,0.05866208399426147,0.06999933257584992,0.07174509177271396,0.06821014199207465,0.06308151409480833,0.06337019917413489,0.06096891563539267,0.057747265036735634,0.06095514319392337,0.05566254613793292,0.061345821903865276,0.0642935861589281,0.06656945664486766,0.05547390356360496,0.06463873362181302,0.06512173258730017,0.06473782530921904,0.06661656079093206,0.05985017526568566,0.07233291240435716,0.06707705047524187,0.051239880019066736,0.06759222575305991,0.06629701221450043,0.06282438938933427,0.055256584416272944,0.07165453887500534,0.06680253268498443,0.062072631394855186,0.05945797210309439,0.06774976872175298,0.06339533978127451,0.06426217152082175,0.06396964487761461,0.04975406072356174,0.07667136114800074,0.060693286686551065,0.05846196788948989,0.06829416181011391,0.0623999715622519,0.05782138230831323,0.055420340940571425,0.056137898246772885,0.06617709038482913,0.0625678479039089,0.07174622380855061,0.06660000977630723,0.0630641063594487,0.06701347495626572,0.062050250310838294,0.06013549625410179,0.06766908258317733,0.061599954889645925,0.057726087427724844,0.0650644822969114,0.06347983890619677,0.053774601630859675,0.055610792617864294,0.0584725692461536,0.0730339768315112,0.06251504742635947,0.06386210544408422,0.06266334426015431,0.07265213321016356,0.0743518171705861,0.07072279229379516,0.06775113055512774,0.057261814657321525,0.06585659563351348,0.054675726325574936,0.06126388274778378,0.06759075761826305,0.06112504853040454,0.07055893325433976,0.05967297046885229,0.0661392101672039,0.05922912914397608,0.06706089678879343,0.05970583829101401,0.05851944300781997,0.06841264379832179,0.059911524956839796,0.06796493826186142,0.06662080790361982,0.06275349716515269,0.06288995937426434,0.06770098086428003,0.06293344518168036,0.06148400136549471,0.06622955327602989,0.053304725131733906,0.05740226069448677,0.0636451358510613,0.06909290769687429,0.06359944896537914,0.056957133989261305,0.06545923398265946,0.06687913493853843,0.07197258242530888,0.06166453751284905,0.07034065114994739,0.07025391155561808,0.06600553199368142,0.06446213591866537,0.07389369762268262,0.06479885741486016,0.0699620342172732,0.06529910111901419,0.05671237480127267,0.059615008257356994,0.06383403683408535,0.057576309180582264,0.05869876980456569,0.06384489155665138,0.07148037988554001,0.06359019004491856,0.07234925528038127,0.0664858099897042,0.05992503864916052,0.053827457762232966,0.0754099563588678,0.057766243307116175,0.06751244739270365,0.06797804810363967,0.054692262998568775,0.06709055626523572,0.06995941443474377,0.06962667865740416,0.07079627817512116,0.06031339944469845,0.06445788580627124,0.062165636843433285,0.07218388319340102,0.07267817624911269,0.06194535342259869,0.06232980752373606,0.07047144465620517,0.07339453357692591,0.06404359280241867,0.07410635500258955,0.05764516742576899,0.052935024323406674,0.05733626445774778,0.06635061161613756,0.06652084131171991,0.06846750792708922,0.06705464797111832,0.06423382233224081,0.0678769685699761,0.06256530661546043,0.06399340946458068,0.06854232026376056,0.06442742300606089,0.0729914008530935,0.06543221744931832,0.06364914502769747,0.061232693092678106,0.055824719841899836,0.060098351713113604,0.06963089456861168,0.06632239115749759,0.06395229108647965,0.06533452547156407,0.056591202592305004,0.05534354008793786,0.06763569531148068,0.06667869993112824,0.06271090631461518,0.06626840864253053,0.06654271233822952,0.0708078939775342,0.06287919016099869,0.06793050396602765,0.06541587541408334,0.059493540128532654,0.059580793826155554,0.07216680075113042,0.05437074677874375,0.06073060401984694,0.06723385907778753,0.06708252966264378,0.06598073028635222,0.06683441364914669,0.06950475212680748,0.058324432976573776,0.06523294996011193,0.06314558941594088,0.06885330537412004,0.06299167376824802,0.07191087464470791,0.06626458481232539,0.06422856443884675,0.0702173723368699,0.06682051121699997,0.04569455933317586,0.05708380230828515,0.06372251620018599,0.058917316824863425,0.06458682207103283,0.05750274462545571,0.06852928067246211,0.06761080369248568,0.057728739984172064,0.0609529524900071,0.056422719340624386,0.0778913956646222,0.06269463182049874,0.06639165987596321,0.07550611446979158,0.05629345628942824,0.05330667592069767,0.06594871750830827,0.06800387710444158,0.05295891523723549,0.051390949303780395,0.052365847950465935,0.05998765070618253,0.06482851293342143,0.0624020960234763,0.06321453052480991,0.067503929638419,0.07000505057451381,0.06692653743503443,0.05572751560254797,0.06104129212057387,0.0652421296965621,0.06890675765465396,0.062345125239711634,0.06342922698242084,0.06215472198885083,0.06685501163228558,0.05759800270629416,0.07016130897529545,0.057703168019729675,0.06298192552267583,0.06181900716606337,0.05979513110215681,0.06998464495697748,0.06543239201830908,0.06897173275295276,0.06761370591867219,0.06342644273731068,0.07001147741116182,0.06792574512024885,0.07040740609116823,0.06856559273628056,0.05946090667086901,0.04669139729289597,0.062053839145262454,0.06244355913424042,0.05278505408479309,0.06956104627032285,0.05552003891517411,0.07085670374737502,0.058950865232819394,0.059076343638639645,0.06410333485984528,0.06940417605074464,0.055215658181958796,0.06099989367092143,0.062399385390962674,0.057376651838616594,0.06963273303624362,0.06297324952799567,0.06705420982790086,0.06576722485795494,0.06604839093412426,0.06672615533854818,0.06785180629406484,0.07115047508390182,0.06846248924768306,0.06198183300506573,0.0630306294061776,0.05816166687771175,0.06554392751824131,0.058550832015215395,0.05590706852632987,0.061955298535819936,0.06708870749308288,0.05490288592008566,0.06058607492025572,0.061644296431804184,0.06538897506640615,0.06965832070015343,0.06986162772911694,0.05640696566862913,0.0587729958684409,0.0690949537178473,0.057978223709153606,0.06287272826135745,0.06252363393616377,0.06403116489822346,0.06479969396951324,0.07461277251657389,0.06812606144437539,0.05611208360756094,0.06585035350376423,0.05460160817179619,0.07087559791768214,0.07028867212863985,0.06162654259311789,0.07303618100627203,0.06044637607369223,0.06551252263593621,0.07031331774437356,0.06318131658183307,0.05642609705218138,0.06127425657671284,0.05656024738231619,0.05407367168594805,0.057256753572749994,0.06085261077219803,0.06064391224404301,0.057846251331858445,0.05237473491556048,0.058857062645150046,0.05632427079340031,0.05809798725718984,0.06180972564141448,0.07191795294282236,0.06722103333501189,0.07262932210945952,0.06151562344281854,0.06297092747473815,0.062200144442669324,0.06051946166045777,0.07365144604152017,0.05545898169888609,0.0635917009178407,0.07351769797975738,0.07143400660721294,0.06920486926503464,0.06849257140341394,0.05188101231990684,0.06833652846823937,0.061681443592870616,0.07215274440867565,0.06591276655735614,0.05287865831582611,0.0598609504375079,0.054305799340566216,0.0517055874659868,0.06792965055092613,0.05708143423913439,0.07225047016927627,0.05611212673689359,0.05617009367848781,0.05793384983198975,0.05026441426388231,0.06906146091147804,0.06338852701069657,0.06818279982904962,0.0670436173025043,0.06723975066744316,0.0601048759724969,0.059471991479745676,0.05521667721059842,0.07291378547431507,0.06810421740345118,0.07090343877926861,0.06539950418488297,0.07130255239264618,0.06755626289799316,0.05027318803049803,0.0644036452240669,0.06628791881933357,0.06652088010343203,0.0628251981253379,0.06468918311219168,0.0609411070077414,0.06818101396569233,0.06807719512268207,0.060048157637840716,0.05658855839143431,0.05418376252215425,0.0589610429964572,0.0660796819440696,0.07008856748913879,0.06492079952687313,0.058626993254780796,0.058711728569933494,0.058245751218963616,0.06495714283713497,0.06589298657875929,0.07204588664921929,0.07341626876889912,0.06239877474781113,0.07000351053755147,0.06335950913121563,0.07416773595628334,0.0674731789168142,0.07108640323450326,0.06065634778270429,0.06285132439565702,0.05846598763188243,0.06163673080152719,0.06309609771420005,0.05772719541551636,0.07144053977697185,0.06837058572605453,0.06480698141606442,0.07132627997277333,0.06607873686065777,0.05970077663947094,0.06609570410966696,0.06196357974231819,0.0693966430823979,0.06301116864443917,0.06526738133082108,0.060054807781231595,0.054573503349844475,0.06505653642972534,0.0667001184143165,0.06951348777146699,0.060723243918146874,0.06703226587832337,0.054428316056460445,0.06731055595951044,0.06800828058187205,0.06412209040992228,0.060121526995002464,0.05938343575871628,0.06337316333670429,0.06460692562269343,0.06505307519075765,0.05064170937202382,0.06869762629739465,0.0599710277467687,0.06201612279211499,0.06095774446157805,0.06628844476157966,0.0627492194661447,0.06529020069697514,0.06442197185791174,0.07317590310817546,0.06514991953432406,0.0705716865945638,0.06677500114676407,0.06708267922432234,0.06779688592165922,0.06897649136178259,0.06631576568340114,0.057920866951368145,0.07232373812554521,0.0689505416232421,0.06317702839531919,0.06392203182084069,0.06293405647601649,0.0644689273954775,0.0671455377900815,0.06190875549172441,0.0649182620269747,0.06830907070337358,0.05976774885326514,0.05446721631829847,0.060516852576228165,0.05799248955835565,0.06707590014043283,0.0625016187771554,0.06865804844549397,0.06199889513671416,0.07143469729533783,0.059166461916708976,0.06743414267492223,0.06343784332779444,0.07092296127646995,0.06497942409891139,0.0644144332488981,0.06336803798506085,0.06812266313790644,0.07058657349137296,0.06210221093115401,0.055494683400650655,0.07181537555548301,0.06886431631632922,0.0648869167059418,0.07312248079155945,0.0697220121029889,0.06557281125471442,0.051889133258037,0.05858610655932915,0.058152401095468466,0.06888415982415118,0.05655228516316405,0.06746276698106127,0.055289670972927536,0.05933934945333319,0.05565344559006031,0.06910305453583915,0.06570057368299942,0.06804548721829629,0.06705393589864583,0.07303448520842225,0.06549903094735732,0.07142857721055812,0.0683272650964464,0.06774811526947014,0.05965322767407624,0.05668712986128234,0.06039232306258445,0.0608278029168026,0.0635628841013488,0.06081613912964949,0.06828584607122706,0.05875136695832342,0.06509103674587574,0.06991492371578789,0.05122107972378136,0.06628235494839108,0.05963297187774205,0.06322238431929342,0.05034773253117489,0.07126805592848917,0.06504701537252,0.06660527593953901,0.06551215792339385,0.06674113038034496,0.07505505845759199,0.06890053254124875,0.07288620180229213,0.06627435987257708,0.06999066550044064,0.06233929154650165,0.05390999389495363,0.059355384242074455,0.06984437999693682,0.06268466417465095,0.05716570909375382,0.0589975634562392,0.05924281024810059,0.055516346661815805,0.06869609391746609,0.0543459591423352,0.06587315112253453,0.06692920208454532,0.0643679688186978,0.07127389590623867,0.06507474894692838,0.053027953884031115,0.0690390460216791,0.06715380302054072,0.05756978594083471,0.06755009136269535,0.06623764534773409,0.06696599278599424,0.07059115939521836,0.061832849472765156,0.060703030044896716,0.06429213871126767,0.06132910371428188,0.06257296811860394,0.06047840117455247,0.058948808710757625,0.06834481651644823,0.06859359188321795,0.06158374081940493,0.0692751592206006,0.06092345666231967,0.0625470384569064,0.05964078889500727,0.06838022172348765,0.06959009549593081,0.05305688873291424,0.06414538459753491,0.0738348337795394,0.05979379031536855,0.05565341274331923,0.04886061892535663,0.05870739784341853,0.06306706999673023,0.06403200750600474,0.07031206415534633,0.06635758934414894,0.06060737822296078,0.0635598198362963,0.05882215905712691,0.0680011259019431,0.06652853853520943,0.05670792208511777,0.06382639999688693,0.06749838555786386,0.06579585341387531,0.06732900322974336,0.06550981076620849,0.05796136790708035,0.053171186957337066,0.06463270797082755,0.068252616906251,0.06513286606097667,0.06377994921062957,0.068987279661749,0.05382444520130397,0.05118430394276234,0.06958060313387009,0.0701717212540832,0.06508665776864708,0.07271320487197291,0.06851004751736559,0.06677363364067239,0.07072261371768407,0.06905268692681396,0.06085673234496375,0.0657879301626051,0.0685101722767734,0.07069435793629007,0.06756634230292274,0.06596582056935718,0.06976386710384852,0.06534576852736851,0.06670277312765605,0.059415270424193596,0.05327236463878876,0.06969249930333202,0.06449793444111188,0.057515659865753334,0.0630280264553366,0.06993439112347756,0.0659244709255607,0.06694168253733579,0.061245617443860825,0.061487077409359214,0.050949071013529634,0.060013969507699085,0.06073992974832387,0.07018169567287974,0.07758698924187216,0.06905568572369236,0.05982903284694149,0.05379161616600422,0.06598078770600921,0.05659317649167263,0.06688627198403468,0.06559400628441994,0.0597967576906745,0.058281633884780144,0.05334293016850215,0.05322929413874361,0.050552192579099084,0.07398138036488099,0.07641369054808388,0.06316299599351882,0.07056979480814388,0.06003168255808925,0.060371845050655695,0.06663369750066689,0.060105275741635114,0.06776932151381887,0.06970257408521258,0.059970048414240834,0.055394496844103726,0.06168103127771893,0.0598327454819597,0.05793399860282889,0.06359678059369725,0.05503631577730262,0.05467888953528245,0.05316207641054534,0.06284289413139255,0.05369141359315273,0.06260295193967083,0.05752362764482195,0.06840027026550341,0.06411862559508365,0.06896435039389615,0.07353638273288467,0.0681559520927353,0.04894395671437283,0.07642440548675689,0.06354695750732832,0.062419371609451844,0.0570173921036736,0.06152579777216049,0.06529752364781305,0.06853126752147576,0.07137533063178339,0.051128399671263855,0.06301415059405116,0.06840522963789882,0.06519723110893644,0.06259060047480985,0.060299381659684584,0.05573553655509221,0.07188185537794631,0.07277374881640077,0.06597910584038073,0.06043885124315919,0.062342948304400354,0.06646287595079708,0.0680403862620704,0.06729970157810321,0.06614073638279908,0.06205244343882055,0.0673892575616821,0.05967605928137086,0.0585592996504867,0.06173553071526402,0.06070296700703644,0.05969773207962012,0.07580183123479972,0.06483812798440439,0.06001174150766065,0.06068904370182254,0.06897127772081715,0.05662769090075015,0.06396742398450514,0.055101220897639774,0.06048340742586492,0.07023680345631164,0.06616476937096646,0.06862155284847228,0.06345095014391203,0.07120267763143556,0.0666363858752794,0.07489753600026555,0.05429261226107904,0.06570342733858464,0.06576888129637978,0.05159571284801173,0.07232363503174644,0.07137729746948714,0.06077540452823086,0.06304852629764912,0.06920432845667207,0.05324694624426406,0.07229122979788709,0.06295801499846937,0.06914939976215412,0.062259067368887,0.06985942811629745,0.06555182085367081,0.06902591460418514,0.05874025049908108,0.06762455771523485,0.05648300194315163,0.06492625746413735,0.06201663256709041,0.06810748046862067,0.05390296439709913,0.0651317942977481,0.0664193771586418,0.053098113114089074,0.0747965855586241,0.06711398063451965,0.05713135262707582,0.061643356857239755,0.06800010796452938,0.07171753708552617,0.07116689979852589,0.07379107106049733,0.06614756983440004,0.05138691825807834,0.061108119494405186,0.062235822050898226,0.062165861392539705,0.059038569070277255,0.05974725093436299,0.06912646629469874,0.060164253179148725,0.06667361026371899,0.0537857466983649,0.06195037548076279,0.07164371074504791,0.06365552842551847,0.06851605147255034,0.06594841071144024,0.06548699064253477,0.06995916518745916,0.05698952063062186,0.06868706027608384,0.06843914865970435,0.06906632163418334,0.06220738332973108,0.06133735266323919,0.060183659766306,0.06810117429686444,0.04889101367269828,0.06671905172339054,0.0597564978153189,0.05423556164699522,0.06501770113235633,0.06443694028585176,0.0639275552774686,0.07190372538437406,0.06474571839677026,0.06743668708767928,0.06434509763365551,0.07163483700987636,0.05422406049828972,0.06462473137459143,0.06763536030039519,0.0734653731215838,0.0713799073208304,0.07043782415726407,0.07546229906687775,0.06283302006221911,0.07077056310217877,0.06619548657285648,0.06436934268082745,0.04757845025678092,0.06344543778588388,0.06862213152727945,0.06764008728104842,0.05720823541892351,0.05609161430435803,0.07018460508001342,0.06587672730193503,0.06421652091549936,0.06442020606935359,0.0690678370321118,0.07033399728459598,0.06227491992019953,0.054835917705109594,0.05265447981940097,0.0592546221967933,0.05357969627475837,0.05889461261057878,0.06633852797246242,0.06900387180671122,0.05817094444231488,0.052069194247161485,0.058703935758927114,0.06297298239652971,0.05589634701815824,0.06570949955232759,0.06378441981539443,0.046224264004310005,0.073033211674649,0.060252741648303465,0.06271978405328713,0.06081777493133768,0.05966651073846063,0.07340987660214601,0.06707253804788145,0.06594863863843618,0.06286576234303923,0.06639284081099955,0.05720745159776647,0.06494958960813482,0.05991074855079967,0.06436606051591123,0.06574756546794178,0.060768457787853926,0.0506470525581343,0.05224664769921804,0.07027302112457418,0.07406525688542187,0.049701821213261874,0.06522937650577788,0.057655356612164005,0.06907129156254088,0.048993095619952354,0.06959696168858039,0.06402639417874567,0.047855511149364094,0.05370565976188677,0.06578082338754457,0.060354171873165226,0.06267683970639527,0.05800573415184905,0.0598150796843792,0.06257231138409818,0.059545358182859755,0.07325798276714476,0.05993893610320081,0.0710938541253516,0.061490358187800735,0.07031512200933597,0.0553793023688775,0.0686378042477408,0.06175973717226249,0.06288394223822269,0.062475064540709405,0.05619605172826856,0.060502979201263546,0.06490511601798332,0.05609544583240263,0.06354433905910542,0.06952536088859318,0.06494783577156522,0.04683751820473857,0.06459743668224818,0.059272348670904376,0.07315352781347541,0.05674856108661183,0.06633418132979936,0.06296836199694142,0.06542812751729438,0.049735232063896906,0.061634183099684416,0.06893432189329474,0.05393880364073697,0.05871468762135457,0.06824886057148705,0.06998202300729932,0.05536994039637139,0.06686555413615075,0.06404521387147794,0.0629121547839036,0.0646514242291829,0.05242663072767083,0.06922016909349386,0.05776875381431805,0.058773006574811916,0.0717681704007723,0.06878725662782913,0.0640709739264314,0.07422528670419487,0.06958758854985037,0.067523959574409,0.05718814152684967,0.06732789543371245,0.06358250301374807,0.06327614917643584,0.06843441430176656,0.058537857665577134,0.06802029394130851,0.06956883576156157,0.06475052007642212,0.059493139936262276,0.07014140632546768,0.058285332333430875,0.0622875415877948,0.06593847723137916,0.06046431899091405,0.0713673047127014,0.06779966630018641,0.0562997436276972,0.06265844711213372,0.05170382449818163,0.05978356345836433,0.06968062684263984,0.07257922901063599,0.07229327588036291,0.06956144201813788,0.056619889314595453,0.059594999595175766,0.05911247059565938,0.060376139162246434,0.05636834513793622,0.06898807527569462,0.0545771647211048,0.061847037253568736,0.05905003830801343,0.06423044163483849,0.05303370938654034,0.05737676646953508,0.0666765918389031,0.06053666932185295,0.06308952117103454,0.058187787502888055,0.06298988185532711,0.06277109923516858,0.0585879519912082,0.05358512669442955,0.06701257080102055,0.06530585513159791,0.05667923354186587,0.07259424194101705,0.07078031444199828,0.06741152487547679,0.062284067287984475,0.06999914375554367,0.06499277829954228,0.05239214978756217,0.05882452210733994,0.053014296839457256,0.056255710368435075,0.0657834620250772,0.054879786611080825,0.07226608868501028,0.0631194840240582,0.06543430609075572,0.06327296976759789,0.06316033547515232,0.06055215211608812,0.06224159222052703,0.06296627228423612,0.06319424188241651,0.07001538733859841,0.07158128809758374,0.06467711653883994,0.06937845419504993,0.06533908951192953,0.057961585811523775,0.05513017948434214,0.06287720231594596,0.05872322109673726,0.06636920317164104,0.0672728372041044,0.06383846963877793,0.06723045344925896,0.06768001721486155,0.06573149463498903,0.05366097372813851,0.06647317119971782,0.06308698976370379,0.05598717631937841,0.06509225806042483,0.06157297027000635,0.05793534156260245,0.05523836651443809,0.06849677998456379,0.06302518581443459,0.05742893755279877,0.06147293871943901,0.06569947383920961,0.07029560461650908,0.0637407433461364,0.04827948398529891,0.07011717512650478,0.07096616098813664,0.06974111130775021,0.0504519704602424,0.05675240041259564,0.06591524123602285,0.06701008543597878,0.06786376533389145,0.07094708081081841,0.06898616199918642,0.06656807826452889,0.052369569410555976,0.057238057852716975,0.06259663795950644,0.05190163605828382,0.06558806933045264,0.06193986676948005,0.058305324196547555,0.06046468922077204,0.07102284623342031,0.06971894627274905,0.06780313969080984,0.05690551503906141,0.06067347495726105,0.07096256332612218,0.06553463695226844,0.05529519908651159,0.06140597199103602,0.054734900167187586,0.05866867813999892,0.06844322263085248,0.05798614862061999,0.05878240430636756,0.06642353758447893,0.06490581990711713,0.05782639603058882,0.07472724664925745,0.06906950630959888,0.06939886013304475,0.07105926347681123,0.06734671990555606,0.06955650438858701,0.051887755228492136,0.05989111298918767,0.055684455278174755,0.06114825447102946,0.066285132563533,0.06158547904567628,0.06263412491391473,0.06158351191533653,0.06874534288081932,0.06582644217702452,0.0649973088982192,0.05921969007226938,0.0591408904518535,0.059282881870383095,0.07424743335356532,0.05785379158265688,0.058775370948381006,0.05703735799579452,0.05896211719826853,0.06318555413389526,0.06676022959159153,0.0737865835299786,0.06039886802819783,0.06745882427863975,0.05785403112951518,0.06157938986211871,0.06524209552064444,0.05396194169587914,0.06545493337860367,0.06374479156732868,0.07668332571916553,0.06627699224617328,0.05533653078977023,0.042505090995543934,0.061290754067376524,0.05974825164342335,0.06840933882078992,0.05352572763754194,0.05424216972685433,0.061913377114335005,0.06572467175562088,0.06263051220892868,0.049909087382803696,0.06594814343203106,0.06938271236733412,0.07043771410052743,0.06694787235694846,0.06645396111496253,0.07084442416806966,0.055851793079162784,0.061670024426298785,0.06648438444715676,0.06437255322951319,0.06466282327738343,0.06807364749626549,0.06759192011996346,0.06391056318403524,0.06391256600124715,0.06476695750087222,0.06571872913859667,0.07044985386992988,0.07441772728095844,0.07202204823968106,0.06073925358166071,0.06758490826313301,0.06379144153852641,0.07292465135104145,0.050548845669666775,0.06111935902029174,0.07061670507642126,0.06496886594298824,0.06601296803444232,0.0660670846129603,0.0661237167043642,0.05574663380550916,0.06616463681205503,0.06612169992762866,0.07298494186291221,0.07482720723733657,0.05924654161749934,0.06355356439183636,0.06879066782222346,0.067796270434549,0.06416176712876234,0.06047238973639505,0.06277243669746978,0.06068082702247442,0.06945040757318649,0.05841975058603978,0.06387280883247942,0.05767318471714719,0.05742818400792443,0.05444213167248137,0.06636338486350168,0.053861106201208264,0.05224712564723229,0.0680915728143939,0.06339370701398783,0.07066697970961633,0.06113878137592485,0.05482224627850904,0.0653996396985293,0.05882896647111367,0.062441017871116465,0.07165094812629008,0.04947785732694288,0.06410844806677526,0.06140711122928832,0.05414934536833972,0.06377067185563216,0.07357999140735857,0.0633887526797541,0.06264600793099176,0.06317843859002292,0.058970628757834945,0.06851612433424661,0.06101050283795865,0.06839033555115909,0.06817004049070838,0.06529764815375855,0.0677357270113651,0.06469880060353995,0.05068753743585863,0.06519442697453798,0.056548944626312284,0.05768496581155756,0.0546687556367936,0.06841063371904349,0.06354870025794263,0.05758361239760999,0.06591840886249403,0.060017488329865365,0.060493329042125524,0.07081536495749188,0.0678536291200438,0.06052714398554561,0.05502431898125374,0.056146683358620095,0.05489666062475916,0.060835276163118304,0.06207748412342919,0.06305071539746601,0.07250704262757998,0.0626282290470737,0.06327539317455508,0.057179099363483255,0.07157258430593713,0.06331085490947655,0.06362265224388362,0.06444393423053599,0.0650464992679218,0.06639800242326203,0.06223692856002441,0.06586107568218294,0.059594963722041185,0.06832871925218076,0.06437825293306196,0.0705282173161906,0.06522654060889992,0.058231325310486706,0.06186140699878557,0.06311620531093248,0.06133977939993958,0.06114492634399704,0.06927355693689827,0.07114596559463326,0.04796902638713796,0.06885990811610468,0.07640733412619591,0.054777749472971135,0.06777618434437907,0.0666261192924125,0.06308053648300264,0.06918342100166557,0.06920448595249565,0.06870349247929025,0.07332775346722528,0.05712513064552389,0.07116387510545369,0.0658785728595663,0.0676313959775357,0.0668588574398189,0.062021946558984964,0.07161573075227406,0.061847559368209945,0.06288057157578066,0.06417108195888145,0.06378936472005692,0.0684978651444686,0.07337471729291795,0.07148510937993488,0.060525695398662094,0.06220464303522804,0.07323476363094081,0.059628964953499015,0.06008474814850196,0.06393714231403916,0.06252308937001425,0.05294968670908236,0.05354417437055897,0.06719055700226191,0.05962873824044361,0.06575226472051503,0.069177131694006,0.056504100451103795,0.06434003267380542,0.05944762828132816,0.053549545538593174,0.05901305387210118,0.06658608699682571,0.06899845142414957,0.0471536981413172,0.0703365408031632,0.06878071499087204,0.05733071768977223,0.057770883685667726,0.05789978906984529,0.06626878288930524,0.05811981947264454,0.06103234098316629,0.05507910494172677,0.0541174373296216,0.06617068939454424,0.06185276918340835,0.05893415696646433,0.06472616912187579,0.06523808125369501,0.055738942944992875,0.05621302467000706,0.06054941660247801,0.06641935803632223,0.0658081289201621,0.05594052183731264,0.06402495672451564,0.06509892938376087,0.057226692840004006,0.055873607695682226,0.06388404345927673,0.07215257625938068,0.06370006424874347,0.07102461550667777,0.05370175693109232,0.07111505835478651,0.06256202057806488,0.06514593061986698,0.05490489071175061,0.06306477644436156,0.0645116582677498,0.06328388644759983,0.06183886946943253,0.05754627729320261,0.05358954479236012,0.071777742612467,0.058799046391340606,0.06106601894867829,0.06994104314362196,0.06454892175883029,0.0571413582270921,0.060182770224562476,0.06420702834289753,0.06574294846353476,0.06521347487723396,0.07169797819555088,0.05997215224655612,0.06133422805995992,0.06533828670259481,0.06332184027103341,0.05732930189258256,0.05430328862119102,0.05853091024763838,0.06370325004313279,0.07711720472690436,0.07141925502403157,0.06214311368292624,0.06556731087104276,0.06429752152066202,0.0625395653590882,0.06114857324160706,0.06663844290666796,0.06026567264884532,0.05639296037945438,0.0611395029747044,0.06344681255935479,0.06501697226681075,0.0555851092200365,0.05943677023897849,0.06604200854891515,0.07180061442973612,0.07038515037189763,0.06310608333923301,0.0557201306998716,0.0669009411635451,0.06708311142544734,0.07176661549199773,0.06384171149030975,0.052178748603787795,0.06346719420944293,0.06745993765723249,0.06945592817273243,0.06420324647405382,0.061118120484606527,0.06233543754874407,0.06167731855717991,0.05510531065398115,0.06913021575642223,0.06459371778854439,0.06884726514521708,0.06589058908199788,0.046278940974837174,0.06942153451982293,0.0676539173596396,0.057043974723193586,0.06481694823476469,0.059414559201810144,0.06371863874428413,0.06104260828072572,0.05243665253038897,0.05440649880844771,0.05529270852974816,0.0655376866698558,0.05835231621384118,0.06088312824405559,0.06943730293287279,0.06673264390581485,0.06523941241666341,0.0762230215085912,0.06265187320096621,0.07367050977074059,0.06998579966904614,0.0674659544388918,0.06310001074999814,0.07364550065293204,0.05731061761258288,0.06279429267811003,0.05537318150479526,0.06438200235044925,0.05533370288769025,0.0655749433108774,0.06635581778273808,0.05633060693643827,0.07468698902039123,0.07237664277661754,0.06660780710061952,0.06672052000126605,0.05644692123510843,0.07321790748613102,0.0626239411562105,0.06535805610884421,0.06891942180075684,0.058799117566262694,0.06309954531270533,0.06960024616592081,0.05874162113694592,0.07106321046450037,0.07207589053662841,0.06583964472882897,0.06400836875123411,0.07112612048307804,0.05662432383280898,0.06382840978064343,0.06143524217666998,0.06617846408857794,0.07151226372241773,0.06836319471497021,0.07175926054463408,0.06548855931968127,0.06681427546143623,0.06032621956315757,0.061927055202862104,0.06467584122383882,0.06693397721123989,0.055114346034421426,0.057403647290100945,0.07301917614142711,0.05118997647720007,0.06951010512705398,0.06782725271134465,0.058385896795047694,0.07230493040823033,0.07456924637706427,0.065979000179927,0.06409262656111525,0.06568846350633101,0.05535219972578721,0.06989573831287284,0.05951336225562099,0.062256071276720934,0.06800684667888839,0.060035302367690585,0.06202782073175013,0.07085899348253522,0.062493221316046337,0.06806292745351401,0.07263882336422461,0.06771566306940417,0.06994320391587774,0.04793084829046999,0.07106124596431912,0.06440681626365069,0.05855017618450734,0.06655428518803264,0.06334838884475147,0.061800732398452234,0.07001808344671054,0.05148085591148992,0.055842042578996956,0.06932174722784762,0.06863734479923829,0.06315776316953971,0.06407099931142107,0.06838559592991314,0.07210862180699223,0.04874488574886397,0.06046579741464843,0.059422513268711646,0.0627681520385444,0.0644336358876471,0.06867861918798504,0.0683662040423097,0.06283724519550188,0.05560829982350938,0.06365064387905639,0.0649692721791297,0.06403708938805566,0.05634371367128696,0.07131278993351112,0.05967922129413178,0.06591530573664076,0.061299066895019676,0.04937150887091997,0.06239946587274957,0.06696834340313579,0.06126796815113818,0.06945850509763041,0.05771312960720594,0.07011132962324423,0.060619237316746816,0.07183992873979374,0.05639053028929254,0.05998153275440405,0.06708700251975747,0.05957129296236125,0.06328318755764026,0.06899252460955207,0.05845535021704858,0.05889611714356616,0.05532455339760202,0.06437017142015874,0.05616211255556906,0.06742802057030448,0.0653830021218608,0.06696356633373896,0.04853558341738487,0.07352077790947265,0.055438758641332825,0.05938823648657438,0.062447586801136504,0.05539096877058087,0.052584240230500054,0.06706110481168448,0.0591957795691297,0.06065929833677915,0.05786588026699939,0.0642369827488454,0.056977307477436026,0.06164077387144645,0.0666300094509732,0.06020502664345943,0.0568348361453977,0.058936768533945375,0.06250077356989309,0.05477270311714172,0.0599408851108041,0.06542426880397453,0.06923319228970157,0.06407114418566573,0.056756486713614926,0.05555971745747727,0.056454521821491195,0.060751445380418684,0.062157924473022395,0.06079343796681602,0.06709615577639533,0.056152471951273875,0.058284128619150985,0.057156601988697604,0.057439315755459304,0.06346496156434381,0.06464249173316827,0.06991123937839391,0.057820949470965624,0.062940901636271,0.057244894560382495,0.05678017165166016,0.05668550160376268,0.06222243998232743,0.0594140934532075,0.07136772487435783,0.06790207169308798,0.05454079386270394,0.061532211002064144,0.06589408816001807,0.07101563481746576,0.06512555081828006,0.06996363975054237,0.062406940816192844,0.06394323771437144,0.06966269194676014,0.06937458184739384,0.04779879393270858,0.0678778446660818,0.0755806205519503,0.06321598277696353,0.0693196236382829,0.06768157055565939,0.06670191002946349,0.05769894597601363,0.06925236648525358,0.06295884058708201,0.0651336916066402,0.06060168266609843,0.05431251205037062,0.05987983654964606,0.06030363041713792,0.06367285758809224,0.05769082718286615,0.05167082651318683,0.07015893884082092,0.06780634454861548,0.06745285893118731,0.06965934445515978,0.0485579827368952,0.0649606691254352,0.05725974048583382,0.06285771927096748,0.06377542957740939,0.05484144552950521,0.057634283774989865,0.05626292067000191,0.0647791152727801,0.06900714812643036,0.06842016309229179,0.06104844984373613,0.05893343599293557,0.0577858700451678,0.06803374919451435,0.06857655862471836,0.06717322595369501,0.05844328625166359,0.06827341590539857,0.06585536858528951,0.06338998909782145,0.06704584438822901,0.05544100883361376,0.058860728431472845,0.07360492478629779,0.06649046608720663,0.0500228103256998,0.06670896778669919,0.06419719061922106,0.06796600500674307,0.0631682550636644,0.05283538661868379,0.06963900633634704,0.06642979693651851,0.057426252908811995,0.05646601432168029,0.05835357079109061,0.06249709246721931,0.0608190911424014,0.060480231712693025,0.059562701485116984,0.06632444267696885,0.0669309693238935,0.06706938597456741,0.0690618198059946,0.06492180120961585,0.06094372182373077,0.049956858403918336,0.06444143149311395,0.060498156673272185,0.06692333010613638,0.05778527697723674,0.06310996870828253,0.05906420851963216,0.05786379581230444,0.05152664556999789,0.06598523829980041,0.06743712760688028,0.0469144794818592,0.06563652502939854,0.06340093400956934,0.07518096963908455,0.062311538820732745,0.07644549943717147,0.0668463603518053,0.06786346324836486,0.06681172984057976,0.05907142253194454,0.06781753613943035,0.06832467421765423,0.05928495309027257,0.05442557644309375,0.06189031333818625,0.06086373166334409,0.06220197725198454,0.050119279246137974,0.06302854632653618,0.05729709272140124,0.06928695584166551,0.06431349209306214,0.0494535022653798,0.05375971609891854,0.061688615905502855,0.06863404428927633,0.06555490380847215,0.06509817655724598,0.057744605200959156,0.06406617281623372,0.05764148907875365,0.05923409221265126,0.06945134430125054,0.06678039580997235,0.054089931614715064,0.06561480081577734,0.06609469861189202,0.05565389000280553,0.06493620394084926,0.0656689356062507,0.05212354524290298,0.061658812670972826,0.06815138815286176,0.06838903831533827,0.05929536594837135,0.06375376216580028,0.05872568716790916,0.055768847658407936,0.06749260696483081,0.06643182504704875,0.061715024605104515,0.06284438089160893,0.06468948034802716,0.07237082066315803,0.055350816281436704,0.061309004360111156,0.06790295707337132,0.0602342244792952,0.06084491617232639,0.047218380383495843,0.07040388000718387,0.04961135580179202,0.06252099335301,0.06271159309856274,0.06750868293263572,0.0630875445013539,0.059367295397454334,0.06546664537242054,0.052628261308523244,0.06066454815106448,0.06548822088480674,0.05028123822504469,0.05729029474186368,0.06604121652075273,0.06026336818183387,0.06301374620900665,0.06413315414435841,0.06583419208252721,0.06533596236931968,0.06750575745216786,0.05722910703463814,0.0629561069784545,0.06578161483940523,0.07404793847822189,0.07201551638193832,0.050938493503862814,0.05889139487177146,0.07260196165720072,0.058405781817240654,0.06461642843896939,0.055167932127369035,0.07223898130836015,0.06444327870656576,0.06624324476799054,0.05720783799783892,0.06433226353567162,0.06911433161336475,0.06744944576941259,0.053454316387088446,0.06522986109019202,0.061176613762217814,0.06232162049949685,0.06591949599499058,0.07233748016884742,0.06125734657521236,0.06522737350216432,0.06337899066391528,0.0553828607231695,0.06286953595356395,0.0667159557344563,0.06133245303678949,0.0639962903893884,0.06778766946379372,0.06368870828506369,0.05476027420521648,0.06689186056300891,0.06223946350721453,0.062391922373566465,0.0519905696719824,0.05912302960475825,0.06629821681058726,0.07505462620014827,0.061746447770308736,0.05661861671744068,0.06573730259024183,0.05880345214499633,0.06434163170306754,0.071546444480004,0.0623652844551129,0.06751653870782559,0.06424400097853239,0.06918973553557614,0.06283776094964835,0.061741616261977994,0.06321722467121912,0.06271529047531746,0.07192785290311152,0.07595280850574004,0.060875747064930624,0.06740079304439123,0.07078484043670358,0.07042123358470243,0.05690843200562393,0.061879871045038684,0.0769192716832006,0.052136273004478526,0.07050827424133406,0.0725845303546433,0.07048405898941998,0.05327265361742093,0.06195005489387604,0.06554431281816452,0.07165782129972204,0.06902039402390317,0.06493854848540423,0.057070967641572615,0.07196233695852727,0.061244702899248035,0.06620678624569959,0.06482301780282859,0.06682550586044511,0.06289247673181364,0.0626008000665178,0.05501158033098631,0.06977182647549979,0.06238680153963146,0.05875873329538363,0.07129992392432453,0.06092196421639068,0.06324245584838026,0.05773789266547488,0.06340506766034017,0.053119782645324964,0.07003054627156413,0.056020154023284266,0.0726117741186249,0.06488891144852768,0.054300840998157546,0.06112538176925198,0.06532063946213945,0.05433171085380039,0.05984759230774776,0.06554965865776767,0.06023295047597442,0.05999158804172833,0.06038619246233491,0.06743083428260073,0.06386883496819468,0.06212403445149073,0.06519005112294587,0.065592146920252,0.06318094474720007,0.061103370797765996,0.06374594830934474,0.07140297460351587,0.06422686846343686,0.06148191016413835,0.053579748767815814,0.06732898064211487,0.05525111541694547,0.05308856135265815,0.06791687565939027,0.06454072173474484,0.06803703851542287,0.06882542893897393,0.06928796710353669,0.06587981604706494,0.06597285291057195,0.05598614400879616,0.058694414967729376,0.0621900551382801,0.06031558626617882,0.07156464435139666,0.053866595576789494,0.07244642638279923,0.06716274139196171,0.062229497310797584,0.06160047362473114,0.07444837250770149,0.06155603033254641,0.049574044712707946,0.06010638169779206,0.04405688256723318,0.05887101006483396,0.06496407010587353,0.07443721944608922,0.056098764556858007,0.07028568521416702,0.06195531334854238,0.06649583208041382,0.0673596442507636,0.06519149995740225,0.057589189738339916,0.061511634089259394,0.07218399526630448,0.06365426185487813,0.06209112879436398,0.054406774847518116,0.06465169121021866,0.06203024131744761,0.06357666857640384,0.058956104473202216,0.057656353638753655,0.06130685240219551,0.05591241313765245,0.05867493523219172,0.06841111113045367,0.054628889877862116,0.062097965054437335,0.07284007187743434,0.057425133327843936,0.07053634058857308,0.05069426526314004,0.05133002752190412,0.062055732238955004,0.06523794381781406,0.06244594283514735,0.06010079626493207,0.055920683803820315,0.04680787585955487,0.05432802065118098,0.07166418706565628,0.06710239084714473,0.06706376437128572,0.059847805613284684,0.06494151771436461,0.049495823051793605,0.06550847868217181,0.06309461796740204,0.07209143092011247,0.058994944466195944,0.048474511788025955,0.063293351366978,0.05706580323831327,0.07090156807360781,0.06591008991789536,0.05684079276434191,0.07064180888466406,0.06308031502645325,0.06627418942977907,0.06438182856337363,0.06431415323921365,0.06911576301483693,0.05169442124126441,0.06329654216233663,0.06815409042761977,0.07371111881572945,0.058053005802591706,0.058316679287462564,0.06702682316676511,0.05390864416703957,0.06156376010395599,0.061577183189217004,0.06304893110689565,0.05593015016106363,0.06264797954432907,0.06757009413402354,0.07210168405306619,0.06912112187306071,0.05888632249563418,0.06395576270535183,0.07322446438592481,0.06258984447710489,0.07150659772849496,0.054014466485738266,0.0624927820843146,0.05958458663221694,0.055744856752816305,0.06712463308738623,0.06745009751946814,0.06352109727378483,0.06761915826330761,0.0514154140279153,0.06420393805090895,0.06894611471906112,0.05870015261075046,0.07323811739838118,0.07342347615989962,0.0707278108159189,0.06932496275022137,0.07001681669501361,0.06752419859231125,0.06716169319523099,0.06555184681252721,0.06886152861112227,0.06314315845532824,0.06613326412945433,0.061566641013071154,0.05394621026628797,0.06439974020598078,0.0654570749264172,0.06429891097317092,0.06417623462898224,0.05909061809058315,0.06844071370840957,0.06929986398596316,0.06633582789433201,0.06635721232685782,0.06400910195586304,0.0637688484414844,0.06625589134034651,0.06452046390585792,0.06331605249369684,0.06646403149719457,0.06410419755891446,0.056294444180961384,0.06681802317264388,0.06358746845747795,0.0585900428681657,0.07089334938990671,0.06633166594726107,0.06642844462726143,0.06911505302899885,0.06806498889645353,0.06943286822095482,0.06487305974232652,0.06146632380597303,0.07241605131901271,0.0484348104417741,0.05854226671522412,0.058591820057848604,0.06390584038032021,0.06528639307188873,0.06693938384560846,0.06589083883509081,0.0677724781796526,0.06585213160422687,0.06530699714841741,0.06771291096062634,0.058511822490286226,0.0701422144606955,0.061985987578138835,0.07343764677376718,0.06350496710802205,0.060702232006549214,0.07109078708758715,0.07078936040360717,0.07126940935443393,0.0702922079445845,0.05618653742867583,0.07193429263056249,0.06622633178774678,0.06674124005168336,0.06610403501287786,0.07093986668910501,0.0647964394068388,0.06302354367602975,0.0651535522823464,0.06237985012784178,0.07237836530911226,0.052101418384898915,0.06610800285772975,0.06980603672224245,0.05114831204209057,0.06055539215176779,0.07006441508302816,0.06547433827603055,0.056735858512969896,0.06079632392161023,0.0626343682303204,0.059292522428185515,0.0643169716352633,0.0594963025164114,0.0607106624625202,0.06956575608467055,0.06921110012057655,0.06505619986517625,0.07218808416674082,0.06103405539356832,0.06362472808526667,0.05996303470452641,0.062358193041665864,0.06046488918590724,0.0639362589305486,0.06159486138951242,0.06065629417249377,0.07523186705118745,0.06594723312282443,0.05281705720083287,0.06748661800605527,0.060827464790240704,0.05181914397610334,0.05455005231543028,0.05098133222871412,0.056144773485216654,0.056653925563369276,0.07143567297720718,0.06532401571092217,0.07204242158104729,0.06169431070774574,0.07194384737131501,0.07265124548495093,0.053442084310496296,0.057940860916005484,0.06261345781962951,0.06102274346858015,0.06370932706306692,0.06678886626499556,0.05900296266830116,0.06912751355589444,0.05951068863608665,0.06039923415369981,0.05391704847239458,0.06761240050091692,0.06889055869697899,0.05707869539864187,0.07340083244555667,0.07257025765312056,0.0655238350845671,0.06673932878370797,0.05266927857839557,0.05821435183515186,0.0657759512411733,0.07113156261196091,0.06558206167505642,0.058555705335507996,0.06337423609867282,0.06131809152341699,0.05684642621024267,0.053469522080873966,0.06681902369212653,0.04601013207860485,0.07187150795488137,0.06505773733740834,0.06179052072574503,0.06264523259529614,0.0584097460535672,0.07333213133743118,0.0615889604653022,0.06463654132610139,0.063144152918605,0.06847411465888362,0.06317603788448078,0.05667236597409529,0.06066704773512363,0.06474451441491356,0.06714889974600238,0.06122014877338114,0.06101470519477462,0.0687032603165526,0.07046425281366842,0.06803246979025548,0.06548414650033663,0.051951023957223105,0.0709994588875994,0.0574196398112168,0.0674344235802807,0.052206576475474435,0.06135668655906709,0.05139346864439433,0.06643349928678569,0.06713285506963196,0.068381987648406,0.06413437219254636,0.058130369318396546,0.06607933459741469,0.07352791002717597,0.05119206563414718,0.06470268474755453,0.06571797041143981,0.05676631918548118,0.07405312225504823,0.061554825029508674,0.06319903660225482,0.06340486515399535,0.07059766561828859,0.06618493653721574,0.06754284219302398,0.0636923249404319,0.06473026252045382,0.053071628798842585,0.06558620010591942,0.06860520480014326,0.059626232407174545,0.06054950172402419,0.06638482196487405,0.06899693705433169,0.07287356059445516,0.06812968103010589,0.06177869864542262,0.06054156659584982,0.06224725113877993,0.06526284149781501,0.06767766951526955,0.07021750855622014,0.06724229102612869,0.06457181789934714,0.06267018257526522,0.06851689227138122,0.07309275926722797,0.062151345000695794,0.058102614689447114,0.06709877370753412,0.05992184837683488,0.06897105730448339,0.06762063283541525,0.059350143174542694,0.06419700884452922,0.05672447079369442,0.05077024050497718,0.05674836258470249,0.06762390638623314,0.07408489051949314,0.06688430658874568,0.06957092866196762,0.07146952971454232,0.0598058990128629,0.0690457035321513,0.07206185667632117,0.06313299764026001,0.06192461636954066,0.05798023218441263,0.059008721697380986,0.04993908757814569,0.06632861308325533,0.06825457295775045,0.059529285838124446,0.05548582503865568,0.052663173057451246,0.0633463898164544,0.06887869487536363,0.07004361876422574,0.0650427192764122,0.06162827691667977,0.052034398754257485,0.06689186652345795,0.06739477146797479,0.07204399755378968,0.057391652940472646,0.0618334586539832,0.062482193502379414,0.05226462899582177,0.06877711290177327,0.06955160660865704,0.0658243138307108,0.07063939454645432,0.06596994519949384,0.05627621175350717,0.05924492547078884,0.07091914248944262,0.07130997888783902,0.05775011579922297,0.05751301825399619,0.058403222659782185,0.060091568196579084,0.06132701562213463,0.062648935891788,0.05273593802044463,0.07196017261657342,0.0735645604774399,0.07077957835229365,0.07151359063830154,0.06251322091285544,0.06415570171317073,0.05815499889333513,0.055871587762215666,0.07048244982302726,0.06807353947551478,0.061920335608656146,0.06834315870152777,0.06358093834622384,0.07148876325622722,0.06529414018717059,0.06066226814782353,0.0635446138621085,0.059584471449761686,0.06578832343218868,0.06983140055453332,0.05940425810155664,0.05085745846383432,0.06845675199312862,0.0531958485408408,0.05757093327697799,0.06494252596581657,0.06143534385971385,0.05545573688525727,0.0633528597978735,0.07265310668217276,0.06595720549335038,0.06889316838881193,0.06818336861152546,0.07635058599546526,0.06376261594729737,0.06798996498579091,0.06647393544140298,0.06076133900641723,0.06558712088641808,0.07346645510420494,0.07404545034419735,0.06447324497953097,0.05987915143803696,0.06181082548139498,0.06251758465249332,0.055017760759508885,0.06786173629485241,0.06707642725362158,0.05873047992595633,0.06635183427739676,0.047173554719936665,0.06480410677980314,0.06821865186722567,0.06974675666429221,0.06054280714392975,0.05694898292144408,0.05187547207064727,0.061938324122539565,0.0718508232315344,0.06675128263588323,0.06850674467242955,0.04390969740483317,0.06881667102808064,0.06980263881646276,0.06093833722620527,0.05903105004439335,0.06392433792120597,0.06377988675042581,0.05367332357277914,0.05212128679618551,0.06859841174450533,0.056469576026330806,0.07182431797326586,0.06520068260121462,0.055561758053860516,0.05525289894315513,0.06252560073742444,0.06057823159348798,0.060652493511611186,0.07026013457876669,0.06293863165278675,0.0643692145254108,0.06408240146019556,0.06767590544823024,0.0688455792952492,0.07233275951449211,0.06719421792996752,0.05255093633804851,0.05450031246552739,0.06600963773004825,0.06700212532834227,0.06650247026618195,0.06712508109007312,0.04454079521699239,0.061156313686938114,0.07194306800583622,0.06527656954405753,0.056560051910292175,0.06905714278519362,0.05935148991079651,0.06885629968993975,0.06255835009632414,0.05608453748549476,0.06247710500397834,0.07678191154523858,0.07361995051018601,0.07315647691905,0.0657845846922091,0.06220199153867082,0.058342392039585037,0.06556644388223046,0.06668761956975708,0.05351726303950748,0.0659714054970369,0.057226272594389435,0.05893834424379574,0.06718668966886068,0.06264919600742105,0.0523123450260894,0.05841416263347045,0.06708288005060933,0.07004737939314018,0.06114136299000906,0.051557837941567874,0.06627351305769619,0.05518496328261435,0.06431319944273738,0.07099793241946473,0.05077816784255468,0.06625702011578286,0.06389429766816412,0.05873414567026905,0.05465650820459958,0.061567436365675486,0.06969350696437868,0.06368048129940244,0.07425537842713038,0.06171066511036651,0.05902173287399815,0.06648308880904386,0.062298903799944065,0.06287240632903773,0.054418217694676425,0.0615141156665283,0.0509861274334982,0.04975161647953974,0.0630051676513437,0.05154699335535583,0.0654418376380104,0.047830201357458094,0.06628823668601899,0.06829959142031643,0.06888300001331332,0.06649310742844286,0.06432458803999619,0.06989898960455695,0.06842452418172612,0.06472133547877487,0.06877803606533966,0.07553697774339027,0.05447028921450986,0.06480142645696446,0.06408437396537123,0.06456659774746978,0.06742106213119362,0.05947089895339569,0.051121704445036026,0.05571440056014275,0.06072549728317609,0.05569869404952113,0.061606927125301685,0.05746069590068255,0.05621794307157206,0.06200805204351932,0.06915958919209506,0.07136363149678769,0.06051939305736513,0.05588509966154186,0.06963006564428659,0.05731391476066716,0.059253164508850945,0.07006550701005534,0.06401854437345586,0.06440803247758237,0.06683098742216345,0.05507158785399817,0.0691570004049856,0.056947425958270056,0.06172769697111457,0.05113642079811637,0.07342594264803434,0.059562522191810435,0.050251742426546966,0.0699164290994165,0.06596163363942578,0.06587472093932922,0.05535942456579225,0.07083519596196511,0.06984154124496998,0.06859384459085371,0.06966390637292565,0.05839900566128357,0.06191681713864304,0.06209634383476196,0.06264677763129886,0.059833188570633564,0.07043130346295753,0.058535691964261984,0.05569057263541621,0.059045118714031467,0.0654981552363357,0.061161193891172444,0.06321247522115506,0.060963537996335165,0.07254089311318296,0.05194859510601478,0.05724485639446601,0.06821846545128424,0.06859421959291914,0.06271056135689987,0.06989815226412294,0.0687841534700852,0.06631690010251158,0.0627777568091098,0.053544746176305454,0.06240759606114681,0.059109532523250116,0.058724898938097715,0.07217784134038945,0.06319243973546751,0.06731153198695496,0.05428970352725947,0.06654649024273934,0.06557649454798092,0.061855041544142765,0.06940607160686194,0.06017449105301949,0.06576219211781324,0.055598288554108095,0.05666391148151018,0.061130501105858215,0.06559119986742455,0.060752694893749126,0.06793339756954707,0.0727523741247593,0.07670099792615201,0.05853752013460349,0.05728891962608812,0.059682800710010066,0.07220485039266367,0.07642883767947818,0.05547280427421601,0.06906498090660737,0.05771134069708073,0.05510288776024927,0.05952954723752661,0.07442282498176421,0.05787901295563071,0.05808070452035694,0.06822599555046199,0.05766415000875104,0.06826571782144661,0.06529087316390633,0.053487874446040404,0.06275158483914123,0.061862525157884804,0.0733593598658055,0.06361654719399762,0.07263186664821507,0.05781638121456542,0.06293339458189284,0.058314928449396314,0.06649948993629928,0.06043826986225738,0.06196758429337156,0.06741568130476197,0.06592627095035103,0.05794678206967081,0.06874983103516197,0.053694734073507625,0.05993452424120373,0.07163462001795629,0.06422804914114871,0.07303444648524952,0.05947808280792201,0.0598842947270849,0.06564023425358506,0.06709049006245886,0.06008642385503282,0.0707354369250903,0.07113644036069543,0.062753521872572,0.06354986676958094,0.06987878182039313,0.07235594793281733,0.06250899954254838,0.0648332018231458,0.0701814879288532,0.05455965141438389,0.06776149529992616,0.07466035476800803,0.055335396858446406,0.06660837677779657,0.061834402714738716,0.06591071191579728,0.06368179304499766,0.057267409537690724,0.06716821909615688,0.06507066595842216,0.06167949181640109,0.059364792629902366,0.0679296769081644,0.0681248534593957,0.06618169309158341,0.06526216894971895,0.05516822827592363,0.054959369055679747,0.05929590626972646,0.06481493861791651,0.06209105733785202,0.05657292718068946,0.061793374261289925,0.05500335600585972,0.060647540753380205,0.053493756360422376,0.06692871502240105,0.061727141004186366,0.06107436738121319,0.06242252374211978,0.06279843051321221,0.07081610182984319,0.06542255039081599,0.06175902220430233,0.06467894556995891,0.06491076686559442,0.05059746793111028,0.06657840107281449,0.057938065546519535,0.05830645231108131,0.06409456775671213,0.0616768791161223,0.06478315193032276,0.06994980354027602,0.06703787114611945,0.062201372427139225,0.06131330409812465,0.05954819193756811,0.06747932684065291,0.0686510605267275,0.06404073532335255,0.0731128246897694,0.05624134896015394,0.054767395938398925,0.06358389958247998,0.06170248805808788,0.0645370471813781,0.06353552712778546,0.06663192208183204,0.06421626273512757,0.06725341041030158,0.05573355925561661,0.05415593191064849,0.06487521706978891,0.06536478248840527,0.059552506439884835,0.06264632935730809,0.06913151745927879,0.06043161567748076,0.06940907812029938,0.06556156324951021,0.06270369908004095,0.0692743358124599,0.061813746608845734,0.0612750529767127,0.053098784450628114,0.06181735981598568,0.06225825703706706,0.061229665083418455,0.05734349090088703,0.06634586906401044,0.061826787668270676,0.06031360651217286,0.06331437773895199,0.050550405229161946,0.06532010241725758,0.07129088891472607,0.053772431246533955,0.07306494974125899,0.0755357677545666,0.061054468235322235,0.05586612011346451,0.049065851115051615,0.06500685108681073,0.06516315459257319,0.058924089684665185,0.051188176465951786,0.06402064245227793,0.0660823823271473,0.057693888045022526,0.06486934262660961,0.04763493597625954,0.07106440193418721,0.06519632835716169,0.0714147766698851,0.05220040344371245,0.05179628597569831,0.05692480781056686,0.07007376302698134,0.06432125843646769,0.05518392819144498,0.06303791173128379,0.05968675185450688,0.06636609343078521,0.07144606096306882,0.07416562424561388,0.05661064747683162,0.06100547057517761,0.05373725380671625,0.06849231169769705,0.0597138517052553,0.06777996121958357,0.07012727968217633,0.06231247431837776,0.06080782698620592,0.06448384070499141,0.057335410350316196,0.05634064451262916,0.07455942653475109,0.05528477847870114,0.047470309490869105,0.06526214603664235,0.06840521681133074,0.06986327669323819,0.06395281727214736,0.06714614327669964,0.0638788966805747,0.056010231463075835,0.061030928739022244,0.06485477331236071,0.07420358400977559,0.05335894634782737,0.060133685389181304,0.05947040733411431,0.06110704238345935,0.06586143253414752,0.06506070936843725,0.06664847460023375,0.06419934683352932,0.07180441011250502,0.0680220033879534,0.06295459062794417,0.07145719328845074,0.056713971257614654,0.054062450102202786,0.06827666973262086,0.0654085351438911,0.06823806032508128,0.07314528273020277,0.049430232804882054,0.07135018052285617,0.06789849854248482,0.06740384137996835,0.05536800758813441,0.0713064027275375,0.0626822685732119,0.05304923953798596,0.058938131959127046,0.06587940587287733,0.07147982650879986,0.06282740716469716,0.053024410646264675,0.06349823776372951,0.06425487533081886,0.05880602592934301,0.06779019839173922,0.0579993540037977,0.07092765027427628,0.06645982825393726,0.06104439171013196,0.06932381640033271,0.06595530816899624,0.06133306642891012,0.052127711020530945,0.06264069165783756,0.05514227603478354,0.0665581488463747,0.06302845829484036,0.06579568872164593,0.06397662725286007,0.07470998240768131,0.06535146825504512,0.06850567443783187,0.07042722242201577,0.054301338957882736,0.07415576390114631,0.06290858490368753,0.06857807538948757,0.06255642911140012,0.06971132575812858,0.06673774648273276,0.06472113332149983,0.06499364247860998,0.061815104776277004,0.06587169737648016,0.05402821562243931,0.06736825410207947,0.06690859649243408,0.062245583999231836,0.06139166064503574,0.06971941455563067,0.05409094499671932,0.06613801971292999,0.06151263376241788,0.053770434609066955,0.06408052853124421,0.07054185328044205,0.056654635612496285,0.0665560206227778,0.05924287514943831,0.060623995621177686,0.06679030579094328,0.07305284921531455,0.07061135948420383,0.06051691907515733,0.0635488621499522,0.06625251908543245,0.07034074766374257,0.07505452950218752,0.07441039906624305,0.07262060241243395,0.05980519447492231,0.05311514858541995,0.0678377769796705,0.06183965571679212,0.06067185271795683,0.06530085367913449,0.06171779715362411,0.05934470350138116,0.06625290303471557,0.0667167747239121,0.055528109267193065,0.06219620407370395,0.06499952781094151,0.07369533434394632,0.048854674295451275,0.057294933246724286,0.05344936732051484,0.05060319157257652,0.05831685808054157,0.06117951545088372,0.05856159161083835,0.06368305753903028,0.07037725384612907,0.06775388426186679,0.06956825789124049,0.06920859432544518,0.06104431902187635,0.0725892088376744,0.061718297618574114,0.061130605545011735,0.07378343245831645,0.061057175393734134,0.061357835688136235,0.06470310564843099,0.05756589958358925,0.055868638108010724,0.0610968874167359,0.05148992559122876,0.06627366919698884,0.061085108295402826,0.052910125236380126,0.06581002351403248,0.06518099131530705,0.0705866156208324,0.06451064681952934,0.0667775471537661,0.06373448493711265,0.05710944453323635,0.06683973612629476,0.06714309711659694,0.06312846011642678,0.06233831015618666,0.05944332255761418,0.06369321519688873,0.0561102291550548,0.06563365364033844,0.06658207076842537,0.06577679989413904,0.0635696774618632,0.060643130070954746,0.0705393839937809,0.05230519397796447,0.06715652798457457,0.05281724232124323,0.05817118532979,0.06499948197292138,0.059615233089821404,0.07237827087040721,0.05165128218409555,0.07104756100774859,0.06542222100701972,0.06669444240242467,0.06784746199422392,0.0627556762761027,0.062112616105524654,0.06311247059581018,0.05293964770283083,0.049031200452535124,0.06338442647455109,0.05256847465417149,0.06494997537975175,0.06205369945827801,0.0683650069892847,0.06155034796575961,0.0640066148455121,0.06499561953518113,0.05084033184884258,0.06553523776630879,0.058424097898333555,0.06341409299055656,0.0664862568107844,0.06740316363079259,0.06254824672205699,0.0743848208323353,0.06881616207617433,0.0637461603684045,0.0706454813376394,0.06757793832076174,0.05854981175408565,0.06575522454559549,0.04934007875255726,0.051703236134531955,0.0565193658136061,0.06552747177926382,0.06659438478842397,0.05161938292543998,0.06157872427784166,0.060824255440675115,0.07398113303672599,0.05341102554792097,0.0567503600478464,0.06475248548157131,0.06444942760269465,0.07205624646641906,0.06746270931372395,0.06437101600228057,0.06547661046211378,0.06744848119927838,0.06001712004306382,0.06029793129817479,0.06587797188883328,0.07663684733363815,0.0627213633759058,0.07287677284364745,0.06697193502101051,0.06727078476481071,0.06749330623281352,0.05993961034524245,0.05638110568997854,0.06396433913639843,0.06377586409471059,0.06546811063771027,0.0708324542184377,0.06124354066736323,0.0637732620219614,0.06657597962132505,0.05174318330117342,0.05261840455965788,0.07411176375545966,0.07439763059925347,0.07286793980324177,0.06672568070327009,0.06546360446825777,0.06376141436593827,0.06467638924435157,0.06469229487805525,0.051565942331273845,0.07082499093779908,0.06231000885221233,0.06930234386863687,0.061209813007056316,0.06612423568575829,0.06965103122977198,0.059132859872262106,0.05195624978978536,0.06989852358285892,0.06722160849968342,0.07057284235082004,0.05993316952247478,0.056837272374757414,0.06948921499466656,0.06710262621382562,0.05888109125651486,0.06743124646067183,0.06954053860203618,0.05834132129537606,0.06539975477227146,0.07106964535786141,0.052456403864480444,0.058198032543004555,0.053355835008970634,0.06118368958934042,0.06921654859205528,0.06738710548146397,0.07298115502075303,0.07250119584427525,0.05949210947226208,0.06422955661640123,0.0669321212774396,0.06511787640088174,0.06406164638367742,0.05736137746825232,0.06790086993424493,0.06544932959615725,0.06333851011388578,0.0657240826015572,0.056077596661997256,0.060480778738607355,0.0650776336349872,0.05731738255037558,0.0637306443239313,0.07008830636379923,0.05835265241782736,0.07140104350371287,0.06571960981910187,0.06459732875434131,0.05682493129919065,0.06494734213509518,0.04870522653170855,0.06844406339478348,0.07252471814153835,0.06639035578386633,0.06800883092757176,0.06251054122187796,0.06405340890104597,0.06502473153295737,0.06737208395612598,0.06741042131970168,0.05580315599960546,0.07258229058739729,0.060029280078200443,0.060930182386189524,0.059685455831689216,0.07033255756457096,0.06768366002908824,0.06573592865621777,0.06859819759823113,0.06870603449754054,0.06515036850313782,0.061585809432042206,0.06704190766791546,0.06569913405343183,0.054228695595905776,0.05880981771868987,0.06398239903860717,0.06144740725574648,0.06639588472666333,0.06340808235129389,0.058830578897500194,0.06510516412732392,0.06362113482573337,0.05625514263886082,0.0649672303631769,0.06712783249772755,0.06817085014410687,0.06334130565444737,0.06489858145744203,0.05646368827786253,0.0636060875474061,0.07062391242965177,0.06512132302483647,0.05531192283903853,0.06251167261508335,0.063336397856598,0.06552328931682314,0.0687778424262564,0.06701885296700694,0.06347096301832533,0.05689868376572716,0.06440349918418736,0.06424910429209627,0.06028878813268686,0.06538934967525398,0.06386523005303393,0.06718306347831582,0.06326994252438425,0.06198998491779594,0.05640366567774749,0.05628752795263542,0.06371896586061528,0.06310615408678985,0.057314741750272166,0.06101346433039698,0.07181774618982568,0.055176241889360295,0.07150267260530835,0.07655248224184794,0.05954356886424312,0.0712939364835882,0.06471742698958038,0.057072993384332665,0.06603289356012712,0.06868059546393299,0.06375835366645924,0.06028601275681099,0.05390818239559558,0.06017136807010305,0.07365096962742144,0.06724232676328279,0.06080499021520636,0.06894765047099666,0.06532036866595073,0.06926716701771889,0.07240103994068738,0.06510509073296725,0.06025659921373454,0.06428640466738961,0.057518442756071986,0.07272704881626837,0.06458872122350363,0.05443609863036312,0.05667790150547207,0.060587617281804625,0.05941155325919624,0.060627532986709295,0.06339886436365125,0.0664129570739325,0.0702947054641162,0.0713285862145158,0.06381235020104481,0.06023396603065978,0.05998733947031546,0.06784567591029309,0.06600219086195015,0.06694161926141554,0.06372364313377099,0.04912701108418194,0.07133761056960802,0.05473167806684887,0.05765913206317179,0.06254782286023258,0.06318156670012062,0.06984815964792992,0.06715626365510202,0.06251920158320229,0.06429665539855178,0.06390456472450862,0.06873885826523818,0.07055179242859895,0.0643915696173529,0.061502097001574794,0.05635787426301844,0.07059388110006533,0.05572942434831124,0.06500606585799097,0.06587500667959903,0.07166247051665885,0.06600395033244691,0.07302874073742034,0.06916220610325777,0.06921866990870096,0.061567527844207916,0.06952898611157493,0.06560104029007344,0.054473945292055334,0.062376231278717574,0.06911518243792689,0.06410960253742401,0.07323984903337638,0.06968552306951287,0.07257772848840127,0.06140871956579005,0.07111453106902973,0.07274362342702807,0.05770256115438663,0.06154880108857868,0.06270264564102601,0.05860048288164868,0.06271546171541334,0.06479437107882266,0.07092844192537973,0.05914328630421986,0.05893855829096141,0.06644309691498847,0.055049431505888154,0.07135590999514854,0.07203315595166544,0.05407347163855561,0.07069557279918762,0.05895975725422631,0.05284213965696132,0.06422377636328779,0.05865526576849506,0.06357058794747136,0.06887725805903058,0.058589280832601176,0.067465958322006,0.05847865587888888,0.06614915119411052,0.05822177319650909,0.06948532715660125,0.06019234394316406,0.06518283606828056,0.06072582001420518,0.060673670917766445,0.05505845490822265,0.06306388628258824,0.06448820802278837,0.06306392424453733,0.06589603567598085,0.06521697632108772,0.05926698063300309,0.05945895223723017,0.06610297593914513,0.07007966559571216,0.06319436912179839,0.06057266328043207,0.07028827520594452,0.06529772271372966,0.0558722946263047,0.06784604995309099,0.060975078830529356,0.05163682442335913,0.06345849044079063,0.06157371746325956,0.0613500548581815,0.06190251964168093,0.06376586698398445,0.07053866716121215,0.07054246729394216,0.06774324673915463,0.05788980835582713,0.05232083186307601,0.06598114189995677,0.05478725800682012,0.06299824680810338,0.058688070456620195,0.06191745352294715,0.06338266785621041,0.06977730397220783,0.05807410509344972,0.06707159946363013,0.06037006328556041,0.054084801050098955,0.06217691069423705,0.06650879871957369,0.06934774588577663,0.06398982442602827,0.05615637870587149,0.0502419695309834,0.06828345868411474,0.07556600669597835,0.05545227517665566,0.05634613548936829,0.06856350575979298,0.06214300086635529,0.06488319280842178,0.07095076008899551,0.05468823235444599,0.049830991734464175,0.05250326713014006,0.06511711306558779,0.068050901169526,0.05974815530424238,0.06203195971910915,0.058091601846311125,0.058280377913184356,0.06717893824462964,0.06510969498956101,0.05751595459169326,0.06901494825259405,0.07505060426856673,0.05166647047411363,0.06276936800963448,0.05766608934528218,0.07265770686101183,0.07107522241147401,0.07315544928874179,0.05078799712843024,0.0606716442498587,0.06372732274354453,0.06101604156270746,0.05963477385682429,0.0540381309890534,0.06891601456640788,0.06608244384623774,0.07103702756802488,0.07453555476613134,0.061845368613737565,0.06868694847667345,0.05667502474952374,0.06054753685691917,0.06258431505574977,0.06626676302824705,0.06361743620095339,0.053736666637454134,0.06987886788686754,0.058711293561177116,0.06301898627625593,0.0629037469587929,0.060366867483778106,0.056327878495794965,0.06495646819346762,0.06606930401341934,0.06622017307482492,0.057629874903712866,0.0694101483539187,0.05798584529747914,0.06334201187309323,0.04409829542187045,0.06419734422401036,0.06811997208863863,0.058604933762336005,0.061684108197464664,0.06216404247046425,0.06341258483580421,0.06653269963142552,0.07065968463786511,0.06292109334833396,0.06513825799656944,0.06717038582319118,0.06170363404434395,0.05320250086485633,0.0756667684888643,0.07427705076831218,0.06644641095806356,0.07400520868753924,0.06084950071664645,0.0701796940397626,0.06142222697336659,0.06329089614212577,0.05877078235209754,0.06697986551110538,0.07074861156224546,0.06609804280486037,0.058976816431402936,0.06804691299817855,0.06036734831904461,0.05600193446927386,0.06817593910108471,0.051035686530190416,0.0603536413628524,0.06100858083871128,0.0632865810352117,0.06652981748873349,0.058936373637066206,0.054831589401529074,0.06306244533276381,0.05312172615529047,0.06301867610129912,0.06970676479480145,0.05657786273783575,0.05830374507557719,0.06264292358600404,0.06855736322946518,0.06076622273604221,0.05056370858605616,0.05738992634179341,0.0694009719120583,0.05996069568607507,0.06266882378368807,0.06360877065414827,0.06442358302230858,0.06429519819069464,0.0560206950154714,0.06168666153360197,0.05529140056112605,0.06687992618176261,0.06803783007086717,0.054929632727471395,0.05807858302426695,0.06202566619307855,0.06589799796356322,0.07426811357069901,0.06534702744221069,0.057850481528431016,0.07320139439762385,0.06494990090061294,0.06230956245682739,0.06817093438603786,0.061969368965847156,0.061768986900565315,0.06350854193063199,0.05886941246319034,0.06308071507361787,0.06312275913141721,0.06062477005060216,0.059528394835013904,0.065254728498397,0.05989786379073581,0.06925409535143182,0.06110735138285689,0.060754116301435984,0.0495315505994352,0.06368188974524949,0.05571227646032507,0.06198205752844436,0.06500199268396807,0.062151632717107566,0.0652769517558942,0.058898529756072224,0.07322772669271506,0.05732461228565976,0.06845702106260343,0.06698364895642145,0.06297366307735797,0.061511327572379614,0.06924410008859269,0.06715498688408211,0.06565687805135104,0.06418047568640896,0.06246896677749621,0.05649203163960494,0.059600083207082474,0.061448843337130056,0.06169435925418095,0.057579937705016904,0.050498805275557515,0.055271849549042976,0.055778532984714094,0.06680584450177587,0.04823566898969872,0.05746043413424341,0.07225596022298242,0.07375579261996926,0.06487980101109958,0.06594957419112864,0.0634164670876115,0.06806369620442648,0.06263313885747876,0.05603905751053779,0.06693775444342734,0.06200641433956613,0.05528958692266092,0.06752425822314082,0.059559748046333884,0.05523785466246044,0.06033904576937359,0.0650253004011147,0.07049939355917707,0.07636932327078824,0.06754864041779679,0.06344007236825974,0.07218737380764861,0.06407868299109104,0.06411704669761875,0.06421096718363939,0.06196078983292482,0.059936285057266836,0.06414843266985129,0.06860227274515815,0.06233417698073774,0.0645325765362508,0.06225589025640869,0.06508496944183904,0.05344917009671195,0.04727342031278163,0.071265760040344,0.06335663293048605,0.06610452470886032,0.061936458566176265,0.060257678810808686,0.0659489367829755,0.06090922334115091,0.05748190895797845,0.06372644147223264,0.06831890700359203,0.06748108491016316,0.06891380623460877,0.07071622658380733,0.05394227009983742,0.05584454010879086,0.06804070358435332,0.06240668745015511,0.06579216639621896,0.07491066109138489,0.05709480402818384,0.052371148050835106,0.059038369654024134,0.06753640727990767,0.06213589567043548,0.05398991735925372,0.06220327093294086,0.06786172635336815,0.06808879995939317,0.06627326597063778,0.06368954256198495,0.056678788604681674,0.062316622030278976,0.0682475729741604,0.0670572293738346,0.059486773332490146,0.056451641662446914,0.06800455168451748,0.062343017608016825,0.07080843284380013,0.05643630568150429,0.06345299227294077,0.061820767507462376,0.06107315545929935,0.06004337501248858,0.0690192749026726,0.0634617692899466,0.06357061435083439,0.06165364665114277,0.06095266860034447,0.05859438853776501,0.06192840172661881,0.06415261053994026,0.06636601915327872,0.06944416556821505,0.07140049927027231,0.06997821591585249,0.05386096953390629,0.06871062105888588,0.06670803753742698,0.06068658878580291,0.05620209453417349,0.0672459338472095,0.06003079856766505,0.06518012949083084,0.064531582335567,0.05631799397589647,0.07332744134573718,0.06028983325336254,0.06818831563413652,0.0578030061663688,0.06615420197250024,0.05678394553319344,0.06235390480217043,0.07057613920573616,0.06479596898939767,0.07037104048007065,0.061986218860735284,0.0580784620434182,0.06830723778672862,0.06251390399207803,0.051275269564008186,0.07324062072845655,0.06867962826589293,0.06722821690664593,0.0665684946951344,0.0615661501973731,0.06775887600128125,0.06447310015597887,0.06090781133859448,0.06879562119310952,0.06473349039616594,0.06029438251988885,0.06676203784104913,0.0747528387626839,0.060524466323306686,0.06942249406185877,0.06766051375303206,0.055892802447077575,0.06338683272654208,0.06168727174847984,0.06327058670610028,0.06508316107913578,0.07337623368799154,0.05575045721740285,0.06029112511092842,0.06591378119540556,0.06807940976096524,0.0617533308526721,0.058653195401303794,0.06758173457973707,0.054692016330317526,0.06566170529624588,0.07094009107908766,0.06480845924179843,0.059422453554923815,0.06410100826043907,0.06186048494003066,0.06711462788383454,0.0681477216026589,0.06563153401720645,0.056024099963944377,0.0651039066780253,0.06277023819903979,0.06302306341191269,0.06336288095558121,0.06365511045905851,0.06835987896575735,0.05845272255885192,0.06829056549478155,0.07550166450673432,0.06420497046549281,0.06886504729537338,0.06396638046053751,0.05956894682567232,0.07215117407514193,0.06445580053568731,0.07234364089491813,0.0613288508349061,0.06591652039179209,0.059845623988259274,0.06373956651576282,0.043866459156763675,0.06842204848571867,0.06424323324219755,0.06986992427675223,0.060758857196419574,0.06101665528348481,0.058189212424511624,0.06403170688788898,0.0751106176547423,0.05056272234684328,0.06854851715302948,0.06523304244879688,0.06263559814963199,0.06287296145697814,0.05761575596039874,0.06276263872107488,0.0739815874076902,0.056442584097177145,0.07150984539413807,0.06620576093124886,0.065126664236897,0.061666739931010864,0.05275748689062043,0.06159493536849992,0.062118270534422475,0.05790500702935367,0.05954126192234527,0.060366368461867305,0.06307579988019502,0.07061001371539358,0.06491277028321715,0.05846367848238221,0.06161707440976863,0.06374243473365249,0.06802995706759468,0.07261693178451226,0.05827240857257991,0.049380602562761694,0.06335625724695652,0.06141571907699816,0.06657472128378206,0.05938647360766566,0.06264265891658095,0.06495853030442847,0.06298070060398636,0.06560691169876545,0.06462479464983822,0.06422597302106023,0.06862814734264486,0.07151691250126718,0.0547319502473661,0.06299912231327498,0.059822447366339605,0.07189474003226042,0.06204694851184212,0.07504518790105001,0.0650940198533698,0.06745914238578006,0.06309332850415092,0.05345235506694368,0.06618982333901413,0.0638523554955607,0.06440997666324758,0.06587739090264465,0.06234237319753715,0.07153179438536722,0.0701893656516476,0.06493285911396417,0.06962426268858757,0.04854276480100479,0.06704759987144082,0.07494878534806726,0.05617057579452203,0.06511439240011696,0.06748309360004828,0.05695203501207535,0.06382490690981632,0.07448089097274094,0.06510746355765346,0.059369832235655486,0.07363913187191759,0.06494613855344558,0.058633741882954865,0.06233602520012487,0.05573453609263446,0.05967219343570253,0.06832575271996277,0.059607199816905855,0.06726093132722863,0.06038456653364763,0.06630069068955824,0.061442133484080244,0.06990360789375138,0.05556929064745385,0.06785432237611422,0.06848159766799256,0.06788408649618133,0.07311670975861134,0.049064918015522915,0.06914745083357957,0.0677076917733613,0.06778906814963831,0.07285168760330002,0.06339282881640063,0.052005883331823516,0.05613647948266207,0.06294644955538956,0.06764031039844007,0.06734083144792513,0.06205390917360991,0.06456737353927289,0.05400040276179413,0.06659124021647142,0.061658881032215916,0.06845258408614621,0.07222594483554151,0.0710033885708713,0.062016429648388736,0.06386401949218654,0.07573388537033723,0.06419616777880632,0.05070419110810717,0.045920315621844544,0.07056264991988553,0.06163547156080168,0.07226639476626795,0.053864109154635645,0.06137318359545238,0.061797322965800205,0.06592771232207631,0.06788169901869343,0.06830835561802849,0.06078632064912057,0.05478915338192464,0.07478116355687403,0.07213588058310068,0.059293841947337673,0.06558746063170565,0.06788311416898778,0.05474808975128462,0.05778463077466789,0.06343527716386067,0.07146999962868125,0.05868056646321274,0.06318384659405757,0.06721853552753808,0.06067895245119077,0.057054561531569764,0.05921374122645488,0.07171071022058942,0.06106759783254574,0.06246943974203835,0.07734578946414987,0.07074606371678366,0.058919158017956164,0.050425938351162575,0.07275954583222452,0.06557169910137933,0.06347950038331987,0.06331376528021879,0.06952300596447941,0.05256322810593342,0.07147276874756697,0.06931575861300975,0.07371025753701885,0.06871789447392582,0.05332652625350896,0.0665166214728374,0.06692263582871441,0.06847558969760699,0.0744533334405291,0.0645869710300272,0.06802857751388287,0.05888960933166032,0.06798035894047182,0.05139569779425273,0.06554367951909858,0.06268001364157053,0.04916255469602028,0.0652844567129986,0.059619653724526336,0.06937462569303413,0.057177343230881264,0.06793568243598114,0.06399306087989631,0.07138166615349763,0.06763454008483621,0.06103633605732578,0.05226656614408581,0.06142029561155431,0.07100320499974867,0.05936434623002462,0.06564593240720197,0.06081254451495091,0.06477725277327176,0.060871026049323915,0.06473778293015132,0.0675242019822121,0.049690046669516943,0.061860730005936324,0.059119094991754426,0.06270589262538827,0.06474287114213303,0.07447431300585039,0.06976236761058816,0.06610985120978288,0.06773085887693611,0.05628056728793724,0.058261869548345155,0.062299696888517976,0.057482101718568684,0.053718732367896815,0.07059208538543962,0.06283856829499318,0.07124576238369204,0.07268547202061121,0.05874934060274323,0.06186844259313862,0.06630171467171635,0.06490338351811323,0.07385937363605435,0.06655842273537366,0.052654443233420015,0.053960981105182576,0.06454967288597563,0.06433414367214099,0.06567091673715239,0.07030273493906394,0.04926471013480185,0.06505097195260956,0.07362845532060022,0.0564179390657634,0.06719263459188143,0.05217290920527071,0.06340326842644424,0.06631881299528508,0.0638046009642787,0.06744798546479336,0.07042954720495374,0.060292946625347704,0.06355380899643934,0.06016436433710051,0.06831727132585466,0.06266038071148383,0.05477019829696317,0.0629023757284263,0.05199633427508994,0.059771260827607336,0.05926704093582425,0.06502896334936745,0.07421697484320658,0.0659915801904907,0.06691118240100608,0.0671393149225124,0.06327861743258863,0.05683326122263897,0.06908579414212246,0.05772832378730776,0.06451244640203492,0.0645878157622084,0.07057721637743605,0.07053243815123161,0.0680788400953968,0.05434205943039783,0.06513692465353911,0.054944602462977346,0.056318266349692725,0.07206858951870795,0.062230893262676897,0.05747707372421797,0.060620467051140506,0.0690721942787795,0.06806631944143499,0.05815424064610987,0.06588677431928072,0.05231973607623899,0.06922975477555293,0.05892497622413118,0.05568910730041655,0.05228161387033772,0.06914282138863043,0.06782961938281923,0.051671402285560636,0.06450495558277836,0.06273299300454784,0.06403777189069154,0.05497837019394573,0.05781985285642427,0.07068394222398008,0.06239621200427108,0.058576446897329534,0.06312659026826392,0.06294181193353364,0.06754213222356856,0.05694245822176233,0.06881450831503709,0.054308407517404414,0.06522187107232894,0.058536787874319804,0.07090840317893154,0.05902987460587024,0.07075818904082884,0.06377128300997309,0.06212463972144965,0.055970480166411485,0.055443208230395155,0.06563961249024247,0.0578441412210353,0.06561760039195325,0.06433833448725285,0.0699213209360513,0.07168357344778192,0.071082618963385,0.06068733243031266,0.06892647592009164,0.0674611009318669,0.058146057543228914,0.06636670001308073,0.06556123158031085,0.06562739792532812,0.06882729069694325,0.06691653501093062,0.06836166692336038,0.05711208346105362,0.06146786171067826,0.05952067595994432,0.06633499869805062,0.052395587251749906,0.05163434439197135,0.05434373598321114,0.0640542552679525,0.06461203574446622,0.042847710749225866,0.06622828570996722,0.05588249447124055,0.06619950931398906,0.07606261967392389,0.056970785402313553,0.07276768585876445,0.0628526043844442,0.06981735646470118,0.06287692311512513,0.06624851017197542,0.06356224589328666,0.06978647806113632,0.07043147729135563,0.053633349316608264,0.062175559246368084,0.06735945662269391,0.06507786945192251,0.06307616785693632,0.07052099666943945,0.06004628042818647,0.07063895515057303,0.0632786978469754,0.061839753380292455,0.0696992398250281,0.05694883891203842,0.059767468620799485,0.07464327206820107,0.06469794990071229,0.059272736405160606,0.0590518150924116,0.05900082330144357,0.07164410613257421,0.063678670872553,0.06058146893154931,0.07367082141087458,0.06503994455422535,0.06412873429234364,0.0628291058685383,0.07296864391587041,0.06069525757995814,0.060415184637641076,0.05561988956652406,0.06932369399678753,0.07073012288935021,0.05058265034664185,0.06032171087624648,0.061564181622304114,0.05458065976896476,0.06151515741543936,0.055546014011025664,0.05696721691959301,0.0628265762843859,0.05116587065725284,0.06418601146584418,0.07150477174527826,0.06603782020369574,0.06210159842650678,0.06643971985841693,0.06155489613551204,0.053259371949714376,0.05464199189690867,0.06563640346737229,0.06601985315458883,0.0659936769763515,0.06369157659253324,0.05894344698400307,0.06026117150592186,0.06850189959632841,0.06652083676675771,0.057314450870856085,0.05275621184233342,0.06526118447373583,0.051695503533252064,0.06300387338440222,0.06612621895717279,0.06805741070775059,0.05200844710139044,0.06589915662693122,0.06472379724574634,0.06089528340983896,0.0649384221758253,0.057166124994740455,0.06589176378846304,0.05682575379538979,0.05849007374171475,0.06596551637155633,0.06141606488178422,0.06834260473580046,0.0676818158027283,0.06746685628262089,0.06829945181931421,0.06869601297175826,0.05901791719726007,0.06304080093568472,0.0663324569338112,0.0628617290483749,0.06341264440220017,0.060141285561904734,0.055993031364641926,0.06543485985166833,0.05672486798736921,0.06518423071817817,0.06360064977480429,0.0632766204204323,0.0633461005038204,0.06360631921843679,0.061278603533587245,0.07082238803053431,0.06239752395258259,0.06677592016832647,0.0632661380261289,0.06270413096943539,0.054229587927385955,0.05750837565822041,0.0674465403989789,0.056334466004244944,0.0672355430053573,0.06723836903406977,0.0667696343405016,0.04830831709795745,0.06490237239726329,0.06523489917445115,0.06478751208143506,0.07004155515474267,0.07290937996361868,0.04776888151221276,0.0752456624865526,0.06393469621891956,0.05519618249555408,0.06573147453918801,0.0728222045847927,0.06979776885679398,0.06999786739182637,0.0679214984625173,0.062275048320593544,0.06732644532448333,0.0562011817050884,0.06576743031096088,0.06328943626124736,0.06274475143843561,0.06718201115695088,0.056714987242493464,0.06706820611357564,0.07048225499105915,0.06806087715889997,0.06278161035461863,0.06031075848935131,0.0638395773592719,0.0712026899550658,0.0720301325849086,0.06969771563832844,0.05716423468609941,0.06499982314125927,0.049564946300956494,0.069481813947776,0.05814937056355614,0.0608240270718063,0.062028946574475934,0.06503208953578621,0.07644862954297751,0.06423266116788486,0.07391576819946588,0.0674635283224315,0.07016089556115132,0.058291129660280785,0.06257412717695192,0.06087013956702248,0.058683278800852556,0.06004580032562472,0.05622398784813851,0.07729512456992385,0.06640231557340501,0.07079092320469786,0.0585037471444991,0.058516981018456954,0.05737761843995781,0.0652148695277325,0.0694747359197339,0.06271100538173933,0.05971094411092587,0.05416241925088845,0.06821168281603347,0.059633150947259475,0.06677797742558672,0.057925995603596875,0.06851114756969368,0.054836435083720964,0.06813941738242041,0.06176332870953473,0.06607244615955016,0.06462486909731766,0.06605950259087334,0.06059590089191341,0.06291021649066064,0.0648278242126121,0.06009290332721047,0.06387165969209516,0.06380112638268448,0.07251959398085331,0.049347209286766745,0.058461658142709635,0.06010008568538333,0.059825009928035286,0.07033971964667865,0.06842919879466913,0.057557811499906394,0.06955718439712055,0.05135534786715301,0.0565608463055281,0.06636247619458033,0.06255392245942076,0.0623102125990699,0.06179381110734312,0.06963677858067818,0.05803969715670858,0.06973568026056168,0.07171281850368558,0.07357792673082787,0.06081541416756346,0.06528763683055691,0.06886505590065453,0.05646153370576601,0.05286095244491891,0.07528727463347945,0.05410349317700145,0.04905189635872584,0.06451005370889173,0.06956281738700998,0.06595921477669028,0.07034517350681585,0.06459555578446893,0.06749266109116772,0.06605016998061813,0.06408101758627822,0.0657095532771098,0.06250194628481166,0.06629704464273506,0.06910612505513347,0.052492379506165004,0.055449866059469985,0.06654687159384393,0.06116241610682434,0.06213115545931088,0.06440221130210326,0.06607508843835513,0.0695220761149376,0.054227675446753026,0.06403216440355808,0.06529819156816738,0.055822776962685214,0.06300325279199308,0.06272899985209071,0.06798773005014028,0.059945451687695314,0.0682038894559312,0.06517987383839888,0.059030352476375905,0.06705391680068278,0.0675978708971662,0.06616335045644686,0.06030019249617154,0.05680885633241423,0.0596919283821012,0.05311772585056392,0.06946844179927625,0.06763693832533785,0.06351646586931514,0.06231255706683615,0.0524089988751701,0.05703831614282742,0.06727126927222604,0.056494971112819654,0.05547325878784348,0.06375003490054396,0.05855834812804858,0.06009526211722967,0.0662358320244952,0.05853315421542672,0.0660103463041199,0.07196578579656404,0.058763853405335145,0.07221253015203123,0.058871416198260376,0.06726594736223482,0.056204979477066964,0.06734785498077128,0.05939096906826882,0.06888992721839013,0.07276451568252543,0.052308594466233205,0.06070153841955886,0.05903725994496513,0.06293817527886388,0.07081083279934447,0.06657197940428827,0.05985361393730386,0.05662016762971184,0.07138900053096432,0.06550275812563472,0.06253182120005951,0.057173935321168504,0.0626260361273955,0.05922030617388525,0.07148647385893359,0.06156559983755346,0.06371502526293091,0.06473307121866295,0.05734858718908807,0.059397967071981084,0.05877600562520978,0.06560948288366433,0.06178058897988318,0.06686019384459133,0.0675907301019168,0.07180210408407584,0.07113040842248478,0.07084066009711502,0.0682634276245545,0.06510325766807773,0.05571523900096282,0.05559543186315624,0.05985067255923422,0.0697309029784678,0.05524194492483664,0.05947133313308162,0.06611997596644342,0.07469783601180588,0.05872449548595371,0.065558918582603,0.07029024444769393,0.06615333187993086,0.06598510186201628,0.06155051804160262,0.06947909599026456,0.07397765370744816,0.06176844353210148,0.06766253548085482,0.06984085850769212,0.06273845446933483,0.07215138939350392,0.06052195491455706,0.06639571465614127,0.07282841396207436,0.061526428371425225,0.06162079948165108,0.05954955759081207,0.07014201732896541,0.06707949907436692,0.063025388697843,0.057491820547996675,0.06147614583989481,0.06385688848497806,0.06081991723713063,0.06043910204531669,0.06721559737262317,0.06123186115657931,0.048497171558281885,0.06197007841528115,0.06159876438175051,0.06128549693377599,0.0649833330595749,0.06360555477283072,0.05799790687743579,0.05088879236445321,0.059304412658368,0.07497349270121723,0.05609622264363977,0.07309541580921677,0.05267270910687477,0.06443544483493538,0.06036855331100571,0.06504987703530127,0.05407058610505516,0.06936615179773387,0.06497187991738634,0.06707288910638598,0.060881352893478555,0.06378873376716063,0.06428985775612878,0.06454540271691586,0.06922609476390992,0.06200371085820418,0.06762674183985404,0.07070257796286288,0.05177140906269194,0.06587221914051405,0.06812318702524359,0.053988541638377636,0.05177120760404845,0.0652192146645429,0.06885675591255086,0.06104407912483019,0.06449568226557596,0.06720168774681182,0.0538898637476741,0.06518193658362095,0.06443852998976263,0.06484007929721378,0.06212292795254515,0.06369257468571307,0.06257320189614285,0.05582678358001104,0.06834313100293762,0.06363151594524383,0.06520812496776834,0.051926981646812906,0.05992646388741017,0.06553781594172672,0.0628045671989736,0.06971054775466945,0.06795002986336593,0.0670677284370503,0.05261641329452918,0.06236754677877641,0.0638530874004664,0.05987204093785182,0.05161507404668268,0.06460956642322471,0.058149565037817263,0.061374413374515956,0.06710369158219595,0.07088849071751621,0.05988827289667073,0.05900277068069591,0.06215114471166951,0.0652038483875374,0.061358744210853744,0.06655147203273556,0.06929110886388712,0.06069677746721433,0.05642244178660713,0.05016992344169313,0.05800235475004587,0.05643851923357107,0.05429358634237627,0.05521625386753483,0.05110073369791344,0.06280563120106061,0.06436672081197736,0.0663019679526464,0.05274145522651651,0.07678140723508609,0.06243683332897953,0.057080506551345514,0.06072290977853942,0.06981672255453793,0.06633953098531947,0.05394415650618414,0.06390155102943886,0.06474896669808786,0.06567015033790333,0.059533874117123825,0.05966905659444777,0.0739113728567618,0.06046066109093039,0.06290400849211294,0.05566347522654591,0.052405238081052546,0.05424310070819732,0.06565027361965003,0.06300923354070417,0.05093741092652733,0.05695736541461381,0.06552624009033456,0.06667277808000208,0.059384708187030136,0.059758235147597524,0.063504437711,0.05705488486626266,0.05400926031373912,0.06806360660115607,0.05556218817563621,0.07152283311768051,0.06200014765127271,0.07260948537957707,0.06589285583256227,0.06280499378224781,0.07603688787911753,0.07127550487452143,0.06402481164706833,0.05987174863157434,0.07496372839989916,0.06532348378415548,0.06616724209597562,0.06095704075315131,0.06119366361468435,0.07025023326636434,0.059685555249651606,0.0648071230172544,0.06721780640532246,0.06726891216908337,0.0730725886549331,0.0628851231486206,0.06790534095037831,0.05912203676511395,0.07327676847182055,0.06572799637229307,0.05959452455815693,0.060879513423429954,0.0655267411970142,0.06186706107218467,0.061820737632274865,0.056111281213629086,0.07177499449970398,0.06122036675718737,0.06659483894110133,0.0651088394379465,0.05260990384366924,0.0603001898863057,0.06387165285672511,0.056703469981713814,0.057699620780612505,0.07071671223072164,0.06472368673671289,0.0582697869176944,0.055065683434790055,0.061348548062453316,0.06087926527363108,0.06746926141759774,0.06714976232007794,0.06852280159460258,0.05499862950969614,0.059491520758260585,0.05292945036709709,0.05649861472593882,0.06006411036144306,0.06718266486557231,0.06760861152007289,0.06912141226174842,0.06563466635633025,0.06310219433565019,0.058799508233265865,0.068147215656987,0.05804213836620674,0.05889582175869383,0.06578603848575978,0.06398760563319389,0.05338287811053326,0.04701596243377333,0.05967461575855351,0.07141322111998291,0.06909906109955699,0.07102867085132197,0.05927314244530678,0.0696469548581643,0.06510186930236048,0.05883553117073878,0.06644795187906483,0.056044870663203385,0.06649727451579698,0.057668887642784224,0.06269239265919571,0.058514073398158256,0.06304229017230147,0.07077392484850681,0.056482416190123244,0.058938986691315115,0.053331985096652615,0.05668381782417589,0.06146211321157279,0.07073111702178358,0.06574256133132386,0.05608632133666862,0.05721385743217575,0.0710921155398678,0.06554111292688458,0.06725785821046454,0.058802550805573464,0.06546913603178717,0.06544859449248677,0.07086917548910632,0.06757003936084191,0.059284327895523584,0.06104831002044245,0.06883355954529977,0.06429700403121819,0.06874694331237974,0.0650098401869417,0.05687121223786851,0.056881960291077824,0.06786386084609802,0.06785307109451255,0.0689027351118898,0.06364066608119878,0.0660578897993558,0.06342695048662042,0.06508937601400037,0.061493336665817265,0.06677110384511717,0.0678077143743349,0.06156870779746368,0.05809375032674137,0.059874708344760566,0.0724017801428137,0.07274825647747507,0.07083056339781658,0.051568139470974404,0.06191847200285502,0.06951476809229751,0.06434062323057839,0.07219052236138038,0.07019832601864916,0.057603174507094,0.06863004364397823,0.054804475556004775,0.0687747114927411,0.06660934543711955,0.07061157595411324,0.0644179338804571,0.05183221672317593,0.059001618664493584,0.06113664790999096,0.0705008559283255,0.06624569293893168,0.0615920202856072,0.06676644332752305,0.06479458680570527,0.059573625272590344,0.06480427278528131,0.06738698050342351,0.0707923145984709,0.06281992424749341,0.0742068751493464,0.06535497453754863,0.06723664929811435,0.06498976656833255,0.07046079661852231,0.0542372005660348,0.06696073811780194,0.05941599713260763,0.05247714873589903,0.061440634702009465,0.05661775314871574,0.06587179924027009,0.06534520761627019,0.06785897580365824,0.07108886566968445,0.07131557437364033,0.06581057463479276,0.062304868080625075,0.05935061385698262,0.05228684819372307,0.05898102000306353,0.05131783601252536,0.06575432940539094,0.056201852578217,0.06633802901145784,0.055529213944543826,0.05752084005375997,0.06233713587302085,0.07327231325857393,0.05980019465209525,0.06600075519248334,0.06587336462494095,0.05569360093040665,0.06241181773946819,0.0678247605048049,0.060372684990962405,0.05522276516240299,0.06339331267717994,0.0688617256202482,0.07345070830624939,0.060009413450979585,0.05848333282173907,0.07461509187230184,0.06884710657500746,0.07221652640615835,0.06971876550211464,0.05511423858271519,0.057274354111028616,0.06437533296555215,0.05861104561040509,0.06244191755071924,0.06877324038445617,0.07284632452013509,0.07045480122476931,0.06633779866169458,0.06962493321423141,0.0606950199830505,0.06588100515543466,0.06444408476651575,0.060360048272429384,0.0597428812206388,0.06498362143398412,0.05933766293188952,0.06640396071552791,0.0648191794044849,0.06964660681686288,0.062181436667901754,0.05784267334706086,0.06320354097586,0.06571767653379199,0.05911601507862516,0.06721737557283278,0.06772242512973864,0.06166457445488715,0.06520579541031317,0.06225994255226171,0.06297299739306156,0.06694219671340199,0.07411193049093176,0.055708424126119,0.05871055607204632,0.07233008337211716,0.06762884708237159,0.047179536487704614,0.07035655528841894,0.0682043465587117,0.06186090091919224,0.0614287616534649,0.05890461786734459,0.06267697574997363,0.06779957695666533,0.0662148354282106,0.056344920119956565,0.05620820246590055,0.06221365902651346,0.06719724192688395,0.06510801037735103,0.0744231859408411,0.06262550950473184,0.06342399658105581,0.07224441657180938,0.0645435222977066,0.07097090306091144,0.0620853366446677,0.066741513103689,0.05671231467238379,0.06342915561280354,0.06401877613511832,0.06178780690463353,0.07226702985947114,0.05857282028942873,0.06006092020677204,0.055253070057225934,0.0668192636016319,0.06432970275783802,0.06078974019693885,0.06329719700480121,0.06778739005134013,0.05795815120740755,0.06371646780180742,0.0692567647293971,0.06934939228662061,0.06514079029221169,0.05940720485726882,0.06836920910774827,0.053494430642252025,0.07449196173357041,0.061777305795345745,0.05930332931677213,0.07039884304387811,0.05768134472880307,0.0653077786733407,0.0611899453436095,0.06742984831115831,0.05258803357201773,0.06186049856374877,0.05973693900927002,0.05579775429079609,0.06731018263112756,0.06418110441003774,0.07156268764735854,0.054915050093707785,0.05988241917390853,0.06618065143649907,0.056804719950134266,0.06443056931578317,0.06412078604180582,0.05412295271152151,0.06538149780323951,0.058000278936101976,0.06891669329934266,0.06432478716865608,0.06857544628387935,0.07241988012279796,0.06459063431954296,0.0667893090332505,0.059156007416947584,0.059870267598426424,0.061037585157954405,0.06092264555313404,0.060733358360764766,0.06616198252351019,0.0621550078918745,0.05517675546582808,0.05909465347321453,0.06487306786483617,0.06148619107511121,0.06489521230383077,0.0634681816546577,0.06703719407778788,0.06332690653150776,0.0645995440896256,0.07283301346368307,0.05593076963029828,0.04989112394104752,0.07389433281863075,0.07188756720971815,0.06345795466700307,0.06506195260061362,0.05747793605582617,0.06329170389933611,0.06763957616312863,0.07113524214546905,0.06740027645774266,0.06150063125116905,0.06631730683394785,0.06530145658507511,0.052441532890191725,0.06607282510272938,0.053847764648983165,0.05752439158626911,0.06784598967050623,0.05846622239807157,0.0617469197820258,0.06378855135202707,0.07022486366796335,0.07204624207935423,0.060298717724861894,0.06691165356540416,0.06272794063723014,0.0701153255818858,0.06133785589612932,0.07072136969929294,0.05783813759241879,0.0663185986817621,0.06254020307983432,0.0694214324977509,0.05384849363155661,0.05990605751295964,0.06204296775432015,0.06307714654431607,0.06773067323393334,0.07294681136926179,0.06394703225049037,0.06744885595162088,0.07204990447651219,0.06997110300581119,0.06893211598922784,0.07219720392835578,0.06232920794903102,0.06288467533938599,0.06934738865206735,0.06933242390870292,0.07279317154124276,0.06993632454610138,0.06267811679607421,0.06009457368085233,0.07298056595333527,0.06824898188646228,0.07008774261790635,0.07014380413267712,0.05404991405754386,0.06435020290309351,0.06496471596284371,0.05731600198507515,0.05905584396853196,0.059093645178093845,0.060202367426200123,0.05615073876692931,0.06761459183629995,0.07127380408753026,0.06036079483394255,0.051716678319890234,0.07319900125512194,0.06861423130860961,0.069169166345313,0.060945827749277606,0.06044267955895048,0.06813427831758598,0.06659598054436894,0.06405695731251773,0.07049390025750805,0.053338037085186724,0.050561708722859866,0.06494461578676552,0.06149315022304164,0.06003968994450888,0.0673016214340243,0.058308527868536526,0.07034611693288928,0.060178228121102795,0.06319469258547677,0.06243768675806257,0.058522183881198044,0.05055343849343964,0.06105110198601528,0.058203585051086054,0.05516779294733756,0.07552419754512335,0.06620693018307999,0.06526622295554349,0.04728801323307394,0.057299471268344365,0.06609572976308609,0.07480959370108167,0.06429851619794863,0.06685599537009103,0.0704044599900587,0.06488724087216378,0.06276103193037322,0.06114455015099613,0.05541548547007427,0.060738924968202115,0.054587388168167834,0.05787960974378181,0.0601649653637666,0.07019864504893247,0.06808544018264302,0.06229363714787247,0.05138601685708212,0.06602989049552675,0.05788387263019616,0.06792102140232352,0.05864426696597415,0.0684436836092691,0.05136191104606801,0.0737588223444168,0.06861930726363516,0.048320459943395094,0.06837031878270626,0.07060082598733605,0.0608483856088242,0.05892315824938443,0.0682826063864509,0.05429965108536558,0.06719253610125386,0.0647802738192291,0.061566040994759326,0.06700048386429461,0.059728869677071555,0.06987793317124313,0.06709101703828138,0.055489500845229967,0.06462115669132065,0.06426470714527702,0.059242627603303316,0.06653696175990244,0.05884310994710186,0.06312118465187247,0.056797398767976806,0.07526805573662296,0.06626258530089482,0.06749209838833634,0.07038937395251627,0.06470559457358283,0.058284254643900954,0.07060424349216611,0.062205803074240416,0.056232572846448795,0.06029242264974527,0.053889155515787365,0.07086362817886672,0.0636405086309208,0.05858215456125827,0.07154635566961545,0.061501393295334356,0.06060188344833321,0.05934620215216199,0.06829578849363231,0.05740864955474967,0.058143510046104414,0.07379894068853754,0.06566626532730718,0.06182906087852321,0.05847083572667279,0.06044774068769383,0.053479157248789544,0.06654424816670948,0.05554095237420038,0.05248192992107589,0.06415707179875639,0.05831272795798859,0.0663711681621875,0.06774353894807386,0.06600794340307853,0.0594931585873341,0.07195901007161598,0.06096603149620218,0.05047996482299755,0.059183898360421275,0.0632346689726156,0.06320682501862573,0.05589083297075851,0.05714349214800095,0.054915490320583904,0.06076801393783718,0.06048708569097373,0.06336263536157832,0.06564877377556513,0.057255499594822346,0.06880898335316926,0.07176732710344064,0.06351699169226249,0.04976362293949314,0.056712605681472905,0.05786836101476953,0.07097301110694934,0.05795913425152646,0.06076409800568361,0.050272416905193545,0.06736115133903423,0.041616946825015246,0.05799618142977908,0.07139022517827345,0.07162609030392744,0.057695233127396696,0.06580145222948763,0.05537493937177174,0.05999994731914615,0.07017024153287323,0.05329251917771077,0.0607413788688181,0.07585460171403141,0.07261323291412494,0.06996353412680117,0.06657174415417486,0.06072265106600308,0.06460113031102116,0.06495797076738424,0.04749018107824542,0.06729708481808466,0.05171275600926756,0.06364123444351319,0.05933981866012979,0.0704946231382117,0.06828547297472234,0.0603566359563716,0.06354977772500986,0.05645149076363852,0.0583369269302568,0.05949345610631743,0.07565231889758253,0.06169425505318442,0.07288355759075345,0.06435600809051337,0.06686424115075199,0.05829908577007668,0.058043842672730536,0.06448439034775566,0.06377482235500966,0.0631636631258512,0.060265944919444155,0.06702491166731975,0.067222445443097,0.05936743691581434,0.06528880625872868,0.048027622542582166,0.07044798459914792,0.060115274808774696,0.06397170850591061,0.060055931923972836,0.07023589728961624,0.06958056741675406,0.07342687580391305,0.06533089693761401,0.05955755207207805,0.06415627965225315,0.06252294802656258,0.07825202832753853,0.06613474077343079,0.06965706775610236,0.0634109442127514,0.06741237922520649,0.0671485646245378,0.0633243756770139,0.06868009933499672,0.05353805507327407,0.06744986857246578,0.064837963046643,0.07367210867117088,0.06438563097339933,0.06533559719357887,0.06812282768907382,0.06814746856619022,0.06498813907297421,0.06085480834073795,0.05737412332283452,0.06192881554795826,0.06503046122533317,0.05060385009284772,0.05761621073131402,0.06365246500493646,0.06677855581248551,0.06412026840197621,0.06484534393348154,0.06887142880822555,0.06710593958128341,0.0668361467077623,0.06782687396208617,0.06184010053184003,0.06296694025189561,0.05273700372504271,0.0636168958580556,0.05397899642265664,0.06981468768182944,0.06014238188723818,0.06923736707801434,0.055846219429365754,0.06511939933703967,0.057214795233033085,0.06192386299501901,0.057992335369824594,0.06604954784764655,0.06704670237413764,0.06166038153498362,0.06979478827661065,0.07568793366438972,0.0639155962746963,0.06910192053765796,0.05756852482010563,0.06729077454324463,0.06020265600957304,0.05893375972621682,0.06254287716374186,0.05621796227746749,0.0704824507223874,0.06514204196295892,0.06197381940431355,0.05405052190886239,0.05529387015484953,0.060424827297118545,0.06686134571179739,0.07128902346264843,0.06477295706638753,0.05504105772419122,0.07050783416154394,0.06892545838839143,0.06320526251488863,0.0612246412970021,0.07331046984920093,0.05664823171970324,0.06174276046229241,0.06727057624531332,0.0684413684000767,0.07148325986344613,0.06147899844662299,0.06234479374918442,0.07079366792504271,0.06821495458286865,0.06399102659769823,0.06019115220019847,0.06491334767054213,0.0532997576164025,0.05903338945996462,0.07147138976766912,0.05454017119324979,0.052888014099904235,0.06282694661819338,0.05973921068614885,0.0627250328220634,0.06970674360137692,0.0650240682924689,0.06415280033051338,0.074550093083074,0.07216463872032008,0.06018664001426435,0.06730541117855468,0.057649437347416954,0.07265787108411437,0.07373462536852723,0.06217014121103172,0.0512085784368641,0.058807467192277835,0.061076831284361595,0.06865221302984266,0.06152897279299228,0.06509789813569053,0.06756188813879925,0.060557799420661,0.06300773136502702,0.056281811214690905,0.06825437041033261,0.0645612708226041,0.0724318757598898,0.049756999133741625,0.07279139886423108,0.06657085196650714,0.06679968594804797,0.06500310912491197,0.059366141473840284,0.06514603545754101,0.05665387574538681,0.051117343848954964,0.07578792973672976,0.05795807235011277,0.05407631173671242,0.059266280858653694,0.06657586686593084,0.06774486927900787,0.059681592203225664,0.07248749607768254,0.06368190769228818,0.061841387805502,0.06322499904459822,0.0558818253334938,0.07158827837205153,0.055127018863230154,0.05874774836102856,0.05858455818629579,0.07366254130253497,0.06752111551474246,0.05464541875101599,0.05946892881218217,0.06827883019871968,0.06149653901795657,0.071519521457732,0.07105959966748489,0.06285283133845146,0.06744944107435866,0.0726484025052982,0.054072243391795394,0.05724390248823971,0.06617707869278061,0.04754890003634332,0.06543192500814193,0.06320580895155395,0.06786059214702454,0.06499207937335194,0.06546843954024423,0.06043124847250486,0.06363280181590389,0.06581601946523563,0.06310999567937473,0.04811811933526711,0.07120499461419288,0.060320480195737325,0.06373060961560521,0.07289770414271865,0.06587155714536622,0.07232148944932151,0.05454734987755602,0.05913397203943389,0.057096657651411156,0.07061104380110858,0.07284739517375727,0.061744072557567094,0.06704204882714665,0.05912286539782644,0.05812697641963967,0.046212716839176056,0.0629114093427033,0.07378080905586651,0.05164208247153891,0.07369399183067751,0.06002527903155972,0.05340203014181859,0.047684240667734724,0.0629768350293487,0.05320174385241207,0.07017912424970266,0.07254317966836349,0.05384286962197703,0.05650529949094811,0.06456171507827335,0.06099141045707246,0.06393669758992324,0.0667455697340857,0.06871171642490731,0.06025508852075834,0.06978310086024545,0.06524536913053416,0.07568994772640503,0.05849785657848184,0.07319324706205585,0.061134265226789886,0.06474592583840656,0.058423183219627174,0.06324896682857686,0.06878564135768472,0.06301280553649412,0.05247100982128025,0.06748817509044316,0.07195014062536054,0.057854648655585775,0.06454616485703804,0.0644048126744868,0.07197891885127802,0.06452673254820479,0.0705199051641709,0.07578056927443777,0.06738942493405983,0.05271112442751772,0.06918129839979567,0.056628774949961276,0.06596854846610065,0.05778217730447741,0.04926918401863076,0.06287516023691317,0.063297676977603,0.06559885754160197,0.06140706487596704,0.06900368661703177,0.062423052869085977,0.06446181546017624,0.05207383583230338,0.0614225760094054,0.07062947348123122,0.06704497694645352,0.061981243688751346,0.06075049999060338,0.06635575973158522,0.05562855203140613,0.059729829509778024,0.06979601141544457,0.05629289426493754,0.0701943219428921,0.06403670088815092,0.056424752325982486,0.06582139379142779,0.056698381148809124,0.05912363587277686,0.06599829241736875,0.060194780981341164,0.06685507174363672,0.056357918863009705,0.06331504454515187,0.05794167146845634,0.06684841892865176,0.05303281989128283,0.06857150994268779,0.05611819277526327,0.05327988469701499,0.06885575314255708,0.06368862514719248,0.05417363049845801,0.05503254525947125,0.05414606054644404,0.06277264235907126,0.05561456226945717,0.06237815598631194],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Home Growth Rate"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

In [19]:
random_kwargs = kwargs.copy()
random_kwargs.update(
    {
        "investment_return_rate": simulated_investment_rate,
        "home_price_growth_rate": simulated_home_growth_rates,
    }
)
simulated_break_even = RentVsBuy().calculate(**random_kwargs).value
render_plotly_html(px.histogram(simulated_break_even, title="Simulated Opportunity"))
# Print the p05, mean, and p95 of the distribution.
print(
    "p05",
    np.quantile(simulated_break_even, 0.05),
    "\nMean",
    simulated_break_even.mean(),
    "\np95",
    np.quantile(simulated_break_even, 0.95),
)

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>                <div id="b92a9242-da0d-4cd3-b98c-92d1127f7591" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("b92a9242-da0d-4cd3-b98c-92d1127f7591")) {                    Plotly.newPlot(                        "b92a9242-da0d-4cd3-b98c-92d1127f7591",                        [{"alignmentgroup":"True","bingroup":"x","hovertemplate":"variable=0\u003cbr\u003evalue=%{x}\u003cbr\u003ecount=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"0","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"0","offsetgroup":"0","orientation":"v","showlegend":true,"x":[-163752.79912253097,755961.2940887003,330205.631171118,567080.3269891818,559912.5346818115,400127.7399280681,32734.152488363907,425496.1076106336,-9482.018595113885,-45236.34422775544,394682.06651527015,29065.156667289324,634810.9970661411,48406.22480386775,409652.05565485917,67594.19482170325,-26536.276656177826,-68180.03371325508,448260.0931354598,219575.42936025606,188712.62260686234,619660.3778101634,347936.9410413522,273228.4567219317,511348.97871713014,307662.0822088267,22454.732143003494,-151708.18988001067,21610.466182193253,37929.53330166498,4937.22885834286,-242328.12975362595,204248.0241621579,316352.6887080306,171563.28794974647,-340197.9888238688,-259722.92687209323,-229746.0080546476,249851.6616112548,23078.445252196863,784280.2595049958,38268.39979557088,676263.7177841929,728013.5411449899,214546.3826906965,838945.9759306516,432485.3464813051,-212330.55751069402,1166089.4431703808,285557.9337002253,-342591.3371413662,829726.0886567463,43567.342016407754,45986.727608986665,105240.47380291251,563010.7207311452,-137003.4802137348,-164186.12497092132,-294640.1167138838,888943.3582736114,290788.15344229946,-61234.40941404272,412098.92856252147,1146207.71366703,-109997.29993182654,453912.16722028237,-649296.0525031509,531003.1745252758,470468.8752167765,267699.95528443437,564853.1217077742,359385.79332427355,-712075.5410031956,550922.7169583458,-282558.0717205084,553539.0094064008,378150.33044130495,409247.1206205217,535356.4572363673,-390179.17832529475,691387.1037350032,659634.5611050194,-234923.50938816695,315847.65000330145,665000.1664669709,-183317.82360446407,-34927.27212712355,-141459.43491726601,10607.865812957287,-22683.03212145623,-364357.69954520976,225425.1443699994,669668.032177587,-508682.37756180856,-6905.981279040221,619005.545758564,-47062.49794067303,-230493.28654525848,-29112.769410635345,167987.99346209038,847193.7326171026,-107938.85450143227,-4758.1924227895215,1026166.9910346051,289989.59243353363,361252.76755851926,-255639.10832147626,231364.88761756243,429702.0119351167,116408.46149234287,-326946.72522395337,172498.43755989708,-400183.7345577199,79464.17124471953,564522.6898864629,-72407.3114105314,275635.3259872701,496233.6287136087,129655.68163480517,70731.41620899737,61937.75282496633,1268964.3854161473,494655.18669056194,593043.1145743714,518296.1294201808,537208.0089843394,731627.5637936885,238878.97195357317,1062346.5871671285,136608.20451217704,293545.1855761502,616729.2259802283,619584.3958573774,-851116.0113675059,-6087.30011480907,314303.6225962243,-106011.23224156396,-115379.09575453121,349034.77983344207,1333371.0980523317,554552.2749525667,210343.40198977757,728342.5464772661,167541.86523775337,330332.4041054067,-189667.7864712053,994905.9512301218,1013562.8428690499,227935.29172431678,-379532.63585118344,865366.6334730699,245384.89208331564,-798451.7538375338,125320.45302718924,-420781.07148775365,82483.94085434638,268710.46691327356,91640.17503997823,215023.08875931753,1176753.5943035726,948549.173223299,675003.6726472084,437716.63387590507,-612446.8526700183,398299.6295154649,39269.342599016614,-439377.12948138686,715467.7837381228,-16043.621691423468,134263.01366707124,975774.3304638173,13578.319442362059,527061.1808631504,93824.31442205003,888108.5947248908,-110453.2661447064,-194122.19933384284,380447.7979689669,436856.2775500566,-242295.27567427745,-643096.3787873979,-87549.10676237522,-52318.751747006085,-434915.68938957155,519172.8217665027,-378817.23637423245,148583.87712774565,-514931.11617224244,810065.144859584,346558.0505323368,-250486.02928693406,598542.4797120974,1033388.6551651256,-445611.0310316412,150843.42876417935,628488.1011172323,470454.0534910266,779428.7443306693,4510.66938567115,182847.05497317947,-76186.34337343415,518417.1156745474,985946.350521761,262657.79609719664,653793.6453645006,337389.3799725338,955129.5935560912,470344.4266905999,-171414.22761781514,430323.0566161643,-443081.07826723997,-93023.45962773124,220797.33113485435,439081.0477413805,628626.6320169116,-192677.71622665692,-577119.1576372669,39617.93431519065,698774.29347939,629811.5910104075,552830.6489027818,-528361.7979438589,-315305.8467083704,673022.516178898,674327.9978691437,-168698.9025247246,388996.3698516092,-164206.90129187843,328229.9043264468,62195.34511463018,-342623.0876263981,47704.201483954675,282170.2446302143,172087.62583855167,455006.73819271475,891069.835163129,-418035.2018941543,254388.72148193745,710418.863036186,683633.9194438513,109976.03325559292,295001.05937029514,104653.42063495284,608184.0291220285,84130.56119756494,331913.3725863183,-374227.626715594,313282.24951390177,-191939.44879048318,662653.5354030002,221015.71579783037,386538.8079837579,120490.48520117346,316257.83241203334,276847.6378947096,-161135.37811375456,689588.0260693002,-272284.005950538,442434.78525311965,673988.4160316475,-220166.41211880185,348062.6188155324,50215.26183240954,-7014.6016256809235,11241.75828556111,273557.2932170378,141374.9111286737,-303128.2561363834,214537.64841145463,125820.09998368612,79599.93538075779,-392432.7020266731,524507.5956450528,756492.3502094881,132745.3813216705,-521173.854912844,381592.0262102722,391024.14851627685,252882.1533827018,570582.6302179485,145776.03037604503,-5254.842741294764,27464.800238999538,529992.9998910567,967168.9340989762,-386404.5832275413,176461.21328759007,677639.4995861817,474107.0982322851,697946.2085537757,508004.8004727089,542258.5081873196,-209531.27936854446,-469075.8166761678,286512.5498635769,-268499.4902880234,532918.9725628397,370372.1024912046,-579390.676563472,668849.58066861,-7519.592787302099,669259.1273753843,793624.6841987115,143143.73951278068,-280969.278099861,423061.65940381493,530826.8415062474,350770.1989748138,928084.852132387,500647.1300038663,815903.5027683345,45622.126297879964,-79279.10933815362,569168.3828137158,119669.81669510622,32867.70745232748,-485886.3872027537,1170.914302793797,360922.35625737486,534957.2380774559,-349803.73401070386,162726.95683794934,189297.21276173973,202062.13442442426,-362009.99901451403,-90772.68998513604,-224820.76479221322,276195.839079333,557979.1656136531,318598.52871327894,-125999.04702808103,272788.7494532033,16822.507763216738,468832.6997860139,-423781.3465586738,138865.32898234902,138570.88433685713,-101330.53448380996,48824.301570005715,218173.10148674808,542451.7186711212,584403.7807509592,-391384.3973485958,45788.236825391185,-776494.5689715603,399796.7757713143,349381.4502087487,145911.1967459959,618699.4753179536,-570762.7151678596,1012421.377774328,-169860.4382235147,457669.68969412753,84039.68103661481,-454740.5919440896,-590219.7371163762,1031713.3810598813,424205.4060809468,150955.87099537067,766513.2467273632,98186.83608634071,463211.77890662523,480217.39512544917,960254.5658423728,-454887.02203578316,-150391.0913974545,820480.0876329434,303818.18002753844,908308.3148584487,24700.94679732714,216566.98678979743,576376.3420961881,137845.20333146304,712592.1320221317,578619.6189943054,-316310.73085916555,1169146.5690705306,277447.7512156069,-130609.76469727885,-200597.23412711686,-219316.2183541851,756754.7715572384,-479723.49019143404,97951.93864681106,-64865.42358472152,1231188.333788203,496349.961893308,160310.8056230289,200541.19282730483,464478.4917098372,-9190.041294613853,-59227.716499713715,496583.34778133314,-448388.6522775949,349713.54524264345,-284847.60433676746,92658.42819871753,-219856.15304734884,-49058.18360800482,112486.2621580367,228184.5692970897,866559.631999773,257629.41907496192,-251581.83112006215,147421.48954419466,987819.5878457897,276939.4862825852,313863.19369501714,565383.8469835818,-64659.91360147437,1011278.0029441025,-252136.4547908553,236938.47053577192,295679.9299233188,900102.9396042572,272472.58511016006,-337895.1753466199,-370054.67077250406,-587862.1372613125,447542.20539608505,40112.90066125058,237757.95415309165,-82528.19319387618,291970.171473776,-519908.9585187738,462180.19568877853,742596.5698576476,-377302.60448758723,327307.04483333696,307281.0527913142,926122.4785369583,-45781.85327609675,1137401.730631873,76858.56028186809,23424.438956896774,-93907.95629090024,731642.9579221443,437252.4438118236,642981.2615477818,757210.637308913,597237.8007020955,603431.6215441213,-61608.465790777,891715.678248391,656192.1472311732,-356022.9078177037,671223.7935393099,-522324.2186069754,873529.8261663499,107428.94353248924,-121270.28717287304,-27585.74411433749,423764.6843534354,389625.1065794956,420130.4809613242,107227.16979732737,-221444.25137405982,442531.68048484065,462516.7294443976,376410.25454218825,338085.9836008651,864500.4812552654,322401.5827564439,541424.0127669135,622396.7911449419,-768242.0609574122,425957.0706499312,788301.6858761539,225983.033876437,868847.3938563378,309288.8719748091,-174409.06766145863,-164286.36371651385,126128.95997220604,658900.2341372278,270095.4140528855,-17560.720189467072,-153667.2540336526,975122.9952089479,855556.760100015,467257.70873518847,-4273.675472614355,-825576.0971268928,85060.8180837608,-125114.43931519287,890417.1611346453,159800.68521337677,505422.1401390098,-351787.36635008734,212137.29277022835,306641.701792642,-393487.81143039465,181240.93456634367,64682.04700166825,-147713.85494957166,354087.5879895296,-127277.06181633193,-389921.6727024489,-110498.91904229252,753199.5321195703,467432.7574780886,639690.3264261349,1038158.8162913788,201062.5964496499,1610.0154603552073,-523678.9703096207,-404598.96410294576,-466299.45598535053,115069.46164031746,425247.9730959181,549002.984922173,-70175.17834303807,651209.2778306277,936421.4406686719,-357037.952693196,439464.23626950895,-188865.78501134925,453062.57426892966,-279628.31039835466,243497.12406990957,-284963.73062074604,367425.38874999713,1152271.253349409,111186.76278271712,300226.7079939968,-62306.59971345076,900355.2961769411,144716.56765097566,518489.5759510575,-340538.12929773843,-340556.0098261144,203707.25370229036,732886.784002596,473187.24070303,734606.8141061324,1155289.8294805456,602604.6196576869,210412.03096477874,500503.00878783595,-102068.16870785085,468689.2525845128,183996.19984746212,37770.30199835915,55271.28165381262,-626844.2674133899,380228.6488399692,-385565.4629244949,57478.757522400934,765355.0046279514,370573.4631952187,-33641.906338091474,983186.541759151,-220221.32076448016,592830.7886208161,562727.3680197052,-238100.3679135819,212525.8214967642,293152.40720572975,893976.6646823441,1284681.6917503937,654092.0474262345,291974.35377543047,-367927.0313497465,-520226.1353127919,-120610.9080236787,586602.5076186233,468985.2741598976,-764302.372153875,1002165.2008696841,-272621.3551367475,-1073005.805246057,747680.0789396097,-425062.88588755997,-713612.8809404233,236136.32444767235,-335507.90011894424,2897.2003807718866,-345324.0135573824,149802.70080395252,-495963.07464366313,-33126.708199054,-97991.93241009256,411730.3147200411,-307333.9221639605,829597.9676395082,255409.2118633599,701456.7191674667,512064.76073454414,80941.75526791718,22648.843025290873,-119607.20922126109,181990.97039452335,96252.1283010291,906325.9395213714,1075451.8347718809,-58706.352348290384,309642.4942210964,916993.8646593373,-701157.6036066124,-480243.28032163484,-362776.850436348,-115967.52323544631,382157.00743126543,-73264.30174350878,74103.68810126232,214693.82390894834,-24247.919516769703,940268.850354735,-485071.3653448806,571629.8709132215,667402.8654957265,887395.3406413076,-205525.68635816406,-679574.3218308627,611658.5011570216,80810.47444292996,-458525.4727693922,356386.54616130097,-108520.72940824833,177361.6478595375,181608.1250073216,677161.0631834576,-413991.5039641005,324950.2587789004,150400.79569242056,295946.9029067755,70288.45668394445,75253.34326439304,69366.99718806101,-229038.0902852388,35871.38277077582,621413.1414211197,833993.4372414914,186794.7776213265,-389615.35744643817,-220475.7211392764,-52251.052461309824,752999.0740340152,164794.48903249577,-425505.5166946952,231358.5811610301,320599.8855620916,599930.7455825484,-134462.20432879543,949277.5441321335,175946.96465788316,-399006.3639268526,90186.89571193838,360777.8825985049,276004.43718453404,-167859.02189558512,565431.8417847594,259142.30235961685,109345.95242560748,-771719.4490086082,-67257.0693123634,180540.82860242156,805187.435442361,-549774.0253116726,-479905.5571348863,227563.69680217933,-358427.9704090534,-341785.50205356674,307701.8591382266,-340813.12808853574,-121804.43282992905,-196007.17656214768,351225.97739311913,-639522.3690178683,164158.3276087453,-1032745.6533998973,-227851.1178499246,-467161.3662121985,510116.2353513683,-64782.14789998159,1102528.7320205849,-119287.2020387156,-151488.6530846143,799808.6890282254,1097969.9543351615,706920.0488054985,64116.59964959789,386860.48781124083,-445707.91277132276,584149.5881710597,590438.8178740386,88564.53985186107,610492.9907588796,192266.7380555435,187386.93368124915,-2692.2026312313974,-169320.9126452217,-26978.097469072323,-293544.35051656375,309410.66252312623,109209.87999550207,65910.64963479852,174143.6122914413,303561.8831829461,136266.25464967825,-127252.07002294529,526492.2661915361,637619.0669263578,-549941.9229925992,-434019.721930515,58110.85223507183,6177.239631769713,5849.846592311747,-360485.97439364437,217014.13396898843,844834.3604961634,1014396.202965585,434075.48743856,-56160.23039668519,-59532.7756408602,-243758.52070471924,214732.4147723061,713219.2234512125,1290591.747493043,94408.39601753093,451778.9119297513,299519.8012578627,1096190.8744430942,-290219.9244234166,-156758.9289545128,149971.8142611347,338687.4940329511,335231.99915262964,71011.26024198066,736416.4047303342,-266925.73775789374,677417.6416225494,-497718.3278052551,-91334.87036265898,310268.10060159164,-147462.6940643345,18011.3855198822,566404.2714121458,948394.8230260108,-90005.56734106317,270337.79112677183,-512467.88334751036,-31231.333603022154,300134.0731645725,487275.32199722505,-140013.24115746794,-487850.1100477637,566300.0710691521,764490.6302925688,449339.1599200065,-528309.9270589971,360286.9705293416,640869.1769723813,-436587.9078272409,567555.485914791,-276332.3202876258,-610937.7568780398,890384.3293596441,535394.2800345416,169487.96295338636,180566.67554578884,-291823.89771359414,-194104.07123347325,-419502.0742752077,560950.0733003737,445664.8677855176,214366.90289976727,262321.00872393325,3729.860350924544,33300.158578353,-91678.5886597205,472205.4970384161,452778.85309658526,116461.95412827143,-43737.99917261023,57671.35326027498,370927.1954251458,244174.08802215708,-592514.3789863996,-323072.78585117403,-149896.3125785687,108907.51221252605,124869.26663412899,-357298.94776552357,410313.5205869614,-265736.3109426694,-298580.31613070006,255844.67919418495,-44261.193240097724,92437.50582266785,117710.80015830602,1012633.7236272553,161435.00423083128,-559872.276127832,653714.9274456562,-126041.38453617739,72959.13712583669,567631.4909136705,7141.963875641581,749087.1849242812,168437.91384729045,421615.3541679261,230844.57407777756,-517869.4903581999,939263.2758144429,43077.629691661336,443766.5318098278,-404394.81113168644,-132808.3382400619,433470.99709329894,490290.8941741637,-74191.57629826339,211293.2916603731,1619646.0496148206,6088.148258635774,769710.042748108,391604.6472204644,330692.27237915574,724298.3453003019,645008.4431201173,-491931.1276089144,257526.26863868255,91863.34512244957,553433.6045276704,415892.8347739335,-468228.9687807071,-353992.17336840834,-408327.4111624784,531443.9273968833,1119654.8247745128,1058777.4496304444,443324.4027274442,340456.9189158636,1073097.4734400883,392158.7614172553,218619.3384233825,114157.37723420793,-388201.9818270644,338661.2790860068,667243.4185764138,1038071.8686130107,615491.9814143614,473003.89765255386,411097.35887044016,-175506.60105776554,591984.3487263368,-320281.04830718227,264508.9190759398,617044.9885521531,-157620.70984052774,-173830.26024406822,731880.6130978181,-748381.4735756144,-279903.42382233776,212825.3395736264,-410408.7198032127,-261223.57545883907,-270679.92208636925,290649.72640598984,-5957.568608095404,731271.871100896,7715.484711783007,341753.1994720027,-274888.0118323965,1221659.3326928704,-58324.41480001109,676027.4447963443,-12659.094441781752,98760.77389447391,613568.3200313295,-353550.75762899173,-65632.66052772757,515705.2331972234,910146.893280599,158317.55072332127,215840.61465909285,708915.9047838706,-811464.0122230854,185926.06598516228,104956.29500617646,-40889.11670063343,840926.7895315639,996694.3642664463,308778.5671328972,571408.5212394339,-293556.9562627529,792044.867801378,153801.98186151963,831936.15692434,182874.2712553381,68198.57739026565,-250303.1970962435,116968.78140763985,-258758.47236340446,-30807.36298937816,1048943.034441234,266955.190305586,416799.12862917315,-862788.818443758,1047056.9908543997,197.83909065183252,-771493.4661294008,-59348.13248954341,-57852.364241215866,669074.76328072,-200363.11342392676,323313.86235897616,-461366.209989117,551526.5885476605,375263.226772496,-301315.66597845545,638133.9256193326,1142848.239773022,-69826.4872778696,-241171.1836246634,78495.72422762029,254283.91925479984,139985.57972627552,341619.1240407303,230725.08103234554,571344.5221222392,443332.5552188046,642922.8564701397,1135273.7303097514,-541755.1800984726,53598.6998519022,-125794.82908066502,36356.88515748875,138228.03109426284,119213.7374275485,73057.88182938658,90835.13845034922,-96046.48559477413,132892.1591281807,195349.43541959114,272326.27080033254,284971.8099367125,155528.64754250553,111202.05404803855,-44173.80681444425,-355220.459420952,428837.51557696145,96.82949591055512,284990.97020621365,-185675.58130346797,279835.42063450534,315137.8769244645,749995.8642937555,189240.7490581167,-221883.674986457,343660.23112702975,-313947.14478407754,-706342.2232428514,-324979.02999682305,-552996.7167403004,25016.00077788625,139931.66376411822,167677.57278254908,-845529.3333663379,1048070.0687996931,764004.1457124758,-270590.8248024741,73004.44479011605,474568.2575027058,391221.3517696131,247541.2516441485,-266317.5718054767,-568338.0644610506,-259754.25044421153,85127.8423572341,480303.6766025773,182801.81819173344,-406828.03470119834,23249.93914747471,63307.01018883381,105948.96472962899,309717.9359562504,318885.34603407374,180984.8778483607,244453.0968908784,-426191.67647697404,73266.48877345584,290702.69794419315,204647.25155115826,-235309.35749314306,374756.18526647706,444041.4039717489,337545.59499199456,-601259.8964796886,-209027.28876718273,884393.345255929,-173963.4366507344,-401848.98974486394,-173938.9487109366,-1108525.9150151357,14948.115524486639,-49535.99221824575,291853.24583142065,970708.4586321195,86019.63286276395,-398511.6441457099,87065.26040260727,-302689.0417636195,-177980.279456357,1507345.4823550954,-633616.9386912882,364495.43837557035,164208.71676816652,111768.3327457835,53387.90284708887,-115237.29904661234,-155771.29887878988,86734.32137377933,-2732.929964000825,348947.68121675425,1001327.3763681371,308536.82173563074,176284.9677350619,429128.1626649201,269386.59881613636,327081.1069999747,397500.16497156536,-312967.7084667608,743111.1273745028,790651.9642600031,10097.290613483638,493425.11428716546,50563.40472578863,25770.142851034645,-857435.8613779354,506339.38136622636,475142.03903003223,596694.5119120767,227835.67460164102,310950.35898569925,95906.31297278171,445387.85462438734,393476.3923866586,105094.3829876082,858980.5332704377,10055.295442247763,-94605.77287007263,477267.52553539164,479960.9233898674,495808.88854951225,147457.49413857982,68635.62103270134,371949.41643007426,1185276.4340065583,268594.925670512,655796.7843727027,350652.98248578375,-273137.79903599154,763332.248319387,-592091.8853771712,285652.4078053171,313843.8158824318,-266401.20585268317,201653.55491503375,802881.3780060876,182131.46373891598,1052101.8729850831,107962.31399995368,868867.2059671362,19416.771453328896,1291076.4071369148,244853.67373815132,421889.0266455556,961159.7917428617,218481.68798425235,284337.4950271002,785664.0258434801,12206.023849239573,356746.5888927579,-621940.407915913,-9155.23890893627,1128170.6295750942,229181.77655089367,-119663.23952857126,442165.13716011075,363191.542383817,276937.0029140082,22595.781659468077,-532081.545934102,-225568.9862481528,197544.1220416911,243470.34852373973,71237.60172148049,-102076.74704818102,564213.3418043922,-163040.3962111785,-31033.441943814978,87852.47804819653,-416144.7979999939,-474487.32518109307,-3655.5060491086915,430760.5340712555,581012.9375078925,970845.0837590238,-304408.2969534178,460247.4121051654,476613.44884001557,-51519.791474344674,880145.10082513,51268.45155906677,-107136.40172868781,333433.38912149053,-754516.3198819794,432065.013961514,191096.09790314035,-118284.04782472039,611990.9896873678,-307312.82966087665,291685.5687709199,242253.2126900945,-163876.2261977829,-581739.7159719737,-462094.09537669085,803260.7961886716,198587.59251313144,-146431.40448051458,646139.5026051374,397171.0698805074,-135537.26438317867,579750.5776837994,-312465.02815765794,-245037.0545622832,865784.1723123314,-6287.554564736318,328027.70628034044,33166.9947402738,426094.75568607636,712335.0080217589,404297.00190946227,368862.43968289206,201881.6015220089,-101419.34582233103,274030.1640144591,46956.52172780782,142675.29929497465,299889.9227213743,735529.4672842799,85386.56532413047,258188.5916450657,682707.7519914177,-460853.0219814882,-505771.0857787612,849252.2318044389,-275143.31843360234,542672.9844952645,-241828.1829480948,348374.8889489891,407206.6941362866,106451.78465399845,246898.7832800001,429572.3837166447,948575.2634069175,-316949.1548243365,-310485.54649484856,922272.6826099278,527018.526213001,-214846.718292139,583420.5135107441,455274.26367816096,-160253.02690428495,287825.5877653146,-690325.5131666893,595958.3886460224,148273.03419046197,292448.7214320954,691752.6521241511,-249127.03015253227,145487.32564551197,245763.99830357125,649993.0056053596,-194240.53171115275,313130.03696182696,90051.20938629564,254640.8112762808,361833.55121798906,528145.9984543808,392700.4357934664,376947.1060758997,-657517.1509757414,785186.1127975257,290313.7352098441,435530.2497633165,467266.8294259086,321287.38575714454,84398.35461772,481700.87403173046,381918.8736710083,-444024.8558790311,580812.9671879928,988466.6300081261,557063.3418501266,-960239.2750212231,105869.822292523,359014.1457613008,-35105.219714282546,776416.7223474295,-126606.49828558974,-252462.77439600835,378627.2330235364,719888.6951525509,-192831.6091473368,583505.8971080477,237971.10323610483,638573.9452044205,-509105.026880377,-60397.827650196385,1095516.5998326368,316359.2487652642,-169719.82642809767,-295438.3339865301,438269.67799270945,354241.2786716367,575062.9035176216,-94107.77880060673,214765.41539050033,555314.5051903697,63338.258156429045,490970.4626136222,-84857.62643166305,87588.13826401671,600413.5530477432,-890631.8306598538,304319.2184660556,-411626.60558322445,1166761.8725945293,565823.168331834,-217811.77374548884,99239.00096003106,135732.92888453137,180358.91683378816,-71209.0770247099,518633.0794325294,-29866.112118849996,-6538.023139060475,-163354.39844258362,354357.1740493211,-259879.09988602903,-65968.21121648792,180759.7453680802,-50434.478069450706,116375.49281994207,303858.43210941926,721829.5185616426,-636560.2155461484,-422139.19097735686,899052.9716368522,903819.2121164701,697889.3459210489,650376.7198337456,283197.6225301707,-246956.12331278855,237111.9319031057,124193.77648657933,54046.90502259787,-289010.45831589913,411044.4837831438,-276384.8392350334,829.6328364843503,409065.0905763707,872941.6258460972,315505.7955061132,815928.4183363407,617000.6144773811,-128094.13181374269,550458.7996251858,868075.1388102612,603639.0267683947,146160.80142607586,855582.7083143294,405489.4042437379,-633776.8332203124,399400.70673235087,-8852.425206075422,105922.84679210419,443312.0593368253,-510198.3904668153,503105.3291666759,349128.07956797164,273162.8152507525,797423.7249565083,-240948.37772528548,342634.62197632296,-120815.22348718485,-95963.1938461517,570734.752038464,27718.64553253725,136174.75505035324,464639.52785548195,-354094.9363101637,-617695.9059414654,479793.97820459446,387738.0261840867,-619724.9584155837,-35639.38989179814,-435904.0828406578,317408.0249829809,-89824.3549150792,382828.89343461767,259884.02168941125,168133.07675930392,426518.9929589606,232538.78834593715,502613.91399564315,-102539.3340696413,-989936.7797855255,162009.04274517577,692821.1126183211,-662260.3046045126,652855.4623632818,896068.4498323854,352697.458334615,847342.2576462142,635487.1662686802,386761.41071796743,299799.4159261794,165838.70935674477,-447958.2129348088,284403.73703978676,-77689.3882871843,103160.28644335223,-461411.3047786094,307816.36347494787,-364847.11486490816,708690.3345094211,976126.5555102592,1154773.2985716634,118791.47564212792,555703.8804618344,90396.9127647276,27260.642524886876,174073.6190106175,774348.5880309269,291619.1872587949,685542.1685101497,-109226.88262191368,879300.526297085,848031.0912213465,10965.445183338132,447684.05134875746,979008.9060523296,119388.1064967406,344558.69439847115,1011744.186178681,179924.09916433739,-175597.90973600186,-643301.9727126681,730987.8069792856,-307802.5404287358,614021.5286308951,384417.1844794336,-120355.82790882606,44648.73944384558,-167365.41418212652,-868724.0341074271,100260.26231563836,227781.0669999346,-659998.7943237396,-323896.82570435293,-212692.84780574637,-16039.88796019461,1060675.8133606445,-52547.280596322846,455688.51487671817,-313382.83269465296,-3628.884746589698,16613.63125993358,82199.36333750421,-182353.69655360747,-306712.93827205105,332579.3987575695,301668.17695255624,590082.506546522,-20908.27270013513,-219751.61021274142,20696.31066658115,438319.117645673,1205259.4594793352,-299298.8503081179,-60340.45548509294,84959.79484351818,283132.8381758644,-436678.061278624,606657.2915178551,108018.58221002854,145757.33424093714,552410.6490448951,-650646.208757085,215874.35814256314,919476.7569512874,47281.8275772091,-216791.1266999785,-278661.0181169971,875918.6805602955,342602.5684678047,920122.6611030563,-84573.17743888265,531945.7828043073,-41378.62870057998,394556.93485708814,-676762.1091633537,-75718.86524667917,354664.5765375402,-269216.1626015054,579605.0165316178,-70230.58575822692,138912.22137036175,171042.0644103433,664121.2609142689,-141063.8678859193,378485.5178999314,-284471.3625537623,127235.95057000034,33315.772527989,-293078.936078541,49223.16575239319,-689757.82364036,98437.40714342985,530217.2187795104,124758.95023097005,258970.76016388088,47696.34317463124,896509.4088879968,1083453.8155969768,929158.0277359169,928515.0919806967,-114882.120169817,472154.87792644463,-75256.39932067879,113837.57194823353,283436.34760258347,-357730.164091866,119708.05800768733,17398.365745469928,-71981.79285801202,-73228.73340380844,-493714.31328590866,229110.94785175286,377406.15762405284,-564347.448657847,384122.95180115616,454472.1842213739,-399593.91319518397,411926.5210905052,-571142.3939954988,-46156.45624607708,-608706.668573509,201112.67494779872,1010843.8683532099,-163029.16104110237,1178927.3292466113,-270271.3444266515,852373.5467482377,631878.3497745558,559094.197164827,193146.6706642597,-168629.91167104477,986552.5667888769,-37340.11873328639,18528.749710550997,-456643.09487234754,546631.2331351843,519943.08403772535,608088.3666305437,-38448.70610205922,149411.5794741013,199329.36621458642,112249.17247123085,378492.7286035931,695857.0287563303,-486460.45660986146,281199.3172141877,-379739.2760168086,-58489.87656883057,231090.60533894692,296708.3490542066,590584.204143005,-28217.208577459212,-266632.3753233468,369512.0310645662,242748.77714854758,1352017.0089213694,-777556.8956137162,429190.30815403024,-32956.4068322964,-281381.0773203727,551991.8304820568,551695.1148470184,28366.490725714248,-116387.90919008432,191656.3271678798,-691009.979771236,971165.6547190673,18226.905328196473,-257228.5665014456,-210089.123162881,455729.8147552011,-508335.78219268285,143614.62984107807,-204507.6641994845,-355648.6145186238,-97990.58726567915,-325553.20563843753,116517.33825237164,144363.1615927904,209709.26452064095,-220266.40094709396,315001.4328544205,-154251.31517213676,260095.90345446626,815074.7967990753,235550.26631591655,448801.0055868365,913180.6464813864,631172.3949474758,-154841.76999641815,268637.6753250607,-393393.53285480826,550037.5306149428,463431.80872620223,551349.5178908142,518373.7127758325,550566.9065233385,-15096.665939091705,-666323.1777475034,158064.8556249151,551019.547836849,181157.78046461195,436032.6669201916,-190931.32941159746,-846220.7131896093,493480.2073882343,165117.02908710763,1261072.8429397815,886358.8181443722,187052.51845979784,764326.5600170107,-349715.6928233998,-214001.6659566844,455996.08822853584,-477397.6190050361,83513.16640282143,-410672.71843907423,1080855.597994748,851719.4121542512,289436.2693743212,419734.6339150779,717274.7812015386,257226.61453834502,178082.17967281258,203629.51818097616,379574.0424526562,-202305.41314457916,-181819.40604711417,213151.43323675217,199827.17245432548,375818.507136615,424963.27024321025,-289908.4119297974,-128307.76965126395,870813.2827912932,867076.4809967177,-345571.76271241484,-104749.61586818751,-355041.6796899494,1326053.360247917,755371.419939355,285.2234735228121,-155453.7812306867,-402715.75142569,1096520.499492223,216995.11204308458,763820.4758009654,-110657.89413363859,20918.816791826393,-45450.24402454635,204692.23950442718,385764.1598045938,341425.18242251687,80514.92096025264,89671.95273190783,-142302.14355445188,-44861.25604359899,20809.212577915285,-269192.1020823573,423223.3383262018,91153.32576709753,69639.39838411054,-72186.20805310318,-391193.01022896776,525860.4284304767,516250.6674173237,-183344.54489753256,-736923.248198552,-572090.8981997306,-627830.8453082861,654594.356580086,11673.50681415433,-82521.17129463982,-884737.6545642521,326914.11931241537,-387824.6832084721,327505.33424002957,494008.57967912266,341932.7835308206,244850.99491510028,406756.6878472292,-77524.45704213763,268086.6339898333,595246.5410236786,271725.7991637364,-378933.0600138088,292596.0193769103,309211.17832992226,-389124.27455443004,13876.751201425679,677840.5144261639,512757.02655614074,275255.4526742301,521723.38595879637,485662.71946443664,538344.8247020477,48239.17359727388,1023132.587716281,437071.07141671656,-469962.9330676354,893104.8200078395,819441.4357891874,623981.8603609358,205032.27312647086,470236.21052731154,806953.9236041941,496182.1578431837,-58857.907191582955,383771.0706575187,-489090.16140724905,-315602.1454193201,-48317.77188729914,-312367.2980224923,295001.7514366433,286801.7618311094,-238015.27532263147,1340963.4452305166,-502651.5628352626,-266907.67990981694,27984.570814992767,57483.08967411006,-545632.8680033949,171436.27317119017,170218.12051383313,-85705.00342093641,442845.9942658949,145903.19400303066,-75472.62873594323,382956.9930051612,50320.726439723745,1014491.8531728927,310133.40834326716,468949.47469779593,828111.3761702306,945341.87088709,-206527.01286326582,36997.287590067834,61299.66381423548,-372968.6914023184,-287043.42491791584,176194.95399741363,175313.3435325902,219489.31499009486,15619.387603661045,-111982.19505924406,-337033.52369624097,127614.91455167439,851882.394153581,70574.96455473686,-170155.36990063358,210505.08630232094,-215226.09184479155,27190.168589180335,-515976.745743555,-75550.78364972817,196675.0421967716,292888.7976026931,357910.57279246626,32708.263181664515,401720.79525308963,1038437.920210402,467538.2614296493,-270791.1787558226,23580.69417170249,-80557.13511017663,-111449.28623033594,127220.13795038313,-13019.260181487072,153110.52788180113,740354.6388915223,194968.0673127859,512023.28361288784,601422.7634392134,494676.4537815298,-105681.91208293708,189822.65345116053,537470.7242593262,541217.9995312584,398647.72423555236,-178464.21101534646,-122273.58186477888,70891.01119879168,-163510.36015585857,733748.1315519628,772735.6624635467,718583.410448405,38803.931300282944,826747.0063108779,637294.2785617239,146105.0892502386,412192.711837891,66726.42289918521,-225555.8862050264,882751.1434850646,-204460.16168367025,157509.59210623289,910165.303588487,1010701.5448049377,847610.2182746204,1040233.8364832061,346702.5479514203,-413503.98262223415,419351.0795254358,983588.7295214883,134885.20145701244,846381.555379292,195484.96627188893,783695.5309092295,340708.47457627626,-120187.67358730827,210438.48530702805,634226.3850332606,476557.9836917035,345013.4145385376,-212268.1843272159,537010.9861249332,593096.6635927684,717262.6829576963,223446.8460946679,-740561.7355119498,-292225.48079612013,-32370.305128960405,172892.01888998272,821198.1657555206,549176.2403253568,438974.4929964775,-216446.26877785847,202222.87243341468,-688185.8228811552,-340964.7462551086,-328574.98933318444,555736.2925997903,-347246.2406989476,1008274.6354526277,564916.1213138867,-60441.74293357879,294056.73342088656,-319114.20578093035,795012.2437958848,-377262.9872183935,771915.0382657754,-192945.1698214938,-515320.7291684984,84283.33814008208,862046.9829873284,175164.45378645696,245001.24445365788,109168.34395861719,334355.6055531092,206334.04880471015,-345573.7308955882,388491.48210429214,872796.246730974,776071.9934862908,199327.89568258682,231707.46826297743,-38964.33308605151,-264152.87807136495,208894.42233352922,262479.0324020898,587092.064159147,-280399.7824986675,203048.56798384758,291239.6307299938,-233939.098071374,506016.4472829434,190878.02361332765,627387.3373663924,-160567.5525351842,954060.1148340395,205289.56371996785,116057.81697319681,-320599.62287510186,315760.12202771567,663036.8431696305,508910.233105069,426776.7709110817,-428523.0896751229,687143.4903704873,241366.62486817688,134292.46968680946,44548.79714313941,-320191.59793372406,569901.9202867178,-49038.09808611963,-764392.3588343128,-221617.49213320948,828243.0469790148,1028143.9941339493,20221.645801558625,-1091528.4054378262,156469.93790147826,939645.0553107914,-217496.9686605907,188805.73914889945,-446638.01907589845,858178.6304564178,1013837.9893001297,159080.67114382237,260375.10472802585,-226571.86315643322,1114307.441522601,-165134.95048687374,318017.8711246671,369181.2050340222,853673.4238655563,490721.66775584035,-185985.80446019815,352058.7175354278,112959.1245004544,-47450.43708231114,223454.82968061464,58288.05116754444,249328.34343280736,-117925.0282082893,304538.7471189578,698814.5892533953,283806.633798277,-48433.63378551649,927680.8364326919,698510.6790533648,-65846.33641345287,37281.79285712307,-103699.28786330065,263299.20380249154,-31215.34134695353,376988.4567118753,1284916.3754834612,799028.9855939806,-228781.5573706471,85489.60313887196,-899972.8098081942,83049.23736265302,-579885.9662780184,689979.6157466047,-167439.73864650773,556260.6161532286,308255.2619060804,888122.9993297257,-109956.40607213229,294025.117861948,-255035.79972675582,808486.0455456004,-288158.8572770576,191467.39619851438,-146016.76318818284,142033.693075364,478752.2303659492,1012015.6394205121,473634.053716328,589762.1058127722,353178.05473010615,-185983.59493951686,454882.5682630744,-97822.88967245398,1225324.0487225666,746947.6176347672,261122.09360676724,465899.89075294416,-459554.75965714734,629014.7683991976,722378.8355829241,483727.2261258657,-190060.32558415644,609887.9280515043,-460794.1493031285,149650.02141328854,631639.3802442774,401443.26186249265,269972.9307573498,226863.98903680267,134147.2312914771,788220.050193035,387846.4813278774,-130084.96280720783,539192.1358066835,333530.1380395577,299079.29744595056,1003036.8913472146,217171.60546433413,236422.5075076446,-4977.8872444690205,34776.719737222884,213716.28687962983,400528.9300113283,186518.65537814144,424021.25975799887,-64956.2475182889,-91458.14346192079,727441.1301351283,945167.5280027585,99311.35966396378,39066.52431927901,472720.3762735836,49615.73125216877,142443.7647954058,388072.81383487023,-143845.08822941035,997852.2422300684,-560478.3709152206,585224.2818053602,-312433.7255526008,879195.0849126466,-191055.3281925246,78330.90206633508,-203673.01473893272,604186.3232842223,435833.6755508678,-300430.39004675904,481094.62055488443,908738.1204475183,924253.9806098826,-373977.99459282914,-606415.997867025,609964.5993905179,-167288.98343371507,558460.957710295,961898.4073975673,-38989.849188543856,263817.7545935735,-439328.17149008997,-177459.7964407499,-445631.07636502525,296981.9689074978,78849.87571129715,68897.80945535516,-354438.72619340057,905210.6375290737,386596.9150120807,1134764.7247146755,-89482.88184341695,100080.02138319705,98627.0919410754,1066184.7344797347,-111592.98131634714,45816.38429631619,276143.45611934364,89694.09409351181,-59696.27468599705,497600.84148048516,-16928.9503490841,-375448.69709700905,464273.45267942874,-34504.377748136874,-556175.3384828572,-41606.24206526857,628770.7596615301,-52421.53086923668,264148.9160329527,-366090.2085429027,71531.42129505705,571980.7685340964,-486746.65366931725,888667.3220551596,146117.88220047485,707031.8162787431,317630.57073717983,-356179.6420887802,-283966.5992125049,177690.34040455753,873139.7357102688,-258545.73592975596,-479784.6705346522,423340.9390858705,385467.8090105746,-329783.27128098626,257387.8177317828,-359185.79230557336,293514.4197815298,702203.0768752978,323394.2195904944,692193.3212115895,651807.4943193775,105209.49966291757,-330585.2383804815,459359.8833096721,401562.9548104927,387638.6728058248,-309759.9729884807,426078.7612818349,392030.8199312864,-91038.98482080176,197714.358922238,664049.7528205006,637449.9609620832,429534.24646527227,-235505.2824041862,534798.3155182418,371757.60420752596,545252.4304291853,-210811.66980670253,188686.65571011137,4731.403597708326,561110.2184912739,-114627.15320992656,823320.7173654814,513134.1436227197,478144.3820687039,-745343.856772352,160240.78102565138,80261.46265171887,475673.65004096925,-434223.1790412441,302695.3716839277,756390.107169597,-376534.7929092604,-387877.5489382455,35938.61871048529,21488.777099486906,-250020.93305327045,-108736.31296336837,502173.74037710764,577660.5937691364,-174605.26133694407,845838.7394362278,-599177.3808695981,-33844.79732062481,478954.33559390483,-292122.1077881488,136830.5859429785,116975.1760155079,848471.2837985558,177052.93881583586,834889.8013106119,154360.274882989,-305670.8927304102,29233.43232494034,835739.4993749633,803657.0609960919,-428379.2957818657,785825.1319400421,700594.5816253615,389892.3560110829,835296.3488436523,-225039.74417632446,-756743.3419907731,705211.4090403905,10092.046872875653,371539.7293847832,-125394.67201121058,-170980.3645036635,-114939.09021983389,931134.7742165113,170473.29583126213,732615.3022904559,450254.263217133,895925.3251431538,-195212.34005703125,171456.6095038401,685268.9134326642,224942.51964999456,646977.8862007414,270513.7795486017,1101448.6632564524,137717.07633780967,-262960.5154123013,653681.1329418998,-5790.8696787511,343184.51188581996,-196352.92572482768,1264107.3851038106,756866.8952556406,-407899.86167848064,-262638.0794594898,341514.1263825791,787945.0405996824,393145.34938035393,151746.33372116275,928430.3112461441,-276130.38702274533,249212.04731657635,693530.3078777059,1103306.050453423,85800.36783778714,357770.6439185813,-529367.8648273749,476732.2900403002,-119665.43613887113,471214.7997305207,-415693.5353922914,207461.4535380029,66127.3486768473,-192264.6711516264,-192764.53666123934,-83049.45424429979,407572.3304300378,31951.215546295978,173526.3880423829,169103.8352973098,-321386.26631326694,146700.29709014855,1112357.0906868945,-297735.8292472656,-150702.9860956082,750416.1542208027,807196.3124505994,-131487.82600862207,-492543.20036104694,577144.9811967076,718927.2124668271,182105.9036314697,306604.8826822536,-32451.78608553158,-450826.4891340835,-31371.41563514294,-58492.01009263983,314829.6014771024,354162.8905681758,282333.46155499574,236650.54341400787,654734.4083097759,1161384.8086842569,-34977.406945921946,312841.5765245403,610128.1836307542,377196.2767229355,848906.7968799411,223453.56198730879,-766388.9940634393,333275.82906755246,-129191.02049020072,-44722.49599801656,201966.1804489689,-27778.46312611457,-594728.9302673377,488240.74236763595,128685.49229719117,785828.3343537259,-294136.0731187607,-121539.47075127065,256930.48443114338,714980.0195985166,618023.9263658393,707194.6997672333,111814.28233970888,163374.53785856813,-316447.7397916394,74763.68134215334,14075.866533123422,447739.3570639617,288757.57149542123,320610.79827809334,-417641.26582208695,148405.6380959917,-257842.87913490832,543454.5290319538,79000.47031362215,1024392.9190481915,251691.26537799602,235394.400370582,151583.59939396894,310556.12383328564,391324.71611742396,718626.9559515999,566163.5303051253,60733.30988893146,-336675.6264376715,490693.47729356494,63710.82202275563,804150.3465554421,315244.58862933656,513512.4933697954,8383.204325981904,-551448.3098296653,3914.3200845168903,-55049.41732699983,453541.2553581721,-94685.14945452474,-156809.9193738806,444331.0935589168,-988277.212008598,-104223.58731894335,-284342.2940664957,301352.1931961491,71170.25568479672,634751.2773705716,200128.9527616836,249031.35412767017,222394.3627592125,553119.463156167,32631.284148682375,-310753.75508698076,-196266.15192003874,442945.9673764594,358064.3395701335,219931.27816919563,273524.2322870381,-151333.33350909175,-209142.73758814298,136238.7717225696,38325.713506949134,219346.83213320002,367160.96272838954,540960.1966198403,-174327.50720161013,366519.01760715386,216777.8364257319,-7200.227036792785,18669.227169571444,376049.77776468825,-494893.7668900597,996884.5416035526,271126.8847876829,65506.19360218244,144684.38830617443,-23683.406496938784,-25919.9255872434,285704.83474199055,511820.14750279067,-404041.77479741303,154836.79776341515,-97123.68239449617,-494650.52495266683,383246.6163179693,-379793.64475722425,113739.06621663459,191248.93257178413,5221.454470120836,-352562.4516201117,122075.96793813817,275081.2463487829,-378330.6198239941,-320766.9866861529,292856.2375134281,70185.4237161926,185128.427237547,406954.6669087438,-414656.23730070353,492834.8522236091,-242265.7871912308,25590.07281027548,82550.24986363016,636591.0965920626,-158127.7776627671,614261.3447036673,535888.2983919214,175506.35050133755,631380.1240124395,24515.662584260106,169785.49307841202,5462.301599638537,-298231.82261230703,-182269.39569370775,746449.5348044992,552683.367960582,70328.08387880772,710096.0826920504,358543.9282349711,-187638.56808814825,633472.2256502802,34655.276369961444,406335.7267360091,-229485.209645655,585189.7545925439,49008.140805106144,-817934.7682776069,611437.8183296202,575391.7093267799,-117303.737953512,145187.78537752572,980754.7478248617,-61974.96569492528,643220.9276163946,105692.76913242508,223104.13460594742,781966.3348371675,307461.13729984406,255289.9495465262,107123.2594033461,-40413.47804908408,529061.494267514,171928.21036243252,195902.01154005202,374530.05718346126,18567.878288113978,724544.356261069,319840.45505940635,200430.1038782755,340408.1331494693,-716993.7818214356,-239096.70366173843,-298743.2776226071,-161093.27131157508,-36900.356620314065,397115.43672235124,-318688.08850454073,-875901.3277225539,384688.840848024,503597.4249054361,-139472.38930952922,-405020.18305686815,709700.3544325829,403187.9284659568,809842.463185484,569204.9479506928,568121.1494891178,33491.101331774145,135851.8909057635,-5668.004302706569,-218101.62039893772,621690.9421547195,344005.4743720712,-8126.639288786333,-46321.67287464859,-148140.67509970488,242717.95316987624,155861.04832184268,-139945.82339447783,-347723.7372903656,51550.50237557711,-491768.1221454486,549480.5023859795,10512.886742386036,143270.54570824513,-486096.93977318285,154595.03364029666,913308.6675760704,187767.53295454616,-135282.35357793514,175327.3893593736,20080.919127081055,-506022.656621072,400421.6939504193,206918.5390894753,-646195.4369341703,-500286.24374655867,143592.05359856412,-338122.7608029777,90640.29291732237,141039.9438654245,612262.9774434436,500886.8793840511,-422588.386671938,618416.428635357,62408.549858091865,102903.10511509655,1249839.4409873718,-148965.5886841179,580760.787000122,146648.62591099413,132898.4317754372,-196881.81453434145,-117425.28137672739,267361.03929579817,822880.721773704,-410707.32866024086,-223320.43349586567,-640617.0726610105,848881.6524327369,376485.29249444185,357108.19216651237,-1008115.1829823984,-220297.207204883,548538.9072223147,491767.6338426685,-765586.9221424749,-269427.48660072125,-457161.4424966241,824392.3603776167,-159972.6685106149,526825.0643545245,-413709.17804486863,345200.6012746291,298414.2675539786,-425310.17252369085,793177.225899687,684778.3676988152,-597881.8907735609,-96622.12445172062,-24582.951698539313,299278.265329625,-34614.50106974458,455638.38071037875,970608.5952721282,929229.7330458369,230717.1487689265,470676.5935392929,416278.09114232194,75018.10339416051,485662.1295035714,558634.6830929797,-135418.629762277,390703.57323152525,568688.1408877587,-66730.56167540373,-133501.10791094787,468119.5776365716,-10634.969945917837,793120.1032561462,440752.9087371053,341253.5177550111,330866.886374922,362596.5289642303,516074.31927651074,328476.1891815467,313846.8931664615,-647808.0195487218,-291870.09092040686,-99010.97529600747,-30912.66400866583,59071.19513168512,1250687.7336894413,222356.4750869777,-220741.00607779901,260430.95167584997,594229.3811253593,156956.16200520378,211823.2722493857,167460.56423019245,398687.4113783096,-123002.01735859318,355252.731138811,683175.6892478964,254940.23269877583,-13133.993817651179,-410667.54806228867,288032.2273441744,-287475.8817653679,51515.420285623055,488731.6247054683,251938.3971402105,784793.3785193083,-166871.53861383302,347267.53429535264,315153.7340093879,145217.0488831997,-75804.13953248644,630956.2523707631,1257273.8248716546,-158314.70698478539,77509.06503707124,23739.70987669751,335433.9368428746,58135.18713237159,-334742.43000208633,156370.951841095,349778.3499602196,-9505.776719378307,-483160.83602071134,25297.68145504361,50274.04289073078,458338.1019664458,736840.6727347174,397992.2101094923,-28383.623263434507,491391.6530645862,149135.7183843339,54104.93391034519,-95753.78389589395,-117000.84220552677,1438966.0781138763,696173.1623727996,-679297.6065580659,549558.6928006355,444080.90941749467,90210.74472937733,264769.12069549644,332993.40136277815,261678.75107998727,999899.4189934009,54448.3058557841,531905.7634582012,279697.4384561544,24099.285219592974,642967.9959207387,-76470.4997396525,22458.587943009567,293057.5485314089,-495178.5758824409,393613.3646328277,188479.32326086052,48157.215854560956,-417609.4124805592,258898.11278238567,550090.3760233815,423849.62892691465,541402.749227752,328297.60330964206,-33537.38985787332,760012.1537065199,392814.0396632864,-102683.92811327754,-545384.1678517475,-490255.8311429252,25106.689764572307,188247.27965999767,348006.49700521026,82699.08176807826,-24801.665460933,-332774.2604288389,550411.3430931051,462085.1403977745,-1470.858208885882,665134.958958629,-608903.885581139,2939.7286455216818,303475.7646134412,738390.271118822,333452.20016168617,254821.546606089,105199.7005813159,-347990.87897785846,49923.85333063733,-390170.0174037032,-171756.19718719134,-491415.03686252097,657217.8244576817,-221279.9661208177,118789.28056161804,35988.144110864494,-141427.92135358043,781691.39686045,716317.4091769727,1060954.396465703,-177388.4290458765,-124383.2570838849,312918.8069372913,364995.774849616,299643.91080823727,352196.06109555904,209081.4580175546,1217447.712239706,969389.2447978416,659115.2919214023,-59949.60498504806,417455.521278569,20777.62284513889,107711.98766043456,755298.3019107124,328164.2877414739,216962.95942887245,140621.58937939443,678708.723097783,318645.73207211355,49910.94884095853,105549.30426434474,-322381.30839490984,693763.8644737704,208949.54012172995,199840.17609683285,488725.52782519115,-114155.78601525305,478980.871600199,65741.62409827719,182397.61109142937,-125562.80500929849,556406.5341634341,-345708.7006303021,73929.71103032492,5069.225414431188,204583.2427735692,473923.20607990585,-301851.87523998553,-748830.3270762926,314524.1049235957,-20703.10674454132,-133938.80100247776,-160751.91645532614,192086.9872236289,-552169.9315781794,1173661.1247159583,58616.52280151844,408948.6519598211,172868.5286415373,348246.1538749994,60591.08717891015,126500.23247074569,291511.80618898524,596462.4059632784,716252.6720284652,-23153.8226695098,161697.86707885517,-224894.50774667412,-360184.6971141151,-438043.2757796161,-248290.59741400369,197058.56917119864,282918.7238000687,327956.5535141574,683463.6806917815,620152.25671722,312003.25999841234,252579.70601097774,943046.729995979,322023.06432999484,-927835.8159792917,526064.122075107,-535934.6842659428,-229978.6915959278,547045.0335142519,209640.99803857272,-368692.0570404818,689377.9844282321,999003.2594659594,-74797.30762164248,881539.2478167834,861974.090634767,880012.3525569635,610884.3073060084,583536.2932083383,159199.4481592467,-25735.396346285008,74818.5258402247,9953.000708397944,88496.11163586937,485794.47266948875,-102164.94976596907,125128.03386464017,1195292.6895005796,905298.7645501867,355152.6324835648,675944.5178548228,-994197.3395657488,279837.4575592517,683110.6361741801,392149.204713054,66300.21545310458,170722.7573672277,740206.0325635299,401796.5037324019,840457.1052594939,721433.9940314265,-393026.0463582901,-238087.32985128462,-183190.86286912858,-692775.5712543088,1075757.571723471,446012.2545702122,-106978.12909592642,447889.54050432704,-72347.53780325456,-35165.45322273113,-142611.11985541414,612598.1858572452,-736888.6651520603,537333.3514323905,467263.4130861508,445879.03610920766,141674.96049664915,385145.0894638137,155162.4368490451,-278193.58757203864,-248523.39575978415,686099.5723840515,401093.1370173665,190326.13649340626,-292259.2011844972,1019668.9507406102,-443179.90568937035,482083.3816474946,735064.4414568227,362829.2834288301,-253659.3877213481,-30205.208481515758,846752.2870811112,181833.98082138272,90658.34473370202,-126417.46385058574,-39804.45747320913,-132592.04559080163,-59527.98780257208,-435520.90752663254,-751623.3354055765,1077930.403937527,-197835.88723290293,310173.82580043655,547991.0875355378,724232.9288682058,-13722.033087683842,87484.55583400931,679866.331628622,156927.37357753748,562752.6503894026,41281.042068485636,311861.05834715953,-173948.83196863253,403766.2243144084,876810.6474795518,64265.12834226014,-263274.0145634748,753921.6634382829,487098.2821701239,46235.874117821455,1198962.3617068287,549713.0812490168,128138.50406042021,249298.12354294304,619113.9430200867,1103906.4707469703,254477.7507849047,826682.8460818538,324161.90699771745,819062.0030376259,80054.8316158671,-290616.89518814115,550366.5536136325,165612.61899062293,-96861.32624019776,779645.9324810128,59687.80558359437,-714349.4318799935,372782.7737843101,685061.8192069489,-520365.94356860314,-438083.61441028677,880604.9681743262,650106.2650010651,-216149.49644878833,-393261.3707881556,620626.9303291566,-74091.71220775787,-927113.7468720204,572746.6089932676,42128.96830139635,272958.68256116007,-473668.4585920642,247165.91478031734,54439.332361223176,343234.958194545,-365826.43051371677,1107066.236446979,-705711.9208701267,-435315.3000620769,444308.0850814637,1024303.4257854177,328985.4813161483,480217.61913030455,73259.61936775316,782569.1932795732,81781.58151228586,35936.63736517215,438341.385381659,40896.76932120696,433824.3713951586,-315223.0888012247,429345.1304500061,165674.45121956104,132237.17344627436,548243.654394683,266586.72098309686,615565.61300423,454511.2863223227,560897.9469983033,302086.98688088823,-358314.59679213446,-66394.260605169,-35682.44338680431,354689.01179261296,-79082.20702418406,82707.30136756878,1027103.2286453443,519992.3009333024,176764.00363511592,-597248.8710871674,-611629.5586328451,586166.7020139974,-488311.974321831,311115.41838147864,-125085.3296327698,96126.6001344421,-296566.31837172527,-601939.0468033883,88496.79672939144,264781.27137407847,480002.0279042884,-172007.99108303804,543502.0889638755,-416849.24114199774,-1212934.075507501,-197599.31521023717,650884.0661499929,320775.513283222,419443.69971844414,-181842.1026858203,50662.08363862848,297954.19673121953,56069.99484945554,-921396.794807272,664313.8926121802,340110.02965195524,405158.26668558083,-101887.65704124235,1011279.2713688975,-223180.45975255128,-515676.8841697825,-119253.12219393533,14150.075349430554,-343575.4923906452,72748.89875962166,428495.4670302514,544177.8185524419,-299304.0243047173,596767.0756705292,-319927.1819642731,209948.47220671922,-117879.92615713645,199935.4735044716,868505.1158569851,-167264.6323585012,-83732.57415362448,-688580.5836939509,697772.8389147324,146357.274482755,-660393.945075097,109572.47907771263,583022.3071386237,202344.60487711988,-206664.34510133648,346607.46468886686,719175.6390282516,158136.66250863904,-712133.235518354,426222.9467521147,367538.3183532576,117561.4284063126,280479.4462014567,394735.546817509,584029.8893315769,467910.6934618987,656303.4928429811,102798.1615684363,364885.64245637925,597782.7470939304,405487.4854474594,634834.8644932844,-180789.74705820484,1158538.261270451,452452.9086281359,511509.4996738662,25038.022875662427,597564.9754758533,38278.09567872714,-166312.16035462683,894421.725277551,564782.7508949419,-512092.52029248653,552985.5573274698,18412.20770091703,-334212.84295763634,1060074.071799444,121809.66224869806,-64496.393675146624,323257.2905701422,580273.3187764231,307400.7994443984,-40394.46870854683,540082.4188259337,1010247.3320461079,-159105.92110968474,564273.3103597723,590812.0247428878,213428.42014364526,1743687.452147916,167708.7916069408,-435556.8039168813,29476.316724495962,597593.7725306689,204428.7068011416,417692.5644232691,237067.06401760038,641763.2318420033,240296.97760239383,619004.0854470911,-520105.3706361889,586663.3039230118,202653.32088241586,660940.8278563572,600585.8969121887,293036.00175976707,312591.17505628476,-95792.64487117343,-322444.8699746968,-694744.4878771589,618659.9242420499,361671.33623308176,322847.8836924997,168061.8856595126,-49678.8259812505,655434.102829284,514891.8985288236,-252887.60304891504,-244329.98192671034,462479.3315575137,107435.86553437402,843923.9607301271,60209.490674719214,-188461.83351010922,-17216.288686808664,404179.1182546895,470670.81472270656,819577.6437615417,313843.3529544892,191286.01295426907,-226973.94410240883,555160.6810765229,560890.5978460358,-210704.5247358363,587314.9514403194,-574415.9930186793,273791.61369651,-18270.43019837234,-3976.825793945696,914841.8265351593,404292.4875940187,14270.28259447869,371694.21452170657,-29159.224164307117,-488493.64882603707,69367.12677293876,560082.5334365051,553879.5704804435,-861241.4603449171,8412.513665236533,507052.8250825852,37721.32572969003,188968.70244687563,435071.03714513034,1067752.188764868,478632.47792512365,404497.8953216495,17487.1262429785,18351.917118179612,-108898.19879833795,-338391.10900438763,555953.1252469458,7914.374604687095,58351.69338813005,528000.8130527339,-38297.927880734205,-846875.0595503761,402533.7060051989,148268.46362720057,108914.62616775976,597567.8083023638,619248.6738098962,-45549.30212102551,135659.86608032882,636108.420339853,484185.07981711766,335966.8817791105,251354.7906553112,757441.1720856228,-554113.992907776,606113.6780857611,27684.3516025329,347353.4568255944,865618.547626981,-148086.22659026925,-389722.34854385117,-75088.18683973653,25371.012407694012,-574294.3342851307,193307.20219748374,753025.1466345284,327947.47284110496,362767.16848737234,170039.49598070653,561375.7089544423,277725.6107893628,10658.960306216497,398670.7740475205,169671.46863683173,-246240.29932628432,937620.1404274371,548361.0104276203,60672.85586137418,-163582.52161629265,610616.998270282,-317196.17795680324,-27825.716869189404,-783510.8850104539,116066.32082754094,-699348.5544378324,-235324.6084034536,-675382.1775627448,-67473.69482436357,399469.8076214022,80952.72859736206,80022.00481205201,561735.6331951064,-267813.48260661634,-229456.19678677525,737023.1252796235,165014.83632535348,-52825.148232290056,-606548.2788035416,444752.1630519936,-765297.338079822,731732.1386776473,102062.72755243536,20549.564141092356,48161.528420343064,-114858.22712507471,330746.5401158589,-167402.34081134526,446727.35060731834,512182.83057059254,-23139.05854597455,-144390.5643711076,83169.02468148991,-351023.23900681734,-327442.4486220067,1044455.2402532548,-342476.51736969035,77745.63484251034,214065.35591956787,-219854.3158782632,20010.385082181543,-7026.943618501537,467042.62760416046,284334.8332041339,-195693.35155600123,1056012.2236114792,-27643.01178772701,521823.5597795858,-170326.75246829586,530948.1233244664,513111.22318678815,330562.988693865,650689.4431032948,429090.59158672765,-339295.02840740094,-165805.58242610563,-6226.741919900756,493791.07296189107,-440956.2962910719,262081.8602608065,-573658.8564587533,-38234.02514945809,-345113.7600697661,139660.40365862846,638348.8507195874,106188.92058919137,191761.99134124583,63086.31619713036,415743.7220582254,508821.02762990026,167552.2023520479,453847.29436323745,621333.0139763798,602781.8173268554,-421365.23372246884,66188.4503839463,840325.6765871206,-243790.96711805603,24020.34715492511,65766.30667724973,170501.4557069135,-304907.4785346971,269801.67951363605,688675.9335525548,469247.22594122915,1464937.5166881653,785587.4888410475,359341.07940058457,511646.8074564738,-26487.549158630893,736313.7487737685,777813.4837172148,109031.76695160661,239743.78767589293,712156.3903208841,-815673.2344145738,-158095.26480850345,171226.19326801505,683573.9957009689,70674.65712872427,611239.5739997677,243320.43044828577,-286888.9896579748,-46936.363730286714,129056.91607855959,356200.65643500304,720996.8067774358,379815.14311615285,323974.93073908053,-418278.4919930715,-59702.20231805742,882288.4091579788,241397.67634912208,232762.89061520668,1062969.6696584662,-47607.91652482655,948998.624004194,181485.497534554,1072436.4013990448,733410.1960818716,104994.34789488418,238906.95916728908,145317.06161390897,154196.96014255704,-125867.28041559551,842219.9090722953,-625100.2417420037,-65868.12320048455,118162.16162246233,1224343.3772040661,-284908.42332209274,459697.687873194,440395.50907270703,1279119.4155938209,552340.1882168241,-449933.5797651275,930315.6565192831,752222.7272231989,339305.0762404981,-555960.3944777059,-168862.4925015415,302919.5526574962,-149572.10280850995,1278499.7969026295,-62235.731877524406,-288406.5592610366,340168.7042288077,-2695.2411939911544,-232373.43900832674,-53687.70729228156,74009.62011129968,612907.3139691851,946518.4017330739,896488.9175880002,316474.6422395413,107186.5996544701,-707513.353326478,458211.255091283,-120499.31275272835,609439.7368999417,79619.45927947201,36380.672426600475,286205.7921363758,316960.1344990935,558613.6667546295,-871313.2912187758,-856978.6047899085,495361.2358004316,205526.8020538669,-176314.25027919048,-142431.25786381355,-315357.0574890035,-781514.3978598458,428803.69974867627,832995.2612148081,-207053.1623069346,-220491.85549717303,496517.1956011052,-268978.1944308139,812914.3271102621,1282201.6559453944,632029.3993141525,-408714.0176178231,227461.11441688472,155587.0437949272,-178620.49607667187,298725.9819399365,-620583.7526629255,599755.9224987966,874182.036508867,25656.227038900368,872139.5446523936,458274.4731161329,787686.068557567,231751.37819986697,652585.4618009562,-207246.1094691581,24953.598713951185,-208096.7230079933,-264699.42520396365,701409.4023665786,-10544.559458849952,94412.56269122753,-11194.08074894594,393352.0010170671,-230069.82097580098,618639.046586514,-120910.87647283403,16213.960307813715,406097.9335518307,-28978.72335377382,-66067.39111034106,-139153.12182928622,922606.6886159731,298669.7082616943,215859.0550704063,341050.42900525825,361434.7483432293,413633.9339656299,355681.92523989594,708779.9007490762,32143.473450228106,461004.3700626767,-411213.1992100538,259133.14105312992,-74364.21203239681,287835.3676648862,-308900.9361053221,500032.5973831103,31753.707268572878,650954.8987540193,179898.0512931198,-218346.35748733114,-392693.47593473736,886892.4093246749,145841.85001636017,255718.69957941351,190058.5014643632,438708.14502621954,-508476.59819375025,-1051525.2557521923,374068.3083749418,577815.4232830857,498007.98228680016,628920.1401301739,36412.34525590716,805463.2554974761,-49503.806993582286,151230.98246191256,-253798.0617169561,79658.68400071049,-235121.3151527634,-325706.1930486378,-536968.2298352444,275428.62196522206,-434415.8724323292,755801.7435999517,-466489.27656763745,-133264.04484248767,422514.07725310745,370631.55157766026,69933.90151221585,-80852.08376535308,-367758.3151763906,203367.3494059788,-139534.63572730264,502186.5767879863,730797.1611036165,366672.68461400643,204.44595644483343,215615.539339046,1028312.654767965,-254877.22889098572,-243299.63417986827,-448769.8595961372,856977.4817290991,446479.9569399832,-113536.83543946175,-372866.38324748585,1041787.6940824538,913936.3348081317,-171041.8585735862,943171.650019641,503775.1984939212,-639351.2031466351,475608.85217741085,-110091.01012130035,1302909.0797454384,510387.4153292356,138777.61301921634,240698.9485661192,-64442.8117701998,-272494.6038312912,1354691.8023795304,601025.4518206753,485716.9328141832,204779.26233019726,510572.3690799866,-262697.4790152302,832941.9398978669,722119.358359437,22373.115952009335,-431765.84301708825,818679.344130856,181513.2115794327,-118207.38303973945,59053.50519827567,421848.85281324433,121166.0920725041,89675.6533867484,52588.90285849711,-87086.29118314758,-205878.11346958438,-40696.549134409055,912917.396579315,1047041.87999511,229255.21265264368,-36100.246382856276,304933.79694595886,-146896.9042276321,-307374.33572507603,90863.29015375441,348058.8473232724,897701.719818816,-391868.93497472815,130728.0669924072,1044413.0541067659,488030.47001723805,610080.9558146405,573717.1134320819,-562123.2010017927,402372.3289597705,-452726.36147225834,1152329.7840674445,331764.42982766917,309158.1867203065,102974.43187681958,-513297.3117179144,317139.0577672017,547589.5785008669,-190365.4834210202,231527.7060951367,171754.01625130558,228654.55268020602,153854.3335360177,536550.1689609438,-353826.33355089976,20857.74797094334,878067.8912658221,-288145.9708008184,-803265.1804771991,433817.21459695743,614528.0472336672,1029569.7520768801,371179.7546011959,435455.43686814886,-248794.14673019154,487974.957453053,-89504.09320370946,-110614.57354091853,-115823.68898304692,468739.1185359992,354014.7474346943,136979.905759315,739398.2154498557,-336299.2568559055,489659.9135965626,468872.4042102541,780478.5202710326,618362.2201315016,-634586.9380245022,-298565.3424086338,293304.13457839657,167806.49046775,-157890.6684769676,-324493.51145610446,-123907.55706726387,11477.247305728495,141282.87987053348,86057.15250158403,-104899.15715585463,317164.25610234775,513712.767290595,394994.3908547042,28789.914435836952,-7812.073419936001,155790.58261248097,748933.6822062572,-567265.7302077492,704580.2761695231,573915.6247275379,-21096.405452054925,550910.4146404322,295189.2024164847,545109.895979228,215919.85383504815,358007.1233552238,164787.0065999655,389596.84525192855,702905.6056535449,528970.7600317122,221246.52201376343,136980.10035447776,-345393.1681155483,907255.5834848261,-7665.678737010341,-492030.065829115,980252.8909262754,-19985.61575636035,-331487.68141362583,-60450.1227243077,-145480.92636843445,390704.02581145475,-334511.99994604057,-1002236.1883032885,-84753.8872067798,485945.55185451545,-744743.9061095212,-157387.32757383818,240178.5584623823,897092.4241695479,1143322.4282765905,-28997.717836360913,618270.9839162757,794498.9662617203,134915.60678344313,-297403.57713889703,-311948.45145024545,284549.0892956676,-783638.9549522293,189875.17397144344,337899.789495694,733162.4551959988,482376.7421158687,265904.94340785826,-276888.3105924716,170400.35608111462,875563.1935120155,300500.8820153149,-128501.84931933321,153173.7452733079,-127633.05719238427,574181.0888965721,-234760.58181352448,-634002.9470313173,-229793.25018857606,270141.6758251656,-627515.6180827492,804366.5325929979,866925.8351691863,150382.92507257266,-460390.57657394605,304643.88146908116,668248.9350069026,-2596.9606875306927,415828.3588551483,-72387.32209682977,316782.67644020775,-146318.0676165889,456301.9646346257,-968470.1913425208,620798.4082935299,675277.7573058372,844133.0476598376,-39156.41038163286,-209130.07825143915,251663.8289457038,-430121.9297525049,318810.92712728167,454569.4873103504,294226.7615762176,-121398.19420234626,-65570.47686463455,564324.3937189206,-241439.9361137601,503221.3206461258,-68345.80819420936,-96662.1146065616,-971167.985033781,-692942.5798056731,520583.6385514322,557879.4884598539,686446.9142233748,409428.3205990135,349851.03427088493,-236236.71564031625,123407.99479595013,-193875.0012819986,-86222.42567069456,417472.8771702801,-100029.55405487213,27474.985156028997,2806.5874762604944,1022930.8912875163,345448.0312910024,-59534.09941762639,192724.57590605598,613064.8870980749,956512.7934120744,184099.23684967263,576475.6192235253,296701.7372926953,-164583.64514947962,674585.9313566745,-596184.9394073002,860547.5043103592,186707.3734993562,569147.557072707,322112.03652269626,314944.3468841957,139342.9456844665,215294.16573481634,-539668.8763033245,418449.5842197095,85103.09668191802,25576.80960553838,178347.70704273134,740072.6207991587,241817.07410375495,858793.2308831178,176519.21567901643,529070.0943519245,-287833.94110244606,-231500.0533301239,335668.46423321357,-138526.42143336963,-50135.64032402355,398237.71010618284,428867.5227429173,44539.890038120095,110000.33472006628,683406.6517363717,327288.30912902625,775433.6620080099,87560.0667936746,191352.1609896198,340033.5885588536,160591.54212231608,452908.8391697253,395859.55991314584,370235.7887690631,-16644.61479858402,168892.30645190505,-51529.75329114776,176735.01382374438,398658.6895584846,573632.8112642802,812687.8129586047,493660.4632576476,311671.0652246908,-327112.19929795526,235552.85691733006,32930.08942615148,686808.7717039813,96366.84594909567,388095.7083906112,363458.3394479868,-874476.4383389149,178592.59430192877,-225839.87914414518,659181.5473162681,140050.60472817486,191263.66725936718,641188.6168369437,645100.1844011913,684348.1329086432,-355870.61631321674,-104072.32499746094,630110.4310635184,221667.9402914783,237919.75322466856,102345.61393565638,250583.31868998194,322334.7584269317,-143321.62995863752,-894939.702859432,295535.36169449147,201746.29313769657,-75730.98844762752,465426.97000183444,18195.604647982866,845734.5342250317,-128037.40904356726,-385082.054625981,-34975.10008967016,-3780.3213292281143,868564.1542884335,382867.49506347766,503964.151399232,688735.3320886143,499894.2146440996,-215573.82261733664,233315.98832917167,189768.97219378315,849759.751407084,-438528.2983583957,-153867.16306527657,88161.56750497455,418005.67313287733,-343999.4925913201,964209.6615170492,472218.4148349534,202745.57466227002,376856.91744717816,-218907.76534764422,178265.93641449464,-131005.03381306026,538014.9836516608,-156834.99812959507,-37145.92150526121,-413773.5407561469,-18110.503963520285,-179816.57330809627,-310555.3635514444,139008.96325023938,507400.34253614536,688470.422765546,236404.50130378315,-322098.35740519036,350203.78025145875,824.6963923270814,569735.7222842001,-44220.27293154737,367498.1645229291,-519085.84979067696,73302.52501374576,466198.24506331235,355185.91978837457,-598263.65857914,-331412.4582952219,-260336.88509979704,390924.1130870851,300172.6838720427,544045.7911894196,761572.3048197981,918991.2474236228,517680.3755197013,99961.06070404034,592047.9833922824,657578.8818860971,779520.9754671343,835700.0639638519,-200737.25612502964,-13394.29515054822,97059.63898303872,1517230.3731750625,-249351.1220042156,-294272.53168992326,23024.871920516714,46460.49681541789,832970.1894949642,569696.6084891451,410044.35310917534,162017.0376098277,658821.9185179253,238661.74539113278,891291.1566976104,481594.09318816755,622341.9380734409,-372916.4358031475,359588.08431206597,111285.33725895826,716143.1485648281,76438.3362047323,-31394.123409093823,825538.8046010961,373049.36839978164,-490147.2048358931,-32214.6140278494,-237679.4937710506,255596.74788200855,330131.8170976578,590229.7612510431,195829.13063843083,573254.9612055016,190184.76865420397,-556868.7959810821,430496.34000286506,310411.84943647403,928723.9383309674,408024.3053383948,257135.74777659774,-187161.8503105631,282305.5437214896,619022.9124166416,-134753.5084500676,257181.21329951938,-319152.8098011636,-808000.7651073197,-715868.3545269524,410438.75656832894,550976.5957829282,67878.42790497374,-535006.1476021232,377968.9541190965,-58192.34193872241,-314117.29729300924,106425.67000506073,584884.1831881744,509886.8278666558,477976.9858139781,436854.9245359441,598362.8898315686,49392.304984352086,201462.4118767241,428464.7505816752,246163.48091961397,-130430.583889117,151624.28583435295,1159835.9947021408,-113982.53800133942,561503.3468269161,797679.3544852096,-386180.79902960453,89797.70314636454,-143128.9350309223,196829.08491626428,-373534.36238660244,-329053.1630006172,-178904.9068580214,-1068797.6120853878,363628.1221300056,-120600.38107004343,-428552.25779913366,97826.23181387363,-152033.77876709076,148406.30847719545,581706.4608949074,177740.9518797025,-368996.6148342029,165919.9208611478,868148.9026452978,641300.4131820239,231013.2382598035,-496032.8034745783,313609.9169352199,-211965.88376442483,-256295.62164682988,1027596.7656078506,298988.7506899843,494636.2238489813,936293.3285332238,1164771.1036521546,-589015.0965348617,76900.13159792963,-769754.5538489644,257085.63602849282,512620.28998934384,569679.1337357755,275857.6279070042,324926.1149040824,437834.9761752491,344248.2690269472,-101773.46951013803,-318303.1944713462,90868.1938766283,-458116.8440980604,221197.1487627658,-684821.4962137239,819491.256831625,308504.1938508493,155602.45525615802,-296680.17032167874,-731394.697518562,242316.23211499723,-380809.30920712696,1017908.9671838321,5933.3759728511795,261856.30213423818,-304866.8512509316,617209.3460288886,134407.2370734983,299136.62037572917,141009.60786760552,822923.208648121,215443.11484678695,228769.09187111398,624872.881753196,344611.43865495455,113003.1697739875,704764.3430602481,-283720.1547794109,-335750.1681614602,471383.53948624525,46685.08987804875,871583.2721413178,230020.4586774176,165795.84985629935,636133.0622999787,545760.7656758339,503320.8941910132,460365.36115981126,791078.9134092988,416997.9313322683,84428.51907075895,138382.918821658,544718.1140359496,-609267.1467111944,-222591.8826275973,600304.8803718663,227314.46169598913,164939.8609833978,-8379.621710098349,259334.15106617287,660499.5205755078,-522581.550483936,486661.1261370359,144593.13858979894,625763.9163602758,-242458.77782616252,48457.22199852951,-75022.4546319046,599276.3934410214,-225662.9968765797,452560.5031231027,-242993.49250832433,10574.623665601015,-639007.0036587939,-636670.9396508327,79737.60594946658,-577575.529580636,-95565.87440056028,952315.890813339,383328.63675247505,-203172.2591219116,374512.4772904874,-283687.0001728344,-282493.72445257194,373429.01304066414,706220.0333879753,41913.20534505742,323049.35168609116,555106.427810051,-625249.8673674292,20937.079815382604,-285325.7925822227,739652.5249843858,-242971.61814306676,-279765.74026437383,391443.6264638775,213503.02741201455,-11124.688393326942,-273631.82172968425,-852035.2107393984,69354.11348113557,78064.31761127291,410654.1924594133,-3274.2510753599927,952332.2435265584,9188.804162959568,1018636.1592404665,-65854.4933978985,-51638.35699932417,584034.8746854523,-61949.18656447576,6830.574492739979,-107163.93376917625,347874.9104178832,-939646.0785564973,175225.58969947416,560713.2885552607,502121.05947897467,-51309.92577438848,576956.2308293232,425656.94534148276,-203234.24668948865,-330955.8557304591,333151.6933797193,433086.72067652596,377672.37799664075,-351454.4289296749,862888.5899953227,456037.20913529117,745139.4322545938,286107.62388932006,933239.3762471238,162150.498261909,185493.09070687927,-289452.5675677997,728187.5739473831,166970.587375049,412591.76782013476,624448.3815998654,-86272.94729837636,142086.5999703398,227267.15634496696,586012.4940251601,90672.06376894191,-226367.49810265936,-750.0743636367843,-542719.9783912897,-121379.14441686496,642373.3465454369,35465.15141930664,451549.62319761375,548.9079898963682,-34384.173348030075,-108778.77142909961,630592.2027177862,336233.789689729,396398.5866215755,-721106.1938418597,244506.2894732016,191762.29700058652,-59120.11387062771,-572187.6605296526,403438.5351633397,-300156.3275804366,1164736.9594792735,1596042.5671687368,-480987.9148299461,-15024.888798331376,-262160.19786157645,-494307.9707401139,1040932.23933317,-312702.84300363064,1087193.940993783,556588.5379065764,-271094.22813727893,667950.4699942237,514401.00089973444,-259154.44349874603,-277296.6903228243,608249.1466955347,512252.73643277306,523571.610520686,-42430.244425449055,-644292.27205777,-55326.284411086235,211629.07041075313,281891.6200584341,-504960.50165955024,1232391.3669558,846043.6542850924,508775.83425729163,-16222.138178880792,-459925.47816227516,211768.1316734138,650682.7326677712,84325.49082751991,-40941.98729685694,271096.2957292199,-239625.5673078075,230428.3255396448,342906.1638560025,-164160.15863778163,532762.8128916444,151952.93687346298,-267391.6024142192,117452.85856323689,654853.2651771186,-171155.2623267779,51007.13184466353,1029057.2845489108,-236951.35958687356,-465924.60028304905,540051.2931268192,-122166.34511224553,89325.2142893793,-428995.63274455164,732482.4470148445,611001.0053414633,967403.5445340937,1285537.1905307653,422677.6712300414,-577986.8104956085,-631755.7672293177,612381.3382926993,-234278.07536185626,342919.9126005075,11202.297772742342,78463.85908513516,53113.83915505465,-114790.43463216955,685691.4497115128,-63939.02730580419,155360.17637293087,-505138.7626574999,-128148.05656793108,-20306.201203230303,19186.77525543049,109030.46837017918,379635.9004471693,1275861.1487238756,537309.9651371855,351013.3815090512,532665.8486351436,568342.8318584855,430132.77430671826,282956.73277818365,-346643.37437160965,-189638.70883932337,-23670.235252258368,-190214.07424731506,304199.05235215835,-126459.56840025261,-489896.0835993956,84463.58341547567,-538611.733777456,839403.4267959252,302753.02098011086,283079.4652632163,-752670.0903013358,115759.71678893315,200882.82694979338,122398.4261144111,420254.5339898914,1367628.454273162,904764.565950775,790545.6454831897,226710.0154570304,636783.3105442119,-241093.12612646678,818419.0765741621,27773.87468038313,175756.66447108611,-526174.8976908741,173514.16155247856,-317692.5086428621,61322.600559277926,581061.3314696318,-407086.9021910974,718986.4624747927,1037369.6883992231,76788.20281576458,-164185.47016675957,-43671.88736555679,643111.9464756879,-360271.0032317224,-420477.43739733193,-10084.490289676934,339231.5544123347,120123.83726689313,-342728.27140563214,654518.2653531851,-137338.00099593727,334906.08256375697,646909.8786268667,-478984.89871965745,315247.2940583825,-178180.53013793705,-125523.29437332042,-524818.4774969551,50127.93751614727,260735.66430699872,-96966.08823101781,432572.1947688265,451929.94306435157,-171052.82931089168,87688.96573988954,2408.3017512965016,161734.95464252075,564460.1257938859,365425.4388242024,-1092208.9294907614,28907.48880021088,-204165.46218395606,-347508.7268567425,284734.7776234932,271179.7423842689,799354.5436920077,87797.74506422412,261917.6284999363,767611.1725903018,-424177.52172844065,480386.4757334795,-541166.4573328854,684797.143645694,344905.9881240777,725127.0493399934,811435.6389672663,-277097.7756286813,307484.5541305519,250164.23791230842,584632.610835738,1009196.8172629867,753623.6874428065,-27730.522691211663,387990.8735428122,41333.372099177446,950345.3543435661,-372591.12599411234,588223.0926436596,458203.53419150645,926958.0635811174,-441482.38405437814,582638.0498129334,-167092.7711293432,313137.6795030553,-248154.57009171043,-543777.0829781038,-349465.0644340068,224281.1451148442,203202.1024226048,-589945.879792653,-107282.48536098143,-396794.90783328563,665198.0408875155,805941.9339888105,-923777.5351810635,252340.94201546395,524781.5494049666,-755301.3605512369,-98392.70243717078,-879073.2054115259,-482198.27156620426,182386.98570780316,678677.2874513697,193427.16624704376,841807.0832426413,-101654.41010408942,-697348.125941637,904030.9893155918,481168.25223678164,-510424.54770469153,723513.7189239599,-216862.90436476516,895192.1383012016,454548.02023984864,-19447.39013696648,156869.186931849,80691.25325802667,850077.5171106975,783628.3311991361,-187615.62609819882,1126498.8454511869,-61327.019187591504,394017.4958522259,-560414.3506271532,153005.95966751873,505493.895586601,105319.89426026586,-319956.4507747819,75211.8011346194,350738.4873200711,-11957.900557952933,676931.7983670868,-502104.42078963853,1593679.1777274846,-94978.79781058151,-359405.4173997864,296656.5815134798,642664.8704655282,910380.2591144764,22813.947373645846,-18536.556636053137,150559.4482525047,-222381.82829091884,425824.77234590985,387465.80197703,-37430.96738843992,169395.60169922328,623830.0704942751,-223266.9400787456,448418.5706003099,-60378.48495544866,658956.2114619417,829689.3208752656,689773.4227938028,317080.75809465395,907603.7086637127,223823.70949638076,-427119.14820051985,424278.8792383452,-138517.16853829566,352485.4302888601,97865.33483380359,156726.88956629578,734621.7173445532,481597.8863009671,522329.4872500608,110890.44090404967,301955.12045231555,630512.8611636963,-719926.4782734918,499809.9929227256,961711.9940345604,333412.2561502978,774130.2901323652,-532810.6575048408,328606.78781630937,834224.702722643,106843.44611896947,1376213.3552935175,243212.3417888577,-694101.1065374692,646866.6341188671,-51119.86110154353,469266.29240396107,551596.5534341712,-112233.99660965381,-302868.0597267761,551050.0884682522,149712.4904794977,-405813.4877521782,963453.0952181774,118623.1813039314,-273218.1097514285,36870.7569625848,-410062.1393772857,564634.2794323368,-434829.1847034078,-539583.7428458214,-147196.6012474266,395534.56168775633,283951.66071769455,-205010.51516688755,-2841.4758987678215,-445255.8217105558,244756.04620841565,711224.4984509358,-339624.62344116904,-769676.6280309409,103514.07835002802,151652.41229233192,561645.1286674202,-291152.4722872735,-154977.13090322632,125456.76450827159,-140258.80379712256,-878545.8191420175,-13589.186216433067,-405949.7561939128,680943.6373883113,-42606.486821553204,368538.7611642261,962365.3836115315,90335.83016111795,166489.29093600996,191848.7208884759,129326.62757988414,-599025.8123376574,44774.2174451896,561190.3999096453,1315076.2179495238,-154077.33779052878,-738321.3579411362,197998.18698890926,-554351.0744281327,327215.7593604056,417297.2699654079,953130.0517293816,-160389.29356930684,-193720.87184940884,499042.4896327141,622448.3860723143,-84708.54610870779,-338247.53824738506,106719.47154683061,338800.07219930505,588033.5399033399,-1147718.1501098392,79743.86109992489,55663.69725925848,697463.6460007494,740043.2341568959,694275.2174286521,790884.1942012059,-548772.6663413001,-233819.82391749742,155670.83523743926,-211932.799528745,658893.1703484762,-207608.67428081296,-198067.69422822166,-299217.1801469466,278191.9919197536,203412.31185437413,447688.7560902317,571423.4463770199,64562.128590723034,4866.561161705293,-25949.479020018596,423515.46816841327,-68696.05221823277,378802.5248209904,270322.06330424873,87467.47541920329,343546.0751685109,-1056550.3749422967,389969.169759206,279524.7199137877,-50984.57947928738,65724.10011074785,158733.9339208873,-937263.2994659375,-524508.2674153713,359506.40565769654,492458.52717591636,151907.24581375392,404683.775327181,-254650.83853324503,-418309.9111348032,-158766.90466457605,121003.85246120673,354004.16596505465,91890.71188771725,-333081.30664482526,574324.6617411568,581473.6027737316,105957.96509874891,660668.9424630525,-358895.2334903106,197294.09100377187,5010.628423896618,321953.30094352877,191474.7866131803,-533292.3093852587,581053.115872107,-98223.21842023358,-52966.4981427989,530830.1085547865,194734.20521311695,99505.22242926806,654960.7696500616,817933.1258264426,425169.23892927775,-492371.3578199982,-41193.972258360125,-317343.12571523385,-432282.6974264765,-332124.23967962107,321895.61156695336,-268430.07515915297,-427179.4252799279,584401.0640848926,379384.21991366474,1037900.5572297545,300706.73677717336,-446069.92379811313,541859.1123718973,-175010.44574318384,580002.6582988766,287197.2953441711,276044.88763282914,176707.3753409502,80956.35211701132,180378.61980003119,129593.42918265657,-196919.34390538605,171840.68127861805,-467332.72843482764,206459.30162570206,900416.1595162866,781846.5760662518,317598.9025717848,547466.8235063534,325299.0954764527,-193383.2145181517,139564.10860167397,-230811.15341948532,108712.90524151782,93967.36934498698,327288.88810254494,491416.18744341563,503910.8356256387,-311270.8058154378,1631756.8357879422,-174892.1907679052,650433.8285320788,-166314.4480973552,231200.68636688218,368924.01222807914,-139105.20925292093,360057.8520130003,273871.93184528966,816043.5149157741,610471.526714047,-37373.99031481566,152049.84101660037,-25399.331629190594,434493.6561843902,-780723.1394350715,-58931.942770688795,-339173.31906293286,320277.23114862666,132282.3480421137,-995263.6790209208,-171068.8097259882,51499.79536798736,-600776.0436530276,117181.04575923877,-231898.06550766155,-143729.73100149585,-579292.1760080792,-722401.1563571487,-407290.34889599634,-491683.2819017945,-49451.7911813329,19509.177271865774,-1043031.6996079865,227383.20520493947,124193.7614312293,43455.51616873266,399184.68279297836,483768.9724766896,-44715.913513229694,260562.73642014572,323157.792920331,102551.00162397511,122280.0419989326,483469.5057377005,-121812.53658106597,264710.39897110127,-274103.72460509,743946.3241218468,892923.9729069262,-336772.19543123664,232753.23965891637,-523740.3402498467,491441.1934827692,329320.00614126306,355857.0864107269,-400661.7766378997,1157456.507116064,1026689.3888700507,-399335.256170901,410430.19936871575,392515.2019323143,-755753.8698086035,211849.66318236198,202996.33762541553,228376.3359497781,-494769.4599337932,653613.683818582,661190.9579446088,126860.57573887659,-107943.8908037818,347991.5888050329,887351.7666332466,-83814.72219257755,-315309.5729329963,82996.87787178718,315035.66477465816,449797.80454635806,421639.6305254232,113680.01505778218,677664.9899745984,-236772.74839013908,116270.68423054367,170797.13255887013,-120448.24347583158,-586041.5906779245,128167.10065191751,-211028.12699028663,-624074.5443140594,-128546.16013583075,952038.4084931742,439540.0014962931,-649384.2379924888,-26401.98466612585,-568975.9719185806,-243381.26644361624,-417597.8891391554,351998.60594730964,53759.60195513442,-105608.60992165003,356720.29922093824,717117.1076940908,-140052.99524537474,119331.67576777982,887851.372348974,-404655.4567175121,-430595.32516232505,-39386.4512861548,301116.24828301463,-481969.1985910791,171902.9708853159,424088.821286676,692695.163832874,-607717.5415824484,-42825.810008847155,-584202.9687419641,278730.3721254431,-475657.1556303473,323464.93558579776,306212.8869867516,435078.4637755011,-433635.7241573883,619652.6216373714,1254978.5096009444,-37163.93604756612,-466375.8376719048,152453.5130979824,-636741.4476339701,395648.573791211,328646.3306982601,-117325.46545392228,-189151.61769166123,825057.4899081646,-152032.07204128942,-157363.17998449504,233744.32964087278,468620.31694007246,73118.2308419696,571863.1115428857,57491.391991647426,-90105.58167009475,619294.7011659369,453599.0099575892,308504.5243096105,-718533.9425458703,556736.3857936086,-25155.229167683516,648557.7940736911,850120.2861523149,1139794.8835210633,43658.627135502174,-147493.72438702825,-473128.7332132077,1516747.3930731472,420219.90858696634,482999.0268370551,105306.83289750246,22611.597970929462,-64529.5404368774,505232.2807754027,505998.37018988095,310190.2899660305,-16029.91822100291,220387.82523975428,-8810.669053600635,324578.2846331089,460049.4487721473,509502.8010318554,-272938.4220818444,518339.1373175471,-267051.9299419611,84235.03650558135,1026215.2506862623,-121095.7978238212,30436.488300862256,-669325.606400515,368803.15082979854,-686746.3614391645,32086.59018583875,101275.44423186593,-115517.19742380828,311880.9459011825,-318437.52705821814,273797.5817876733,209273.64264167473,-669019.5279597405,399854.4971741955,118885.62669363245,60406.82673915941,-140888.5113443453,270433.07146619307,47059.89580198238,630905.1824011323,317499.47545610834,1064496.1690659658,-510889.4967227024,736590.168515637,495390.2522376403,-661185.8776243012,641389.6173107312,-60973.32287783595,-164505.29279667605,215287.69295940222,988578.3783060103,-887856.9585113451,-496624.4324968071,-269315.4453009716,102045.50346317934,279549.6402794323,718572.2995894263,717849.3147920747,-23733.877708078828,291690.5789693226,277612.8945203377,378799.48838408384,446471.9868446733,50609.065409010276,47143.17290303297,490297.96656297706,-130420.11883279542,46515.92933226656,921898.4629974626,1038678.0857626311,-64413.142379825935,865838.5752826701,91309.70962763345,124528.35670102295,766142.6975447112,100354.41793473903,1196076.0604756935,12301.545292838942,360108.6106887683,181969.33462809818,942921.7681949902,500210.735279541,468682.41392103676,470061.4575318019,-531123.3032420352,-77977.85747967241,808708.8847959517,821190.2514725518,-262300.9415465249,-504752.0699941749,-419746.37568819197,143810.20225094026,-230909.4230979411,320977.9905356183,-92827.48839868186,278003.03934837924,377834.6168451309,-156903.88319361722,639741.2780272658,307093.66348951403,-40306.36620778404,273773.28766141227,359503.8153082151,-141612.58826355217,-203167.64461586252,1050423.6537704142,749996.3672612077,161822.4943053648,209494.4620742225,1073792.7845357656,362295.3775735586,177518.74625829235,-284131.5945321545,-682717.259920358,-85000.55732616084,894380.3908613548,-316765.43726214767,104717.5355186616,1236582.226525941,-84328.4265248673,-111812.45526891993,933469.3164124424,-544185.2498125928,-285606.4854223775,56448.165428051725,790854.585345909,563973.5244353353,195329.21404349292,-633036.5584427845,636446.3930582572,-132280.56653858395,1344873.2269157327,619794.0752966963,-279607.7859008787,464424.99476830475,817942.3408326688,143072.16027838364,314350.46514090477,-947855.0445532692,346470.6036028685,971845.1762350327,-770149.1539174351,919656.4762509284,-349299.67701077415,132663.80171225173,657825.0259826374,691746.5696074273,176636.13950671442,693321.3649848867,426537.3496920592,314675.5795959062,335022.47318141954,338019.3932951479,537007.743084155,869227.6201148322,608469.9255894404,-152837.5087836897,271048.7777627264,1123206.3699009037,630102.1586261494,195668.18270089198,506591.9590090099,625420.5654470639,367302.046810518,-248726.965809579,51107.728526820894,-480899.62291699834,83123.15300088096,-47856.069619649556,645511.4966114131,-74451.88674063748,-135513.03067688178,664583.8860111358,-106054.00650681276,-75370.77153983014,223696.36860588612,835658.0022696224,1084864.3467896902,54845.070822695736,-236964.38170337258,650192.4449577047,-257310.46778526204,11944.030558436178,-16032.23259913316,439582.42266113916,201509.10119608557,282643.66710943496,172762.05572235864,224296.44892089767,-545999.4664180493,291831.47541475995,999256.928380549,1043416.8220851398,-325957.85252755834,424861.2077177963,-79360.18523062812,-72359.04075359507,462529.84260139614,394431.7992147743,494104.7727714111,361865.37543544173,332464.21634852793,89549.93305305019,-2523.8136859694496,685971.0965098222,317598.6117249313,488360.143170496,-47309.91124534048,915078.8737022006,336307.10765000666,-331317.37439985294,-484215.9590772162,362518.1978806006,264236.4596862667,796508.7779370337,523528.5949809286,449780.3269720692,689987.1022481839,-20620.536179815885,308659.454802325,329290.47887036065,243862.82288314495,770957.4402796756,401121.75559902564,512533.7146936166,-65517.94051046437,335659.46918666037,18314.297933856025,407019.7085812888,807784.9718165491,-197582.01944967452,-139671.99602207122,-129036.41732450202,164837.14831829444,-759543.0026643244,593168.716532608,-412161.95845769206,206813.1485650884,89758.10202862322,376585.7941503157,903290.2442184649,597369.5723315361,-310530.55398769956,650624.7624719432,272511.5933434698,577557.3492265246,55772.43878003303,576037.9042992075,1198754.3535994901,27881.869359445293,788543.6138883978,-124792.15051471954,-169352.2645521937,440825.44658935745,-293555.83961451193,-737708.7716070246,-625038.7077794755,-691527.9534724499,15593.62694702996,1201381.4931070642,-148925.77042791853,132854.76305597555,61850.803085641935,681897.9266453888,-272640.7359761046,1050113.5142237553,-232368.2362057711,-517292.02473334805,924058.9053532858,-183311.42552999966,338603.6592713599,-25900.660898815375,269699.8128087637,-392652.44941952266,-87415.99325593282,446896.06876348844,212324.55793163553,1206981.4300483079,-439796.49094052566,40528.877866122406,176113.84566366905,457922.7264610124,892618.7400706857,73083.99234760925,-81573.1582586579,339252.3716028151,186289.71033646213,344494.3161122091,674851.1322761499,108074.85967662372,-32305.229245974217,-346305.1055757962,-169904.7935183281,-347229.752886252,644278.9281949224,-243264.61084009847,-279588.60568017233,-175973.64002742013,103434.91612598626,-655056.8286803095,-401596.922405852,511681.381166514,-246262.86861433415,66063.18961948063,1133694.5716414787,818686.9796365243,31969.89590215357,424205.80673375493,146489.92930441909,-400888.2087685992,14042.242680433206,268543.62282392103,-93575.84353004955,204167.8004678283,-5358.494118309114,80983.52648621565,285811.7222326114,-52997.0725813061,-172262.23722945852,-117106.27418252034,56098.82322982373,671548.9529748638,402545.9330010363,-1032549.4532990358,327130.9406395457,698792.5527783884,-271876.0963020199,-69894.78779955208,504619.3181527918,-96073.64335078327,-380964.60555524565,208586.15240599262,-44572.40542392526,349342.49853552086,73166.43230169406,-76802.90132326772,-354082.4248545398,884692.5444582566,-548631.7941214629,-561890.7174299415,352627.32252194267,-234178.0227683331,43075.662664979696,-289964.9913005787,-470088.8559581889,730475.730626435,-165949.23837979045,797401.9751491109,303438.56906814594,-564834.1473869509,476331.23587508826,25177.275715729687,119004.58366865572,109893.94445889303,583936.3337009656,91096.5909640342,148787.28324171389,353840.0095143751,-660414.7715081424,207213.31047414988,-195073.35259486455,288355.12856010674,62041.74258423969,284122.98528390704,392470.49487985065,590245.4548406089,1157377.7434362955,826267.280620743,351511.6055705063,-369823.5806088578,562078.5095400382,-676035.2870610361,-223224.94063931983,518288.60080996295,-282457.32550644735,610033.3635583902,589294.202172152,283683.1130452035,-446001.35537881125,263435.60031875176,171230.24476481183,349396.5484716669,353662.9250146565,181615.65557128517,25304.83399848966,-142405.1236262261,-62472.33798771771,286730.85297143925,-127337.64561626641,-99998.57513825828,114993.8592585302,-250180.6620152914,-724389.6057828832,322409.3842513659,627033.8930271566,858575.5911919288,-56852.5256915139,-278762.11595420307,481233.8116701981,651683.7604511301,463379.47077129316,179319.9983064644,712651.8614737839,933600.0966935041,193396.09593255725,-331662.4599512699,1408289.016838354,-162201.55373960547,742941.6824613349,337258.7198542049,182186.37395981,-211599.8723237305,-54027.717764335684,116264.55545341689,180306.27994488366,672604.0639758371,550936.6078830534,414539.3241133732,568646.9793651993,588484.8760676654,-37652.47999418061,-571287.9363861629,1389857.516423149,-266071.8452852038,242453.17514398135,-12591.664568564855,-483935.3164387811,629000.1612710105,735506.2446229924,335313.62398208724,1103754.9294112953,28193.782465812284,90362.91971775703,199613.45037314994,960217.2078610603,713004.8584858095,200660.23838693695,427998.3541237279,609415.6000320856,825556.7847024044,156420.14408684708,667081.6430987678,-287754.19066159334,-292850.754455097,174103.92282288615,459258.96164974384,264873.52588771936,584697.7426776718,395519.6243943665,49149.998201114126,242480.01322403364,138537.12151299883,40546.70141871972,389578.12799404515,-309201.0431079506,888029.0744896792,335439.7196367332,396877.44575859467,366977.0220036027,-244623.96432191692,-104279.62390362518,333224.4147751052,76133.42744119931,617509.1817161231,205819.10925127752,-397391.84022711776,177762.9912949209,179118.2225996023,159058.18937417073,96190.41494813655,-43584.62906169612,127179.37782406388,278724.8121384145,758840.7453456984,307222.8755532447,637699.5399939492,313236.8116186969,-187205.64917968167,477385.7666220786,-797345.8682715325,-159537.29313717643,735552.7417115574,621073.7756266715,293675.5937183988,786226.5575755122,908139.7843251792,-829421.4730864856,209346.4327123221,399307.1680049617,818004.7915339246,13050.797270359471,812990.0384169952,175233.80929757748,369876.2044220674,589335.7075476479,708205.4828856592,-475616.45199810853,197513.27837993437,558458.0784731745,183892.58115301142,239146.24156103795,-689111.9948233706,427748.55340750376,823522.3671654346,223480.07445541257,213076.6252868129,-476839.3569476409,1337597.0425241143,81485.438048054,-102757.0132414964,906874.7396344221,-73081.14692316903,-646856.3066965942,392433.4209751352,268306.4182431968,-419644.64086065395,-356170.149463119,160057.39038803382,-297358.2920413264,-33516.697593424,252556.5074729924,109600.65997911477,431582.74346813606,670362.7019118592,526962.2791578337,-294920.5999260689,337014.86447342485,245610.63367284276,529525.073312826,622171.0691894344,229534.1292748456,-63943.37832071958,-160510.49802659778,99984.81346905138,194194.60709258774,-637623.2439672854,-42945.76246868959,714251.4419199913,338544.309835366,663230.1453067246,407026.398510552,859191.324963639,313672.36163726775,243401.02682423498,918730.5817036009,159898.84321488393,314515.7149374634,563054.5848506512,-12437.234545458574,-800276.6302326813,574323.7427780242,-344430.9651011354,87236.02066674689,444350.5381668741,173915.07987429947,515265.02063958766,-24175.72040510364,85076.29319048859,-355527.3669643658,450346.7325884537,-560210.6524156868,196012.6593402354,226137.9500504816,328312.081778239,816223.8755827062,-197864.81887634797,157328.76696154522,307555.7373697837,297510.1994433417,332712.39260268304,-254299.87941896357,390781.84225052735,352051.7384546208,146343.92057629488,267215.03903328674,137067.76741320826,605892.8278354495,-158512.66131693963,64164.033963184804,-265812.3436157466,354553.4424124891,232176.7861437262,200695.3701657448,-61725.36041805241,60747.81617090106,485092.7680559051,1352940.5416333696,205291.3768430301,238272.67575046886,666500.796352502,-74835.5878989012,471959.6920825299,-79379.00668057892,-4601.437215175945,267686.59027262963,1028075.3099093512,631072.2471037437,58276.57630057959,202939.59390174644,-732773.6954122563,613610.8495888975,1097904.2556683435,309197.48419354204,432177.7341652787,450398.1231892016,656410.4330492425,492798.6315783798,74922.36840882897,-195775.377985843,-102131.79515378596,-8010.724940226413,-737517.1849466306,-507851.085115768,74522.89191283286,-200580.71052508336,217181.85504320776,-300614.0611293963,450827.85022963537,-437994.07621799316,388635.6360274488,-390182.527660002,650508.6695807581,68071.08620411856,200782.30962826544,338036.38361813314,503345.7243052819,457647.82169308234,-554755.7989773066,588653.3323206343,-374203.24284831993,4073.161210323684,923030.6680422467,494758.40852423804,380472.94026976265,-187395.20843162015,-183673.0596077782,411945.14173549274,751021.2047144133,974021.7258792114,867835.3825087165,-541848.8495844295,351393.64505969314,23403.921011514496,-146068.1822211668,540234.0210318789,-409495.47150185145,995410.7144175693,-140327.60143695492,-108308.34640182415,-541382.8994897045,-906986.0367297404,829288.7621154683,-181108.2314451309,850839.3430870869,315980.7115957425,1098052.3627444855,-541823.8982607382,-385421.36175222276,-499954.8619565908,1312282.8596371426,427769.01649061637,886204.5735425446,448497.2324201148,472470.75964486506,106815.54132734891,-1002277.7824779633,-90208.59150272748,-109416.14743896574,23278.12889812002,309142.6285059401,423100.3951294101,344972.96800847026,796898.7223768369,244573.32238685992,177984.0685239518,-143622.3272428615,-482049.1985212201,-600966.6020156578,770623.9423986827,566565.426537951,-86233.07344975881,-91178.64015143737,-440145.45031144563,420834.0565369958,639352.4823898738,90069.76665805047,546420.7080907412,554191.4277932253,145568.6310638506,303848.59416140616,176300.92761747353,333563.61124981754,328740.3171577086,-76063.94418991217,-594694.2709614336,-26656.500504310243,16494.341627221555,173999.48858451005,76936.41478290083,-229131.59514414426,1265001.3432654398,265751.7909086668,-326104.8498445186,542202.1142579792,652382.1666598246,-306745.9500571727,435301.99266812997,15460.9031699365,130370.74473766657,-179342.16462155432,240307.65519786393,8229.844256207813,16217.295143025927,-140765.88923963252,-238046.80303230416,795233.339348495,195384.71791403275,-141017.86180675635,-573208.482025601,624812.4895447702,320115.5161666111,177048.63781093806,472127.74117060285,-45517.82924359571,377863.067002886,371973.4708465985,291813.7503567212,-557055.4702165909,120234.88064131187,53554.78035910521,-174739.83690365963,288119.8999642953,333161.9348078938,-340639.68778372137,-357383.5617463286,269017.9561078246,265657.5578629263,307061.5422158069,323726.0696822144,481084.4535804135,-161937.124016203,574406.4596422617,319074.0452119028,53677.34250855865,509201.8028930442,445798.1692460254,939361.0489854189,-262619.56723861257,303609.18104709405,279580.83236942533,-378112.1005312372,584365.1758685843,-11627.227753738407,383134.8092213003,151268.66689846897,274599.150378081,-547343.9948757971,219858.98731334787,-348863.09027046664,1087337.6624955628,-55685.1550114085,650798.6223858874,-327667.38179207966,522789.7041328228,-262767.0328627154,45601.78440491855,828979.8913196423,825246.7211685576,491870.1362131564,52366.89057961153,-375643.6315927417,702073.5509977075,496767.5375150391,673041.7533154353,-134745.03508127015,779850.6983112926,613426.8502069884,171061.07144191908,796714.4564841031,483191.61657269904,754869.070919429,-676943.2035772223,132808.21560675604,-216075.52140791016,268749.4738525511,-407232.06471409136,-288945.24143606704,-579468.0554593913,-402380.34624811774,81043.75447633257,152207.27781145275,150403.01336093992,490205.1320019043,-361358.2034254833,608338.5482659233,63841.47115452401,749955.4029491176,410874.2816251223,644417.1904175095,-476270.3347045663,117986.19262027647,712401.0677775429,-240693.16179624572,3294.810431255959,-567832.6239244672,437465.9788645017,-336200.98789331596,-178587.60449887812,345371.9265257637,-553776.0554128,522835.8266833653,-49732.467748858966,506295.1348707876,-358180.54695126833,933666.8494661809,197800.1957987016,708485.4856948629,291187.61373503366,467841.4988159649,740016.1697473587,124126.55007410888,1059010.3407962737,-265822.3413832872,636361.1018090756,411436.9951644456,-663885.1318421182,-275785.15838218667,798815.1719207722,326865.9427312822,335509.73856938584,483174.13211290864,-71110.14398787636,-442067.0436007031,378479.27821300924,-525874.8750445163,80412.90110576665,232068.11501463735,87191.42120699026,368811.9471265995,241600.90740650194,-130012.29358960502,1028286.1169023891,-65073.86838706629,-727146.6135505806,237483.8299346869,122596.86150882812,203777.8854099526,586522.7844790276,68325.69631659519,-78767.29354581889,704063.7152353507,383841.0264153527,-253595.57191539323,151231.24227868346,-284524.8655371354,995753.8104665,301879.7481191,-398972.11445548804,414728.27857578546,-196595.66470858315,141108.34576018387,-333605.5943147633,317550.70744686946,538846.4426053772,-993742.9070305224,-314496.2678890419,947044.5737147685,13996.587670669425,-63005.561571876984,-41587.694783806335,-380217.02904346725,105918.30397654511,-41030.989678246435,-136518.88786444766,410695.73167059803,345811.96110639814,433948.8193013198,-140219.65830896143,-90054.77623122279,575915.4327224176,-370993.13163038297,-135599.95296454523,1068360.2439477125,611646.4702657438,-86268.75801040232,244565.02125390572,-129030.95365034137,-538551.1621361952,-273730.1240992374,295237.83480488276,443839.6793539878,443442.30408332916,542252.803608058,70074.0191399483,-499963.6908199815,178541.73222175194,525965.2228597626,237960.61358346883,475693.14335461333,436564.7883077306,285694.2288083355,811285.5344856922,490993.4502747636,556980.9623874547,-79416.94052058365,644696.827475816,262731.198500101,474270.18474474037,-307047.27305487497,461946.7296330645,263271.7062134333,558228.19561793,219615.01451231493,-591557.6462518158,449916.96502764337,-296698.4217824922,20329.912012415007,246431.43801998533,277695.5841068723,774866.0070370217,674514.5248956066,-345167.5023165764,-226460.90812147968,-910822.9924682709,-329924.7106245961,51927.017893147655,584221.8692350122,822105.6885678424,227417.9307795735,-277948.908906762,-468450.58970146114,248508.09162534866,-575166.779587022,884554.5754757011,458356.75288284104,109235.08138191793,-2492.7922432580963,-1038052.1100339363,36136.801084435545,-107711.87534356397,843080.7870595413,1117496.003113599,619806.8718435382,403075.064477758,-222912.50770245725,-497315.94715084974,459943.24573777197,75783.41289726831,734134.692108782,473780.8811546373,165175.0508786249,-518982.96304071136,-28739.062977026217,-228147.40048278635,-92834.52930706833,-202451.95700361114,121428.44154254207,-742794.6928867567,-676765.406987844,475162.76890544314,231161.33443134744,343712.1666145404,-694621.3515650933,377197.66368577257,-93471.60664497875,315463.1243810812,1034825.0439623394,740599.0724081914,-685582.0391535836,726653.4963442334,179185.57694062544,91453.54566355981,-42561.28366112523,79694.601428675,130706.912308631,856939.6766314795,737023.5396490577,-638305.403659455,-322896.26758676115,756107.7222953234,-170386.6749990466,36767.114819095004,277166.4702863083,-407544.0556036616,1018028.6359379566,910350.758840939,-73154.76911988342,135238.7912586704,281510.616508625,366799.1966296113,540095.952420644,510241.0888697696,344625.7719016131,570338.4539811509,573807.4867668636,261728.963366942,-410520.7582785897,-333506.35459428513,128927.43687812937,-111350.49687093357,694135.317690982,199250.40293212375,81762.54926232295,-295197.41874318337,48991.86306228815,193513.04037872516,186965.22289108438,-237877.87090334622,-113462.5448091845,763620.7639342397,162810.87365854206,521176.7489831704,-309780.6645971327,50704.88341656979,-352930.77215470234,243002.26192471432,-304033.8733920073,593733.3835330298,431503.1126241926,-780163.8138093203,875117.0634225807,1084000.0092074242,-408544.53184246225,226774.78655941226,708001.33768349,54750.706286120694,490866.52204530314,173298.55117137963,591264.6984009952,57366.768732304685,282316.3610258051,217989.07439570734,654307.6078296616,-462646.53696807986,400447.47699476127,142649.70353750046,238542.97056312952,-267069.437986705,429524.7156151002,-456538.3522202382,284756.7605269244,791740.8785135206,-851523.4240912492,582213.5573229073,224631.11733382102,-542041.3447121684,200314.37660326902,781660.2967668325,785852.7264219709,340446.8667982309,1017599.4828784945,324454.1835439247,-487711.6056772405,118743.68441293575,23384.11954433564,-391950.1107644094,-45875.42979420023,-443558.5199282551,365247.7639399697,197353.0977811576,5886.823354062159,-67521.09966683574,140648.3225601185,935327.4274623394,-92880.87963534985,340997.94611570146,276019.15198062547,571249.9894163376,531782.80829367,-8640.246840971988,564054.1793445079,612227.2255748734,1029784.3489133059,660366.2925319998,-214098.12438577833,-282853.7279682611,1082401.3787249327,-152266.78478454566,125519.33513698308,448006.2337639369,-767350.577521089,1227.6679979213513,25769.317548616324,398103.6166087389,224836.6685747062,-83094.6163869272,192066.49907050794,487861.6604415821,914983.1023456319,-520758.48230136046,359363.64691520575,442318.0936454721,779536.5904025426,665898.7510330393,832834.1243352154,1185210.818804917,267994.9838537276,1019711.4162279423,-378210.8094454608,378860.51863981737,-624662.6730680235,59200.04897493636,731538.1944947848,-68718.93061579857,39862.91757285362,-469407.7430961365,943249.8903218755,261516.15876197163,-137621.53377861017,616171.3978117597,133009.15587533684,102379.39544657618,341233.65310624754,-341590.62469250616,-485051.03882356174,384154.13696600543,-476749.3132749484,34209.88967067469,27447.768165606074,1079785.2631308418,43743.84806000767,-680194.8953015311,-421122.45135002397,722805.2569965846,-558545.8899001665,571042.0006342339,227303.32952883653,-712626.4643111748,204834.21245154692,-296892.6329789241,136598.97148422524,41997.117052298505,60753.913796455134,807170.6783677675,451761.1528513739,590500.5370527115,338739.93466154113,775498.6425301605,66938.67179170577,-365426.76966241095,-74663.76104290271,201459.19992805133,883923.2879230985,95124.5352150118,-380375.8034375333,-662478.540160398,589189.0232188879,1352320.6570165046,-965297.545093134,843246.1905661686,-13061.823034381028,265957.2449647784,-356946.79694881616,685750.3814671845,406439.52140866267,-721960.694127131,-38515.8310912,785943.7456939807,-411651.5209402116,160436.97369735455,-26776.606947903987,-108194.22663601348,391778.40799241466,67782.83780698059,377218.76454751147,306327.45902409684,1009420.2689173138,111163.24028995866,913414.8353637885,-237379.77357180277,704226.3807333387,557234.3435616321,253008.14119850472,-523290.7990903319,-419313.449689114,-67491.9941482218,-68893.1928963759,-405341.8509360342,160379.20383431762,135957.25887464313,144564.10096532572,-243799.91134435846,-38610.991646325216,107796.6355718621,1350011.9241562104,-162878.6762771923,481320.67925517773,205446.52909862716,548042.7427026187,-722394.7992302433,-284569.3028705232,671774.9518481237,-45374.73691869574,88637.98953633243,389083.4202397838,685617.4774197419,149035.5549947992,301618.3651943272,778184.9636926609,-185879.59399849968,143749.02961596102,-827899.7726259716,728600.5632195203,134238.67826965265,-67016.84056482138,345210.00195122,861787.1341653424,274699.0313666584,597203.3123398856,750865.6793284798,348310.61392152496,76391.22928139521,145900.87093059393,-83908.87314257119,839988.5345614147,250798.1076411493,-504678.79211732047,400827.3435017243,611795.3083388112,-80248.8178959717,245599.44800271653,576947.0446179197,-103385.69687801646,733541.3333549406,685961.7758610346,253935.6135858139,406766.4619814083,133488.14892616728,-324124.24752609665,55561.24457353866,-292076.9728833642,-211542.01858332055,-46294.16419444559,662826.4035629961,519094.09309681226,756371.7877975414,-574346.7331827008,-26501.62000070652,-57034.8496309286,411879.1696941764,-911524.4360755621,405750.61608543387,-648478.7173545812,628081.5559913521,536241.5010227894,385312.1631666827,-874145.126344467,-85017.64907389414,209921.20913711656,-40505.0612661303,175886.24097369658,168173.5590263931,336378.01376354555,447748.7712237728,-202315.90005611535,344878.7917743493,227220.45912417723,381121.045165699,-219432.36104750494,722634.4498445569,476264.30652491795,475070.3742796951,-121448.12444250379,354736.45173316495,735198.0985333631,-808074.134170888,552872.5982639526,-142231.41709880112,-412717.8088058131,862198.5194023568,-722304.8442895613,1017081.9312898153,119283.87525751768,597752.969755868,-97005.61922160862,-302107.5627038954,135278.70190310804,-303931.52967831166,135715.80826212466,298373.3458175007,548973.4959559035,461103.86679340247,155937.09838355985,1064337.9462570385,322632.419871544,121214.35732317716,-77410.393005535,-205911.96236257674,107560.76863763109,-31416.215584842488,350564.26752471225,-25192.502219117712,36116.102727896534,648991.6325881858,135790.34738548892,121267.74759791931,707712.1815915005,487656.9966996508,-452322.85879156925,50059.537129486445,138075.03815404465,108965.86420772225,-120028.95353301708,501372.8248737524,3076.2387252952904,-414097.4627357377,289822.8869514335,-77414.40374600096,390052.2089870181,605499.2740277904,-136868.22648963262,74319.62481550314,-95168.44644671306,428533.6697113076,-299811.9184457534,-356890.9422839796,531639.064073198,264162.33071627794,982218.9224688769,495883.4828439248,717766.0481959404,505614.40532678925,-495593.1926941932,-4473.744326561224,264226.60432757763,-121889.69573886553,436460.72118494427,540033.5185144846,380242.37259820616,-244710.79969131807,741904.215603597,996110.8319395161,640071.802268697,-227153.51459380286,813292.5464580532,770261.8393659322,625380.2644440583,-411145.9174681003,160758.3110951502,5961.346321952995,-141822.64993604086,-189246.74620554084,-231095.43725809734,284312.35404155264,697036.3151875767,634968.0689564738,-117570.22127670329,1187340.2292562108,432638.77996151615,378000.45697825216,440207.50378097314,681256.226439144,594924.0957094938,-126427.4125428279,-19190.410621151794,-537945.597693719,-2488.406460353639,275751.48755611805,-123056.65949248569,529671.3381982413,-375750.74179555196,64180.24608611455,-165027.65419117594,639101.5040902868,-383970.7997778477,6905.307006787974,186757.6543828072,1094372.101946887,400420.40628180327,-36561.94041276071,-91803.75235337112,-375790.8570255474,366190.03568839934,755831.72119623,255732.52362611098,-148417.61367645627,540769.246745822,65985.26290320233,-231765.871955737,-410617.4505732404,306238.71305048466,373268.06948097656,700327.7776259976,820644.3141854038,649204.9628921878,-873636.8292470109,-592588.4145022712,322616.97376405867,365703.9120995356,852998.7208323311,-866641.1430221326,-657323.3819109276,155218.4875302422,563231.6537718824,296374.6157934256,-255104.197589878,286994.55012150994,577594.691206438,561525.5341514642,1071587.324779595,225416.548056792,460724.12745790323,-224910.6577238585,164207.85196940787,636181.89444697,279056.6073932927,319735.0877170963,963098.6083081607,398876.71648154967,-235562.33884673798,492975.19965385646,205418.41420506733,477307.4445844111,351972.6440552464,602313.6244448274,1012325.28644204,-225304.51495829085,93347.0447380445,211703.353869637,441001.7948623691,-653774.6745265108,-521488.14061912056,524511.1137765292,414582.5947671281,762786.3412348623,63262.76201907499,332776.4668638306,-30920.46955380682,693708.2090838319,6546.107159719337,993920.4931565612,145713.72912672115,-247051.91356279748,-54816.41090827482,322569.61604408687,250589.90478162654,256444.47060180316,115514.82127797557,240712.86367743416,18030.016826004256,870515.5398760973,142703.73835935397,-180357.98295889376,170055.59612818994,620522.422630422,-749450.0307375062,376132.76377340127,-8903.030599620193,-402859.1755785607,449285.1989218225,204490.5936661698,424402.16354449885,-223073.23645453947,297122.08258633455,548253.3942160015,31458.011218696833,-203226.9177735569,542879.6694571902,-516251.56895694137,124479.90237677284,407827.07530579064,-527792.0329341358,315126.8104287437,213746.26644459926,-258432.30767411506,137380.5607768786,113422.55446023215,305492.59505735757,426653.6196421981,-360621.5745646213,118513.48113007937,432662.23628436634,201030.7212338047,322007.55299689574,562118.9285279941,-1003395.085010082,442304.3952529514,-677688.6070370008,-625022.3661940647,-680635.210688612,550481.9806963634,-464598.0434581968,-474452.1376524782,815957.0301120263,-41087.65459485026,-167983.5964375427,969850.5814230596,706413.8304018844,-14111.968893978745,98068.98449617717,-507676.0772467549,-600252.7467169478,101318.99216813035,-108339.97519468516,240803.59231458418,534721.9406458926,72088.83812635625,459874.3681717706,-329695.3473784821,1030205.9552662596,630365.6783348126,164686.50935557252,-55900.181768481154,406755.1333049289,-152381.90970117552,-264678.2055445169,391807.61478285585,179570.34131712327,497484.2200377048,212970.4215766634,653932.4490660755,481003.90785565926,-257509.92763509694,297138.6240599635,449374.23849502904,-53230.544556317385,165480.92824296746,725379.7445194209,941550.5352361612,51280.51675576833,819596.9328065128,1171123.6669555283,-221639.36256140796,728173.5704328017,618465.1986493086,189622.96784349345,544710.2107715467,510981.24572041957,308926.74329214124,939051.7072843765,-759150.7235799315,520199.43639821233,236909.68094354076,493035.5069362563,848022.2127933786,-246530.36174656218,963413.875439303,-96527.49485198781,624831.4082759898,203599.6854831148,284777.96903670114,361738.018974476,1426097.0347197438,799989.4696548437,265732.43260485027,-248980.31497926638,687833.8766234028,-503405.99506720295,388932.47793354606,476178.25384260947,-238120.3280198085,-608820.0871890574,-40786.94341435237,218551.87142309453,-562255.7238863078,-12987.136079218239,367210.06159380963,-123927.02691990463,-199052.0007111649,-37836.866018128116,-96729.45877710264,-398766.0524013215,651370.2009878559,1020629.9183962117,-615563.4172551881,920331.0376532632,600119.1982461186,-275597.301419036,-378425.90259778686,-427067.76629849896,162070.1961828177,232268.17450021347,-71326.05259346077,-416772.6301981085,-347454.93401646195,392456.84764023405,173543.74502492277,145570.53056563716,-374043.23705498176,191466.77573189326,-555631.1667923494,-324774.0544357593,68807.61988930032,403618.1598933339,340297.6587690604,-705999.0603511524,35415.857381530106,380138.23820337513,103691.8508130461,-703374.443541457,83628.4693979132,495633.3617430199,205281.3318804889,358633.8244812796,116268.28617302934,1282724.6125697163,-221116.77955693798,539300.2151092826,-261170.46725451155,435423.3178789965,-75089.52904755948,-96685.88936047908,296910.49978442024,102647.67325078743,-34310.078215976246,843741.9690138376,-345381.09866493987,415710.566595417,98099.81784969382,661614.7053893972,-159433.35381969484,-219636.48913589446,160341.46267730696,916229.5641896618,651330.098839608,259365.6947228606,-397730.46373889083,655495.5363720031,366453.4737435044,-85445.92631619284,-384678.7818248491,103926.04991033254,276691.6285680267,336654.2519225683,746920.5719349328,990054.9906767542,-42355.70227433089,436116.6512658503,282760.06943957787,145651.312431572,-347265.67477124045,-277279.09062322415,-268627.80493151257,34661.4371042694,231570.19959312025,226983.89757472556,268554.0077587338,-535673.4155990114,-135621.8635042878,660818.3884855178,591391.8750345795,824223.5314713139,366354.1663027359,-60543.70550217107,426354.408459187,572330.9095371845,1115866.83929813,551801.2763009677,-639258.0542469523,561313.2050906587,37238.30969639495,1088050.6387724113,-5583.61296124151,46113.30154147651,646415.3976452551,-329505.16672955314,-356732.63571564807,363170.72527182195,-335309.1276381258,744450.6862065704,776548.9090396208,-1089961.3581254145,875243.536312127,724875.2363534002,35296.87473730836,157418.35007269913,11885.887102168985,-328905.1773447115,-531829.6603107452,-578066.4830967099,-351666.6970928707,40998.08749553282,-250648.27803610498,-382842.96956322435,-294827.81339943316,629569.495210819,288308.18229398504,-210608.149691856,1481185.8711269689,150502.01533530466,1055920.543981255,551277.3577089938,606732.2178713284,-231896.17719965242,701393.6227043779,45203.83531438559,-10972.471590562724,271145.67620533286,481647.5449051908,-252543.12940060627,580572.363328991,503097.9127222379,-269073.02908977726,969083.3795433715,1136615.9107166412,-143231.6508121118,721760.9765058067,-203089.69999184692,438182.2085302803,626268.5414953665,333429.1010593651,316144.2743506259,211210.29656897113,259506.43443068722,100110.0556528829,261001.8389714714,548593.4405875085,786414.107065199,948226.3647550624,436589.1940435879,439352.51118898066,-156585.9062455981,808079.36967379,509559.2866575462,374636.6121403016,889940.7587423581,463494.70892973756,759738.676045503,510339.45573558565,515660.62616628036,209367.78261668235,-282527.2613615091,52304.64021507092,531515.7402330623,-100452.97665978782,-613021.582768776,838453.3240480521,199151.2875731038,-59304.96106778644,65700.62746531563,74031.8392216973,1654691.9109713743,1074472.5588645516,262491.9122403683,-218839.52794076642,35900.43625108618,-116981.82809231663,321175.859837431,-106399.02691007219,458037.2559849643,57805.32512844121,-180474.6216070517,709793.4205144353,539933.9239993868,33517.35026665125,-94943.4722307953,702577.053495653,713190.7187525411,344857.4308490399,-961830.0119608522,656679.0584161235,199485.5941101387,236181.10543658584,-58449.32935325289,430497.6031678552,383421.555515226,862020.8064103913,-1028553.1230285428,-615806.6423385749,565462.1231885478,279347.4143326059,-90791.07418725966,122049.62173557794,-304778.3337379047,1137192.4824528233,-692060.6275829736,-535890.7011222383,-642817.4537075143,152446.83445136668,223341.20842550136,-234250.63937535742,670584.62319702,417559.3533371715,-910612.5220508645,525954.8678315766,398428.0526484465,-12608.777323215734,-445164.6549778492,883285.5837650676,-331608.1063170531,-316685.30355373956,593510.1616639434,-305128.8467030332,675056.7888057115,827243.9901962406,-460031.7406152203,537863.2565573328,-334268.2844950659,464046.5474355947,83644.30006828438,699073.2266933857,-249400.7073743823,-58970.28558462346,720226.7486279006,279398.0393242715,-188094.74708740506,1103235.9065991049,518727.1407049252,196515.14917433728,9611.949193376582,388781.1049218476,-473163.1717428523,-17527.9775509974,-107832.54758179141,589333.8992341934,-586605.5107968764,399806.39418731024,-397757.43877461413,-277350.18676715996,-239906.71145973355,-345244.85843474744,-126631.80474287644,412828.06418448314,6480.109877749812,283533.9720336669,496051.0061582015,1080.2873419676907,-169356.140223437,213939.56292633386,-140072.57063738676,302814.1625748952,-706343.7377938894,-72094.22405892937,-93977.61461399589,-181156.8131621587,-209961.1724237618,119625.5582313938,824838.796334825,-40996.387068501674,-278341.76969750505,-223332.91327491822,-379585.66625378,98369.09444102831,38797.72553327028,-481190.2287957831,-4044.8809825573117,206222.46327796485,-323535.7223305553,-255781.90311488276,-96079.09826766886,-44173.514116630424,451474.5264367615,485959.58121113526,-145856.12462425185,645800.8253634125,255401.94771607453,-83309.93405271694,279870.931257579,446477.7049892307,115141.57696168963,996060.010157906,541225.632926885,37810.892027062364,-287850.0827489868,660087.6647656369,1057021.8028779412,468932.75680946046,83048.36562683433,442229.21018145094,135015.48960952973,838492.3387938822,638832.5110999234,-589968.978763494,162340.83465225622,1211732.3286970835,328593.70410876395,786523.811839309,299592.5534481369,1116099.2113459138,-404279.7649506624,840994.95591299,10564.84054685384,172444.83841752866,128393.37428419385,167108.90987701295,70535.65636181319,-33278.85266769538,98103.59417842561,-369153.6145681427,-273052.5796512058,457329.05251156073,303758.4105869788,117907.0033118506,548858.3289605752,-341089.2823682248,63592.89133266499,-260609.82044225605,238463.3631732175,718804.9939348968,-726416.4354795166,-269510.0939708231,-436014.37598535744,185200.51058883173,523123.4698246331,732516.8307965458,95191.0781435091,465762.0488007106,31387.054724416696,282178.40315877926,228553.42223118013,857866.8518928774,459869.62641957356,181010.77200558363,919864.4985770965,45526.68175617978,-243117.12442129897,30121.6118503958,-290632.60327924136,850449.9258166654,383098.2573001818,-776468.9902685294,133757.13724210905,-69643.0366652892,683643.4881215137,345911.01821967494,-534733.1512888614,849013.8373332839,43480.91199577786,-217020.99036346562,-445653.4410309987,-194252.71283759363,65572.99850887014,214321.79416743712,-198573.18325153645,197728.04742885055,85339.30131085217,666552.6154846153,102704.06660344172,616794.3618749119,213288.59771365207,101265.57489403011,-518048.57676930074,215668.45527453534,140541.68801315734,162320.86818399746,-115009.14860053593,304981.14024024317,376772.78599431785,-160413.47299630567,-291837.67024749797,-54176.50830956781,141881.8571460126,-901623.7276938714,857969.5191080924,478029.2632822064,944056.6213241988,-12700.253966117743,1213249.9148375504,344355.10474641575,108271.3795878212,290072.3539900258,-57407.89540492417,-169070.50143255107,832314.0266802264,-263936.5968483947,-480183.8608181714,-38345.08156589605,185698.4728758051,-6102.450491537806,-593475.177759947,575755.5267359717,-377468.99070381327,878185.7263114955,308827.0015821294,-631315.9919781613,-143825.53560108738,34713.716786365956,-16967.60744817462,357320.97005478386,-147059.997721483,-263569.1148270536,173728.0318949623,280308.17347758356,-94419.93697591964,1051920.6074046432,610158.366259764,-274408.83869480854,611267.2466068594,1057639.3072455924,-541319.305256336,395761.07463613106,375694.6705221315,-649426.6826287643,207248.24686994636,385824.1966860276,1016497.0781666916,392901.221399365,824932.9604436324,-134300.82375917071,-254094.71982536418,322754.9731650981,487288.4584878022,-146006.2716490226,-231258.09583610017,59020.64506641822,805403.1066127731,-53133.07312359847,160484.5599793815,576468.2517512692,-84700.12311161635,110749.10529576894,-826411.2364205553,680183.3615807127,-495493.5993015731,428256.29846787173,-338631.58138369815,294184.98115038034,-224743.2687662784,-203780.04604172334,392139.1219477216,-446478.24717012,469125.7150192405,376833.2717836909,-311099.4737842581,-528193.0546608642,199170.96295558847,-156982.84355796967,530070.5708043915,514272.89285255875,616311.996286138,697648.8882821975,399125.16958544124,-294401.4725193656,667787.8345471877,387985.9288338814,1034066.0561502636,1433538.1738545913,-273693.96332041314,342981.5304522319,1003005.3852927871,-460021.42357807094,-280508.5432525999,-326528.26963095646,757952.5058082929,200899.7244267189,198422.89684646623,38008.15527710272,143170.10266554402,938318.8398873354,114915.24652127782,-549848.3629084476,557708.7485764348,222220.69404104073,-702707.9089561268,210428.31156113464,961572.7003689683,-418747.9087653593,1210.6354652480222,419989.6129258424,61559.671121375635,479799.76335660275,163586.88936174894,255476.1158876391,378182.8454596638,487865.92420626245,641302.7375852056,-280806.9241507924,191747.4161810996,730413.7852775615,385116.2851140294,-865319.2292420007,102834.81996845966,908536.4718710324,1014271.6164490338,-17673.871111929882,-105582.76159105962,704787.9638042944,-161381.515750797,528776.3368631131,618721.1940986305,154789.95276035694,-149464.829371918,-138847.9226939492,124344.69336650101,115940.51184118027,-414470.25735007506,79476.25918447413,68488.36444956344,460874.65109388996,1009226.5415788675,-462705.6863733162,98336.96925758338,268425.6384460181,1071619.8645772953,251936.6940078754,-287177.66132344306,1005691.4639817965,-254745.7525365157,1183421.5821380303,750422.9588312446,1151529.240907521,-27974.77157010231,-488360.92279818794,-10901.274251922965,536729.0122819613,543457.344360529,239259.71723608347,88438.99274480948,771089.8446179479,107505.29168295301,84421.26442220202,557913.1330063734,438404.4755598032,863363.0768794799,296407.01984415855,-15423.387976424769,1004344.9815868628,353672.558201835,264380.8497968158,829727.2191563435,-340317.2835297738,195550.84563191514,-519949.01109400205,252729.9360752646,-86443.37430625036,714274.8684368287,-526054.8125014086,322863.9975099317,186684.3141259062,-135772.94406996435,323331.0111018829,778308.1237559542,-555877.1261212765,-141656.9103539926,871332.7422716939,-230397.39036716847,-515322.6463730261,292714.0211296873,496811.48272339953,323875.426076537,-529436.6373635936,688240.8101322558,674711.9999225899,1530.5384716144763,-294104.6628447026,286874.4741745419,938015.9171597506,-298871.18316259887,-165717.2848036224,-273400.3239529631,277687.89052594174,-130303.90913544735,-564304.6835904913,812841.8916524854,-439731.3748160284,498236.4724973426,625812.399773893,296415.19647570373,295544.39633440226,262727.7149931714,242899.9212008207,120373.47617490403,46598.042668755166,136650.6234007231,302598.52912242105,-284014.8632526421,311214.69277335424,656372.3624937842,75455.27764562378,89132.20183249284,389010.3734141686,140951.943297551,-344569.58166424115,-386532.79021706944,-507362.86098083016,63705.35388810141,-71698.68299174728,1380885.3409608896,-207118.0045213923,772107.8556765593,-420020.42535346095,575599.5527977981,-42090.3323241584,261124.66840095306,-594209.7225958011,-217200.9579285495,468737.705416698,-17258.572646504734,-258706.77716733934,51018.180213310756,-111340.77412882727,131064.87897993391,104183.80750694126,-402573.64252116904,-486760.0189135303,-697182.4111861326,-55414.18392248219,-205777.86388196424,342835.40008123405,-346477.45336356014,-468288.08293307247,986784.6672251099,-25108.44069770258,215821.99186124187,-239933.9539166526,107752.15894472413,-262744.3138864348,-452070.77510364354,-124581.85042085964,504918.57943354966,-240518.49239823408,-326111.82798127295,-287571.92062745383,213940.6649398366,70946.22211172385,397541.29509685235,-340973.6577152824,557429.5547967306,-851833.1011262417,195984.74241882982,332008.5308456435,667611.3224125342,294256.7818055111,-112451.42969130888,94113.67977674864,-303924.5856740284,770533.326175252,558233.2322716741,-296150.1500856918,673059.4676248133,-567860.6315083504,641966.2687652069,904898.6081482971,-72283.53409577208,651217.6698395126,-897136.1571847643,499657.98391060904,435030.71836259775,1127120.3232072918,187261.15214162972,-158103.49253350357,380836.8491923553,-578233.3169782446,98772.61773202987,-130288.81470427755,-311301.0441537169,-514764.3604768729,-243616.93227860006,687754.2179482821,505298.4735539784,645861.1380498093,2607.2796559813432,771734.5924779805,271291.1110575041,-559787.5750702163,377752.7693588496,-461171.457420981,-116173.62942305114,-30249.49651871808,-427.9241287377663,36556.449904385954,297309.5861762515,609570.5474067223,14190.827042483259,-616632.3026830102,123731.33689428773,57542.0370734958,-301588.07044149097,1071591.3595098061,795025.7599134711,792648.380528375,834578.9347593039,663956.5936022205,900937.6032260852,-115540.76167368516,298825.9613192328,522191.4823166933,119590.80692364462,-482020.38054080633,-153043.60274279723,-530931.7791110026,623954.8347978783,367678.0588919101,-151058.95334793674,812190.2904683799,-343811.03900605673,877734.7579893027,140879.01084102597,-163534.3185345796,591138.1276138336,19336.3646782781,-45225.15793184191,99213.24122267542,-221042.27787211398,194287.8102770592,-401135.4607889191,389162.9646648527,-297865.9446768323,203793.38709716313,124551.89284726419,50105.58366382122,234369.77279530466,83277.504995625,492037.07947373716,151313.38104223646,513474.1873561386,407157.3865114879,209208.2685630978,322190.74536536355,184578.07729765354,-699950.1984072218,-7786.5535917212255,350757.5302354018,517053.0335877198,171756.47130426345,443411.15584072843,189933.38847019523,838017.8056432884,94508.77929916093,652268.6974093406,478412.8036051793,-91299.35633127159,195917.70604458032,147346.19833276747,859853.4426690582,-49760.40941275656,623627.8675010842,651754.3505366277,465032.2919202675,512578.69282428315,972102.8706197641,-415928.356324994,553300.2800389607,631725.846833278,968970.7037818702,248251.38691184716,338099.03633707203,471697.9184876005,41518.54591881856,-27701.028974756133,105519.78746887902,632549.9505840153,-461650.0200530542,396662.886171523,671787.07345318,-107182.72686814284,-136415.19714539032,1180442.3247835836,327068.55488106376,-232362.22805146826,-200257.30910865357,80191.17030256847,371986.04698980646,638046.3267957848,-163969.0179513325,-118527.90346308053,801211.9778887457,597294.1614615219,-368.79690254246816,407311.06340495683,-72991.5613715793,11902.737159986049,-388307.6042538453,275612.8028092622,390576.6725419243,172706.40959039424,-59684.65013681399,326542.6532643591,645126.273287077,768291.6336560976,31531.456501882523,180562.09777533542,-250622.6831932906,-589741.7757996223,-404154.23456971813,-170423.2119142143,129078.87532476475,-824202.3929965412,301697.07603750285,547186.117341361,1114770.304876973,128904.2833755305,794460.6666920106,918127.4585347064,-531042.4192173695,-296928.121171894,-127421.87179197883,154960.51142642414,364874.97579351254,403495.1793869757,-210963.43767870544,520035.57287616516,-22458.979759781156,128222.98129876191,-555840.0859908457,210605.48465139745,375413.15968432557,-259548.69500148995,833871.2709261263,933752.0407640701,279515.7194813993,446399.60755359335,-586233.5208319123,-38736.4414172424,126822.95787410019,576233.9486260894,401510.77295453846,225084.18242032686,25568.58986473549,-13960.941433747299,-560583.0499823615,-751625.9945350252,783344.3880659188,-432215.67373044346,830471.8458684739,85251.45444677724,601547.5946382172,96097.3950119582,357632.376608646,1231573.4583325088,292481.52651649807,246512.97140683047,223856.34146385267,1082358.666413399,582189.4803971611,-201700.7726242682,-95727.36789472168,128521.76393260062,612753.8891827371,100781.9296729085,357900.16712743416,470329.79004885117,449965.5748905912,-69094.27364098141,-300737.9511829191,43780.500336039346,296415.8242481295,-186260.92125958018,787023.4079840854,-608874.6848650812,-86981.81890669279,-345213.39297745796,797540.6377850091,623562.1617871681,336798.7730858461,692098.5204466605,7120.468694753479,-46691.8475756133,1268517.3308102968,-611529.4476900212,465388.31555280695,591245.8624498406,48224.29151169583,1082744.5425313436,-170615.70160479425,676887.7480592751,729860.1731841159,373886.9110501497,819556.5091540809,527598.255957331,242394.7891479847,499824.5497367983,-453131.63647553045,222223.58366862964,719045.8179105562,77272.00351219764,-387865.4828104186,931517.642860014,804384.4123421554,737962.2634730074,502592.30341356667,243013.04743734282,570116.4114252138,493863.09288291447,211975.88489190862,675220.0898678582,314030.1730122529,169492.3143601413,695478.5966220177,463888.75867657876,524895.7727627768,396267.6778815314,606432.604470782,434018.4657264836,319535.2628145218,171346.11605359428,293872.959177054,581448.6682158839,-475450.30873774923,-349274.37518341653,45176.92252510926,-481559.7063236409,-350238.8357231603,379193.5876306263,789587.9433608567,155988.08494030498,424246.5009740363,860399.2880506455,-252012.9421699443,433646.13601782033,230775.773645123,202535.8386031757,-152649.40599553613,3698.1529781469144,-128313.83206623932,-756925.5319329684,48662.24199546594,791190.7139161327,-250079.33508456405,-375850.96371863224,-603507.3763105008,854613.6484796153,194672.93798205396,451911.88938134816,-151831.17946811626,128519.84599198587,-428995.49774355674,471195.59702807013,535892.615970342,631682.9349844311,-150231.78209908726,177157.37149635283,475937.91788178775,-47871.46200723201,578862.9091539839,832742.8043206036,-111732.30459258333,475003.6932630427,53627.182169245556,62106.43762640422,-246285.1885795272,142039.66384987347,400023.2268451564,-444027.19542146334,238548.59932986507,-729710.2147524664,201328.31466000993,-216558.61776581453,264493.2658587517,-728959.3683950654,1166947.4615816316,591179.2147031873,131995.84715870582,349809.61505094497,274180.1363570411,131669.26816257508,-68718.18614677759,-928105.3532818346,513053.258830396,541366.4769028998,-34127.53425132157,918257.9745002817,504457.21632201644,848584.6034025452,154993.7607798418,75474.78374905745,-123460.84880298935,-524818.8322868017,762724.8928937507,395849.48590138275,-76382.57611461775,93403.96628887393,452998.1635022131,147085.27369863307,-573812.0007426301,417513.1967863771,4653.814336797688,-397329.95236111386,130343.86613450386,316829.64008053066,32438.375757914968,134473.45979571668,-135770.45541367866,1074512.0652863686,393442.41421495145,904878.09473215,903252.8673323542,300717.5125969602,866935.6852453724,419338.8526213006,423773.4714008365,425129.5429246952,-768445.9198877327,120592.77109566098,-232173.61181111215,-501377.2753146207,489419.1040365738,2795.1353660915047,-170714.73885284597,30543.95024469681,-211376.5845204906,334043.3191761286,594429.8242116147,684726.800086631,-644599.3857316789,-718420.7729085693,60639.719582675956,343568.94971294515,502570.31039812276,808249.6154421242,405065.12044027355,-709026.3881641168,319905.97023542505,354284.85596418893,281932.7426838926,28517.02696544258,-101264.3191218148,-241872.46516423067,-495514.1361373542,-912704.4921755577,1094613.0808540294,-750379.672262595,585841.961270846,221137.97314954503,272621.23106577573,-643437.994318102,-33750.74931176612,270263.1286034151,135396.22699452052,762308.0422499217,115252.40753070544,96467.31543102488,208206.54953761958,499514.28352423897,439874.9469419508,510480.1986858803,236151.13963924348,-57810.29805250373,-103332.74613335775,-173454.20861757966,704168.7877708874,313333.43945428124,311517.0860392521,-849469.6835476977,-476809.43020316307,948346.1407308644,330378.77404593816,130670.0420758077,660668.0837066583,-645440.3198744878,1029773.7066751402,393014.2280016793,-113098.11685867514,-314567.421503772,1151302.3428236772,907895.1060412978,948691.8714249986,-193360.83253394905,-91668.36606694339,-259888.24210249214,534648.7605187036,580007.2896884005,-195334.68717033835,-58267.858099403325,-222787.42460370203,-202885.58815379208,901313.4035602892,-28018.138138022274,255477.54424224608,-109249.58318965416,-13605.294192883186,1163081.2291922197,50322.70389276743,-492408.1383105335,416686.9559552558,-351624.54442178877,-201860.41381014232,1040306.3349848422,-586527.6305953893,301703.23574838974,463293.7962865536,25748.576978564728,-395117.3309811405,-85323.5996790384,730851.8269670098,-101415.63556508673,679559.699787009,-49858.90705587249,509784.91999154864,-154028.1197103411,-218709.55206930032,455327.51944447076,-724387.637093978,-127854.78720703768,-430893.4614527002,-557141.9011835956,157928.8946992741,-700230.9273500578,-25399.346358443145,-621892.7645357058,585703.4534055772,528378.4905698677,296845.0674923379,271480.5871111695,624147.3579262164,397717.99519334175,684761.4176921397,593084.0520517421,796425.5338513781,1099687.4909285284,467811.62304013455,134412.89099172316,97635.63692094339,594315.2930509541,194957.27021009848,238796.9013007544,-342065.5593473511,-357616.6574065201,237307.01427873597,-269134.28992966516,21633.827380724717,-131239.13213889766,-416126.8522563912,-10221.12358565256,899819.9793104981,726518.6273945826,527067.5089366236,-91506.57273678761,248427.47875386802,-171397.16092297388,-227836.98105789674,269774.2612069957,245936.2148854318,124839.03673626296,344429.17968673585,-357199.986231714,151273.25366686704,145922.89846690744,187628.67764349375,-511243.6023132419,921870.8376246677,385889.7000465514,-501535.77029383555,1156522.4345340324,-158109.30360474298,764230.8537111422,-318936.5035492382,298079.1537607843,841971.448694801,-167007.59064348135,1135826.2785496227,-143528.52201257972,-146757.71429513302,130729.83322452102,-446576.8907641773,-48458.1636163611,334716.0144773382,-64327.67939886823,-436966.5264377785,-367319.5662997775,312433.72974793334,-269544.6516778362,441688.2523794784,-103837.53236542037,489420.09829522856,-118118.59606476361,45084.37061658595,371806.21184894955,631556.5798733956,-356192.5821057814,856803.4971618517,933040.0528993174,401292.24986358033,297504.2228194587,-828872.4740162743,116507.63134479988,-39797.159552487545,276768.5862198919,863676.4227128061,56907.044340582564,78865.1008915673,-430096.8378934618,570478.5787240337,278463.4532060772,12361.269971093629,860007.7497276664,277891.5126639451,-189824.84777947143,184698.7773464257,14695.437272816896,339660.0559393647,155053.80710048182,175337.11611755937,54359.857896558475,908909.3391291262,1139457.3924449724,-493213.29877247335,108398.97726331,-375912.79887045827,574153.0198115371,442593.6506089922,-190977.41282863123,588387.9048168133,-40035.05548084527,159319.89949859958,-174216.40417414857,561291.6620317027,-69647.27147981618,-359742.70706226584,169944.2025902341,-48806.968490817584,221193.46084264386,141558.13180543203,-579309.8975174455,468705.39577607485,141883.82542043133,841291.4630302698,700708.3943708884,1202658.7001001895,99420.84855745081,488810.83882751595,-164823.97305786004,308998.21282439027,163919.3643245888,132693.0469261175,451986.44436197914,498402.87343969336,-45220.63729095878,781691.8200211478,-240465.2937006331,-505868.42512807436,602623.1801220798,-201230.79570132075,1329607.048654799,-353210.4422620004,80227.41978516197,94878.54028581455,16993.602063974366,204802.09209154826,928572.9445032212,367127.76510603447,571473.2472263002,261348.1947047389,491846.46240615565,761244.4154209327,149956.7158792396,564375.7897644811,699070.4428274632,94987.05873166304,730291.0724960128,725061.4372284231,-156520.78335042577,-198813.8921363037,228676.08194239344,529630.8857887983,746304.6332990103,-373415.866192108,403253.7694355869,116007.14344156627,52567.95328944037,-172103.17703158502,246977.7736229673,60119.598313695285,122936.51386028063,45698.54925516248,-379758.1855207379,-29828.04434579611,140904.89625240443,23829.442413044162,-81545.16133944178,-245586.81447179755,-184415.67595637962,-310235.01893502055,447554.7704318282,-373066.2098056087,392746.1323041525,-451379.1449544714,-167998.44538543513,63104.139141798485,469033.2535119932,262727.67818128644,347819.35092227766,640135.520847844,-473115.40685172286,-225399.08221145207,-360603.9430506765,711484.8488458069,19044.90424017748,11930.457366073504,525961.5263150991,-179652.04574353527,530194.0474741473,1319638.537403145,911642.6510685198,133250.17345458502,88464.24643606087,-173581.4944267124,14042.388950362336,273553.5271396702,219780.4777897806,498893.8183219321,-340630.80355691677,-690767.1683326685,16600.313795884605,175943.44384404272,43758.65407887474,159905.94902412035,857768.7031056937,229811.86058503576,62850.12403583666,-131187.68336416595,93060.95332799153,125512.03879349539,153896.8366764849,-358054.8988666488,-193735.71598617686,654412.1096650073,-254007.60599534074,519376.15566344094,440419.7423369442,-29258.654944153503,8265.471058057621,277051.81080110883,62342.85085917078,-165487.7872018139,705014.5459768637,202862.06587365828,192873.92553536687,-591517.137909432,532373.4769208301,-61234.45543145854,-135860.24362050043,-190742.84785131644,-308069.6156614907,-97056.69399766717,455502.1262951279,75146.41332283616,526433.373948467,766092.1845235606,378599.17464685347,-324477.39019560907,-277291.05985008064,648412.1071383613,275366.65273991274,-353924.0768440203,-579385.7038624822,298394.14027157845,86788.860400951,-168672.3190604532,491763.4261704744,-1124028.779291206,948518.1673262273,-14622.778579048812,1044311.0114187994,-102538.00385750318,-432293.4543536012,110342.71249079518,584877.4069143934,257469.38770031556,-269830.7126824977,337221.156386646,-109898.50908062421,193201.67562964652,339073.8313930677,738465.3766999547,71309.82695499249,353701.0136878621,-727353.8150943927,951215.1250583436,-305581.7480020197,393557.20609184634,716125.9036372537,-174000.96642648708,-59607.71295221336,617984.7231708127,38183.740725894924,-599797.2173697744,926105.4153871089,-867670.9388863896,-969599.0793410966,667099.0386643596,129.2302152197808,956271.0674922736,194313.68470615242,-125078.80903625954,395829.3013897538,-184649.36418433813,449027.7642981005,269147.9162298646,153392.67205910617,-435317.104668024,423093.2877301802,-72179.6354978932,113118.26093803765,1114312.773065276,-184116.3132752506,269728.58654897194,-103147.51738218637,724228.5768534923,363151.8886700561,225952.89765665494,822137.2175194924,-275998.0649594716,-430764.3382925559,444701.43460369855,661093.8963506436,414226.7089843503,158289.93583438173,-284547.7240873629,652228.9668587116,363433.255891345,762793.8431294463,-173386.31765850773,834166.2169517325,488817.74350611353,-122142.363932034,-743246.9938991717,225427.93919170275,451876.21079114825,479732.1599700898,-461173.4097333001,381673.39072130807,224914.7794406875,-75478.07189415256,169519.50936400658,-548968.7412576629,403846.07691798965,150093.6422481574,-243647.86754487688,769972.0250309943,118840.2715414525,-408025.00532063004,21493.34191338811,-239901.73589899018,414237.99594209995,383506.04979116004,-196883.17311192397,174704.84863268258,-357183.6315304446,933341.3175134808,73164.60845778091,781100.940727625,42400.602438535076,-180251.45760725066,630450.27315054,-224362.44637563964,508837.5237167026,368404.6949266922,-270932.0485191862,321535.0844218144,-39252.08868719963,394470.9087442374,282345.7068297593,-41056.40271273535,-238992.75757716084,714302.7178081316,538735.9397855857,382206.34642097587,627554.266475419,705565.4327035043,-11706.883151431102,171626.24931522366,463259.25716647925,-199674.31326627033,334788.8748005782,1160665.030371414,-890343.6181101021,124820.80359252123,-74924.48665011907,373293.93040497694,219296.5578350881,1046907.0863160426,590986.9306458225,555054.9562715804,-33694.825700413436,143410.49217531737,948818.6653360953,419218.2162066144,1140039.8083807281,466473.7896061875,-370525.8712599813,7896.671574514825,439554.928786831,192433.09480440617,-477621.38976142555,-153816.29199520452,-112674.67833642615,652877.8599622825,758526.6779056145,683393.3982241787,-790827.3761343285,159595.17399271764,403840.5343905203,788166.1559959599,-375869.8670097075,-297715.5722503532,-22395.177229658235,-342833.75433413265,-7520.970562561415,32455.076895263977,137550.48742918624,-161839.54944994394,957458.4956826442,769627.8482562848,610740.9458937054,871860.2815460302,170998.3660874013,386162.7892824258,216998.82628031634,526912.1034354744,470958.26618710486,623738.8724299124,326744.38784780493,105623.04202137701,-276595.9280167762,-295800.5980298086,-62771.69064049702,-484124.0514767375,246953.73202672135,2521.7355299871415,-33959.062475857325,460391.0374388732,-6254.25980104832,803842.8210683931,-394570.4501630594,302226.2913380903,-5768.912624160759,314048.050636319,346346.99037741683,522616.05116887856,-143979.9741835692,-176493.55398349697,341635.36376652727,471126.5819707704,-135527.99071595073,-53928.80134010641,566179.5314116664,707489.1114252526,6880.931259894744,-57716.682927449234,759406.5910432162,-505993.6951825586,78926.44117728341,-510109.1536310888,288810.3646269608,202421.03230602993,-536079.0099384072,630706.7120681205,-655681.6974886148,270476.52200978715,281935.6413960182,189010.68580355775,93814.1341845016,239994.8050051094,-316696.45215578517,136349.89386088075,-404844.0860522366,-353071.4452912642,460266.8265842893,-383294.4995762664,833379.4912457382,-64668.53016581107,40369.9693575385,444804.11288220296,412581.43844557647,-229161.9098763452,-576058.7939823638,758848.2262892537,14346.847624173388,517485.4145716545,834426.8636575197,417884.86164538795,508621.26514826296,568955.4417594275,-181307.59082529927,-220005.5063580759,853805.4894464696,156699.87986519886,90228.2676237775,209174.09345248388,-992049.5122686066,-446921.6177820149,161782.4588107164,766094.2363193333,-275699.28090538085,-907240.9218054637,-272055.9114633282,247853.00675018504,1223087.3941149116,-342832.9366061785,-423778.65626784274,387282.854142244,-169473.54808436334,378640.0887986403,1034529.1494691768,569748.0851109112,-82041.16843138542,1241647.0858430006,268972.8907663431,64882.37207036093,-5264.323605758604,891305.3409098843,-10514.409919522703,207683.74873765046,314801.36481627356,-100948.58909331216,298883.94344939245,-206982.04593302356,-53494.88814712828,-17430.142547771335,254676.1010995512,837009.7546074819,455095.245015156,-32663.32481972268,496083.3913470418,384738.4116035979,-841110.5283137211,-630139.1964934291,302322.4372685184,406822.91616432834,1153768.6852953597,182379.8003707067,381596.30346838385,-97325.13822232466,-7895.0500351041555,655600.0220592082,-1128616.7653731285,640029.2413763008,233616.56119115,408065.8658179669,117572.39167418098,22987.872973649297,167669.14639857272,-304811.7029972179,288955.7185417353,439985.4906560811,767421.6751031852,481312.14722140506,408367.2610281566,-358049.8598150769,428903.32786060637,257974.11284270138,-53353.88875137316,176045.84063440096,813992.5118755596,51603.13845470548,610723.7758033681,845240.9059003359,14034.433342597447,-457922.86704106815,-408082.2818914661,473432.2475331244,299318.8812278821,146907.66040106956,600122.3901586314,644749.4520033109,35509.527556579094,-31556.176097060554,411710.49662441993,915493.3879548232,440360.3992300341,-300135.1675695898,95653.19871118013,556605.0348553373,-84278.28889683867,167334.12934952555,132689.69512747088,29343.19235582743,187606.70133127132,152604.97598931938,-432191.57194808265,139811.29755620612,-522310.3486661473,350084.84335108986,730622.9029641906,454873.2796326806,209692.3573203506,705763.4149120036,-607314.5418653574,464984.52236713376,384166.83771834895,645341.668001981,911525.1697534989,-199348.3194618756,13413.789385021664,78814.97457564622,3114.9643735014834,433363.8111802074,-492530.0707327677,745161.1281719343,-305564.179252231,-440581.27577487193,22707.843788806815,404218.3260015552,339081.38670942886,-39492.985631573945,588419.4244865053,784826.6315957359,867678.2403029851,511824.5537680853,-23740.05716625275,572281.5183275901,-169510.3724379558,236738.81259997562,254225.89437753055,422869.7959152716,112718.23941244837,-95773.17056771228,357656.51532401517,341625.8417986268,-187762.5057615782,-178197.00009688362,-179035.14634375134,186444.65501064574,1030476.4523190763,-345869.53632897884,443481.78285949165,68885.86808977602,75921.79770611087,767346.4774658391,382633.3125272342,-234214.50445385324,-341599.0423781271,427432.17782829236,747391.4449789757,481321.5049365186,724422.0918984716,194328.54757977044,455195.0066315492,609170.544873117,100891.69301698124,48324.928492936306,774433.411389201,698040.6122775115,474241.3381102141,-285216.7811338529,310302.08936548047,-345659.5138027929,-602852.0194234457,38952.55020449078,354895.41343182186,-225465.5736886314,69096.23692067806,1235553.8292661314,-26446.678970348556,490073.4077721131,1085156.1287436802,7207.089960367419,753770.9290174549,332946.785746295,123138.41077401675,406857.76751254685,415731.28416456655,358241.68840852566,152786.28332327912,-80501.74625566974,313248.70188695844,609565.9410488387,178350.3036262365,-71977.37690432416,427364.6692879498,248660.03881001333,472904.59468404064,547057.7879021312,190128.43811938027,372795.9614952761,-331772.74625939084,-149483.39606288075,1013006.9037171071,590598.3692695647,-665582.7528212359,-56523.51852460904,418259.3827340794,-424605.930729778,272604.1746999058,400002.12831836194,502655.61208613496,735625.8512191833,1186379.4692451423,-76872.12483953126,-279201.9312012689,974.5384646155871,679918.9711997965,272031.4088995061,613206.1329389345,41763.499661070295,94789.6886860868,710435.0051240106,-334338.34371042997,-233447.7864365615,768358.0816625715,-404666.4166892986,259141.99978505587,-431957.9473922951,-158389.41518003168,-74999.97458192846,24316.703677358106,499208.9368019104,892414.4353288221,443195.2698606816,-208318.46718625957,-18269.10434791399,450809.6448949394,-685218.8943962394,476350.04209147627,427522.26827323204,788659.2620555111,485124.3238415802,752751.4663050016,926179.7222110797,612962.6987834754,62483.92913584225,342534.6545055909,4773.603944874369,177680.55517776543,185701.12328149285,50948.08005109755,-11025.035610648338,645978.3986852523,457782.4184074141,937734.571914956,216279.01462476468,464585.99792295694,702554.2602123315,186974.61445158115,-109124.85109132528,-165441.24460446974,136722.91720089316,437877.3448669738,316529.3507790505,345462.04345739447,-115888.54415107006,-448464.14111972926,753563.3295941888,-634473.1280097808,573228.3512587938,1001142.9993776665,-339576.45840143226,1014335.0962543194,-133923.09247276187,-329544.66761359107,292379.15038414113,-771431.7348094364,321348.6534793703,1038718.8965215404,-23960.507277227938,117060.55851738947,-76505.3830820825,437488.4837159738,-230348.16509487573,978714.1382227992,-23954.84916288918,769949.7102470384,283721.94123164564,-169063.31995177316,-515458.6290725684,-49940.58838775009,369002.66429534974,630155.1736451164,310147.5830742833,554822.3922546869,52238.51343712909,-114172.66269551218,374398.398571447,1143740.3248822745,-174251.77946754405,43485.57973374054,639473.3609125763,507849.45685765054,404252.3643140062,-348750.63441342954,-43962.935770823155,-116345.08013699623,367848.51265641674,403975.17357886536,-665967.3572378214,451776.66375815356,331316.9487210857,710872.7097302554,799513.0122818481,269148.9872958069,-5850.046333250124,-610636.7043720349,637993.8011620683,-74088.13928880496,-28641.928249492776,222631.55838853912,-32950.37478380464,-1610.5626824949868,455409.49089183845,7250.084068289958,288569.8362717731,-147442.88829018455,-524252.78013752867,73243.11758543598,777895.1929959711,595717.3806479219,579542.7438695775,256777.23020761833,-601389.0136681446,24220.10055194935,987876.599850975,-219276.48023727303,-468027.7073462503,519563.9827991077,-410895.31899581803,300804.2920729574,217802.16955766547,187579.86466190312,-885918.7894970649,-135220.69611569727,356706.577429072,513152.5788760483,203522.91942611337,11504.27576953685,138737.9353166609,384832.1847574883,482884.26408053096,583617.904769979,-122093.75841656746,306614.1187759079,1036514.2828055564,-508830.62567145284,28997.050618280657,-115860.45794800343,1113501.7181115788,982946.4970889525,838159.1115105045,-536768.6967226751,159369.51470519044,-321521.35278852656,278778.0922858021,-255892.632741753,-184617.48206422897,-138149.85531957448,-40939.61310511315,683997.5733178444,461529.0032505756,-362544.2635472184,780758.6713790046,-77913.10645335307,-367617.9739929554,98961.56153736217,125629.50892639393,-381884.89505673293,-491934.67919646064,266907.73845563224,185075.11427774886,-28779.969027307816,327656.6813604585,-56553.17051949864,-523325.7495878772,-179332.29428137606,601894.8206854244,333387.385235121,-473283.96453889366,476057.72844144935,-80310.26496848138,119957.45314950729,-537003.9933054417,285867.68890561676,636388.4895557612,31340.49659870332,-75638.66524547804,200960.55196337774,717638.0704469574,231024.78176745027,1018990.2613770636,622085.6501031527,-25189.52948797401,810113.339136248,178458.42178687965,-236759.7934177476,984008.0311943633,424925.5006605601,155953.3915426135,981015.511726833,-497513.0314098275,1170298.9535083827,166207.29295219528,370030.0541865239,74352.72408673912,962722.167424927,499929.07407427626,157456.00670433417,-118002.4577582269,770413.1590401414,548507.5192364682,-689971.6134984419,205505.64197522448,-920536.3673367533,-323358.59678050363,16288.138300597202,-158423.7257793108,460246.7439259202,-127503.84655109607,-82323.88009392889,268460.2401984134,-210838.7109648441,256030.566238428,612962.4308584947,-225489.0260123359,-139216.55862148805,374544.89193625003,470458.42883956013,132452.6817437387,-493981.6487467452,525049.6186056896,541070.2907513268,288141.094042887,-127280.40358841838,530748.7310718014,549737.939019036,430266.3279368379,76062.45679226704,133876.97501735156,-415078.62400989793,520821.94193130964,-92160.6029745983,-706961.5290022572,-81110.07507632487,-129318.93347779848,631747.0054636165,407299.4006753247,397892.78383153025,-78112.76768148039,518534.12661254033,697421.3206516504,-79548.67375825718,468108.54742941493,52798.30483924784,-269070.8503252552,-272704.0982827218,505149.3229118113,68929.31890047109,96605.80964135798,-36097.99661112577,97938.69029171066,50617.23990578996,-100405.02458562749,562907.8375559277,360343.3487999784,-407373.38335487014,-360278.8869585246,340323.97977097426,311411.1811020789,-46294.95064900676,136384.15345706232,531713.3417640845,67654.08014977584,193275.80103937443,169824.25011172704,-595584.6355209728,173142.33786203386,450982.1221082667,-526187.4800402182,-494397.1964676399,465330.0709494157,-126809.52401158307,371048.4762672237,95771.80894344766,-37664.24084499432,21174.985270644538,80041.57343506394,539937.2006970062,359993.3060377203,-461541.6685884674,-1033665.391117583,-279320.95414604386,-174773.20624153968,471489.8174975151,-628174.2526903213,-141737.31087663164,778569.3143767295,290253.54883272946,-206820.06424006214,176669.8174806824,-163047.41396541474,611386.5997590423,603055.6756284256,-19499.46003750339,179468.52999877045,-66473.82914055139,-594832.0773964175,354397.02727976115,233456.10179436253,44042.8943903842,-590113.6016582251,-423758.48977047764,781069.8000275753,492102.95531749073,393852.79979282664,8930.670892993454,420200.90488941176,183754.3728973777,79320.73896040721,226704.7130989777,-16261.437510138843,236160.47016300727,-226985.12824004143,707027.5148991314,-752999.148437053,18779.64151663799,210072.43356181588,448967.088221123,-78059.32991835661,-725192.5742561778,340200.2535506962,-34856.743085529655,204322.6891784286,-380304.0835832604,-133325.47496590158,398296.92797837174,-201507.84709742386,18655.43405337911,50391.126436243765,713428.4074252388,485378.64861563407,766537.7162771611,435569.77752301935,27646.28388692625,-575975.5862380499,365858.49071352463,-3530.4649232402444,-102635.84405899746,552825.3820461957,-238349.63211929612,132792.50699137826,229987.9172022692,363755.8258247203,92608.85788946366,-699979.9287854852,-275458.5799056869,625370.4124630811,-60101.3056659014,46892.97454855172,336057.26747834357,228456.67734874832,19438.848562849686,493563.52731451066,537817.5728722606,262467.7178389486,-695182.7592049339,451656.79668921325,-60279.84890184691,945904.9609421585,-355584.89755109046,-233982.846830816,595830.8952065934,-412584.4756564051,-58677.838710708544,591023.134152872,-25385.632269810885,137504.86522578448,190032.2564843516,191033.90735091502,-420721.1172703239,928588.6766398465,221538.84928988107,721118.209232511,422042.5909203384,403025.339896949,629665.7419697959,-881315.7672872548,419752.0995468176,75619.0144806318,271768.14429902425,148842.16524650482,-265004.2122044042,153576.262555324,-85116.41489733616,93020.95720880106,180051.62804153468,-52077.42590038711,246891.1559759793,363164.97596482234,300218.5891771717,436082.64477767004,-380276.111058197,-163599.7061211327,487754.68540295074,582845.4507797691,246272.54150713235,363646.19351980696,-372560.9837605478,750958.5490147313,59916.938572946005,-172899.57775856508,581427.240195082,268936.60582532035,320987.90867982805,206518.92305487115,578409.6582104634,255665.14906688686,217113.06559799472,-681987.465528599,734871.7904011165,-114831.15690949047,-482434.2695728447,304243.6507533961,671348.2171501829,219109.87239881046,825406.6301399148,515760.92648003297,-90208.84580177488,578688.4048074139,140767.7716815006,-1211.3417656109668,320256.9311841037,923162.450443557,218427.35320127243,-298847.9435171457,293343.34232812,568059.0345800645,236175.17299286788,-466888.4471940617,523472.48506350815,-127770.12159485882,289842.42393556563,429357.1096109324,578571.7428887812,-561542.5864689033,137035.92635559198,570122.9859177978,809736.0060181534,736326.6763854474,610048.8594321762,-818527.2222994738,68494.08414077293,130465.45757824974,186553.5635619904,1553.5271920762025,388313.7298401627,88334.25368602155,-479098.38495642925,977474.875982767,1020033.704914466,91642.2064950075,733173.774575938,61787.12747155363,-566276.3245218657,839813.1850792575,-25059.23335985467,363506.2360079824,108091.63675572211,-43569.09651933564,-106300.07815075153,422048.4665573435,-547872.397903817,394907.5259051877,165791.8913965146,612512.8984314925,64273.79806568753,327942.0107977777,-162718.823008365,303307.39415900875,273073.74837286305,-334080.9637289308,446085.25577905774,552029.9353624764,144799.28227621783,823888.5946077518,214575.38405547757,727188.290887946,955573.1520107212,-91491.83151367772,581013.9525427017,300132.90896083694,83354.33826572495,-552688.0970641356,-449361.6889083814,-477092.5071548936,-277718.93269937905,-27977.1461935821,-508100.2427433799,-257622.86569053726,-231063.43717584386,1297659.662466098,456301.8898564251,-393953.4540548697,-465484.8211536198,31802.52637895476,321489.9589729365,230507.0838757865,-125811.4598131883,-357978.7682342662,426916.3390688668,148555.73630206287,-397891.05015834235,72798.76036410406,341955.0807076767,385901.7038301178,223293.13135408424,-39126.18764430052,427031.5268427003,494605.7418739521,938797.3532935646,512751.5625821003,-649078.8156507676,260211.52228557924,-558993.7570260176,744000.2799412175,514648.37108989945,560201.4743559854,147865.4489909676,856365.2789754998,-258557.91387459636,-557203.0532768844,433750.5534419259,-254693.5765169412,-4605.85927102156,179944.265992078,146164.03756674053,429739.0970813292,181238.53049250133,167635.63217530586,645188.9987767907,-475657.81645180495,86682.68430352258,828383.7956166388,-176316.72014322458,-94269.26943228906,676015.6295570084,-807461.2942030197,659126.0502400785,1171800.6123738,341819.42195145227,-62321.881967438385,889331.0330134965,-58030.783931853715,-390323.1557074939,190736.45806446671,-286593.69265619945,114769.85775720235,487114.4378818716,151435.22693626583,482967.5612945366,-602218.7144225123,237932.08632425964,396874.9277838678,514878.4485304444,-588331.9370844001,928071.3195851073,728494.189879505,30770.24666086724,650519.4218992926,55214.44087507203,661434.8431133535,622344.786992847,678051.0236788904,568330.2095853952,823788.9998684872,-451925.0256840596,-24302.784415383823,212506.748390893,791609.8768297969,182593.28810972627,-11403.94400900323,694716.9564769189,212025.25064705336,305202.5491296151,165019.04837791296,883523.5366209978,1131875.592231718,1276829.301346331,416431.04981566453,55665.62190223299,1417793.876768428,273123.21327030426,-794955.855847918,-952171.048371207,744131.4389837612,54733.96674625436,272911.5003932025,61269.57313387981,107930.33254151512,339186.52021026006,193245.7421867787,79138.49689483922,920824.252294492,103608.98594980547,-782215.8998226458,1397590.5675592385,745747.45264395,-407475.9001301979,374318.1604529619,144762.99694680236,-9860.49238000065,-308106.38528410904,518533.2766880896,806345.8574279924,-382885.30309868045,197840.04169280222,178311.3482962828,144406.03936663782,83428.90928818751,43669.53380154958,1042849.5638789954,299937.50725708436,479004.6966194776,1099551.3349963967,174565.48720663227,28688.632988978177,-1259993.7111059967,668416.2119568503,477670.49443126144,-74104.70861510001,106020.41113558691,565036.5397265693,-603056.5942930034,263169.91870431975,542615.1956356508,693367.2702001869,675756.8886240381,-331657.9291221616,329944.7484250027,278357.9176053512,-50325.073900284246,774789.9512410178,-174433.91865496384,251987.5793097564,140448.02569246432,90916.46760573937,-610886.9855665122,318549.4643637659,-15590.714765149169,-469702.09063337836,419959.84179550176,-291775.5258237277,444564.0183856641,-272014.51120480103,82396.16625444498,448295.06373575656,608177.3462397573,55166.11795997014,-90703.40090877982,-263313.79719035514,-169324.43790042633,897572.5114696599,-532946.5782264178,501770.83371883444,65659.93312963564,38088.01907673944,31168.903385443147,-139754.356975073,1048424.526686924,-857085.3132776227,-331140.08618747164,-311734.53966188664,335627.2086834805,583000.8823907278,1263544.4758379925,744511.31816631,332112.48187493393,461343.01533360314,-105118.68399484502,62933.74652154185,265453.9142573755,115732.38278915174,-817581.3194958796,179265.29797564866,-6043.445391560905,433341.6530663632,483939.2835334642,21896.419231744483,308697.03025188623,541113.9721229137,417878.06084707566,977659.3719859943,1061791.6255248133,-505655.33135783253,10257.201257266104,372385.8962462819,397595.72661374277,530557.8745394521,592716.5439872402,39806.700229532784,-202310.10802502232,236184.3117246679,141256.83881401364,667404.5958481831,-179205.71829443797,-282153.84563619737,281330.8654981288,101558.663396928,77118.9825271219,996144.5180630339,-236833.86223702086,257703.35752324434,-362284.81585950824,330641.92969544,288619.8296244871,-454866.7086849306,285270.63433573814,-81857.43532944005,-170368.8315964518,307433.53060001926,84352.48616413446,884216.1519493819,-51810.36391390534,153967.39698870853,99510.60325937392,234006.56396540208,3689.8892371151596,687578.0146890515,272488.17141575273,-111967.0250217393,696218.4112114594,467884.81494570524,99978.8097987962,504120.24021083396,-141460.07925066538,29859.846267600078,81875.11658471124,-228861.56570566213,781257.0603570901,-53204.55419955496,-345606.2091953764,-41074.970837217756,735355.720182607,385849.5895457999,135211.9384735152,459566.0396880326,-79229.98345259996,13890.590029915329,-395014.1885432056,-507744.54799851356,-483124.08328648144,684092.8783112573,13204.531709087081,-221546.92338014534,18497.115019047633,606606.123534401,643105.1683483985,-801840.13769635,131099.8806455275,521771.3827262451,-183557.15070593916,-420929.38374379836,-76206.05189109035,71739.08919888921,658232.8862743424,-85532.33698714897,963682.7658008221,-712371.6538370606,284483.4433559533,106005.17819060059,383292.8805315355,-204171.08845270565,577009.5705439942,-46233.19468768779,-274663.33503984986,-185062.1145102214,-81077.53642085195,693922.9360671737,-141761.41112226574,587393.0836256794,229278.28495174227,827375.0302347164,406454.5164082004,873920.3605534304,48633.771177728195,466150.7637766814,51346.62026746245,-200772.66915811598,-1244.505551747512,-79107.45243605226,37232.976007910445,276839.11732151685,423577.8777618315,257109.83039887762,-258494.752859266,405288.75126603246,197386.55913671898,568606.2338840258,-144357.45319659682,-746872.9155969466,-430042.01976108504,272519.2555124131,-351307.7421588735,-673598.3978118333,660513.2990781385,-650409.4781679534,498249.2021753816,1247720.021447572,-242256.4845153545,617268.1706989193,806747.9727399428,742836.6478394647,57735.19124205783,-11562.728903980926,73165.89333627513,697852.881234047,443896.4160259818,-24995.32369943848,517112.841257222,501465.9013518477,-92975.54137802869,243584.07287424663,428280.7749202424,1632.1367990071885,730115.8852555389,531240.8717032415,104369.08727913443,550659.438791242,46067.65732200118,-233165.67792296922,1162974.9378490173,505777.1944607245,99159.07069360418,-191157.44027780788,-139108.96343760984,750385.8974707616,439669.5043476927,20964.84218433546,387885.1398114846,603903.231054584,330926.062405779,124972.794262941,901801.1187919467,-271303.99726592377,-140941.38970919512,-513626.5502920393,377912.23634606646,827627.7064953297,-131789.74463845324,-127482.48967750277,166956.3634261964,43858.2025933438,-141385.31652840972,223588.44162869873,-116318.27668302413,365259.8298230232,-234631.5554106608,438340.1876226994,1266396.7835636665,-82297.51631541317,114153.02529348712,-322754.21516319923,908.3217214811593,-366153.85727579286,-565602.2816417795,-47699.08315329114,578194.8682893207,-37986.010352611076,-25803.999153680634,507250.9417193122,416994.82236096216,979939.308468937,777536.6010277648,-614851.5531434808,-277967.5467293281,77925.48456388805,-859285.8266005768,-283086.1488710996,-28605.386999052484,1028878.8214771538,-507102.4443133129,363825.86129760277,250216.66404535482,-127599.33769413875,779543.952390865,92444.27446207032,571951.5514067928,-313573.407635699,161108.7420017738,-135161.1684418167,335594.0478323307,301598.6084192847,575414.4663928756,458775.0181744844,-81300.74744635448,56646.367026533466,523512.67455449584,233488.42153724283,278491.3472453691,322729.10872949986,161431.28046576632,24452.04307889426,12102.034042554442,123010.16629076237,-9946.165459625423,-241053.07383630378,520114.9075039667,179894.0897090123,-178711.83074380225,181050.06652792916,105769.46493994026,223574.5762999258,314318.67264321307,352819.531302914,13652.849903877359,54399.55694142729,94934.54651332973,-363414.0592223755,-80368.32440385269,130648.70067299437,297490.2203464513,878104.9791841893,473293.2857837109,-231349.70323847118,76100.99269008031,-64947.79546480207,-396245.9056495242,1073173.36370327,400051.20347006107,-968320.7889706362,1133278.0489816149,-289555.1001801668,-456888.03601806145,425035.22920363303,501253.60977715673,905675.6681637582,369102.8605543198,587523.6901333309,-336587.01858405117,144039.19689830812,250754.5958196246,478089.41165227024,482699.8376578335,-224664.59829989355,56792.0861618896,-495948.50457453495,236924.7298506978,971550.7430671747,634426.0768627459,-298927.3611566671,-614049.7281940095,-231129.07891346654,889604.1304451698,490990.5015642643,385811.0016997731,-102927.11194050172,262924.7043213737,-667755.2273786527,-4101.082686262205,357938.0440405072,-138531.9182546637,-294003.03066634387,682799.0415607556,1556559.9951654612,-83099.16245440999,1122019.473571789,718563.5374979028,311355.982785841,-490567.1173614571,-378284.04664947,-222354.7909421078,-77479.71990714036,56451.995936912484,-773116.3550969707,866313.3235440184,13285.28009109199,505063.42669319874,-485031.7341722506,-51361.67746116733,-291880.41349120485,305469.7847431083,574595.846310046,309308.382456596,298373.76322647557,-149130.18831334217,500077.55174810626,-587987.5697976286,433327.4255307666,187733.09762466978,342758.3993188995,-68099.11646428937,541844.1261528851,64005.329598547425,231855.1629188168,-509422.6078701564,474524.2243212699,-264080.811996453,-380347.96677323803,631204.4726250833,-653453.8595649838,-192606.92943088477,167479.2097779354,368404.4167871913,-215178.2767137296,-90712.9580322844,-17471.30397931859,191433.12958970107,434772.6425048737,509089.7467446327,-259187.12565499637,533686.8817923116,-644915.1474049687,157897.88699447038,206097.98890612135,504705.01039587567,-60919.90862700017,-385905.19973520096,939761.3718568208,-310717.12353722285,820332.0576439039,864316.1308471104,830980.4607874705,-370732.22490901314,830136.8359658467,171659.21121069463,-614237.2344089076,-708210.257201998,671634.4549330412,130360.49910602905,-262197.48139289697,133050.16932177916,581433.3795434125,797048.0364157395,1131611.2093260977,220791.87119454984,36703.840833977796,699197.0186685617,411215.3975814516,-230986.32090221625,614899.9186447579,713727.0954014654,615261.3023697818,-372334.2154122172,-513292.61017040163,490696.1857849662,384068.0260849623,186819.80994012393,-67033.74226941308,275952.66013508476,-18731.77580683306,-233961.5302171274,105924.11681707483,481350.9708257951,-349017.3783770092,-72101.50142484577,67102.01252246369,834205.2669900944,589769.579104281,276505.2482154253,280232.6332867411,60848.81154989451,591850.3883052245,201447.3622236494,596976.4361861236,55342.99361315323,-263437.29700573534,-410033.0807998241,-418963.6804785435,575561.7892739871,673083.2126432233,644196.7002027407,171634.20559684467,-151926.0754966531,-315584.9853508421,235686.08931396762,270559.6521237432,23221.13225652324,829970.6472760981,-406343.9966363148,-594454.108073805,-92679.16416996391,98631.9585214979,156227.05799139012,1405462.835305855,-99620.47940893983,895137.9548539799,383462.0596397766,528599.7201857232,245925.97891114,646370.9552408406,-204133.2260463382,221658.65660250792,267416.2845637072,-577710.7059439016,223444.93139805086,-182762.3931716797,-241470.17522675171,571877.9999389867,642851.7924527903,-86845.70936519885,217663.55497424956,974269.5203059618,471228.53864021366,301255.73745515477,-73497.98804229638,47205.20191694051,-413441.86306295777,703036.5860280604,394377.24611567566,647571.6559833046,345230.6350359409,-335016.5552291651,-507753.0756735988,-85703.15455755172,670715.2125410382,-204224.10390990367,295733.87512909574,387382.4039348648,367186.8463240727,514830.7380672628,756986.9893932887,142618.782682803,625820.2269484042,-497983.6724540726,-859450.6656204443,-637256.5903428285,174009.39382558828,-278533.6864842032,-506729.86142945196,319637.41872726986,858144.4514687122,21128.189408566803,284248.7389509133,395954.0596723622,399375.44905136013,944036.4946766284,-148720.2684358037,885260.5634630211,991500.7873268006,-380081.6998570594,561527.8568674289,434131.5876672147,-150721.85598228872,461232.7751170611,684261.3034210035,120037.03501485242,909712.4327196274,164404.365435292,178202.68753847014,303198.152215688,1167601.340523411,424702.85686094034,453196.5316996332,-13062.258405671455,62963.356716145296,469346.6688408265,-19856.914254916832,-207517.65098960046,275336.42520237295,512452.6492103501,-686578.1276353591,-59643.374467942864,-110781.59618046833,-67613.93618345913,132021.739070002,138259.06455075415,64966.06837507337,-340335.3445586357,15302.830461536068,847937.5084280786,211799.65303936508,1054797.5201373817,-177612.51450635353,446677.9207255915,80933.4263098645,-23258.324052938726,-354309.462231454,161912.32314596046,318390.91325307405,251674.70055904007,-405652.7870417675,-118815.13684288366,280073.07651732303,231316.45841870178,826559.3898177035,-52625.71959765302,157655.1450424292,771384.0091754757,-195971.17230241792,367045.3171051885,461885.6193402605,-59237.715520397294,-695287.1940839123,526260.2666805759,538838.5258336165,113667.28654984757,165918.58752664505,51870.12528023822,-750382.7606920875,676266.307940179,370658.74474498536,204568.19202896766,-20241.619105767924,169184.06265225215,15976.19113871362,-419182.083207075,511575.56552966824,140139.89243075857,436466.5228013196,-187638.83281068876,19677.17209553579,168679.54451953992,-107095.94743383722,830895.508296954,633362.2888940931,89026.28776479233,-1081995.2382036871,-220240.30163594568,533959.4276391151,255580.1563547426,-502762.74226499954,-157469.83005230967,337747.99070819,293264.37223537173,537002.064768191,87833.55871840892,468109.1200214862,264698.89368494786,37907.649028778076,162467.4149423521,104723.82111878553,467780.8264508974,27767.58945228625,-244560.02917081304,325163.49921031576,-491679.4544055094,-589882.8010308249,-235831.21532594087,-750122.1459897417,50685.12711626198,-552771.011266028,-9568.07498691883,503545.0636203303,628231.9582613781,194869.3546721954,1075663.9137437092,-175323.79687321465,-486111.75265476014,125361.15809441125,461690.7157183336,-12701.397587122861,-748015.5343572441,85635.11543107964,338775.3508965592,459155.10227990616,-384861.0391151039,85060.47467205254,472082.8710682634,-146523.41406717617,389071.2983896155,-542254.4842104744,-611050.4439900122,-261669.78133126535,103096.451293875,66840.38267795928,-341669.29861180205,-368720.3463220424,371982.3779014484,631079.0906666918,425531.86993032414,-204221.96962793637,512910.3162941332,-299587.1482385243,-566849.2888534334,596148.8341817609,-452786.35621437104,729060.4678921411,-254629.84604432108,433278.1477871244,236250.4675116334,332815.9050046378,1075214.8711321033,576104.1831353242,-306424.01716258563,381413.00773929013,1361074.9638921916,199821.73252076283,576132.3798363917,297400.4697255022,31053.570620469283,722559.6774188424,46743.755983097944,12089.073664183728,388091.9985502572,1136593.1106627304,696638.1805325267,451316.28026574664,335346.8203801187,-15086.970877801068,551514.2482888242,147871.16137995105,-474606.94248734834,-353389.44188282406,-47887.97684890125,286117.6956536281,-226827.0756390444,-182190.01217586314,893138.3759807008,85614.72949250229,298237.62346026395,616268.0703492416,-298313.95932306955,268018.0276130168,667522.0505003096,62438.142941123806,-124584.11831936613,723121.3634713502,-82122.56291944114,10216.877884186339,-160513.61981961876,274543.2307999036,386652.22686738847,741414.2520554555,-275595.17006858485,633658.4286489072,-636891.2020106451,-362985.19086272316,-460040.07865804667,-322283.54566251487,-79196.97114034323,744975.0017643166,184383.1840082137,479633.16813961184,119451.81606141943,-21153.75291760359,-164105.74374271836,426389.73101593135,-335814.54355827905,-311522.72705138614,35027.66133566294,-57892.18370197667,-255052.86319334386,-605753.9223056922,-99502.83754549595,1447299.9364929597,909289.2167151282,1154336.3588809096,136044.1633254513,13616.684389888309,446598.48076129286,125234.5200278121,584443.7893259125,-65694.77715393482,-109522.46583981486,393991.50431226473,301059.5271395873,-315847.6894622557,486452.54380875407,734182.781793335,-603070.6792963902,-84663.41600988433,-1042359.1406974755,-257838.90456983354,417883.3656008509,408363.1281059701,709177.6370560895,73291.61805907497,-471584.80211000517,371649.15590804303,385274.32202004176,-5957.500348760281,-394457.1104480298,682869.0877449,-1776.9795968420804,-23697.592536283657,902426.6154849045,-88533.50440109009,-170827.24254657282,551977.1597193028,459139.62926399615,-60909.336518041324,195887.07276604185,-235763.67181475647,-261919.4651643294,586963.1693670931,150601.039073091,181677.75545465713,-380569.7442263807,102232.9306441485,425389.83179637184,501459.35603783606,-376618.3619983052,-253440.80210301513,171311.7271462339,-163272.07124374947,247723.56006628554,287012.6322282362,953768.8195928484,1102466.301444035,853845.3827091819,-601885.9634536956,-276955.8111337242,843954.9730538349,-71317.98532155761,809724.1520653656,266633.085626754,-250985.3450999707,-9276.750682708807,-658677.3261863478,297672.20589588163,265221.65576751856,177975.1757102413,78602.47194820037,-11392.673179667443,180473.1676210519,134873.44222548185,627848.604643485,319666.6603120761,45836.494111324195,476371.35500929505,-224640.92550957855,-240740.3754490544,833115.7282844242,300018.0889948788,776197.8991177566,211590.54759586975,314838.51703818,-83587.8483967837,80403.23739022529,-96646.08154736925,1083926.9447084195,-7560.912500184961,426100.0536573436,144676.34130517114,-230487.13944739942,261005.10293549858,-226246.17568665743,499581.7001467394,-355467.31458377466,185537.01381288283,425178.257562676,481985.4739051061,222335.4885299271,515159.9120009686,-199223.83611549344,-358329.94705996243,-11465.578515581321,-64770.10898124939,710033.7390885674,-753553.6993561909,104424.86204326339,89202.63292459864,199126.97550788708,-160976.14374509268,470463.8849233547,365581.18538056035,312087.12085739383,384343.64689459885,-293956.7122313818,-51277.08063805895,803727.0058504185,-94204.78430709383,-376769.56446709577,469451.2276601554,451564.2048494583,538965.8055011481,56298.30678841751,-21544.131587571,427394.6876303251,630651.2738391943,833420.1056938125,498769.79032200435,-756426.7751198253,287027.1245385157,198839.32550362684,-77644.15926006902,98066.0164391459,908060.4873061981,866799.4323373027,614709.0922781429,342444.24054521136,379072.78314908314,-27706.399994578212,145020.94459605543,115835.79748686077,495562.8540370469,609888.3776599905,12671.078257372137,-326340.7114003366,424594.5510051837,567397.6214130046,-179298.0855468614,377719.80610229913,-314519.4448065739,-139753.68549967045,370476.2257980574,-94931.131441243,546100.444145008,589171.2209244198,260203.1350772283,645285.8727451293,-339414.3893015743,20567.169882415328,994408.57266948,1072222.7913918393,-111388.54676909512,-211333.82457180228,390060.72932362603,515707.97085015336,-821890.727387365,167698.834841887,804802.7942876378,474794.77427331917,-102559.36902274657,-121027.85872339318,-5906.337044111453,467257.34512755694,65693.58805730892,398413.8075133823,-111893.28183709644,325383.69789411314,132246.23011123203,-184312.12690931186,1419312.0939622303,-199023.8831718131,230972.18404618604,468641.00672235945,66597.6520190048,767518.3865398215,-386442.74149897136,593994.1116666715,-455760.29533791216,163917.39647301845,-83384.94756703591,-175102.43630482722,1140419.9070396945,253811.21724867309,-103747.08249417087,139542.62207102962,772159.0966456551,182847.96873588813,22557.903254479636,610018.8438849747,925709.2256666976,-285093.3449740126,471925.61457939073,701429.694728455,97506.68169608107,364207.67230758537,161354.52505421825,799424.562339698,-91024.33997411607,1258844.1063140323,-509448.6745810234,-287108.3033610149,337477.2641237569,-679883.6707025641,585475.3461460327,667508.5709417101,247223.4520887467,-1007605.5557596502,134047.77472073818,52464.87015015585,-535150.1359582571,192512.12589035928,-278913.2190043791,841645.522820808,34400.36371834995,148507.2304135873,556518.1974048815,-278007.7781954617,341702.0161414929,-94218.4070389918,-291782.5811856892,569898.4326518839,-131363.9579290077,131798.25766563322,416220.5941794729,60489.22978051985,818018.2633052478,395336.36493428657,767507.0107998187,69150.03916042252,-271735.58029456483,-16469.170852255076,192300.37919700053,-305353.64063934935,63652.331275893375,-148621.77259268053,-202589.00893974444,213193.3703906224,225583.1326151872,173393.6223918083,628165.9086687355,90161.50312849274,447231.9118192429,460404.0955519499,507176.50017393753,897034.8817299521,-637599.9262710735,-419795.0282653186,797814.4867485524,845495.2697770079,632286.5105161201,249679.9311617422,402956.8125247916,70455.18535129866,-195396.70481611043,307986.3730052379,208233.60822834866,-262269.283174973,735476.0554830539,208886.45454318402,-219013.0457637799,-48957.5032839654,-695301.2501874678,207106.85464979103,288274.29530797433,89522.7456781608,25793.605306717567,-124595.79987889156,99047.07156996103,884040.511654567,13378.975777749904,443075.8689709436,251921.01126375888,939490.188935129,-46614.210294532124,900995.2589816283,-167285.86789577315,38997.78303606156,-141820.1722827223,416.18743757903576,-184143.31288879877,90369.7247153325,230357.518951762,18515.23482277477,529588.8508601445,885124.4430757291,457176.72486159997,505379.99545744667,-122199.27397966338,332286.9310015589,406673.4856307758,98520.94709207909,-64085.287381260656,20749.223911430687,784146.4526317352,-24126.820522902068,752762.2542030234,918822.0061156284,104519.0223132153,158922.46130167693,970595.8604133325,170696.4859528765,768388.7958836546,296565.0665051653,-539447.0836210293,472044.0764703513,175142.2363330787,-334036.50614924356,171929.25341180153,-186607.3547339621,-113831.79971161205,-68753.32047451707,194789.68877069606,458997.6968180244,293055.2603434995,-126345.5109411818,653242.8277673367,494975.84960728325,50452.45328582823,125240.77696243068,-448933.83357821684,257196.88672731072,957187.8262005164,77892.7455801228,394171.10981881386,-686532.0988215171,-1031387.8509817091,195304.22734649293,350916.810567576,380511.4475131314,884746.9118585708,228110.48299072683,942647.4166555936,-216140.8782810578,-72152.93228088599,-399196.7390027698,-177657.2175632473,-460125.0502650803,183846.22721402673,40373.04028508905,-588762.7432364295,797291.072420626,-119075.59792115353,-83712.03678623214,-383675.83223468554,205944.94049402792,-304026.4679054925,1199083.4420919158,-17083.27959511988,495796.8508289531,605194.914047413,3003.689576618839,-422669.6208579517,320488.6280036876,-531627.4219815563,352539.90993417474,-56058.831992871594,352098.5441511767,-241558.81400390668,769943.5189935165,134615.0863374574,165739.66485489393,-646594.7061788384,919864.3734983546,-369976.1202562228,195520.1251951591,41982.3354422315,962796.4133642116,-666383.968685478,1107881.2814496318,20418.5811495902,-201245.98557112203,383569.1568124383,487003.5993993054,26986.63944591023,-335159.7157128784,436767.6299551176,-239683.0693750414,725123.5317840357,253411.5800390183,315883.4989659153,389013.71923192916,-553810.9204181274,79900.57481813384,238961.20777706103,133347.80271406332,46307.07259717537,118984.68652076507,370881.1865687906,566066.1187611222,-405872.81983774016,-141779.8553545382,-41221.857428452,1159653.6312721395,306849.94112821436,601413.0114637115,638377.2820432121,-108916.85987313045,-159302.46770133544,-144348.1905898056,70668.52478053467,-577251.2059865715,-81256.95917068841,-1114.8367898049764,345389.93278787704,786046.799414234,344724.73222061945,192099.02834752854,299690.1728517185,206911.91421295283,-220030.80010319268,648340.9113659873,-437639.6865534899,15403.680787262972,642355.5447309557,-82987.50215651607,137015.49936170783,-170621.93425404467,234481.72882981878,-214083.82244883105,657494.0018980675,337783.34734820155,244246.53588356916,-58718.10946518881,-303580.4655927359,594821.2151462864,281737.51398361847,-35906.058194209356,-55967.553682052996,687379.0597139238,203936.3257323606,-400384.7332465621,-314180.6135722166,499090.03233737266,585019.2756407135,-465486.236481125,-205469.59991334006,-178065.58582458412,83415.85016300483,-168207.08871917427,352745.77073979797,195764.92721493356,-465344.51943225926,425879.5230777394,216877.31831834372,-328015.23507723166,-853956.6943173867,-94494.58220245223,11922.455877819564,808149.4018383706,-738446.5531236837,334479.96266986756,-721624.5594889033,632983.5564585244,-544108.1711472692,98891.62136063166,264082.5863295491,622290.8121309998,42222.84396416508,78061.30585542275,-28641.771799970884,-336316.28625603067,239742.10195006197,-212757.98819958977,172253.28944031382,965797.5111104436,1239782.6080700648,253512.64382740157,-266421.58477183664,43510.58852711739,116527.46012409357,211904.93969687447,-605275.831332945,354385.544626615,-463844.526479885,371041.99856472574,-50049.024162486196,528923.9597814293,79292.79394289991,-30625.83831335511,319157.5013608979,-457936.2849918348,177845.2703845147,120511.07262553414,829639.1568208197,194715.5100776176,916762.6577019054,-93277.6750239036,922269.0696542305,-16462.580925597344,259001.01511640055,634342.6992987557,-268811.80042269686,686546.6202999209,110589.30593721801,241010.45562237408,127207.64451453136,215532.7457880946,427575.1996340249,-695356.7814848374,970839.0378680909,-261482.54127301183,78978.25413915189,-349871.31274654437,482010.85960433306,-64713.60640950361,910688.0430745464,153323.70394610334,-358410.65309978975,-43659.08715518471,77541.26219653804,872515.5899932054,624726.4341126601,660200.484161281,45009.80574087892,389868.615045425,648409.9615302416,350323.32945675356,738386.8481493313,-644498.6076157102,859345.5584243671,515465.3395480048,1001656.7115099658,716998.3515458433,628986.72979715,15828.763632382266,233449.86987683643,-28181.650785202626,-473103.8556804196,-480278.6106970846,18431.470447101165,-264135.49939026544,-466086.26070874045,-480247.0756125287,205382.64067511866,574207.0448302203,-452494.270978197,-63351.165669752285,399440.65642352076,45012.639148470014,579212.694140851,638393.4262896078,155155.2833950068,175620.66357581364,-209569.4759520404,206659.44394864468,-90110.26340935472,509610.5561536895,-240341.20682923216,533721.27762608,-72989.02253549034,157638.289721224,123654.5688476977,-180006.88207174232,261104.63054425223,-99073.49620047212,-180482.90899991663,147429.51423538616,378699.5779117155,1430263.9351817793,-131801.06185065303,263513.8005168545,-828485.2295585573,567506.40031943,381193.36661427375,-743328.7537264288,159895.07903469168,-89970.82337401574,1027883.3110761326,-144905.84995245654,128646.28907808056,10334.194231858011,-486299.80104320496,275732.93726177514,220148.5784802162,1049676.5567758572,499174.09676977945,-435597.5445772377,125891.0956510664,749078.9282665551,553161.6864278619,-47678.9439380439,1044587.0946549792,-643736.6177289188,411371.3413798646,610586.1340901419,322482.02060066396,877057.5999305216,45989.988616457675,-182185.90224682353,1113768.5691118129,480468.135411832,209684.5296161496,-424447.6165557513,112890.50192629034,-271812.04537064675,-621473.7065270441,913225.0647258232,-890866.1799875963,-272414.74330175156,-260134.56139354082,-116048.07714537904,-29467.31582117174,364780.8463813984,466806.7982949414,224640.68087314954,872593.5839547426,69178.58420284046,452800.5595871783,337011.4045824744,-706890.0799177941,415052.4367065802,655072.4099718179,15013.115057232324,-859600.23564567,381592.1388176433,200567.4905935279,291868.8860891806,75428.54067546176,311134.0898246132,364894.087556927,-66476.27309779916,-282553.36189901177,-493542.60231290106,611583.2357337624,389985.8104796312,267206.6706199711,-897621.8862955722,803751.9502658905,258962.98294368805,54463.260610372294,308469.4116568188,256713.27647404885,65644.52848933358,-260549.1675998005,-663526.3404633389,563989.8238817896,-146167.96249407576,-16447.420849782415,-172144.54886430362,282148.76901980443,450123.1651527579,-56173.73304924555,747789.047176729,438240.5231203004,507074.7034056247,251127.4193138564,-201620.4505139119,744620.8154583131,-88585.0353557393,-148343.70535845542,271613.29777951725,949318.8671423788,81083.9498068397,-415139.8275015517,-133302.81821162207,88376.72986856895,-59306.77932817256,1129525.6931455196,448420.87506742496,151776.5897492878,208845.74641879555,334715.95747601194,-517144.24099177215,289355.4917790964,276705.7219180693,-265925.94035554654,465037.6992253065,27048.040852684993,24742.323968769517,-451374.1895465241,436303.8367096996,-39767.16300758254,-580517.0886866096,877218.7837039148,-157730.88869111845,-832404.3342675364,761481.3418839192,182703.91182937566,-575947.9723702311,683236.841129832,36464.29159210017,484741.21446461044,-441700.20984662976,139527.70694588963,-38907.511892226525,945744.5741354367,905855.7580560087,-195262.48841416277,1045286.6782990347,-303522.8481049589,-89168.28974578343,-856248.7926683801,-29899.35612997692,916708.883484398,-146914.7742542564,91992.13770314073,-66762.44867830211,-83894.71805824479,-689518.20429478,-295951.01884096535,-126870.34631381836,662713.0249732998,795431.5281819827,377033.18267748924,-255822.60059114406,102997.11730941199,-280339.0390669871,684861.5442592497,-203513.7120360434,663204.1653692722,68540.5371280685,460647.7497832328,571569.5322613665,1052117.446971801,23329.39947056491,1213774.7657641284,-492085.9435483562,218092.75592183787,-255212.9351941934,176639.27869019425,-114628.64426836092,-52114.88021904742,-179864.6146735954,754446.6701705377,689393.9739802959,162797.64565333957,182902.2617145353,224934.33341779467,902921.597186699,815656.0505126608,319492.167355502,696596.8772103828,757952.1490496071,-390848.4776998265,871737.0670213904,-401790.64032352064,695902.1672103261,17026.090469664894,-302399.09838183736,-594410.9963813946,160687.63680324797,479169.2002194864,329756.5738660032,384311.6562649561,247672.9355709548,727153.1487623705,-520879.0549191232,317861.4070840725,675771.7736503929,119006.4176709801,675643.6300291938,44199.2531520878,694430.8442517291,-412037.24197427696,-232041.53896712232,296590.6931798742,-185813.21356786136,877738.3605157416,670100.4441042119,97159.03898993647,715433.7381124515,171558.00477135368,-870458.3591927709,-44951.208510160446,-333230.03612389416,960482.2485266947,-705644.0567546682,538892.3774164063,-181037.8926305417,730160.7193499375,-167026.41158281593,920176.9465870755,91084.6148273414,-215066.12876949785,980159.188318612,355683.4064908675,-475226.36124712136,134745.74745311867,-751080.7101577148,377125.92971279286,-616218.5204880368,-316232.28562878864],"xaxis":"x","yaxis":"y","type":"histogram"}],                        {"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"value"}},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"count"}},"legend":{"title":{"text":"variable"},"tracegroupgap":0},"title":{"text":"Simulated Opportunity"},"barmode":"relative"},                        {"responsive": true}                    )                };                            </script>        </div>
</body>
</html>

p05 -547910.3677146992 
Mean 185272.82030055262 
p95 916774.2180497767


In [20]:
# A little magic to automatically write my blog :)
import subprocess

subprocess.run(
    [
        "jupyter",
        "nbconvert",
        "--to",
        "markdown",
        "--output",
        "~/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md",
        "rent_vs_buy_blog_post.ipynb",
    ]
)

[NbConvertApp] Converting notebook rent_vs_buy_blog_post.ipynb to markdown
/opt/anaconda3/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['application/vnd.plotly.v1+json']) is not able to be represented.
  {%- elif type == 'text/vnd.mermaid' -%}
[NbConvertApp] Writing 1158832 bytes to /Users/jakeetaylor/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md


CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'markdown', '--output', '~/Documents/jakee417.github.io/_includes/markdown/rent_vs_buy_blog_post.md', 'rent_vs_buy_blog_post.ipynb'], returncode=0)